# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)



## Introduction: Business Problem <a name="introduction"></a>

This project is focusing on helping a new client with finding the best location for a new food truck business 
called "Hot Potatoez". The client is interested in expending the business to Chicago, IL. The client
would like to see a short list of neighborhoods where food trucks potentially would be in high demand. 

Potential locations would be locations that are not crowded with a lot of restaurants or fast food chains. One of the most preferable  
locations would be parks or any other outdoor public places.

## Data <a name="data"></a>

This project requires two pieces of data. 

The first piece is the list of all Chicago neighborhoods and their 
latitude and longitude. This piece of data will be taken from the website called Data.Opendatasoft. The data is in Geojson file.

The second piece of data will be acquired through Fouresquare. This piece of data will provide the most common venues 
for each Chicago neighborhood. For example, a Chicago neighborhood Chrysler Village (Lat: 41.7764, Lon: -87.7518) has an 
airport terminal, airport service and heliport as the most common venues, which do not look like the best food truck 
locations. 

After the two pieces of data are combined, the results will be clustered into sets of a number of neighborhoods 
that have appropriate most common venues for a food truck business. Each cluster will be examined to see if the 
neighborhoods in this cluster will be potential candidates for a food truck.

### 1. Download and Explore Dataset

First, we need to import all the needed libraries

In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Load and explore the data
Neighborhood has a total of 228 neighborhoods. In order to segement the neighborhoods and explore them, we will 
essentially need a dataset that contains the neighborhoods as well as the the latitude 
and logitude coordinates of each neighborhood.

In [17]:
new_data = {"type":"FeatureCollection","features":[{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.68472573428465, 41.845086420677205], [-87.68542261104949, 41.84484968665212], [-87.68544500538192, 41.845435134547266], [-87.68546192584819, 41.84616821826134], [-87.68548031952074, 41.84708341986971], [-87.68550675609418, 41.84799826819348], [-87.68552430031912, 41.84879132904806], [-87.68555051725787, 41.84977163830361], [-87.68558565763647, 41.85068494830626], [-87.68560526478207, 41.85159909517087], [-87.68561238693383, 41.85203242348418], [-87.68496341931541, 41.852034692645496], [-87.68429192840915, 41.852040216203456], [-87.68318, 41.85207], [-87.68316965475312, 41.85159929126761], [-87.68314986200994, 41.850929360727676], [-87.68311537507094, 41.849829131463274], [-87.68308298567239, 41.84879075643091], [-87.6830675836062, 41.84805923059039], [-87.68304126470393, 41.84714438227497], [-87.68301414640354, 41.84616910757611], [-87.683, 41.84574], [-87.68355106345064, 41.8455350590474], [-87.68472573428465, 41.845086420677205]]]}, "properties": {"city": "Chicago", "name": "Heart of Italy", "regionid": "403329", "geo_point_2d": [41.84865731753687, -87.68433056046746], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.54227994499989, 41.75285039400005], [-87.54215662699993, 41.75284338800008], [-87.54140430899989, 41.752845319000066], [-87.5413044299999, 41.75284001400006], [-87.54091042999993, 41.75279701400006], [-87.54085021199988, 41.75278837200004], [-87.54054721199991, 41.75273437200006], [-87.54047128599987, 41.75271742300003], [-87.54011712399992, 41.752622108000075], [-87.53986596099989, 41.752553794000036], [-87.53980712699989, 41.75253560600002], [-87.53762312699989, 41.75177760600006], [-87.53758324999988, 41.751762699000025], [-87.53585525099992, 41.75106969900008], [-87.53574909799991, 41.75101882700004], [-87.5353843869999, 41.75081367700004], [-87.53141799799991, 41.74922159800007], [-87.53136101099993, 41.74919639800003], [-87.53075601099991, 41.74890339800003], [-87.53069970799991, 41.74887362400004], [-87.53064565299988, 41.74883994100008], [-87.53059411799988, 41.74880251600006], [-87.53054536099994, 41.74876153700006], [-87.5304013609999, 41.74863153800003], [-87.5303150799999, 41.74854207300006], [-87.53022266899987, 41.74840928200006], [-87.53010317299992, 41.74819690100007], [-87.53004191399991, 41.74806358600004], [-87.53000341499987, 41.74792350400008], [-87.52976341499992, 41.746657504000034], [-87.52975667899992, 41.74661691700003], [-87.52949144099992, 41.744756276000075], [-87.52921054499987, 41.74311358100004], [-87.52920727099989, 41.743093008000066], [-87.5290442709999, 41.741986008000076], [-87.52903467799989, 41.741855], [-87.52903467799989, 41.741655], [-87.52903467799989, 41.741479], [-87.5290483399999, 41.74132283400006], [-87.52907894699989, 41.741200315000064], [-87.52911570199988, 41.74108754500007], [-87.52905416499992, 41.74099966100004], [-87.52901663399989, 41.740927262000056], [-87.52895563399991, 41.74079526200005], [-87.52893038999989, 41.74073497300007], [-87.52890959099993, 41.74067301100006], [-87.52889334699994, 41.74060970100003], [-87.52888174499991, 41.740545379000075], [-87.52872015199992, 41.739416], [-87.5296276109999, 41.73940885500007], [-87.54465116399987, 41.739290559000025], [-87.54660253799992, 41.739407314000054], [-87.54660111499992, 41.73744892000006], [-87.54808899899989, 41.73743], [-87.54808899899989, 41.73838537200003], [-87.54818190299991, 41.73894279500006], [-87.54827480699991, 41.73931441000007], [-87.5484606139999, 41.739871833000045], [-87.54883222999987, 41.74042925600007], [-87.54929674899994, 41.74107958300004], [-87.54976126799993, 41.74154410200003], [-87.55022578699992, 41.742008621000075], [-87.55059740199992, 41.74228733200004], [-87.5511548249999, 41.74265894800003], [-87.55134063299992, 41.743123467000075], [-87.55134063299992, 41.74405250500006], [-87.5513411869999, 41.744642049000035], [-87.5505052159999, 41.74465585600006], [-87.54948489099989, 41.744683846000044], [-87.5484605479999, 41.74469491200005], [-87.54785, 41.7447], [-87.54785383999989, 41.74526059200007], [-87.54786342499993, 41.746283301000055], [-87.54789602799988, 41.74711864200008], [-87.54793651999992, 41.74804995900006], [-87.54796028299988, 41.74897700400003], [-87.54797421499988, 41.74981290600004], [-87.54799272699994, 41.75083454700006], [-87.54800499299994, 41.75148464200003], [-87.54800120099989, 41.75195988900003], [-87.54716046899993, 41.75197115200006], [-87.54623143799994, 41.75198741300004], [-87.5431459999999, 41.752154], [-87.54235826799992, 41.75285942100003], [-87.54227994499989, 41.75285039400005]]]}, "properties": {"city": "Chicago", "name": "The Bush", "regionid": "403359", "geo_point_2d": [41.74514133089662, -87.53976477689002], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.66585814199996, 42.02312979900005], [-87.66525902799997, 42.02302994600006], [-87.66487578299994, 42.022264323000044], [-87.66484027999994, 42.02224569100007], [-87.66475265999995, 42.02221564000006], [-87.66470502799994, 42.022175946000054], [-87.66464302799992, 42.02204194600005], [-87.66455202799993, 42.02183194600007], [-87.66451902799996, 42.02175494600004], [-87.66451982599993, 42.02174344400002], [-87.66425902799993, 42.02142994600006], [-87.66422523399993, 42.02125253000002], [-87.66419928199994, 42.02121969000005], [-87.66413204599993, 42.021196946000025], [-87.66405501399996, 42.02119728100006], [-87.66395500599997, 42.02123024100006], [-87.66390313299995, 42.02124480900004], [-87.66386502799996, 42.02123894600003], [-87.66383702799996, 42.02120294600006], [-87.66373702799994, 42.02075494600006], [-87.66365702799996, 42.020392946000065], [-87.66363702799997, 42.020261946000055], [-87.66363634099994, 42.02020525500006], [-87.66363172499993, 42.01982446500005], [-87.66363103599996, 42.019767680000015], [-87.66360831399997, 42.019547664000065], [-87.66360502799992, 42.019515946000055], [-87.66360824399997, 42.01949109700006], [-87.66361595699998, 42.019431496000024], [-87.66364473099995, 42.019361662000044], [-87.66375548399996, 42.01920869600007], [-87.66378979999993, 42.01910662900006], [-87.66379079299996, 42.01909618600006], [-87.66379398799992, 42.019055452000025], [-87.66379082999997, 42.01903855900007], [-87.66377802799995, 42.01897294600007], [-87.66353162099993, 42.01842106200006], [-87.66345902799996, 42.018329946000044], [-87.66345902799996, 42.01832102900004], [-87.66345902799996, 42.01827325900006], [-87.66345896799997, 42.01826393600003], [-87.66328002799995, 42.017874946000056], [-87.66308023399993, 42.017383001000034], [-87.66306802799993, 42.01735294600007], [-87.66307990799996, 42.01732661700003], [-87.66314169999995, 42.01718967300007], [-87.66314202799997, 42.01717748500004], [-87.66314202799997, 42.01713225900004], [-87.66314164599997, 42.01712002400006], [-87.66305702699995, 42.01707399600007], [-87.66295702799992, 42.01707894600002], [-87.66288802799991, 42.016987946000015], [-87.66284202799993, 42.01685694600006], [-87.66277902799997, 42.01673994600002], [-87.66266302799991, 42.01632794600004], [-87.66266395199993, 42.01627848300006], [-87.66266408599994, 42.016271297000046], [-87.66266501899996, 42.01622139600004], [-87.66268043099994, 42.01616402000007], [-87.66268785199995, 42.01613659600007], [-87.66268060399993, 42.016106248000014], [-87.66266448699996, 42.01604362800003], [-87.66251202799995, 42.01595394600002], [-87.66248702799997, 42.01587194600005], [-87.6624400279999, 42.01576894600002], [-87.66237002799991, 42.015619946000065], [-87.66226102799996, 42.015387946000025], [-87.66203302799993, 42.014903946000054], [-87.66197502799996, 42.014726946000046], [-87.66193602799994, 42.01450894600002], [-87.66188202799995, 42.01442894600006], [-87.66177802799994, 42.01426794600006], [-87.66171702799991, 42.014079946000024], [-87.66166302799995, 42.01394194600003], [-87.66152502799994, 42.013638946000064], [-87.66137302799994, 42.013295946000014], [-87.66118402799995, 42.01286894600001], [-87.66108002799994, 42.01266494600002], [-87.66106502799992, 42.01260794600005], [-87.66106502799992, 42.01257348500007], [-87.66106502799992, 42.01255125900007], [-87.66106497899993, 42.01251640700007], [-87.66101102799993, 42.012389946000035], [-87.66074302799993, 42.01209894600003], [-87.66053702799995, 42.01186494600005], [-87.66037202799997, 42.01162094600004], [-87.6593425, 42.010563493000056], [-87.65926467699992, 42.01049664400005], [-87.65915002799991, 42.01039794600006], [-87.65894202799994, 42.01016294600003], [-87.65884406299992, 42.01002775900002], [-87.65865563999992, 42.009867454000066], [-87.65860802799992, 42.00982694600002], [-87.65864694099994, 42.009813726000054], [-87.65865497499998, 42.00981099700004], [-87.65867955999992, 42.00980264500002], [-87.65864794199993, 42.00975626000007], [-87.65849802799993, 42.009548946000045], [-87.65837902799994, 42.009417946000035], [-87.65727765299991, 42.00821063900002], [-87.65685702799998, 42.00786494600003], [-87.65655655599991, 42.00753351100006], [-87.65620874799997, 42.007211936000026], [-87.65596371399992, 42.00631298700005], [-87.65594837099994, 42.00625669800006], [-87.65596085899992, 42.00625597300006], [-87.65708003799992, 42.006235964000034], [-87.65715311699994, 42.00619249500004], [-87.65721776399992, 42.00602264800005], [-87.65723834599997, 42.00584554800002], [-87.65723996599995, 42.00583148400005], [-87.65723840799993, 42.005787890000065], [-87.65723302799994, 42.00564494600006], [-87.65723164999994, 42.005587860000034], [-87.65722740599993, 42.00541202000004], [-87.65722602799994, 42.00535494600006], [-87.65714702799993, 42.00503694600006], [-87.65706102799997, 42.004816946000055], [-87.65697302799997, 42.00461394600005], [-87.65691802799994, 42.00444294600003], [-87.65690854199993, 42.004325466000054], [-87.65690502799995, 42.00428194600004], [-87.65690837799995, 42.00426899400002], [-87.65693486199993, 42.00416658900007], [-87.65702079499992, 42.00391363200002], [-87.65702200399991, 42.00389342600004], [-87.6570218499999, 42.00385625000006], [-87.65702175299995, 42.00383625500005], [-87.65702130699997, 42.00374447900003], [-87.65702103099993, 42.00368758600006], [-87.65700002799996, 42.00353994600003], [-87.65689902799993, 42.00309894600007], [-87.65679402799995, 42.00274094600006], [-87.65667702799998, 42.00239094600004], [-87.65654502799997, 42.00210594600002], [-87.65639602799995, 42.00183294600003], [-87.65601302799995, 42.00107194600002], [-87.65599895099993, 42.00100404500005], [-87.65599602799993, 42.000989946000054], [-87.65599891799995, 42.00094742300006], [-87.65600299399993, 42.00088744200007], [-87.65603564099993, 42.00073417700002], [-87.65604089699995, 42.00070955900002], [-87.65603576999997, 42.000676443000025], [-87.65603002799996, 42.00064094600002], [-87.65581202799994, 42.00000394600004], [-87.65571502799995, 41.99971894600002], [-87.65561402799993, 41.99950794600004], [-87.65548602799996, 41.999275946000054], [-87.65528502799997, 41.99903394600004], [-87.65514302799993, 41.998896946000045], [-87.65497702799995, 41.99868394600002], [-87.65487702799997, 41.99855194600007], [-87.65480643399995, 41.99840829600003], [-87.65479059099991, 41.99837662800007], [-87.65476458599993, 41.99835771500004], [-87.65480440599997, 41.99835224300005], [-87.65518515599994, 41.99828931200004], [-87.65596292999996, 41.99811584600007], [-87.65606291399995, 41.99815302000006], [-87.65620891599995, 41.99816702200006], [-87.65701491599997, 41.998165022000066], [-87.65862691499993, 41.99815602200003], [-87.65891491499995, 41.99815502300003], [-87.65928591599997, 41.99812102200006], [-87.65949191499996, 41.99810202200006], [-87.65965991499995, 41.99809602300006], [-87.66054391599992, 41.99807102300007], [-87.660552, 41.99807092200005], [-87.66175191599994, 41.998056022000064], [-87.66302391499993, 41.99804702200003], [-87.66420691499995, 41.998028022000014], [-87.66541691599996, 41.99801702200004], [-87.66661891499994, 41.99801502300004], [-87.66780791599994, 41.99800602200002], [-87.66903391599996, 41.99799902200004], [-87.67023391499995, 41.99796802200005], [-87.67055691599994, 41.99796002200002], [-87.67067591599992, 41.997958023000066], [-87.67085991499992, 41.99795402300002], [-87.67197491599993, 41.997933022000026], [-87.67222091599993, 41.99792802200004], [-87.67344791499993, 41.99790302200005], [-87.67496891299997, 41.997878077000045], [-87.67513891599995, 41.99787602300005], [-87.67545391699996, 41.99787102200002], [-87.67680402599993, 41.99794686500006], [-87.67699691599992, 41.998402022000036], [-87.67761191599996, 41.99980002200003], [-87.67790591599997, 42.00039602300005], [-87.67838791499994, 42.00146402200004], [-87.67926291599996, 42.00329602200002], [-87.67964191599997, 42.00414502200004], [-87.67972791599998, 42.004338023000045], [-87.68011991499995, 42.005204022000036], [-87.68041691499995, 42.00580302300005], [-87.68071091499993, 42.00643302200006], [-87.68115591499995, 42.007355022000056], [-87.68127291599994, 42.00762602200005], [-87.68183091599994, 42.00882402400003], [-87.68188391599992, 42.008956022000056], [-87.68226791599993, 42.010053023000054], [-87.68229991599995, 42.010159022000025], [-87.68269191599994, 42.01126202300002], [-87.68308391599996, 42.012308023000045], [-87.68322091599995, 42.01275002300007], [-87.68368491599992, 42.01404302200006], [-87.68387891599995, 42.01458202300005], [-87.68406791599992, 42.01515002200005], [-87.68416791499997, 42.015602023000035], [-87.68429391499993, 42.01653602200001], [-87.68441391599997, 42.01765202200004], [-87.68444891599992, 42.018003023000055], [-87.68460837299993, 42.019373893000065], [-87.68461541199996, 42.019429946000045], [-87.68460941899995, 42.019429946000045], [-87.68360472099991, 42.019429946000045], [-87.68315801699997, 42.019430030000066], [-87.68273350199995, 42.019465896000035], [-87.68195802699994, 42.01953043200005], [-87.68075801299992, 42.01953047400007], [-87.67945819699996, 42.01953032000005], [-87.67927376099996, 42.019438348000044], [-87.67925815299996, 42.01942978700003], [-87.67860373199991, 42.01931080100007], [-87.67854328099996, 42.01929981000005], [-87.67815820499997, 42.01922974100005], [-87.67775804599992, 42.01912994600007], [-87.67705804599993, 42.01912994600007], [-87.67645804599991, 42.01912994600007], [-87.67395800499996, 42.01913009200006], [-87.67331428899993, 42.019222051000035], [-87.67325919199996, 42.01923009000005], [-87.67330153199997, 42.019266990000055], [-87.67331497999997, 42.01927871100001], [-87.67341895799994, 42.01937055100006], [-87.67348893899997, 42.01943173000006], [-87.67431693699993, 42.020153360000045], [-87.67439052899992, 42.02021621700003], [-87.67462099399994, 42.020416952000055], [-87.67485693199994, 42.02062258100005], [-87.67529407299995, 42.02100356600003], [-87.67568865699995, 42.021347461000055], [-87.67709504899993, 42.02257318700003], [-87.67711162199997, 42.02258839900002], [-87.67715811099997, 42.02262913000003], [-87.67728869599992, 42.02289084000005], [-87.67732945899995, 42.02297236500004], [-87.67735725099996, 42.02302921500005], [-87.67732946999993, 42.02302948000004], [-87.67382350899996, 42.02303019400005], [-87.6724510459999, 42.02302994600006], [-87.67115103799992, 42.02302996500003], [-87.66642811899996, 42.023119195000035], [-87.66585814199996, 42.02312979900005]]]}, "properties": {"city": "Chicago", "name": "Rogers Park", "regionid": "269605", "geo_point_2d": [42.00961387156982, -87.67010887599854], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.7420571861749, 41.77507961197302], [-87.74216243236447, 41.774662220098804], [-87.74215300027504, 41.774832000406924], [-87.74345299994606, 41.774832000406924], [-87.74465300040329, 41.77493199962064], [-87.74595300007421, 41.774832000406924], [-87.74700979641257, 41.774832000406924], [-87.7469583310749, 41.775570773486606], [-87.74697979485194, 41.77613956357667], [-87.74700488669141, 41.77695187556669], [-87.74702945841491, 41.777846286302655], [-87.74703800396176, 41.77823218319221], [-87.74647743529897, 41.778243947885244], [-87.74533938637366, 41.77826804284639], [-87.74403778842975, 41.778289458042494], [-87.74265483958915, 41.77831342631368], [-87.74219993638071, 41.778318720544426], [-87.74216408580752, 41.777597725684], [-87.7421317968927, 41.77694835973134], [-87.74209849630188, 41.77621780606684], [-87.7420793044504, 41.77573033304232], [-87.7420571861749, 41.77507961197302]]]}, "properties": {"city": "Chicago", "name": "Clearing (E)", "regionid": "403337", "geo_point_2d": [41.776563579685735, -87.744547513021], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "MultiPolygon", "coordinates": [[[[-87.63854, 41.8528], [-87.63853249799993, 41.85311510000001], [-87.63856158099996, 41.853931852000066], [-87.63864500999993, 41.85442399500003], [-87.63870494799993, 41.854782916000026], [-87.63876225399994, 41.85512607400005], [-87.63883135099994, 41.855564794000024], [-87.63888593599994, 41.85642005000004], [-87.63891538799997, 41.85709007000001], [-87.63894004799992, 41.857652886000025], [-87.63894691599992, 41.85809902300007], [-87.63895591599996, 41.85887202300005], [-87.63898291599996, 41.85991102300004], [-87.63898878899992, 41.860009152000025], [-87.64043991599996, 41.85999602200007], [-87.64092391499996, 41.85999802200006], [-87.64192591499992, 41.85998502300003], [-87.64292591599995, 41.85997302200006], [-87.64384891499995, 41.85995002200007], [-87.64402091599997, 41.85994702300007], [-87.64412091499995, 41.85995302200006], [-87.64420691599994, 41.85995902100005], [-87.64428291599994, 41.85995702200005], [-87.64442283999992, 41.85995797300006], [-87.64445, 41.860569977000054], [-87.64452081199991, 41.86197208400005], [-87.64456792699995, 41.863252321000054], [-87.64469386199991, 41.86484144400003], [-87.64467558799997, 41.866307019000025], [-87.64484328699993, 41.868195059000065], [-87.64494978399992, 41.86930135700003], [-87.64505143999995, 41.87000097300005], [-87.64527289399994, 41.871629515000045], [-87.64560567299996, 41.872836132000025], [-87.64561197799995, 41.873690875000065], [-87.64560016299995, 41.87472869900005], [-87.64556004299996, 41.875738908000066], [-87.64547964299996, 41.875734260000044], [-87.64548307099993, 41.87576258600006], [-87.64401268299996, 41.87574], [-87.64163146299992, 41.875681835000066], [-87.64004582599993, 41.87562], [-87.63888784299995, 41.87563411200004], [-87.63701597799994, 41.87555440100004], [-87.63666007599993, 41.875608635000034], [-87.63633038499992, 41.87565887500004], [-87.63337470299994, 41.87565], [-87.63260851399994, 41.87565694000005], [-87.63135478899993, 41.87567748000002], [-87.63069771199996, 41.87569028600006], [-87.63067410199992, 41.87522212800007], [-87.63064858399996, 41.874035091000046], [-87.63062179999996, 41.87292088900004], [-87.63062, 41.87223], [-87.63058503199993, 41.87152598900007], [-87.63052912799996, 41.87006429600006], [-87.63048634799996, 41.86915943600007], [-87.63044763999993, 41.86790605800006], [-87.63043833099992, 41.867440045000066], [-87.63040521699996, 41.86637366200005], [-87.6303618099999, 41.86525509300002], [-87.63033211099994, 41.86448978900005], [-87.63030623899992, 41.86379399100007], [-87.63027922299993, 41.86288898500003], [-87.63025, 41.86170357600002], [-87.63026, 41.860658408000056], [-87.62730582899994, 41.86080144000003], [-87.62736004599996, 41.86240120900003], [-87.62743765999993, 41.86483952700006], [-87.62748376699993, 41.86602321000003], [-87.62751211799994, 41.86746095100005], [-87.62753264599993, 41.86846208300005], [-87.62757257599996, 41.87006826100002], [-87.62760689199995, 41.87125071400003], [-87.62761784799993, 41.87228007600003], [-87.62761, 41.87228], [-87.62762095799994, 41.87257221600003], [-87.62764, 41.87361849100005], [-87.6276716199999, 41.87508131400005], [-87.62768795499993, 41.87564139600004], [-87.62723787899992, 41.87557397800003], [-87.62668590799994, 41.87566177900003], [-87.62564115499995, 41.87567431900003], [-87.62480534599995, 41.875690816000024], [-87.62426, 41.87568], [-87.62425894899994, 41.87568864900003], [-87.62382509099994, 41.87562789000002], [-87.62257483099994, 41.87558321700003], [-87.61619115999996, 41.87564705300007], [-87.61529147499992, 41.875656050000025], [-87.61530383999995, 41.87504098000004], [-87.61531568399992, 41.874431946000065], [-87.61621514399995, 41.874431946000065], [-87.61625018699992, 41.874431946000065], [-87.61624977899996, 41.87437563500003], [-87.61623311599998, 41.87387505000003], [-87.61623121499996, 41.87381800800005], [-87.61622921099995, 41.87376157000005], [-87.6162281, 41.873730309000045], [-87.61621202799994, 41.873277946000016], [-87.61615407399992, 41.87166112900006], [-87.61615303399995, 41.87163210900007], [-87.61615405099995, 41.871604316000045], [-87.61623689899993, 41.86936740600003], [-87.61625095399992, 41.868987921000034], [-87.61625171799993, 41.868931232000065], [-87.61624776999996, 41.86892785700007], [-87.61546284099994, 41.86831735600003], [-87.61535204599993, 41.86823194600003], [-87.61485200199995, 41.86823211600005], [-87.61339194799994, 41.868473277000064], [-87.61189226599997, 41.86872098400004], [-87.61184597299993, 41.86867224200005], [-87.61180298299993, 41.86862594600006], [-87.61172210099991, 41.86852520400003], [-87.61164610099995, 41.868415204000016], [-87.61161045699997, 41.86835931700004], [-87.61157902699993, 41.868300956000034], [-87.6115519799999, 41.868240439000026], [-87.61152946499998, 41.868178094000015], [-87.61150546399992, 41.86810309400004], [-87.61147633999994, 41.867985166000025], [-87.61146971299993, 41.867941274000025], [-87.61143299499996, 41.86765703500003], [-87.61106546199991, 41.86767069900002], [-87.61102677199993, 41.86767130700007], [-87.61086280499993, 41.86767035400003], [-87.61075716599997, 41.867698659000034], [-87.61059620699994, 41.86771229700002], [-87.61012197799992, 41.86770726600002], [-87.6099640589999, 41.86770543000005], [-87.60804585199996, 41.867745052000025], [-87.60800666099998, 41.867771836000024], [-87.60796205299992, 41.86779594000006], [-87.6077380529999, 41.86790894000005], [-87.60762941999994, 41.86795506700002], [-87.60746042099997, 41.86801406700003], [-87.60734608299995, 41.868045696000074], [-87.60710908299994, 41.868094696000064], [-87.60697391799991, 41.868112097000036], [-87.60683991799992, 41.86811909700002], [-87.60674365899993, 41.868118968000026], [-87.60656165899994, 41.868108967000055], [-87.60649415599991, 41.86810269900006], [-87.60634915699993, 41.86808369900006], [-87.60620462099995, 41.86805250000002], [-87.60591162099995, 41.867963500000016], [-87.60579517499994, 41.86791910500005], [-87.60563317499995, 41.86784410600006], [-87.60556133899995, 41.86780683600005], [-87.60542991499993, 41.86771437100003], [-87.60528290299996, 41.86758882100002], [-87.60520698199994, 41.86751586500002], [-87.60519505499995, 41.86750122700004], [-87.60519348599993, 41.86744261100005], [-87.60513696099996, 41.86533042400003], [-87.6051354109999, 41.86527247800006], [-87.60515827699997, 41.865246285000055], [-87.60518555899993, 41.86521899900003], [-87.60518769299993, 41.864848816000055], [-87.60519160999995, 41.86476994400004], [-87.60520243499994, 41.86469171900005], [-87.60522008299992, 41.86461474700007], [-87.60524441799997, 41.86453962000006], [-87.60527525199993, 41.864466919000044], [-87.60531234899995, 41.86439720500004], [-87.60535541999995, 41.86433101500005], [-87.60540413399994, 41.86426886100002], [-87.60542418899995, 41.86424579500004], [-87.60552292699997, 41.864149079000065], [-87.60565133899995, 41.86405916400002], [-87.60579341299994, 41.86399291400004], [-87.60594483399994, 41.863952341000065], [-87.60600010599994, 41.86394435500005], [-87.60612410699994, 41.86393035600002], [-87.60619881899996, 41.86392424000007], [-87.60627377699994, 41.86392503300004], [-87.60634834299996, 41.86393272600002], [-87.60642188299992, 41.86394725500002], [-87.60649377299994, 41.863968496000034], [-87.60643469999994, 41.86369000700005], [-87.60616494999991, 41.86296182500006], [-87.60611612599993, 41.86294008600002], [-87.60606676799993, 41.862914896000056], [-87.60599333199994, 41.86291116200004], [-87.60588283399993, 41.862898659000045], [-87.60573141299993, 41.86285808600007], [-87.60558933899993, 41.862791836000035], [-87.60546092599996, 41.862701921000046], [-87.60541294099993, 41.862658624000055], [-87.60537994099995, 41.86262662500007], [-87.60531707899996, 41.86255907300004], [-87.60522716399997, 41.86243066100006], [-87.60516091399994, 41.862288587000066], [-87.60512034099996, 41.86213716600002], [-87.60510736399992, 41.86201611600005], [-87.60508536399998, 41.86145311500007], [-87.60508480099995, 41.86140310500007], [-87.60509236399994, 41.860946521000066], [-87.60506047799996, 41.86019091700006], [-87.6050628369999, 41.860077695000015], [-87.60507283599996, 41.85995869500004], [-87.60508244, 41.85988302800006], [-87.60509842199994, 41.85980844700003], [-87.60512066599992, 41.859735488000055], [-87.60514901199994, 41.859664676000065], [-87.60518325599996, 41.85959652100006], [-87.60522315299993, 41.85953151300003], [-87.60526841499994, 41.85947012000002], [-87.60530724699996, 41.85942202300003], [-87.60536093399996, 41.85936231600004], [-87.60541973499991, 41.85930763900005], [-87.60548318099994, 41.85925842700003], [-87.60555076599994, 41.85921507400006], [-87.60562194999993, 41.85917792600002], [-87.60569616599996, 41.85914727800007], [-87.60577282099993, 41.859123376000056], [-87.60585130199995, 41.85910641100002], [-87.60596677099991, 41.85908748100007], [-87.60608668499997, 41.85907756500005], [-87.60608449899995, 41.85905129600007], [-87.60602517899994, 41.858594918000044], [-87.60602045199992, 41.85854959300007], [-87.60563840899992, 41.853697538000056], [-87.60560987899997, 41.85362568500005], [-87.60558797899992, 41.85355154300003], [-87.60557288799995, 41.85347572000005], [-87.60556473099994, 41.85339884200004], [-87.60556357499993, 41.85332154100006], [-87.60556942999992, 41.85324445400004], [-87.60557442999998, 41.85320045300004], [-87.60558234099993, 41.853145834000024], [-87.60562291399997, 41.852994413000026], [-87.60568916499993, 41.85285233900003], [-87.60577908, 41.85272392700004], [-87.60585291799995, 41.85264591000003], [-87.60586891799991, 41.852630910000066], [-87.60592230699994, 41.85258466100002], [-87.60597915999993, 41.852542744000054], [-87.60603912799995, 41.85250541900006], [-87.60610183999992, 41.852472915000064], [-87.60616690899995, 41.85244543400005], [-87.60623393499995, 41.85242314400005], [-87.60630993499993, 41.85240114400004], [-87.60640383499997, 41.85237934000001], [-87.60648494599995, 41.85236881500003], [-87.6074879459999, 41.85228481600006], [-87.60752476399992, 41.85228249200002], [-87.60766109299996, 41.852276690000046], [-87.60773601499992, 41.85226479800007], [-87.60778751599995, 41.85225814100004], [-87.60791751699996, 41.85224514100002], [-87.60798524799996, 41.85224094100004], [-87.60822589299991, 41.85223511900005], [-87.60863754199991, 41.85221841700006], [-87.60867891399992, 41.852217691000064], [-87.60886191399997, 41.85221869200006], [-87.60899875599993, 41.852232001000054], [-87.60878677699996, 41.85198209100002], [-87.60871451399997, 41.85188362400003], [-87.60853251499996, 41.851596624000074], [-87.60849569499996, 41.851532945000024], [-87.60846420199991, 41.851466469000044], [-87.60843824699997, 41.851397642000045], [-87.60834124699993, 41.85110464200004], [-87.6083150549999, 41.851007677000034], [-87.60829205499992, 41.850898677000025], [-87.60827907, 41.85082003300003], [-87.60827309999993, 41.850740548000026], [-87.60827419199993, 41.85066084600003], [-87.60828819099997, 41.85041984600007], [-87.60830414199994, 41.85029562800002], [-87.60838552099992, 41.84988873300005], [-87.60852192199995, 41.84918084300005], [-87.60853130499997, 41.84913783900002], [-87.60860361299994, 41.84884034200007], [-87.60860543699992, 41.84860991800007], [-87.60858467099995, 41.84838028600006], [-87.60948229999994, 41.84832099000004], [-87.61463153099997, 41.84798084000004], [-87.61630352999995, 41.84797129600001], [-87.61787012599994, 41.84795288100002], [-87.61811525099992, 41.84795], [-87.61992506899995, 41.84792283100005], [-87.62145891399994, 41.847903584000036], [-87.62305893999994, 41.84790481200002], [-87.62461647599994, 41.847934521000056], [-87.62569733899993, 41.84765507300004], [-87.62623428999994, 41.84725844000006], [-87.62698295699992, 41.84720558500003], [-87.62696825799996, 41.84630639900006], [-87.62694614299993, 41.84556802300004], [-87.62692660699994, 41.844985185000034], [-87.62689088799992, 41.84386893500004], [-87.62687945099995, 41.84240311800004], [-87.62684831699993, 41.84135937100006], [-87.62680914599996, 41.84031046200004], [-87.62675821299995, 41.83884914400004], [-87.62674, 41.83828], [-87.62738226199995, 41.838274603000045], [-87.62863660899994, 41.83827], [-87.62940272799992, 41.83826453400006], [-87.62976881999995, 41.83827485200003], [-87.62976890899994, 41.838254803000034], [-87.63103806199994, 41.83822005300003], [-87.63104926399993, 41.838777704000044], [-87.63103275199994, 41.83982565600006], [-87.63150214699992, 41.845188010000065], [-87.63141330899995, 41.846233489000035], [-87.63156305799997, 41.84735021800002], [-87.63207950299993, 41.847366945000026], [-87.63205087699997, 41.84776802800007], [-87.63190953499996, 41.848673859000016], [-87.63192477199996, 41.84957948100003], [-87.63195188499995, 41.85055336100003], [-87.63197596899992, 41.85125052500006], [-87.63198, 41.851739642000034], [-87.63199654799995, 41.85250198200003], [-87.632, 41.85289], [-87.63295680399993, 41.85288465500001], [-87.63532568299995, 41.85286], [-87.63637087699998, 41.85285601900005], [-87.63769395999996, 41.85280292600003], [-87.63854, 41.8528]], [[-87.61125448699991, 41.858490529000036], [-87.61132939599995, 41.85857404700004], [-87.61133142599991, 41.858524322000044], [-87.61133781899994, 41.85844789100001], [-87.61135070099992, 41.858372283000044], [-87.61136997899995, 41.858298048000044], [-87.61139551199992, 41.85822572500006], [-87.61142711399997, 41.858155841000034], [-87.61146455599996, 41.85808890300005], [-87.61150756499994, 41.858025399000034], [-87.61155056499994, 41.85796739900007], [-87.61159938699993, 41.85790716000002], [-87.61165321599997, 41.857851350000026], [-87.61171165299997, 41.857800384000015], [-87.61177426299996, 41.857754641000035], [-87.61184057999996, 41.85771446200004], [-87.6119025799999, 41.85768046200006], [-87.61202741399994, 41.85762391400004], [-87.61211031999993, 41.85760169900003], [-87.61213664599993, 41.857535885000054], [-87.61217640699991, 41.857398982000056], [-87.61217582499994, 41.857124476000074], [-87.61216977, 41.85708554700005], [-87.61206748899994, 41.85685592500004], [-87.61204391299997, 41.85679758600002], [-87.6120028839999, 41.85664344500002], [-87.61197549599994, 41.85659778800004], [-87.61193088699996, 41.85654822400004], [-87.61183593299994, 41.856466497000035], [-87.61168234399995, 41.85637693600005], [-87.61098338799994, 41.85595280000006], [-87.61091975099995, 41.855910372000025], [-87.61081975099994, 41.855837372000046], [-87.61072523499995, 41.855757877000016], [-87.61060823499994, 41.855644876000035], [-87.61053799899996, 41.85557383700006], [-87.61042326599993, 41.85638710400007], [-87.61042117099996, 41.85642847900003], [-87.61041396899995, 41.856529709000014], [-87.61052383199993, 41.856637056000054], [-87.61062131399994, 41.85675004400007], [-87.61075731399993, 41.856937044000055], [-87.61080757499995, 41.857014163000024], [-87.61089420599993, 41.85716325000003], [-87.61096108699995, 41.85730641300005], [-87.61098237299993, 41.857372796000035], [-87.61103137199993, 41.85754879600005], [-87.61105773099995, 41.85768132100002], [-87.61107173099995, 41.85779632100007], [-87.61107832199995, 41.857905], [-87.61107302899994, 41.858002425000045], [-87.61103955699996, 41.85830957800004], [-87.61111395699997, 41.85836368800004], [-87.61116307299991, 41.85840208000006], [-87.61125448699991, 41.858490529000036]], [[-87.61066681899995, 41.86390150900006], [-87.61071973299993, 41.86402102200002], [-87.61083235799995, 41.86425238300006], [-87.61092765099991, 41.86442681700003], [-87.61105052499991, 41.86462353100005], [-87.61110271099993, 41.864668059000046], [-87.61115021599993, 41.86471754900002], [-87.61119257199992, 41.864771513000036], [-87.61122935899992, 41.86482941600007], [-87.61126021399996, 41.864890686000074], [-87.61135641399991, 41.864885159000025], [-87.61137730899996, 41.86488420200003], [-87.61185737099996, 41.86486781000007], [-87.61189116499996, 41.86479533900006], [-87.61193819699994, 41.864722273000034], [-87.61195819699996, 41.86469427300005], [-87.61200107999998, 41.86463892700004], [-87.61204165899991, 41.86459732700006], [-87.61230711899995, 41.86394852200003], [-87.61240095099997, 41.86365451500005], [-87.61251329899993, 41.863091982000064], [-87.61251078599996, 41.86299791800007], [-87.61248940299996, 41.86287716600002], [-87.6124397849999, 41.862760840000064], [-87.61242673199995, 41.86272851400001], [-87.61236810299992, 41.86257481300003], [-87.61216825999996, 41.862171709000044], [-87.61207465199996, 41.86201407500003], [-87.61197492699993, 41.86194392100003], [-87.61186407999996, 41.861833073000064], [-87.61177416499997, 41.861704661000026], [-87.61170791399995, 41.86156258600004], [-87.61166734099996, 41.86141116600003], [-87.61165382599994, 41.861248484000036], [-87.61155607899995, 41.86114407300004], [-87.61154605499993, 41.86112975700006], [-87.61150592099992, 41.861187073000046], [-87.61138903799996, 41.861302147000046], [-87.61138149899992, 41.861341479000025], [-87.61137460199993, 41.861420583000026], [-87.61136075499994, 41.86149877000002], [-87.61134006599995, 41.86157543100005], [-87.61131269499995, 41.861649968000044], [-87.61127885599994, 41.86172180100004], [-87.61123881299993, 41.86179036800006], [-87.61119287699994, 41.861855136000024], [-87.61116487699995, 41.86189113600005], [-87.61114392099992, 41.86191707300002], [-87.61107252899993, 41.86198846400003], [-87.61091424799994, 41.86222480800006], [-87.61080697899995, 41.86251742000007], [-87.61065167099994, 41.86331325400004], [-87.61063905699993, 41.86336855200005], [-87.61063035799997, 41.86340181600007], [-87.61062600799991, 41.86352838100002], [-87.61066681899995, 41.86390150900006]]], [[[-87.63854, 41.8528], [-87.63936699899993, 41.85125189700004], [-87.63998789399993, 41.85025985400004], [-87.64120480999995, 41.84943102300007], [-87.64205049799993, 41.84901976600001], [-87.64242935999992, 41.84876856300007], [-87.64250891599991, 41.84881302100006], [-87.6426009159999, 41.848845023000074], [-87.64263691899997, 41.84885720800002], [-87.64273791799994, 41.848815022000046], [-87.64335191599996, 41.84867902200004], [-87.64340597799996, 41.84866984500007], [-87.64344391399996, 41.848713024000055], [-87.64348491599996, 41.848786022000056], [-87.64363591499993, 41.84895602200003], [-87.643801, 41.849217], [-87.64424006999994, 41.849457056000055], [-87.644721, 41.84972], [-87.64470924699997, 41.849720993000055], [-87.64494337199993, 41.85009447600003], [-87.64520675499993, 41.85057831400007], [-87.64538163699994, 41.85095950200007], [-87.6455909869999, 41.851540764000035], [-87.64572551099995, 41.85215163700007], [-87.64580144499993, 41.85264156700003], [-87.64297597899996, 41.85269077100002], [-87.64082035399996, 41.852733683000054], [-87.640824, 41.852728], [-87.64043944699993, 41.85273793500004], [-87.64004605799992, 41.852748099000046], [-87.63947570299995, 41.852760450000055], [-87.63854, 41.8528]]]]}, "properties": {"city": "Chicago", "name": "South Loop", "regionid": "269607", "geo_point_2d": [41.86077111977609, -87.62556832609671], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.7180210359999, 41.99720697000003], [-87.71844012599988, 41.99719749300004], [-87.7183853959999, 41.997164066000025], [-87.71601412999989, 41.99571575700008], [-87.71135966099988, 41.992855797000054], [-87.70961670799993, 41.99176446600005], [-87.70940498399992, 41.991623819000026], [-87.70902028099992, 41.99026176100005], [-87.7091713959999, 41.990259978000026], [-87.7102186319999, 41.99024], [-87.71217703199991, 41.99023], [-87.71326065499993, 41.99022071000007], [-87.71439422199988, 41.99020219800006], [-87.71563075199991, 41.99018148700008], [-87.71717642599991, 41.990156074000076], [-87.71815537699989, 41.990144997000066], [-87.71876, 41.99014], [-87.71944372599992, 41.99018], [-87.7206811419999, 41.99016958200007], [-87.72202056799993, 41.99016], [-87.7232574529999, 41.99016], [-87.72531892599993, 41.99004], [-87.72727632099989, 41.98997032500005], [-87.72857043799988, 41.99007191100007], [-87.72862312499987, 41.99030147300004], [-87.72867064899991, 41.99050854100005], [-87.72871, 41.99068], [-87.72875, 41.99127018600005], [-87.72875238399989, 41.99224949600004], [-87.7287582399999, 41.993280200000065], [-87.72876850999988, 41.99389435900007], [-87.7287767959999, 41.99446685100008], [-87.72879205199987, 41.99498176800005], [-87.72880179699989, 41.99565094200004], [-87.7288071589999, 41.996578574000054], [-87.7288152769999, 41.997173893000074], [-87.72881632299992, 41.997229946000054], [-87.72726004599988, 41.997229946000054], [-87.72626004599992, 41.997229946000054], [-87.72526004599987, 41.997229946000054], [-87.72501304599989, 41.997229946000054], [-87.72415904599988, 41.997229946000054], [-87.72295904399992, 41.99722995000006], [-87.72156404799993, 41.997235941000035], [-87.72035904599991, 41.997229946000054], [-87.71915904599987, 41.997229946000054], [-87.71865906299989, 41.997229911000034], [-87.71838645899992, 41.997220084000034], [-87.7180210359999, 41.99720697000003]]]}, "properties": {"city": "Chicago", "name": "Pulaski Park", "regionid": "403269", "geo_point_2d": [41.993307701317775, -87.72063113644991], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.74552591563983, 41.938760022407365], [-87.74681001609804, 41.938725128558325], [-87.74682694928438, 41.939238478530534], [-87.74685686632922, 41.940152856208286], [-87.74688127201638, 41.941067240740914], [-87.74690123057741, 41.941982221853394], [-87.74691, 41.94239000000019], [-87.74692247812209, 41.94295151549421], [-87.74694332884033, 41.94355816035167], [-87.74695436145957, 41.94398291619719], [-87.7469789513854, 41.944831687407394], [-87.7470077834036, 41.94549752367644], [-87.74701671362106, 41.94602616744325], [-87.74630977147596, 41.94603623379483], [-87.74493423945631, 41.946063997149366], [-87.74420568696996, 41.94607386746603], [-87.74347865483485, 41.94608640250286], [-87.7422499999999, 41.9461], [-87.7422497933254, 41.94610000354688], [-87.74137178276658, 41.946115068476985], [-87.74048127956365, 41.94612248933701], [-87.73845740900452, 41.946144635853976], [-87.73619074436577, 41.946167136949455], [-87.73548015108564, 41.94618042035644], [-87.73484197765174, 41.945795736540184], [-87.73359520502021, 41.94504589013486], [-87.73239710739091, 41.94431420353767], [-87.73167595469398, 41.94386995128704], [-87.73025768264688, 41.94300068566356], [-87.7284815259211, 41.94191244867433], [-87.72754375639506, 41.94133871443075], [-87.72711592765161, 41.94107835895091], [-87.72711591596176, 41.9410780222399], [-87.7271119157773, 41.94091802205897], [-87.72710791559304, 41.940793022592196], [-87.72707791600861, 41.93988802314303], [-87.72704678058079, 41.93901789040873], [-87.72953291496191, 41.938986022033326], [-87.73191391500065, 41.93894902212636], [-87.73289191601629, 41.93893702247256], [-87.73384491587956, 41.93892502191938], [-87.73444891584286, 41.938916022403674], [-87.73512286147638, 41.938905076755375], [-87.73591691606343, 41.938893022242965], [-87.73692291567194, 41.9388770224048], [-87.73813991511423, 41.93886402180536], [-87.739359915594, 41.93884802196728], [-87.74058391535891, 41.93883202212911], [-87.74070091625562, 41.93883102208293], [-87.74082591482319, 41.93882902199057], [-87.74208091062114, 41.938813022152495], [-87.74312391553248, 41.93879802236042], [-87.7437889156097, 41.93878802189952], [-87.74429791569054, 41.93878002242998], [-87.74511991581033, 41.93876702183055], [-87.74552591563983, 41.938760022407365]]]}, "properties": {"city": "Chicago", "name": "Kilbourn Park", "regionid": "403287", "geo_point_2d": [41.94217275140614, -87.73827926845259], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.62285313799997, 41.90203080300005], [-87.62016766299995, 41.901597662000015], [-87.61975402799997, 41.90153094600003], [-87.61965578599992, 41.90139340700004], [-87.61941926499995, 41.90106227800003], [-87.61937966099995, 41.90100683200005], [-87.61941723699992, 41.90100622500005], [-87.62411739399992, 41.90080912100006], [-87.62831571699995, 41.900702096000025], [-87.62833437299996, 41.90122746900004], [-87.62835276899993, 41.90163137900002], [-87.62833333299993, 41.90219667400003], [-87.62845231999992, 41.90271072800004], [-87.62867154099995, 41.90357466300002], [-87.62869, 41.90393], [-87.62940416099995, 41.903918841000056], [-87.63061825599998, 41.90389882300006], [-87.63141, 41.90389], [-87.63145453899992, 41.904625899000045], [-87.63151279299994, 41.90640962900004], [-87.63155847999995, 41.90827075600003], [-87.63160491899995, 41.90988854800002], [-87.63167391599994, 41.911174022000054], [-87.63019891499991, 41.91118502200003], [-87.62885091499997, 41.91120402200004], [-87.62764191499994, 41.911233022000026], [-87.62694691599994, 41.911255023000066], [-87.62634290699997, 41.91127397000001], [-87.62616291499994, 41.91133602200005], [-87.62597888899992, 41.91139997700002], [-87.62592095199992, 41.91144110100003], [-87.62573891399994, 41.91148602100002], [-87.62559891599992, 41.91148502200007], [-87.62540791599996, 41.91147202300003], [-87.62514691599993, 41.91146002200002], [-87.62469631699994, 41.911441373000045], [-87.62534604399995, 41.910050198000064], [-87.62535468699996, 41.91003167700005], [-87.62534638099996, 41.90999234500003], [-87.62524302799993, 41.90953094600007], [-87.62488802799996, 41.907948946000026], [-87.62463802799992, 41.906834946000046], [-87.62443102799992, 41.905914946000046], [-87.62405402799997, 41.904230946000034], [-87.62285313799997, 41.90203080300005]]]}, "properties": {"city": "Chicago", "name": "Gold Coast", "regionid": "403303", "geo_point_2d": [41.906010802574, -87.62729235987793], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.67390565773391, 41.98327703441705], [-87.6745660489883, 41.983263424687344], [-87.67458991566761, 41.98411502219399], [-87.6746029153675, 41.98445802271673], [-87.67461891430638, 41.9864230224649], [-87.67460191442214, 41.98651702140205], [-87.67456691550665, 41.986610022991066], [-87.67453191659132, 41.98669002488027], [-87.67451491670695, 41.986780022733704], [-87.67452991559975, 41.9882370224473], [-87.67455991608328, 41.99012902242763], [-87.6745569159451, 41.99043402209795], [-87.67456691820466, 41.99052702278758], [-87.67461287086232, 41.99063813042712], [-87.67465691515882, 41.990728022160646], [-87.674720912713, 41.9909020238862], [-87.6747329159641, 41.991267022725005], [-87.67475491517962, 41.99191702211069], [-87.67484791586935, 41.9942790221729], [-87.6748969154301, 41.995456022469234], [-87.67492291572927, 41.9966880217036], [-87.67496791600496, 41.99784202273842], [-87.6749689133531, 41.997878076558614], [-87.67344791518597, 41.99790302195307], [-87.67222091618213, 41.99792802220614], [-87.67197491563421, 41.99793302243669], [-87.67085991549781, 41.99795402250539], [-87.67067591600929, 41.99795802268975], [-87.67055691591976, 41.99796002188257], [-87.67023391541964, 41.99796802225129], [-87.66903391586177, 41.997999021881704], [-87.66780791600458, 41.998006022204336], [-87.66661891515517, 41.998015022619335], [-87.66541691550495, 41.99801702181214], [-87.66420691548612, 41.998028022319225], [-87.66302391491314, 41.99804702229565], [-87.66175191563354, 41.998056021811365], [-87.66055199954262, 41.99807092211929], [-87.66051871779256, 41.99772596846654], [-87.66046825814665, 41.99690284470814], [-87.66045031390402, 41.995941614453464], [-87.66042799027497, 41.99511564017435], [-87.66040894220696, 41.994450486437735], [-87.66091231956648, 41.99442990721727], [-87.66215037517927, 41.994392026419916], [-87.66352677941642, 41.99436920712858], [-87.66462624151886, 41.99436], [-87.66620689607161, 41.9943521048314], [-87.66833716027219, 41.99433260885892], [-87.67008999999987, 41.99431], [-87.67019, 41.99431], [-87.67016, 41.99276], [-87.67009999999989, 41.9916], [-87.67007, 41.99067], [-87.67002, 41.98978], [-87.66832525677401, 41.98860631186093], [-87.66678188564347, 41.98765264211384], [-87.66551973408278, 41.98686820481223], [-87.66439720338904, 41.986175202085654], [-87.66284077439137, 41.98521652740211], [-87.66181305814993, 41.98459263676286], [-87.66073445342803, 41.98394093587361], [-87.66008, 41.98353], [-87.66007816157598, 41.98352848327984], [-87.6609850559568, 41.98351563403249], [-87.6621526662879, 41.983503338800986], [-87.66290877307947, 41.98349620898816], [-87.66442088576308, 41.98346354211069], [-87.66723978154901, 41.98341618976198], [-87.66909350343585, 41.983366082490846], [-87.67163846693491, 41.983320874625306], [-87.67253071502827, 41.983305446420204], [-87.67390565773391, 41.98327703441705]]]}, "properties": {"city": "Chicago", "name": "Edgewater", "regionid": "269578", "geo_point_2d": [41.99093836075318, -87.66920284730631], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.63315602799996, 41.94273094600003], [-87.63319659399995, 41.942719882000056], [-87.63320717599997, 41.942716996000044], [-87.63396818099993, 41.94250944900005], [-87.63425500399994, 41.94243122500006], [-87.63462296699996, 41.94178880200007], [-87.63465508799996, 41.941731445000066], [-87.63462239799996, 41.94173094600006], [-87.63425602799995, 41.94173094600006], [-87.63423595799995, 41.94159045600002], [-87.63419656399996, 41.941314696000056], [-87.63398218799993, 41.94077347100005], [-87.63398345899995, 41.94071683500004], [-87.63399073699996, 41.94039247800003], [-87.63399086799994, 41.94038663600002], [-87.63399201299994, 41.94033562700002], [-87.63399087499991, 41.94032906100005], [-87.63394748299997, 41.94010747200007], [-87.63272531999993, 41.937591253000036], [-87.63271927299995, 41.937585049000056], [-87.63270586399993, 41.93755119600006], [-87.63235602799992, 41.936830946000036], [-87.63220357399994, 41.93648835200002], [-87.63155502799997, 41.93503094600003], [-87.63150380799996, 41.93495923800002], [-87.63129393199993, 41.934665412000065], [-87.63057657799997, 41.933661116000046], [-87.63055502799993, 41.933630946000044], [-87.63057598399993, 41.93360475100007], [-87.63094393199997, 41.933144815000034], [-87.63095443399997, 41.933131688000046], [-87.63094452199991, 41.93308695100005], [-87.63092155499996, 41.93299077100004], [-87.63090838399994, 41.93293561200004], [-87.63092051799997, 41.93293381800003], [-87.63099300399992, 41.932923098000046], [-87.63115802099995, 41.93291201300007], [-87.63128900099994, 41.932888134000045], [-87.63134409799991, 41.93285747000003], [-87.63134899099997, 41.932845500000035], [-87.63141605099995, 41.93285416900004], [-87.63188294899997, 41.93291919300002], [-87.63204291599997, 41.93282702200001], [-87.63210088999993, 41.93279297700007], [-87.63252798199993, 41.93271184000002], [-87.63256591499996, 41.93275902300006], [-87.63260779899996, 41.932835151000056], [-87.63264391699992, 41.93285102000004], [-87.63272491599996, 41.93286102100006], [-87.63380891499995, 41.93284002200004], [-87.63404491499993, 41.93282202300002], [-87.63618191499994, 41.932858022000055], [-87.63793391599994, 41.932916022000015], [-87.63883891499994, 41.932892023000065], [-87.63936491599995, 41.93290002200006], [-87.64035391599995, 41.93288902200003], [-87.64131291499994, 41.93286102200005], [-87.64250291499994, 41.93283802200005], [-87.64272091499996, 41.93283702200005], [-87.64357391599995, 41.932822022000025], [-87.64488691499996, 41.93280302200002], [-87.64583091599995, 41.93277402300003], [-87.64655991499994, 41.93275902200002], [-87.64664591499995, 41.93276102200007], [-87.64764491599998, 41.93275002200004], [-87.64899180799993, 41.93271705800003], [-87.64900349299992, 41.93312480000002], [-87.64903286099994, 41.93401441200005], [-87.64906484199996, 41.93499020900003], [-87.64910481599996, 41.93587631700006], [-87.64913, 41.93684922200002], [-87.64920022399996, 41.938628550000026], [-87.64923225899994, 41.93943734200007], [-87.64926032399994, 41.940571380000044], [-87.64928295599996, 41.941299679000046], [-87.64932317399995, 41.94219520900002], [-87.64934662899996, 41.943001634000034], [-87.64936797099992, 41.94397269000005], [-87.64939911199995, 41.94511247500002], [-87.64942021499995, 41.94591989700006], [-87.64944153899992, 41.94672770400006], [-87.64949396999997, 41.94818322700006], [-87.64954050999994, 41.94947733300005], [-87.64956, 41.95061250000003], [-87.64959366699995, 41.95182834300003], [-87.64962, 41.95275], [-87.64859208099993, 41.952809076000044], [-87.64689711599998, 41.95290410400002], [-87.64573106199997, 41.95297316100004], [-87.64105232299991, 41.95339304800005], [-87.64060446499995, 41.953433240000024], [-87.64053244199992, 41.95343970300006], [-87.64051280999993, 41.953440593000025], [-87.64022002799996, 41.95269494600006], [-87.63991502799995, 41.95197894600004], [-87.63939102799996, 41.95075994600005], [-87.63894702799995, 41.94984694600003], [-87.63869102799997, 41.94932394600004], [-87.63845502799994, 41.94885494600003], [-87.63812702799997, 41.948276946000064], [-87.63801853399995, 41.94807970200002], [-87.63319762899992, 41.94277670700006], [-87.63315602799996, 41.94273094600003]], [[-87.63692694199992, 41.94099849900003], [-87.63657840499994, 41.94063411900003], [-87.63611149099994, 41.94064702100007], [-87.63596903299995, 41.94066198800003], [-87.63590656699994, 41.94069750000004], [-87.63563484299998, 41.94113905300002], [-87.6356308, 41.94125764000005], [-87.63562504099994, 41.941333105000055], [-87.63560104099996, 41.941535105000064], [-87.63558975099993, 41.94160590400002], [-87.63557285799997, 41.94167557800005], [-87.63555046799995, 41.94174368600004], [-87.63552272399991, 41.941809794000044], [-87.63547972399991, 41.94190179400004], [-87.63545103699991, 41.94195684300006], [-87.63550325999998, 41.941950386000016], [-87.63559325999992, 41.941943386000034], [-87.635663, 41.94194067800002], [-87.63581916599996, 41.941954341000034], [-87.63592984599994, 41.94198117900003], [-87.63620984599997, 41.94206817900004], [-87.63627624199992, 41.94209169700007], [-87.63634059399996, 41.942120340000024], [-87.63640250599995, 41.94215393100006], [-87.63646159999995, 41.94219226500006], [-87.63651751199993, 41.94223510600005], [-87.63656989999996, 41.94228219200004], [-87.63671389899996, 41.94242219200003], [-87.63677405799993, 41.94248671400003], [-87.63689998999995, 41.94263582100006], [-87.63699083599992, 41.94276533900005], [-87.63702560899995, 41.94283182900004], [-87.63778760999996, 41.94444982900006], [-87.63781685499997, 41.944519352000036], [-87.63784017299996, 41.94459108100004], [-87.63785739799991, 41.944664511000035], [-87.63786840899995, 41.944739126000044], [-87.63787312999993, 41.94481440200007], [-87.63787152599994, 41.94488980800002], [-87.63786795999992, 41.94494615100005], [-87.63789842799994, 41.94499219400005], [-87.63896495999994, 41.945282185000046], [-87.63903658599995, 41.94530491300002], [-87.63917271199993, 41.94536839000005], [-87.63922261099997, 41.94526156800002], [-87.63931005599991, 41.94511079700003], [-87.63936572899995, 41.94502617800003], [-87.63942220999994, 41.944950042000066], [-87.63924633999993, 41.94466403000006], [-87.63895509299992, 41.944291351000054], [-87.63887894499997, 41.94426307600001], [-87.63873624999994, 41.94419525600004], [-87.63859816199991, 41.944112677000064], [-87.63850887599992, 41.94405166300003], [-87.63836187499996, 41.943937663000035], [-87.63833492699993, 41.94391592100003], [-87.63822039099993, 41.94380062000003], [-87.63801970499996, 41.94355516800003], [-87.63795904799994, 41.943471595000055], [-87.63785704799993, 41.94331259600005], [-87.63779463999995, 41.94319771400005], [-87.63761863999994, 41.94280871400007], [-87.63759132599994, 41.942741189000046], [-87.63756956699994, 41.942671676000025], [-87.63755350499996, 41.942600630000065], [-87.63754324499993, 41.94252851700002], [-87.63745268399997, 41.94163332200002], [-87.63736364299993, 41.94152216300005], [-87.63692694199992, 41.94099849900003]]]}, "properties": {"city": "Chicago", "name": "Lake View East", "regionid": "403295", "geo_point_2d": [41.94183768196671, -87.64198485315828], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.77524993413351, 41.99659697344265], [-87.77512090930128, 41.99657593919951], [-87.7744979156567, 41.9967860217251], [-87.7740509256285, 41.996908075012755], [-87.77354691588567, 41.996935021398855], [-87.77266591574403, 41.996874021284754], [-87.77175191857779, 41.99667102181968], [-87.77143686628591, 41.996529088118976], [-87.77125391630594, 41.99627202231301], [-87.77110478982767, 41.996001067377534], [-87.77109454072087, 41.99592979465879], [-87.77102986281311, 41.99548002177709], [-87.77107779128116, 41.99503997631374], [-87.77120096922776, 41.994828594452215], [-87.77123981313798, 41.994761935619984], [-87.77142591614086, 41.99461302318027], [-87.7725849156073, 41.99393602255014], [-87.77320794972155, 41.993544062936216], [-87.77337003093356, 41.99336808809796], [-87.7734449975186, 41.993124027387644], [-87.7733299652378, 41.99265700406203], [-87.7731270188329, 41.992291936874715], [-87.77300091499858, 41.992184023627125], [-87.77226991547607, 41.99177102257564], [-87.77187691624574, 41.991622022901886], [-87.77163991341507, 41.99156102368732], [-87.77126091572939, 41.99155402246531], [-87.7708878706565, 41.99152014590385], [-87.77080676170202, 41.991466121830626], [-87.77077978204095, 41.99129002108776], [-87.77081991608512, 41.991114025565246], [-87.77091491596792, 41.99091102250271], [-87.77129291360733, 41.99028502332381], [-87.77137300183128, 41.99000452838027], [-87.7713470402029, 41.989810977192576], [-87.7712190172154, 41.989613934836626], [-87.77087291493113, 41.98936402313712], [-87.77062291329955, 41.989215024362736], [-87.77008691557114, 41.98910002355812], [-87.76926491545125, 41.98905902256761], [-87.76864791219069, 41.98900502367549], [-87.76809287131883, 41.989072894609606], [-87.76758491625016, 41.9893230230458], [-87.76695194282892, 41.98978406857803], [-87.76661291551106, 41.9899390226628], [-87.7661829154747, 41.99010902150625], [-87.76599891598606, 41.99015602277348], [-87.76581291640554, 41.990203155341554], [-87.76547491611323, 41.99014902155148], [-87.76502091407116, 41.99002701952486], [-87.76467591525488, 41.98983802160462], [-87.7643229151708, 41.989581022348254], [-87.7641879386253, 41.989492943648], [-87.7640049158004, 41.98948602426419], [-87.76390989793144, 41.98950597212622], [-87.76377791612845, 41.989574022924984], [-87.7635339156726, 41.989757022367435], [-87.76309391517513, 41.990190021642626], [-87.76279595269982, 41.99042009879479], [-87.76228090467845, 41.9905491982708], [-87.7621799180091, 41.99051502133574], [-87.76205088238513, 41.99042706314443], [-87.76196280008732, 41.99031908784373], [-87.76148878363368, 41.988863044109614], [-87.76149586399612, 41.988762013373474], [-87.7615409164322, 41.988629025228896], [-87.76165080998707, 41.988470881248844], [-87.76176991439776, 41.988423025625735], [-87.76206796140973, 41.98839614758817], [-87.76217608419913, 41.98833511689727], [-87.76217004705056, 41.9882139980051], [-87.7620679155442, 41.98799702217633], [-87.76183791483452, 41.98763102419082], [-87.76177298108591, 41.987576859823605], [-87.76149591345849, 41.98755702347753], [-87.76110991455118, 41.987611023268826], [-87.76069591615143, 41.98776002294258], [-87.7604818532269, 41.987861199368936], [-87.76036536354579, 41.987966191920115], [-87.76021790295673, 41.988054668273655], [-87.76007044236773, 41.988113652509284], [-87.75989348966094, 41.98817263674491], [-87.75977552118978, 41.98826111309809], [-87.75968704483631, 41.9883200973339], [-87.75953958424715, 41.9883200973339], [-87.75936263154037, 41.988231620980365], [-87.75915618671574, 41.988054668273655], [-87.75900872612668, 41.987907207684486], [-87.75889075765552, 41.98770076286006], [-87.75871380494863, 41.987523810152986], [-87.75853685224183, 41.987523810152986], [-87.75835989953504, 41.98746482591736], [-87.75824193106358, 41.987287873210825], [-87.75812396259242, 41.98705193626814], [-87.75812396259242, 41.9868159993258], [-87.75797650200336, 41.98649158602975], [-87.75788802565, 41.98622615696951], [-87.75777005717883, 41.98610818849825], [-87.75762259658974, 41.985960727909436], [-87.75750462811848, 41.985783775202556], [-87.75726869117607, 41.98566580673112], [-87.75703275423365, 41.985606822495846], [-87.75682630940894, 41.98554783826022], [-87.75667884881986, 41.98537088555315], [-87.75664935670223, 41.985134948610806], [-87.75676732517339, 41.985016980139726], [-87.75703275423365, 41.98495799590392], [-87.75726869117607, 41.98495799590392], [-87.75738665964732, 41.98484002743267], [-87.75738665964732, 41.98460409049033], [-87.7573276754117, 41.98451561413679], [-87.75720970694043, 41.984368153547635], [-87.75709173846936, 41.98425018507673], [-87.75694427788018, 41.98410272448757], [-87.75673783305568, 41.9839552638984], [-87.75656088034879, 41.9838078033096], [-87.75647240399533, 41.9837783111917], [-87.75629545128854, 41.98360135848481], [-87.75623646705274, 41.9834244057781], [-87.75623646705274, 41.983247453071215], [-87.75623646705274, 41.98309999248205], [-87.75626595917065, 41.98295253189324], [-87.75632494340627, 41.98274608706845], [-87.75632494340627, 41.98253964224384], [-87.75617748281728, 41.98245116589048], [-87.75600053011048, 41.98239218165485], [-87.7558235774035, 41.98239218165485], [-87.75567611681453, 41.98239218165485], [-87.75544017987208, 41.982362689536956], [-87.75526322716529, 41.982362689536956], [-87.75502729022277, 41.98245116589048], [-87.75485033751607, 41.98253964224384], [-87.75467338480908, 41.98262811859719], [-87.75449643210239, 41.98271659495055], [-87.75440795574893, 41.98292303977534], [-87.75431947939524, 41.98318846883558], [-87.75417201880643, 41.98339491366037], [-87.75405405033524, 41.98351288213127], [-87.75387709762838, 41.98354237424918], [-87.75375912915712, 41.98360135848481], [-87.75358217645031, 41.98360135848481], [-87.7534052237435, 41.98360135848481], [-87.75325776315445, 41.9835718663669], [-87.75316928680108, 41.98354237424918], [-87.75305131832965, 41.983483390013554], [-87.75290385774076, 41.9834244057781], [-87.75281538138731, 41.98336542154247], [-87.75260893656251, 41.98327694518912], [-87.75232175605781, 41.98313389273106], [-87.75270593853502, 41.982641686051025], [-87.75308801757356, 41.9821842721091], [-87.75365848187852, 41.981524439571515], [-87.75405246691946, 41.98109055031312], [-87.75477826017944, 41.980280513192895], [-87.75517999999988, 41.97985000000033], [-87.75557137089805, 41.97992696068154], [-87.75592899669964, 41.97986679612998], [-87.75611387400947, 41.979852963326984], [-87.75617791563063, 41.97992002217329], [-87.75624891620545, 41.97999602207939], [-87.75627596871158, 41.980025161012435], [-87.75628167120712, 41.97981689613811], [-87.75628199686705, 41.97980500248298], [-87.75523091604387, 41.97776402261378], [-87.75494445083275, 41.9772257231684], [-87.75494106611156, 41.9772213083106], [-87.75413492903027, 41.976725626204384], [-87.75259596137076, 41.97578594740011], [-87.75179999999982, 41.975300000000104], [-87.75098423174028, 41.97523915484559], [-87.7499535151316, 41.975240566888864], [-87.74856177794423, 41.975259572393234], [-87.74793101277801, 41.97524838492255], [-87.74784991574892, 41.9730480219589], [-87.74783891524174, 41.97278102224163], [-87.7478329158644, 41.97263202256805], [-87.7478279156341, 41.972527022224526], [-87.74780291538097, 41.97162702210636], [-87.74778091526608, 41.97070902205776], [-87.74776491542791, 41.97018302299194], [-87.74773091565922, 41.96974902277112], [-87.7477319157054, 41.969243021929174], [-87.7477059136076, 41.96897602041353], [-87.74766398541584, 41.96885597621103], [-87.74758284588447, 41.96878007163313], [-87.74752091677081, 41.96864001931423], [-87.74750286288113, 41.968541021944986], [-87.74752991538706, 41.96827902245835], [-87.74755091545586, 41.96795002168151], [-87.74756191506349, 41.967774021662684], [-87.7474709153654, 41.96550902247448], [-87.74746591513495, 41.96460302207977], [-87.74744051096675, 41.96428421922507], [-87.74747652446852, 41.961702352697564], [-87.74746787422197, 41.96113576155308], [-87.74745900660386, 41.960617642971975], [-87.74837596710233, 41.96060486004827], [-87.74928756027066, 41.9605895328771], [-87.75056262125901, 41.96056380406264], [-87.75183694521985, 41.96053888424638], [-87.75329370735342, 41.960525494110804], [-87.75645093023488, 41.96048795435], [-87.75857582224745, 41.96046456314369], [-87.76106572755464, 41.960434262418744], [-87.76349485610254, 41.96040975627709], [-87.76604491722684, 41.96038354351589], [-87.76792685487389, 41.96035773032235], [-87.76902153261629, 41.96033980779009], [-87.77005297518853, 41.960326308540374], [-87.7713891113268, 41.96031065949511], [-87.77284583650213, 41.96029336802943], [-87.77393885476185, 41.960278799109844], [-87.77515296783382, 41.960264483870546], [-87.77630648214686, 41.96025013850192], [-87.77770466559099, 41.960227300564156], [-87.77873664293332, 41.960210303422684], [-87.78001101487153, 41.9601947071965], [-87.78116496401115, 41.96018049646294], [-87.78231815935436, 41.960158242674076], [-87.78371455012208, 41.960144499579066], [-87.78498958890597, 41.96013393779615], [-87.78608208375861, 41.960115309989824], [-87.786725, 41.960132], [-87.78675999999989, 41.96183100000016], [-87.78677264379992, 41.96190686280047], [-87.78696, 41.96303100000015], [-87.78665999999986, 41.96443100000017], [-87.78665999999986, 41.96454401756152], [-87.78665999999986, 41.96633100000017], [-87.78675999999989, 41.967231000000204], [-87.78675999999989, 41.968331], [-87.78693213272057, 41.9683453443935], [-87.78792245440452, 41.968427871200575], [-87.78791691549316, 41.968659021988685], [-87.78789286492415, 41.96961092467904], [-87.78788491581673, 41.970145022139704], [-87.78784891595586, 41.97193202177668], [-87.78781291609499, 41.973772022957256], [-87.78777094421905, 41.97558080392705], [-87.78775691531222, 41.976208021933836], [-87.78773791533561, 41.97737702276099], [-87.78769391600548, 41.97918802260477], [-87.78765991533758, 41.98099902244872], [-87.78763991531488, 41.98189702247471], [-87.78763791612207, 41.981974022427075], [-87.78763591602981, 41.98204002277129], [-87.78762891570709, 41.98229302274267], [-87.78762591556873, 41.98242502253208], [-87.78762191538446, 41.98253902239105], [-87.78761791609936, 41.982687022918086], [-87.787614044518, 41.98282915536896], [-87.78638991525081, 41.98281502252331], [-87.7859099156074, 41.98281002229285], [-87.78395491542078, 41.98283802178501], [-87.78351991605324, 41.9828410219231], [-87.78215383458617, 41.98284997045906], [-87.78129782328931, 41.98237035955641], [-87.77971605180474, 41.98148158075315], [-87.77854391319822, 41.98083115225916], [-87.77796, 41.98051], [-87.77612771346877, 41.97947395595252], [-87.77501514252923, 41.978853315979336], [-87.77380926758825, 41.97818991126915], [-87.77169595443283, 41.97700895254651], [-87.7705162767874, 41.97636081265524], [-87.76930449190723, 41.97561882650148], [-87.76836999999983, 41.97515000000012], [-87.76835, 41.97640000000017], [-87.76894504933179, 41.977043735186214], [-87.77007072400917, 41.97827336588179], [-87.77148790780163, 41.97983356778279], [-87.77365129783658, 41.98220920030016], [-87.77476889008669, 41.98343053855719], [-87.77573857238541, 41.984492450018706], [-87.77728430489674, 41.98618537820212], [-87.77785291562724, 41.986795022525946], [-87.777825915282, 41.98775802285022], [-87.77782091595083, 41.98832202276829], [-87.77780691620484, 41.98948002218873], [-87.77779791578993, 41.98990002266383], [-87.77779291555929, 41.99015302263486], [-87.77773991581427, 41.99166102200147], [-87.77773691567607, 41.99283302206736], [-87.77772575598904, 41.99381612493688], [-87.77773084866003, 41.993819061894925], [-87.77773189182612, 41.99601157472915], [-87.77773189182612, 41.99707401298102], [-87.77750191563524, 41.99707702254836], [-87.77720691552643, 41.99712402201704], [-87.7766989154916, 41.99714502208592], [-87.77635391577631, 41.99712402201704], [-87.77621889786205, 41.99709707113462], [-87.77573091493723, 41.996833022092964], [-87.77524993413351, 41.99659697344265]]]}, "properties": {"city": "Chicago", "name": "Jefferson Park", "regionid": "269588", "geo_point_2d": [41.973773677496744, -87.76850079826066], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.68355, 41.779160000000125], [-87.6835385939874, 41.77841464927419], [-87.68350052777986, 41.7766251056323], [-87.68343702784264, 41.77450885384328], [-87.68339408801852, 41.77296396083643], [-87.683350634679, 41.77117510231725], [-87.68330007830444, 41.76946566941588], [-87.68326434019845, 41.76826033982011], [-87.68405069918111, 41.76826], [-87.68513550304883, 41.76824919195135], [-87.68622060411062, 41.76824], [-87.68773580281197, 41.76819592222032], [-87.68893310689788, 41.768157281551694], [-87.69012597248954, 41.76813133401255], [-87.6912111743564, 41.76812486408103], [-87.69251332840605, 41.76812], [-87.69307999999985, 41.76811], [-87.6930466592584, 41.767065323432305], [-87.69302, 41.76623], [-87.69251495744828, 41.766244311812024], [-87.69121085064245, 41.76628363778925], [-87.69012654916575, 41.76631025084723], [-87.68882502631163, 41.76633827825722], [-87.68752087957766, 41.76638863084732], [-87.68632909935708, 41.76641], [-87.68502714181773, 41.76641279147334], [-87.68394242428157, 41.76642750124133], [-87.68321458356792, 41.76642869841995], [-87.68319624248585, 41.76572331499309], [-87.6831366759162, 41.763771673722005], [-87.68309677246758, 41.7623058723205], [-87.68303204176145, 41.75994791177102], [-87.68299330788908, 41.7583189313458], [-87.68296883886875, 41.75750101910946], [-87.68297091529453, 41.757501018990716], [-87.6830429141169, 41.75747702328041], [-87.68321491575027, 41.757455022266335], [-87.68406298811497, 41.757433009561076], [-87.68390584777823, 41.757484033495665], [-87.68427491514983, 41.7574710221045], [-87.68490691550463, 41.7574710221045], [-87.68549491562975, 41.757457022358516], [-87.68649493924326, 41.75743715363691], [-87.686571915813, 41.75740202252112], [-87.6866528925672, 41.75736589135932], [-87.68761291433684, 41.757363023421256], [-87.6878639151153, 41.75741502042254], [-87.6880039161728, 41.75740702275166], [-87.68881991601626, 41.7574480219437], [-87.68992191555321, 41.75736102242988], [-87.69265091544577, 41.75733302203846], [-87.69276591535082, 41.75733202199237], [-87.69316191561867, 41.75732702176183], [-87.69578391507552, 41.757293021993135], [-87.6972419157344, 41.75725902222453], [-87.70013991532333, 41.757229022640296], [-87.70131491552758, 41.75717702204198], [-87.70157491582141, 41.75716502238808], [-87.70247291584738, 41.75713502190457], [-87.70252391550041, 41.757134022757754], [-87.70257991538372, 41.757132022665495], [-87.70302591615771, 41.757117021974125], [-87.70449391547893, 41.75710702241242], [-87.7079109156097, 41.757050022482844], [-87.70963091575547, 41.75707302264382], [-87.71097491565608, 41.75705702190638], [-87.71151691545943, 41.75707102255155], [-87.71182191602904, 41.75711102259678], [-87.71225594952477, 41.75721597077982], [-87.71230591585672, 41.75877702205198], [-87.71245391458511, 41.75990802112735], [-87.71242691513906, 41.760502022428284], [-87.71240991525478, 41.76087402248909], [-87.7125349138223, 41.763720023278346], [-87.71245573841018, 41.7641560658595], [-87.712570080911, 41.764302964717025], [-87.71257891495125, 41.765175022899676], [-87.71265691584874, 41.76607702221073], [-87.71275191483207, 41.76691502216878], [-87.71270491536359, 41.767822022609586], [-87.71265991598709, 41.768686021967504], [-87.71262391612639, 41.769370022021256], [-87.71264091601056, 41.76990602244744], [-87.71266691541044, 41.77073702208285], [-87.71268695410389, 41.77139100816307], [-87.71268791547907, 41.7714310225975], [-87.71268891552515, 41.77147002259673], [-87.71275591501636, 41.772349022646104], [-87.71274391536264, 41.773233022926014], [-87.71275991610017, 41.77413502223681], [-87.71277691508514, 41.77504402187069], [-87.7128439154757, 41.776053022516656], [-87.71282891568364, 41.77686302208328], [-87.71284791565999, 41.77787502196823], [-87.71287004797516, 41.778703150068644], [-87.71272478408501, 41.77870589210138], [-87.71272476996289, 41.77871051157675], [-87.71140842977483, 41.778731859504], [-87.70856152622655, 41.778786482858465], [-87.70595571512897, 41.778824028092764], [-87.7036771592307, 41.77887681528703], [-87.70253917231062, 41.778908219276744], [-87.7014814251485, 41.77892304607333], [-87.7000168344301, 41.778935066386886], [-87.69903815036753, 41.77894609589574], [-87.69741187954683, 41.778966326182704], [-87.6959466915061, 41.77898674890079], [-87.69521208482801, 41.77900333673998], [-87.69407337827764, 41.77903249879621], [-87.69277452066595, 41.77906], [-87.69138760566186, 41.779067271388406], [-87.69000716086, 41.779104920327214], [-87.68895042315476, 41.77910377396031], [-87.68772916340411, 41.77910599083943], [-87.68659002687473, 41.7791151217328], [-87.68431189144101, 41.77913000000013], [-87.68355, 41.779160000000125]]]}, "properties": {"city": "Chicago", "name": "Marquette Park", "regionid": "403148", "geo_point_2d": [41.76812205831823, -87.69822843623105], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.58581439883125, 41.75148253732185], [-87.58720150616423, 41.75147], [-87.58868760717554, 41.75146320654718], [-87.59119589239536, 41.751425072860926], [-87.59249641549728, 41.75141425503891], [-87.59370487234892, 41.75140428102138], [-87.59444805887476, 41.751398174922286], [-87.59538168438127, 41.75139053864597], [-87.59644391608803, 41.75137702178048], [-87.59657491583145, 41.751367022218766], [-87.59683891541043, 41.751323021989165], [-87.59694691589225, 41.75129802263565], [-87.59715497404366, 41.75127312130772], [-87.59733591763595, 41.75122902395137], [-87.5975749151634, 41.75122902215281], [-87.5979039159399, 41.75122802210672], [-87.59909891526746, 41.751218022545004], [-87.59950691608825, 41.75120802208393], [-87.60029891572461, 41.75120802208393], [-87.60147291588277, 41.75118602196931], [-87.60273391555448, 41.75114702197016], [-87.60392691568914, 41.75113802245455], [-87.60511291550102, 41.75111502229366], [-87.60514294644142, 41.75111488040876], [-87.60516872548529, 41.75232366003466], [-87.60524848214702, 41.75408508938655], [-87.60528733657745, 41.75622249637555], [-87.60530585550819, 41.75770817457113], [-87.60533525561827, 41.758449993649464], [-87.60538343411893, 41.75966562620924], [-87.60544179163277, 41.76142721543733], [-87.60546797789054, 41.76319094890611], [-87.60552722369145, 41.764863156997045], [-87.60552384271584, 41.765732879549155], [-87.60555691528347, 41.76835602204419], [-87.60558291558262, 41.76937002202133], [-87.60559191599754, 41.77118002271857], [-87.60558791581326, 41.77149102266542], [-87.60566204584497, 41.77299007645905], [-87.60565058993265, 41.772990146283206], [-87.60543191590826, 41.772990022758364], [-87.60446791522313, 41.77301602195814], [-87.60325291604808, 41.77301602195814], [-87.60280891520719, 41.77301602195814], [-87.60205791593462, 41.77303202282704], [-87.60084791592529, 41.77305502195781], [-87.59958491509346, 41.77306102209598], [-87.59833291501536, 41.77307502196906], [-87.59722391508552, 41.77306902278053], [-87.59598691585305, 41.77307202189989], [-87.59442091523232, 41.77310202119218], [-87.59357891516113, 41.77310002209565], [-87.59285791613043, 41.77309902207266], [-87.59205885507399, 41.77313885780719], [-87.5920369492309, 41.773156097808794], [-87.59187291591081, 41.773178023044466], [-87.59173098915788, 41.773223089957156], [-87.59152291593783, 41.77326702094808], [-87.59131391606708, 41.77327802120159], [-87.59120291575698, 41.77326602282437], [-87.58877691604945, 41.77328102222041], [-87.58641691511485, 41.77329702213949], [-87.58625208225426, 41.7733060139109], [-87.58623561320015, 41.772669796163974], [-87.58622999999974, 41.77201655384489], [-87.58619137813075, 41.770810995841956], [-87.58623361416623, 41.76902924115096], [-87.58632467822457, 41.76830104579237], [-87.58632999999985, 41.76699974778702], [-87.5863146118959, 41.76551277887316], [-87.58629783811433, 41.7647692008953], [-87.5862534579603, 41.763656074128896], [-87.58622102822414, 41.76291416877236], [-87.5861918928566, 41.76217137677344], [-87.5861496901457, 41.76096724220289], [-87.5861181590199, 41.76003531592221], [-87.58610285519954, 41.75900984117408], [-87.58609345218038, 41.75817380789041], [-87.58607610111055, 41.757617042116834], [-87.58601058089606, 41.75641280287002], [-87.58598340022252, 41.75557561507218], [-87.58595938522187, 41.75473999895028], [-87.58591999999985, 41.75278546378954], [-87.58583111093344, 41.751722459895205], [-87.58581674478968, 41.751712105785636], [-87.58581591520228, 41.751613022883404], [-87.58581591180152, 41.75154088604859], [-87.58581439883125, 41.75148253732185]]]}, "properties": {"city": "Chicago", "name": "Grand Crossing", "regionid": "276349", "geo_point_2d": [41.762230073051356, -87.59579161120412], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.67205191558631, 41.97608602260525], [-87.6743889693547, 41.97605002274438], [-87.6743939156258, 41.97695402214763], [-87.67440796753264, 41.97771603209211], [-87.67436691617993, 41.977857022403974], [-87.67435091724096, 41.977936022448354], [-87.67437691574156, 41.97875702252214], [-87.67446091511701, 41.97966402296303], [-87.67448891550845, 41.98059202257307], [-87.67452591541539, 41.981491022645145], [-87.67452191613033, 41.98238702257877], [-87.67456595323202, 41.98326000796258], [-87.6745660489883, 41.983263424687344], [-87.67390565773391, 41.98327703441705], [-87.67253071502827, 41.983305446420204], [-87.67163846693491, 41.983320874625306], [-87.66909350343585, 41.983366082490846], [-87.66723978154901, 41.98341618976198], [-87.66442088576308, 41.98346354211069], [-87.66378994345294, 41.98347717265245], [-87.66377118209353, 41.982539104678395], [-87.66372201844403, 41.98123316729848], [-87.66368260607555, 41.98034146246024], [-87.66364048192507, 41.97938108427748], [-87.66361363961857, 41.97876371122729], [-87.6635614181649, 41.97752279957448], [-87.66353412686175, 41.97683661823837], [-87.66351391591672, 41.976227021910375], [-87.6647439159583, 41.97620002246449], [-87.66839391513825, 41.97615802232682], [-87.669536915666, 41.97612002237403], [-87.66961491566427, 41.97611502214349], [-87.6696679154094, 41.976111022858404], [-87.67205191558631, 41.97608602260525]]]}, "properties": {"city": "Chicago", "name": "Andersonville", "regionid": "269567", "geo_point_2d": [41.97974888786532, -87.66905093797907], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.66628880977532, 41.89608181013116], [-87.66736999999989, 41.89606], [-87.66738347347751, 41.89663935953593], [-87.66740452458174, 41.89767645868977], [-87.66741591212396, 41.89828611231301], [-87.66743959037372, 41.8990773015872], [-87.6674714890962, 41.900054672889404], [-87.6675009933979, 41.90084635805616], [-87.66750954570449, 41.901761824516385], [-87.66738543400258, 41.90292449423574], [-87.66743176018609, 41.90457224446711], [-87.66763175095457, 41.90578939647873], [-87.66750358550327, 41.90670674524456], [-87.66755021090182, 41.90811200951227], [-87.66757390657328, 41.909638449789114], [-87.66761999999989, 41.91063], [-87.66677580186501, 41.91064493941534], [-87.66628714949073, 41.91065247500598], [-87.66519, 41.91066], [-87.66507792408522, 41.91061533243723], [-87.66459517433793, 41.909770384862476], [-87.66401379119435, 41.909363791194586], [-87.66347258783232, 41.908854909454725], [-87.66295431525475, 41.90839534554683], [-87.66181282559283, 41.907039238389316], [-87.66118832711652, 41.906258316459265], [-87.66111524142526, 41.90519317016374], [-87.66115177983791, 41.90446655233607], [-87.66114034011832, 41.90344000000012], [-87.66104795581289, 41.902199668597376], [-87.6608017841833, 41.901150464061864], [-87.66039822464376, 41.89973736313705], [-87.65962222864215, 41.898465014444874], [-87.65911896434339, 41.89741980111122], [-87.65854381869718, 41.89629381869715], [-87.65842021980995, 41.89620605365613], [-87.65927075102218, 41.89619394268711], [-87.66030835607853, 41.89617465183002], [-87.6611006853658, 41.89616000000014], [-87.66183547578389, 41.896151178697345], [-87.66299433287907, 41.896129752881464], [-87.6638469713946, 41.89612], [-87.6645200876154, 41.896108664719755], [-87.66549561470349, 41.89609208628789], [-87.66628880977532, 41.89608181013116]]]}, "properties": {"city": "Chicago", "name": "Noble Square", "regionid": "403310", "geo_point_2d": [41.90243127813083, -87.66417170368335], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.7135779160356, 41.80058102272588], [-87.71357591504413, 41.80016102225104], [-87.7135449154138, 41.79978702209806], [-87.71350491536867, 41.799305022362546], [-87.71349991513813, 41.79924902247925], [-87.71348491534587, 41.79888302269498], [-87.71348291525379, 41.79882302262714], [-87.71347691587651, 41.79866502253839], [-87.71345991599209, 41.79800302259943], [-87.71343691583138, 41.79706202238996], [-87.71342791541637, 41.79672102195948], [-87.71338991636252, 41.795265022291986], [-87.71333491562604, 41.79334602196622], [-87.71333191548786, 41.79324602275252], [-87.71330691523451, 41.792443022609255], [-87.71329991581115, 41.79222502245256], [-87.71328191588098, 41.7916070227433], [-87.71323791565139, 41.790234022405244], [-87.71321291539807, 41.789590022396645], [-87.7131869150989, 41.78926802284216], [-87.71316791602176, 41.78870302287774], [-87.71313591544605, 41.787798022529124], [-87.71310491671481, 41.78688902199623], [-87.71307391978252, 41.78598196759602], [-87.71320491592871, 41.78598102240871], [-87.7155509152533, 41.7859510219251], [-87.71797291538326, 41.78590902178778], [-87.71938191558216, 41.78589102185734], [-87.72039991574394, 41.78586502155819], [-87.7228129154589, 41.785821022227786], [-87.72288091589549, 41.7858200221817], [-87.72294091506386, 41.785819022135605], [-87.72407791621451, 41.78580502238953], [-87.72530091593336, 41.78580502238953], [-87.7265309159748, 41.785796021974726], [-87.72776191516301, 41.78578002213638], [-87.72895791543596, 41.78576002211385], [-87.7301979159385, 41.785750022552314], [-87.730939915968, 41.7857430222295], [-87.73142491584177, 41.78573802199897], [-87.73174291611109, 41.78573502186077], [-87.73264891560645, 41.78572702239134], [-87.73388091574026, 41.78571702193034], [-87.73511891525133, 41.78570702236882], [-87.73631891570847, 41.785702022138274], [-87.73741191572998, 41.78567102250795], [-87.73755391508121, 41.78566402308459], [-87.73755491602638, 41.78572002206871], [-87.73756591653374, 41.786293022401665], [-87.73763591616311, 41.78671502206932], [-87.73778191659805, 41.78744002221448], [-87.7378499152358, 41.78762302255629], [-87.73792691428872, 41.788033022570154], [-87.73793491555698, 41.78822202228914], [-87.73791991666411, 41.78862202274127], [-87.73794891530297, 41.788967022456816], [-87.73797091541769, 41.78952102191398], [-87.73804191509332, 41.79088402179112], [-87.7380559157387, 41.79131502277295], [-87.73807391566913, 41.791945022136204], [-87.73807291562304, 41.79221202275285], [-87.73807291562304, 41.79259202228301], [-87.7380559157387, 41.79275102241784], [-87.73802091502453, 41.7929200221142], [-87.73799591567074, 41.79300502243504], [-87.73794791525671, 41.7931600223856], [-87.73785983385852, 41.79333500167426], [-87.73785252058286, 41.793877689750246], [-87.73720332409643, 41.7945268862365], [-87.73585319762712, 41.79527695649734], [-87.73579391578939, 41.79528802245296], [-87.73563434808312, 41.79533879841622], [-87.7355238700684, 41.795373953572245], [-87.73550501376512, 41.795394227409794], [-87.73468080961602, 41.7957474577593], [-87.73346023809314, 41.79623568636848], [-87.73255741423134, 41.796660544656575], [-87.73269792850374, 41.796657807521704], [-87.73044949500355, 41.7975376293261], [-87.72814999999981, 41.798410000000196], [-87.72806117287388, 41.79836314301941], [-87.72678491599218, 41.79885702239583], [-87.72661091516592, 41.798906021956505], [-87.72603591563997, 41.79909802271313], [-87.72581691543745, 41.79918402218086], [-87.72339091512305, 41.80013302275906], [-87.72332875938037, 41.8001579177915], [-87.72333091595448, 41.80022302241107], [-87.72333191600073, 41.800283022479], [-87.72333704933095, 41.800467154167926], [-87.72330191551714, 41.800467021967535], [-87.72326591565626, 41.800468022913], [-87.72259791903798, 41.80058702390179], [-87.72117691558786, 41.80122902201948], [-87.7201229155654, 41.80170502237781], [-87.71957991571607, 41.801912022926516], [-87.71885091538671, 41.80220802218211], [-87.71789391533905, 41.80258502247336], [-87.71737491569661, 41.80279002203059], [-87.71604391549577, 41.8033040223419], [-87.71532188940887, 41.80356997614753], [-87.71461120191753, 41.80378752919902], [-87.71383402168259, 41.80392339986947], [-87.7136759700159, 41.80397003415629], [-87.71366091536515, 41.80339202190171], [-87.71363191582691, 41.802291022410664], [-87.71358591550496, 41.801348022109174], [-87.7135779160356, 41.80058102272588]]]}, "properties": {"city": "Chicago", "name": "West Elsdon", "regionid": "269611", "geo_point_2d": [41.79285386973492, -87.72418842228458], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.59349891523203, 41.73677802240869], [-87.59524492736823, 41.736747024576836], [-87.59743491590444, 41.73675102206327], [-87.5978879152024, 41.73674302259392], [-87.5985999156477, 41.73672402261739], [-87.60013991558993, 41.736696022225956], [-87.60037403429514, 41.736692170429656], [-87.59955771381993, 41.73968602538756], [-87.59881448328441, 41.74312346661504], [-87.59870303064689, 41.74394078595693], [-87.59781452349841, 41.743955889298796], [-87.59781391538858, 41.743972022177566], [-87.59781399483794, 41.743973939159524], [-87.5962141821009, 41.743988162602214], [-87.5946334891171, 41.74403783540408], [-87.59351923335034, 41.74406356693512], [-87.59249778318134, 41.74408743949602], [-87.59082527168387, 41.74411657038963], [-87.58989571685618, 41.74412], [-87.588780690676, 41.7441239437758], [-87.58757312143211, 41.74413], [-87.58673677555636, 41.74413645412083], [-87.58573999999977, 41.74416000000021], [-87.5857526928374, 41.74358882231638], [-87.58576642626788, 41.74293868660455], [-87.58575217323863, 41.742007795741664], [-87.58573363426427, 41.74126478410722], [-87.58570125515413, 41.74015085435852], [-87.58568080091902, 41.739222242736524], [-87.58566062997944, 41.73829366406967], [-87.5856398522216, 41.7375503944107], [-87.58562972091364, 41.73690895564486], [-87.58680791526736, 41.73688102265977], [-87.58780691545233, 41.73686102173806], [-87.58813591532963, 41.73685302226853], [-87.5890269150329, 41.73683002210755], [-87.58932091599478, 41.73682702196936], [-87.59026791558135, 41.736817022407664], [-87.59056491578222, 41.736813022223295], [-87.59140291574046, 41.736804021808396], [-87.59176891552453, 41.73680002252323], [-87.59249691580806, 41.73679302220059], [-87.59299491538174, 41.73678902201632], [-87.59349891523203, 41.73677802240869]]]}, "properties": {"city": "Chicago", "name": "Marynook", "regionid": "403358", "geo_point_2d": [41.74034880470148, -87.5925835751646], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.6464974549999, 41.85300464400006], [-87.64647388299993, 41.85262985700006], [-87.64580144499989, 41.85264156700003], [-87.64572551099988, 41.85215163700007], [-87.6455909869999, 41.851540764000035], [-87.64538163699987, 41.85095950200007], [-87.64520675499993, 41.85057831400007], [-87.64494337199993, 41.85009447600003], [-87.64470924699991, 41.849720993000055], [-87.644721, 41.84972], [-87.6442400699999, 41.849457056000055], [-87.64428375799989, 41.84945063200006], [-87.6451204169999, 41.84943355700004], [-87.64553020899987, 41.84939940800006], [-87.64624734499992, 41.849296960000046], [-87.64732304999988, 41.84900669000005], [-87.64807433499992, 41.84873349600007], [-87.6486548739999, 41.84859689800003], [-87.64949153299993, 41.848426151000055], [-87.65065260999994, 41.84834077800007], [-87.65142097099991, 41.84823833000006], [-87.65195028499993, 41.848152957000075], [-87.65247959999991, 41.84803343400005], [-87.65299183999988, 41.84796513500003], [-87.6535382299999, 41.84796513500003], [-87.65410169399989, 41.84803343400005], [-87.65469930699992, 41.848118807000034], [-87.65529692099992, 41.84823833000006], [-87.6559457579999, 41.84830662900003], [-87.65642384899989, 41.84830662900003], [-87.65693608899993, 41.848272479000045], [-87.65738003099993, 41.848118807000034], [-87.65780689699993, 41.84789683700005], [-87.65845573499989, 41.847538269000026], [-87.65927531899987, 41.84707725200007], [-87.65965096199992, 41.84709432700003], [-87.6600266049999, 41.84692358000006], [-87.66021442599991, 41.846633311000055], [-87.66026564999991, 41.84620644400006], [-87.66053884499988, 41.84595032400005], [-87.66086326299987, 41.84572835300008], [-87.66147795199991, 41.84538686000007], [-87.66265610399994, 41.84480632100007], [-87.6637830319999, 41.84427700600008], [-87.66493572899991, 41.84459248900003], [-87.66583446699988, 41.84479220800005], [-87.66584711199991, 41.84479220800005], [-87.66585825299987, 41.84504614200005], [-87.66588921999993, 41.84628686200005], [-87.6659647959999, 41.84909702700008], [-87.66601849099992, 41.851414869000045], [-87.66601047599993, 41.852259824000036], [-87.66600999999989, 41.85226], [-87.666, 41.852697292000066], [-87.6660159549999, 41.85379613600008], [-87.6660342409999, 41.85446754500003], [-87.66606841799988, 41.85544304100005], [-87.66609459699991, 41.856298442000075], [-87.66612615799991, 41.85715324500005], [-87.6661552059999, 41.85825125000007], [-87.6661813339999, 41.85922702100003], [-87.66618775399988, 41.859626022000036], [-87.66618756299991, 41.859626022000036], [-87.66612591599991, 41.859626022000036], [-87.6637959149999, 41.859645022000045], [-87.66135891599991, 41.85968402200006], [-87.65957191599993, 41.859716022000036], [-87.65890391499993, 41.859727022000065], [-87.65768991599991, 41.859743022000075], [-87.6564769149999, 41.85975902200004], [-87.6545229149999, 41.859791023000064], [-87.65342891599991, 41.85980702300008], [-87.65284191599993, 41.85981802200006], [-87.65223391599993, 41.85982602200004], [-87.65092691599992, 41.85984002200006], [-87.65047491599992, 41.859852022000034], [-87.64984691499988, 41.859858022000076], [-87.64877291499992, 41.859882022000065], [-87.64766291499987, 41.859900022000026], [-87.64685791599992, 41.85990202200008], [-87.64674291499992, 41.85990402200007], [-87.64670291499992, 41.85990502300006], [-87.6466997139999, 41.85990510300006], [-87.6466807019999, 41.85941111600004], [-87.64666305099989, 41.85886205000007], [-87.6466302509999, 41.85782164100003], [-87.6466146839999, 41.85708976600005], [-87.64658501799993, 41.856175674000035], [-87.64657290599989, 41.85574880800003], [-87.6465481209999, 41.85507866800002], [-87.64651891499993, 41.85440841700005], [-87.6465144999999, 41.85367497700007], [-87.6464974549999, 41.85300464400006]]]}, "properties": {"city": "Chicago", "name": "Pilsen", "regionid": "33420", "geo_point_2d": [41.85335508616178, -87.65670971378704], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.8119099339999, 41.98236670500005], [-87.81086091199987, 41.98236512700002], [-87.80960278199989, 41.98237], [-87.80803030499992, 41.982376913000046], [-87.80684692499993, 41.98249928400008], [-87.80685, 41.98239], [-87.80688993099992, 41.980659075000055], [-87.8068837729999, 41.97889755200003], [-87.80689479499993, 41.978229933000065], [-87.8069051809999, 41.97744071500004], [-87.80692178199989, 41.976653519000074], [-87.8069358899999, 41.975985579000046], [-87.80695, 41.9754], [-87.80695958099993, 41.97494971600007], [-87.80714575599988, 41.97445378200006], [-87.807161, 41.97453], [-87.80826099999987, 41.97443], [-87.809461, 41.97443], [-87.81009779299987, 41.97437693400006], [-87.8106609999999, 41.97433], [-87.81159739599988, 41.97433], [-87.81190791599992, 41.97433], [-87.81196631599991, 41.97433], [-87.81206099999989, 41.97433], [-87.813161, 41.97433], [-87.814461, 41.97433], [-87.81537862599991, 41.97425353100003], [-87.815661, 41.97423], [-87.81685536199987, 41.97413047000003], [-87.81686099999988, 41.97413], [-87.81686099999988, 41.973537495000066], [-87.81686099999988, 41.97353], [-87.81686099999988, 41.97333], [-87.816961, 41.97243], [-87.81686099999988, 41.97153], [-87.816961, 41.97093], [-87.816961, 41.97056809900004], [-87.816961, 41.97053], [-87.81778832699987, 41.97041181100007], [-87.818361, 41.97033], [-87.81848962899991, 41.97033], [-87.81855199999987, 41.97033], [-87.81851840699993, 41.97068645900003], [-87.818516, 41.970712], [-87.81870518899987, 41.97070118900006], [-87.818761, 41.970698], [-87.818761, 41.97283], [-87.818759, 41.973223], [-87.8190339159999, 41.973223], [-87.8191738549999, 41.973223], [-87.81936099999987, 41.973223], [-87.81936099999987, 41.97343], [-87.81936099999987, 41.97413], [-87.81981804299993, 41.97413], [-87.82186099999987, 41.97413], [-87.823761, 41.97413], [-87.82462471599987, 41.97404371500004], [-87.82456570499991, 41.97453189600008], [-87.8244647779999, 41.97531655300008], [-87.82434669999991, 41.976227470000076], [-87.8242032339999, 41.977204782000044], [-87.82406171599987, 41.978234081000025], [-87.82392821599987, 41.979203998000045], [-87.82381967499991, 41.980055385000064], [-87.82371929999991, 41.980778887000035], [-87.8235815179999, 41.98180962700007], [-87.8234637819999, 41.98272007700005], [-87.82336029799987, 41.98339063600007], [-87.82330665599991, 41.98374673300008], [-87.82336199999989, 41.98453], [-87.8223003019999, 41.984316260000064], [-87.8212349059999, 41.98418398400007], [-87.81988841199988, 41.98384789400006], [-87.81863127899992, 41.983615107000034], [-87.8175647449999, 41.983323034000065], [-87.81713056499987, 41.98321071000002], [-87.81648537899989, 41.983029004000045], [-87.81580298599992, 41.98286640100007], [-87.81512129899993, 41.982709505000045], [-87.8145464329999, 41.982589951000065], [-87.81369151799991, 41.982459569000056], [-87.81290440799994, 41.98237925800003], [-87.8119099339999, 41.98236670500005]]]}, "properties": {"city": "Chicago", "name": "Oriole Park", "regionid": "403168", "geo_point_2d": [41.97834213442986, -87.81598758657618], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.6848991599999, 41.68445], [-87.68387767999988, 41.68444489900007], [-87.6824828389999, 41.68445386400003], [-87.68136977399988, 41.68447151900004], [-87.68081, 41.68448], [-87.68078018499993, 41.683847922000034], [-87.6807577569999, 41.68337907100004], [-87.68070257399992, 41.68226828600005], [-87.68068724399988, 41.68189730400006], [-87.68065918299992, 41.681155502000074], [-87.68064999999989, 41.680506295000036], [-87.68063657899994, 41.679484186000074], [-87.68060864999988, 41.67855542100005], [-87.68060367299992, 41.67733247400007], [-87.6807500129999, 41.67733247400007], [-87.68195002599991, 41.67733243500003], [-87.68324997599987, 41.67723255500004], [-87.6843500499999, 41.677332392000025], [-87.68564999899989, 41.67723251200004], [-87.68626249499991, 41.677232474000036], [-87.6864246249999, 41.677232474000036], [-87.68645008299987, 41.677232671000056], [-87.68642532499989, 41.67728977800003], [-87.68605065899992, 41.67813268400004], [-87.68557150299992, 41.67909099600006], [-87.68555637099989, 41.67912125900004], [-87.68555102799988, 41.67913194600003], [-87.68554732299991, 41.67913375500006], [-87.68555303399994, 41.67917944100008], [-87.68556146799993, 41.67924691900002], [-87.68564732299988, 41.67993375500003], [-87.68574431599987, 41.680903686000065], [-87.68575096799991, 41.680932480000074], [-87.68574796199992, 41.680959531000035], [-87.68565414199989, 41.681803917000074], [-87.68565102799992, 41.68183194600004], [-87.68565049599991, 41.681862309000046], [-87.6857510279999, 41.682732594000065], [-87.6857510279999, 41.68278948500005], [-87.6857510279999, 41.68357525900006], [-87.6857510279999, 41.68363237800007], [-87.6857510279999, 41.683689485000045], [-87.6857510279999, 41.684392567000074], [-87.6857510279999, 41.68444968600004], [-87.6857510279999, 41.68447525900006], [-87.6857510279999, 41.68450679300007], [-87.6857510279999, 41.68453194600004], [-87.6848991599999, 41.68445]]]}, "properties": {"city": "Chicago", "name": "Beverly Woods", "regionid": "403372", "geo_point_2d": [41.680813288197335, -87.6832377372429], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.631371, 41.932729500000036], [-87.631053, 41.932332], [-87.6310582819999, 41.932260688000035], [-87.63097899999988, 41.932215], [-87.6309129999999, 41.93213], [-87.630859, 41.932028], [-87.630835, 41.931905], [-87.630881, 41.931554], [-87.631084, 41.930856], [-87.631125, 41.930658], [-87.63115699999989, 41.929861], [-87.63115, 41.929719], [-87.6310699999999, 41.929153], [-87.630964, 41.928749], [-87.630809, 41.928398], [-87.6304399999999, 41.927799], [-87.63019599999988, 41.927513], [-87.6297799999999, 41.927094], [-87.629691, 41.926986], [-87.6295389999999, 41.926676], [-87.629524, 41.926524], [-87.6295263609999, 41.92639239900006], [-87.62962533699991, 41.92637590300005], [-87.63289534099992, 41.92583090200007], [-87.63394668799991, 41.92570219700007], [-87.63474853099989, 41.92567981700005], [-87.63620768499992, 41.925649548000074], [-87.6375817189999, 41.92562509700007], [-87.6384715149999, 41.92560626000004], [-87.6398476979999, 41.92559189300004], [-87.64098053199989, 41.92557437100003], [-87.6426805639999, 41.925552953000064], [-87.64559706399987, 41.92550228000004], [-87.64697303099989, 41.92547927100008], [-87.64874164099989, 41.92546030400006], [-87.64876999999989, 41.92546], [-87.6487762769999, 41.92657203800006], [-87.64881730399989, 41.92778488100003], [-87.64884929699988, 41.928755352000046], [-87.64891299399991, 41.930534390000034], [-87.64897609799992, 41.93231335800004], [-87.64898970099993, 41.93271710000004], [-87.6476449159999, 41.93275002200004], [-87.64664591499988, 41.93276102200007], [-87.64655991499987, 41.93275902200008], [-87.64583091599988, 41.93277402300003], [-87.6448869149999, 41.93280302200008], [-87.64357391599991, 41.932822022000025], [-87.64272091499991, 41.93283702200005], [-87.6425029149999, 41.93283802200005], [-87.64131291499987, 41.93286102200005], [-87.64035391599991, 41.93288902200003], [-87.6393649159999, 41.93290002200006], [-87.63883891499994, 41.932892023000065], [-87.63793391599994, 41.93291602200007], [-87.63618191499994, 41.932858022000055], [-87.63404491499993, 41.93282202300008], [-87.6338089149999, 41.93284002200004], [-87.63272491599989, 41.93286102100006], [-87.63264391699992, 41.93285102000004], [-87.63260779899989, 41.932835151000056], [-87.63256591499992, 41.93275902300006], [-87.63252798199993, 41.932711840000074], [-87.63210088999993, 41.93279297700007], [-87.6320429159999, 41.93282702200007], [-87.6318829489999, 41.932919193000075], [-87.6314160509999, 41.93285416900004], [-87.63134773099993, 41.93284569900003], [-87.631348, 41.932845], [-87.631371, 41.932785], [-87.631371, 41.932729500000036]]]}, "properties": {"city": "Chicago", "name": "Park West", "regionid": "403308", "geo_point_2d": [41.92919993516452, -87.63987334862385], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.74086540263771, 41.92421472136127], [-87.7415479157292, 41.924209022550166], [-87.74157491607463, 41.9251340229214], [-87.74160891584313, 41.92619902255157], [-87.7416359152892, 41.92709102230092], [-87.74163791628064, 41.927146022138324], [-87.74168191561097, 41.928174021861444], [-87.7416889159336, 41.928367022664155], [-87.74169391526475, 41.928525022753], [-87.74172091561002, 41.92940302275618], [-87.7417219156562, 41.929440022663144], [-87.74174991604752, 41.930401022895246], [-87.74175091609362, 41.93043202252548], [-87.74178391581623, 41.93150702261665], [-87.74184891521516, 41.933196022232735], [-87.74192791525952, 41.93517602267243], [-87.74199991588044, 41.93687702284168], [-87.74200391516554, 41.93697402281649], [-87.74208091062114, 41.938813022152495], [-87.74082591482319, 41.93882902199057], [-87.74070091625562, 41.93883102208293], [-87.74058391535891, 41.93883202212911], [-87.739359915594, 41.93884802196728], [-87.73813991511423, 41.93886402180536], [-87.73692291567194, 41.9388770224048], [-87.73591691606343, 41.938893022242965], [-87.73512286147638, 41.938905076755375], [-87.73444891584286, 41.938916022403674], [-87.73384491587956, 41.93892502191938], [-87.73289191601629, 41.93893702247256], [-87.73191391500065, 41.93894902212636], [-87.72953291496191, 41.938986022033326], [-87.72704678058079, 41.93901789040873], [-87.72700503815916, 41.93764847249624], [-87.72698861048805, 41.936671777204836], [-87.72696312190914, 41.9357567791841], [-87.72692924399792, 41.934720401271996], [-87.72690831592041, 41.93392590050122], [-87.7268773295482, 41.93270607964952], [-87.72686, 41.93169], [-87.72685748092573, 41.93168604955766], [-87.72685891580602, 41.93168602187474], [-87.72685004327779, 41.931241947505015], [-87.72682474648434, 41.9304506092786], [-87.72678554947639, 41.92922805065884], [-87.72677243111808, 41.92861804698984], [-87.72674892424713, 41.927704035498806], [-87.72671670204126, 41.926726628583324], [-87.72668560376123, 41.92556857019334], [-87.72667338203279, 41.92501553698726], [-87.72676879384628, 41.92448415933321], [-87.72762400233135, 41.92438885458059], [-87.72859902757811, 41.92437415424736], [-87.72963704099904, 41.92436143022767], [-87.73116247329897, 41.92434331322537], [-87.73244479198719, 41.924325540628466], [-87.73342029883945, 41.9243147943184], [-87.73476339846378, 41.9242977496186], [-87.73573959761836, 41.9242818764616], [-87.73732592381168, 41.92425596846212], [-87.73842414349478, 41.92423786539685], [-87.73946140119853, 41.92422532165667], [-87.74086540263771, 41.92421472136127]]]}, "properties": {"city": "Chicago", "name": "Belmont Gardens", "regionid": "403317", "geo_point_2d": [41.93162224570338, -87.73432784533544], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.68424495399992, 41.82365750500007], [-87.6842196409999, 41.82299610300003], [-87.6842249289999, 41.82299611100006], [-87.68422291499988, 41.822942023000046], [-87.6841999149999, 41.82243002300003], [-87.68419091599989, 41.822246022000044], [-87.68418191499987, 41.82201002200007], [-87.68413791599991, 41.81932302200004], [-87.6840829149999, 41.81751602200005], [-87.6840739159999, 41.81658202200002], [-87.68406391499988, 41.815694022000045], [-87.68395491599993, 41.813525022000064], [-87.68391091499988, 41.811971023000055], [-87.68389291599992, 41.81108402200004], [-87.68382191599993, 41.808350023000045], [-87.68381978299992, 41.80830389000003], [-87.6838110759999, 41.80801121500007], [-87.68378091099991, 41.80699554400007], [-87.6837669459999, 41.80622598600006], [-87.68374763199994, 41.80529863800006], [-87.68372876499988, 41.80486940000007], [-87.68372178299988, 41.804667723000065], [-87.68424399799993, 41.804613174000046], [-87.6843108339999, 41.80453593500003], [-87.68498491599988, 41.80453002200004], [-87.6857269159999, 41.804531022000035], [-87.68577891599993, 41.804531022000035], [-87.68598591499993, 41.804531022000035], [-87.68693791599989, 41.804522022000064], [-87.6869769159999, 41.804522022000064], [-87.68819791499992, 41.804511022000035], [-87.68823891599992, 41.804511022000035], [-87.6893959159999, 41.804503022000056], [-87.6894359159999, 41.804503022000056], [-87.69232191599988, 41.80446302200004], [-87.69400791499991, 41.804439023000036], [-87.6940529149999, 41.80443802200004], [-87.69410691599987, 41.804437022000045], [-87.69612091599991, 41.80439702200003], [-87.69710291599992, 41.804382023000066], [-87.69769991499992, 41.80437302200004], [-87.69928291499988, 41.80432802200004], [-87.70087291499992, 41.80430102200005], [-87.70261391599988, 41.804257022000044], [-87.70378291499989, 41.80424502300007], [-87.70383291599988, 41.804244023000074], [-87.70387891499988, 41.804244023000074], [-87.70399291599993, 41.80424302200002], [-87.7061169159999, 41.804211022000054], [-87.7077479149999, 41.80418902200005], [-87.70856291599993, 41.804177022000026], [-87.70875091599991, 41.80417202200004], [-87.71005991599989, 41.804135022000075], [-87.71113991599991, 41.80411802200006], [-87.71158791599991, 41.80411102300007], [-87.71254091599988, 41.80407002300006], [-87.71294391599989, 41.804053022000055], [-87.71349591599993, 41.80399202300002], [-87.71367596999993, 41.80397003400008], [-87.71367791499992, 41.80406302200002], [-87.71367991499991, 41.80411102300007], [-87.71368091499993, 41.80415702200007], [-87.71370991599989, 41.80514202300008], [-87.71376991499992, 41.805651023000046], [-87.7137929069999, 41.80602899100005], [-87.71380291599989, 41.80658502200003], [-87.71380591599987, 41.80672702300006], [-87.71381491499992, 41.80726902300005], [-87.7138409159999, 41.807913023000076], [-87.7138459159999, 41.80804102200004], [-87.71384791599989, 41.80809102300008], [-87.71384991599989, 41.808138023000076], [-87.71385191599988, 41.808200022000044], [-87.7138699159999, 41.80880102300005], [-87.7138729149999, 41.80922902300005], [-87.7140299149999, 41.81113902200006], [-87.7140419159999, 41.81181002200008], [-87.71405391499988, 41.81250002200004], [-87.71403691699992, 41.81267102200007], [-87.71410191599993, 41.81536702300007], [-87.71426191599993, 41.82134902300003], [-87.7142689159999, 41.82210402200006], [-87.71423491699993, 41.82235402200006], [-87.71424491499991, 41.82324602200003], [-87.71424591499992, 41.82328902200003], [-87.7142509159999, 41.82358902300005], [-87.7142509159999, 41.823914022000054], [-87.7142509149999, 41.82413802200005], [-87.71421491699992, 41.82458302200006], [-87.7142469149999, 41.82554702200007], [-87.71425191599988, 41.82604202300006], [-87.71426091499988, 41.826195023000075], [-87.71426295899992, 41.82627510300005], [-87.71426207599988, 41.826275381000066], [-87.7143114929999, 41.82812296000007], [-87.71432292899992, 41.82855051400007], [-87.71386700999992, 41.82868982300004], [-87.7102221289999, 41.82988814000002], [-87.7071764059999, 41.83081184200005], [-87.70490459699992, 41.831610720000064], [-87.70230824299992, 41.83243456300004], [-87.7006355919999, 41.832933862000054], [-87.69946223999993, 41.83333330100003], [-87.6988880469999, 41.833582951000025], [-87.69831385299992, 41.83375770500003], [-87.69771469499993, 41.83385756500006], [-87.6970656059999, 41.83398239000007], [-87.69374526899989, 41.835055882000056], [-87.69062465199993, 41.836079445000045], [-87.68850263199991, 41.83675349800006], [-87.68720445499989, 41.83712797200006], [-87.68615592799992, 41.83752741100004], [-87.68508452299993, 41.837864138000036], [-87.68509826699992, 41.83744692900005], [-87.68495849999992, 41.83634898600007], [-87.6848709039999, 41.83558547400008], [-87.6849299159999, 41.835566022000066], [-87.68493057899988, 41.835565857000056], [-87.6849013749999, 41.834723294000064], [-87.68471063399991, 41.83366288200005], [-87.6846, 41.83260871800007], [-87.68439398299992, 41.83098127200007], [-87.68442075099993, 41.82992254400005], [-87.6843957969999, 41.82894697300008], [-87.68437504899991, 41.82805258600006], [-87.68434178899989, 41.82707087200003], [-87.68428498899992, 41.82528394800004], [-87.68424495399992, 41.82365750500007]]]}, "properties": {"city": "Chicago", "name": "Brighton Park", "regionid": "273040", "geo_point_2d": [41.81889082483459, -87.69839302690764], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.62653999999986, 41.83097], [-87.62654021681088, 41.83020932287118], [-87.62652084407006, 41.82944343849882], [-87.62648965325026, 41.82832751700938], [-87.62644865752442, 41.826934821369846], [-87.62644, 41.826098188137465], [-87.62642732252061, 41.8247751254132], [-87.62641692221297, 41.8236869497701], [-87.62649391543518, 41.82368602209056], [-87.62757891508797, 41.82367502248277], [-87.62862091545645, 41.82366902220631], [-87.62898504465097, 41.823664889821565], [-87.62898582706111, 41.82370803659457], [-87.62935004798636, 41.823700151339], [-87.62934986002807, 41.82366102183759], [-87.62961691550333, 41.8236580225986], [-87.63003191574762, 41.823637022529795], [-87.63026791583458, 41.82363402239171], [-87.63049591555261, 41.82362902216108], [-87.63065626909773, 41.823628050273804], [-87.63075917675701, 41.82742305727711], [-87.63080333660939, 41.82909415906349], [-87.63083093651726, 41.83013859767979], [-87.63085598126086, 41.830950061777926], [-87.62953, 41.83097], [-87.62952718134642, 41.830973428299004], [-87.62952704625297, 41.83096589002624], [-87.62899878809024, 41.83096515977687], [-87.62835789456835, 41.83097553480891], [-87.62710414368213, 41.83100140619695], [-87.62653999999986, 41.83097]]]}, "properties": {"city": "Chicago", "name": "Stateway Gardens", "regionid": "403347", "geo_point_2d": [41.82732833516787, -87.62861445442577], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.6674675110474, 41.968901884914374], [-87.66740324088782, 41.96845752797823], [-87.6672891429363, 41.967918132863524], [-87.6671817999036, 41.967415920763706], [-87.6670506747851, 41.966811058026735], [-87.66686813385812, 41.965981089873864], [-87.66680745994113, 41.96570441703208], [-87.66656951880614, 41.96461162418315], [-87.66641804287728, 41.96391250501135], [-87.66625016886366, 41.96310751816972], [-87.66606648919768, 41.96219604402094], [-87.665950806081, 41.96165418014629], [-87.66667991608395, 41.96162102210819], [-87.66771091601481, 41.96161502196968], [-87.66917291503728, 41.961596021981606], [-87.66923391509418, 41.9615910228159], [-87.66927400926811, 41.96158834972535], [-87.66930891597383, 41.96158602220069], [-87.67082391581842, 41.96156702221244], [-87.67163691517088, 41.961559022028105], [-87.67165991575133, 41.9615580220051], [-87.6724109150239, 41.96154902224719], [-87.67396091530064, 41.96153202087426], [-87.67418991510759, 41.96152702218976], [-87.67451391588065, 41.961520022028424], [-87.67516391602325, 41.961503022086355], [-87.67523791498051, 41.961501022040174], [-87.67599191577192, 41.96148902221327], [-87.67746791511671, 41.961462022040685], [-87.67897091608411, 41.96145102223677], [-87.680609915139, 41.96143902196009], [-87.68142491598732, 41.96142002197183], [-87.68222591601301, 41.961406022098934], [-87.68385291524106, 41.96138102197234], [-87.68506991591188, 41.96135402224953], [-87.68627091521385, 41.96133702280732], [-87.68749391507328, 41.96132402200777], [-87.68833791519053, 41.96131402222685], [-87.68863891588326, 41.96130302292281], [-87.68870275085966, 41.96130020615182], [-87.68870691499983, 41.961300023], [-87.68877991499991, 41.96129802300002], [-87.68992191599982, 41.961295022000044], [-87.69115591599986, 41.96128602200019], [-87.6911607569999, 41.96128604400018], [-87.69116, 41.96129000000016], [-87.6911799199999, 41.96219635800019], [-87.69119199799981, 41.96274590600018], [-87.69122214799991, 41.96350128300007], [-87.69124208299989, 41.96411927900004], [-87.69126557699988, 41.965080986000125], [-87.69129999999988, 41.965710000000115], [-87.69129255400024, 41.965713380106095], [-87.69039932003834, 41.96571612703731], [-87.68854528320084, 41.965735306798024], [-87.6880644051831, 41.96574532494416], [-87.68743723598917, 41.96576589276183], [-87.6870480499035, 41.96587954213298], [-87.68685, 41.96596], [-87.6866799999999, 41.965700000000176], [-87.68642140483995, 41.96579353423884], [-87.68600016878136, 41.9659016958641], [-87.68503916002558, 41.96591529495254], [-87.68399999999983, 41.96593], [-87.68400108224107, 41.966319227012605], [-87.6840367021401, 41.967214152227065], [-87.68406445023992, 41.96810603820387], [-87.68404833401662, 41.969000297983705], [-87.68407214111959, 41.969686739034906], [-87.68408642578139, 41.97009860386568], [-87.68410972912073, 41.97078512016495], [-87.68412790994665, 41.9713366000555], [-87.6841482790939, 41.97195446885145], [-87.68416, 41.97231000000015], [-87.68469449214194, 41.97230561895796], [-87.6860001032497, 41.972289917008354], [-87.68716840317344, 41.972275385048185], [-87.68812979797738, 41.97226177513801], [-87.68854186779325, 41.97225083518702], [-87.68901999999989, 41.97224], [-87.68900032818662, 41.97175214383054], [-87.69065999999985, 41.97189], [-87.69122339589904, 41.97188130581032], [-87.6917730781599, 41.97187282293281], [-87.69246018123602, 41.97186221921099], [-87.6930785741545, 41.97185267616126], [-87.69389999999989, 41.97184000000017], [-87.69387478591823, 41.97119703891722], [-87.69384616483539, 41.97009856981079], [-87.69382873284033, 41.969205736101415], [-87.69380731221196, 41.96796905904608], [-87.69379052093062, 41.96721343888693], [-87.69375966106757, 41.96618305688481], [-87.69387, 41.96569], [-87.69472767699989, 41.96568167300018], [-87.69582853499985, 41.96567098500018], [-87.69695999999986, 41.96566], [-87.69790597699986, 41.96565524000003], [-87.69806857099984, 41.96565442100005], [-87.69857832699991, 41.96615284900008], [-87.69918076699985, 41.96678206500009], [-87.69960916999985, 41.967197080000055], [-87.69993047099986, 41.96747821900014], [-87.70039903599982, 41.96789323400011], [-87.70082743799983, 41.968308248000156], [-87.7010525049999, 41.96850330600013], [-87.70122906599994, 41.96865632500009], [-87.70148342899992, 41.96885713900019], [-87.70152359199989, 41.96912489000005], [-87.70165746799994, 41.969339091000094], [-87.70200554499988, 41.969633618000046], [-87.70225990799987, 41.969834431000045], [-87.70244733399991, 41.970075408000184], [-87.7026347599999, 41.970343159000045], [-87.70290251199985, 41.97077156100014], [-87.70315687599984, 41.971146413000056], [-87.70327736399986, 41.971387389000114], [-87.70342462699986, 41.971668528000194], [-87.70358527799993, 41.97194966700009], [-87.70369237899983, 41.97231113200007], [-87.7037727039999, 41.97271275900017], [-87.70389319199984, 41.9731143860001], [-87.70404045499986, 41.97338213700014], [-87.70417433099992, 41.97377037700011], [-87.7042126239999, 41.973908231000145], [-87.70424126899975, 41.974011353000144], [-87.70424126899975, 41.97421216700007], [-87.70429481899986, 41.974412980000075], [-87.70434836899993, 41.97470750700002], [-87.70438853199987, 41.9749618710001], [-87.7044822449998, 41.97524301], [-87.7045357949999, 41.97543043600005], [-87.70456257099987, 41.97560447400007], [-87.70460446899988, 41.9757553090002], [-87.7044880399999, 41.97575593500008], [-87.70373172199989, 41.97576000000015], [-87.7020132839999, 41.97576533400006], [-87.70022681699977, 41.97576533700015], [-87.69823445199988, 41.97577490500005], [-87.69740866699985, 41.97579218600014], [-87.69569074299991, 41.975810359000135], [-87.69493477699987, 41.975814941000166], [-87.69362915399995, 41.97582541200005], [-87.69287326099992, 41.975834100000064], [-87.6917735379999, 41.975843747000056], [-87.69094885199986, 41.97584901700014], [-87.68978054599992, 41.97585648200014], [-87.68908006719748, 41.97586058194399], [-87.6873063790763, 41.975885580920384], [-87.68579502602711, 41.975919032193076], [-87.68503935276244, 41.97593458111202], [-87.68366406084796, 41.9759540381592], [-87.68249622311414, 41.975961904190754], [-87.6813951509869, 41.97597465321872], [-87.68029575982675, 41.97598411822209], [-87.67899000599245, 41.975992648983606], [-87.67850894016661, 41.97599583491619], [-87.67754680208417, 41.97600675400511], [-87.67692851605236, 41.976019286917946], [-87.67603510913148, 41.97603361289001], [-87.67521015000887, 41.97604], [-87.67473181682531, 41.9760521409308], [-87.67438899168468, 41.97605407509846], [-87.67438896917726, 41.97605002287235], [-87.67205191579302, 41.97608602280268], [-87.66966791520461, 41.976111022929274], [-87.66961491583208, 41.9761150220718], [-87.66953691583291, 41.976120022187146], [-87.66839391506899, 41.97615802216347], [-87.66838984200054, 41.9761580690655], [-87.66836072499981, 41.97580138600002], [-87.66826679399978, 41.97497114500016], [-87.66813078199988, 41.97386950400004], [-87.66797583199978, 41.972933898000086], [-87.66785517599992, 41.97216899800021], [-87.66771905499984, 41.97106839100019], [-87.66763138199988, 41.970238412000164], [-87.66755164599992, 41.96948358200006], [-87.6674675110474, 41.968901884914374]]]}, "properties": {"city": "Chicago", "name": "Ravenswood", "regionid": "33597", "geo_point_2d": [41.96928841789519, -87.68273409945928], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.61541625047785, 41.774803668198345], [-87.61720943904248, 41.774803668198345], [-87.61888170774762, 41.776940455988345], [-87.61888170774762, 41.779448859046404], [-87.6188314558956, 41.78005425294125], [-87.6187669153784, 41.780056022680085], [-87.61788691618221, 41.78007602180334], [-87.61775691558562, 41.780079021941695], [-87.61759991554304, 41.78008302212597], [-87.6171439161068, 41.78009302258705], [-87.6162399149049, 41.78011402265584], [-87.61551382837871, 41.780138035453284], [-87.61550991542858, 41.779426022417645], [-87.61550891538256, 41.779254022582904], [-87.61550291600531, 41.7783010227197], [-87.61547591566004, 41.77742902209369], [-87.61544591607581, 41.776455022161606], [-87.61541625047785, 41.774803668198345]]]}, "properties": {"city": "Chicago", "name": "Parkway Gardens", "regionid": "403355", "geo_point_2d": [41.77764255397448, -87.61704426029556], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.56564699999987, 41.774134], [-87.56443555199982, 41.772276446000156], [-87.5642409999999, 41.772158000000125], [-87.56405599999985, 41.77208300000013], [-87.56377199999989, 41.772073], [-87.563535, 41.77208700000011], [-87.56337399999983, 41.77206400000016], [-87.563313, 41.772026], [-87.56326499999987, 41.771964], [-87.563232, 41.77187], [-87.56323699999986, 41.771676], [-87.563318, 41.77153100000011], [-87.56332699999977, 41.771288], [-87.5632609999999, 41.770873], [-87.563195, 41.770672], [-87.563108, 41.77045800000015], [-87.5629699999999, 41.770223], [-87.562866, 41.770091000000164], [-87.5625009999999, 41.76956700000015], [-87.56245587999989, 41.76952242], [-87.55934599999989, 41.769334], [-87.55973310199984, 41.76825011400008], [-87.55941437599995, 41.76777356700017], [-87.559413, 41.767326000000175], [-87.55936199999987, 41.766781], [-87.55936499999987, 41.76670300000014], [-87.55937899999986, 41.766655], [-87.559413, 41.766599], [-87.55947699999989, 41.766547], [-87.55983899999983, 41.766422], [-87.56043199999985, 41.76615800000017], [-87.56048899999988, 41.766071], [-87.56051198199972, 41.76593863000014], [-87.55924599999987, 41.765034], [-87.55754599999989, 41.763134], [-87.5567559999999, 41.76267300000013], [-87.556244, 41.76224000000018], [-87.55591459199982, 41.76199435300001], [-87.55582699999988, 41.761948000000174], [-87.55541666399994, 41.76176838400006], [-87.55484599999988, 41.7619600000001], [-87.55475166919369, 41.76185852581994], [-87.55452314731812, 41.7616126987043], [-87.553367, 41.760369], [-87.55167999999983, 41.759383], [-87.550646, 41.758933999999996], [-87.54764599999986, 41.75853400000015], [-87.5466459999999, 41.758734000000175], [-87.54345583009307, 41.75991892025116], [-87.54314599999985, 41.760034], [-87.542446, 41.7595340000001], [-87.54034499999989, 41.757535], [-87.53969901401919, 41.75763657012289], [-87.53968233299977, 41.757593753000414], [-87.5396612129999, 41.75752273300005], [-87.5396460119998, 41.75745021600018], [-87.53964168399979, 41.75742326699999], [-87.53963836099989, 41.75740030500005], [-87.53963336199965, 41.75736230500045], [-87.53962567799981, 41.757245], [-87.53963934099977, 41.75708883400012], [-87.53967991399966, 41.756937414000205], [-87.53974616499974, 41.75679533900008], [-87.5398360799997, 41.75666692700029], [-87.53987213399985, 41.75662649700015], [-87.5399441339997, 41.75655049700008], [-87.53999157099969, 41.75650399200005], [-87.54004231099972, 41.75646111600041], [-87.5400960769999, 41.7564221020002], [-87.5401525769997, 41.75638716400015], [-87.54021150099979, 41.75635649100019], [-87.5403117419998, 41.75630893200007], [-87.54316685399962, 41.754622907000105], [-87.54321470299983, 41.75438702300013], [-87.54321154899976, 41.7543512280001], [-87.5431380099998, 41.75402920300019], [-87.54306501399988, 41.753794888000066], [-87.54267570099974, 41.75293750500009], [-87.5425582619998, 41.75289299800006], [-87.54248609299978, 41.7528741540001], [-87.54235826799984, 41.75285942100003], [-87.54314599999985, 41.75215400000019], [-87.54623143799986, 41.75198741300018], [-87.54716046899992, 41.751971152000166], [-87.54800120099985, 41.751959889000034], [-87.5487393259999, 41.75195000000019], [-87.54985417199987, 41.75195000000019], [-87.55087597499993, 41.75193984000006], [-87.55221999999986, 41.75191], [-87.5522680019999, 41.751929234000066], [-87.55235390399983, 41.75190188800011], [-87.55245691599993, 41.751902023000035], [-87.55282891599988, 41.751902023000035], [-87.55401291499987, 41.75188402300015], [-87.55409491499991, 41.751882023000164], [-87.55466691599989, 41.75186802200016], [-87.55524291499994, 41.751861023000025], [-87.55547491499986, 41.751860022000024], [-87.55648091599988, 41.7518480220001], [-87.55689291599987, 41.75185202200008], [-87.55768591599984, 41.75183902200007], [-87.55842091499991, 41.751833022000106], [-87.55890791499988, 41.75182402300016], [-87.55988791599978, 41.75182102200018], [-87.56013691499986, 41.751820022000096], [-87.5613289149999, 41.75179702200015], [-87.56202491599986, 41.75178602300004], [-87.56247691499985, 41.751773022000116], [-87.56377291599988, 41.75175802200011], [-87.56496891499982, 41.75173802200006], [-87.56619291599984, 41.75172902300012], [-87.56739691499988, 41.75169102299999], [-87.56861591599994, 41.75168502200012], [-87.56923891599988, 41.75167602200008], [-87.5698669159999, 41.751666023000055], [-87.57107691599991, 41.75165402200004], [-87.57236791499989, 41.75164202200011], [-87.57345591599993, 41.75162002200006], [-87.57471891599981, 41.75159502200013], [-87.57593191599989, 41.75159002300016], [-87.57717191499982, 41.751578022000146], [-87.57836391499985, 41.75156402200005], [-87.57966491499984, 41.751549023000145], [-87.58001491499986, 41.75154702200007], [-87.58089391499993, 41.75153902200011], [-87.58162091499976, 41.75151902200006], [-87.58206891599988, 41.751522022000124], [-87.5828109159999, 41.751512023000096], [-87.58329291499982, 41.751512023000096], [-87.58435191599986, 41.75148702200008], [-87.58514991599992, 41.75147402300015], [-87.58544678099987, 41.75146907800019], [-87.5854989159999, 41.751468022000104], [-87.58553491499987, 41.751468022000104], [-87.58557691499989, 41.751469022], [-87.58568091599989, 41.751469022], [-87.58575591599985, 41.751469022], [-87.58581211015132, 41.75146889377227], [-87.58581439883125, 41.75148253732185], [-87.58581591180152, 41.75154088604859], [-87.58581591520228, 41.751613022883404], [-87.58581674478968, 41.751712105785636], [-87.58583111093344, 41.751722459895205], [-87.58591999999985, 41.75278546378954], [-87.58595938522187, 41.75473999895028], [-87.58598340022252, 41.75557561507218], [-87.58601058089606, 41.75641280287002], [-87.58607610111055, 41.757617042116834], [-87.58609345218038, 41.75817380789041], [-87.58610285519954, 41.75900984117408], [-87.5861181590199, 41.76003531592221], [-87.5861496901457, 41.76096724220289], [-87.5861918928566, 41.76217137677344], [-87.58622102822414, 41.76291416877236], [-87.5862534579603, 41.763656074128896], [-87.58629783811433, 41.7647692008953], [-87.5863146118959, 41.76551277887316], [-87.58632999999985, 41.76699974778702], [-87.58632467822457, 41.76830104579237], [-87.58623361416623, 41.76902924115096], [-87.58619137813075, 41.770810995841956], [-87.58622999999974, 41.77201655384489], [-87.58623561320015, 41.772669796163974], [-87.58625208225426, 41.7733060139109], [-87.58615991599989, 41.773304023], [-87.5860569159999, 41.77330302300009], [-87.58571791499983, 41.77330902200014], [-87.58504991499989, 41.773321022000076], [-87.58386491499991, 41.77334502300002], [-87.58339991599988, 41.7733610230001], [-87.58263991599989, 41.773370022000044], [-87.58139022877899, 41.773400741495784], [-87.58137203193967, 41.77257346622848], [-87.58134160780888, 41.77080888190661], [-87.58131430904164, 41.76904037219265], [-87.58127441689285, 41.767370820896154], [-87.58123, 41.76605000000012], [-87.58069804513214, 41.76605831189653], [-87.5796762969952, 41.766080000000116], [-87.57846817104196, 41.76609441520709], [-87.57698227691972, 41.76609433623862], [-87.57627999999983, 41.76610000000017], [-87.57629720479785, 41.76681495613162], [-87.57635958303067, 41.76885693596485], [-87.5763986660511, 41.77052928675781], [-87.57641535476304, 41.7724815398926], [-87.57640406305788, 41.77344011606979], [-87.5752849159999, 41.773458023000124], [-87.57405191499984, 41.773480023000175], [-87.57278791599991, 41.7735060220001], [-87.57159591599986, 41.77351502200014], [-87.56996791599988, 41.77352602200016], [-87.56828391599991, 41.77353602300019], [-87.56739091599984, 41.77354302200015], [-87.56674486899985, 41.773524852000065], [-87.56625799999989, 41.773844500000045], [-87.5662677879999, 41.77384929400009], [-87.56603436406087, 41.774405154873506], [-87.56564699999987, 41.774134]]]}, "properties": {"city": "Chicago", "name": "South Shore", "regionid": "269608", "geo_point_2d": [41.76061500881076, -87.5680755242311], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.60636091547298, 41.802093022276686], [-87.60634991586527, 41.80155802279606], [-87.60634191549654, 41.80128802294061], [-87.6063239164654, 41.800309022778066], [-87.60630791572785, 41.79887302223399], [-87.60629491602786, 41.79851902300273], [-87.6062369160523, 41.79670702221332], [-87.60616491543128, 41.794877022392846], [-87.6061149158244, 41.79405402222695], [-87.60610191612453, 41.79303802305691], [-87.60607091559481, 41.79199202250399], [-87.60604691538792, 41.79120002196834], [-87.60596291601239, 41.78946202189204], [-87.60597891585056, 41.787608021864585], [-87.60604696844781, 41.78709502159947], [-87.60594391603587, 41.78627202233278], [-87.60593591836498, 41.78573396605671], [-87.60611991605496, 41.785734021814505], [-87.60664191583561, 41.785734021814505], [-87.60833591568222, 41.7857210221147], [-87.60959291606912, 41.78570602232255], [-87.61001991596713, 41.785700022045916], [-87.6108349157643, 41.78566902241558], [-87.61193291601643, 41.785630022416534], [-87.61324791547943, 41.78560002193292], [-87.61448991607425, 41.78558102195639], [-87.61565827733648, 41.78556816207186], [-87.6156590509001, 41.78556815355743], [-87.61560191607212, 41.783765021882076], [-87.61556691535814, 41.78193702305288], [-87.61553491658093, 41.78057802246082], [-87.61551382837871, 41.780138035453284], [-87.61623991490482, 41.78011402265584], [-87.6171439161067, 41.78009302258723], [-87.61759991554304, 41.78008302212615], [-87.61775691558562, 41.780079021941695], [-87.61788691618221, 41.78007602180352], [-87.61876691537832, 41.78005602268026], [-87.6193869160794, 41.780039021896556], [-87.61999591537386, 41.78002302205848], [-87.62018591513895, 41.78000902231232], [-87.6203949157799, 41.77997802268189], [-87.62124591543794, 41.77991402242978], [-87.62202791551259, 41.77989302236099], [-87.62268391607438, 41.77987502243055], [-87.62294891569938, 41.77987502243055], [-87.623657915107, 41.77987502243055], [-87.6247169162592, 41.779907022107146], [-87.62528204752437, 41.77992088875337], [-87.6252959159695, 41.78028602249058], [-87.62531291585375, 41.78071802171994], [-87.6253587848746, 41.78306115400583], [-87.62543891626609, 41.783062021851805], [-87.62536794838695, 41.78349716128659], [-87.62537992294402, 41.78406397469027], [-87.62539230946349, 41.78463065795788], [-87.62540019179401, 41.784991274582886], [-87.62541628286088, 41.78586723454212], [-87.62542992318923, 41.78679477687661], [-87.62544689832225, 41.78756732282667], [-87.6254738010654, 41.78880484901699], [-87.6254872486267, 41.789372696444026], [-87.62551433984707, 41.79035277485437], [-87.62554476740904, 41.79133286721382], [-87.62557162994472, 41.7921566516416], [-87.62559988936965, 41.793083434336005], [-87.62561110093723, 41.79349545944941], [-87.62569999999982, 41.79457000000027], [-87.62569459435413, 41.79551008223827], [-87.62574374032107, 41.79732012307496], [-87.62577348693877, 41.79844029938969], [-87.62580051526366, 41.79955288159161], [-87.6258461694998, 41.80108389874519], [-87.62586501989969, 41.80185502209748], [-87.62586191597983, 41.80185502209748], [-87.62583291554222, 41.80185502299693], [-87.62580791528916, 41.80185702308911], [-87.62492191581644, 41.80185702218975], [-87.62422591610878, 41.80186202242028], [-87.62265391648997, 41.80188202244282], [-87.62097391549014, 41.80190202246561], [-87.61936591511123, 41.80193202204968], [-87.6184589155698, 41.801948021888016], [-87.61798691539593, 41.80195702230274], [-87.61792791627363, 41.80195702230274], [-87.61771691554047, 41.80196402262563], [-87.61674691579275, 41.80197602227927], [-87.61660391549627, 41.80197802237145], [-87.61653491591288, 41.801979022417534], [-87.61643491579979, 41.80198002246372], [-87.61636591531715, 41.8019810225098], [-87.61628691527275, 41.801982022556075], [-87.61613699904008, 41.80198550916245], [-87.61611491633748, 41.801986022740344], [-87.61608691594589, 41.80199102117215], [-87.61561291568002, 41.80199502225578], [-87.61493791514171, 41.80200002248633], [-87.61416091529757, 41.802013022186216], [-87.6124619161198, 41.80204502186282], [-87.61125791547813, 41.802068022023626], [-87.60999191557578, 41.802087022000144], [-87.60782791564755, 41.80212202181474], [-87.60647491533204, 41.802155022436715], [-87.60636390931603, 41.802153025941934], [-87.60636091547298, 41.802093022276686]]]}, "properties": {"city": "Chicago", "name": "Washington Park", "regionid": "275927", "geo_point_2d": [41.79219425905019, -87.6165506385566], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.75589580253842, 41.91763107192232], [-87.7558609732618, 41.91647122993934], [-87.75582868823756, 41.91544465413274], [-87.75583491151082, 41.9154440250072], [-87.75643891597078, 41.91558402156821], [-87.757552916061, 41.91571302211863], [-87.75818991574698, 41.91578702193263], [-87.75877391658702, 41.91585502236912], [-87.75906091542753, 41.915902021837795], [-87.759676915944, 41.91600202195078], [-87.75975691513509, 41.91601502255005], [-87.76058591557769, 41.916102022963145], [-87.76145891535062, 41.91622002300666], [-87.76316491575022, 41.91642102237963], [-87.76399691543173, 41.91649102290838], [-87.76492191580297, 41.91652002244635], [-87.76553491528188, 41.91652102249244], [-87.76560091562635, 41.91652102249244], [-87.7656079260681, 41.91652102249244], [-87.76564349575432, 41.917812150227384], [-87.76572470776023, 41.919464804626585], [-87.76578920715725, 41.920865421876854], [-87.76588605012955, 41.923371127917825], [-87.76590811791128, 41.92390775947266], [-87.76533749730311, 41.92391], [-87.76417672609819, 41.92393138373769], [-87.76295738274496, 41.923960000000115], [-87.76173681117642, 41.923960000000115], [-87.7603348206667, 41.923985979983705], [-87.75911328854635, 41.924005215415], [-87.75789465997657, 41.92402537721371], [-87.7566734575809, 41.924050856703346], [-87.75607, 41.92398000000017], [-87.756061647439, 41.92361248731597], [-87.75604483184935, 41.92263486644968], [-87.75603644759472, 41.92178081618041], [-87.7559824648942, 41.91994909728011], [-87.75589580253842, 41.91763107192232]]]}, "properties": {"city": "Chicago", "name": "Hanson Park", "regionid": "403324", "geo_point_2d": [41.920035789451866, -87.76073005468055], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.82626100019446, 41.937629999998045], [-87.8264801966439, 41.93761173386614], [-87.82651958335138, 41.937608451340694], [-87.827460999947, 41.93752999988274], [-87.82822005831667, 41.93758838926478], [-87.82876099961794, 41.937629999995806], [-87.8296692501163, 41.9375601343652], [-87.83006100018815, 41.93752999988274], [-87.83121428446334, 41.93752999988274], [-87.83126951722518, 41.93752999988274], [-87.8324610002034, 41.93752999988274], [-87.83376099987444, 41.93752999988274], [-87.83470910472774, 41.93761619090613], [-87.83486100021867, 41.93762999999599], [-87.83602795361685, 41.9375402342258], [-87.83605204705698, 41.93809417643843], [-87.83608245371629, 41.93863896241734], [-87.83612656301483, 41.939429254015806], [-87.83622619145251, 41.941370496341015], [-87.83629721250617, 41.94283611262695], [-87.8363568694674, 41.943684128846506], [-87.83646004424946, 41.9446268881613], [-87.8358730013524, 41.94464135711071], [-87.83453831447564, 41.94467870120413], [-87.83344651493005, 41.94471519623185], [-87.83223252030218, 41.944751436992455], [-87.83089444706002, 41.9447842932706], [-87.82974186176943, 41.94481594042341], [-87.82858720607119, 41.94484998662633], [-87.82725067497393, 41.94488323312576], [-87.82658016550891, 41.944904303230516], [-87.82658, 41.9449], [-87.82657, 41.9444], [-87.82646218257464, 41.94234437555136], [-87.82641610520454, 41.94161865697244], [-87.8263583745046, 41.94052267766912], [-87.82635504751853, 41.94003693770585], [-87.82631092890603, 41.938274661846926], [-87.82626100019446, 41.937629999998045]]]}, "properties": {"city": "Chicago", "name": "Belmont Terrace", "regionid": "403282", "geo_point_2d": [41.941170337615524, -87.83128725693491], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.66009209306269, 41.98397986193279], [-87.66007816157598, 41.98352848327984], [-87.66008, 41.98353], [-87.66073445342803, 41.98394093587361], [-87.66181305814993, 41.98459263676286], [-87.66284077439137, 41.98521652740211], [-87.66439720338904, 41.986175202085654], [-87.66551973408278, 41.98686820481223], [-87.66678188564347, 41.98765264211384], [-87.66832525677401, 41.98860631186093], [-87.67002, 41.98978], [-87.67007, 41.99067], [-87.6700700828403, 41.990672568051515], [-87.66888787670261, 41.99069260423542], [-87.66613924054047, 41.99072718922389], [-87.66380180155171, 41.990751013979036], [-87.66208427276328, 41.990770696192065], [-87.6609161690148, 41.99078998129571], [-87.66029, 41.9908], [-87.66028970223944, 41.99079996701649], [-87.66027294563568, 41.99030542343715], [-87.66024393808021, 41.98941066459494], [-87.66021679055095, 41.98838208660433], [-87.66020733421212, 41.9879683717822], [-87.66020101802374, 41.987692038542626], [-87.66016895958944, 41.98652430788761], [-87.6601596514137, 41.98624971670456], [-87.66013867441995, 41.9855634883991], [-87.66012173083492, 41.985082387058206], [-87.6601051154194, 41.98446429360196], [-87.66009209306269, 41.98397986193279]]]}, "properties": {"city": "Chicago", "name": "Magnolia Glen", "regionid": "403259", "geo_point_2d": [41.98828218380367, -87.6638611211376], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.80601291598552, 41.93791402208043], [-87.80662896269746, 41.937908180586824], [-87.80663, 41.93791], [-87.80667021200746, 41.938819796668476], [-87.8067309599981, 41.94046198393678], [-87.8067777158286, 41.942039190690146], [-87.8068013854014, 41.94277294744417], [-87.80683981181863, 41.943865107284246], [-87.80686079935249, 41.94441078316624], [-87.80688880767305, 41.94516980012885], [-87.80630214038088, 41.94517940575404], [-87.80490760693216, 41.94520800966555], [-87.80387449266281, 41.94523341411476], [-87.80265921725363, 41.94525872230239], [-87.80132372280714, 41.945280266839305], [-87.80029081326923, 41.94529577877282], [-87.79895528058364, 41.94531999999989], [-87.79767930450294, 41.945345626719565], [-87.7965262834076, 41.94536924125567], [-87.79525169693943, 41.945395080562456], [-87.79391620115807, 41.94542226777698], [-87.79276346268826, 41.94544726952014], [-87.79124545061886, 41.945477171784276], [-87.78936274831192, 41.94551871490406], [-87.78814954200195, 41.94554293800003], [-87.78699537388894, 41.94556721628751], [-87.78619, 41.94558], [-87.78623293302142, 41.9450127250719], [-87.78620082875536, 41.94422426856159], [-87.78616152805336, 41.94331514522898], [-87.78611673247899, 41.94228484701689], [-87.7861, 41.9419], [-87.78608216903204, 41.941494345480756], [-87.78604462096793, 41.9407070258089], [-87.78600109179843, 41.93979681515166], [-87.78596739967573, 41.93882239157054], [-87.78593018540255, 41.93823293668137], [-87.78702191560559, 41.93821102138211], [-87.78716791604035, 41.938208022142945], [-87.78826291615422, 41.938187022074324], [-87.78840891568962, 41.93818402193606], [-87.78948591587304, 41.93816502195953], [-87.78962991621565, 41.938163021867254], [-87.79066291526992, 41.93814202179865], [-87.79085000303043, 41.938139074720326], [-87.79194591595599, 41.938120022583206], [-87.79314291627503, 41.938108022029866], [-87.79437591555548, 41.938087021961245], [-87.79558091534388, 41.93806502184636], [-87.79681791570805, 41.938049022008286], [-87.79802091540427, 41.93804002249275], [-87.799263916045, 41.93801502223969], [-87.80110191533491, 41.93798902194053], [-87.80247691576524, 41.93796402258664], [-87.80293891547804, 41.937956022217925], [-87.8038829158258, 41.937941022425676], [-87.80415191563532, 41.9379370222414], [-87.80521891535768, 41.93792102240306], [-87.80538591496179, 41.93792002235705], [-87.80601291598552, 41.93791402208043]]]}, "properties": {"city": "Chicago", "name": "Schorsch Village", "regionid": "403284", "geo_point_2d": [41.94171514733618, -87.79640761435786], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.68830469899993, 41.946722724000026], [-87.67372245199992, 41.94692668200001], [-87.67371616799993, 41.94694017000005], [-87.67361089999996, 41.94694224100003], [-87.67329881899995, 41.94640485300004], [-87.67299893999994, 41.94588914200005], [-87.67278874799995, 41.94558902000005], [-87.67209746999995, 41.944601982000044], [-87.67151095899997, 41.943784025000056], [-87.67058174, 41.94245766800002], [-87.66956022599997, 41.940978206000054], [-87.66901277299995, 41.940208083000066], [-87.66865, 41.93968], [-87.67053275499993, 41.93966798900003], [-87.67125712299996, 41.93965297800002], [-87.67167350999993, 41.93964434900004], [-87.67239409399996, 41.93962941700004], [-87.67343287499992, 41.939620847000064], [-87.67343996899993, 41.939620538000035], [-87.68813813699994, 41.93948916300007], [-87.68830469899993, 41.946722724000026]]]}, "properties": {"city": "Chicago", "name": "Roscoe Village", "regionid": "403292", "geo_point_2d": [41.94303060091173, -87.67962073409387], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.72637373439652, 41.91439424757358], [-87.72635216200783, 41.91363218089306], [-87.72630886666542, 41.91220115551966], [-87.7262687923841, 41.91092075248931], [-87.72640985081988, 41.90977841754782], [-87.72713485881512, 41.90976682074892], [-87.72817268249314, 41.909719042830766], [-87.72963540746575, 41.909709776613106], [-87.73067347400647, 41.90971451298725], [-87.73152776046601, 41.90970802132902], [-87.7330533793201, 41.909690000000225], [-87.73415178228726, 41.909686058109685], [-87.73763026759764, 41.90962777034467], [-87.73909471722763, 41.90961263197517], [-87.7401931548398, 41.90959377427862], [-87.7408040087793, 41.90959000000012], [-87.74133271476212, 41.90957737917592], [-87.74220794633378, 41.909558335830354], [-87.74324538898513, 41.90955164265172], [-87.7443439032033, 41.90954284827813], [-87.74562542541405, 41.909524751203186], [-87.74598969056683, 41.909520513589904], [-87.74605999999989, 41.9095200000002], [-87.74605201975906, 41.90957586168508], [-87.74599500002363, 41.91022999992127], [-87.74584737192001, 41.91073208251881], [-87.74582299571061, 41.910975549327134], [-87.74583137435398, 41.911165449505916], [-87.74585970077379, 41.9117142568056], [-87.74587376900205, 41.91214099306687], [-87.74589135630274, 41.91268961626245], [-87.74589905749619, 41.9129334873806], [-87.74591665740643, 41.913666129582076], [-87.74596976789934, 41.914416719812586], [-87.74597491565275, 41.914417022632215], [-87.74603440210691, 41.91442531476596], [-87.7459776498086, 41.914886638334266], [-87.74602239446332, 41.91580581868407], [-87.74604585092082, 41.916235071854146], [-87.74608519497228, 41.91732754750879], [-87.74610388627059, 41.91842654099186], [-87.74613128163308, 41.919098813070036], [-87.7461880581859, 41.92025533827489], [-87.7462, 41.921659584035275], [-87.74622021104902, 41.922819801364064], [-87.74625043193996, 41.92373422132657], [-87.74626, 41.92416], [-87.74544218073888, 41.92417000000015], [-87.74452701529597, 41.92417754974517], [-87.74312336778948, 41.924195338473524], [-87.74208615177517, 41.92420572652128], [-87.74154791572913, 41.92420902255015], [-87.74086540263767, 41.92421472136135], [-87.73946140119845, 41.92422532165674], [-87.73842414349474, 41.92423786539702], [-87.73732592381154, 41.9242559684622], [-87.7357395976183, 41.92428187646167], [-87.73476339846363, 41.92429774961868], [-87.7334202988394, 41.924314794318555], [-87.73244479198704, 41.92432554062854], [-87.73116247329892, 41.92434331322544], [-87.72963704099887, 41.924361430227734], [-87.72859902757807, 41.924374154247445], [-87.72762400233131, 41.92438885458066], [-87.72676879384619, 41.92448415933337], [-87.72675496766911, 41.92344801265185], [-87.72665273993553, 41.923100022690555], [-87.7267788015017, 41.92266087295177], [-87.72707099662492, 41.922644187830215], [-87.72726808574568, 41.92241998235167], [-87.72688280000051, 41.92225615285753], [-87.72674991527816, 41.92202102241512], [-87.72661483890779, 41.921845058368945], [-87.72658991599661, 41.92173802016167], [-87.72657391525898, 41.921445022843166], [-87.7265589154669, 41.920744022905076], [-87.72652791583657, 41.91981502234948], [-87.72650104409433, 41.9189108889472], [-87.72648075269652, 41.91830050206145], [-87.72645436703503, 41.917264333068715], [-87.72641864543068, 41.915922422769036], [-87.72637373439652, 41.91439424757358]]]}, "properties": {"city": "Chicago", "name": "Hermosa", "regionid": "273964", "geo_point_2d": [41.916953750844634, -87.7362713025328], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.628208284597, 41.89701623767707], [-87.6282, 41.89666000000012], [-87.62770318645191, 41.896671040301065], [-87.62657021901073, 41.89669213573277], [-87.62519406609873, 41.89670487230838], [-87.62431, 41.89671], [-87.62435719896705, 41.896043953517754], [-87.62435, 41.895235545987596], [-87.62433162932014, 41.89442786808464], [-87.62428988668535, 41.893622164304055], [-87.62427421190212, 41.89296725973135], [-87.62426209850781, 41.89216539627696], [-87.6241430162682, 41.89134937416779], [-87.62409541624118, 41.890540380504966], [-87.62423794561916, 41.88989438066611], [-87.62433402268671, 41.88939470244686], [-87.6244078330344, 41.88909850908387], [-87.62441091564342, 41.88909902287891], [-87.62454991485635, 41.88912502227888], [-87.62478991602694, 41.889191020824526], [-87.62500491784374, 41.889212156690924], [-87.62504591883442, 41.88920102308409], [-87.62511291382896, 41.889197022000445], [-87.62519191567198, 41.889178021124636], [-87.62556791591715, 41.88902602221181], [-87.62571191536063, 41.88895702082977], [-87.6260179159762, 41.88874202261027], [-87.6269059191385, 41.888122021909375], [-87.62698791572393, 41.88808602294787], [-87.62738991716753, 41.8879210225364], [-87.62781791711181, 41.887832022930496], [-87.62791591533409, 41.887830021939024], [-87.62808891611418, 41.8878110228617], [-87.62834391617764, 41.88777302290882], [-87.62954191564341, 41.88772102231033], [-87.6297389157312, 41.887717022125884], [-87.6310162774779, 41.8877429891115], [-87.63103484700069, 41.88825222152499], [-87.63103999999987, 41.88879993265291], [-87.63106201155229, 41.889532704694446], [-87.63108063730708, 41.890448284991884], [-87.63110456598763, 41.891179844997524], [-87.63111, 41.891958161585954], [-87.63115135748899, 41.89287429956388], [-87.63117066490003, 41.89365192845592], [-87.63120044904991, 41.89442875647401], [-87.63122652045193, 41.89534467706759], [-87.63123, 41.89616913349695], [-87.63123, 41.89663], [-87.63205057943567, 41.896618193101645], [-87.63388024415309, 41.89660000000012], [-87.63516288435399, 41.896573526956466], [-87.63653540850699, 41.89656057652498], [-87.63767937914288, 41.8965426708038], [-87.63866999999986, 41.89653], [-87.63868019766409, 41.89695320306364], [-87.63870396600527, 41.89776830026625], [-87.63873033044798, 41.89830040911], [-87.6387455090483, 41.898778535021314], [-87.63875722988887, 41.899060982833205], [-87.63877953748339, 41.89949598092844], [-87.63880514990696, 41.89999542318568], [-87.6388179561186, 41.900245144314255], [-87.63882999999987, 41.90048], [-87.63887364322643, 41.90134201034257], [-87.63888, 41.901526425042256], [-87.63889158868157, 41.90174217989287], [-87.63891819288966, 41.90197084266221], [-87.63895626934394, 41.90278639103477], [-87.63895626934394, 41.90377233045945], [-87.63822952384396, 41.90378259088512], [-87.63563826512801, 41.90383450590851], [-87.63377482094442, 41.90385439957842], [-87.63215632027983, 41.90388197505081], [-87.63141, 41.90389], [-87.63061825602814, 41.90389882317193], [-87.62940416111162, 41.903918841232745], [-87.62869, 41.90393], [-87.62867154094015, 41.90357466309902], [-87.62845232046027, 41.90271072831868], [-87.62833333271368, 41.90219667425732], [-87.6283527690709, 41.90163137934049], [-87.62833437344818, 41.90122746896494], [-87.62829989859641, 41.900256653681666], [-87.62826916937992, 41.899367573261266], [-87.62824622313518, 41.898720470919685], [-87.62824, 41.89839300386861], [-87.6283514242391, 41.897999231492136], [-87.628208284597, 41.89701623767707]]]}, "properties": {"city": "Chicago", "name": "Near North", "regionid": "269597", "geo_point_2d": [41.896849160395014, -87.63108242930801], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.62761784816306, 41.87228007613114], [-87.62760689186854, 41.87125071405947], [-87.62757257593843, 41.87006826087166], [-87.62753264596844, 41.868462082861505], [-87.62751211800625, 41.86746095093961], [-87.62748376679129, 41.86602320995735], [-87.62743765987736, 41.864839527050094], [-87.62736004610505, 41.86240120908372], [-87.62730582904815, 41.86080143975586], [-87.63025999999975, 41.86065840802345], [-87.63024999999973, 41.861703575993936], [-87.63027922321389, 41.86288898492104], [-87.63030623876315, 41.86379399081846], [-87.63033211110495, 41.86448978893962], [-87.63036180978737, 41.865255092947876], [-87.63040521686844, 41.86637366203803], [-87.63043833061396, 41.867440044891524], [-87.63044764014532, 41.86790605787279], [-87.63048634814604, 41.869159436037606], [-87.63052912778764, 41.87006429618051], [-87.63058503195843, 41.87152598921734], [-87.6306199999999, 41.87223000000016], [-87.63010074953718, 41.872241465778885], [-87.62940420815606, 41.87227408818251], [-87.6289151237891, 41.872284605416], [-87.62828829713443, 41.87228658540923], [-87.62761784816306, 41.87228007613114]]]}, "properties": {"city": "Chicago", "name": "Dearborn Park", "regionid": "403344", "geo_point_2d": [41.8665216234067, -87.62894556876412], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.60444210101703, 41.72202006652601], [-87.60444004185958, 41.72199288406111], [-87.60389891799588, 41.722009023294355], [-87.60382791472304, 41.72202601958113], [-87.60346791611458, 41.72200602225662], [-87.60338891517104, 41.722013022579254], [-87.60311695119606, 41.72202510856808], [-87.60324291563545, 41.72140002220097], [-87.60345291542323, 41.720460022037756], [-87.60370191521007, 41.71949202238247], [-87.60392191545867, 41.71850502185112], [-87.6041589155917, 41.71743902217494], [-87.6043649151951, 41.71653102258735], [-87.60457691507507, 41.7156380227919], [-87.60476991497842, 41.714741022812184], [-87.60496791601167, 41.71386702299317], [-87.60503691559506, 41.713565022561944], [-87.60509491557065, 41.71331102254439], [-87.6051539155925, 41.713054022388825], [-87.60520791538363, 41.712822021587066], [-87.6053289155654, 41.71230602208278], [-87.60563191514355, 41.71097202264315], [-87.60584191493133, 41.71012002293903], [-87.60614691550093, 41.708783022461866], [-87.60623991619066, 41.70837302244801], [-87.60645291521733, 41.70745402235349], [-87.60645991553996, 41.707425022815265], [-87.60646591581651, 41.70739802247001], [-87.60666191585831, 41.7064920220753], [-87.60685991509291, 41.7056030224643], [-87.6070559160339, 41.70463602285503], [-87.60723691538406, 41.70373702278297], [-87.6074309153337, 41.7028270222038], [-87.60762991551368, 41.70193202231617], [-87.60792591566837, 41.70066602241384], [-87.60805041036338, 41.70007043756494], [-87.6087192288335, 41.700054399098825], [-87.6093683585727, 41.70004], [-87.61020479307355, 41.70003455621034], [-87.61187662522947, 41.700007573996366], [-87.61299142680959, 41.69999460985887], [-87.61382724607226, 41.699985366998874], [-87.61531327738948, 41.699961037354804], [-87.61652137352878, 41.69994548054491], [-87.61772983240158, 41.699931150288386], [-87.6190301298926, 41.69990966450181], [-87.62014468797254, 41.69989137600034], [-87.62085, 41.69988], [-87.62078214386248, 41.70112998592466], [-87.6207445138059, 41.70197078525099], [-87.62065473320713, 41.70298477789489], [-87.6205287493128, 41.70447650797124], [-87.62047241489547, 41.70512998721201], [-87.62040203450027, 41.70577296194804], [-87.62021732437752, 41.70690380037663], [-87.62006679346791, 41.70783929353565], [-87.61995599883346, 41.708562777665406], [-87.61987144514747, 41.70939957454831], [-87.6199612174886, 41.70994757564064], [-87.6198586524817, 41.71033070919982], [-87.61986498356261, 41.710609276757296], [-87.6199100851552, 41.711172095245644], [-87.6199795054391, 41.71162599710183], [-87.62007587274871, 41.71238077703585], [-87.62016728129443, 41.7131266002847], [-87.62039682466026, 41.71497554524423], [-87.62042716063398, 41.715812104091675], [-87.62044760964802, 41.71683267536477], [-87.62049655921241, 41.717854786218055], [-87.62052906072876, 41.7185998744941], [-87.6205559675226, 41.71961949924061], [-87.62057114476323, 41.72054853435913], [-87.62058, 41.72138397364302], [-87.62058226695405, 41.72178765356451], [-87.61990397055536, 41.7217958778693], [-87.61832321376392, 41.72180529222074], [-87.61711623084142, 41.72183022948606], [-87.61600166657674, 41.72184668776489], [-87.6147939622313, 41.72184566636681], [-87.61408, 41.72184], [-87.61330499371535, 41.72195643637091], [-87.61200595913786, 41.72196587393497], [-87.61089071974733, 41.721995878041106], [-87.60977713368557, 41.72201], [-87.60875492878297, 41.72201865665921], [-87.607269161743, 41.72205096995507], [-87.60568990353407, 41.72205957652057], [-87.60444210101703, 41.72202006652601]]]}, "properties": {"city": "Chicago", "name": "Rosemoor", "regionid": "403379", "geo_point_2d": [41.71155743376761, -87.61291560873518], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.64201372299992, 41.89245144100004], [-87.6414322879999, 41.89188118800006], [-87.6410395709999, 41.89142017200003], [-87.64097127199993, 41.89107867900008], [-87.6405614799999, 41.89054936400002], [-87.6396906719999, 41.88957610800007], [-87.6393491779999, 41.88920046500004], [-87.6393491779999, 41.88889312100007], [-87.63924672999988, 41.88855162700007], [-87.63899060999994, 41.88812476100008], [-87.63880278899987, 41.88713443000005], [-87.63864154499991, 41.886841258000054], [-87.63861496799989, 41.886792936000056], [-87.63837592199988, 41.88636606900007], [-87.6379661299999, 41.885853829000034], [-87.63788010299992, 41.88570635500007], [-87.63813705399988, 41.885709142000046], [-87.63839794299989, 41.88570620700004], [-87.6393744189999, 41.88571680100005], [-87.64047274299989, 41.88573779700004], [-87.6417546449999, 41.88571601100005], [-87.6433413689999, 41.88571], [-87.64480596599991, 41.88570441900002], [-87.64559825399992, 41.885689131000035], [-87.64599041099989, 41.88570457100007], [-87.64610694299994, 41.88570915900004], [-87.6462941059999, 41.88637478500004], [-87.6467201649999, 41.887141070000034], [-87.64711547599988, 41.88767418300005], [-87.64727978399989, 41.88788543600002], [-87.64772421799994, 41.888470599000065], [-87.64799133299992, 41.88875406800007], [-87.64826635199994, 41.88903978300004], [-87.64824091499989, 41.88904002200006], [-87.64822433599993, 41.88904014000006], [-87.6487147369999, 41.88956682700005], [-87.65110313099989, 41.891142214000055], [-87.65290537599991, 41.892363853000056], [-87.65192000099994, 41.89253679200004], [-87.65085873799987, 41.89265836800006], [-87.64980146999993, 41.89268033700005], [-87.64818735899989, 41.89235899000004], [-87.64664811599992, 41.89238044900008], [-87.6451910739999, 41.892398507000046], [-87.64300620399989, 41.89243816100008], [-87.64241405799991, 41.89244608400007], [-87.64201372299992, 41.89245144100004]]]}, "properties": {"city": "Chicago", "name": "Fulton River District", "regionid": "403313", "geo_point_2d": [41.88929815903607, -87.64445398066268], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.64319832703174, 41.90371243982024], [-87.64318009181912, 41.90316920200266], [-87.6431374761561, 41.902118440988815], [-87.64308542594681, 41.90082492985826], [-87.64305421189617, 41.9000134538318], [-87.64300636480277, 41.89863839423334], [-87.64294315580251, 41.897105219993335], [-87.64291, 41.89648], [-87.64383866999991, 41.89646511600005], [-87.64383799999985, 41.896454], [-87.6441414479999, 41.89645780500007], [-87.64447599999987, 41.896462000000156], [-87.64447610799981, 41.896446241], [-87.64657039699989, 41.89640936800006], [-87.64810570599991, 41.89640000000018], [-87.64932206999993, 41.89637532500017], [-87.6503761029999, 41.89635597400008], [-87.6514269129999, 41.896325894000086], [-87.6523156499999, 41.89632000000013], [-87.65320920399992, 41.89630170900013], [-87.65409836699988, 41.89627848100016], [-87.65652593599991, 41.89623290200002], [-87.65842038992388, 41.89620617466253], [-87.65854381884847, 41.89629381884866], [-87.65911896417155, 41.89741980105569], [-87.6596222288209, 41.898465014222545], [-87.66039822482178, 41.89973736306856], [-87.66080178409159, 41.901150464031026], [-87.66104795590634, 41.9021996687989], [-87.66114034005903, 41.903440000000195], [-87.66115177991884, 41.904466552168095], [-87.66111524121249, 41.905193170082015], [-87.66118832705811, 41.90625831622983], [-87.66181282579626, 41.90703923819479], [-87.66295431512724, 41.90839534577356], [-87.66347258791602, 41.90885490922752], [-87.66401379109702, 41.90936379109749], [-87.66459517416885, 41.9097703849313], [-87.66507792402822, 41.91061533214571], [-87.66510573699989, 41.91066401300009], [-87.66381157299983, 41.91068072000014], [-87.66259557799985, 41.9107071670001], [-87.66089673299989, 41.9107233750002], [-87.66008710299985, 41.910734882], [-87.65903490199986, 41.91075245900012], [-87.65711062499987, 41.91077733900005], [-87.6567575479999, 41.910780144], [-87.65639799999988, 41.910783], [-87.65599391599986, 41.910790022000114], [-87.65548891599991, 41.910799022000155], [-87.65409091599989, 41.91081502200006], [-87.65308991599993, 41.91083502200011], [-87.65109491499977, 41.910874022000144], [-87.64947391499987, 41.910898022000175], [-87.64826105699989, 41.91090986400009], [-87.64824491499986, 41.9109100220001], [-87.64743091599976, 41.91092202300012], [-87.64716791599994, 41.910931022000064], [-87.64693691499984, 41.91093702200003], [-87.64660691499988, 41.91094502300007], [-87.64605191599988, 41.91094802200005], [-87.64584691499981, 41.91094902200013], [-87.64546091599992, 41.910954022], [-87.64433091599987, 41.910968022000105], [-87.6437389149999, 41.91098302300019], [-87.64357191499987, 41.911000023], [-87.64341855396204, 41.911029704194384], [-87.64340343905083, 41.91045864901329], [-87.6433740299483, 41.909812799870586], [-87.64332890880193, 41.90802730224031], [-87.64325712510589, 41.90551738909738], [-87.64322360484749, 41.90446546781291], [-87.64319832703174, 41.90371243982024]]]}, "properties": {"city": "Chicago", "name": "Goose Island", "regionid": "403298", "geo_point_2d": [41.90389627469062, -87.65229588274904], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.68569472978758, 41.89575683911626], [-87.68681999999988, 41.89575000000014], [-87.68683908341643, 41.896335224772955], [-87.68686438957201, 41.89731033595356], [-87.68688, 41.89810255276589], [-87.6869042272004, 41.89889630081247], [-87.68693205180347, 41.89993438296222], [-87.68695085304154, 41.90078838687346], [-87.68697703218982, 41.901705288297045], [-87.68699, 41.90255763899036], [-87.68705, 41.90302], [-87.68557423883502, 41.9030395152234], [-87.68356051659272, 41.90308013080576], [-87.68069256770679, 41.90312925066186], [-87.67855631564532, 41.90316387264643], [-87.6772520943022, 41.90318166701809], [-87.67725, 41.90274072472563], [-87.67723355134109, 41.901888129505174], [-87.67717936050457, 41.900969826621925], [-87.67715274521582, 41.899872362481645], [-87.67711759153214, 41.89926250563334], [-87.67710999999986, 41.898163581344264], [-87.67710009806173, 41.89743294185082], [-87.67706580045628, 41.89639454732419], [-87.67705, 41.89591], [-87.67861554001854, 41.89588433540952], [-87.68081287243581, 41.895839156783346], [-87.68331422144745, 41.89580186541788], [-87.68569472978758, 41.89575683911626]]]}, "properties": {"city": "Chicago", "name": "Ukrainian Village", "regionid": "403312", "geo_point_2d": [41.89946241233393, -87.68203311340594], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.71694893293171, 41.75912897352687], [-87.71641892199037, 41.758983020756105], [-87.71691256614747, 41.75921442440809], [-87.71613775885038, 41.75889412457219], [-87.71614108184524, 41.75884592630725], [-87.71525591604406, 41.75841202231398], [-87.7146289150205, 41.75811002278194], [-87.71407291547109, 41.75785602186519], [-87.71353791509108, 41.75757202316295], [-87.71295291600369, 41.75736502351335], [-87.71268491534096, 41.75729402293851], [-87.71254391603594, 41.75726902178607], [-87.71225594952477, 41.75721597077964], [-87.71182191602912, 41.75711102259678], [-87.71151691545953, 41.75707102255155], [-87.71097491565608, 41.75705702190619], [-87.70963091575557, 41.757073022643645], [-87.70791091560979, 41.757050022482844], [-87.70449391547902, 41.75710702241242], [-87.70302591615781, 41.757117021974125], [-87.70257991538382, 41.757132022665495], [-87.70252391550041, 41.757134022757576], [-87.70247291584748, 41.75713502190457], [-87.7015749158215, 41.75716502238808], [-87.70131491552758, 41.75717702204198], [-87.70013991532342, 41.757229022640296], [-87.6972419157345, 41.75725902222435], [-87.69578391507552, 41.757293021993135], [-87.69316191561879, 41.75732702176183], [-87.69276591535082, 41.75733202199219], [-87.69265091544577, 41.75733302203828], [-87.68992191555321, 41.75736102242988], [-87.68881991601626, 41.757448021943524], [-87.6880039161728, 41.75740702275166], [-87.6878639151153, 41.757415020422364], [-87.68761291433684, 41.757363023421256], [-87.6866753838882, 41.75736582416951], [-87.68769990206818, 41.75691292793149], [-87.68911545394042, 41.75631215797691], [-87.69143883360641, 41.755292130833446], [-87.69337269825989, 41.75441935424546], [-87.69442767827746, 41.753934144220416], [-87.69620736583897, 41.753128777857754], [-87.69738321612961, 41.752594771141304], [-87.69906513453736, 41.751873355568954], [-87.70042897952754, 41.751252454243314], [-87.7016741546089, 41.750683400649294], [-87.70233, 41.75038], [-87.70230694708768, 41.748883568699235], [-87.70229175636405, 41.7483291072916], [-87.70225727224238, 41.74730258465662], [-87.70222537448878, 41.74646412942107], [-87.70217915826032, 41.74525811476982], [-87.702147046749, 41.74442321547539], [-87.70211686083506, 41.743402362234704], [-87.70209890970729, 41.74275253140333], [-87.70209, 41.74243], [-87.70208489713563, 41.74219526824115], [-87.70205010150642, 41.740893146698824], [-87.7020225969406, 41.740057908221594], [-87.7019956994393, 41.739036474768575], [-87.70204, 41.73878], [-87.70351859701447, 41.738755560379936], [-87.70602638597681, 41.73870790192675], [-87.70788366857681, 41.73867950223936], [-87.71057750185787, 41.73864805394551], [-87.71203, 41.73862], [-87.71214177915152, 41.7428250351872], [-87.71268164123944, 41.74240451103454], [-87.71375377838002, 41.741566173441605], [-87.71480929519123, 41.74074671756423], [-87.71589386053809, 41.73989799000425], [-87.71715391431762, 41.73891696440679], [-87.71840928034838, 41.73793121699038], [-87.71968902667447, 41.736907169922894], [-87.72109718543598, 41.735786601173096], [-87.72155100027733, 41.735033000425865], [-87.72158196701173, 41.73503644114665], [-87.72150580784752, 41.736253263846585], [-87.72153972891499, 41.73708883281543], [-87.72158409848771, 41.738297053121194], [-87.72160521981311, 41.73894674093808], [-87.72163871281712, 41.73996762212308], [-87.72166070208203, 41.74080229577888], [-87.72168652368555, 41.741731218462895], [-87.72172832306647, 41.74275602669761], [-87.72174874982721, 41.74340374222398], [-87.72177627811016, 41.744425473489265], [-87.72180382008872, 41.74526078257232], [-87.72181489167708, 41.74609644249518], [-87.7218776061181, 41.7485096606015], [-87.7219444088499, 41.750555515082176], [-87.72191, 41.75325], [-87.72196294239097, 41.75328775604791], [-87.72196491593925, 41.75331202284378], [-87.72199691561575, 41.75373802269571], [-87.7220007880964, 41.75380015865342], [-87.7220430409432, 41.753799878964266], [-87.72213696793465, 41.754877148944054], [-87.722058863615, 41.75479289236252], [-87.72206391510687, 41.75487602299282], [-87.72206678664215, 41.75500515394524], [-87.72214404559907, 41.755004888645274], [-87.72216191602713, 41.75558202226217], [-87.72216391611931, 41.755742022443094], [-87.72217191558867, 41.7563020221768], [-87.72217091554258, 41.7568030227882], [-87.72217091554258, 41.757018022806506], [-87.72216891545048, 41.75783402264968], [-87.72214991547395, 41.75893602218681], [-87.7222219151955, 41.7608410227664], [-87.72223491579477, 41.76120302326576], [-87.72224302228359, 41.76140716127243], [-87.72217291563476, 41.76138702185492], [-87.72210291510599, 41.761367021832214], [-87.72172291557577, 41.7612570230568], [-87.72146591542001, 41.76116202227498], [-87.72123191542528, 41.761051022554355], [-87.71918093959175, 41.759975939725464], [-87.7189918796181, 41.759961152173396], [-87.71744091514171, 41.75933102240841], [-87.71694893293171, 41.75912897352687]]]}, "properties": {"city": "Chicago", "name": "Ashburn", "regionid": "269570", "geo_point_2d": [41.74934720073128, -87.7106224667427], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.59065760765178, 41.72223858723394], [-87.59061182883887, 41.72157328252164], [-87.59056967214696, 41.72044786919751], [-87.5904977472918, 41.71868600359796], [-87.5904643700227, 41.71785194064957], [-87.59041325650901, 41.716460173576955], [-87.59036369193512, 41.71506829676431], [-87.59115921078815, 41.7149736103925], [-87.59199534512521, 41.71488070657718], [-87.59301461505366, 41.71495696060854], [-87.59394490932515, 41.71492936290158], [-87.5953401503445, 41.71489704671576], [-87.59654757096975, 41.71487704048398], [-87.59747635862455, 41.71486169903907], [-87.5988699243469, 41.71484433396382], [-87.60016999493035, 41.71482710752186], [-87.6012861027646, 41.71480284743076], [-87.60240000933267, 41.71477048750127], [-87.60444323419468, 41.71472789756277], [-87.6047692631125, 41.714744052470486], [-87.60457691507507, 41.7156380227919], [-87.6043649151951, 41.71653102258735], [-87.6041589155917, 41.71743902217494], [-87.60392191545867, 41.71850502185112], [-87.60370191521007, 41.71949202238247], [-87.60345291542323, 41.720460022037756], [-87.60324291563545, 41.72140002220097], [-87.60311695119606, 41.72202510856808], [-87.60294591633387, 41.72202502223323], [-87.60138291533147, 41.72208502230099], [-87.59839991542162, 41.722146022414755], [-87.5971439150809, 41.72215402188437], [-87.59595591607615, 41.72216402234519], [-87.59476591518045, 41.72217802209135], [-87.59356491557651, 41.72219202183733], [-87.59163591568904, 41.72221802213649], [-87.59065760765178, 41.72223858723394]]]}, "properties": {"city": "Chicago", "name": "Cottage Grove Heights", "regionid": "403383", "geo_point_2d": [41.718415638600064, -87.59723319114183], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.7419996012905, 41.79303456729892], [-87.74187, 41.79298], [-87.7397267335174, 41.79301400989444], [-87.73799950018345, 41.792992834730775], [-87.73802091502453, 41.79292002211404], [-87.7380559157387, 41.79275102241767], [-87.73807291562304, 41.792592022282825], [-87.73807291562304, 41.79221202275267], [-87.73807391566913, 41.791945022136034], [-87.7380559157387, 41.79131502277269], [-87.73804191509332, 41.790884021790944], [-87.73797091541769, 41.78952102191371], [-87.73794891530297, 41.788967022456646], [-87.73791991666411, 41.788622022741], [-87.73793491555698, 41.788222022288956], [-87.7379269142889, 41.78803302256997], [-87.737849915236, 41.787623022556126], [-87.73778191659805, 41.7874400222143], [-87.73763591616311, 41.786715022069146], [-87.73756591653374, 41.78629302240148], [-87.73755491602658, 41.78572002206853], [-87.73755391508121, 41.785664023084415], [-87.73760091544916, 41.78566202209296], [-87.73789491551167, 41.78563602089442], [-87.73807491571512, 41.78559002237122], [-87.73824091527331, 41.78552702216509], [-87.73839891536207, 41.785439021705905], [-87.73848391568289, 41.785375021453696], [-87.73857191524279, 41.78529102207798], [-87.73862491318927, 41.78522402348633], [-87.73867391634742, 41.78512502251996], [-87.7387209158159, 41.785056022037296], [-87.73892291523497, 41.784761021928695], [-87.73901887559339, 41.78460798789623], [-87.73920391739625, 41.78443902417278], [-87.7394159136789, 41.78432702440571], [-87.73965090652506, 41.78425996915663], [-87.73988691560525, 41.784243022332326], [-87.74020291578258, 41.784226022447974], [-87.74059591501289, 41.78422702249416], [-87.74115591564596, 41.784230022632435], [-87.7414139374314, 41.784215889786786], [-87.74147281841425, 41.78423431390759], [-87.74191121901752, 41.78353314091369], [-87.74212650005283, 41.78318215990694], [-87.7422974223306, 41.782755594051785], [-87.74227155737765, 41.78094175411975], [-87.7422179341844, 41.7789835648261], [-87.74208999999989, 41.77832], [-87.74265483958915, 41.77831342631368], [-87.74403778842975, 41.778289458042494], [-87.74533938637366, 41.77826804284639], [-87.74647743529897, 41.778243947885244], [-87.74753562542162, 41.77822173957645], [-87.7489254549322, 41.77819271962353], [-87.75014569761466, 41.77816600828444], [-87.75136317696808, 41.778144295703015], [-87.75250306022254, 41.77812955810689], [-87.75437404725477, 41.778101680474514], [-87.7561635399503, 41.77808439345167], [-87.75836042147074, 41.77806722837973], [-87.76080144646336, 41.778033579814384], [-87.76164, 41.77802], [-87.76164483043509, 41.77849541741685], [-87.76167358026986, 41.779470556072006], [-87.76171579157595, 41.78036729444409], [-87.76175569204075, 41.781183037279064], [-87.76179501354939, 41.7822415067483], [-87.76181458092191, 41.78297332648405], [-87.7618699078073, 41.78452108998008], [-87.76189721524484, 41.7854140537113], [-87.7619182448978, 41.785916321729324], [-87.76191893279982, 41.78591706079795], [-87.76195381566778, 41.78679920736352], [-87.76199108439658, 41.787774218425014], [-87.7620221137816, 41.788589847975665], [-87.76208807255615, 41.79029615144369], [-87.76216552412406, 41.792001329216276], [-87.7621999999999, 41.79271], [-87.75909098713248, 41.79274104085969], [-87.75673246824606, 41.792750000000126], [-87.75429221575666, 41.79277471369326], [-87.75193188458934, 41.79283139590571], [-87.74835067857583, 41.7928723396358], [-87.74639824029828, 41.79289254425467], [-87.74444567261963, 41.79291110818453], [-87.74194999999986, 41.79298], [-87.7419996012905, 41.79303456729892]]]}, "properties": {"city": "Chicago", "name": "Chicago Midway Airport", "regionid": "403332", "geo_point_2d": [41.78587574924793, -87.75071677388192], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.60110195499992, 41.659098118000074], [-87.59691924699989, 41.65907], [-87.59496882799992, 41.65908335100004], [-87.59357213399993, 41.65908158100006], [-87.59355813199993, 41.65907295300007], [-87.59315173899992, 41.65882253200004], [-87.59285067299992, 41.65838325600003], [-87.59244300499992, 41.65786222800005], [-87.59202190599989, 41.657435968000065], [-87.59170291499991, 41.657113908000035], [-87.59141600499991, 41.65673696800006], [-87.59122010999992, 41.656431372000036], [-87.59085, 41.65597], [-87.59114223099992, 41.65546140100007], [-87.5915576229999, 41.65535895700002], [-87.59208950799987, 41.655275500000066], [-87.5928289049999, 41.65534051700006], [-87.59273857599992, 41.651613208000065], [-87.59273857599992, 41.65021089200006], [-87.59271147899993, 41.64962478800003], [-87.59379289299991, 41.64955269400007], [-87.59498244799993, 41.64930036400004], [-87.59653247499993, 41.648975940000064], [-87.5982266899999, 41.64868756300006], [-87.5997406689999, 41.64861546900004], [-87.6002092819999, 41.64872361000003], [-87.60172326199992, 41.64912012900003], [-87.6032732879999, 41.649408506000036], [-87.60475926599989, 41.64955002700008], [-87.60470320199992, 41.65019424900002], [-87.6046268459999, 41.651518255000035], [-87.60471721899992, 41.65183611100007], [-87.60519999999988, 41.65182], [-87.6050627199999, 41.65241728000006], [-87.60492142399991, 41.65250625100003], [-87.60479397399988, 41.652534131000024], [-87.60474, 41.6529], [-87.60478447599989, 41.65308492500007], [-87.6048791239999, 41.65347846400004], [-87.60498064499988, 41.654031853000035], [-87.6050558509999, 41.654591074000045], [-87.60511526899987, 41.65533010600006], [-87.60510177299993, 41.65570123000002], [-87.60510865899992, 41.656350988000035], [-87.60512132099988, 41.656908123000036], [-87.60513, 41.65729], [-87.60631, 41.65734], [-87.60629380299989, 41.65830369800005], [-87.6062299999999, 41.65913841700006], [-87.60388703299989, 41.65918], [-87.60110195499992, 41.659098118000074]]]}, "properties": {"city": "Chicago", "name": "Altgeld Gardens", "regionid": "137228", "geo_point_2d": [41.65425647941322, -87.59882486249272], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.68414827918792, 41.9719544687027], [-87.68412790989359, 41.97133660011085], [-87.68410972924148, 41.97078512032973], [-87.68408642556287, 41.970098603730946], [-87.68407214123937, 41.96968673906965], [-87.68404833403348, 41.969000297967376], [-87.68406445048005, 41.96810603840746], [-87.68403670228057, 41.96721415245401], [-87.68400108248234, 41.96631922702496], [-87.6839999999999, 41.96593], [-87.68503916005142, 41.96591529490485], [-87.68600016856286, 41.96590169572795], [-87.68642140468005, 41.965793534477335], [-87.68668, 41.9657], [-87.68685, 41.96596], [-87.68704804980739, 41.965879542265874], [-87.68743723597848, 41.965765892523535], [-87.68806440536639, 41.96574532488822], [-87.68854528340191, 41.965735306595796], [-87.69039932007695, 41.96571612707445], [-87.69239181965933, 41.965710000000136], [-87.69386999999985, 41.96569], [-87.69375966113532, 41.966183056769445], [-87.69379052086155, 41.96721343877352], [-87.6938073124242, 41.9679690590921], [-87.69382873268097, 41.96920573620242], [-87.69384616467104, 41.97009856962134], [-87.69387478583666, 41.971197038834276], [-87.6939, 41.97184], [-87.69307857430927, 41.9718526763223], [-87.69246018147224, 41.971862219421865], [-87.69177307832001, 41.9718728228655], [-87.69122339579819, 41.97188130562034], [-87.69066, 41.97189], [-87.68900032837338, 41.97175214366083], [-87.68902, 41.97224], [-87.6885418675868, 41.972250835373906], [-87.68812979795523, 41.9722617752756], [-87.68716840334704, 41.972275385096246], [-87.68600010349961, 41.97228991701636], [-87.68469449228402, 41.97230561891579], [-87.68416, 41.97231], [-87.68414827918792, 41.9719544687027]]]}, "properties": {"city": "Chicago", "name": "Lincoln Square", "regionid": "269591", "geo_point_2d": [41.968922468346506, -87.68889798656545], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.72196294199993, 41.75328775600007], [-87.7219099999999, 41.75325], [-87.72194440899993, 41.75055551500008], [-87.72187760599988, 41.748509661000064], [-87.72181489199993, 41.74609644200006], [-87.7218038199999, 41.745260783000035], [-87.72177627799991, 41.74442547300002], [-87.72174874999988, 41.74340374200005], [-87.7217283229999, 41.74275602700003], [-87.72168652399992, 41.74173121800003], [-87.72166070199994, 41.740802296000076], [-87.72163871299992, 41.739967622000044], [-87.72160521999989, 41.73894674100005], [-87.72158409799992, 41.73829705300004], [-87.7215397289999, 41.73708883300008], [-87.72150580799988, 41.73625326400007], [-87.72158307199992, 41.73503528500003], [-87.72245210499993, 41.735131844000065], [-87.7228520699999, 41.73493242500006], [-87.7238520459999, 41.73493194600007], [-87.7250520459999, 41.73493194600007], [-87.72563704599989, 41.73493194600007], [-87.72635204599992, 41.73493194600007], [-87.72675198699993, 41.734831318000026], [-87.72675904199988, 41.73483100900005], [-87.72745204599993, 41.73483194500004], [-87.72865308799992, 41.73493186800003], [-87.72985301699991, 41.73483203000006], [-87.73115308499989, 41.73493187400003], [-87.73155300199993, 41.734832203000046], [-87.73235304599987, 41.734831946000035], [-87.7336530459999, 41.734831946000035], [-87.73475301499988, 41.73473203800005], [-87.73585304499991, 41.73473094700005], [-87.73645304599992, 41.73473094600006], [-87.73673669399992, 41.734660280000064], [-87.73693470599993, 41.73461077700005], [-87.73698155499993, 41.73459933800006], [-87.73725296699988, 41.73453164400006], [-87.73783009999993, 41.734583882000024], [-87.7379506049999, 41.73459427100004], [-87.73835309099988, 41.73463074900006], [-87.7395530879999, 41.734730868000035], [-87.74094843799989, 41.73473094600006], [-87.74104873999993, 41.73473135300008], [-87.74105416799992, 41.73473147400005], [-87.7411429239999, 41.73473147400005], [-87.74115296099988, 41.73473155100003], [-87.7411429299999, 41.73478858600003], [-87.74102801299993, 41.73543947400008], [-87.7409360599999, 41.73596071500002], [-87.74085402799992, 41.73643137800007], [-87.74085402799992, 41.736488485000045], [-87.74085402799992, 41.73727425900005], [-87.74085402799992, 41.737330946000036], [-87.7408871239999, 41.737700760000045], [-87.74095402799989, 41.73833159400004], [-87.74095402799989, 41.73838848500007], [-87.74095402799989, 41.73867425900004], [-87.74095402799989, 41.73873094600008], [-87.74099931599993, 41.73890850800005], [-87.74100012699994, 41.738917128000075], [-87.74100177299992, 41.73892651200002], [-87.74105333999992, 41.73913161700005], [-87.74105301299993, 41.739188587000044], [-87.74105301299993, 41.74007436100003], [-87.74105301299993, 41.74013147400005], [-87.74105301299993, 41.74018858700003], [-87.74105301299993, 41.740343454000076], [-87.7410530159999, 41.740400558000026], [-87.74105400099988, 41.74045765400006], [-87.74105402799994, 41.74197425900006], [-87.74105403299988, 41.74203113600003], [-87.74105556099991, 41.74208819600004], [-87.74108980599992, 41.74350735800004], [-87.74109077199994, 41.74354343100003], [-87.7410913339999, 41.743564436000035], [-87.7410923, 41.74360050900008], [-87.74109286199987, 41.743621502000046], [-87.74109382799989, 41.743657575000036], [-87.7410947949999, 41.74369367700006], [-87.7410963229999, 41.74375075500005], [-87.74109787799992, 41.74380782000008], [-87.74114293299992, 41.74546153000005], [-87.74114448799989, 41.74551860700006], [-87.74114604299989, 41.74557567200008], [-87.74114876799987, 41.74567567800004], [-87.7411489499999, 41.74568236800007], [-87.74115032299991, 41.745732755000056], [-87.74115069399991, 41.74573944000007], [-87.74135032299993, 41.749332755000054], [-87.74137275599989, 41.749422486000064], [-87.74137969599988, 41.74943383300006], [-87.74138162399991, 41.749445918000035], [-87.74145324899989, 41.74973138200004], [-87.74145271299993, 41.74978811000005], [-87.7414482549999, 41.75085136100005], [-87.7414482439999, 41.75090846900007], [-87.7414486889999, 41.75096557700004], [-87.74145233699991, 41.75127437200006], [-87.74145265799987, 41.75133148000002], [-87.74145227999989, 41.75138858100006], [-87.74144923199991, 41.75162644800007], [-87.74144888699992, 41.75166434700008], [-87.74144869799991, 41.75168355200008], [-87.74144860499989, 41.751721448000055], [-87.74144853999988, 41.75174051900005], [-87.74144997899991, 41.751778676000065], [-87.7414491179999, 41.75205126000008], [-87.74144902799989, 41.752107946000066], [-87.74146934699989, 41.752392862000065], [-87.7414721359999, 41.752446092000056], [-87.74147449199987, 41.75250308400006], [-87.74147888999993, 41.752563865000056], [-87.74148131599989, 41.75259166300008], [-87.74147886699987, 41.75262098000006], [-87.74146001699992, 41.75288240100007], [-87.74145864199991, 41.752939475000055], [-87.7414553929999, 41.753074316000024], [-87.7414540279999, 41.75313137800003], [-87.7414540279999, 41.75314461600003], [-87.7414540279999, 41.75318848500006], [-87.7414540279999, 41.753201735000054], [-87.74138200499992, 41.75319800100004], [-87.74133791499989, 41.75319602200005], [-87.7412909149999, 41.75319302300004], [-87.74014591599988, 41.753194023000034], [-87.7394429159999, 41.75314202200007], [-87.73893291499989, 41.75311502300007], [-87.73843791599987, 41.75304802100003], [-87.73821381999991, 41.75298486300004], [-87.73820003399992, 41.753050169000055], [-87.73724891499987, 41.75303202300005], [-87.7366957769999, 41.75304590100006], [-87.73670405199992, 41.753117144000036], [-87.73352291499991, 41.75311702200007], [-87.73224391599989, 41.75313602200003], [-87.7316999159999, 41.75314702200006], [-87.73038991599991, 41.75317502200004], [-87.72996691599988, 41.753182022000026], [-87.7291779159999, 41.75319602200005], [-87.7275949159999, 41.75322202200004], [-87.72678491599991, 41.75324002200005], [-87.72643991499987, 41.753248023000076], [-87.72577591499991, 41.753264022000046], [-87.72534391499993, 41.75326202200005], [-87.7244509149999, 41.75327102200003], [-87.72202778299992, 41.75331215600004], [-87.72202804599992, 41.75324889200003], [-87.72195978199991, 41.75324889200003], [-87.72196294199993, 41.75328775600007]]]}, "properties": {"city": "Chicago", "name": "Scottsdale", "regionid": "403367", "geo_point_2d": [41.744011291050654, -87.73146912400465], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.56444, 41.704570000000125], [-87.56443544604305, 41.70373207193451], [-87.56443, 41.7027300000001], [-87.5631999999999, 41.7027300000001], [-87.56319606625703, 41.698105695881814], [-87.563602736737, 41.69839543364085], [-87.56439371249515, 41.69961349078177], [-87.56486432778365, 41.70038959318723], [-87.56532749385423, 41.70115341091772], [-87.56567321591, 41.70172590065102], [-87.5657677577941, 41.70215964029389], [-87.56585642371465, 41.703548739910104], [-87.5658104597551, 41.70456697738259], [-87.56505389593373, 41.70456551900783], [-87.56444, 41.704570000000125]]]}, "properties": {"city": "Chicago", "name": "Trumbull Park", "regionid": "403384", "geo_point_2d": [41.70171099398419, -87.56443122168251], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.66618775399988, 41.859626022000036], [-87.6661813339999, 41.85922702100003], [-87.6661552059999, 41.85825125000007], [-87.66612615799991, 41.85715324500005], [-87.66609459699991, 41.856298442000075], [-87.66606841799988, 41.85544304100005], [-87.6660342409999, 41.85446754500003], [-87.6660159549999, 41.85379613600008], [-87.666, 41.852697292000066], [-87.66600999999989, 41.85226], [-87.66601047599993, 41.852259824000036], [-87.66601849099992, 41.851414869000045], [-87.6659647959999, 41.84909702700008], [-87.66588921999993, 41.84628686200005], [-87.66585825299987, 41.84504614200005], [-87.66584711199991, 41.84479220800005], [-87.66718257399991, 41.84479220800005], [-87.66982885699989, 41.84396836500007], [-87.67157640299989, 41.84339417200004], [-87.6722005269999, 41.84301969800003], [-87.67349870299994, 41.84242053900004], [-87.67482184499993, 41.841796416000065], [-87.67609505699991, 41.84134704700005], [-87.6769188999999, 41.84099753800006], [-87.6775180589999, 41.840298519000044], [-87.67816714699988, 41.839998940000044], [-87.6806636409999, 41.83924999200006], [-87.68273573099991, 41.83852600900007], [-87.68440838199987, 41.83807664000005], [-87.68508452299993, 41.837864138000036], [-87.68507, 41.838305], [-87.68512556699991, 41.83825737100004], [-87.68513869699987, 41.83872818900004], [-87.68519515099987, 41.83963927000006], [-87.68523199899991, 41.84092156500008], [-87.68526594699993, 41.84177601800006], [-87.6853098419999, 41.842750498000036], [-87.6853689599999, 41.84384957100007], [-87.68541, 41.84452], [-87.68542261099991, 41.84484968700007], [-87.6847257339999, 41.84508642100007], [-87.68355106299991, 41.845535059000035], [-87.683, 41.84574], [-87.68301414599993, 41.84616910800007], [-87.6830412649999, 41.84714438200005], [-87.68306758399989, 41.84805923100003], [-87.6830829859999, 41.84879075600008], [-87.68311537499993, 41.84982913100002], [-87.68314986199991, 41.85092936100005], [-87.6831696549999, 41.85159929100006], [-87.68318, 41.85207], [-87.68429192799992, 41.85204021600003], [-87.68496341899991, 41.85203469300006], [-87.68561238699992, 41.852032423000075], [-87.68561930799987, 41.852453504000025], [-87.68565350799989, 41.85349243800004], [-87.68567512399991, 41.854465301000054], [-87.68570142999992, 41.855320043000056], [-87.68573783499993, 41.85635717300004], [-87.68575650499989, 41.85703031600006], [-87.68576972899989, 41.85740238500004], [-87.6857690649999, 41.857402517000075], [-87.68576937399988, 41.857410155000075], [-87.68508491899989, 41.857579578000056], [-87.68343714699994, 41.85800677800005], [-87.68264377499989, 41.85837295000004], [-87.68156178299992, 41.85869118300007], [-87.68130491899987, 41.85866802300006], [-87.67880991499992, 41.85888202300002], [-87.67603091599992, 41.85933402200003], [-87.67506788699987, 41.859457195000054], [-87.67488191599992, 41.85936602300006], [-87.67471492999991, 41.85929384800004], [-87.67388991199991, 41.859518020000046], [-87.67359991599993, 41.85951902200007], [-87.6724599159999, 41.85953002200005], [-87.67113091499992, 41.859552022000045], [-87.66930591599991, 41.85957202200006], [-87.66869291599991, 41.859578022000036], [-87.66687291499993, 41.85959602300004], [-87.66637091599993, 41.85960502300008], [-87.66623991599992, 41.85962002200005], [-87.66618791599988, 41.859626022000036], [-87.66618775399988, 41.859626022000036]], [[-87.67698264099991, 41.84102145200006], [-87.6769790529999, 41.84098044200003], [-87.6768408109999, 41.841033504000045], [-87.67692991999988, 41.84102598800006], [-87.67698264099991, 41.84102145200006]]]}, "properties": {"city": "Chicago", "name": "Heart of Chicago", "regionid": "403330", "geo_point_2d": [41.850207007705045, -87.67589239709841], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.68611952099988, 42.01949873900008], [-87.68548842499987, 42.01946102800007], [-87.68495904599989, 42.019429946000045], [-87.68461541199989, 42.019429946000045], [-87.68460837299993, 42.019373893000065], [-87.68444891599992, 42.018003023000055], [-87.6844139159999, 42.01765202200004], [-87.68429391499988, 42.01653602200007], [-87.68416791499992, 42.015602023000035], [-87.68406791599992, 42.01515002200005], [-87.68387891599988, 42.01458202300005], [-87.68368491599992, 42.01404302200006], [-87.68322091599991, 42.01275002300007], [-87.68308391599992, 42.012308023000045], [-87.68269191599987, 42.01126202300007], [-87.6822999159999, 42.010159022000025], [-87.68226791599993, 42.010053023000054], [-87.68188391599992, 42.008956022000056], [-87.6818309159999, 42.00882402400003], [-87.68127291599987, 42.00762602200005], [-87.6811559149999, 42.007355022000056], [-87.68071091499989, 42.00643302200006], [-87.6804169149999, 42.00580302300005], [-87.6801199149999, 42.005204022000036], [-87.6797279159999, 42.004338023000045], [-87.6796419159999, 42.00414502200004], [-87.67926291599989, 42.00329602200008], [-87.6783879149999, 42.00146402200004], [-87.6779059159999, 42.00039602300005], [-87.67761191599992, 41.99980002200003], [-87.67699691599992, 41.998402022000036], [-87.67680402599989, 41.99794686500006], [-87.6754539169999, 41.997871022000076], [-87.67513891599991, 41.99787602300005], [-87.67496891299992, 41.997878077000045], [-87.67496791599989, 41.99784202300008], [-87.67492291599989, 41.99668802200005], [-87.6748969149999, 41.995456022000035], [-87.6748479159999, 41.99427902200004], [-87.67475491499991, 41.991917022000045], [-87.67473291599991, 41.99126702300003], [-87.6747209129999, 41.99090202400002], [-87.67465691499989, 41.990728022000035], [-87.6746128709999, 41.99063813000003], [-87.67495591499991, 41.99073102100003], [-87.67523591599988, 41.990777075000025], [-87.67705491499991, 41.99059702300006], [-87.6783839159999, 41.99057602200007], [-87.67979991599991, 41.99055302200008], [-87.6817339159999, 41.990518022000046], [-87.68369291499992, 41.99046802200006], [-87.68562691499993, 41.99044902200006], [-87.68714991499989, 41.99042002200007], [-87.68749591599992, 41.990413022000034], [-87.68836591599988, 41.99039502200008], [-87.68954665499989, 41.99037602700007], [-87.68955, 41.99038], [-87.6912239839999, 41.99036671700008], [-87.69266672599989, 41.99036], [-87.69376635299993, 41.99036], [-87.6950042059999, 41.990351335000064], [-87.69617204299992, 41.99033608100005], [-87.69740886699991, 41.99033], [-87.69850849399987, 41.99033], [-87.69933, 41.99033], [-87.6992961469999, 41.98927490200003], [-87.69924216699988, 41.98762501600003], [-87.69919228199988, 41.98597492200007], [-87.69917398599993, 41.98538304900005], [-87.6997667189999, 41.98573289800003], [-87.7003752459999, 41.98609355200005], [-87.7010323129999, 41.98648728800002], [-87.7019844719999, 41.98719044400006], [-87.70270831999991, 41.987610587000056], [-87.7040666469999, 41.98839439900006], [-87.7050404209999, 41.98899410500006], [-87.70601890399988, 41.98958534200004], [-87.70689004199993, 41.98996609700003], [-87.70771213499994, 41.990589144000054], [-87.7085315829999, 41.99107895000008], [-87.70933822299989, 41.99159416000003], [-87.70933299999989, 41.991576], [-87.70940498399992, 41.991623819000026], [-87.70961670799993, 41.99176446600005], [-87.71135966099988, 41.992855797000054], [-87.71601412999989, 41.99571575700008], [-87.7183853959999, 41.997164066000025], [-87.71844012599988, 41.99719749300004], [-87.7180210359999, 41.99720697000003], [-87.7178812039999, 41.99721030500006], [-87.71684704699992, 41.99723494200002], [-87.71555902699991, 41.99722999400005], [-87.71439305999989, 41.99728591700005], [-87.71288604999992, 41.99723993800006], [-87.71175870599988, 41.99723062700007], [-87.7114590129999, 41.99723047400004], [-87.71105901599991, 41.99723003200005], [-87.71094875999992, 41.997239404000084], [-87.71042927699992, 41.997283563000046], [-87.7100120199999, 41.997319019000074], [-87.70980604299989, 41.99733393300004], [-87.70950200999991, 41.997356631000045], [-87.70950055799992, 41.997413473000044], [-87.7094584739999, 41.99906132200004], [-87.70945701299989, 41.999118408000065], [-87.70945523099992, 41.999175459000064], [-87.7093997949999, 42.00095034500003], [-87.7093980099999, 42.00100741400006], [-87.70926002799987, 42.004630378000044], [-87.70926002799987, 42.00468748500003], [-87.70926002799987, 42.00668725900004], [-87.70926002799987, 42.00674437800006], [-87.70926002799987, 42.00680148500003], [-87.70926002799987, 42.00827325900007], [-87.70926002799987, 42.008330378000025], [-87.70926002799987, 42.00838748500006], [-87.70926002799987, 42.00924625900005], [-87.70926002799987, 42.00930337800007], [-87.70926002799987, 42.009360485000045], [-87.70926002799987, 42.01007325900007], [-87.70926002799987, 42.010130378000035], [-87.70926002799987, 42.01018748500007], [-87.70926002799987, 42.01167325900006], [-87.70926002799987, 42.01172994600005], [-87.7093740959999, 42.01174085700005], [-87.70937673299993, 42.01179889300004], [-87.70937816399992, 42.011857111000026], [-87.70937909399991, 42.01191405200005], [-87.70940133299987, 42.01266941100004], [-87.70940289999992, 42.01272667000005], [-87.70934612599987, 42.01362191900005], [-87.70934509999994, 42.013639293000026], [-87.70934685799993, 42.01367917800006], [-87.70934629099989, 42.013696402000036], [-87.70944440599992, 42.017169407000026], [-87.7094459399999, 42.01722662500003], [-87.70944601099987, 42.01728359000002], [-87.7094460129999, 42.01731536100003], [-87.70944568599991, 42.01737233000005], [-87.70945764899993, 42.01747660100005], [-87.70945790599988, 42.01751337800003], [-87.70945790599988, 42.01753342800002], [-87.70945904199992, 42.01757047300003], [-87.70946034299992, 42.017627557000075], [-87.70949201499988, 42.01897252500003], [-87.7094921859999, 42.01902927500002], [-87.70929801299988, 42.01902947400002], [-87.70905899699993, 42.019029520000046], [-87.70866478899987, 42.01906531300005], [-87.70756336599992, 42.01916553900008], [-87.70685902099989, 42.01922944800002], [-87.70435902099989, 42.01932945400005], [-87.70205900399989, 42.019329496000054], [-87.69985903199992, 42.01942943000005], [-87.69817995499992, 42.01935647100003], [-87.6975590689999, 42.01932890000005], [-87.69660001399988, 42.01928304200003], [-87.69512573399992, 42.01942319000005], [-87.69505902899992, 42.01942942600004], [-87.69482555299992, 42.01942962000003], [-87.6934590369999, 42.01942996600006], [-87.69055903699989, 42.01948896700008], [-87.68925302799991, 42.01951599200003], [-87.6890892619999, 42.01952403100005], [-87.68884002299987, 42.01953544800005], [-87.6879590119999, 42.01953047600005], [-87.68665902699988, 42.019530446000026], [-87.68611952099988, 42.01949873900008]]]}, "properties": {"city": "Chicago", "name": "West Rogers Park", "regionid": "403169", "geo_point_2d": [42.00335651010399, -87.69498307093737], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.71026003014478, 41.68450120579197], [-87.71025, 41.6841], [-87.71091136185663, 41.6840866391545], [-87.71124900023227, 41.684130000302865], [-87.7127350003833, 41.684130000302865], [-87.71515000019073, 41.684133000441044], [-87.71755000020599, 41.684133000441044], [-87.71915000021598, 41.684133000441044], [-87.72000550537761, 41.684133000441044], [-87.72004391226, 41.684964492948346], [-87.72010310590804, 41.686452369364716], [-87.72017619578912, 41.68840041641364], [-87.71657936134478, 41.688403119237385], [-87.71435200728199, 41.68852622770978], [-87.71304901636621, 41.68855], [-87.71055, 41.68859], [-87.71035871219884, 41.68747394156818], [-87.71032226076552, 41.68645330143763], [-87.71026003014478, 41.68450120579197]]]}, "properties": {"city": "Chicago", "name": "Mount Greenwood Heights", "regionid": "403369", "geo_point_2d": [41.686302161547324, -87.7151673045351], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.59132456399992, 41.816895208000055], [-87.59125753699992, 41.81682311200006], [-87.5911509029999, 41.81673140700006], [-87.59104906899994, 41.816646546000065], [-87.59092392699992, 41.816558921000066], [-87.59089919899989, 41.81653739000005], [-87.59080619799991, 41.81645339000004], [-87.59070757499987, 41.81634883500003], [-87.59050857599992, 41.816100835000036], [-87.59046387099993, 41.81604006600008], [-87.59042441599989, 41.81597576400008], [-87.59039048999992, 41.81590838200008], [-87.59036232999993, 41.81583839300003], [-87.59034013599988, 41.81576629000005], [-87.59031324599992, 41.81566382900006], [-87.59019239099989, 41.815456659000056], [-87.59007846899992, 41.81535762600002], [-87.58993590899988, 41.81525878000007], [-87.58986089099992, 41.81519364000006], [-87.5897620789999, 41.815093073000064], [-87.58970271599989, 41.81501385200004], [-87.58963271699989, 41.814908851000034], [-87.5895916249999, 41.81484089100007], [-87.58955668799987, 41.81476957000006], [-87.58952817999992, 41.81469544600008], [-87.5895063229999, 41.81461909500006], [-87.5894912869999, 41.81454111300006], [-87.5894772869999, 41.81444611400007], [-87.58946767799993, 41.814315], [-87.58946982299992, 41.81429048100005], [-87.58946028099989, 41.81426106300005], [-87.58942326799988, 41.81413244000003], [-87.58938375999989, 41.81403995300006], [-87.58932981299994, 41.81398055600004], [-87.5890766779999, 41.81374205800005], [-87.58900588199991, 41.81368137600003], [-87.58896496499992, 41.81365943200005], [-87.58884018199988, 41.813612292000066], [-87.5887750029999, 41.81358469100007], [-87.58871219699992, 41.81355205100004], [-87.58865215299993, 41.81351457200008], [-87.58859524199993, 41.81347248700006], [-87.58854181699989, 41.81342605600007], [-87.5884922089999, 41.813375568000076], [-87.58825520899988, 41.81311456800005], [-87.58821146599989, 41.813062577000025], [-87.58817177299991, 41.81300743200005], [-87.58813635699993, 41.812949448000026], [-87.58810541899992, 41.81288895500005], [-87.58807091699991, 41.81281470000004], [-87.5880546319999, 41.812801587000074], [-87.5879942699999, 41.81278228200005], [-87.58788481899987, 41.81276148700004], [-87.5874749579999, 41.81271839100002], [-87.58733342199989, 41.81269191900007], [-87.58719342199991, 41.812653919000056], [-87.5871276269999, 41.81263332200006], [-87.58706360299992, 41.81260774500002], [-87.5870017259999, 41.81257733900008], [-87.58694236099987, 41.81254228200003], [-87.58688585499992, 41.81250278000004], [-87.5868325419999, 41.81245906600008], [-87.58670954199992, 41.812350066000064], [-87.58665435999987, 41.81229832800005], [-87.58660427199992, 41.81224164300004], [-87.5865597199999, 41.81218051100006], [-87.58652109699993, 41.81211547000002], [-87.58648874399992, 41.81204709500007], [-87.58640216399992, 41.81192266100003], [-87.58638618899992, 41.81188840200007], [-87.58631607899989, 41.81181807300003], [-87.58622335199992, 41.81168471800003], [-87.58620782099987, 41.811657043000075], [-87.5866489999999, 41.811533], [-87.58717536199991, 41.81144078200003], [-87.59122611899993, 41.809879552000034], [-87.59187013199991, 41.809640778000066], [-87.59268247699991, 41.80956462300008], [-87.59623602699989, 41.80952333700008], [-87.59748970299994, 41.80951127400004], [-87.59874371499988, 41.809495856000055], [-87.59992932899988, 41.80949], [-87.6011840389999, 41.809485825000074], [-87.60229863699992, 41.809474156000064], [-87.60306565899991, 41.80946724600004], [-87.60386681099992, 41.80945470900007], [-87.6038880699999, 41.81028314800004], [-87.6039035459999, 41.81097958300005], [-87.60393026399991, 41.81182013600004], [-87.6039799999999, 41.81292907200008], [-87.60398422199994, 41.81348634700002], [-87.60399260599988, 41.81425347300006], [-87.6040052469999, 41.81515956500003], [-87.6040291239999, 41.81620418800007], [-87.60404, 41.81668], [-87.60404390599989, 41.81672193200005], [-87.60403791499992, 41.816722022000064], [-87.60184091599989, 41.81674802200006], [-87.60061691599991, 41.81676102200002], [-87.5994019159999, 41.816782022000034], [-87.59878491499987, 41.81679102300005], [-87.59746291499991, 41.81681502200006], [-87.59666891599991, 41.81681802200005], [-87.59644691499993, 41.81682802300008], [-87.59625791599989, 41.81683902200006], [-87.5959789149999, 41.81685002300008], [-87.59517191599988, 41.81688102200007], [-87.59503191599991, 41.81688102200007], [-87.59485291599991, 41.816874023000075], [-87.59232499999987, 41.816992], [-87.5914288629999, 41.817104650000026], [-87.5914037169999, 41.816990937000064], [-87.59132456399992, 41.816895208000055]]]}, "properties": {"city": "Chicago", "name": "North Kenwood", "regionid": "403351", "geo_point_2d": [41.81311941267225, -87.59643851247085], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.68935778163619, 41.9830508904097], [-87.69056391546954, 41.98304302224127], [-87.69109891495025, 41.98304802247164], [-87.69176291588069, 41.98304302224127], [-87.69300491557611, 41.98303302178027], [-87.69420891531841, 41.983029022495295], [-87.69495391638549, 41.98302402226474], [-87.69505091546093, 41.98302402226474], [-87.69516591536608, 41.98302302221857], [-87.69667391563198, 41.983013021757664], [-87.69787791537428, 41.98300902247249], [-87.69908591530104, 41.983006022334315], [-87.69909112877318, 41.98300595258949], [-87.69909, 41.98301], [-87.69912502966, 41.983915766924014], [-87.69916467750102, 41.98508194005923], [-87.69919228245969, 41.985974921778904], [-87.69924216720385, 41.98762501611689], [-87.69929614657948, 41.98927490172733], [-87.69933, 41.99033], [-87.69850849405165, 41.99033], [-87.69740886719451, 41.99033], [-87.69617204282086, 41.99033608095262], [-87.69500420572368, 41.99035133544256], [-87.69376635323064, 41.99036], [-87.6926667263736, 41.99036], [-87.69122398365899, 41.99036671680292], [-87.68955, 41.99038], [-87.68954649919992, 41.99037602881746], [-87.68954691535404, 41.99037602212236], [-87.68959804810673, 41.99037515067927], [-87.68954191602306, 41.98854802239416], [-87.68948491609348, 41.98671002220478], [-87.68942791526462, 41.98484302247751], [-87.68935778163619, 41.9830508904097]]]}, "properties": {"city": "Chicago", "name": "Arcadia Terrace", "regionid": "403255", "geo_point_2d": [41.98668929073946, -87.69434751757403], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.6122339159999, 41.685391022000026], [-87.61234991499991, 41.684798022000045], [-87.61251091599989, 41.68402902300004], [-87.61251321799992, 41.68401875700004], [-87.6127719159999, 41.682865023000026], [-87.61298791599992, 41.68205102200004], [-87.61316891499987, 41.68126802200003], [-87.61326291599993, 41.68080402200007], [-87.61366691599991, 41.67880102300005], [-87.61380191599991, 41.67815102200006], [-87.61402091599987, 41.67725002200006], [-87.6142419149999, 41.67634202200008], [-87.6144949149999, 41.67533102200007], [-87.61472491499987, 41.67441402200005], [-87.61494891599992, 41.673462023000056], [-87.6151159159999, 41.67266602200004], [-87.61535391499987, 41.67153502200005], [-87.61550791599988, 41.67075602200002], [-87.61574091499992, 41.66958102200003], [-87.61588591599988, 41.668970023000035], [-87.61643391599993, 41.666622022000055], [-87.61652091499988, 41.66623402300006], [-87.61688041599987, 41.66463602300006], [-87.61695391499988, 41.664311023000025], [-87.6176159149999, 41.66167302300004], [-87.61809491599988, 41.65969302200004], [-87.6181429149999, 41.65947702300007], [-87.6182278729999, 41.659142969000065], [-87.61824491599991, 41.65907302200008], [-87.61825535799993, 41.659023436000034], [-87.618322, 41.659024], [-87.61890063599989, 41.65903327300002], [-87.618946, 41.659034], [-87.619346, 41.659134], [-87.619846, 41.661134], [-87.62174599999989, 41.661734], [-87.624646, 41.661334], [-87.62499299999988, 41.661102], [-87.625046, 41.661066], [-87.627946, 41.659334], [-87.63084599999988, 41.657534], [-87.631346, 41.657434], [-87.631575, 41.657434], [-87.635131, 41.657434], [-87.635201, 41.657434], [-87.636663, 41.657434], [-87.6412469999999, 41.657434], [-87.646847, 41.657034], [-87.64675820099987, 41.657655594000055], [-87.646747, 41.657734], [-87.646847, 41.658234], [-87.64683648199988, 41.658475927000026], [-87.64675452099992, 41.660361017000064], [-87.646747, 41.660534], [-87.64675124099993, 41.660644272000056], [-87.646847, 41.663134], [-87.64672256999992, 41.663134], [-87.646347, 41.663134], [-87.645147, 41.663134], [-87.643847, 41.663134], [-87.64264699999988, 41.663134], [-87.641347, 41.663134], [-87.64148042399994, 41.66446823800003], [-87.641547, 41.665134], [-87.641547, 41.66659976800007], [-87.641547, 41.666834], [-87.64155277399992, 41.66690328500004], [-87.6415577589999, 41.66696311300006], [-87.641647, 41.668034], [-87.64159793799989, 41.66832837300006], [-87.641547, 41.668634], [-87.641647, 41.670434], [-87.642947, 41.670434], [-87.64414699999989, 41.670434], [-87.64534699999989, 41.670434], [-87.64664699999989, 41.670434], [-87.647747, 41.670434], [-87.64796243499987, 41.670434], [-87.6489469999999, 41.670434], [-87.650147, 41.670434], [-87.65144699999988, 41.670434], [-87.652647, 41.670434], [-87.65394799999989, 41.670334], [-87.65457999999988, 41.670334], [-87.655148, 41.670334], [-87.656348, 41.670334], [-87.657648, 41.670334], [-87.658196, 41.670334], [-87.65864799999989, 41.670334], [-87.659548, 41.670334], [-87.66104517399991, 41.670334], [-87.66134799999989, 41.670334], [-87.661248, 41.672134], [-87.66134799999989, 41.673834], [-87.661448, 41.675734], [-87.66135373599988, 41.67752501600005], [-87.66131491299987, 41.67752501200004], [-87.66132191599989, 41.67796602300006], [-87.66132991599993, 41.67853902200004], [-87.66135791499991, 41.67936402300005], [-87.66142091599991, 41.68117402200004], [-87.66146191599994, 41.682366022000046], [-87.66147391499992, 41.68310002300006], [-87.6614769159999, 41.68333002200006], [-87.66152504799992, 41.68479114900002], [-87.66039991499989, 41.68481302200006], [-87.65998491499992, 41.68482002200004], [-87.65911891599988, 41.684831022000076], [-87.6583489149999, 41.68484402200004], [-87.65789091599991, 41.68485302200008], [-87.6567029159999, 41.68487502200008], [-87.65546991599989, 41.68489402200003], [-87.6542949149999, 41.68490902200004], [-87.6530769159999, 41.68492302200007], [-87.65187191499989, 41.684947023000056], [-87.65063891499992, 41.684974022000056], [-87.6494549149999, 41.684998022000045], [-87.64820691499992, 41.68501302200007], [-87.64758791499989, 41.68501302200007], [-87.64694491599991, 41.685015022000066], [-87.64551491499991, 41.68504902200003], [-87.64535391599989, 41.68504802200003], [-87.6443009149999, 41.685042023000044], [-87.64219691999992, 41.68505996700008], [-87.64211991599988, 41.685060022000066], [-87.64205491499989, 41.685060022000066], [-87.64095691599991, 41.685064023000045], [-87.64035191599991, 41.685079022000075], [-87.63971891499992, 41.68509502200003], [-87.63851191499991, 41.68512902200006], [-87.6377179159999, 41.685152022000054], [-87.63727591499989, 41.68515702200005], [-87.63604591599992, 41.68517202200007], [-87.63486591599988, 41.68518702200002], [-87.63354391599992, 41.685203022000046], [-87.63243991499988, 41.68520902200003], [-87.63124391599987, 41.685199023000045], [-87.63004991599989, 41.68523202200002], [-87.62946291599988, 41.68525102200004], [-87.62885991599988, 41.68525502200003], [-87.62762591499988, 41.685263022000065], [-87.62648191599992, 41.68526602200006], [-87.62557191499988, 41.68527602300008], [-87.62526091499988, 41.68527802200003], [-87.62514691499989, 41.685279022000074], [-87.62499691599992, 41.68528002200007], [-87.62410591599989, 41.685301022000026], [-87.6228569159999, 41.685320023000074], [-87.62281791599989, 41.685317023000025], [-87.6227709159999, 41.68531602300003], [-87.62075991599993, 41.68533302200005], [-87.61874491499991, 41.68536302200004], [-87.61801591599992, 41.68536402200004], [-87.61742591599992, 41.68536902200008], [-87.6167199149999, 41.685375023000056], [-87.6156439149999, 41.685391022000026], [-87.61437991599989, 41.685402022000055], [-87.61311891599989, 41.685413023000066], [-87.61249391599992, 41.68542502200005], [-87.61222693699993, 41.685428075000054], [-87.6122339159999, 41.685391022000026]], [[-87.62058719699988, 41.661735678000056], [-87.62052806099989, 41.66170945000004], [-87.62047096399992, 41.661679040000074], [-87.62041619999988, 41.66164460300006], [-87.62021919999994, 41.66151060300005], [-87.62014692699992, 41.66145592000003], [-87.62009655799991, 41.66141030400006], [-87.6197935579999, 41.66111430400007], [-87.61973307899987, 41.66104907300006], [-87.61968766099993, 41.66099015900004], [-87.61953566099992, 41.660775159000075], [-87.6195069059999, 41.66072872900008], [-87.61974031899987, 41.66162623600006], [-87.61984438999988, 41.66186335200006], [-87.6202766519999, 41.66205510200006], [-87.6207280319999, 41.66226462400004], [-87.62142721799994, 41.66226859600004], [-87.62184887499988, 41.66225515000007], [-87.62309383499989, 41.66221417100006], [-87.62313435899993, 41.66219745800002], [-87.62349900299994, 41.66203486300008], [-87.62273101899989, 41.66203231700006], [-87.62266840499987, 41.66202992600005], [-87.62199840399988, 41.66198092700005], [-87.6219651039999, 41.66197786800006], [-87.6212601039999, 41.66189986800002], [-87.62120397299991, 41.661891860000075], [-87.62096397399992, 41.66184985900008], [-87.62084506599989, 41.66182058600002], [-87.62064806599992, 41.66175758600007], [-87.62058719699988, 41.661735678000056]]]}, "properties": {"city": "Chicago", "name": "West Pullman", "regionid": "140913", "geo_point_2d": [41.673652960028406, -87.63561718181813], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.66743176018609, 41.90457224446711], [-87.66739712859197, 41.90334045257915], [-87.66818274297823, 41.903341396512786], [-87.66964592469971, 41.903304493037766], [-87.67019394354992, 41.9033], [-87.67086525791251, 41.903289999999885], [-87.67190353714435, 41.903277747363546], [-87.6727578693891, 41.903261602902504], [-87.6745271067079, 41.90323565786589], [-87.67550304312012, 41.90321309650928], [-87.67666232427207, 41.903189713648395], [-87.67855631564532, 41.90316387264643], [-87.68069256770679, 41.90312925066186], [-87.68356051659272, 41.90308013080576], [-87.68557423883502, 41.9030395152234], [-87.68705, 41.90302], [-87.68702474672597, 41.904204278069486], [-87.6870773244175, 41.90597595326075], [-87.68711992510998, 41.907868790012984], [-87.68715983146821, 41.90939483169036], [-87.68713, 41.91032], [-87.68636562255364, 41.910326589460766], [-87.68538870991097, 41.91033854838368], [-87.68423051658607, 41.91036], [-87.68300893217334, 41.91036854209547], [-87.68087362892597, 41.91041856029741], [-87.6788591902199, 41.91045267433113], [-87.67654016390617, 41.91049599781459], [-87.67550277303758, 41.910511987430276], [-87.67446555753156, 41.910528999048964], [-87.67306223480517, 41.91054604725573], [-87.67184186875723, 41.910561905342746], [-87.6705596178507, 41.91058258680889], [-87.66927835349298, 41.9106038091851], [-87.66848523609093, 41.91062092301127], [-87.66761999999989, 41.91063], [-87.66757390657328, 41.909638449789114], [-87.66755021090182, 41.90811200951227], [-87.66750358550327, 41.90670674524456], [-87.66763175095457, 41.90578939647873], [-87.66743176018609, 41.90457224446711]]]}, "properties": {"city": "Chicago", "name": "Wicker Park", "regionid": "7930", "geo_point_2d": [41.906828722007425, -87.67731224757341], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.68605554299987, 41.86676330800008], [-87.68602003099991, 41.866610133000044], [-87.68604101999993, 41.86660985100007], [-87.68783148899988, 41.866585194000024], [-87.68874700499991, 41.86657571600004], [-87.68972320299989, 41.86656547200004], [-87.69074491799994, 41.86655175300005], [-87.69075383699992, 41.86656436100007], [-87.69073116099989, 41.86656461200003], [-87.69052782399989, 41.86656682000005], [-87.6904747849999, 41.86656757900004], [-87.69058454199988, 41.866743139000036], [-87.69064909799991, 41.86684984100008], [-87.69071411799989, 41.86695620100004], [-87.6908422089999, 41.86705662700006], [-87.69086213199994, 41.86716760800004], [-87.69087163199991, 41.867220531000044], [-87.69095013099988, 41.86729408600006], [-87.69094843499992, 41.86738767400003], [-87.6909779839999, 41.86746159100004], [-87.69100523099988, 41.86753618100004], [-87.69103018099989, 41.867611102000055], [-87.69105237299992, 41.867686350000035], [-87.6910727239999, 41.86776227400008], [-87.69109032199991, 41.867838182000064], [-87.69111166399993, 41.86795253100006], [-87.69112794599992, 41.868068223000066], [-87.69113998399989, 41.86814924600007], [-87.6911506089999, 41.86823403400006], [-87.69116175599991, 41.86835827400005], [-87.69116977399993, 41.868474264000035], [-87.69117310099993, 41.86855455200002], [-87.69117550999992, 41.86863483400002], [-87.69117608299989, 41.868715107000064], [-87.6911854, 41.86903824800003], [-87.69119005099991, 41.86919954500006], [-87.6911927719999, 41.86929465800005], [-87.69124000599992, 41.869389865000066], [-87.6912228279999, 41.869794488000075], [-87.6912120349999, 41.87004873400007], [-87.6912131219999, 41.87010424800007], [-87.69122428299991, 41.87019579200005], [-87.69126344799989, 41.87030348800004], [-87.69122396699991, 41.870625717000046], [-87.69123249499994, 41.87101167500003], [-87.69124659499992, 41.87103322600006], [-87.6912965209999, 41.871109536000056], [-87.69115384699988, 41.87111215000004], [-87.69119133799987, 41.872021691000036], [-87.69122884499987, 41.87293162100007], [-87.69126627599988, 41.87383969100005], [-87.69131158299989, 41.87493879200007], [-87.69132466699992, 41.87525618800003], [-87.69133029299988, 41.87539265600008], [-87.69134345799992, 41.875712032000074], [-87.6912299669999, 41.87571288200006], [-87.68954028099992, 41.875726285000034], [-87.68628500099992, 41.87576005700003], [-87.68526886099991, 41.87577062600006], [-87.68408093599992, 41.875783908000066], [-87.6828276239999, 41.87579792100006], [-87.6799594489999, 41.875820182000034], [-87.67788379099993, 41.87582411200008], [-87.67550449199987, 41.875843637000045], [-87.67267662299989, 41.87588509600004], [-87.67305364699989, 41.875572781000066], [-87.67391174799988, 41.874854700000064], [-87.67444950899993, 41.87440374200008], [-87.67543758699989, 41.87356358200008], [-87.67605965499988, 41.87301905200006], [-87.67663360199991, 41.872558341000044], [-87.67748930499992, 41.87182735700008], [-87.67911329199988, 41.870467640000065], [-87.67994841199992, 41.869773142000035], [-87.6811104309999, 41.86881409700003], [-87.68259156299987, 41.86759948600006], [-87.6835899999999, 41.86669], [-87.68438325599992, 41.86665258400005], [-87.68490101899992, 41.86662816300007], [-87.68593, 41.86672], [-87.68605554299987, 41.86676330800008]]]}, "properties": {"city": "Chicago", "name": "Tri-Taylor", "regionid": "403321", "geo_point_2d": [41.87187560908095, -87.68429521577427], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.6486483269999, 41.92267926700003], [-87.64857914499991, 41.92057391900004], [-87.6485230579999, 41.91887558400003], [-87.64849999999988, 41.91818], [-87.64940132899991, 41.91815765300002], [-87.65045355999989, 41.91815], [-87.65158686999987, 41.91813914300006], [-87.65271962299992, 41.918117510000044], [-87.65409559399987, 41.91808566200005], [-87.65522903899992, 41.91807255500003], [-87.6566058529999, 41.91806320100004], [-87.65821999999989, 41.91803], [-87.65820330099989, 41.91766261400004], [-87.65817999999989, 41.91714], [-87.65944031099988, 41.91710148300005], [-87.6613031239999, 41.91707477800008], [-87.66227445399993, 41.91706100100003], [-87.66403499999988, 41.917018], [-87.66425933899988, 41.91701252100005], [-87.66438647099991, 41.91747564300004], [-87.6645471219999, 41.91787727000006], [-87.6649219739999, 41.91815840900005], [-87.66524327599987, 41.91843954800003], [-87.66552441499991, 41.91873407500008], [-87.66577877899992, 41.91908215100005], [-87.66622056899988, 41.91971136700004], [-87.66652848299991, 41.92005944400005], [-87.6671041479999, 41.92056817200005], [-87.66751916299994, 41.92103673700005], [-87.66798772799989, 41.92150530200006], [-87.6681617659999, 41.921759666000064], [-87.66826886699992, 41.92206758000008], [-87.6685901689999, 41.92225500600005], [-87.66908550899988, 41.92240226900008], [-87.66962101199991, 41.922429044000054], [-87.6703707149999, 41.92241565700004], [-87.67095976899991, 41.92248259400007], [-87.67140155799989, 41.92253614500003], [-87.67187012299989, 41.92267002000005], [-87.67227174999991, 41.92287083400002], [-87.67271353999989, 41.92320552300003], [-87.6733695309999, 41.923674088000034], [-87.67383809599993, 41.923995390000066], [-87.6742531109999, 41.92431669200004], [-87.6744137619999, 41.92451750500003], [-87.6744673119999, 41.92482541900005], [-87.6745169079999, 41.92506100300004], [-87.67425399999988, 41.925064], [-87.67425520899988, 41.925070301000055], [-87.67166414099992, 41.925099836000065], [-87.66891206199993, 41.92513351200005], [-87.66813975199992, 41.92514524600005], [-87.66761677999993, 41.925153192000025], [-87.6664023519999, 41.925174074000026], [-87.6651074409999, 41.92519548400007], [-87.6639740669999, 41.925204767000075], [-87.6633045869999, 41.92521016300008], [-87.66275974099992, 41.92521455500007], [-87.66154655199992, 41.925236721000026], [-87.65992705099988, 41.92526840800007], [-87.6579846569999, 41.92529742200007], [-87.65660867399991, 41.925320186000064], [-87.65571702999993, 41.92533484200003], [-87.65426096899992, 41.92536336400007], [-87.65272239199992, 41.92538241400006], [-87.65078046599987, 41.925417763000034], [-87.64876999999989, 41.92546], [-87.64874164099989, 41.92546030400006], [-87.64870400499989, 41.92421343200004], [-87.6486483269999, 41.92267926700003]]]}, "properties": {"city": "Chicago", "name": "Sheffield Neighbors", "regionid": "403306", "geo_point_2d": [41.92181307403807, -87.65924842685138], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.78013982853508, 42.00008100093778], [-87.78007347126999, 41.99971603598015], [-87.77997393537224, 41.999400838971034], [-87.77994075673978, 41.9990856419621], [-87.78002370332119, 41.9987206770041], [-87.78002370332119, 41.998422069311395], [-87.78002370332119, 41.99799074708831], [-87.78002370332119, 41.99754283554916], [-87.77999052468854, 41.99711151332642], [-87.77989098879098, 41.99686267358241], [-87.77967532767961, 41.99669678041956], [-87.77955920246553, 41.99669678041956], [-87.77932695203802, 41.99669678041956], [-87.77914446955904, 41.996713369735964], [-87.77896198708021, 41.996746548368435], [-87.77879609391738, 41.99681290563354], [-87.77861361143839, 41.99694562016357], [-87.77853066485703, 41.99702856674508], [-87.77836477169454, 41.99711151332642], [-87.77824864648066, 41.99711151332642], [-87.7779832174204, 41.99711151332642], [-87.77783391357389, 41.99711151332642], [-87.77773189182612, 41.99707401298102], [-87.77773189182612, 41.99601157472915], [-87.77773084866003, 41.993819061894925], [-87.77772575598904, 41.99381612493688], [-87.77773691567607, 41.99283302206736], [-87.77773991581427, 41.99166102200147], [-87.77779291555929, 41.99015302263486], [-87.77779791578993, 41.98990002266383], [-87.77780691620484, 41.98948002218873], [-87.77782091595083, 41.98832202276829], [-87.777825915282, 41.98775802285022], [-87.77785291562724, 41.986795022525946], [-87.77786191604221, 41.98650702274015], [-87.77788191606476, 41.984683022296835], [-87.77789691585701, 41.9837800229396], [-87.77792890310833, 41.98284889576672], [-87.777943342789, 41.98209243503515], [-87.7779524481181, 41.98122742875841], [-87.77796, 41.98051], [-87.77854391319822, 41.98083115225916], [-87.77971605180474, 41.98148158075315], [-87.78129782328931, 41.98237035955641], [-87.78215383458617, 41.98284997045906], [-87.78363353372103, 41.983679178883996], [-87.784396823263, 41.9841146070561], [-87.78586718671941, 41.9849436144431], [-87.78826753199006, 41.986277091928514], [-87.789989239919, 41.987235837756835], [-87.79209586215413, 41.98841464384247], [-87.7938914421852, 41.98942132636079], [-87.79619991594124, 41.99070895279809], [-87.79813214372614, 41.99182930294621], [-87.79999948627889, 41.99293762988755], [-87.80089516936332, 41.99346367166391], [-87.80291001618131, 41.99468288138056], [-87.80591034400628, 41.99645451957138], [-87.80666369604577, 41.996894304711944], [-87.80664834114513, 41.99770780040107], [-87.80665591594789, 41.99858802205291], [-87.80664791557898, 41.99945202231038], [-87.80666791560178, 41.99994002232233], [-87.80667296529496, 42.00077605276525], [-87.80667060094697, 42.00082169099533], [-87.80624232222009, 42.000782756565606], [-87.80566199999984, 42.00073], [-87.80564430795921, 42.00079192214235], [-87.8055789799999, 42.00102057000015], [-87.805462, 42.001430000000106], [-87.80513255525759, 42.00145745372864], [-87.804262, 42.00153], [-87.80385655499985, 42.00149624100017], [-87.80346959278425, 42.00146402102273], [-87.80306099999984, 42.001430000000106], [-87.80263340951656, 42.001465632540395], [-87.80186099999985, 42.00153], [-87.80171859554807, 42.00147303860656], [-87.80171379899983, 42.001471120000126], [-87.80136099999986, 42.00133000000018], [-87.80126099999985, 42.00093000000012], [-87.79986099999985, 42.00093000000012], [-87.79957733199981, 41.99998538500003], [-87.7995609999999, 41.999931], [-87.7993989087782, 41.99993087531456], [-87.79893310763147, 41.99993051700598], [-87.79826099999988, 41.99993000000016], [-87.79798790395952, 41.999957309680546], [-87.79754550199992, 42.00000155000004], [-87.797261, 42.00003], [-87.797261, 42.00083], [-87.79712596499984, 42.00083794300006], [-87.79700587041616, 42.000845007405466], [-87.79621136435779, 42.000891743175394], [-87.79610334227166, 42.00089809743207], [-87.79556099999985, 42.00093000000012], [-87.794061, 42.00093000000012], [-87.79349486199979, 42.00096330200004], [-87.79236099999984, 42.00103], [-87.79175904299987, 42.00097527700005], [-87.79130196634193, 42.00093372424308], [-87.79126099999989, 42.00093000000012], [-87.79116099999987, 42.00043000000014], [-87.79077441999986, 42.00054045100013], [-87.79071029352666, 42.0005587731649], [-87.79065821499992, 42.00057365300004], [-87.79046099999987, 42.000630000000164], [-87.79032508318161, 42.00070001784556], [-87.79027934299994, 42.00072358100001], [-87.79027317294808, 42.00072675950706], [-87.79022613282483, 42.00066162700731], [-87.79014318624348, 42.00057868042598], [-87.78984457855059, 42.00042937657936], [-87.78974504265312, 42.0004127872633], [-87.78956256017412, 42.00036301931443], [-87.78938007769521, 42.00031325136574], [-87.78908147000243, 42.00016394751929], [-87.78884921957456, 42.000031232989265], [-87.78866673709572, 41.99993169709152], [-87.78833495077048, 41.99981557187772], [-87.78806952171016, 41.99969944666392], [-87.78785386059869, 41.999599910766165], [-87.78775432470113, 41.999599910766165], [-87.78763819948733, 41.999649678715045], [-87.78742253837576, 41.99969944666392], [-87.7871405199992, 41.9997326252962], [-87.78689168025538, 41.9997326252962], [-87.78672578709262, 41.9997326252962], [-87.78652671529748, 41.9997326252962], [-87.78632764350226, 41.9997326252962], [-87.78619492897222, 41.9997326252962], [-87.78597926786067, 41.9997326252962], [-87.7856972494841, 41.9997326252962], [-87.78538205247499, 41.9997326252962], [-87.78521615931241, 41.9997326252962], [-87.78510003409852, 41.999782393245255], [-87.78500049820096, 41.99984875051035], [-87.78486778367082, 41.99993169709152], [-87.78483460503836, 42.00004782230549], [-87.78461894392683, 42.00016394751929], [-87.78446964008054, 42.00016394751929], [-87.78417103238768, 42.00014735820289], [-87.78393878195998, 42.00004782230549], [-87.78365676358341, 41.99996487572416], [-87.78349087042085, 41.99996487572416], [-87.78329179862553, 41.99996487572416], [-87.78312590546292, 41.99996487572416], [-87.78294342298393, 42.00011417957042], [-87.78289365503525, 42.00032984068179], [-87.78289365503525, 42.00052891247692], [-87.78287706571892, 42.00076116290489], [-87.78279411913768, 42.00099341333249], [-87.78272776187241, 42.00124225307667], [-87.78251210076104, 42.00150768213674], [-87.78236279691468, 42.00182287914568], [-87.78218031443578, 42.0022044334199], [-87.7819480640079, 42.00250304111278], [-87.78179876016155, 42.0028182381219], [-87.78156650973386, 42.00295095265211], [-87.78126790204098, 42.00295095265211], [-87.78100247298083, 42.002884595387], [-87.78073704392048, 42.00271870222433], [-87.78053797212536, 42.00233714795011], [-87.78047161486035, 42.00202195094117], [-87.78027254306494, 42.001557450085606], [-87.78017300716755, 42.001043181281354], [-87.78017300716755, 42.000628448374854], [-87.78013982853508, 42.00008100093778]]]}, "properties": {"city": "Chicago", "name": "Norwood Park East", "regionid": "269602", "geo_point_2d": [41.9936607996992, -87.78931212343441], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.61212978799992, 41.65027484100005], [-87.6121932859999, 41.64994785900006], [-87.61260949499987, 41.650021307000074], [-87.6131862489999, 41.65005735400006], [-87.61426766299988, 41.65012944800008], [-87.61592583099988, 41.65016549500006], [-87.61715143299993, 41.65023759000007], [-87.61762004599989, 41.65027363700005], [-87.61880960099987, 41.650525967000064], [-87.62014464799994, 41.65078026100007], [-87.61986156499992, 41.65146808200005], [-87.6197619999999, 41.65171], [-87.61954599999989, 41.652234], [-87.617614, 41.659018], [-87.61661396399988, 41.65903035500007], [-87.61559291399992, 41.65904], [-87.61401347099988, 41.65904009500007], [-87.61243418399991, 41.65905], [-87.61234128099993, 41.65820937900007], [-87.6121554729999, 41.654864842000045], [-87.6120625689999, 41.652728054000065], [-87.6120625689999, 41.651148689000074], [-87.61212978799992, 41.65027484100005]]]}, "properties": {"city": "Chicago", "name": "Eden Green", "regionid": "403381", "geo_point_2d": [41.65432291543977, -87.61550763259083], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.6188314558956, 41.78005425294125], [-87.61888170774762, 41.779448859046404], [-87.61888170774762, 41.776940455988345], [-87.61720943904248, 41.774803668198345], [-87.61541625047785, 41.774803668198345], [-87.61541291545394, 41.7746180220184], [-87.61538791610016, 41.77312802258249], [-87.61538312307452, 41.77284006700497], [-87.61538, 41.77284], [-87.6144210582097, 41.772855466803115], [-87.61344781325556, 41.77285175968174], [-87.61219163581188, 41.77287265632829], [-87.61093777671215, 41.77289273370299], [-87.60975227629704, 41.772913112138916], [-87.60863710054674, 41.772944161520456], [-87.60752346923637, 41.77297403018721], [-87.60633969245816, 41.772990000000135], [-87.60566204206407, 41.772990000000135], [-87.60558791581332, 41.77149102266535], [-87.60559191599758, 41.77118002271849], [-87.60558291558277, 41.769370022021256], [-87.60555691528363, 41.76835602204412], [-87.60552384271593, 41.76573287954899], [-87.60552722369158, 41.76486315699688], [-87.60546797789063, 41.763190948906036], [-87.60544179163283, 41.76142721543716], [-87.60538343411908, 41.75966562620916], [-87.60533525561833, 41.7584499936494], [-87.60596935843303, 41.75843977026511], [-87.6073628771905, 41.7584110594228], [-87.6082912204214, 41.7583920132133], [-87.60977725838562, 41.758367361652326], [-87.6108917945831, 41.758348783142665], [-87.61191312395307, 41.75832752585243], [-87.61312167109624, 41.75831], [-87.61442398870446, 41.75830202612085], [-87.61609452779685, 41.758264122017266], [-87.6175804425442, 41.75823334556317], [-87.61925256235907, 41.75820783790496], [-87.62064704124502, 41.758180323968936], [-87.62241317930403, 41.75814256379325], [-87.624456094125, 41.75809325520341], [-87.62715014745713, 41.7580900000001], [-87.62891535306994, 41.75808261912997], [-87.63030795853581, 41.758048367365255], [-87.63138000000001, 41.75802000000018], [-87.6308421662814, 41.75896308512501], [-87.62997533394899, 41.760809572319005], [-87.6290911353703, 41.76269669814608], [-87.6284908341588, 41.763970509021604], [-87.62803132676106, 41.76493061663217], [-87.62724407858366, 41.76638339256728], [-87.62659849224026, 41.768017006651206], [-87.62557786151977, 41.76981232231454], [-87.6251171057076, 41.77132904649581], [-87.6251688530367, 41.773074444138935], [-87.62518420939637, 41.77419899595806], [-87.62517302315145, 41.77438697684865], [-87.6251, 41.77446], [-87.62511765886967, 41.77525849513428], [-87.62514, 41.77633], [-87.62516255267575, 41.77674346572406], [-87.6252199999999, 41.77755000000014], [-87.62525, 41.77816214101047], [-87.62525, 41.77889600016205], [-87.62528204752437, 41.77992088875337], [-87.62471691625927, 41.77990702210697], [-87.623657915107, 41.77987502243055], [-87.62294891569947, 41.77987502243055], [-87.62268391607438, 41.77987502243055], [-87.62202791551267, 41.77989302236099], [-87.62124591543804, 41.7799140224296], [-87.6203949157799, 41.77997802268189], [-87.62018591513903, 41.78000902231232], [-87.61999591537386, 41.7800230220583], [-87.6193869160794, 41.78003902189638], [-87.6188314558956, 41.78005425294125]]]}, "properties": {"city": "Chicago", "name": "Park Manor", "regionid": "403356", "geo_point_2d": [41.76669732431505, -87.61741433099748], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.80662976699992, 41.93790886700002], [-87.80660191499997, 41.93750102300003], [-87.80655791599995, 41.93627602300005], [-87.80649791599993, 41.93443600800002], [-87.80656032799996, 41.934330917000054], [-87.80757992499994, 41.934409305000024], [-87.80786025799993, 41.93443086000006], [-87.80906025299997, 41.93443086900003], [-87.81026025599992, 41.934430858000034], [-87.81146024399996, 41.93433088800003], [-87.81276026099991, 41.934430850000055], [-87.81406024899995, 41.93433087800001], [-87.81526025599997, 41.93433085800005], [-87.81629003399993, 41.934245054000016], [-87.81646024899997, 41.934230879000026], [-87.81646025299993, 41.934287587000014], [-87.81646025299993, 41.93597336100003], [-87.81646017099993, 41.93603083300007], [-87.81656033499995, 41.93783090500004], [-87.81750018999995, 41.937830869000024], [-87.81764492899995, 41.937830869000024], [-87.81776025599997, 41.937830858000034], [-87.81782790499992, 41.937825231000026], [-87.81896024899996, 41.937730879000064], [-87.82026025299994, 41.93773086900005], [-87.82146025599994, 41.937729859000044], [-87.82276024899993, 41.937629878000045], [-87.82396025299991, 41.93762986900003], [-87.82506025299993, 41.93762986900003], [-87.82612850399993, 41.93762990900006], [-87.82615538699996, 41.93762991000005], [-87.82625494599993, 41.93762991700003], [-87.82626107199994, 41.93762993500007], [-87.82631092899997, 41.93827466200002], [-87.82635504799997, 41.940036938000034], [-87.82635837499997, 41.94052267800003], [-87.82641610499995, 41.941618657000035], [-87.82646218299998, 41.942344376000044], [-87.82657, 41.9444], [-87.82658, 41.9449], [-87.82658016599997, 41.94490430300004], [-87.82609582799994, 41.94491952300005], [-87.82534, 41.94495], [-87.8241144369999, 41.94548868800006], [-87.82276800399993, 41.94613052400006], [-87.82092990199997, 41.947012658000055], [-87.81914247899991, 41.94786777100005], [-87.81905371399993, 41.94920581200005], [-87.81911443499995, 41.95017733700006], [-87.81923587499993, 41.95127030300005], [-87.81927986299993, 41.95227291100007], [-87.81928212699992, 41.95232947400005], [-87.81836199799993, 41.95232951600007], [-87.81734740399995, 41.95241402500004], [-87.81716202699992, 41.952429432000024], [-87.81695192899996, 41.95242947400003], [-87.81677007699992, 41.95242947400003], [-87.81596199799995, 41.952429516000045], [-87.81551304199994, 41.95246688800005], [-87.81476202699996, 41.952529432000055], [-87.81380122699991, 41.952529474000066], [-87.81346101299991, 41.952529474000066], [-87.81226101299993, 41.952529474000066], [-87.81096101299991, 41.95253047300002], [-87.80976100399991, 41.95253049700005], [-87.80845301499991, 41.95259146600005], [-87.80718519199995, 41.95263268100006], [-87.80712363899993, 41.951997680000034], [-87.80708503199996, 41.95102769200002], [-87.80705667699992, 41.950239197000016], [-87.80702716799996, 41.949329349000024], [-87.80699125499994, 41.94823763800007], [-87.80696281499995, 41.94738445300003], [-87.80692128999998, 41.946108686000045], [-87.80690107199995, 41.94550215600002], [-87.80688880799991, 41.94516980000002], [-87.80686079899992, 41.94441078300007], [-87.80683981199994, 41.943865107000065], [-87.80680138499994, 41.94277294700002], [-87.80677771599993, 41.94203919100004], [-87.80673095999995, 41.940461984000066], [-87.80667021199994, 41.93881979700005], [-87.80662976699992, 41.93790886700002]]]}, "properties": {"city": "Chicago", "name": "Belmont Heights", "regionid": "403283", "geo_point_2d": [41.943295319096364, -87.81496234075838], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.5489164040314, 41.72284100886722], [-87.54841950725744, 41.72231666783814], [-87.54779203397345, 41.72164470238233], [-87.54676460043697, 41.72062800046003], [-87.54721570618572, 41.71952589733856], [-87.54851635959898, 41.71664587906642], [-87.5503744359035, 41.71190778448983], [-87.55083895497961, 41.71042132344633], [-87.55167508931659, 41.70837743951134], [-87.55251122365374, 41.7070767860981], [-87.55344026180592, 41.70605484413053], [-87.55409058851262, 41.70559032505443], [-87.55520543429535, 41.70503290216312], [-87.55687770296936, 41.70391805638032], [-87.55762093349121, 41.70336063348892], [-87.55873577927382, 41.70150255718446], [-87.55950136964873, 41.69973722941826], [-87.55945999999987, 41.69552], [-87.56096598860107, 41.69549500433869], [-87.56170901443608, 41.695482671959574], [-87.56245193188302, 41.69564961682524], [-87.56282354714381, 41.69722898168404], [-87.56316, 41.698080000000125], [-87.56319606625703, 41.698105695881814], [-87.5631999999999, 41.7027300000001], [-87.56443, 41.7027300000001], [-87.56443544604305, 41.70373207193451], [-87.56444, 41.704570000000125], [-87.56505389593373, 41.70456551900783], [-87.5658104597551, 41.70456697738259], [-87.56578125462256, 41.705213957335395], [-87.56644679593767, 41.70605484413053], [-87.56951262184025, 41.70930647766353], [-87.5712777943295, 41.71144326541374], [-87.5729500630037, 41.713394245533514], [-87.5714641374855, 41.713440300726944], [-87.5701634455669, 41.71345388795375], [-87.56876845900321, 41.713465378382104], [-87.56756310865666, 41.713480794768664], [-87.56644721871059, 41.71350420118911], [-87.56523975952507, 41.71352958387369], [-87.56412465898721, 41.713543994685516], [-87.56273483316568, 41.713579325765096], [-87.5618027214797, 41.71362333834863], [-87.56068675939368, 41.71363], [-87.55972, 41.71364], [-87.55973745265113, 41.71441664297506], [-87.55977377427223, 41.71627400488565], [-87.5597988153299, 41.71785299644777], [-87.5598088179886, 41.71850316926717], [-87.55986020998733, 41.71938637797725], [-87.5601229800772, 41.719795394975485], [-87.56038331953245, 41.72010416967817], [-87.56081, 41.720350000000145], [-87.56066972675126, 41.72038339839255], [-87.56067262237598, 41.72138423067972], [-87.5607237349761, 41.72270695529384], [-87.56010891570656, 41.72269402249482], [-87.55986991548134, 41.7226980217799], [-87.55976791617536, 41.72269902182599], [-87.55844891562863, 41.72272802226352], [-87.55827091551727, 41.7227300223557], [-87.55814991623477, 41.72273202244779], [-87.55804791602958, 41.72273302249388], [-87.5579829157312, 41.72273402253997], [-87.55660391511661, 41.72274602219395], [-87.55518191521782, 41.72275702180158], [-87.55451991617805, 41.72276302207829], [-87.55386491566252, 41.72275602175547], [-87.55300491558967, 41.72274802228613], [-87.55253891569227, 41.722761021986024], [-87.55152291562284, 41.72277602267738], [-87.5510469152647, 41.722792022515726], [-87.55040191521, 41.72281202253824], [-87.54970391541025, 41.72283402265314], [-87.5489164040314, 41.72284100886722]]]}, "properties": {"city": "Chicago", "name": "South Deering", "regionid": "140440", "geo_point_2d": [41.71156244257977, -87.55829433741066], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.58562691568586, 41.73690902215193], [-87.58562972091364, 41.73690895564486], [-87.5856398522216, 41.7375503944107], [-87.58566062997944, 41.73829366406967], [-87.58568080091902, 41.739222242736524], [-87.58570125515413, 41.74015085435852], [-87.58573363426427, 41.74126478410722], [-87.58575217323863, 41.742007795741664], [-87.58576642626788, 41.74293868660455], [-87.5857526928374, 41.74358882231638], [-87.58573999999977, 41.74416000000021], [-87.58571730314658, 41.74479460629342], [-87.58577852588392, 41.7464697535028], [-87.58585406533224, 41.74832664276018], [-87.5858063159954, 41.749897335509644], [-87.58187945257508, 41.74706243769546], [-87.57715348664681, 41.74351537601651], [-87.57213289312047, 41.73980363677494], [-87.5685358833736, 41.7371923821632], [-87.56860591528482, 41.737191022560545], [-87.56876591546566, 41.737187021477], [-87.56892991583094, 41.73718302219183], [-87.5696579152149, 41.73716502226139], [-87.57085891571832, 41.73715102251558], [-87.57213691527454, 41.73714302214669], [-87.57331591566307, 41.73713302258515], [-87.57448891577505, 41.73710702228599], [-87.57568291595572, 41.737080021940564], [-87.57694391562752, 41.737053022494685], [-87.57815491569255, 41.73703302247198], [-87.57935891543485, 41.737008022218916], [-87.58057991596085, 41.73698202191976], [-87.58274291584291, 41.73696702212768], [-87.58484691570338, 41.73692702208236], [-87.58520991534938, 41.73691902261291], [-87.58543191569014, 41.73691302233637], [-87.58562691568586, 41.73690902215193]]]}, "properties": {"city": "Chicago", "name": "Stony Island Park", "regionid": "403128", "geo_point_2d": [41.74135496267677, -87.58002528543601], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.64070489568995, 41.80529429693964], [-87.6414370197707, 41.80520533774718], [-87.64198351234278, 41.80520543388584], [-87.6425329054633, 41.805190976171964], [-87.64418140316377, 41.805159699916786], [-87.64535, 41.80513], [-87.64539837316204, 41.806575070475226], [-87.64542820054243, 41.80749028366349], [-87.64544407102603, 41.80822232132269], [-87.64549, 41.810054635177465], [-87.64551519755355, 41.8114272944343], [-87.64556339197036, 41.81270819123191], [-87.64560574066043, 41.8140873421738], [-87.64567211021426, 41.815465598106385], [-87.64573237263858, 41.81701989837922], [-87.64576668848457, 41.81812113357133], [-87.6458167675977, 41.819856714523176], [-87.64586142863992, 41.821140716118535], [-87.64590561713491, 41.82288581257547], [-87.64592332608169, 41.823454992215964], [-87.64592191527483, 41.82345502223417], [-87.64586591539143, 41.82345502223417], [-87.64471091610908, 41.823465021795876], [-87.64348091516825, 41.82347702234905], [-87.64226791591051, 41.82350702193311], [-87.64105291566102, 41.82352402181746], [-87.63982891499683, 41.82354802202453], [-87.63861891587725, 41.823560022577695], [-87.63730891574534, 41.82356602195496], [-87.63656891580808, 41.82357202223167], [-87.6364399152575, 41.82357302227778], [-87.63630980741779, 41.82357372192024], [-87.63630980741779, 41.82150677184418], [-87.63622239000279, 41.81847630145692], [-87.63622239000279, 41.81763126644491], [-87.63622239000279, 41.81699020540146], [-87.63604755517272, 41.815795500729465], [-87.63604755517272, 41.81530013537771], [-87.63604755517272, 41.81448423950418], [-87.63610583344939, 41.81358092621574], [-87.6362806682795, 41.81273589120374], [-87.63642636397118, 41.81183257791539], [-87.63660119880109, 41.81107496031855], [-87.63677603363125, 41.810142507891634], [-87.63698000759963, 41.8094140294331], [-87.637183981568, 41.80859813355975], [-87.63750451208979, 41.80781137682443], [-87.63794159916483, 41.8070537592276], [-87.63820385140993, 41.80658753301414], [-87.63881577331507, 41.80615044593915], [-87.6395733909119, 41.80568421972561], [-87.64018531281705, 41.805451106619024], [-87.64070489568995, 41.80529429693964]]]}, "properties": {"city": "Chicago", "name": "Canaryville", "regionid": "403340", "geo_point_2d": [41.81484508196885, -87.64112349648103], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.58686091591176, 41.72227602211225], [-87.58700491535508, 41.722268022642716], [-87.58856691541219, 41.7222720219278], [-87.59063691550398, 41.722239022205294], [-87.59163591568904, 41.72221802213649], [-87.59356491557651, 41.72219202183733], [-87.59476591518045, 41.72217802209135], [-87.59595591607615, 41.72216402234519], [-87.5971439150809, 41.72215402188437], [-87.59839991542162, 41.722146022414755], [-87.60138291533147, 41.72208502230099], [-87.60294591633387, 41.72202502223323], [-87.60311695119606, 41.72202510856808], [-87.60338891517104, 41.722013022579254], [-87.60346791611458, 41.72200602225662], [-87.60382791472304, 41.72202601958113], [-87.60389891799588, 41.722009023294355], [-87.60444004185958, 41.72199288406111], [-87.60444210101703, 41.72202006652601], [-87.60568990353407, 41.72205957652057], [-87.607269161743, 41.72205096995507], [-87.60875492878297, 41.72201865665921], [-87.60977713368557, 41.72201], [-87.61089071974733, 41.721995878041106], [-87.61200595913786, 41.72196587393497], [-87.61330499371535, 41.72195643637091], [-87.61408, 41.72184], [-87.61410018203651, 41.72277846470174], [-87.61416621102217, 41.724634489559456], [-87.61421601755262, 41.72593521113727], [-87.61423, 41.726900000000114], [-87.61422588806654, 41.726901821278624], [-87.6142260756752, 41.72690192418964], [-87.61423191587244, 41.72701602207465], [-87.6142657510652, 41.7279031250157], [-87.61427, 41.728165952084346], [-87.61428392867866, 41.72890804016769], [-87.614309108022, 41.729745979343335], [-87.61432282563969, 41.73020997942879], [-87.61435108735826, 41.73151001848104], [-87.61437131818462, 41.73244063649834], [-87.6143916765111, 41.73337128125924], [-87.6144181583773, 41.734300804112785], [-87.61445081228783, 41.73532193177041], [-87.6144731274029, 41.73596819788925], [-87.61449200457288, 41.73648000103436], [-87.61339996323295, 41.73649354082933], [-87.6124709263831, 41.73650973984799], [-87.61163518302938, 41.73652518996128], [-87.61024204736646, 41.73654873532453], [-87.60931161721476, 41.736565841886126], [-87.60791858845745, 41.736589487199204], [-87.60587526856727, 41.73662857649359], [-87.60487, 41.73664], [-87.60420312581385, 41.73664474661429], [-87.60308870978577, 41.7366589292474], [-87.60169494315254, 41.73667583378455], [-87.60037403429514, 41.736692170429656], [-87.60013991558993, 41.736696022225956], [-87.5985999156477, 41.73672402261739], [-87.5978879152024, 41.73674302259392], [-87.59743491590444, 41.73675102206327], [-87.59524492736823, 41.736747024576836], [-87.59517891533267, 41.73662702264258], [-87.59508091531177, 41.73644802248513], [-87.59488191603107, 41.736085022839326], [-87.59483791580148, 41.73600502274886], [-87.59452391571624, 41.73555802282795], [-87.5941249153102, 41.73505602217064], [-87.59354391550781, 41.73432302255595], [-87.59326791537563, 41.733942022080335], [-87.59321491563051, 41.73386902231251], [-87.59313491554005, 41.73377402243015], [-87.59299491538174, 41.73360802287186], [-87.59271491596455, 41.733275022810204], [-87.59259591587488, 41.73313402170653], [-87.59207791627858, 41.73229702269386], [-87.5917179158716, 41.73171702203826], [-87.59151791564551, 41.73134902216182], [-87.59099891600306, 41.7302640234084], [-87.59057491624321, 41.729506021741905], [-87.59035191585627, 41.72910802318078], [-87.5895169160365, 41.7277060224052], [-87.58860391621835, 41.72612402232547], [-87.58850288458301, 41.72591004393704], [-87.58839791571583, 41.7256290219907], [-87.58758591515756, 41.723672022611275], [-87.58755791566551, 41.72362502224323], [-87.58745291532189, 41.72344802217814], [-87.58734695809979, 41.72327791901342], [-87.58729291514113, 41.723191022921775], [-87.58726191551062, 41.72314002236954], [-87.58722291551156, 41.72307402292434], [-87.58695095603325, 41.7226249878379], [-87.58683991584297, 41.722518022475555], [-87.58677791568294, 41.72245902245397], [-87.58662390498682, 41.722288144973184], [-87.58666091568568, 41.72228602257315], [-87.58686091591176, 41.72227602211225]]]}, "properties": {"city": "Chicago", "name": "Burnside", "regionid": "137634", "geo_point_2d": [41.72896055405266, -87.6023654107906], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.80666402799993, 42.01877225900005], [-87.80666402799993, 42.01788648500006], [-87.80666402799993, 42.01782937800004], [-87.80666402799993, 42.01777225900007], [-87.80666402799993, 42.01718648500007], [-87.80666402799993, 42.01712937800004], [-87.80676241499991, 42.01535798500003], [-87.80676400099989, 42.01532943100004], [-87.80675873599992, 42.01530218900005], [-87.80666402799993, 42.01352894600005], [-87.80666402799993, 42.013472259000075], [-87.80666402799993, 42.011686485000034], [-87.80666402799993, 42.01162937800006], [-87.8068638899999, 42.01072956500007], [-87.80686402799992, 42.01067225900004], [-87.80686402799992, 42.009086485000054], [-87.80686402799992, 42.00902937800002], [-87.80695906499993, 42.00826865100003], [-87.80696395999989, 42.00822949200005], [-87.8069554239999, 42.008212792000045], [-87.80668700599993, 42.007217448000056], [-87.80666790999992, 42.00714743000008], [-87.80666273099988, 42.007129253000066], [-87.80666793699992, 42.00709031400003], [-87.8066906529999, 42.00690944400008], [-87.80676395999988, 42.00632949200006], [-87.80686080199987, 42.005361197000056], [-87.80686397499991, 42.00532947100003], [-87.80685714899992, 42.00530536000007], [-87.8067635859999, 42.004556861000076], [-87.8067640279999, 42.004528946000036], [-87.80676723599991, 42.00449846400004], [-87.8069608849999, 42.00265880000005], [-87.80696397199993, 42.00262947500004], [-87.80695723499989, 42.00260295900004], [-87.8067630929999, 42.000828861000045], [-87.80667341599988, 42.00082070900003], [-87.80667296499989, 42.000776053000024], [-87.80667232299992, 42.00076474100006], [-87.8066679159999, 41.999940022000075], [-87.8066479159999, 41.999452022000064], [-87.80665591599988, 41.99858802200002], [-87.8066487279999, 41.997707592000026], [-87.80719098199992, 41.99769], [-87.80765495099988, 41.99758082700003], [-87.80820346499989, 41.997392866000034], [-87.80853410999991, 41.99728465500004], [-87.80887251199992, 41.997205286000046], [-87.80929019599994, 41.99711782700006], [-87.8098152449999, 41.99709151100006], [-87.8101837489999, 41.99700211800006], [-87.81031189599989, 41.99707110100007], [-87.8114669159999, 41.99703202200003], [-87.81345691599992, 41.99699602200006], [-87.81538991599989, 41.99694302300008], [-87.81660791499989, 41.99692502300006], [-87.8188509149999, 41.996853022000025], [-87.82039091499989, 41.996806023000026], [-87.82129700299991, 41.99676404400008], [-87.82135847999989, 41.99675905700007], [-87.82151352399987, 41.99675655600003], [-87.82161379999988, 41.99675493900008], [-87.82171576499988, 41.996752784000066], [-87.82171901299994, 41.99680347400004], [-87.82186301299991, 41.99752947400008], [-87.8219633399999, 41.998629617000056], [-87.82196301299989, 41.99868658700005], [-87.82196301299989, 41.99897236100007], [-87.82196301299989, 41.999029474000054], [-87.82196301299989, 41.99908658700007], [-87.82196301299989, 42.00047236100004], [-87.8219629519999, 42.00052956000006], [-87.82187015199989, 42.00099377700007], [-87.8218627659999, 42.00102921500007], [-87.82187015399988, 42.00105089700008], [-87.82194498699992, 42.00127539700007], [-87.8219633399999, 42.00132961700006], [-87.82196301299989, 42.00138658700006], [-87.82196301299989, 42.00177236100006], [-87.82196301299989, 42.00182947400002], [-87.82196301299989, 42.00188658700005], [-87.82196301299989, 42.00210156400003], [-87.82196301299989, 42.00215867700006], [-87.82196301299989, 42.00217236100006], [-87.82196301299989, 42.00221579000003], [-87.8219629519999, 42.00222956000003], [-87.82186935899993, 42.00269774400003], [-87.8218627659999, 42.00272921500003], [-87.82186935999992, 42.00275486300006], [-87.82195721299988, 42.00310627400005], [-87.8219632739999, 42.00312971800007], [-87.82195721399994, 42.00316339300007], [-87.82182361899991, 42.00394247000008], [-87.82160453899989, 42.00408486000002], [-87.82158227799991, 42.00409923300002], [-87.82160454499989, 42.00414197300005], [-87.82185941399989, 42.004622686000054], [-87.82186331699988, 42.00462966200006], [-87.82185941799989, 42.004679807000045], [-87.8218572299999, 42.00471007000004], [-87.8217639889999, 42.00602945000008], [-87.82167206299991, 42.00721937900004], [-87.82166303199993, 42.00732943700006], [-87.8216630129999, 42.007386587000035], [-87.82166301499987, 42.00767235600005], [-87.82166321299988, 42.00772932200004], [-87.82166484099989, 42.00778653800005], [-87.82176147299991, 42.011372403000045], [-87.8217620419999, 42.01142916000002], [-87.82099301299989, 42.01138647400005], [-87.82064951499987, 42.01136746500002], [-87.82020042299987, 42.01134202800005], [-87.81996304599993, 42.011328946000035], [-87.81936402799991, 42.011328946000035], [-87.81936032299988, 42.01133075500007], [-87.81936212499991, 42.011387805000076], [-87.81937052099993, 42.01165369200003], [-87.81937232299991, 42.011710755000074], [-87.81937602799988, 42.01170894600005], [-87.81986338599994, 42.01172920700008], [-87.81986402799991, 42.011786485000066], [-87.81986402799991, 42.01197225900005], [-87.81986402799991, 42.012029378000065], [-87.81986402799991, 42.01208648500004], [-87.81986402799991, 42.012097385000054], [-87.81986402799991, 42.012154504000065], [-87.81986402799991, 42.01221161100004], [-87.81986402799991, 42.012872259000055], [-87.81986402799991, 42.012929378000074], [-87.81986402799991, 42.01298648500005], [-87.81986402799991, 42.013772259000056], [-87.81986402799991, 42.013829378000075], [-87.81986402799991, 42.01388648500005], [-87.81986402799991, 42.01467225900005], [-87.81986402799991, 42.01472894600004], [-87.8199292719999, 42.01507549800003], [-87.81995951899992, 42.01523093100007], [-87.81996322399992, 42.015229122000044], [-87.81997345699993, 42.01522934600006], [-87.82056288099993, 42.01522960800008], [-87.82132543199991, 42.01551538100005], [-87.82136286099988, 42.01552915000007], [-87.82140263699992, 42.01575036400003], [-87.82141568899992, 42.01582378900007], [-87.82142257199989, 42.01586585000006], [-87.82166433999991, 42.01722961700005], [-87.82166401299989, 42.017286587000044], [-87.82166401399991, 42.01853233000007], [-87.82166415499994, 42.018572384000024], [-87.82166401699992, 42.01858943400002], [-87.8216639969999, 42.01862928100006], [-87.82130351699992, 42.01862894600002], [-87.8209640459999, 42.01862894600002], [-87.81906308799992, 42.01862886800006], [-87.81871229399988, 42.01859963500004], [-87.8178630169999, 42.018529030000074], [-87.81666301899989, 42.018629024000056], [-87.81536304599987, 42.01872894600005], [-87.8141630459999, 42.01872894600005], [-87.81286304599989, 42.01872894600005], [-87.81166304599991, 42.01872894600005], [-87.81136429699988, 42.01872894600005], [-87.81036300399988, 42.01872910400004], [-87.81016193999992, 42.01876003700004], [-87.80922815499989, 42.018904135000064], [-87.80906303399989, 42.01892939500003], [-87.80810328599992, 42.01892947400005], [-87.80776309299992, 42.01892886100005], [-87.80691917699993, 42.01885214100007], [-87.80666402799993, 42.01882894600003], [-87.80666402799993, 42.01877225900005]]]}, "properties": {"city": "Chicago", "name": "Edison Park", "regionid": "269579", "geo_point_2d": [42.00770448895082, -87.81412882143049], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.595154074843, 41.71042132344633], [-87.59552569010378, 41.710142612000496], [-87.59691924733215, 41.71004970818537], [-87.5987773236368, 41.71004970818537], [-87.60128572664787, 41.710142612000496], [-87.60267928387623, 41.71042132344633], [-87.60388703347417, 41.71042132344633], [-87.605767650771, 41.71042132344633], [-87.60563191514355, 41.71097202264315], [-87.6053289155654, 41.71230602208278], [-87.60520791538363, 41.712822021587066], [-87.6051539155925, 41.713054022388825], [-87.60513979423303, 41.71311553408794], [-87.60397993728948, 41.71311553408794], [-87.60286509150677, 41.71311553408794], [-87.6019360533544, 41.71292972645724], [-87.600914111387, 41.71274391882698], [-87.59905603508253, 41.71255811119645], [-87.59840570837582, 41.712465207381314], [-87.59756957403886, 41.712186495935576], [-87.59691924733215, 41.71190778448983], [-87.59599020917996, 41.711257457783304], [-87.59552569010378, 41.71088584252234], [-87.595154074843, 41.71042132344633]]]}, "properties": {"city": "Chicago", "name": "London Towne", "regionid": "403382", "geo_point_2d": [41.71143321083543, -87.600928687721], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.6857693738678, 41.857410155001695], [-87.68576939690348, 41.857402450978675], [-87.68764811888289, 41.85703032092319], [-87.68818189484996, 41.85692198586973], [-87.68896191423381, 41.85678602281895], [-87.68997091600652, 41.85655202282145], [-87.69018291594654, 41.85651302187231], [-87.69073191505989, 41.856377022835034], [-87.69152191623112, 41.85619202211806], [-87.69164691596464, 41.85617602219881], [-87.69310391582148, 41.85584802278327], [-87.69430091598083, 41.85557702283243], [-87.69538906701287, 41.855348010062436], [-87.69553702314944, 41.85531687114255], [-87.69556179176354, 41.85555316617481], [-87.69608491523023, 41.85555402275201], [-87.6988209158378, 41.85550402199878], [-87.69914191604182, 41.85550602204496], [-87.6991979159834, 41.85550602204496], [-87.69924991583288, 41.85550402199878], [-87.70285952952689, 41.85544960439592], [-87.70283612718566, 41.85611509493291], [-87.70288745083533, 41.85788714690243], [-87.70294595017796, 41.859716113772116], [-87.70297298292725, 41.86069185702559], [-87.70300748575265, 41.861850193892195], [-87.70304030800999, 41.86331397992538], [-87.70309323008321, 41.86496162477687], [-87.70321, 41.866420000000126], [-87.7024781168776, 41.86643199804322], [-87.7013188658457, 41.866451532168995], [-87.69985345877305, 41.86646533120363], [-87.69887726094017, 41.866475469212105], [-87.69759577693112, 41.866480000000124], [-87.69668036389305, 41.86649039421236], [-87.69539874804003, 41.86651000000011], [-87.69423920505594, 41.86652], [-87.69277451912845, 41.86653000000017], [-87.69179906917583, 41.86653759877013], [-87.68972320277615, 41.86656547186828], [-87.68874700517863, 41.86657571616915], [-87.68783148896124, 41.866585194153416], [-87.68604102067675, 41.866609948418], [-87.68600011008742, 41.86630667883824], [-87.68595999999982, 41.86526917757752], [-87.68594689775796, 41.8645383192918], [-87.6859382821127, 41.86350969267788], [-87.68590000198029, 41.86246576521251], [-87.68587858207586, 41.86191496766947], [-87.68586436357765, 41.8615493491406], [-87.68583932377666, 41.86069423184434], [-87.68582330898168, 41.859962213163826], [-87.68580614984685, 41.85935409896749], [-87.68581596558485, 41.8593539332254], [-87.68581, 41.85935], [-87.6857926718111, 41.858923436223805], [-87.68577872831149, 41.858251792209444], [-87.68579101201878, 41.85794469952838], [-87.6857693738678, 41.857410155001695]]]}, "properties": {"city": "Chicago", "name": "Douglas Park", "regionid": "403560", "geo_point_2d": [41.86124544158485, -87.69467625530679], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.57957231599991, 41.803784655000065], [-87.5801496909999, 41.80253367600004], [-87.58738731399988, 41.80254707100004], [-87.58734687699987, 41.80245001600007], [-87.5874049159999, 41.80245002300005], [-87.5878109159999, 41.802447022000074], [-87.58788791599991, 41.802447022000074], [-87.5879649159999, 41.80244402200008], [-87.58916091599988, 41.80240402200007], [-87.59047591499989, 41.80235802200007], [-87.59177891599987, 41.80233702200007], [-87.59338391499993, 41.802311022000076], [-87.59499891499989, 41.802297022000054], [-87.59664191499988, 41.80227702200004], [-87.5982149149999, 41.802266022000076], [-87.59907191599987, 41.80225802200004], [-87.59974291499991, 41.802248022000065], [-87.6013339159999, 41.80220802200005], [-87.60154791599989, 41.80221402200004], [-87.60283991499993, 41.80219602300007], [-87.60374491499992, 41.80219402200002], [-87.60375306599988, 41.80219386800006], [-87.60373999999987, 41.80282578200006], [-87.60373999999987, 41.80345288300003], [-87.6037631809999, 41.804637360000065], [-87.60378587099987, 41.805403144000074], [-87.60380998499993, 41.80651904200004], [-87.6038241349999, 41.80742463300003], [-87.6038396939999, 41.808260478000065], [-87.60385225299989, 41.80888741200004], [-87.60386681099992, 41.80945470900007], [-87.60306565899991, 41.80946724600004], [-87.60229863699992, 41.809474156000064], [-87.6011840389999, 41.809485825000074], [-87.59992932899988, 41.80949], [-87.59874371499988, 41.809495856000055], [-87.59748970299994, 41.80951127400004], [-87.59623602699989, 41.80952333700008], [-87.59268247699991, 41.80956462300008], [-87.59187013199991, 41.809640778000066], [-87.59122611899993, 41.809879552000034], [-87.58717536199991, 41.81144078200003], [-87.5866500049999, 41.811532234000076], [-87.58665098399987, 41.81147525800008], [-87.58665007199993, 41.81028948400006], [-87.58664978399989, 41.81023226800004], [-87.5854959689999, 41.809043455000044], [-87.58494109099989, 41.80846930800003], [-87.58465932299991, 41.80817775500008], [-87.58377032299988, 41.80725775500007], [-87.58361598999991, 41.807098012000026], [-87.58094929299992, 41.80433268400008], [-87.58004429399992, 41.80400968400004], [-87.57957191699988, 41.80384077400003], [-87.57955002799989, 41.80383294600005], [-87.57957231599991, 41.803784655000065]]]}, "properties": {"city": "Chicago", "name": "Kenwood", "regionid": "274178", "geo_point_2d": [41.80592692719289, -87.59298858741], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.6108149067486, 41.83127310382766], [-87.61062793949722, 41.83119089050556], [-87.61057874977455, 41.83119044446031], [-87.61038087804869, 41.83073366829276], [-87.60991491121352, 41.829849066856035], [-87.60899509313822, 41.8280801862766], [-87.60867005910272, 41.82744228697957], [-87.60818876550326, 41.826497531006645], [-87.60772602646247, 41.82557983931459], [-87.60709038687892, 41.82432884651759], [-87.60689, 41.82387], [-87.60689293755483, 41.82386565701963], [-87.60689581333035, 41.82387817753075], [-87.60696991566681, 41.8238640222021], [-87.6072229156381, 41.82385702187921], [-87.60794191550653, 41.82385302259431], [-87.60796491566752, 41.82385302259431], [-87.60962796804448, 41.8238848433057], [-87.60970189231527, 41.82391815239505], [-87.60984191585594, 41.823916021901056], [-87.61261688998972, 41.82387096856532], [-87.6143569153392, 41.82384602227167], [-87.61547591566004, 41.82383202252551], [-87.61671291512486, 41.823823022110695], [-87.61693691555793, 41.82382202206461], [-87.61698491597188, 41.823821022018514], [-87.61702891530228, 41.823821022018514], [-87.61712591527692, 41.82382002197251], [-87.61724480352858, 41.82382101275251], [-87.61726430710398, 41.82449593580471], [-87.61729304905111, 41.82602815073546], [-87.617311018468, 41.82672446563586], [-87.61733495308094, 41.82791073036551], [-87.61736712655576, 41.82986230290296], [-87.6172999999998, 41.83112], [-87.61730017644463, 41.831130939581506], [-87.6155371703371, 41.83122000000011], [-87.61414309380095, 41.8312255298414], [-87.61233197331651, 41.83125317681943], [-87.61109, 41.83127000000015], [-87.61108295463623, 41.83126579837719], [-87.61092091613148, 41.83126102323469], [-87.6108149067486, 41.83127310382766]]]}, "properties": {"city": "Chicago", "name": "Ida B. Wells - Darrow Homes", "regionid": "403349", "geo_point_2d": [41.827281256883, -87.61295570729986], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.76434784988675, 41.87122352955194], [-87.76422348116662, 41.870030476915886], [-87.76415714640892, 41.86886869337283], [-87.76412931868559, 41.867954110058406], [-87.76414999999987, 41.8664287217472], [-87.76413198672729, 41.86563099958642], [-87.76425600021594, 41.86563099958642], [-87.76538357377348, 41.86553703572272], [-87.76545599977379, 41.86553100037272], [-87.76675600034419, 41.86553100037272], [-87.76785599978903, 41.86553100037272], [-87.76915600035925, 41.86553100037272], [-87.77045700007638, 41.86553100037272], [-87.77165699963442, 41.86553100037272], [-87.77285700009163, 41.86543100025964], [-87.77335891441409, 41.86539239146552], [-87.77415699976257, 41.86533100014647], [-87.77405838460547, 41.86720469982295], [-87.77405699964947, 41.86723099959633], [-87.77406343699658, 41.86735330649312], [-87.77415699976257, 41.869130999945824], [-87.77425699987555, 41.87033100040304], [-87.77425699987555, 41.87063099984292], [-87.77425699987555, 41.871120568250944], [-87.77327336660615, 41.870972475264985], [-87.77205226403055, 41.87090815327574], [-87.77058441976097, 41.87084120164679], [-87.76875605477835, 41.87085436197641], [-87.76747752527803, 41.87090199024006], [-87.7657717934569, 41.87107491450653], [-87.76477450996983, 41.871177402137526], [-87.76434784988675, 41.87122352955194]]]}, "properties": {"city": "Chicago", "name": "The Island", "regionid": "403326", "geo_point_2d": [41.868235906052284, -87.76916201499455], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.66540663699992, 41.82334798500004], [-87.6654035749999, 41.823142022000056], [-87.66541291499992, 41.823142022000056], [-87.66558191599992, 41.82313902200006], [-87.6660909149999, 41.82313602200007], [-87.66722791499988, 41.82315602200003], [-87.66776991599991, 41.823169023000034], [-87.6689969159999, 41.82320402200008], [-87.66918891599991, 41.82319602200004], [-87.6693049159999, 41.82319502200004], [-87.67019891599993, 41.82318402200006], [-87.67040091499989, 41.823172023000026], [-87.6714019149999, 41.82316202200007], [-87.67161791499987, 41.82315302300008], [-87.6726409159999, 41.82314402200005], [-87.67279691599991, 41.82314102200006], [-87.6738409149999, 41.823126022000054], [-87.67507091499992, 41.823104022000045], [-87.67626491599991, 41.82308602200003], [-87.67653091499993, 41.823082022000044], [-87.67684891499992, 41.823078022000054], [-87.6775969159999, 41.823070022000074], [-87.67879391599988, 41.82305002200006], [-87.67991791599988, 41.82304702200007], [-87.6809889159999, 41.823036022000046], [-87.68162991599989, 41.82303002200006], [-87.68260491599993, 41.82302802200007], [-87.68319191599991, 41.82300402200006], [-87.68350891499993, 41.82300202200007], [-87.68416091499992, 41.82299602200003], [-87.6842196409999, 41.82299610300003], [-87.68424495399992, 41.82365750500007], [-87.68428498899992, 41.82528394800004], [-87.68434178899989, 41.82707087200003], [-87.68437504899991, 41.82805258600006], [-87.6843957969999, 41.82894697300008], [-87.68442075099993, 41.82992254400005], [-87.68439398299992, 41.83098127200007], [-87.6846, 41.83260871800007], [-87.68471063399991, 41.83366288200005], [-87.6849013749999, 41.834723294000064], [-87.68493057899988, 41.835565857000056], [-87.6849299159999, 41.835566022000066], [-87.6848709039999, 41.83558547400008], [-87.68495849999992, 41.83634898600007], [-87.68509826699992, 41.83744692900005], [-87.68508452299993, 41.837864138000036], [-87.68440838199987, 41.83807664000005], [-87.68273573099991, 41.83852600900007], [-87.6806636409999, 41.83924999200006], [-87.67816714699988, 41.839998940000044], [-87.6775180589999, 41.840298519000044], [-87.6769188999999, 41.84099753800006], [-87.67609505699991, 41.84134704700005], [-87.67482184499993, 41.841796416000065], [-87.67349870299994, 41.84242053900004], [-87.6722005269999, 41.84301969800003], [-87.67157640299989, 41.84339417200004], [-87.66982885699989, 41.84396836500007], [-87.66718257399991, 41.84479220800005], [-87.66584711199991, 41.84479220800005], [-87.66583460399991, 41.84450713000005], [-87.6658206049999, 41.84324926100003], [-87.6658099999999, 41.84166193700007], [-87.66578999999987, 41.83922212300006], [-87.6657657499999, 41.838485521000045], [-87.66577119999992, 41.83848225100006], [-87.66575999999989, 41.83828147200005], [-87.6657309159999, 41.83789915900007], [-87.66570518299993, 41.83716434300003], [-87.66569719999987, 41.83666475800003], [-87.66569230099991, 41.83625328200008], [-87.6656874299999, 41.835928933000034], [-87.66568224499989, 41.835664477000066], [-87.6656763379999, 41.83528266200005], [-87.6656727139999, 41.83501815400005], [-87.6656563899999, 41.834665313000066], [-87.66564727799991, 41.83440106900007], [-87.66563849499991, 41.834048568000064], [-87.66563286699991, 41.83351951200005], [-87.66562052399989, 41.83296255800006], [-87.6656079079999, 41.832579865000056], [-87.66558895699988, 41.831843087000024], [-87.6655726199999, 41.83110791300003], [-87.66556413199991, 41.83072596200008], [-87.66555281099993, 41.83013830300006], [-87.66552784799988, 41.82902118100008], [-87.66550059099988, 41.82796352400003], [-87.66547638299993, 41.82708232800007], [-87.66547387299991, 41.826493718000044], [-87.66547866799992, 41.82605252000006], [-87.6654556489999, 41.82537601600006], [-87.66543361799994, 41.82472960500006], [-87.66542522299991, 41.82434765000004], [-87.66540886099992, 41.823848227000035], [-87.66540663699992, 41.82334798500004]]]}, "properties": {"city": "Chicago", "name": "McKinley Park", "regionid": "139408", "geo_point_2d": [41.83249629006703, -87.67445463613439], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.60593447800368, 41.78573393466706], [-87.60591724904452, 41.784989443776425], [-87.60587373604145, 41.78317585584522], [-87.60584137692383, 41.781365112064705], [-87.60582123923776, 41.77948403188011], [-87.60579267309942, 41.77767029397908], [-87.6057416421022, 41.775789813772334], [-87.60568878999275, 41.774049471235585], [-87.60565058993265, 41.772990146283206], [-87.60566204584497, 41.77299007645905], [-87.6063396924581, 41.77299000000022], [-87.6075234692364, 41.77297403018736], [-87.60863710054659, 41.772944161520535], [-87.609752276297, 41.77291311213898], [-87.61093777671208, 41.772892733702975], [-87.61219163581183, 41.772872656328374], [-87.61344781325548, 41.77285175968183], [-87.61442105820962, 41.772855466803186], [-87.61538156153725, 41.77284003350251], [-87.61538791610018, 41.773128022582554], [-87.61541291545379, 41.77461802201857], [-87.61541625047776, 41.77480366819842], [-87.61544591607576, 41.77645502216168], [-87.61547591566, 41.77742902209376], [-87.61550291600524, 41.77830102271979], [-87.6155089153825, 41.77925402258299], [-87.6155099154284, 41.77942602241763], [-87.61551382837865, 41.78013803545336], [-87.61553491658084, 41.7805780224609], [-87.61556691535816, 41.78193702305296], [-87.61560191607205, 41.78376502188206], [-87.6156587930456, 41.78556815639559], [-87.6144899160742, 41.78558102195647], [-87.61324791547939, 41.78560002193299], [-87.61193291601647, 41.78563002241652], [-87.61083491576423, 41.785669022415654], [-87.61001991596716, 41.7857000220459], [-87.60959291606906, 41.78570602232262], [-87.60833591568216, 41.785721022114785], [-87.60664191583554, 41.785734021814584], [-87.6061199160549, 41.785734021814584], [-87.60593447800368, 41.78573393466706]]]}, "properties": {"city": "Chicago", "name": "West Woodlawn", "regionid": "403353", "geo_point_2d": [41.779287436019615, -87.61065690012529], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.57295006300238, 41.713394245689194], [-87.57127779416459, 41.71144326520713], [-87.56951262191994, 41.70930647783183], [-87.56644679596872, 41.7060548440655], [-87.56578125481114, 41.705213957167786], [-87.56581045983664, 41.70456697725512], [-87.56585642385714, 41.70354873995513], [-87.56576775789698, 41.70215964014697], [-87.56567321595489, 41.70172590082571], [-87.56532749392699, 41.701153410959], [-87.56486432789167, 41.70038959309369], [-87.56439371224744, 41.69961349089109], [-87.56360273686849, 41.6983954338206], [-87.56319606617137, 41.69810569592132], [-87.56315999999988, 41.698080000000196], [-87.56282354707172, 41.697228981842215], [-87.56247614782731, 41.695752533040775], [-87.56446148399992, 41.69475986500003], [-87.56437122699992, 41.6911495890001], [-87.56428096999991, 41.685914689000036], [-87.56391994299989, 41.68374852400013], [-87.56391994299989, 41.68040901900012], [-87.56382968599988, 41.67643771600008], [-87.56364917199986, 41.667953567000026], [-87.56364917199986, 41.66723022100003], [-87.56364917199986, 41.665934880000066], [-87.56364917199986, 41.66425303500009], [-87.56355891499986, 41.65918], [-87.56599590099988, 41.65918089000008], [-87.57023794399984, 41.65919381800008], [-87.57218969599987, 41.65919985100013], [-87.57218999999989, 41.659204], [-87.57309899999987, 41.659208], [-87.57309875699977, 41.659202661], [-87.57547282499982, 41.65921000000018], [-87.57718774599988, 41.659207086000116], [-87.5807979919999, 41.659197168], [-87.58267392699986, 41.65934452400012], [-87.58377647499977, 41.65964987600016], [-87.58511049599984, 41.65998119700012], [-87.58585744499992, 41.660256769000014], [-87.58693999599988, 41.66041111100017], [-87.5885600749999, 41.660440000000165], [-87.58991305899984, 41.6603393850001], [-87.59355, 41.65908], [-87.59355813199988, 41.65907295300011], [-87.59357213399991, 41.65908158100012], [-87.59496882799992, 41.65908335100009], [-87.59691924699982, 41.65907000000014], [-87.60110195499986, 41.65909811800016], [-87.60388703299989, 41.65918], [-87.60622999999988, 41.65913841700002], [-87.60667414799977, 41.65916000000013], [-87.60964711899992, 41.659153244000095], [-87.61122651499993, 41.65905477100004], [-87.61243418399992, 41.65905], [-87.61401347099986, 41.659040095000066], [-87.6155929139999, 41.65904000000014], [-87.61661396399982, 41.65903035500014], [-87.61761399999989, 41.6590180000001], [-87.6182553580064, 41.65902343523742], [-87.61824491599987, 41.659073022000214], [-87.61822787299988, 41.65914296900005], [-87.61814291499978, 41.65947702300016], [-87.61809491599988, 41.65969302200017], [-87.61761591499986, 41.66167302300013], [-87.6169539149999, 41.66431102300008], [-87.61688041599974, 41.664636023000156], [-87.61652091499975, 41.66623402300004], [-87.61643391599995, 41.66662202200008], [-87.61588591599988, 41.6689700230001], [-87.61574091499988, 41.66958102200012], [-87.61550791599988, 41.670756022], [-87.61535391499984, 41.67153502200014], [-87.61511591599977, 41.672666022000115], [-87.6149489159999, 41.67346202300016], [-87.61472491499975, 41.67441402200015], [-87.61449491499981, 41.675331022000165], [-87.61424191499985, 41.67634202200015], [-87.61402091599987, 41.677250022000145], [-87.61380191599987, 41.67815102200009], [-87.61366691599989, 41.67880102300013], [-87.61326291599994, 41.68080402200003], [-87.61316891499987, 41.68126802200006], [-87.61298791599992, 41.68205102200001], [-87.6127719159999, 41.68286502300012], [-87.61251321786446, 41.68401875687994], [-87.61011158116455, 41.684033409830256], [-87.6081606122415, 41.6840380992448], [-87.60696, 41.68404], [-87.60697449897177, 41.68477944606926], [-87.60702686176356, 41.6854840551463], [-87.60665591579068, 41.68549002276257], [-87.60584391501163, 41.685484022124186], [-87.6045219151187, 41.685506022181606], [-87.60429091614019, 41.6855230230735], [-87.60413291609574, 41.68555702295765], [-87.6038359159954, 41.6856570220646], [-87.60340491500452, 41.68582902198202], [-87.6031099130508, 41.68596002280323], [-87.6027079151276, 41.68605502224461], [-87.6025549151985, 41.68608502203677], [-87.60235791515449, 41.68611702282457], [-87.60215791504157, 41.686143022024524], [-87.60201091575081, 41.686155022801245], [-87.60154187689754, 41.6861849591169], [-87.60151891610221, 41.68623102275407], [-87.60139291494639, 41.686529022877664], [-87.60110291595696, 41.687081022060205], [-87.60090591591303, 41.687431022033216], [-87.60087291510216, 41.687510022055385], [-87.60032791513132, 41.688530022182235], [-87.6000099159462, 41.68911702222174], [-87.59966791520789, 41.689746022829965], [-87.59872691510321, 41.69147002204535], [-87.59826691579282, 41.692348022046936], [-87.59823983575532, 41.6923998468362], [-87.599245214042, 41.692806312922585], [-87.60119276387282, 41.69285505721546], [-87.60286482281398, 41.69283843786814], [-87.60435173189288, 41.69283472092512], [-87.60667461583327, 41.69282479119643], [-87.60769869978877, 41.69281109310613], [-87.60880881892228, 41.69278523792022], [-87.60975804128681, 41.6927664717604], [-87.60949491507425, 41.69389802224855], [-87.60929191584182, 41.69488502206465], [-87.60915991594707, 41.69540402283536], [-87.6089589158108, 41.69623302210708], [-87.60859991608024, 41.697801022773994], [-87.60849091581612, 41.698223022107605], [-87.60751003816006, 41.69823555497372], [-87.60667409422295, 41.69824514713706], [-87.60614999999984, 41.698250000000236], [-87.60620808321016, 41.69899412096917], [-87.60619999999989, 41.699737384847694], [-87.6062152748554, 41.70057333295274], [-87.60623679181897, 41.70122601301297], [-87.60625999999986, 41.70193], [-87.60537317020352, 41.70194103918635], [-87.60444427609629, 41.7019526021858], [-87.60314380297615, 41.70196], [-87.60261999999987, 41.70200000000018], [-87.60263390815194, 41.702898408841726], [-87.60266202086908, 41.70345441007984], [-87.60268799818111, 41.70419658776324], [-87.60269952615754, 41.70475377195429], [-87.6028110562141, 41.70568225707579], [-87.60277265394136, 41.707485929882814], [-87.60555921098444, 41.7074407879464], [-87.60645953169974, 41.70742661298511], [-87.60645291510856, 41.707454022176826], [-87.60623991609528, 41.70837302222408], [-87.60614691575043, 41.70878302223101], [-87.60584191496562, 41.71012002296957], [-87.60576765088639, 41.71042132312205], [-87.60388703323689, 41.7104213232233], [-87.602679283938, 41.7104213232233], [-87.60128572682385, 41.710142612000304], [-87.59877732381828, 41.71004970809281], [-87.59691924716591, 41.71004970809281], [-87.59552569005177, 41.710142612000304], [-87.59515407492137, 41.7104213232233], [-87.59552569005177, 41.710885842761336], [-87.59599020908983, 41.71125745789165], [-87.59691924716591, 41.71190778424512], [-87.59756957401935, 41.712186495967984], [-87.59840570818781, 41.712465207190796], [-87.59905603504109, 41.71255811109829], [-87.6009141111934, 41.712743918913645], [-87.60193605317696, 41.712929726228786], [-87.60286509175327, 41.71311553404414], [-87.60397993714464, 41.71311553404414], [-87.6051397941709, 41.71311553406475], [-87.60509491578516, 41.71331102277228], [-87.60503691579741, 41.71356502278104], [-87.60496791600568, 41.713867022996624], [-87.60476952368292, 41.714742841283325], [-87.60444323409727, 41.71472789778142], [-87.60240000916625, 41.714770487750656], [-87.60128610288214, 41.71480284721546], [-87.60016999496511, 41.714827107761025], [-87.5988699241733, 41.71484433398201], [-87.59747635881216, 41.7148616990196], [-87.5965475709848, 41.714877040242115], [-87.59534015017212, 41.714897046858034], [-87.59394490916247, 41.71492936295095], [-87.59301461502669, 41.71495696080437], [-87.59199534506253, 41.71488070678864], [-87.59115921089405, 41.71497361019645], [-87.5903636919675, 41.71506829688233], [-87.59041325675435, 41.716460173788505], [-87.59046437001126, 41.71785194082488], [-87.59049774714573, 41.71868600379912], [-87.59056967207336, 41.72044786909882], [-87.59061182891928, 41.721573282760986], [-87.59065760776691, 41.72223858715657], [-87.59063691575189, 41.722239022102684], [-87.58856691520606, 41.722272021964024], [-87.58700491517747, 41.722268022821495], [-87.58686091595568, 41.722276022056185], [-87.58666091584276, 41.72228602278674], [-87.58662390499326, 41.72228814498676], [-87.58664988880427, 41.7223169742342], [-87.58422803088669, 41.72237382702796], [-87.58190590306242, 41.722392795210006], [-87.57930480589181, 41.72241689492511], [-87.57896302696174, 41.72242213721776], [-87.5790228818548, 41.72193951521531], [-87.57901324917704, 41.72128933121285], [-87.57908171490418, 41.720083320115286], [-87.57378619717016, 41.71441618775065], [-87.57295006300238, 41.713394245689194]], [[-87.56908684399966, 41.694390865000045], [-87.56907315899977, 41.694384664000154], [-87.56909947299981, 41.69440352000047], [-87.56914496399985, 41.694440854], [-87.56918677299964, 41.69447994200005], [-87.56924359599982, 41.69449772500019], [-87.56926407699966, 41.69450440500021], [-87.56931915499968, 41.69452513500015], [-87.56935453999986, 41.69454069100037], [-87.56946500099976, 41.69459211200045], [-87.56948862899976, 41.69459216300036], [-87.56948405799992, 41.6944763620001], [-87.56930222399978, 41.69445005400006], [-87.56925555099974, 41.694442042000105], [-87.56919823799981, 41.69442867800007], [-87.56914192199976, 41.694411595000346], [-87.56908684399966, 41.694390865000045]], [[-87.5686860359997, 41.69414614600014], [-87.5686430839997, 41.6941059170001], [-87.56860201720492, 41.69406245617494], [-87.5686150389997, 41.69417713600022], [-87.56865407799962, 41.69418940500008], [-87.56870360799984, 41.69420785800009], [-87.56888351299978, 41.6942809750002], [-87.56882080399984, 41.694243845000145], [-87.56877936399971, 41.69421775900029], [-87.56873152799992, 41.69418348000003], [-87.5686860359997, 41.69414614600014]], [[-87.58505748699982, 41.694116459000085], [-87.58506236399968, 41.69411750900035], [-87.58515795899967, 41.694116344000165], [-87.5853685789997, 41.69409068400017], [-87.58513243199988, 41.69394355900017], [-87.58500885399988, 41.693901910000186], [-87.58495284499978, 41.69388086500031], [-87.58491669799989, 41.69386495400015], [-87.58482669699991, 41.69382295400005], [-87.58480923999969, 41.69381457800006], [-87.58475733899976, 41.69378683600008], [-87.58470736299971, 41.69375575900044], [-87.58470223799982, 41.693752309000146], [-87.58452823799982, 41.69363430900016], [-87.58448552699981, 41.693603480000355], [-87.58444003599979, 41.69356614600029], [-87.58440406899975, 41.693532827000105], [-87.58421906999972, 41.69335182700005], [-87.58421208399984, 41.693344917000374], [-87.5841718539999, 41.693301964000106], [-87.58413451999965, 41.69325647300051], [-87.58410024099985, 41.69320863700011], [-87.58406916499966, 41.69315866100006], [-87.58404142299976, 41.69310676000047], [-87.5840171349998, 41.693053156000126], [-87.58400124133314, 41.6930131730001], [-87.58397283499968, 41.693065661], [-87.58394175899979, 41.693115637000055], [-87.58390747999968, 41.6931634730001], [-87.58387014599973, 41.69320896400005], [-87.58382991599976, 41.693251917000495], [-87.58378696399959, 41.69329214600036], [-87.58374147199991, 41.69332948000043], [-87.58369363599975, 41.69336375900015], [-87.58364366099981, 41.69339483600015], [-87.58359175999978, 41.69342257800013], [-87.58353815499979, 41.693446865000105], [-87.58348307699984, 41.69346759500039], [-87.58342676099979, 41.69348467900003], [-87.58336944899968, 41.69349804200016], [-87.58331138499983, 41.69350762800007], [-87.58330047099965, 41.693508997000094], [-87.58320765199979, 41.693520064000026], [-87.5832271369998, 41.69354462400047], [-87.58336410399984, 41.693671900000155], [-87.58349391099985, 41.6937752990001], [-87.58425608899975, 41.693799117000296], [-87.58428681799964, 41.69380060400015], [-87.58434538499988, 41.6938063720001], [-87.5844034489998, 41.69381595800017], [-87.58445567199983, 41.69382797400011], [-87.5845626729999, 41.69385597400047], [-87.58456776199971, 41.69385732200004], [-87.58462407799975, 41.69387440500013], [-87.58467915599986, 41.69389513500007], [-87.58473275999975, 41.69391942300012], [-87.58478466099979, 41.69394716500011], [-87.58481551799979, 41.693965814000144], [-87.58504721399966, 41.69411227100009], [-87.58505748699982, 41.694116459000085]], [[-87.56824463599978, 41.69263575900014], [-87.56819466099982, 41.692666836000136], [-87.56816607799983, 41.692682114000156], [-87.56817040499968, 41.69272019100002], [-87.5681795729998, 41.69274676800017], [-87.56824858199965, 41.692867376000024], [-87.56827457799986, 41.69291624000016], [-87.56829963463227, 41.692970953622066], [-87.56829005999973, 41.69263121900011], [-87.56828970349972, 41.69260422300037], [-87.56824463599978, 41.69263575900014]], [[-87.57928263099974, 41.692332478000175], [-87.57931961499969, 41.6923263720002], [-87.57937818099967, 41.69232060400009], [-87.57943699999973, 41.692318678000106], [-87.57949581799981, 41.69232060400009], [-87.57951662399975, 41.69232221000004], [-87.57961188999973, 41.69233067800003], [-87.5796806889999, 41.69233033900009], [-87.5796826029999, 41.69227118100035], [-87.57968837099983, 41.69221261500012], [-87.5796933649999, 41.69217946900004], [-87.57969260399975, 41.69216881900023], [-87.57969067799975, 41.69211], [-87.57969260399975, 41.69205118200018], [-87.57969837199968, 41.69199261500022], [-87.57970795799976, 41.69193455100021], [-87.57972132199981, 41.69187723900008], [-87.57973840499979, 41.691820923000044], [-87.57973979999981, 41.691816845000055], [-87.57974979999986, 41.69178784500015], [-87.57976913399985, 41.6917368440001], [-87.57979342199982, 41.69168324000011], [-87.57982116399964, 41.691631339000516], [-87.5798522409999, 41.69158136300012], [-87.57988651899979, 41.69153352700007], [-87.5799238539997, 41.69148803600012], [-87.57993101399975, 41.69147996900007], [-87.5799455199999, 41.69146052800016], [-87.57998285399978, 41.69141503600012], [-87.58002308299965, 41.69137208300003], [-87.58006603499982, 41.69133185400017], [-87.58011152699986, 41.69129452000009], [-87.58015936299982, 41.69126024100019], [-87.5801753309998, 41.69124981500004], [-87.5802333309997, 41.69121281500035], [-87.58026733899969, 41.69119216400016], [-87.58031923999972, 41.69116442300008], [-87.58037284399981, 41.691140135000026], [-87.58042792199973, 41.691119405000265], [-87.58048423799978, 41.691102322000184], [-87.58054155099964, 41.691088958000144], [-87.58059961499981, 41.691079372000054], [-87.58061103899989, 41.69107794200011], [-87.58067003899978, 41.69107094200007], [-87.58068058799968, 41.691069753000065], [-87.58068684399957, 41.691067134000114], [-87.58072109399974, 41.69105376100016], [-87.58080009399973, 41.691024761000065], [-87.58082092199984, 41.69101740500035], [-87.5808772379997, 41.691000322000086], [-87.58093455099974, 41.69098695800005], [-87.58099261499976, 41.69097737200014], [-87.58105118099989, 41.690971604000204], [-87.58110999999977, 41.690969678000044], [-87.58116881799967, 41.690971604000204], [-87.58120209399983, 41.69097440600008], [-87.58127009399979, 41.69098140600013], [-87.58129538499985, 41.69098437200001], [-87.58135344899975, 41.69099395900019], [-87.58141076199989, 41.69100732200014], [-87.58146707799992, 41.69102440500022], [-87.58152215599986, 41.69104513500016], [-87.5815757599996, 41.69106942300004], [-87.58162766099971, 41.691097165000194], [-87.58167491099981, 41.69112642700013], [-87.58170471799978, 41.69114618700017], [-87.5817360359998, 41.69111685400014], [-87.58178152799984, 41.69107952000007], [-87.5818293639996, 41.69104524100017], [-87.58187933899976, 41.69101416400035], [-87.58193123999986, 41.69098642300009], [-87.58198484499978, 41.69096213500022], [-87.5820399229999, 41.6909414050001], [-87.58209623899958, 41.690924322000015], [-87.5821535509997, 41.690910958000515], [-87.58220295299971, 41.690902555000086], [-87.58242195399978, 41.690871555000186], [-87.58243061499988, 41.69087037100009], [-87.58248918199976, 41.690864603000136], [-87.58254799999982, 41.69086267800007], [-87.5826068189999, 41.690864603000136], [-87.58266538499977, 41.69087037100009], [-87.58270751799971, 41.69087693800019], [-87.58305151799975, 41.690938938000166], [-87.5830674489999, 41.69094195800006], [-87.58312476199968, 41.69095532100001], [-87.58318107799971, 41.69097240500018], [-87.58323615599983, 41.69099313500012], [-87.58328975999966, 41.69101742200009], [-87.58334166099978, 41.691045164000066], [-87.58337208799989, 41.69106354200017], [-87.58357008799983, 41.69118854300012], [-87.58358963699976, 41.69120124100011], [-87.58363747299973, 41.69123552000019], [-87.58368296399986, 41.691272854000076], [-87.58372591699978, 41.691313083000125], [-87.5837661459998, 41.691356036000215], [-87.58380347999989, 41.69140152800007], [-87.58383775899979, 41.691449364000114], [-87.5838688359997, 41.69149933900008], [-87.58389657799987, 41.6915512400002], [-87.58392086499981, 41.691604844000196], [-87.58394159499984, 41.69165992300004], [-87.58395327199968, 41.69169692500009], [-87.58402627199976, 41.691946925000515], [-87.58403167799976, 41.69196623800007], [-87.5840450419998, 41.692023551000105], [-87.58405524594566, 41.69208255019515], [-87.58408972999959, 41.691987480000236], [-87.58410613499969, 41.69194484400008], [-87.58413042199967, 41.69189124000008], [-87.58415816399982, 41.69183933900015], [-87.58418924099973, 41.6917893630001], [-87.58422351999982, 41.69174152700005], [-87.5842608539999, 41.69169603500019], [-87.58426990599965, 41.69168586900023], [-87.58433590699971, 41.69161286900006], [-87.58436708399981, 41.69158008300019], [-87.5844100359997, 41.69153985400014], [-87.58445552799984, 41.69150252000008], [-87.5845033639998, 41.69146824100017], [-87.58455333899967, 41.691437164], [-87.58460523999979, 41.691409422000014], [-87.58465884499978, 41.691385135000395], [-87.58466294699976, 41.69138344700008], [-87.58473394699969, 41.69135444800008], [-87.5847849219999, 41.691335405000196], [-87.5848412379996, 41.69131832200047], [-87.5848985509999, 41.69130495800042], [-87.58495661499964, 41.69129537200017], [-87.58501518099969, 41.6912896040004], [-87.58504300599968, 41.69128821200029], [-87.58524600599976, 41.69128121200042], [-87.58525596699971, 41.69128136600007], [-87.58527699999978, 41.69128067800001], [-87.58533581899974, 41.69128260300009], [-87.58539438499972, 41.6912883720003], [-87.58544856999967, 41.69129719500005], [-87.58571601399977, 41.69134917200017], [-87.58579854899985, 41.69129910300039], [-87.58581533899982, 41.69128916400019], [-87.58586723999976, 41.691261423000114], [-87.58590348299983, 41.69124454100014], [-87.58598548299985, 41.691208541], [-87.5860028439997, 41.69120113500009], [-87.58605792299981, 41.69118040500033], [-87.58611423799968, 41.69116332200008], [-87.5861715509997, 41.69114995900013], [-87.58622961499971, 41.691140372000135], [-87.58623737499973, 41.691139385000135], [-87.58629437499982, 41.691132384000184], [-87.58634518199969, 41.691127603000055], [-87.58640399999975, 41.691125678000155], [-87.58646281899983, 41.691127603000055], [-87.58652138499968, 41.69113337200009], [-87.58652841599975, 41.69113432600017], [-87.5868501029998, 41.69117926100006], [-87.58686044599968, 41.691139357000495], [-87.58686232199983, 41.69113223800035], [-87.58687940499982, 41.69107592300004], [-87.58690013499987, 41.691020845000104], [-87.58692442299966, 41.6909672400002], [-87.58692662099982, 41.690962814], [-87.58694697099983, 41.6909221130001], [-87.58680929499981, 41.6907152180003], [-87.58677916399981, 41.69066666100046], [-87.58675142299963, 41.690614760000344], [-87.58672713499968, 41.690561156000165], [-87.58670640499983, 41.69050607800005], [-87.58670482999979, 41.690501396000016], [-87.58667282899971, 41.690405396000074], [-87.58665732099976, 41.69035376200024], [-87.58664395799973, 41.69029644900003], [-87.5866343709998, 41.69023838500019], [-87.58662860299987, 41.69017981900014], [-87.58662766299979, 41.69016309900019], [-87.5866246639998, 41.690099099000214], [-87.5866256039997, 41.689998182000124], [-87.58662694499981, 41.68998041800008], [-87.58663694499975, 41.68986341800017], [-87.58664137199976, 41.68982261500008], [-87.58665095799977, 41.68976455100007], [-87.5866643219998, 41.68970723800004], [-87.58668140499971, 41.68965092300009], [-87.58670213499974, 41.68959584400005], [-87.58672642299969, 41.68954224000042], [-87.58675416399969, 41.689490339000294], [-87.58676764599974, 41.68946775700003], [-87.58679664599973, 41.68942075700005], [-87.58681424099969, 41.68939336300009], [-87.58684851999986, 41.68934552700004], [-87.58688585399975, 41.6893000360001], [-87.58692608399969, 41.68925708300018], [-87.58696903599969, 41.68921685400032], [-87.58697703699973, 41.689209920000096], [-87.58694876299971, 41.689175461], [-87.58693051999971, 41.689152472000146], [-87.58689624099972, 41.689104636000096], [-87.58686516399982, 41.68905466100013], [-87.58683742199983, 41.689002760000015], [-87.58683189099972, 41.688991340000136], [-87.5864947099997, 41.68898126600037], [-87.58580124199979, 41.688978401000035], [-87.58511618099979, 41.6890075110005], [-87.58507799999981, 41.68900832200012], [-87.5850540439997, 41.68900800300018], [-87.58434104299985, 41.68898900300003], [-87.58430618099972, 41.688987397000076], [-87.58428293499976, 41.688985570000014], [-87.58353927799978, 41.68891742600005], [-87.58139663699987, 41.68876608700042], [-87.58135014299974, 41.68876158700004], [-87.58065784399976, 41.6886763810001], [-87.57996965599975, 41.6886270170001], [-87.57992892199977, 41.68862316200004], [-87.57951910099968, 41.68857494800032], [-87.57932366099985, 41.68856613100003], [-87.57931497699975, 41.688566477000066], [-87.57928146299965, 41.688762532000204], [-87.57927704099983, 41.68878644900015], [-87.57926367799968, 41.68884376200036], [-87.57924659499969, 41.6889000780004], [-87.57922586499976, 41.68895515600052], [-87.5792015769998, 41.68900876000015], [-87.57918794299981, 41.689035289000124], [-87.57912694399982, 41.689149289000326], [-87.57911283599972, 41.689174661000116], [-87.57908175899965, 41.68922463600008], [-87.57905898999972, 41.68925713700018], [-87.57890998999991, 41.68946013700047], [-87.57889847999975, 41.68947547300014], [-87.57886114599988, 41.68952096400044], [-87.57882091699982, 41.689563917000164], [-87.57877796399981, 41.689604146000036], [-87.57876319999976, 41.68961681400001], [-87.5786261999997, 41.68973181400029], [-87.57859547199983, 41.689756480000206], [-87.57854763599968, 41.68979075900011], [-87.57849766099973, 41.68982183600046], [-87.57844575999968, 41.68984957800045], [-87.57842117199978, 41.68986124400017], [-87.57818217199981, 41.68997024300022], [-87.57815315499973, 41.689982865000374], [-87.5781325609998, 41.68999109900017], [-87.57811859499971, 41.69003607800007], [-87.57810237499972, 41.69008005900006], [-87.57807491699984, 41.69014892900008], [-87.57807042999973, 41.690233964000456], [-87.5781121679998, 41.6903938690001], [-87.57812404199981, 41.69044555100011], [-87.57812584399979, 41.69045486500017], [-87.57815813899988, 41.690625586000166], [-87.57817567799982, 41.69068323800015], [-87.57818468599986, 41.690719920000156], [-87.5782056859999, 41.69081392000048], [-87.57821004199982, 41.69083455100014], [-87.57821962799973, 41.69089261500015], [-87.57822539599985, 41.69095118200011], [-87.57822732199983, 41.69101000000018], [-87.57822561299959, 41.69106540700006], [-87.5782139049996, 41.69125507900029], [-87.57823422699978, 41.691311565], [-87.57823959499986, 41.69132692200047], [-87.57825667799986, 41.69138323800051], [-87.57827004199973, 41.69144055100019], [-87.57827975649978, 41.69149959950015], [-87.57829088499986, 41.69158558400038], [-87.57829639599976, 41.69164218200009], [-87.57829832199975, 41.69170100000015], [-87.57829639599976, 41.69175981900014], [-87.57829062799973, 41.69181838500019], [-87.57828104099983, 41.69187644900001], [-87.57827734899988, 41.69189408500006], [-87.57823990999972, 41.69206439900011], [-87.57822903799989, 41.692206879000175], [-87.57843596599977, 41.69219381400008], [-87.57851368999972, 41.69218517800043], [-87.5785541809997, 41.692181604000304], [-87.57861299999978, 41.69217967800032], [-87.57867181799983, 41.692181604000304], [-87.5787096589996, 41.69218488800005], [-87.57885765999966, 41.692200887000396], [-87.57887838499978, 41.69220337200016], [-87.57893644899988, 41.69221295800023], [-87.57899376199984, 41.692226321000184], [-87.57902027199972, 41.692233862000194], [-87.57911327199977, 41.692261862000024], [-87.57914307799976, 41.69227140500037], [-87.57919815599969, 41.69229213500013], [-87.57925175999976, 41.69231642300018], [-87.57928263099974, 41.692332478000175]], [[-87.57133473999974, 41.68921131700013], [-87.57142473999973, 41.6891143170001], [-87.57144808299981, 41.68909008300004], [-87.57147129799986, 41.689068340000105], [-87.57078300699975, 41.68910129300008], [-87.57075054499964, 41.68910226000008], [-87.57070207699972, 41.68910282900003], [-87.57078122199978, 41.68936600900013], [-87.57078867899978, 41.68939223800008], [-87.5708020419998, 41.68944955100012], [-87.57081162799962, 41.689507615000124], [-87.57081739699984, 41.68956618200009], [-87.57081770799977, 41.68957113900006], [-87.57084470699978, 41.69002113900016], [-87.57084439599986, 41.69013381900025], [-87.57083862799992, 41.690192385000124], [-87.57082904099966, 41.690250449000125], [-87.57081567799979, 41.69030776200016], [-87.57079859499979, 41.69036407800002], [-87.57077786499977, 41.69041915600015], [-87.57075357699979, 41.69047276000014], [-87.57072714699966, 41.69052238100007], [-87.57060549199971, 41.69073452100021], [-87.57057375899974, 41.69078563600009], [-87.57053947999984, 41.690833472000136], [-87.57053334499977, 41.69084137700035], [-87.57033234499984, 41.69109737700021], [-87.57030114599976, 41.6911349640004], [-87.57026091699971, 41.69117791700013], [-87.57021796399981, 41.691218146], [-87.57020317199974, 41.69123083700024], [-87.57015085199984, 41.69127475200024], [-87.57018206999986, 41.69127257900015], [-87.5706226099998, 41.69123155300013], [-87.57064739749985, 41.691229590000056], [-87.57098720699977, 41.691207481000085], [-87.57097240499988, 41.691167078000035], [-87.5709553209998, 41.69111076200017], [-87.57094195799985, 41.69105344900014], [-87.57093237199976, 41.69099538500013], [-87.5709266029998, 41.69093681900008], [-87.57092562999972, 41.69091937200032], [-87.57091162999983, 41.690615371000305], [-87.57091260399982, 41.69051518100018], [-87.57091837199984, 41.69045661500013], [-87.57092795799984, 41.690398551000484], [-87.57093651299984, 41.69035998900013], [-87.57112051299961, 41.68960898900035], [-87.57112532199992, 41.68959023800012], [-87.57114240499992, 41.68953392300018], [-87.57116313499985, 41.689478844000504], [-87.5711874229998, 41.689425240000155], [-87.5712151649997, 41.68937333900003], [-87.5712462409998, 41.689323364000074], [-87.5712805199997, 41.68927552800038], [-87.57131785399976, 41.689230036000524], [-87.57133473999974, 41.68921131700013]], [[-87.57309620799973, 41.68918380400025], [-87.57309760399968, 41.689141182000455], [-87.57309963599985, 41.689120546000524], [-87.57302190599962, 41.689114864000054], [-87.57309620799973, 41.68918380400025]], [[-87.58893529199966, 41.6867489460004], [-87.59157154099975, 41.68670973100045], [-87.59360044599984, 41.68670168500013], [-87.59365658899979, 41.686703217000435], [-87.59384005899967, 41.686713964], [-87.59408705499979, 41.68634666900042], [-87.59356371999965, 41.68617683500041], [-87.59042694099982, 41.685095646000036], [-87.58990921799969, 41.68492669400009], [-87.58964043699976, 41.684838438000426], [-87.58957906099974, 41.6848157800005], [-87.58951949599961, 41.684788720000036], [-87.58946205499974, 41.68475740100003], [-87.5894070429998, 41.684721989000295], [-87.58935104299971, 41.68468298900009], [-87.58928692699972, 41.68463392100011], [-87.58917607999987, 41.68452307299999], [-87.58908616499973, 41.684394661000205], [-87.58901991399973, 41.684252586000085], [-87.58899660399976, 41.684178811000116], [-87.58898260299976, 41.68412681100008], [-87.58896533999959, 41.68404916600013], [-87.58895503199979, 41.68397060100011], [-87.5889440319996, 41.6838436], [-87.58894117299981, 41.68373616300026], [-87.58897057499965, 41.68285041800043], [-87.58891655499981, 41.68289761600007], [-87.58885875799966, 41.682940107000476], [-87.58879759499983, 41.68297759000011], [-87.58873349799977, 41.6830097990002], [-87.58866692099977, 41.683036507000104], [-87.58859833399978, 41.68305752500042], [-87.5884643339998, 41.683093524000114], [-87.5883871649997, 41.6831106590002], [-87.5882249649998, 41.68312428100008], [-87.58719887899971, 41.683110549], [-87.58661418499982, 41.68313254900005], [-87.58657531199991, 41.68344206700009], [-87.58656865899988, 41.683486166000094], [-87.58653934799968, 41.68360467800013], [-87.58652234899988, 41.68365767700016], [-87.58649891699972, 41.68372216200008], [-87.58647063699968, 41.6837846720001], [-87.58643767399971, 41.68384484400046], [-87.5864002189997, 41.683902329000155], [-87.58635849199989, 41.683956790000124], [-87.58632770599974, 41.683993932000185], [-87.58627645099985, 41.684050359000146], [-87.58622060099978, 41.68410224200026], [-87.58616055799969, 41.68414920900015], [-87.58609675199975, 41.68419092200014], [-87.58602964299969, 41.68422708000047], [-87.58582564399987, 41.684326081000044], [-87.58570683099988, 41.684373619000425], [-87.58463383099975, 41.68471662000011], [-87.58458785099977, 41.68472998000016], [-87.58437784999971, 41.68478498000009], [-87.5842274289997, 41.68481098300002], [-87.58411289199982, 41.68482088100013], [-87.58405685099979, 41.68483177800005], [-87.58339299699975, 41.685026496], [-87.58288524199975, 41.68521311400051], [-87.5828247629998, 41.685232943000116], [-87.58142561099979, 41.68563740000009], [-87.58004845899968, 41.68601785600013], [-87.57992980299966, 41.6860421710001], [-87.57956028099981, 41.68609226200043], [-87.5794195019998, 41.68613811300005], [-87.5793099389998, 41.68616631200034], [-87.57915666299971, 41.68619562700018], [-87.57899406599977, 41.68623822900015], [-87.57896226399971, 41.68624953400011], [-87.57962005799978, 41.68633072200008], [-87.58262184399982, 41.686551092000016], [-87.58266734699986, 41.68655559800004], [-87.58372112699973, 41.686687081000166], [-87.58499514599971, 41.68671909900051], [-87.58631586099973, 41.68676567800019], [-87.58664299999971, 41.68676567800019], [-87.58670238499965, 41.686767640000355], [-87.58709365599971, 41.68679353400018], [-87.5872933279998, 41.686799286000124], [-87.58822335499984, 41.68677110300019], [-87.58893529199966, 41.6867489460004]], [[-87.58903034099964, 41.681100834000084], [-87.58904425699981, 41.68103599500014], [-87.58906225699988, 41.680964995000124], [-87.58908386899971, 41.68089264600008], [-87.58911147399978, 41.68082236500005], [-87.58914487699968, 41.68075464800022], [-87.58918384299982, 41.6806899710001], [-87.58922809799982, 41.68062879100001], [-87.58927732799982, 41.68057153900008], [-87.58933118799965, 41.680518619000495], [-87.58936218799974, 41.680490619000125], [-87.58938692699982, 41.680469079000304], [-87.58951533899979, 41.68037916400006], [-87.58965741299991, 41.68031291400008], [-87.58980883399975, 41.68027234100018], [-87.58993043399968, 41.68025934200027], [-87.59000843399983, 41.68025634200002], [-87.59008491099975, 41.680256655000484], [-87.59016108399983, 41.68026346400008], [-87.59023640299986, 41.6802767200001], [-87.59031032399974, 41.68029632800013], [-87.59038231099986, 41.68032214500004], [-87.59159776999988, 41.680817332000075], [-87.59189927099969, 41.68089704800009], [-87.59217880499982, 41.680954263000125], [-87.59295075799983, 41.681044800000066], [-87.59300334399973, 41.681052549000086], [-87.59321691499983, 41.68108585099999], [-87.59336441299976, 41.68101591400016], [-87.59352423099976, 41.68097394200042], [-87.59358362899987, 41.68096454300009], [-87.59372299999968, 41.68095367800016], [-87.59377077099974, 41.68095494700036], [-87.59386477099979, 41.680959948000506], [-87.59393121099977, 41.68096552800008], [-87.5939970109998, 41.68097629600008], [-87.59406176399975, 41.68099218400015], [-87.59412507399983, 41.681013096000285], [-87.59418655199984, 41.68103890300002], [-87.59424581899977, 41.681069445000055], [-87.59441479599973, 41.681074980000076], [-87.59456891399978, 41.68100874700019], [-87.59461641399989, 41.680989913000225], [-87.59477103099981, 41.68094900100004], [-87.59472650299976, 41.68093091200005], [-87.59459304499964, 41.680868392000036], [-87.59448169199985, 41.68080461400009], [-87.59435192699968, 41.680713921000134], [-87.5942384329996, 41.68059989000037], [-87.59416578699981, 41.68051170600019], [-87.59411883999965, 41.680449382000184], [-87.59407747499979, 41.68038322100012], [-87.5940420029997, 41.68031372200014], [-87.59401269099986, 41.68024140900011], [-87.59398976099973, 41.680166826000075], [-87.59397338499988, 41.68009053600016], [-87.59395642899982, 41.67998690800011], [-87.59394467799981, 41.679842], [-87.59395113499974, 41.67973442], [-87.59396113599976, 41.67965142000024], [-87.59397237499988, 41.67958146800013], [-87.59398908599977, 41.679512618000125], [-87.59401116499971, 41.67944529700015], [-87.59403847599977, 41.67937992300006], [-87.59407084699987, 41.679316901], [-87.59410800599993, 41.67925112000016], [-87.59415033899968, 41.679184020000115], [-87.59419842299977, 41.67912091200004], [-87.59425188199984, 41.67906228900007], [-87.59431029999969, 41.679008606000515], [-87.59437322299975, 41.678960281000165], [-87.5944401609997, 41.67891769100019], [-87.59451059199975, 41.6788811660001], [-87.59458396899979, 41.678850992000086], [-87.59464252399992, 41.67883007100005], [-87.59478883399969, 41.67879134100004], [-87.59494499999981, 41.67877767800007], [-87.59499099999978, 41.67877767800007], [-87.59514716599993, 41.67879134100004], [-87.59518008099981, 41.678797780000124], [-87.5952730799996, 41.67881778000019], [-87.59536741299958, 41.678843503000095], [-87.59589141299988, 41.67901750300028], [-87.59597036499983, 41.67904791300013], [-87.59628836499977, 41.67918791400008], [-87.59634872199968, 41.6792172210001], [-87.59646198599974, 41.679277538000385], [-87.59660518699977, 41.679329672000264], [-87.5971185529997, 41.67949690400007], [-87.59723162399982, 41.679542425000186], [-87.59732411499982, 41.67958716700052], [-87.59740890899965, 41.679588856000386], [-87.59748668599968, 41.67959378300036], [-87.59756374499986, 41.67960542400016], [-87.59763950599982, 41.679623694000256], [-87.59771340099981, 41.679648454000166], [-87.59778487499986, 41.67967951800007], [-87.59791336699983, 41.679742116000114], [-87.59846655799976, 41.67990670200004], [-87.59855374499969, 41.67989662500018], [-87.5986288499997, 41.67989111900008], [-87.5987041519997, 41.67989191500015], [-87.59877912299962, 41.67989900800012], [-87.59885323799978, 41.67991234900006], [-87.59892597699977, 41.67993184400017], [-87.59899297699981, 41.679952844000134], [-87.59906214099985, 41.67997766900014], [-87.59912904799988, 41.68000805800043], [-87.59919325099978, 41.68004380800037], [-87.59925431999986, 41.680084680000206], [-87.59928522499966, 41.680107246000475], [-87.59929019399962, 41.6792752420002], [-87.5992751359997, 41.67919734600015], [-87.5988414929997, 41.678997804000154], [-87.59875933899967, 41.67895583600006], [-87.59865959199983, 41.67888899100013], [-87.59853259199981, 41.67879099100019], [-87.59842792099967, 41.67869622000008], [-87.59836092099974, 41.678625220000065], [-87.59831715899985, 41.678575272000366], [-87.59827720199984, 41.67852223100016], [-87.59824126799978, 41.67846638600006], [-87.59820955199983, 41.678408042000214], [-87.59818222899979, 41.67834751600036], [-87.59813553199966, 41.67823297700021], [-87.59808967599967, 41.678186677000106], [-87.59804740699983, 41.67813708100009], [-87.59800896199982, 41.67808446499999], [-87.59797455599991, 41.678029124000304], [-87.59794437999972, 41.677971367000055], [-87.59790237999972, 41.67788336700006], [-87.59786891399979, 41.677803587000206], [-87.59784015699982, 41.67770855400006], [-87.59782215699975, 41.677634553000075], [-87.59781034099974, 41.677578165000384], [-87.59780638699986, 41.67753297400011], [-87.5977904909997, 41.67751092500015], [-87.59774299899968, 41.67743782500014], [-87.59767291299974, 41.677317564000404], [-87.59742319999978, 41.67695779200012], [-87.5973707529997, 41.67687244400052], [-87.59731536199965, 41.67687047300045], [-87.59720636099978, 41.67686347300004], [-87.59710783399979, 41.67685165900013], [-87.59700939899973, 41.67682853000008], [-87.59693524499973, 41.676806642000045], [-87.59679073999975, 41.676795804000044], [-87.5966410419997, 41.676771760000186], [-87.59646404199974, 41.676727760000276], [-87.59636457099984, 41.676696815000106], [-87.59616512999976, 41.67662184800004], [-87.59610248899965, 41.67661145400007], [-87.59594641399971, 41.67657008600019], [-87.59591492599967, 41.67655795100017], [-87.59580192599984, 41.67651195200018], [-87.59569133899974, 41.676457836000324], [-87.59557077799988, 41.676374433000085], [-87.59520851699976, 41.67607739600011], [-87.59496199399969, 41.676055932000104], [-87.59483423699962, 41.676035466000194], [-87.59452548099974, 41.67596289900014], [-87.59402758899974, 41.675982617000166], [-87.59394951499976, 41.67598231800024], [-87.59380151599962, 41.67597531800037], [-87.59368783499971, 41.67596266000003], [-87.59362706999966, 41.67594976700012], [-87.59345406899982, 41.675906767000114], [-87.59336341299978, 41.67587908600022], [-87.59329298199972, 41.675850016000126], [-87.59273101599976, 41.67558968700017], [-87.59241208399985, 41.67546327100019], [-87.59227128499974, 41.675438536000186], [-87.59181451299978, 41.67539285800014], [-87.59174783399978, 41.675383659], [-87.5916398519998, 41.675357654000415], [-87.59108985199978, 41.67518865500038], [-87.59104641399972, 41.67517408600015], [-87.5909097259998, 41.67511092100017], [-87.59083153399972, 41.675066494000134], [-87.5907842199998, 41.67505889800021], [-87.5906200779999, 41.67501510600014], [-87.59054074199973, 41.67498512600015], [-87.59047850299972, 41.67495886400029], [-87.59041841199982, 41.674928004000044], [-87.59036080599975, 41.674892721000056], [-87.59030601099985, 41.67485321300016], [-87.59025433599973, 41.674809704000026], [-87.59004435499978, 41.67461897600005], [-87.58991429099977, 41.67457505800015], [-87.58984651299984, 41.67454908100022], [-87.58978105099983, 41.67451772100047], [-87.5897183309997, 41.674481185000104], [-87.58960233099971, 41.67440718500002], [-87.58950792699962, 41.674337921000514], [-87.58941831199975, 41.67425147200002], [-87.58931097599977, 41.67413251800012], [-87.58923982199983, 41.674117078000094], [-87.58855539599968, 41.67410824700016], [-87.58848134399986, 41.674104233000364], [-87.58840787399983, 41.67409413200015], [-87.58833548599986, 41.6740780110005], [-87.58826467199978, 41.6740559810001], [-87.58819591499979, 41.67402819200002], [-87.58807891499961, 41.67397519200008], [-87.58801079899975, 41.67394078200043], [-87.58794584499978, 41.673900720000134], [-87.58788452199978, 41.673855296000106], [-87.58782727099984, 41.67380483600037], [-87.58777450399982, 41.67374970400042], [-87.58772660099986, 41.67369029700022], [-87.58768390699979, 41.67362704300014], [-87.58759290699972, 41.67347904300015], [-87.5875550749997, 41.67341109500039], [-87.58752325599971, 41.6733401310002], [-87.58749768599975, 41.67326668400007], [-87.58747855699984, 41.673191303000145], [-87.5874660119998, 41.6731145500001], [-87.58744501199973, 41.672938550000104], [-87.5874392269997, 41.672863410000105], [-87.58743975299974, 41.672788049000175], [-87.5874465859997, 41.672712997000176], [-87.5874596799998, 41.67263878100026], [-87.5874789409997, 41.67256592100027], [-87.58751394099986, 41.67245292100015], [-87.58754219799978, 41.6723760180001], [-87.58757670599981, 41.67230171000016], [-87.58761722899969, 41.67223050400013], [-87.58766349099977, 41.67216288500014], [-87.58762164599979, 41.67200934100004], [-87.58760432199969, 41.67193826800012], [-87.5875928309998, 41.67186602300045], [-87.58758724999969, 41.67179308300045], [-87.58758761599982, 41.671719931000084], [-87.58759392599981, 41.67164705], [-87.58762203599981, 41.67142698700018], [-87.58762369499978, 41.671162361000135], [-87.58763076599982, 41.67108436500007], [-87.58760298199974, 41.67105794400011], [-87.58752498199969, 41.670973943], [-87.58748532699968, 41.67092824700013], [-87.58744883399979, 41.67087998700004], [-87.58741566899978, 41.670829384000385], [-87.58738598199973, 41.67077666400028], [-87.5873329809997, 41.67067466500011], [-87.58728591299982, 41.67056758700013], [-87.58727063999979, 41.67052184400016], [-87.58722864099977, 41.67038384400011], [-87.58720334099982, 41.670278166000074], [-87.58719009099968, 41.67014924000006], [-87.58714418199972, 41.668634257000384], [-87.5871031509997, 41.668264073000316], [-87.5870981869999, 41.66813475000036], [-87.58710518699978, 41.667926750000206], [-87.58711834099981, 41.667800835], [-87.58716119499987, 41.66764328500011], [-87.58720647699981, 41.667524155000194], [-87.5872374099998, 41.6674394040002], [-87.58718173799988, 41.66736856500011], [-87.58714973799974, 41.66732256500022], [-87.58710798299974, 41.667256609000084], [-87.58707210599978, 41.66718728100004], [-87.58704237499963, 41.667115102000196], [-87.58701901599984, 41.66704061800043], [-87.58700220399977, 41.66696438800016], [-87.58699206599974, 41.66688698800015], [-87.58698867799964, 41.66680900000027], [-87.58698887799991, 41.66675771700037], [-87.58697047699981, 41.66675037900012], [-87.58598555999978, 41.6663893390001], [-87.58592864899977, 41.66641781800005], [-87.58586978399987, 41.66644200300036], [-87.58580928899974, 41.66646176000014], [-87.5857474969998, 41.66647698000015], [-87.58568474699975, 41.666487581000055], [-87.58563174699972, 41.66649458000018], [-87.58556584099968, 41.666500826000174], [-87.58549965299981, 41.66650220700002], [-87.58543354399976, 41.66649871599999], [-87.58536786999981, 41.66649037000019], [-87.5853029879997, 41.66647721600053], [-87.58518698699977, 41.66644921600016], [-87.58509041299979, 41.66642008700033], [-87.58502626899973, 41.66639390000039], [-87.58427779899986, 41.66605413900025], [-87.58406532099968, 41.665958019000115], [-87.5838818039998, 41.66589521500024], [-87.58390280299984, 41.66597358600007], [-87.58393475599982, 41.66602248100029], [-87.58396157499972, 41.666074370000054], [-87.5839829809998, 41.66612871600012], [-87.58399875299979, 41.666184956000144], [-87.58409876499988, 41.66629846400012], [-87.58417327399975, 41.666395637000264], [-87.58425027399973, 41.666511637000085], [-87.58428949699969, 41.666576487000484], [-87.58432311299971, 41.66664441400015], [-87.5843508829997, 41.666714932000126], [-87.58437260799981, 41.666787540000094], [-87.58438813499968, 41.66686172200019], [-87.58443240799969, 41.66714541400022], [-87.58448776499993, 41.66720135400005], [-87.58453780099971, 41.66726210000046], [-87.58458209999974, 41.66732714800015], [-87.5846202949998, 41.667395958000164], [-87.58465206899976, 41.667467958000074], [-87.58467715699979, 41.667542552000036], [-87.58469535299979, 41.667619120000154], [-87.58479100599979, 41.66810497700012], [-87.5848063219998, 41.668270000000184], [-87.58479241699965, 41.66842752600007], [-87.58478517499985, 41.66846788500048], [-87.58475847699975, 41.66856691500015], [-87.5848446029997, 41.669069125000135], [-87.58505343499982, 41.66995049100005], [-87.58547378599971, 41.671836366000036], [-87.58547935699971, 41.6718632970004], [-87.58575150799976, 41.67328856600011], [-87.58576632199966, 41.673451], [-87.58575763499984, 41.673575696000064], [-87.58575063499968, 41.673625695000375], [-87.58573858699975, 41.67369300200018], [-87.58572145999968, 41.673759200000504], [-87.58569935299981, 41.67382390400017], [-87.5856723929997, 41.673886742000036], [-87.58564073799975, 41.67394735000026], [-87.58560456899978, 41.67400537800044], [-87.5855812099997, 41.67404087800018], [-87.58571716599987, 41.674054341000144], [-87.5858495089998, 41.67408821200019], [-87.5859115089996, 41.674109212000154], [-87.58598158499971, 41.674136260000076], [-87.58604915999987, 41.674169061000036], [-87.58611376199981, 41.674207387000195], [-87.58617493999972, 41.67425097000016], [-87.58623226799985, 41.67429950700017], [-87.58627026799988, 41.67433450700041], [-87.58632301299971, 41.67438729700034], [-87.58637119899979, 41.67444427800016], [-87.58641449599982, 41.674505058000044], [-87.58645260499961, 41.674569219000105], [-87.58648526299972, 41.67463631800007], [-87.58650926299967, 41.674691318], [-87.58653416299987, 41.674754853000195], [-87.58655417299984, 41.67482009300026], [-87.58656917999984, 41.67488666300026], [-87.58657909499966, 41.67495417900016], [-87.58658386299972, 41.675022252000204], [-87.5865974649997, 41.675447558000045], [-87.58669583599983, 41.6762052060002], [-87.58670332199992, 41.67632100000019], [-87.58670217499981, 41.67636641500013], [-87.58669317399979, 41.67654441600003], [-87.5866868559999, 41.676614638000146], [-87.58667505699972, 41.67668414900031], [-87.58665784799979, 41.67675252200019], [-87.58663533599972, 41.676819337000474], [-87.58660765799992, 41.676884184000066], [-87.5865749859997, 41.67694666200009], [-87.58653751999978, 41.677006390000415], [-87.5864965199996, 41.67706639000005], [-87.5864366869998, 41.67714333000002], [-87.58648155199987, 41.677196360000146], [-87.58652239099986, 41.67725254900044], [-87.5865589809998, 41.677311594000436], [-87.58659112399978, 41.67737317300047], [-87.58661864499983, 41.67743695100007], [-87.58664139499976, 41.677502582999985], [-87.58667739499991, 41.67762258300015], [-87.58670165899979, 41.67772483400017], [-87.58670922999968, 41.677776499000274], [-87.58673563599974, 41.67800221000003], [-87.58675580899965, 41.67804827900038], [-87.58678396599984, 41.67812101600009], [-87.58680571399977, 41.67819592000007], [-87.58682089099969, 41.67827242600036], [-87.58682938199969, 41.678349959000116], [-87.58683938199971, 41.67850195900008], [-87.58684118599987, 41.67857664700004], [-87.58683678499968, 41.67865122800008], [-87.58682620799979, 41.678725186000165], [-87.58680952899974, 41.67879801000014], [-87.5867868629999, 41.67886919900008], [-87.58675836699977, 41.67893826100012], [-87.58669736699979, 41.6790702610002], [-87.58666767299981, 41.67913302600016], [-87.58663178599987, 41.6791924680001], [-87.58659007299968, 41.67924797600008], [-87.5865429619998, 41.67929898300012], [-87.5865837999997, 41.679352415000096], [-87.58662045699973, 41.679408797000505], [-87.58665271999982, 41.679467803000236], [-87.5866804019998, 41.67952909300015], [-87.58670334399991, 41.67959230900006], [-87.58672034399964, 41.67964530900017], [-87.58674965999975, 41.67976383400012], [-87.58676324299982, 41.67992847500015], [-87.58674033099977, 41.68111252400034], [-87.58714183399988, 41.68109637100016], [-87.58719381599983, 41.68109468300019], [-87.58768797399986, 41.68109293400032], [-87.58798804399973, 41.6810442740002], [-87.58811076099975, 41.68103292900024], [-87.58823776099969, 41.68102992800026], [-87.58825899999985, 41.68102967800005], [-87.58841516599963, 41.6810433400003], [-87.58854340199974, 41.68107583100005], [-87.58858540199978, 41.681089832000055], [-87.5886491799998, 41.68111381300018], [-87.58871095999977, 41.681142554000516], [-87.58877038599971, 41.681175891000166], [-87.5888271169998, 41.681213632], [-87.58888082799989, 41.681255560000444], [-87.58892858199975, 41.68129604000007], [-87.5890130999997, 41.68138099], [-87.58901698999986, 41.68123331900012], [-87.58903034099964, 41.681100834000084]], [[-87.56752422199968, 41.66351838300005], [-87.56751238799977, 41.663688976000046], [-87.56773158899988, 41.663478585000135], [-87.56752422199968, 41.66351838300005]], [[-87.57003295599976, 41.66254791200013], [-87.56998072499981, 41.66258656300004], [-87.57003957699979, 41.6625733910002], [-87.57016243699964, 41.66255469200034], [-87.57062543699986, 41.66251669200004], [-87.57070981199963, 41.66251374300042], [-87.57170115599969, 41.66252566300016], [-87.57240678599972, 41.66252370000008], [-87.57313407499984, 41.662510819], [-87.5732300169998, 41.66251424500028], [-87.57343264083599, 41.66253234556488], [-87.57362660999974, 41.662549673], [-87.57404164199981, 41.6625807690001], [-87.5734104706631, 41.6623663450161], [-87.57306071599974, 41.662247525000055], [-87.57304241399991, 41.66224108600015], [-87.57289763299964, 41.662173261000135], [-87.57281992799979, 41.66212768600018], [-87.57279718299976, 41.6621139240001], [-87.57273418699988, 41.66207216900011], [-87.57267491699969, 41.66202527300014], [-87.57261979299975, 41.661973569000125], [-87.57256920299993, 41.661917421000105], [-87.5725235029997, 41.66185722500027], [-87.57248301699981, 41.66179340500015], [-87.57243386899972, 41.66170814900037], [-87.5723979739997, 41.661639148000084], [-87.57236817499981, 41.66156730300009], [-87.57234469599982, 41.661493153], [-87.57232771099966, 41.6614172510001], [-87.5723173479997, 41.66134016500012], [-87.57231368499974, 41.66126247300009], [-87.57231674799989, 41.66118475400014], [-87.57238932399979, 41.66030866000013], [-87.57097667699979, 41.66166127000013], [-87.57062310799994, 41.66202068300048], [-87.57058315599974, 41.662058873000085], [-87.57009915599983, 41.662493873000194], [-87.57003295599976, 41.66254791200013]]]}, "properties": {"city": "Chicago", "name": "Lake Calumet", "regionid": "403386", "geo_point_2d": [41.686056575618835, -87.58727797688479], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.64232017099991, 41.89275199600006], [-87.64201372299992, 41.89245144100004], [-87.64241405799991, 41.89244608400007], [-87.64300620399989, 41.89243816100008], [-87.6451910739999, 41.892398507000046], [-87.64664811599992, 41.89238044900008], [-87.64818735899989, 41.89235899000004], [-87.64980146999993, 41.89268033700005], [-87.65085873799987, 41.89265836800006], [-87.65192000099994, 41.89253679200004], [-87.65290537599991, 41.892363853000056], [-87.65378080799988, 41.89295725900006], [-87.65638822099992, 41.89475030700003], [-87.65726436999994, 41.89538530800006], [-87.65842021999991, 41.896206054000054], [-87.6584206449999, 41.89620635600004], [-87.6565259359999, 41.89623290200006], [-87.65409836699989, 41.89627848100007], [-87.65320920399992, 41.89630170900005], [-87.6523156499999, 41.89632], [-87.65142691299988, 41.89632589400008], [-87.6503761029999, 41.896355974000066], [-87.6493220699999, 41.89637532500005], [-87.6481057059999, 41.8964], [-87.64657039699992, 41.89640936800003], [-87.64447610799992, 41.89644624100003], [-87.644476, 41.896462], [-87.64414144799989, 41.89645780500006], [-87.64413008599989, 41.89536442100005], [-87.64411301199992, 41.89440823900008], [-87.64399348899988, 41.894135045000034], [-87.6436519949999, 41.89377647700006], [-87.64331050199989, 41.89341790800005], [-87.64284948599992, 41.89319593800007], [-87.64232017099991, 41.89275199600006]]]}, "properties": {"city": "Chicago", "name": "River West", "regionid": "403297", "geo_point_2d": [41.894493093867446, -87.64970909083841], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.62933196183218, 41.87570186276893], [-87.62877596425398, 41.87570000000014], [-87.62807918562673, 41.87570000000014], [-87.62768795520161, 41.87564139628191], [-87.62767162014893, 41.87508131379822], [-87.62764, 41.87361849051251], [-87.62762095808745, 41.87257221566549], [-87.62761, 41.87228000000013], [-87.6282882971345, 41.87228658540916], [-87.62891512378917, 41.872284605415835], [-87.62940420815613, 41.87227408818235], [-87.63010074953722, 41.87224146577873], [-87.63062, 41.87223], [-87.63062180022808, 41.872920889492455], [-87.6306485839292, 41.874035091456925], [-87.63067410153947, 41.87522212774868], [-87.63069771240305, 41.875690286481614], [-87.63031002809765, 41.87569784258634], [-87.62933196183218, 41.87570186276893]]]}, "properties": {"city": "Chicago", "name": "Printers Row", "regionid": "403343", "geo_point_2d": [41.87398251659355, -87.62915426807518], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.75269, 41.80014], [-87.75267403099988, 41.79965295400006], [-87.75264205099988, 41.798677546000064], [-87.75261090899993, 41.79794727500006], [-87.75255999999989, 41.79706], [-87.75203297099993, 41.79718685800008], [-87.75085884299992, 41.79737143100003], [-87.74924062999989, 41.79766843700007], [-87.74843491699993, 41.797798452000045], [-87.7469779179999, 41.79801094900005], [-87.74599805599988, 41.79815394200005], [-87.74468715999987, 41.79833368100008], [-87.74346726899992, 41.79850259700003], [-87.74293, 41.79858], [-87.74283533799989, 41.79802493600004], [-87.74278624199988, 41.79704787300004], [-87.74275642899993, 41.79615358000006], [-87.74272934199989, 41.795422240000036], [-87.7427008409999, 41.794771941000064], [-87.74253845699988, 41.79425722200004], [-87.7423326349999, 41.793880607000055], [-87.74221922399991, 41.79327617800003], [-87.7419996009999, 41.79303456700005], [-87.74195, 41.79298], [-87.7444456729999, 41.79291110800005], [-87.74639823999988, 41.79289254400003], [-87.74835067899993, 41.79287234000008], [-87.7519318849999, 41.79283139600005], [-87.75429221599988, 41.79277471400007], [-87.75673246799991, 41.79275], [-87.7590909869999, 41.79274104100004], [-87.7622, 41.79271], [-87.76216552399987, 41.792001329000065], [-87.76208807299989, 41.79029615100006], [-87.76202211399992, 41.78858984800006], [-87.76199108399992, 41.787774218000024], [-87.76195381599987, 41.786799207000044], [-87.76191893299993, 41.78591706100006], [-87.76191824499992, 41.785916322000034], [-87.76189721499989, 41.78541405400005], [-87.7618848579999, 41.78500995800004], [-87.76189, 41.78501], [-87.76203387299994, 41.78499401400005], [-87.76218435399993, 41.78492505700006], [-87.76364903999988, 41.78492505700006], [-87.76600881199991, 41.78468094200007], [-87.76682252599993, 41.78451820000003], [-87.7683685839999, 41.78419271400008], [-87.77032149799992, 41.784029971000045], [-87.77300675599989, 41.784029971000045], [-87.77650572699989, 41.78386722800007], [-87.7804115559999, 41.78378585700004], [-87.78431738499994, 41.783623114000065], [-87.78708401399989, 41.783623114000065], [-87.78976927099988, 41.78370448500004], [-87.7912339569999, 41.78378585700004], [-87.79310549999988, 41.78337899900003], [-87.79416332899989, 41.783134885000024], [-87.7950584149999, 41.78248391400006], [-87.79603487199994, 41.781588828000054], [-87.79668584399988, 41.78053099900005], [-87.79798778599988, 41.77914768500005], [-87.7988828719999, 41.77808985600005], [-87.8001848149999, 41.77670654200005], [-87.80105499999988, 41.77583865700006], [-87.80105499999988, 41.775932], [-87.80125499999988, 41.776532], [-87.80125499999988, 41.777432], [-87.80105499999988, 41.778332], [-87.801155, 41.779233], [-87.80125499999988, 41.780032], [-87.801155, 41.780932], [-87.80125499999988, 41.781832], [-87.80125499999988, 41.782732], [-87.801355, 41.783632], [-87.80125499999988, 41.784632], [-87.801355, 41.785532], [-87.801355, 41.786532], [-87.801455, 41.787332], [-87.801455, 41.788332], [-87.801555, 41.790132], [-87.801556, 41.791932], [-87.801556, 41.793532], [-87.801556, 41.793932], [-87.801656, 41.794332], [-87.801656, 41.79511401400002], [-87.801656, 41.795632], [-87.80175599999988, 41.796532], [-87.801758, 41.797514], [-87.80166192199992, 41.79755402000006], [-87.797235, 41.799398], [-87.796856, 41.799532], [-87.79655599999988, 41.799532], [-87.79555599999989, 41.799532], [-87.794355, 41.799432], [-87.79306252199989, 41.799432], [-87.793055, 41.799432], [-87.791855, 41.799532], [-87.79065499999989, 41.799532], [-87.78945499999989, 41.799532], [-87.788255, 41.799632], [-87.78695499999989, 41.799632], [-87.785755, 41.799632], [-87.78455499999988, 41.799632], [-87.783255, 41.799732], [-87.782786, 41.799732], [-87.782633, 41.799732], [-87.78205499999989, 41.799732], [-87.780855, 41.799732], [-87.779655, 41.799732], [-87.77835499999989, 41.799732], [-87.7776549999999, 41.799832], [-87.777155, 41.799832], [-87.77595499999987, 41.799832], [-87.774855, 41.799932], [-87.773555, 41.799932], [-87.772355, 41.799932], [-87.7711549999999, 41.799932], [-87.77065499999989, 41.799932], [-87.7699549999999, 41.799932], [-87.768955, 41.799932], [-87.76865499999988, 41.799932], [-87.76735499999988, 41.800032], [-87.766355, 41.799932], [-87.7652549999999, 41.800032], [-87.76505499999989, 41.800032], [-87.763654, 41.800132], [-87.762454, 41.800032], [-87.761254, 41.800132], [-87.760154, 41.800032], [-87.758854, 41.800132], [-87.757654, 41.800132], [-87.75645399999989, 41.800132], [-87.755154, 41.800232], [-87.7539539999999, 41.800232], [-87.7528494409999, 41.800324046000064], [-87.75269, 41.80014]]]}, "properties": {"city": "Chicago", "name": "Garfield Ridge", "regionid": "269584", "geo_point_2d": [41.79202339648409, -87.77795784740518], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.70589669075292, 41.91740011621336], [-87.70688, 41.91739], [-87.70691155289, 41.917995815491004], [-87.70694723032364, 41.918849679388366], [-87.70697794344139, 41.919825591148836], [-87.70707890566058, 41.921048113213665], [-87.7071090113331, 41.922208929316106], [-87.70717530030885, 41.92391960988477], [-87.7072, 41.92471], [-87.70595683671128, 41.92472188112062], [-87.70388202282551, 41.92474627566522], [-87.70144168462156, 41.92477089472925], [-87.70041, 41.92479], [-87.69925628775985, 41.92408002323697], [-87.6985781386627, 41.92366386251615], [-87.6977371288235, 41.923152963975355], [-87.69605413406902, 41.92211263616772], [-87.69484040243702, 41.92136945410676], [-87.69337734478421, 41.92049141571881], [-87.69213028039087, 41.91972321415102], [-87.69004144334434, 41.91846404439999], [-87.68855, 41.917560000000115], [-87.68929632475528, 41.91755238444126], [-87.69277462036557, 41.917527014080605], [-87.69521563050137, 41.91750598652473], [-87.69655817147513, 41.917495014746315], [-87.69765656547605, 41.917485327789464], [-87.69887687747259, 41.91747578121358], [-87.6996090726256, 41.917465085904055], [-87.70125752775844, 41.91745307179862], [-87.70284414674727, 41.91743688127396], [-87.70455258321397, 41.91742003805976], [-87.70589669075292, 41.91740011621336]]]}, "properties": {"city": "Chicago", "name": "Palmer Square", "regionid": "403314", "geo_point_2d": [41.92055959846124, -87.70030747693006], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.7356262940441, 41.97890134405135], [-87.73552322036544, 41.97653064943945], [-87.73696625186835, 41.9768914073152], [-87.73810006233492, 41.977252165190954], [-87.73820313601367, 41.978901344051536], [-87.73619319927751, 41.978901344051536], [-87.7356262940441, 41.97890134405135]]]}, "properties": {"city": "Chicago", "name": "River's Edge", "regionid": "403273", "geo_point_2d": [41.97787723480816, -87.7367864917939], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.74571, 41.89493], [-87.7456922459999, 41.89437961800006], [-87.74566420499988, 41.89352614600004], [-87.74563356899993, 41.892734408000074], [-87.74559478299993, 41.891879039000045], [-87.7455452219999, 41.89059667100007], [-87.74551775899992, 41.88974312100003], [-87.74548471899992, 41.88906892800007], [-87.7454540129999, 41.88822033200005], [-87.74543367499989, 41.88773085500008], [-87.74541024299991, 41.887180476000076], [-87.7453954069999, 41.886813305000075], [-87.7453820729999, 41.88638502600007], [-87.7453444629999, 41.885470656000045], [-87.7453305029999, 41.885105821000074], [-87.74531169799988, 41.884676609000046], [-87.74527139799993, 41.88364063900006], [-87.74523760599992, 41.88266478600007], [-87.74517536799992, 41.88101272600004], [-87.74513492899989, 41.88004090000003], [-87.74508886999989, 41.87881713800004], [-87.74506551199993, 41.87826887000005], [-87.7450217179999, 41.87723321900005], [-87.74498346099993, 41.87644238300003], [-87.74493888099992, 41.87558371700003], [-87.74490008899993, 41.874669354000034], [-87.74485037699989, 41.87350704600003], [-87.74480402699992, 41.87283877000004], [-87.74482, 41.87175], [-87.74611300599989, 41.87171827000008], [-87.74928786099989, 41.87168008500003], [-87.7507544499999, 41.87169565400005], [-87.7518517589999, 41.87167972500003], [-87.75350018899988, 41.87164904300003], [-87.75496382399989, 41.871600163000025], [-87.75636654299987, 41.871590165000036], [-87.75825900899989, 41.871585702000054], [-87.7605762559999, 41.87152069800003], [-87.7621559969999, 41.871462957000055], [-87.76362092899987, 41.871302119000056], [-87.7643478499999, 41.87122353000006], [-87.76477450999992, 41.87117740200005], [-87.76577179299993, 41.87107491500005], [-87.76747752499992, 41.87090199000004], [-87.7687560549999, 41.87085436200003], [-87.7705844199999, 41.87084120200005], [-87.7720522639999, 41.870908153000066], [-87.77327336699992, 41.87097247500008], [-87.77425902799989, 41.871119946000064], [-87.77425902799989, 41.87112994600004], [-87.7743590279999, 41.872630594000036], [-87.7743590279999, 41.87268748500007], [-87.7743590279999, 41.872987259000055], [-87.7743590279999, 41.87299249900006], [-87.77435902699993, 41.87304437900007], [-87.77435851599994, 41.87304989500005], [-87.7743580129999, 41.87310158700006], [-87.7743580129999, 41.873106827000065], [-87.77435801099993, 41.87457336500005], [-87.7743574969999, 41.87463046600003], [-87.77435622099992, 41.874687541000064], [-87.77434856399991, 41.87497340000004], [-87.77434752099991, 41.87503047500007], [-87.77434746399987, 41.875087579000066], [-87.7743575469999, 41.87637141600003], [-87.77435778699991, 41.87641139200007], [-87.77435809599989, 41.876428580000024], [-87.77435895299992, 41.87646840400002], [-87.77435901999989, 41.87648553600006], [-87.7743590279999, 41.87652551400003], [-87.7743590279999, 41.87757325900007], [-87.7743590279999, 41.87762994600006], [-87.77442387499988, 41.87804306600008], [-87.7744553229999, 41.878231755000044], [-87.77455902799993, 41.880030594000054], [-87.77455902799993, 41.88008748500003], [-87.77455902799993, 41.88115263100007], [-87.77455902799993, 41.88120975000003], [-87.77455902799993, 41.881266857000064], [-87.77455902799993, 41.88197325900005], [-87.77455902799993, 41.882030378000074], [-87.77455902799993, 41.88208748500005], [-87.77455902799993, 41.883973259000065], [-87.77455902799993, 41.88402994600005], [-87.77469878499991, 41.885207968000074], [-87.77471632299994, 41.88535175500003], [-87.77473832299992, 41.88562175500004], [-87.77482432299989, 41.88699675500003], [-87.77484364699987, 41.887406250000076], [-87.77484898899989, 41.88744045100008], [-87.77484731399993, 41.88746206600006], [-87.77482802799992, 41.887711378000056], [-87.77482802799992, 41.88776848500003], [-87.77482802799992, 41.887982259000054], [-87.77482802799992, 41.88803894600004], [-87.77487534999992, 41.88886008100007], [-87.77488098899991, 41.88889045100006], [-87.7748790169999, 41.888915897000054], [-87.7748610059999, 41.88914832500006], [-87.7748600279999, 41.889160946000054], [-87.7748610479999, 41.88920512100008], [-87.77486039799992, 41.889218183000075], [-87.77487600399989, 41.89001466600006], [-87.77487732299991, 41.89007175500006], [-87.77491302799987, 41.89051459400007], [-87.77491302799987, 41.89057148500006], [-87.77491302799987, 41.89088325900006], [-87.77491302799987, 41.89093994600005], [-87.7749553229999, 41.89183175500004], [-87.7749953229999, 41.892816755000055], [-87.77505532299992, 41.89323175500003], [-87.77505710199993, 41.893288807000026], [-87.7750695429999, 41.893687691000075], [-87.77507132299988, 41.89374475500004], [-87.77507317399993, 41.893801802000034], [-87.77510147099989, 41.894673696000034], [-87.77510702799991, 41.894729591000036], [-87.77510710799992, 41.89478648300007], [-87.7751079479999, 41.89538725700003], [-87.7751080339999, 41.89544412600008], [-87.77513632299988, 41.89631575500004], [-87.77519032299992, 41.897205755000066], [-87.77523385899988, 41.89788466100003], [-87.77523539799994, 41.89794116400008], [-87.7752593399999, 41.89833061700005], [-87.7752590129999, 41.89838758700005], [-87.7752590129999, 41.89860144700002], [-87.77525901599992, 41.89865855200008], [-87.7752598629999, 41.89871565500003], [-87.7752600279999, 41.90007325900005], [-87.7752600279999, 41.90013037800008], [-87.7752600279999, 41.90018748500005], [-87.7752600279999, 41.90184576200004], [-87.7752600279999, 41.90190288100006], [-87.77461335199989, 41.90191994000008], [-87.77321012399993, 41.901943657000054], [-87.77192785899989, 41.901964869000025], [-87.77089066499988, 41.901981956000036], [-87.7697309459999, 41.90199101900002], [-87.76851118599987, 41.90202094900007], [-87.7674129899999, 41.90203910200005], [-87.7660702249999, 41.90205568900006], [-87.76491131899988, 41.90207036300006], [-87.76430119899992, 41.902080420000054], [-87.76277533799991, 41.90209687800006], [-87.76161595599989, 41.90211051800008], [-87.7598449249999, 41.90213], [-87.75868792199991, 41.90213916700007], [-87.75722069599992, 41.90215], [-87.75618291099993, 41.902155858000064], [-87.7551453549999, 41.90216892900008], [-87.7536800359999, 41.902186220000026], [-87.75239761599993, 41.90220326300003], [-87.75154348499989, 41.90221720200008], [-87.7492859579999, 41.902228603000076], [-87.74666209499989, 41.90226097400005], [-87.74587988799992, 41.90227094200007], [-87.74585003399991, 41.90091157700004], [-87.74581818599992, 41.899938289000026], [-87.7458, 41.89914003900007], [-87.74579175699994, 41.89834720800007], [-87.74576458199992, 41.89718693400004], [-87.74574533199988, 41.89633484800003], [-87.7457189869999, 41.89535686300007], [-87.74571, 41.89493]]]}, "properties": {"city": "Chicago", "name": "South Austin", "regionid": "403120", "geo_point_2d": [41.886741303109275, -87.76013303105047], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.5826739269999, 41.65934452400005], [-87.58079799199992, 41.65919716800004], [-87.57718774599991, 41.65920708600003], [-87.57547282499989, 41.65921], [-87.57309875699988, 41.659202661000045], [-87.57309899999987, 41.659208], [-87.57218999999989, 41.659204], [-87.57218969599987, 41.65919985100003], [-87.57023794399991, 41.659193818000055], [-87.56599590099992, 41.659180890000066], [-87.5635589149999, 41.65918], [-87.56112143899989, 41.65917054900005], [-87.55910739499987, 41.6592], [-87.55807, 41.65921], [-87.55806539399991, 41.65885997000002], [-87.55804928799992, 41.65838573000006], [-87.5579199999999, 41.65799], [-87.5576481709999, 41.65772880000008], [-87.55617301599993, 41.65649144500007], [-87.5551205499999, 41.655598665000056], [-87.55388953599991, 41.654605831000076], [-87.55183203599994, 41.653016719000036], [-87.5488790169999, 41.650788578000046], [-87.5461631389999, 41.64867956600005], [-87.54370372999993, 41.64675875000005], [-87.54219258199991, 41.64558260100005], [-87.5406019399999, 41.64436678800006], [-87.54060321399992, 41.64436704300004], [-87.54094299999988, 41.644435], [-87.541543, 41.644435], [-87.541843, 41.644435], [-87.542043, 41.644635], [-87.542443, 41.644535], [-87.5429429999999, 41.644635], [-87.5455660289999, 41.64460694000006], [-87.547617, 41.644585], [-87.54987697199991, 41.64456109500008], [-87.552344, 41.644535], [-87.55297026299989, 41.64452361300004], [-87.557844, 41.644435], [-87.5580775479999, 41.64448171000003], [-87.55816336499991, 41.64449887300008], [-87.558344, 41.644535], [-87.558944, 41.644435], [-87.55910292399993, 41.64447473100005], [-87.5597439999999, 41.644635], [-87.5598893029999, 41.644635], [-87.56044399999989, 41.644635], [-87.5606207689999, 41.64464604800008], [-87.56204399999987, 41.644735], [-87.56228354699992, 41.64472219300006], [-87.5630539999999, 41.644681], [-87.56407199999988, 41.644635], [-87.5652589999999, 41.644643], [-87.566444, 41.644651], [-87.567589, 41.644659], [-87.568689, 41.644667], [-87.570019, 41.644676], [-87.571243, 41.644685], [-87.572504, 41.644694], [-87.573809, 41.644703], [-87.5750759999999, 41.644712], [-87.5782439999999, 41.644734], [-87.57861983199989, 41.644734], [-87.5794439999999, 41.644734], [-87.583845, 41.644734], [-87.5842141879999, 41.644734], [-87.586345, 41.644734], [-87.58672349399988, 41.64471748900006], [-87.588133, 41.644656], [-87.58960036899991, 41.644592338000045], [-87.59094499999989, 41.644534], [-87.59465746199992, 41.64458941000003], [-87.597645, 41.644633999999996], [-87.60324499999987, 41.644633999999996], [-87.60574499999989, 41.644633999999996], [-87.6061449999999, 41.644633999999996], [-87.607347, 41.644633999999996], [-87.60844499999988, 41.644633999999996], [-87.6091384829999, 41.64458065500007], [-87.60974499999989, 41.644534], [-87.61114599999989, 41.644534], [-87.612146, 41.644534], [-87.61224837699989, 41.64452912500007], [-87.61224837699989, 41.64687511400006], [-87.61196966499993, 41.64696801800004], [-87.6120625689999, 41.64845447900005], [-87.61196966499993, 41.64919770900007], [-87.61196966499993, 41.649524423000045], [-87.611953, 41.649521], [-87.61187825499988, 41.64989406300002], [-87.61087923199993, 41.64973293000008], [-87.60947339399989, 41.64966083600006], [-87.60763499099993, 41.64966083600006], [-87.6059047289999, 41.64966083600006], [-87.60478726799992, 41.64955269400007], [-87.60475926599989, 41.64955002700008], [-87.6032732879999, 41.649408506000036], [-87.60172326199992, 41.64912012900003], [-87.6002092819999, 41.64872361000003], [-87.5997406689999, 41.64861546900004], [-87.5982266899999, 41.64868756300006], [-87.59653247499993, 41.648975940000064], [-87.59498244799993, 41.64930036400004], [-87.59379289299991, 41.64955269400007], [-87.59271147899993, 41.64962478800003], [-87.59273857599992, 41.65021089200006], [-87.59273857599992, 41.651613208000065], [-87.5928289049999, 41.65534051700006], [-87.59208950799987, 41.655275500000066], [-87.5915576229999, 41.65535895700002], [-87.59114223099992, 41.65546140100007], [-87.59085, 41.65597], [-87.59122010999992, 41.656431372000036], [-87.59141600499991, 41.65673696800006], [-87.59170291499991, 41.657113908000035], [-87.59202190599989, 41.657435968000065], [-87.59244300499992, 41.65786222800005], [-87.59285067299992, 41.65838325600003], [-87.59315173899992, 41.65882253200004], [-87.59355813199993, 41.65907295300007], [-87.59355, 41.65908], [-87.58991305899988, 41.660339385000036], [-87.5885600749999, 41.66044], [-87.58693999599988, 41.66041111100007], [-87.5858574449999, 41.66025676900004], [-87.5851104959999, 41.65998119700003], [-87.5837764749999, 41.659649876000046], [-87.5826739269999, 41.65934452400005]]]}, "properties": {"city": "Chicago", "name": "Little Calumet River", "regionid": "403390", "geo_point_2d": [41.65090702041374, -87.57527043554782], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.7253, 41.874000000000244], [-87.7264019052403, 41.87398695992491], [-87.72859853592877, 41.87396266791242], [-87.72957475017509, 41.87396000000013], [-87.73171059699948, 41.873926480877365], [-87.73310723407532, 41.873877845865785], [-87.73335456288449, 41.873846491016586], [-87.73382570323666, 41.87376746696049], [-87.73421292094432, 41.87369111409565], [-87.73467957313527, 41.87355973093193], [-87.73566376476114, 41.8732255850169], [-87.73606790984228, 41.873057888084475], [-87.73654253182774, 41.87284786502491], [-87.73693788581309, 41.87266783789766], [-87.73738378706831, 41.87246479317884], [-87.73783057491548, 41.8722705480712], [-87.73853162995883, 41.87204709276041], [-87.73915570801294, 41.87191783294529], [-87.73957314391622, 41.871853989883604], [-87.73997652588244, 41.87183351985705], [-87.73997671418134, 41.871841716828506], [-87.7408647721811, 41.87178248788977], [-87.74159693199974, 41.87177090298749], [-87.74379437405754, 41.87174216280161], [-87.74481999999985, 41.87175000000024], [-87.74480402723496, 41.87283877039817], [-87.74485037743891, 41.873507045530644], [-87.74490008860548, 41.874669354373594], [-87.74493888077966, 41.87558371656897], [-87.74498346145775, 41.87644238311644], [-87.74502171770854, 41.87723321901721], [-87.74506551175406, 41.878268870406025], [-87.74508886979368, 41.87881713808972], [-87.74513492887405, 41.88004089966785], [-87.74517536788413, 41.88101272556919], [-87.74523760554166, 41.88266478565136], [-87.74527139845091, 41.88364063941875], [-87.7453116983427, 41.88467660940144], [-87.74533050331716, 41.88510582132308], [-87.74534446267165, 41.88547065618602], [-87.74538207255414, 41.88638502626434], [-87.74539540678164, 41.88681330517997], [-87.7454102431361, 41.88718047579957], [-87.7454336752305, 41.887730855075226], [-87.74545401328173, 41.8882203320469], [-87.74548471896594, 41.88906892846421], [-87.74551775917705, 41.8897431208861], [-87.74554522238219, 41.89059667147116], [-87.74559478341534, 41.89187903928736], [-87.74563356887995, 41.89273440758089], [-87.74566420485743, 41.89352614572455], [-87.74569224572645, 41.894379617525345], [-87.74571, 41.89493], [-87.74489428132254, 41.89494373486432], [-87.74232879909918, 41.89498550987195], [-87.74104623872695, 41.895000693520245], [-87.74037546570672, 41.895006804069645], [-87.7397054943677, 41.89501], [-87.73866601355374, 41.895029693220664], [-87.73787335202371, 41.895047260356144], [-87.73622626468583, 41.89509485192078], [-87.73500632546906, 41.89511617066578], [-87.73390530836083, 41.895135367291225], [-87.73268603091985, 41.895160063001406], [-87.73146575523313, 41.89517958459322], [-87.73024548747343, 41.895199575208956], [-87.72884232115979, 41.89523101834718], [-87.7276833988717, 41.89525647842728], [-87.72646305612945, 41.89528076957184], [-87.72590501247194, 41.89528991782857], [-87.72588390231185, 41.894195357220376], [-87.72584323944731, 41.893220563699984], [-87.72581824440607, 41.89248908778121], [-87.72580400664451, 41.89187841196765], [-87.72578016088094, 41.89102160035523], [-87.72575829344747, 41.89035099906109], [-87.72573, 41.889314437473956], [-87.72573, 41.888582094532815], [-87.72570853476532, 41.88789202772699], [-87.72568283833188, 41.887177988293715], [-87.72564779594254, 41.886019511038974], [-87.72561180420496, 41.88473809855902], [-87.72559358371761, 41.88406826268799], [-87.72558147708708, 41.883396010663446], [-87.72555179849157, 41.882601148720966], [-87.72551896835776, 41.881198101474574], [-87.72549029554175, 41.88003602549566], [-87.72545137246117, 41.879065880375876], [-87.72542810915752, 41.8784532747302], [-87.72539596506861, 41.87729246302442], [-87.72537052297832, 41.87643919701137], [-87.72534066953335, 41.875520755536144], [-87.72531350525479, 41.87454536784837], [-87.7253, 41.874000000000244]]]}, "properties": {"city": "Chicago", "name": "West Garfield Park", "regionid": "269613", "geo_point_2d": [41.88406298081134, -87.73564021003033], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.71568593599991, 41.97175091100006], [-87.71528191499992, 41.97172002300004], [-87.71484891499992, 41.97175402300006], [-87.71465791499989, 41.971779022000064], [-87.71382591599989, 41.97185002200006], [-87.7135059149999, 41.97191002300007], [-87.71338991499992, 41.97194002200006], [-87.71301391599988, 41.97204602300007], [-87.71264091499994, 41.972209023000055], [-87.71238191699989, 41.97233902300007], [-87.71224091599991, 41.97244302300003], [-87.7120649159999, 41.972614023000055], [-87.7118839149999, 41.97281802300005], [-87.71122691399994, 41.973710024000056], [-87.71109991499989, 41.97383002200006], [-87.71106091499988, 41.97385902200005], [-87.71074291599987, 41.97404202200004], [-87.7103709159999, 41.974194022000056], [-87.7100149149999, 41.97429402200003], [-87.7099539149999, 41.97430202200006], [-87.7093709149999, 41.97441402300006], [-87.70898991599994, 41.97454002300008], [-87.70876591499987, 41.974634021000035], [-87.70865691599992, 41.97466102300005], [-87.7085019159999, 41.974686021000075], [-87.70826591599987, 41.97468702300005], [-87.70808891599992, 41.97467202200004], [-87.70739591599992, 41.974566022000026], [-87.70694291599993, 41.97443602200007], [-87.70631191599992, 41.974255022000065], [-87.70559291599992, 41.97406802300008], [-87.70533991599989, 41.97402502200002], [-87.7050409159999, 41.97397802300008], [-87.704329, 41.973907], [-87.7042126239999, 41.973908231000046], [-87.70417433099992, 41.973770377000044], [-87.7040404549999, 41.97338213700004], [-87.70389319199988, 41.97311438600006], [-87.70377270399989, 41.97271275900004], [-87.70369237899989, 41.972311132000065], [-87.70358527799992, 41.97194966700003], [-87.70342462699993, 41.97166852800006], [-87.70327736399989, 41.97138738900008], [-87.7031568759999, 41.971146413000035], [-87.7029025119999, 41.97077156100004], [-87.7026347599999, 41.97034315900002], [-87.70244733399991, 41.970075408000056], [-87.7022599079999, 41.96983443100003], [-87.70200554499988, 41.96963361800004], [-87.70165746799992, 41.96933909100004], [-87.70152359199993, 41.96912489000003], [-87.70148342899992, 41.96885713900007], [-87.70122906599993, 41.968656325000026], [-87.70105250499994, 41.96850330600005], [-87.70120001799994, 41.96850196500007], [-87.70202475899991, 41.96849446500004], [-87.70315881799989, 41.96849], [-87.70367, 41.96849], [-87.7035816939999, 41.96745399300004], [-87.70354128699994, 41.96615473600007], [-87.70350133999993, 41.96510374600006], [-87.7034676889999, 41.96397141600004], [-87.7034203769999, 41.962434780000066], [-87.7033800659999, 41.96121201700003], [-87.7033779159999, 41.961212022000026], [-87.70215591599991, 41.96121602200008], [-87.70092891599988, 41.961231022000035], [-87.6997049149999, 41.96124202200007], [-87.69852191499989, 41.961241022000074], [-87.69851987299987, 41.96124103500005], [-87.69851999999989, 41.961240000000004], [-87.69845202799989, 41.96032814100004], [-87.6983942809999, 41.95862985400002], [-87.69833295999989, 41.956769785000056], [-87.6982775729999, 41.95498503900006], [-87.6982299999999, 41.95388], [-87.6990300449999, 41.95387385300005], [-87.70040594399988, 41.95387777200005], [-87.70113501599988, 41.95387624000005], [-87.7027540159999, 41.95387], [-87.70380661299991, 41.95386432600008], [-87.7049399819999, 41.95385504100004], [-87.70607332199988, 41.953856581000075], [-87.70761164299992, 41.95387], [-87.70874708499989, 41.95385177800005], [-87.70987984299987, 41.95382440400004], [-87.71102600199991, 41.95375944400006], [-87.71230940399992, 41.953691572000075], [-87.71336115599989, 41.95366698900005], [-87.7148985879999, 41.953635971000026], [-87.7161126149999, 41.95361564900008], [-87.71732717999991, 41.95372277400007], [-87.71893518599987, 41.95354345800006], [-87.72004095799988, 41.95431249300008], [-87.72220027799992, 41.955770849000075], [-87.7228683829999, 41.95622689300005], [-87.72473581399991, 41.957514558000064], [-87.72560558299993, 41.95811225900007], [-87.7268412199999, 41.958971], [-87.7276272809999, 41.95950887700007], [-87.72763941099991, 41.95992409800003], [-87.72764676799993, 41.96024777300004], [-87.7276639159999, 41.96083402200008], [-87.72769774299988, 41.96203022200006], [-87.7277340119999, 41.963486380000056], [-87.7277868339999, 41.965187809000035], [-87.72783, 41.96607594700002], [-87.72786441499993, 41.96712447700008], [-87.72791, 41.96813], [-87.72795684099991, 41.969110503000074], [-87.7279995949999, 41.970655429000026], [-87.72801393999991, 41.971171830000046], [-87.72803622699989, 41.972098734000035], [-87.72806004199991, 41.97287118900004], [-87.72811335699991, 41.97452398500007], [-87.72813906899994, 41.97538214900003], [-87.72813804799993, 41.97538215000007], [-87.72722386599989, 41.97539785200006], [-87.72703691499991, 41.97552602200004], [-87.72685691599989, 41.975620023000026], [-87.7265739139999, 41.97575902300007], [-87.72634891599994, 41.97585302300006], [-87.72624491499992, 41.975887022000045], [-87.72596791599993, 41.97593102200005], [-87.72578891499987, 41.97593702200003], [-87.72551991599993, 41.97593102200005], [-87.72522891499993, 41.97588102200006], [-87.72489991499991, 41.975809022000064], [-87.72451091599991, 41.97572002200007], [-87.72426491699989, 41.97562502200008], [-87.72412991799987, 41.97554202200006], [-87.7240246499999, 41.975444701000065], [-87.72402391599988, 41.975444022000026], [-87.7236379159999, 41.97503702300007], [-87.7233469159999, 41.97479802300005], [-87.72303991499989, 41.974618022000065], [-87.72265591499989, 41.97436102200003], [-87.7224669159999, 41.97425102300007], [-87.72228191499991, 41.974172023000044], [-87.72182991499992, 41.97402102300003], [-87.72165191499991, 41.97398902300005], [-87.72139191599989, 41.97396102300006], [-87.72080791599988, 41.97393402200002], [-87.72038691499992, 41.97389702200007], [-87.7201029159999, 41.97385802200005], [-87.71995291599988, 41.97381902300002], [-87.7197329129999, 41.973801020000046], [-87.71941291599987, 41.97368502300008], [-87.71919591599992, 41.97358502200006], [-87.7188839149999, 41.97340502300005], [-87.7186559159999, 41.973316023000045], [-87.71830791599987, 41.97320302200006], [-87.7181899149999, 41.97316202300003], [-87.7180928889999, 41.973123068000056], [-87.7176519159999, 41.97280102300004], [-87.71745691499989, 41.97267902300006], [-87.71724091699988, 41.97256702100008], [-87.71680691599992, 41.97225602300005], [-87.7165159339999, 41.972083973000046], [-87.71622388099989, 41.97201510000007], [-87.7159699519999, 41.97183398100008], [-87.7157849149999, 41.97178002300006], [-87.71568593599991, 41.97175091100006]]]}, "properties": {"city": "Chicago", "name": "Albany Park", "regionid": "269566", "geo_point_2d": [41.9638293533172, -87.71419206500633], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.78395491542092, 41.98283802178475], [-87.78590991560756, 41.9828100222926], [-87.78638991525088, 41.982815022523056], [-87.78761404451808, 41.98282915536888], [-87.78847339299098, 41.98282780851665], [-87.7893682699588, 41.9827231038026], [-87.7907813077788, 41.98262518254288], [-87.79413560941686, 41.98248718260091], [-87.79660003486322, 41.98246], [-87.8001131308393, 41.98241554015533], [-87.8024198129331, 41.98240032618596], [-87.80556573526258, 41.98240747298964], [-87.80684999999987, 41.9823900000001], [-87.80682608486636, 41.983239807713815], [-87.80682666081499, 41.98376362006686], [-87.80680533087487, 41.985336875638275], [-87.80679679662592, 41.98669975824212], [-87.80679144223059, 41.9882204065207], [-87.80678423507483, 41.98900760260201], [-87.80677623229407, 41.98953178473881], [-87.80675798017603, 41.99042319691807], [-87.80673582181458, 41.99131396305315], [-87.80671999999987, 41.992414586355785], [-87.80671376645792, 41.99314855169754], [-87.80671558367249, 41.99398811419475], [-87.80671302321582, 41.99472139724432], [-87.80670378326232, 41.99533988851905], [-87.80670291541651, 41.995703023063264], [-87.80668791562435, 41.99597702220396], [-87.8066639154173, 41.99688302169921], [-87.80666369606884, 41.9968943045679], [-87.80591034400625, 41.99645451957131], [-87.8029100161814, 41.994682881380484], [-87.80089516936329, 41.993463671663925], [-87.79999948627886, 41.99293762988756], [-87.7981321437262, 41.99182930294623], [-87.79619991594133, 41.99070895279802], [-87.79389144218517, 41.98942132636081], [-87.7920958621541, 41.98841464384249], [-87.78998923991895, 41.98723583775685], [-87.78826753199003, 41.98627709192852], [-87.78586718671939, 41.984943614443026], [-87.78439682326297, 41.984114607055936], [-87.78363353372117, 41.983679178883925], [-87.78215439288283, 41.98284996680181], [-87.78351991605336, 41.98284102192293], [-87.78395491542092, 41.98283802178475]]]}, "properties": {"city": "Chicago", "name": "Old Norwood Park", "regionid": "403277", "geo_point_2d": [41.98731163033629, -87.79860778620849], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.67991368910606, 41.80470302207955], [-87.67991368910606, 41.80364042120851], [-87.6798356093538, 41.80157130777184], [-87.6797184897253, 41.80012683235382], [-87.6797184897253, 41.79649612387017], [-87.67975000649842, 41.79375416459997], [-87.6807312115169, 41.79375379164051], [-87.68187074839553, 41.79375], [-87.6826034544049, 41.79374585050673], [-87.68399, 41.793799999999884], [-87.6839569076144, 41.792573011182306], [-87.6838870552221, 41.79062165666478], [-87.6838398753978, 41.788995389716746], [-87.68378865471188, 41.78712530056938], [-87.68376915317344, 41.786450029207266], [-87.68383792918002, 41.786451983865305], [-87.68412991566044, 41.78646202209786], [-87.68498791564129, 41.786445022213684], [-87.6862139154983, 41.78642302209889], [-87.68737691604873, 41.78640902235282], [-87.68867291553522, 41.78640002193782], [-87.68942091584125, 41.78639702269893], [-87.68986291553153, 41.786383022053656], [-87.69107891492774, 41.78634702219279], [-87.69230291559194, 41.78633002230836], [-87.69347991588835, 41.78631502251628], [-87.6935529156561, 41.78631402247011], [-87.69363091565428, 41.78631302242402], [-87.69472391477643, 41.7862910223093], [-87.69597291479461, 41.78626202187179], [-87.69717291525183, 41.78623902261018], [-87.69840891556993, 41.78621702249547], [-87.69960091565834, 41.78620202180404], [-87.70085091572243, 41.786187022011866], [-87.70203791558036, 41.78617202221979], [-87.70322391539241, 41.78616502189707], [-87.70327391589848, 41.78616502189707], [-87.70334091538967, 41.78616602194317], [-87.70446891522597, 41.786134022266566], [-87.70571391505968, 41.786100022498054], [-87.70653291594056, 41.786080022475524], [-87.7069579157465, 41.786080022475524], [-87.70814291551228, 41.78607802238334], [-87.70939791580689, 41.78603102201539], [-87.70966091533971, 41.78602202249986], [-87.71059391607976, 41.786018022315496], [-87.71127091581087, 41.78601502217723], [-87.7127229161933, 41.78598702268508], [-87.7128709140224, 41.78598407740551], [-87.71307391978269, 41.785981967595845], [-87.71310491671501, 41.786889021996046], [-87.71313591544605, 41.787798022528946], [-87.71316791602192, 41.78870302287756], [-87.71318691509907, 41.789268022841966], [-87.71321291539824, 41.78959002239646], [-87.71323791565139, 41.790234022405066], [-87.71328191588098, 41.791607022743115], [-87.71329991581135, 41.79222502245248], [-87.71330691523468, 41.79244302260907], [-87.71333191548786, 41.79324602275234], [-87.71333491562604, 41.79334602196614], [-87.71338991636271, 41.79526502229189], [-87.71342791541637, 41.7967210219593], [-87.71343691583138, 41.797062022389774], [-87.71345991599227, 41.79800302259927], [-87.71347691587651, 41.79866502253813], [-87.71348291525379, 41.79882302262697], [-87.71348491534604, 41.79888302269481], [-87.71349991513813, 41.79924902247907], [-87.71350491536867, 41.79930502236229], [-87.7135449154138, 41.79978702209788], [-87.71357591504413, 41.80016102225086], [-87.7135779160356, 41.800581022725694], [-87.71358591550513, 41.801348022108996], [-87.71363191582691, 41.802291022410394], [-87.71366091536515, 41.80339202190154], [-87.7136759700159, 41.80397003415604], [-87.71349591585313, 41.80399202257983], [-87.71294391558871, 41.8040530217943], [-87.71254091589765, 41.804070022578024], [-87.71158791603438, 41.80411102266944], [-87.71113991606748, 41.8041180220927], [-87.71005991574594, 41.80413502197703], [-87.70875091566002, 41.804172021884], [-87.70856291598712, 41.804177022114374], [-87.7077479152905, 41.804189021768345], [-87.70611691565018, 41.80421102188314], [-87.703992915767, 41.80424302245884], [-87.70387891500864, 41.80424402250493], [-87.70383291558626, 41.80424402250493], [-87.70378291508003, 41.804245022551015], [-87.7026139160517, 41.80425702220482], [-87.70087291493792, 41.804301022434416], [-87.69928291538885, 41.8043280218803], [-87.69769991526302, 41.80437302215616], [-87.69710291562228, 41.80438202257098], [-87.69612091622083, 41.804397022363226], [-87.69410691601246, 41.80443702240846], [-87.69405291532196, 41.804438022454555], [-87.69400791504627, 41.80443902250064], [-87.69232191556847, 41.80446302180816], [-87.68943591563334, 41.80450302185348], [-87.68939591558821, 41.80450302185348], [-87.68823891621368, 41.8045110222222], [-87.68819791522314, 41.8045110222222], [-87.68697691559642, 41.80452202183], [-87.68693791559737, 41.80452202183], [-87.6859859148808, 41.80453102224473], [-87.68577891613074, 41.80453102224473], [-87.68572691553224, 41.80453102224473], [-87.68498491550291, 41.80453002219872], [-87.6843108340717, 41.80453593524114], [-87.68424399825723, 41.80461317351353], [-87.6837189156006, 41.80466802226485], [-87.68185791525049, 41.804671022403134], [-87.68179291495211, 41.804675022587396], [-87.68171991518437, 41.804675022587396], [-87.68158391521042, 41.80467702178039], [-87.68060291495578, 41.804696022656195], [-87.68030091542373, 41.80470302207955], [-87.67991368910606, 41.80470302207955]]]}, "properties": {"city": "Chicago", "name": "Gage Park", "regionid": "269582", "geo_point_2d": [41.79560756243489, -87.69732799038418], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.61449200457288, 41.73648000103436], [-87.6144731274029, 41.73596819788925], [-87.61445081228783, 41.73532193177041], [-87.6144181583773, 41.734300804112785], [-87.6143916765111, 41.73337128125924], [-87.61437131818462, 41.73244063649834], [-87.61435108735826, 41.73151001848104], [-87.61432282563969, 41.73020997942879], [-87.614309108022, 41.729745979343335], [-87.61428392867866, 41.72890804016769], [-87.61427, 41.728165952084346], [-87.6142657510652, 41.7279031250157], [-87.61423191587244, 41.72701602207465], [-87.6142260756752, 41.72690192418964], [-87.61422588806654, 41.726901821278624], [-87.61423, 41.726900000000114], [-87.61421601755262, 41.72593521113727], [-87.61416621102217, 41.724634489559456], [-87.61410018203651, 41.72277846470174], [-87.61408, 41.72184], [-87.6147939622313, 41.72184566636681], [-87.61600166657674, 41.72184668776489], [-87.61711623084142, 41.72183022948606], [-87.61832321376392, 41.72180529222074], [-87.61990397055536, 41.7217958778693], [-87.62139325086571, 41.72177782043101], [-87.62297189525505, 41.72172075436438], [-87.62445593676536, 41.721640000000114], [-87.62460207830155, 41.723609650175966], [-87.62464432466415, 41.72518893455257], [-87.62469635880807, 41.7275103836013], [-87.62443571651393, 41.72902265257756], [-87.62458391552423, 41.729022021914396], [-87.62474841197492, 41.72902002793111], [-87.6247838533302, 41.73011935994631], [-87.62479919892121, 41.730386381660736], [-87.62472986887889, 41.73620557012454], [-87.62472664524756, 41.73627824767154], [-87.62306083465508, 41.73637839076811], [-87.62166773758075, 41.73637654242033], [-87.62018010730321, 41.73640408340306], [-87.61832313537997, 41.73643188993516], [-87.6168368272172, 41.73645056068553], [-87.61497908899412, 41.73647396186422], [-87.61449200457288, 41.73648000103436]]]}, "properties": {"city": "Chicago", "name": "West Chesterfield", "regionid": "403361", "geo_point_2d": [41.72908041395048, -87.61948655501888], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.6075083409999, 41.84575883400002], [-87.6075346449999, 41.84564987900006], [-87.60759264499988, 41.845461880000066], [-87.6076387879999, 41.84534298700002], [-87.6077237879999, 41.84516298600004], [-87.60775816399989, 41.845097339000056], [-87.60782703599993, 41.84499497500006], [-87.6081839919999, 41.84453589100007], [-87.60820222599993, 41.84451189900004], [-87.60822729599992, 41.84446797000003], [-87.60817316399992, 41.84439066100003], [-87.60815410499991, 41.84435590300005], [-87.60808910599988, 41.84423090300003], [-87.60805852899992, 41.844165880000034], [-87.60803324099992, 41.84409862200004], [-87.6080134029999, 41.84402956100006], [-87.60799914199991, 41.84395913600007], [-87.60799054899988, 41.84388779700003], [-87.6079876779999, 41.843816], [-87.6079876779999, 41.843766], [-87.60800134099989, 41.843609834000056], [-87.60800811499989, 41.84356829400007], [-87.60757883499988, 41.84299598300003], [-87.60752214199992, 41.84290928400008], [-87.60747399099994, 41.84285116100004], [-87.60743088399992, 41.84278920600007], [-87.60739312299994, 41.84272385400003], [-87.60736097499993, 41.84265556600006], [-87.60728697499988, 41.84247956600006], [-87.60726094299989, 41.84240967300007], [-87.60724079299989, 41.842337862000065], [-87.6072266609999, 41.84226462900005], [-87.6071976619999, 41.84207162900008], [-87.6071898099999, 41.84199987600004], [-87.60716380899991, 41.84162287600008], [-87.60716216099988, 41.841531532000026], [-87.6071662289999, 41.84140743500006], [-87.60709117299989, 41.84132343300006], [-87.60704280399992, 41.84125909800007], [-87.60683187299989, 41.84107236000005], [-87.60673907899988, 41.840977073000026], [-87.60667556799989, 41.84089157200008], [-87.6064215679999, 41.84050357200004], [-87.60635400499989, 41.840380309000054], [-87.60629500499991, 41.84024930900005], [-87.60626991299993, 41.84018758600006], [-87.6062639759999, 41.84016748100003], [-87.60588888199992, 41.83987744300003], [-87.60585908699993, 41.83985337300003], [-87.6054880869999, 41.83954037300002], [-87.60540374799987, 41.839459259000066], [-87.60528874799991, 41.83933325800006], [-87.60524006299993, 41.839275179000076], [-87.60519644099989, 41.83921320700005], [-87.6051581909999, 41.83914778200005], [-87.60512219099991, 41.83907978300004], [-87.60509150999988, 41.83901585600005], [-87.60506596099991, 41.83894971000006], [-87.6050457029999, 41.838881757000074], [-87.6050308609999, 41.83881241900007], [-87.60502381599989, 41.83875936100008], [-87.60591699999989, 41.838659], [-87.60591649599989, 41.838622224000055], [-87.60940618299989, 41.83854672200004], [-87.61086962399989, 41.83852347600003], [-87.6126117709999, 41.83848363700007], [-87.61470269399989, 41.83844602600003], [-87.6164438449999, 41.83841972100003], [-87.61744999999989, 41.83836], [-87.61746771099988, 41.83919975500003], [-87.61748631999987, 41.83989695900003], [-87.6175, 41.84073053900004], [-87.61751530899993, 41.84142830700005], [-87.61754708699992, 41.842475791000076], [-87.61756995599987, 41.84303227400004], [-87.61758996699989, 41.84351919700003], [-87.61761855299991, 41.844214800000074], [-87.61761440199987, 41.84505047500004], [-87.6176244409999, 41.84568116300004], [-87.61762376099992, 41.84568115300003], [-87.61763225699988, 41.84616519500007], [-87.61766, 41.847210581000056], [-87.61766, 41.84758], [-87.6176645229999, 41.84770211600004], [-87.61787012599989, 41.847952881000076], [-87.61630352999991, 41.84797129600007], [-87.6146315309999, 41.84798084000004], [-87.6094822999999, 41.84832099000004], [-87.60858467099989, 41.84838028600006], [-87.60856366499988, 41.848148013000056], [-87.60855279199988, 41.84810589600004], [-87.60842902599987, 41.847949643000035], [-87.6084014019999, 41.84791301400008], [-87.6082429699999, 41.847691397000055], [-87.6078170199999, 41.84713942200005], [-87.60775016399991, 41.847039661000046], [-87.6077028219999, 41.846945259000044], [-87.6076168209999, 41.84674526000003], [-87.60758628399991, 41.84666352800007], [-87.60753328399993, 41.84649752800003], [-87.60750434099991, 41.84638016500008], [-87.60749071599989, 41.846218179000054], [-87.60749475399987, 41.845903359000026], [-87.6075083409999, 41.84575883400002]]]}, "properties": {"city": "Chicago", "name": "Prairie Shores", "regionid": "140000", "geo_point_2d": [41.843119251303186, -87.61233990583406], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.66275974147766, 41.92521455494724], [-87.66330458663695, 41.92521016348663], [-87.66334096478181, 41.92616389757441], [-87.66337885958885, 41.92745905564774], [-87.66341099953928, 41.92826998618087], [-87.66343632344092, 41.92932058151169], [-87.66346021217323, 41.93013056954904], [-87.66349087762137, 41.93093592604796], [-87.66351548215064, 41.93158267367399], [-87.66355176833615, 41.93248697704413], [-87.66354991577761, 41.93248702192583], [-87.66233891571267, 41.93250202261727], [-87.66134591580413, 41.93251902250161], [-87.66111591509447, 41.932524021832776], [-87.65989391542173, 41.93253202220151], [-87.65869791604815, 41.93256202268511], [-87.65743991561533, 41.93258802208499], [-87.65626891559553, 41.93259702249972], [-87.65582791505199, 41.93260002263817], [-87.65516591601232, 41.93261102224588], [-87.65384591541948, 41.93263402240668], [-87.65313191488214, 41.93264602206066], [-87.65262091560838, 41.932656021622364], [-87.65141791591208, 41.932675022498096], [-87.65021391527051, 41.93269702261279], [-87.64899291564394, 41.932717021736224], [-87.6489897012916, 41.93271710042741], [-87.64897609770225, 41.93231335779457], [-87.64891299397891, 41.93053439031112], [-87.64884929732101, 41.928755352074035], [-87.64881730377924, 41.927784881304326], [-87.64877627716169, 41.92657203768973], [-87.64877, 41.92546], [-87.65078046628912, 41.925417763313376], [-87.6527223917001, 41.92538241418086], [-87.6542609694517, 41.92536336439378], [-87.65571703016609, 41.925334842476516], [-87.65660867362394, 41.92532018567825], [-87.65798465734964, 41.9252974223775], [-87.65992705091821, 41.92526840791955], [-87.66154655182163, 41.92523672148243], [-87.66275974147766, 41.92521455494724]]]}, "properties": {"city": "Chicago", "name": "Wrightwood Neighbors", "regionid": "403307", "geo_point_2d": [41.9289644381659, -87.65614756446251], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.78789286492412, 41.96961092467879], [-87.7879169154932, 41.96865902198844], [-87.78792245441764, 41.96842787104569], [-87.78796000021299, 41.96843099978704], [-87.78796000021299, 41.96883100023918], [-87.78915999977094, 41.96883100023918], [-87.7895998069142, 41.96883100023918], [-87.79036000022826, 41.96883100023918], [-87.7910065669018, 41.968781264133895], [-87.7916599998992, 41.96873100012637], [-87.79286000035643, 41.96873100012637], [-87.79395999980136, 41.96873100012637], [-87.7951600002586, 41.96873100012637], [-87.79645999992952, 41.96873100012637], [-87.79756000027368, 41.96873100012637], [-87.7988599999448, 41.96873100012637], [-87.80006000040191, 41.96873100012637], [-87.80136100011912, 41.96873100012637], [-87.80242613914419, 41.96873011249543], [-87.80256099967698, 41.96873000008027], [-87.80256099967698, 41.96823000041426], [-87.80366100002112, 41.96823000041426], [-87.8040010004056, 41.96820700025336], [-87.80489100006277, 41.96820700025336], [-87.80597399962328, 41.96821799986108], [-87.8065096843874, 41.968174632754284], [-87.80706100026752, 41.96813000030126], [-87.80706100026752, 41.96816774754464], [-87.80706100026752, 41.96893000030635], [-87.80716100038053, 41.970029999751205], [-87.80706100026752, 41.97082999975629], [-87.80700311450585, 41.97135097341055], [-87.80696100015456, 41.97172999987438], [-87.8069895392397, 41.97198685613854], [-87.80706100026752, 41.972629999992456], [-87.80698375390108, 41.973325218189146], [-87.80696100015456, 41.97353000011062], [-87.8069823284758, 41.97363664261611], [-87.80714575597283, 41.97445378190043], [-87.80695958050997, 41.9749497160286], [-87.80695, 41.97540000000012], [-87.8056348206327, 41.975411337753265], [-87.80417811828089, 41.975427791064156], [-87.80302412081176, 41.97544381093332], [-87.80180979085952, 41.97545411940447], [-87.80047401186987, 41.97546517063524], [-87.79919816195917, 41.97548], [-87.79816631302513, 41.97548423266363], [-87.79689078889224, 41.975490000000114], [-87.79561564119255, 41.975495242634885], [-87.79446116191131, 41.9755096473015], [-87.79330800925743, 41.97552436057983], [-87.79203247532917, 41.97553477662333], [-87.79100071632365, 41.975546488061276], [-87.78966511601676, 41.97556430232779], [-87.78863294185915, 41.97557280881788], [-87.78777096306005, 41.97557999197461], [-87.78781291609506, 41.973772022957085], [-87.78784891595592, 41.97193202177643], [-87.78788491581679, 41.970145022139455], [-87.78789286492412, 41.96961092467879]]]}, "properties": {"city": "Chicago", "name": "Big Oaks", "regionid": "403279", "geo_point_2d": [41.97205351941075, -87.79755488021043], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.7313487363747, 41.982700000000165], [-87.73199, 41.982700000000165], [-87.73201497182487, 41.98359274273822], [-87.73204237284436, 41.984519744601485], [-87.73205, 41.98476], [-87.73114255993397, 41.98477475512308], [-87.73042112976725, 41.98478332391871], [-87.72918432868865, 41.9847935710765], [-87.72839855098759, 41.98478430003442], [-87.72839267766, 41.98456934629021], [-87.72837113275796, 41.98384856247943], [-87.72835834713464, 41.98338508363169], [-87.72833999999989, 41.98272], [-87.7288749725497, 41.98271561497918], [-87.7301633108279, 41.98270521181879], [-87.7313487363747, 41.982700000000165]]]}, "properties": {"city": "Chicago", "name": "Brynford Park", "regionid": "403270", "geo_point_2d": [41.9837443591651, -87.73019306866084], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.6051429464414, 41.751114880408764], [-87.60512333676972, 41.75036980927367], [-87.6051380760922, 41.748604692357475], [-87.60507804651792, 41.74646916308629], [-87.60504583128049, 41.744794045556354], [-87.60498511637513, 41.74266096602503], [-87.60495502932558, 41.741731524931794], [-87.60492672825634, 41.740616967129384], [-87.60490853406485, 41.739686834014236], [-87.6048945550089, 41.73903680792048], [-87.60486434183143, 41.73782762227047], [-87.60487, 41.73664000000017], [-87.60587526856727, 41.736628576493764], [-87.60791858845735, 41.736589487199204], [-87.60931161721469, 41.736565841886126], [-87.61024204736646, 41.73654873532471], [-87.61163518302926, 41.73652518996128], [-87.61247092638301, 41.736509739848174], [-87.61339996323295, 41.73649354082951], [-87.61497908899412, 41.7364739618644], [-87.6168368272171, 41.73645056068553], [-87.61832313537997, 41.736431889935346], [-87.62018010730321, 41.73640408340306], [-87.62166773758075, 41.73637654242051], [-87.62306083465499, 41.73637839076829], [-87.62472664524756, 41.73627824767154], [-87.62460828958763, 41.7389466046597], [-87.62469, 41.741822813177635], [-87.62476920201837, 41.7452617978116], [-87.62486892562349, 41.748979049659255], [-87.62492153003943, 41.75089827554095], [-87.62491291630089, 41.75089802218335], [-87.62491288815391, 41.750898021355475], [-87.62500196944762, 41.75260545005808], [-87.62529443761392, 41.75715441667337], [-87.62531520821562, 41.75809221714284], [-87.6244560941249, 41.75809325520359], [-87.62241317930403, 41.75814256379325], [-87.62064704124502, 41.758180323969114], [-87.619252562359, 41.75820783790514], [-87.61758044254411, 41.75823334556335], [-87.61609452779685, 41.75826412201744], [-87.61442398870439, 41.75830202612085], [-87.61312167109614, 41.75831], [-87.61191312395297, 41.75832752585243], [-87.6108917945831, 41.75834878314284], [-87.60977725838562, 41.758367361652326], [-87.60829122042131, 41.7583920132133], [-87.6073628771905, 41.758411059422976], [-87.60596935843294, 41.75843977026511], [-87.60533525561833, 41.7584499936494], [-87.60530585550825, 41.757708174571135], [-87.6052873365775, 41.756222496375564], [-87.60524848214708, 41.75408508938638], [-87.60516872548537, 41.75232366003458], [-87.6051429464414, 41.751114880408764]]]}, "properties": {"city": "Chicago", "name": "Chatham", "regionid": "273222", "geo_point_2d": [41.74739610317927, -87.6149778453828], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.5269961729999, 41.65385053000004], [-87.52680690599992, 41.65361145500008], [-87.52665748399993, 41.65342218700004], [-87.52647817699993, 41.65314326600008], [-87.52631879399989, 41.652894230000065], [-87.526251, 41.652862], [-87.52624907599991, 41.65287271900006], [-87.52567838799989, 41.65264444400008], [-87.5252628789999, 41.65268503900005], [-87.52539771899988, 41.65230862600003], [-87.5253075439999, 41.65212827600003], [-87.5253267019999, 41.648852292000065], [-87.5299628909999, 41.64883313400007], [-87.5299433259999, 41.65199753000007], [-87.53036520499988, 41.652], [-87.5309400699999, 41.65200569100005], [-87.53136106299988, 41.65200378000008], [-87.53197499399992, 41.65199839300003], [-87.53245277899991, 41.65200174200003], [-87.53293234999987, 41.65199], [-87.53339200499991, 41.65198675000004], [-87.53394739099988, 41.65197671500004], [-87.53411626799992, 41.65197652300008], [-87.53500780299991, 41.65197550800007], [-87.53497999999989, 41.65369], [-87.53458078699991, 41.65369], [-87.53427395199991, 41.65368699800006], [-87.5339285559999, 41.65368], [-87.53356455799988, 41.65368], [-87.53318240799987, 41.65367623200006], [-87.5331752739999, 41.65433608200003], [-87.53247376099993, 41.65432074900007], [-87.53248075799988, 41.65367], [-87.5321468799999, 41.65367], [-87.5316399999999, 41.65367], [-87.5313422529999, 41.65367], [-87.5313422529999, 41.65477205400003], [-87.5317445669999, 41.65515521000008], [-87.52967552399988, 41.65676446500003], [-87.5294073139999, 41.656936886000025], [-87.52925091699989, 41.65704859800007], [-87.52909804199992, 41.65710792800006], [-87.52891873499992, 41.65696846800005], [-87.5287892359999, 41.65670947000006], [-87.52865973699991, 41.65644051000003], [-87.52823139499989, 41.655763131000064], [-87.52768351399992, 41.65492636800007], [-87.5269961729999, 41.65385053000004]]]}, "properties": {"city": "Chicago", "name": "Harbour Point Estates", "regionid": "403385", "geo_point_2d": [41.65248546327247, -87.52922029865825], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.68376929353077, 41.78645003319634], [-87.68375593910129, 41.785982475119376], [-87.68373882375403, 41.78541338982254], [-87.68370721993331, 41.784356888367945], [-87.6836785607613, 41.78362451463918], [-87.68365502712066, 41.782649994725524], [-87.68361761198264, 41.781671856118784], [-87.68357999110965, 41.7804496177139], [-87.68355, 41.779160000000125], [-87.6843118914411, 41.77913000000013], [-87.68659002687473, 41.7791151217328], [-87.6877291634042, 41.77910599083943], [-87.68895042315485, 41.77910377396031], [-87.6900071608601, 41.779104920327214], [-87.69138760566186, 41.77906727138823], [-87.69277452066605, 41.77906], [-87.69407337827772, 41.77903249879621], [-87.69521208482801, 41.77900333673998], [-87.6959466915061, 41.77898674890079], [-87.69741187954683, 41.778966326182704], [-87.69903815036753, 41.77894609589574], [-87.7000168344302, 41.778935066386886], [-87.7014814251486, 41.77892304607315], [-87.70253917231072, 41.778908219276744], [-87.70367715923079, 41.77887681528685], [-87.70595571512908, 41.778824028092764], [-87.70856152622665, 41.778786482858465], [-87.71140842977492, 41.77873185950383], [-87.71272476996296, 41.77871051157675], [-87.71272191614722, 41.779644022574146], [-87.7127229161933, 41.78054902292276], [-87.71273791598536, 41.781458022556365], [-87.7127669155236, 41.78236002276672], [-87.7128009152921, 41.783287022330754], [-87.71282491639846, 41.784158022910695], [-87.71284691561408, 41.78508602252073], [-87.7128709140224, 41.78598407740551], [-87.7127229161933, 41.78598702268508], [-87.71127091581087, 41.78601502217723], [-87.71059391607976, 41.786018022315496], [-87.70966091533971, 41.78602202249986], [-87.70939791580689, 41.78603102201539], [-87.70814291551228, 41.78607802238334], [-87.7069579157465, 41.786080022475524], [-87.70653291594056, 41.786080022475524], [-87.70571391505968, 41.786100022498054], [-87.70446891522597, 41.786134022266566], [-87.70334091538967, 41.78616602194317], [-87.70327391589848, 41.78616502189707], [-87.70322391539241, 41.78616502189707], [-87.70203791558036, 41.78617202221979], [-87.70085091572243, 41.786187022011866], [-87.69960091565834, 41.78620202180404], [-87.69840891556993, 41.78621702249547], [-87.69717291525183, 41.78623902261018], [-87.69597291479461, 41.78626202187179], [-87.69472391477643, 41.7862910223093], [-87.69363091565428, 41.78631302242402], [-87.6935529156561, 41.78631402247011], [-87.69347991588835, 41.78631502251628], [-87.69230291559194, 41.78633002230836], [-87.69107891492774, 41.78634702219279], [-87.68986291553153, 41.786383022053656], [-87.68942091584125, 41.78639702269893], [-87.68867291553522, 41.78640002193782], [-87.68737691604873, 41.78640902235282], [-87.6862139154983, 41.78642302209889], [-87.68498791564129, 41.786445022213684], [-87.68412991566044, 41.78646202209786], [-87.68383792918002, 41.786451983865305], [-87.68376929353077, 41.78645003319634]]]}, "properties": {"city": "Chicago", "name": "Chicago Lawn", "regionid": "269576", "geo_point_2d": [41.78259050221284, -87.69820521196333], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.61762410100054, 41.845681157920914], [-87.61795191558109, 41.845686021780274], [-87.61886291530698, 41.845700022425724], [-87.61962891554337, 41.84568402258747], [-87.62048191619294, 41.84567802231075], [-87.62212991571778, 41.84564502258833], [-87.62337791568977, 41.845626022611796], [-87.62353091554789, 41.84562302247353], [-87.62364196092228, 41.84562302247353], [-87.62362228190032, 41.8450504896577], [-87.62360495662374, 41.84449360576475], [-87.62354284966025, 41.84275459230192], [-87.62351617071738, 41.841495975112075], [-87.62345646787995, 41.839335779447595], [-87.62344261535735, 41.838329189529325], [-87.62342589220036, 41.837114005709445], [-87.6233783959505, 41.8356435185678], [-87.62332590375563, 41.83341482394161], [-87.62329037476051, 41.831673235158384], [-87.62328, 41.83103], [-87.62208638490443, 41.83103780140601], [-87.62104029020414, 41.831057742745074], [-87.61978665962738, 41.83106251377683], [-87.61846307389001, 41.8310886435724], [-87.61729999999976, 41.83112], [-87.6173671265557, 41.82986230290303], [-87.6173349530809, 41.827910730365495], [-87.61731101846792, 41.82672446563585], [-87.61729304905103, 41.82602815073553], [-87.6172643071039, 41.82449593580478], [-87.61724480352854, 41.823821012752575], [-87.61712591527684, 41.82382002197268], [-87.6170289153022, 41.82382102201859], [-87.6169849159718, 41.82382102201859], [-87.61693691555796, 41.82382202206468], [-87.61671291512482, 41.82382302211077], [-87.61547591566, 41.82383202252558], [-87.61435691533916, 41.82384602227184], [-87.61261688998964, 41.8238709685654], [-87.6098419158559, 41.823916021901034], [-87.60970189231521, 41.82391815239522], [-87.6096279680444, 41.82388484330569], [-87.60796491566745, 41.823853022594285], [-87.60794191550649, 41.823853022594285], [-87.60722291563802, 41.823857021879284], [-87.60696991566674, 41.823864022202095], [-87.60689581333031, 41.82387817753091], [-87.60689293755486, 41.8238656570197], [-87.60689, 41.82387], [-87.60695162965786, 41.824011119363036], [-87.60692999999988, 41.82399], [-87.6056791782141, 41.82207612102299], [-87.60487373899318, 41.82076014211741], [-87.60456138093988, 41.82026834418241], [-87.60440999999985, 41.82003000000032], [-87.60430341684412, 41.81912404192626], [-87.60423919701813, 41.81857412385038], [-87.60413875683155, 41.81767159998532], [-87.60406333983349, 41.81697544304149], [-87.60405086977242, 41.81672182705745], [-87.60404390603136, 41.81672193198206], [-87.60404, 41.81668000000026], [-87.60402912414384, 41.816204187800835], [-87.60400524710462, 41.815159565211346], [-87.60399260613077, 41.81425347298485], [-87.60398422218346, 41.813486346800055], [-87.60398, 41.812929072041804], [-87.60393026388375, 41.81182013587367], [-87.60390354614444, 41.81097958300635], [-87.60388806998098, 41.81028314815203], [-87.60386681086581, 41.80945470900347], [-87.60385225283095, 41.808887412193044], [-87.60383969414022, 41.80826047816836], [-87.60382413494291, 41.80742463285912], [-87.60380998501228, 41.806519041797074], [-87.60378587096818, 41.80540314405646], [-87.6037631810171, 41.80463735995811], [-87.60373999999983, 41.80345288287689], [-87.60373999999983, 41.802825781994734], [-87.60375306630138, 41.80219386786784], [-87.60411391531596, 41.802187022113294], [-87.60621891522251, 41.80215502243679], [-87.60636390931595, 41.802153025941905], [-87.60647491533199, 41.80215502243679], [-87.60782791564758, 41.802122021814824], [-87.60999191557573, 41.80208702200022], [-87.61125791547816, 41.8020680220237], [-87.61246191611974, 41.80204502186281], [-87.61416091529753, 41.802013022186294], [-87.61493791514162, 41.802000022486496], [-87.61561291567993, 41.801995022255866], [-87.61608691594583, 41.801991021172135], [-87.61611491633744, 41.80198602274042], [-87.61613699904011, 41.80198550916244], [-87.61628691527271, 41.80198202255606], [-87.61636591531709, 41.80198102250979], [-87.61643491579974, 41.8019800224637], [-87.61653491591281, 41.80197902241761], [-87.61660391549619, 41.80197802237152], [-87.6167469157927, 41.80197602227934], [-87.61771691554043, 41.801964022625626], [-87.61792791627356, 41.801957022302815], [-87.61798691539587, 41.801957022302815], [-87.61845891556975, 41.80194802188818], [-87.61936591511117, 41.80193202204975], [-87.62097391549007, 41.80190202246559], [-87.6226539164899, 41.801882022442804], [-87.62422591610873, 41.801862022420366], [-87.62492191581636, 41.80185702218973], [-87.62580791528909, 41.80185702308928], [-87.62583291554216, 41.801855022996925], [-87.62586354596684, 41.80185502209755], [-87.62588256625308, 41.80254827072281], [-87.62590082278831, 41.80310509504955], [-87.6259309381752, 41.804009861644175], [-87.62595029463624, 41.8049167942761], [-87.62598243254858, 41.80624018417102], [-87.62600404295227, 41.80707576104893], [-87.62601873038311, 41.80770413686182], [-87.62603979190186, 41.8082613948486], [-87.62605671010833, 41.808889055961366], [-87.6260682078502, 41.809655206971385], [-87.62608317891618, 41.81055846128397], [-87.6261012173703, 41.81167552087014], [-87.62612191983067, 41.812582352297504], [-87.62614655614756, 41.81362880205334], [-87.62616184219475, 41.81425552998742], [-87.62618282399761, 41.81516066789729], [-87.6261916349384, 41.815578096304506], [-87.62622303224991, 41.816149096750536], [-87.6262569191528, 41.81682890223729], [-87.6262923432783, 41.817810656574316], [-87.62632428967582, 41.81885309087657], [-87.62635336122825, 41.81989862392781], [-87.6263679001006, 41.82080310896207], [-87.62638259382123, 41.82150004976847], [-87.6264099999999, 41.8226839728642], [-87.6264099999999, 41.823380751491534], [-87.62641295525796, 41.82368699756749], [-87.62641692221284, 41.823686949770355], [-87.62642732252056, 41.82477512541328], [-87.62643999999987, 41.82609818813755], [-87.62644865752436, 41.82693482136993], [-87.62648965325012, 41.828327517009456], [-87.62652084407, 41.82944343849889], [-87.62654021681084, 41.83020932287135], [-87.62653999999972, 41.8309700000001], [-87.62710414368199, 41.83100140619703], [-87.6283578945683, 41.83097553480898], [-87.6289987880902, 41.83096515977694], [-87.6295270462529, 41.830965890026405], [-87.62952718134646, 41.83097342829916], [-87.62953, 41.8309700000001], [-87.63085598117387, 41.83095006185207], [-87.6308696710589, 41.8313936210125], [-87.63090302501558, 41.832716659793256], [-87.63093813420677, 41.83410933187811], [-87.63096935492541, 41.835362796100654], [-87.6309986248153, 41.83654659609309], [-87.63102965886652, 41.83780175310804], [-87.6310380618968, 41.83822005291261], [-87.62976890917372, 41.8382548034555], [-87.62976881992066, 41.83827485178418], [-87.62940272811464, 41.838264534109825], [-87.62863660876411, 41.838270000000314], [-87.62738226218336, 41.83827460291872], [-87.62673999999983, 41.83828], [-87.62675821280399, 41.838849144005785], [-87.62680914593282, 41.84031046176201], [-87.62684831687594, 41.841359370903604], [-87.62687945119815, 41.842403118217064], [-87.62689088778424, 41.84386893482436], [-87.62692660720667, 41.84498518517287], [-87.62694614288277, 41.84556802300739], [-87.62696825790984, 41.84630639886829], [-87.62698295691557, 41.847205585002904], [-87.6262342899231, 41.847258440028156], [-87.62569733904884, 41.847655073062306], [-87.62461647606963, 41.84793452092236], [-87.62305894012044, 41.847904812184844], [-87.62145891411569, 41.84790358417746], [-87.61992506891089, 41.847922831019794], [-87.61811525112874, 41.847950000000104], [-87.61787012597897, 41.847952881118914], [-87.61766452290573, 41.84770211595952], [-87.61765999999987, 41.84758000000023], [-87.61765999999987, 41.84721058077098], [-87.61763225681842, 41.84616519484612], [-87.61762410100054, 41.845681157920914]]]}, "properties": {"city": "Chicago", "name": "Bronzeville", "regionid": "403145", "geo_point_2d": [41.81835502940534, -87.61751164051734], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.64651998889116, 41.73254994653049], [-87.64638123756782, 41.731882104762825], [-87.64638123756782, 41.73151048949501], [-87.64600962230001, 41.730953066593216], [-87.64545219939822, 41.73039564369143], [-87.64508058413044, 41.73020983605762], [-87.64433735359472, 41.72993112460672], [-87.64201475817084, 41.72993112460672], [-87.63904183602807, 41.72993112460672], [-87.63801989404146, 41.72993112460672], [-87.63690504823796, 41.72993112460672], [-87.63560439480058, 41.72993112460672], [-87.63448954899708, 41.72993112460672], [-87.63373542905406, 41.72993112460672], [-87.63371991577993, 41.72979002224288], [-87.63370391594168, 41.72906202285891], [-87.63370086094471, 41.72892202000249], [-87.63365291538936, 41.72696902260625], [-87.63364191578164, 41.726533022293225], [-87.6336159154825, 41.725734022334244], [-87.63360691596695, 41.725475022086314], [-87.63359891559824, 41.72522202211504], [-87.63355791550691, 41.72399602225803], [-87.6335489159913, 41.72332702199635], [-87.63354391576075, 41.72292502235123], [-87.6335159153694, 41.72166102254116], [-87.63351391527732, 41.72157102288915], [-87.63351211071539, 41.72150603187569], [-87.63435971191075, 41.721494813773255], [-87.63510257674547, 41.72149309894657], [-87.63640391660293, 41.72149367663265], [-87.63779663628758, 41.721458333808165], [-87.63891118711416, 41.72144062601434], [-87.63984041097592, 41.72142780153687], [-87.64132834295782, 41.721541194284065], [-87.64262740009958, 41.72153560178666], [-87.64392857783714, 41.72139], [-87.645043328961, 41.72139523805846], [-87.64634416248259, 41.72139320219926], [-87.64755123061143, 41.72136519902827], [-87.64875886744473, 41.72133861144609], [-87.65006022964218, 41.72143000000014], [-87.6512680100482, 41.721294304666245], [-87.65238262940643, 41.72127913722285], [-87.65368325579595, 41.72126390773949], [-87.65489052236825, 41.72124802422339], [-87.65600505891778, 41.72122966844764], [-87.65730725183876, 41.7213540229014], [-87.65851566364502, 41.72133498206361], [-87.65990781921788, 41.72117281509139], [-87.66092960108803, 41.7211666714157], [-87.66213750004442, 41.721278437499286], [-87.6636317154838, 41.72121738556112], [-87.66365391607496, 41.721253022619486], [-87.66400791530612, 41.721839022652524], [-87.66425691509295, 41.72237802231742], [-87.66458691591572, 41.723049022671276], [-87.66516191544149, 41.724233022391054], [-87.6653799155981, 41.72472802182646], [-87.66556791527098, 41.72515502172465], [-87.66601591613717, 41.72607802290288], [-87.6665149157569, 41.727025022489464], [-87.66718991539587, 41.72823902269285], [-87.66729891592385, 41.728441023011186], [-87.66737291573777, 41.72857702298514], [-87.66742160883602, 41.72866737253067], [-87.66729878548303, 41.728994783224344], [-87.66703556231772, 41.72943725039258], [-87.66693164262318, 41.7296610774269], [-87.66662582111825, 41.7302102191652], [-87.66664582759252, 41.73020940025307], [-87.66644846203128, 41.73058145132543], [-87.6658910391294, 41.73104597041022], [-87.66535119534525, 41.73136987668082], [-87.66535024961003, 41.73132554195622], [-87.66496200095982, 41.73151048949501], [-87.6644974818752, 41.73169629712892], [-87.66392312209722, 41.73209834897354], [-87.66381291531043, 41.73210202269823], [-87.66361191593762, 41.73211202225995], [-87.66340291529667, 41.732124022813025], [-87.66312891615604, 41.73215402239727], [-87.6629709160674, 41.73216502290436], [-87.66281091588645, 41.73219302239651], [-87.66273591512707, 41.73220302285741], [-87.66259991515331, 41.73221702260339], [-87.66251191559348, 41.73222402292602], [-87.6623149155056, 41.73223102234937], [-87.66173991597975, 41.73223102234937], [-87.66052291563818, 41.73224802223372], [-87.65928591527398, 41.732267022210245], [-87.65807891539332, 41.73227702177196], [-87.65685991495957, 41.7322930225094], [-87.65566791487107, 41.732312022485836], [-87.65441191542963, 41.7323370218398], [-87.65338691494541, 41.73235402262335], [-87.65320691564123, 41.732373021700674], [-87.65293291560135, 41.73240202213802], [-87.65202091582938, 41.73241702193027], [-87.65083091493385, 41.732445022321606], [-87.64960191493839, 41.7324680224825], [-87.64872491588058, 41.73245402183705], [-87.64837691602666, 41.73245902206768], [-87.64822891549973, 41.732456022828515], [-87.64801891571194, 41.73248702245902], [-87.64792991610587, 41.73251102176654], [-87.64782991599287, 41.73252002218153], [-87.64723191630611, 41.73253002264253], [-87.64704491488067, 41.73253302278079], [-87.64669291574177, 41.73254702252686], [-87.64656691532953, 41.73255102271123], [-87.64651877372192, 41.7325531226281], [-87.64651998889116, 41.73254994653049]]]}, "properties": {"city": "Chicago", "name": "Brainerd", "regionid": "403378", "geo_point_2d": [41.72651008056152, -87.65063955081521], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.71284691599992, 41.78508602300007], [-87.71282491599992, 41.78415802300008], [-87.71280091499993, 41.783287022000074], [-87.7127669159999, 41.782360023000074], [-87.7127379159999, 41.78145802300008], [-87.71272291599988, 41.78054902300004], [-87.7127219159999, 41.77964402300005], [-87.71272476999988, 41.77871051200008], [-87.71272478399992, 41.778705892000055], [-87.71287004799989, 41.778703150000055], [-87.7128479159999, 41.77787502200004], [-87.7128289159999, 41.776863022000036], [-87.71284391499994, 41.776053023000074], [-87.71277691499992, 41.775044022000074], [-87.7127599159999, 41.774135022000046], [-87.71274391499989, 41.77323302300004], [-87.71275591499993, 41.772349023000054], [-87.7126889159999, 41.77147002300005], [-87.71268791499993, 41.77143102300004], [-87.7126869539999, 41.77139100800007], [-87.71266691499993, 41.770737022000034], [-87.71264091599993, 41.769906022000036], [-87.71262391599991, 41.769370022000075], [-87.71265991599992, 41.76868602200005], [-87.71270491499988, 41.76782202300006], [-87.71275191499987, 41.76691502200003], [-87.71265691599987, 41.76607702200004], [-87.71257891499992, 41.76517502300003], [-87.71257008099988, 41.76430296500006], [-87.71245573799989, 41.76415606600004], [-87.71253491399993, 41.76372002300007], [-87.71240991499988, 41.760874022000046], [-87.7124269149999, 41.760502022000075], [-87.7124539149999, 41.75990802100005], [-87.7123059159999, 41.75877702200005], [-87.71225594999991, 41.75721597100004], [-87.71254391599989, 41.75726902200005], [-87.71268491499988, 41.757294023000036], [-87.71295291599992, 41.75736502400007], [-87.71353791499988, 41.75757202300007], [-87.7140729149999, 41.75785602200005], [-87.71462891499992, 41.75811002300007], [-87.7152559159999, 41.75841202200006], [-87.71614108199991, 41.75884592600004], [-87.71613775899992, 41.75889412500003], [-87.71691256599988, 41.75921442400004], [-87.71641892199993, 41.75898302100005], [-87.71694893299988, 41.75912897400007], [-87.71744091499991, 41.759331022000076], [-87.7189918799999, 41.75996115200006], [-87.7191809399999, 41.759975940000054], [-87.72123191499989, 41.76105102300004], [-87.72146591499988, 41.76116202200006], [-87.72172291599989, 41.76125702300003], [-87.72210291499992, 41.761367022000044], [-87.72217291599989, 41.76138702200006], [-87.7222430219999, 41.76140716100007], [-87.72288195199988, 41.761978312000046], [-87.72849658099993, 41.76376848400007], [-87.73826115299993, 41.76669785600005], [-87.74215502799987, 41.768059380000054], [-87.74215502799987, 41.76811648700003], [-87.74215502799987, 41.76817425900002], [-87.74215502799987, 41.768231378000046], [-87.74215502799987, 41.76828848500007], [-87.74215502799987, 41.76867425900007], [-87.74215502799987, 41.768731378000034], [-87.74215502799987, 41.768788485000066], [-87.74215502799987, 41.77117425900006], [-87.74215502799987, 41.771230946000045], [-87.74225502799992, 41.77263159400006], [-87.74225502799992, 41.77268848500005], [-87.74225502799992, 41.77297425900007], [-87.74225500099992, 41.773031431000035], [-87.7421644329999, 41.77466165100008], [-87.74215836299992, 41.77467835700007], [-87.74205718599988, 41.77507961200006], [-87.74207930399987, 41.77573033300006], [-87.74209849599991, 41.77621780600003], [-87.74213179699987, 41.776948360000056], [-87.74216408599992, 41.77759772600006], [-87.74219993599989, 41.77831872100005], [-87.74208999999989, 41.77832], [-87.74221793399992, 41.778983565000026], [-87.74227155699991, 41.78094175400007], [-87.74229742199991, 41.78275559400002], [-87.7421264999999, 41.783182160000074], [-87.74191121899989, 41.78353314100007], [-87.74147281799989, 41.784234314000045], [-87.74141393699993, 41.78421589000003], [-87.7411559159999, 41.78423002300007], [-87.7405959149999, 41.78422702200003], [-87.74020291599992, 41.784226022000034], [-87.73988691599992, 41.78424302200005], [-87.73965090699991, 41.784259969000054], [-87.73941591399992, 41.78432702400004], [-87.73920391699993, 41.78443902400004], [-87.73901887599992, 41.784607988000054], [-87.73892291499992, 41.78476102200005], [-87.73872091599989, 41.78505602200004], [-87.7386739159999, 41.78512502300003], [-87.7386249129999, 41.78522402300006], [-87.73857191499988, 41.785291022000074], [-87.73848391599992, 41.785375021000036], [-87.73839891499993, 41.78543902200004], [-87.73824091499992, 41.785527022000046], [-87.73807491599989, 41.78559002200005], [-87.73789491599989, 41.785636021000066], [-87.7376009149999, 41.78566202200005], [-87.73755391499989, 41.785664023000024], [-87.73741191599991, 41.78567102300006], [-87.73631891599989, 41.78570202200006], [-87.73511891499992, 41.78570702200005], [-87.7338809159999, 41.78571702200008], [-87.73264891599987, 41.785727022000046], [-87.7317429159999, 41.785735022000026], [-87.7314249159999, 41.785738022000075], [-87.73093991599988, 41.78574302200007], [-87.73019791599994, 41.78575002300004], [-87.72895791499988, 41.785760022000034], [-87.7277619149999, 41.785780022000026], [-87.72653091599993, 41.78579602200005], [-87.72530091599991, 41.78580502200003], [-87.72407791599991, 41.78580502200003], [-87.7229409149999, 41.78581902200005], [-87.72288091599994, 41.785820022000046], [-87.72281291499989, 41.785821022000036], [-87.72039991599988, 41.785865022000046], [-87.7193819159999, 41.78589102200004], [-87.71797291499989, 41.78590902200005], [-87.71555091499991, 41.785951022000056], [-87.71320491599991, 41.78598102200004], [-87.71307391999993, 41.78598196800003], [-87.71287091399994, 41.785984077000066], [-87.71284691599992, 41.78508602300007]]]}, "properties": {"city": "Chicago", "name": "West Lawn", "regionid": "269614", "geo_point_2d": [41.77409860495315, -87.72613078394619], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.6779739151764, 41.744800022573905], [-87.67795091501551, 41.743818022273366], [-87.67788191543214, 41.74283802206464], [-87.6778569142797, 41.742623020247855], [-87.67760891543823, 41.7418880223398], [-87.67747791479555, 41.74140502255803], [-87.67738391495908, 41.74114302217212], [-87.6772959153991, 41.74097802265984], [-87.67727291523829, 41.74093702256877], [-87.67652591497844, 41.73997202215232], [-87.6756509160126, 41.73902102238112], [-87.67525391659802, 41.73852302190808], [-87.67485891547705, 41.73802802247259], [-87.67437791578766, 41.7374790223467], [-87.67375391580175, 41.73675402220154], [-87.67337491631758, 41.7362570217746], [-87.67313991537736, 41.735845022567936], [-87.67307491597835, 41.735710022640156], [-87.67304685083589, 41.7356520955096], [-87.67301991524175, 41.7355960227811], [-87.67279291556964, 41.735124022607216], [-87.6725029147921, 41.73469302252466], [-87.67153891622047, 41.73349402301281], [-87.67072491557013, 41.7325760229643], [-87.66995191501091, 41.73178602252093], [-87.66986191535894, 41.731695022822656], [-87.66940091569205, 41.731305021932236], [-87.6692529160643, 41.731180022465274], [-87.66851691541221, 41.73046302268884], [-87.66824691016063, 41.73016492891259], [-87.66797087045842, 41.72983504859895], [-87.66754591561859, 41.728898022493524], [-87.66737291573777, 41.72857702298514], [-87.66729891592385, 41.728441023011186], [-87.66718991539587, 41.72823902269285], [-87.6665149157569, 41.727025022489464], [-87.66601591613717, 41.72607802290288], [-87.66556791527098, 41.72515502172465], [-87.6653799155981, 41.72472802182646], [-87.66516191544149, 41.724233022391054], [-87.66458691591572, 41.723049022671276], [-87.66425691509295, 41.72237802231742], [-87.66400791530612, 41.721839022652524], [-87.66365391607496, 41.721253022619486], [-87.66361591522262, 41.72119202250555], [-87.66357891531565, 41.7211330224837], [-87.66316391597083, 41.72047002249875], [-87.66259991515331, 41.71945402242933], [-87.6621479159014, 41.718529022957476], [-87.66155591559192, 41.71737002259148], [-87.6612539151605, 41.716787022696806], [-87.66077591560946, 41.715833022787606], [-87.65984391581486, 41.71402902236692], [-87.65983969138534, 41.7140211045819], [-87.66102411998367, 41.714003341120446], [-87.66223117294783, 41.71397449588217], [-87.66371712180262, 41.71395812076802], [-87.66594670007646, 41.71394789782446], [-87.66731, 41.71391], [-87.66723999999986, 41.71286], [-87.66743324308959, 41.712465207381314], [-87.66780485835045, 41.71135036159843], [-87.66817647361142, 41.71060713107678], [-87.6684, 41.71023], [-87.66855472081006, 41.70968002221544], [-87.66867604442565, 41.70912343727544], [-87.66875083060307, 41.70839152129526], [-87.66888237612955, 41.7073660118864], [-87.66914623867336, 41.705583620759356], [-87.66938805207705, 41.70362933651354], [-87.66955213750623, 41.702255345026295], [-87.66971966283246, 41.700940525041936], [-87.66986286899859, 41.69973893992955], [-87.66994356204061, 41.69921421187237], [-87.671100914916, 41.69917902221683], [-87.67271191633245, 41.69915802214812], [-87.67290091515208, 41.69915802214812], [-87.67341291537127, 41.699150021779225], [-87.67410491489454, 41.69914302235596], [-87.67432291595024, 41.69913602203332], [-87.6753369159274, 41.69911502196462], [-87.67648491488711, 41.699107022495085], [-87.67766991555209, 41.699079022103746], [-87.67888291570938, 41.69907202178111], [-87.68006891552123, 41.699058022034954], [-87.68095191575499, 41.699057021988864], [-87.68127791549408, 41.69906102217331], [-87.68128039955639, 41.69906098302205], [-87.6813143521183, 41.700107739609855], [-87.68135622604325, 41.70187437996182], [-87.68142599135297, 41.7036414925095], [-87.68152496776285, 41.70559093650265], [-87.68156892746691, 41.7072622722799], [-87.68162335778032, 41.709217412302586], [-87.68167568908397, 41.710980496109265], [-87.68173174643057, 41.713022871500385], [-87.68175024373649, 41.71363299961605], [-87.68174900015818, 41.71363299961605], [-87.68175193374657, 41.713688744092025], [-87.68184900027117, 41.715532999965276], [-87.68184985013045, 41.71714772277648], [-87.68184993466673, 41.7173080242301], [-87.68185000031725, 41.71743300031461], [-87.68187759061779, 41.717902042618185], [-87.68195000043032, 41.71913300043777], [-87.68204999964404, 41.72093299977463], [-87.68210100019625, 41.721388000064174], [-87.68214999975703, 41.721832999892804], [-87.68214999975703, 41.72244800036317], [-87.68214999975703, 41.72273300001088], [-87.68214999975703, 41.72451100013224], [-87.68214999975703, 41.72533300025221], [-87.68234999998309, 41.72843300015948], [-87.68234999998309, 41.73013300028254], [-87.68245000009625, 41.73093300028763], [-87.68245000009625, 41.73152499969783], [-87.68245000009625, 41.73556530373921], [-87.68245000009625, 41.73563300020489], [-87.68735000023969, 41.735533000091976], [-87.68754999956637, 41.735533000091976], [-87.68759286997886, 41.735533000091976], [-87.68763053229816, 41.73624858415999], [-87.68763053229816, 41.738292468133075], [-87.68764, 41.74267], [-87.68577131674421, 41.74268862492358], [-87.68484232275455, 41.742707319932414], [-87.68372841634556, 41.74273146247069], [-87.6825, 41.74274], [-87.68257888336147, 41.74358917082978], [-87.68260684014183, 41.74451732326256], [-87.68214920709748, 41.74451702386952], [-87.6817775918297, 41.74451702386952], [-87.6808485536601, 41.74460992768634], [-87.6801053231244, 41.74479573532024], [-87.6790833811379, 41.745353158221945], [-87.67871176587, 41.746096388757806], [-87.6783401506022, 41.74693252311024], [-87.67804022218749, 41.74798227256169], [-87.67804891593568, 41.74740102286132], [-87.67804891593568, 41.747349022263], [-87.67804291565913, 41.74660002281032], [-87.67803691538252, 41.74644302276757], [-87.6779739151764, 41.744800022573905]]]}, "properties": {"city": "Chicago", "name": "Beverly", "regionid": "272951", "geo_point_2d": [41.72098494177835, -87.67519041414809], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.62208638490449, 41.831037801405934], [-87.62328, 41.83103], [-87.62329037476057, 41.83167323515831], [-87.6233259037557, 41.83341482394145], [-87.62337839595055, 41.835643518567636], [-87.62342589220043, 41.837114005709374], [-87.62344261535742, 41.83832918952925], [-87.62243594232723, 41.83834290984001], [-87.62111322508156, 41.83836849464095], [-87.61999743296924, 41.83838591184135], [-87.61874321243344, 41.83839592386429], [-87.61745179985606, 41.838360049997924], [-87.6174458084485, 41.837734016748506], [-87.61743435670228, 41.836898039269], [-87.61741988951296, 41.83613176840944], [-87.61740418069427, 41.83543534411645], [-87.6173212611918, 41.83243819389838], [-87.6173, 41.83112], [-87.61846307389017, 41.831088643572315], [-87.61978665962744, 41.83106251377675], [-87.62104029020429, 41.83105774274501], [-87.62208638490449, 41.831037801405934]]]}, "properties": {"city": "Chicago", "name": "The Gap", "regionid": "403346", "geo_point_2d": [41.83471714217079, -87.62037243063493], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.64824491499995, 41.910910022000046], [-87.64826105699993, 41.91090986400007], [-87.64828793699994, 41.91207471900002], [-87.64836154299995, 41.914419744000035], [-87.64846276299993, 41.91717460000006], [-87.6485, 41.91818], [-87.64852305799994, 41.91887558400003], [-87.64857914499996, 41.92057391900004], [-87.64864832699998, 41.92267926700003], [-87.64870400499993, 41.92421343200004], [-87.64874164099994, 41.92546030400006], [-87.64697303099993, 41.92547927100002], [-87.64559706399994, 41.92550228000004], [-87.64268056399997, 41.925552953000064], [-87.64098053199993, 41.92557437100003], [-87.63984769799997, 41.92559189300004], [-87.6384715149999, 41.92560626000004], [-87.63758171899994, 41.92562509700007], [-87.63620768499997, 41.925649548000074], [-87.63474853099993, 41.92567981700005], [-87.63394668799995, 41.92570219700007], [-87.63289534099992, 41.925830902000015], [-87.62962533699991, 41.92637590300005], [-87.62952788999995, 41.92639201100007], [-87.62952899599996, 41.92635748400004], [-87.62952940499996, 41.92633469400004], [-87.62953001999992, 41.92630039500005], [-87.62957383499996, 41.92614960800005], [-87.62957604099994, 41.92614588700007], [-87.62958401499996, 41.92613377400004], [-87.62957653399994, 41.926088014000065], [-87.62898340999998, 41.922741101000035], [-87.62895359599995, 41.92269294000004], [-87.62894802799995, 41.92268394600005], [-87.62895494799993, 41.922635507000045], [-87.62896209599994, 41.92262443900005], [-87.62895449099992, 41.922577919000034], [-87.62876919299997, 41.92153230600007], [-87.62859402799994, 41.92130794600006], [-87.62844702799997, 41.92113294600004], [-87.62838380199996, 41.921020742000046], [-87.62837602799998, 41.92100694600003], [-87.62838369699992, 41.92096421500002], [-87.62838992199991, 41.92092953500002], [-87.62844626799992, 41.92087862600004], [-87.62862942499996, 41.92080675300007], [-87.62863840099996, 41.920803249000066], [-87.62863042199994, 41.92074924200005], [-87.62855704999997, 41.920335215000065], [-87.62849954999994, 41.92026850800004], [-87.62840402799996, 41.920180946000066], [-87.62814202799994, 41.91986994600005], [-87.62783602799996, 41.91957094600007], [-87.62778346699996, 41.919496404000014], [-87.62778102799994, 41.919492946000034], [-87.62778344499995, 41.91943976100003], [-87.62778399499994, 41.91942765900006], [-87.62784224299995, 41.919377611000066], [-87.62823307799992, 41.91917443900006], [-87.62824178399994, 41.919159346000065], [-87.62825063299992, 41.91914269000005], [-87.62824217599994, 41.919101480000045], [-87.62823702799993, 41.91907794600006], [-87.62813302799995, 41.91893494600002], [-87.62788102799996, 41.91868194600004], [-87.62786191699995, 41.91866241900003], [-87.62780578299997, 41.918607], [-87.62785988899996, 41.918606366000056], [-87.62819765299996, 41.91860078300003], [-87.63611, 41.91847], [-87.63685498799997, 41.918347961000045], [-87.6388, 41.91832], [-87.63952389799991, 41.91733775100004], [-87.64049542299993, 41.91563758200004], [-87.64154790899991, 41.91401837200004], [-87.64276231599995, 41.912318203000034], [-87.64323, 41.91108], [-87.64324495099997, 41.91103381400005], [-87.64341836299995, 41.91102974100004], [-87.64357191499992, 41.911000023000035], [-87.64373891499996, 41.91098302300002], [-87.64433091599992, 41.91096802200007], [-87.64546091599998, 41.91095402200005], [-87.64584691499992, 41.91094902200006], [-87.64605191599996, 41.91094802200006], [-87.64660691499994, 41.91094502300006], [-87.64693691499996, 41.91093702200003], [-87.64716791599993, 41.91093102200004], [-87.64743091599993, 41.91092202300006], [-87.64824491499995, 41.910910022000046]]]}, "properties": {"city": "Chicago", "name": "Lincoln Park", "regionid": "269590", "geo_point_2d": [41.92013360497328, -87.6402494720742], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.66813975199992, 41.92514524600005], [-87.66891206199993, 41.92513351200005], [-87.67166414099992, 41.925099836000065], [-87.67425520899988, 41.925070301000055], [-87.67425399999988, 41.925064], [-87.6745169079999, 41.92506100300004], [-87.67452086199992, 41.92507978300005], [-87.67449408699989, 41.92544124700003], [-87.67448069999992, 41.92581609900008], [-87.67449408699989, 41.926097238000054], [-87.67469490099988, 41.92643192800004], [-87.67521701599989, 41.92687371700003], [-87.67619430899991, 41.927435995000046], [-87.67686368699991, 41.92773052200003], [-87.67754645299992, 41.927917948000065], [-87.6784300329999, 41.928172312000065], [-87.67919312399991, 41.928332963000045], [-87.67963491399989, 41.92848022600002], [-87.67978217799991, 41.92861410200003], [-87.68009009199992, 41.929296868000044], [-87.68046494399991, 41.930006409000036], [-87.68078624499991, 41.93070256300006], [-87.68101383399993, 41.93113096500007], [-87.68136191099987, 41.93131839100004], [-87.68206567799987, 41.931606820000056], [-87.6821785529999, 41.93165308000004], [-87.68263372999989, 41.93193421900007], [-87.68281071699992, 41.93222919600003], [-87.68271179099989, 41.93223082000008], [-87.68252219899993, 41.93223394400006], [-87.68233481799989, 41.93223639500007], [-87.68207624699991, 41.93223926400008], [-87.68157916999989, 41.93224511500004], [-87.68128464599991, 41.932248904000055], [-87.6809877449999, 41.932251718000074], [-87.68069230999991, 41.93225478600004], [-87.6804189289999, 41.93225762300004], [-87.68003513199992, 41.93226208100002], [-87.67953804999989, 41.93226830800005], [-87.67921695299991, 41.932272352000034], [-87.67886000199988, 41.932274985000035], [-87.6787511009999, 41.932275789000066], [-87.67856426299991, 41.93227914100002], [-87.67824661399992, 41.93228383000007], [-87.6782170479999, 41.932284056000064], [-87.6782089729999, 41.93227745800003], [-87.67810491599992, 41.93227802200004], [-87.67625791599991, 41.932295022000055], [-87.6756579149999, 41.932317022000056], [-87.67513991599989, 41.93232002200005], [-87.67472891599992, 41.932322022000044], [-87.67388691599989, 41.93232702200003], [-87.67343591699994, 41.93233196800008], [-87.67336370799988, 41.93233270800004], [-87.6720639149999, 41.93234602300003], [-87.6708179149999, 41.93237202200003], [-87.66958591599989, 41.93237602200008], [-87.66838291599991, 41.93239502200002], [-87.66837760199988, 41.93239519200005], [-87.66835614199992, 41.93182924200004], [-87.66829519099988, 41.930371117000035], [-87.66825802999992, 41.92948210500002], [-87.6682496909999, 41.92842979400005], [-87.6682300819999, 41.92778270400002], [-87.6681931819999, 41.92664977900005], [-87.66816378599991, 41.925681121000075], [-87.66813975199992, 41.92514524600005]]]}, "properties": {"city": "Chicago", "name": "Lathrop Homes", "regionid": "403296", "geo_point_2d": [41.92925641154007, -87.67373666689757], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.64592332608169, 41.823454992215964], [-87.64590561713491, 41.82288581257547], [-87.64586142863992, 41.821140716118535], [-87.6458167675977, 41.819856714523176], [-87.64576668848457, 41.81812113357133], [-87.64573237263858, 41.81701989837922], [-87.64567211021426, 41.815465598106385], [-87.64560574066043, 41.8140873421738], [-87.64556339197036, 41.81270819123191], [-87.64551519755355, 41.8114272944343], [-87.64549, 41.810054635177465], [-87.64544407102603, 41.80822232132269], [-87.64542820054243, 41.80749028366349], [-87.64539837316204, 41.806575070475226], [-87.64535, 41.80513], [-87.64418140316377, 41.805159699916786], [-87.6425329054633, 41.805190976171964], [-87.64198351234278, 41.80520543388584], [-87.6414370197707, 41.80520533774718], [-87.64070489568995, 41.80529429693964], [-87.64018531281705, 41.805451106619024], [-87.6395733909119, 41.80568421972561], [-87.63881577331507, 41.80615044593915], [-87.63820385140993, 41.80658753301414], [-87.63794159916483, 41.8070537592276], [-87.63750451208979, 41.80781137682443], [-87.637183981568, 41.80859813355975], [-87.63698000759963, 41.8094140294331], [-87.63677603363125, 41.810142507891634], [-87.63660119880109, 41.81107496031855], [-87.63642636397118, 41.81183257791539], [-87.6362806682795, 41.81273589120374], [-87.63610583344939, 41.81358092621574], [-87.63607914065939, 41.813994664460466], [-87.63595584741462, 41.813971546976994], [-87.63596291305458, 41.81394902290932], [-87.63592591584558, 41.812469022135375], [-87.6359569154759, 41.81185002238002], [-87.63597786951858, 41.81169496535342], [-87.63598691595949, 41.81162802293842], [-87.63599605037336, 41.811566020979754], [-87.63597427886742, 41.81145618704127], [-87.63595478854394, 41.81016008052805], [-87.63595478854394, 41.80781768795818], [-87.63583766891537, 41.80094666975311], [-87.63568150941073, 41.79630092448943], [-87.63564852691304, 41.79435495712467], [-87.63778091591911, 41.79438502219643], [-87.63807791522072, 41.7943910224728], [-87.63903591621373, 41.79436302208155], [-87.63995791554741, 41.794318022705234], [-87.64000091573091, 41.79431702265915], [-87.64028091514808, 41.794311022382516], [-87.64142591576791, 41.79428402203709], [-87.642650915579, 41.7942690222451], [-87.64386891596666, 41.794257022591204], [-87.64501291564113, 41.79424502203822], [-87.6450689155247, 41.794244021992114], [-87.64512391536192, 41.794243021945945], [-87.64596291536623, 41.79422802215387], [-87.6462649148981, 41.794222021877054], [-87.64678891567048, 41.79421002222343], [-87.64751791599993, 41.794193022339094], [-87.6487119161805, 41.79419602247727], [-87.64995891520704, 41.79415902257031], [-87.65017591531746, 41.79415202224768], [-87.65115891566425, 41.794149022109494], [-87.6524009153597, 41.79414202178686], [-87.65358991530985, 41.794135022363335], [-87.65481391507487, 41.79413307623057], [-87.65542691545296, 41.79413202222514], [-87.65599691564773, 41.794121022617354], [-87.65724991585019, 41.79409602236428], [-87.65732291561783, 41.794095022318196], [-87.65884991586034, 41.794061022549414], [-87.6596659157035, 41.794057022365145], [-87.66063091522068, 41.79404702190414], [-87.66088691533024, 41.79403902243488], [-87.66207791537248, 41.79399302211302], [-87.66329791585241, 41.793962022482695], [-87.66373491531213, 41.793945022598336], [-87.66441991541217, 41.79393302204525], [-87.66451191515642, 41.79393102195299], [-87.66460791598435, 41.793929021860635], [-87.66524791580851, 41.79394202246008], [-87.66574291614334, 41.79393602218354], [-87.66694691498628, 41.793921022391274], [-87.66816791551231, 41.79390802179201], [-87.66936891511624, 41.79390402250693], [-87.67059791601099, 41.79388002230005], [-87.67112491602228, 41.7938860225765], [-87.67182391586817, 41.79387402202351], [-87.67226791565074, 41.793855022046976], [-87.67303291584102, 41.793849021770164], [-87.67425791565205, 41.79383202188583], [-87.67546691562481, 41.79381102181738], [-87.67586991531603, 41.79380802257831], [-87.67667491555147, 41.793818022140016], [-87.6771079157262, 41.793807022532214], [-87.67838691532852, 41.79377402280979], [-87.67876502178585, 41.79375799419313], [-87.678766550276, 41.79376872243081], [-87.6795059410407, 41.79375425737002], [-87.67975000649842, 41.79375416459997], [-87.6797184897253, 41.79649612387017], [-87.6797184897253, 41.80012683235382], [-87.6798356093538, 41.80157130777184], [-87.67991368910606, 41.80364042120851], [-87.67991368910606, 41.80470302207955], [-87.68030091542373, 41.80470302207955], [-87.68060291495578, 41.804696022656195], [-87.68158391521042, 41.80467702178039], [-87.68171991518437, 41.804675022587396], [-87.68179291495211, 41.804675022587396], [-87.68185791525049, 41.804671022403134], [-87.6837189156006, 41.80466802226485], [-87.68372178292049, 41.80466772275226], [-87.6837287648555, 41.804869400464305], [-87.68374763242248, 41.80529863761049], [-87.68376694608105, 41.806225985536486], [-87.6837809111063, 41.806995544232365], [-87.68381107638936, 41.80801121529535], [-87.68381978266, 41.80830389039774], [-87.68382191585194, 41.80835002291999], [-87.6838929155274, 41.81108402214353], [-87.68391091545784, 41.811971022561714], [-87.68395491568744, 41.813525022249934], [-87.68406391531613, 41.815694022408785], [-87.68407391577703, 41.81658202197378], [-87.68408291529265, 41.81751602186037], [-87.68413791602933, 41.81932302241942], [-87.68418191535956, 41.82201002217456], [-87.68419091577437, 41.822246022261325], [-87.68419991528998, 41.82243002264923], [-87.68422291545079, 41.822942022868425], [-87.6842249290329, 41.822996110793056], [-87.68416091529078, 41.82299602176037], [-87.68350891491342, 41.82300202203681], [-87.6831919155893, 41.82300402212909], [-87.68260491551037, 41.82302802233597], [-87.6816299155322, 41.82303002242825], [-87.68098891566193, 41.82303602180533], [-87.6799179157553, 41.823047022312586], [-87.6787939161033, 41.823050022450865], [-87.67759691578443, 41.82307002247338], [-87.67684891547837, 41.823078021943005], [-87.6765309152088, 41.82308202212728], [-87.6762649155376, 41.823086022311735], [-87.67507091535701, 41.82310402224216], [-87.67384091531547, 41.823126022356874], [-87.67279691575409, 41.82314102214886], [-87.67264091575763, 41.82314402228731], [-87.6716179153654, 41.8231530227022], [-87.67140191530116, 41.82316202221774], [-87.67040091502383, 41.82317202267873], [-87.67019891560496, 41.82318402233263], [-87.6693049157634, 41.823195021940165], [-87.66918891581207, 41.823196021986256], [-87.6689969159549, 41.82320402235497], [-87.66776991605164, 41.82316902254038], [-87.66722791534892, 41.82315602194111], [-87.6660909150978, 41.823136021918494], [-87.66558191591633, 41.82313902205677], [-87.6654129153206, 41.82314202219495], [-87.66538091564418, 41.82314202219495], [-87.66529491527716, 41.82314302224122], [-87.6646989156826, 41.823153021802746], [-87.66266291535943, 41.82317302182546], [-87.66180491537875, 41.823186022424714], [-87.66050991503892, 41.82319402189415], [-87.65982591588444, 41.82320602244733], [-87.65951391589148, 41.823201023116155], [-87.65894791588107, 41.823287021684514], [-87.6579619153959, 41.82330302242197], [-87.6574689151534, 41.823316022122036], [-87.65710391541523, 41.8233290227213], [-87.65691991502743, 41.823330022767394], [-87.65670491590856, 41.82333802223674], [-87.65651991547448, 41.82334602260547], [-87.65611193174075, 41.823368077578955], [-87.65594693852387, 41.823299890982774], [-87.6538819152802, 41.823359022305546], [-87.65315091575795, 41.82335802225945], [-87.65077891613403, 41.823380022374245], [-87.64820791543092, 41.82341402214293], [-87.6459689156427, 41.82345402218808], [-87.64592332608169, 41.823454992215964]]]}, "properties": {"city": "Chicago", "name": "Back of the Yards", "regionid": "403146", "geo_point_2d": [41.80814497800252, -87.66172986711189], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.69251495744828, 41.766244311812024], [-87.69302, 41.76623], [-87.6930466592584, 41.767065323432305], [-87.69307999999985, 41.76811], [-87.69251332840605, 41.76812], [-87.6912111743564, 41.76812486408103], [-87.69012597248954, 41.76813133401255], [-87.68893310689788, 41.768157281551694], [-87.68773580281197, 41.76819592222032], [-87.68622060411062, 41.76824], [-87.68513550304883, 41.76824919195135], [-87.68405069918111, 41.76826], [-87.68326434019845, 41.76826033982011], [-87.68324702175381, 41.76767624524749], [-87.68321458356792, 41.76642869841995], [-87.68394242428157, 41.76642750124133], [-87.68502714181773, 41.76641279147334], [-87.68632909935708, 41.76641], [-87.68752087957766, 41.76638863084732], [-87.68882502631163, 41.76633827825722], [-87.69012654916575, 41.76631025084723], [-87.69121085064245, 41.76628363778925], [-87.69251495744828, 41.766244311812024]]]}, "properties": {"city": "Chicago", "name": "Lithuanian Plaza", "regionid": "403339", "geo_point_2d": [41.76726925415712, -87.68815701370401], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.62440906599993, 41.88909871500003], [-87.62429091599994, 41.889079022000026], [-87.62420792799989, 41.889062888000055], [-87.6239669149999, 41.88909002200005], [-87.62296391399988, 41.889143020000056], [-87.62269891599992, 41.88910202200008], [-87.62208991599992, 41.88900302200004], [-87.6217139159999, 41.88894002300008], [-87.62073041599989, 41.888835522000065], [-87.6206209179999, 41.88883002100003], [-87.62057891499988, 41.888820024000026], [-87.62053391599991, 41.888816022000064], [-87.62044591499992, 41.88880802200002], [-87.6203130189999, 41.88879744800004], [-87.62036331199992, 41.88846238000007], [-87.62050422599991, 41.887488632000036], [-87.62064345899992, 41.88699774000007], [-87.62064170099988, 41.88636917500008], [-87.62061235799987, 41.88553418400004], [-87.62056218899991, 41.88469678200004], [-87.62064999999988, 41.88449], [-87.62055937599987, 41.88449052600002], [-87.6190908539999, 41.88449905400005], [-87.61790773499989, 41.884520108000046], [-87.61609470199991, 41.88435472800005], [-87.61525840699993, 41.884355914000075], [-87.61356649799988, 41.884352517000075], [-87.61637058499988, 41.88238965600003], [-87.61645179899993, 41.88233234800003], [-87.61637044999989, 41.882331946000065], [-87.6158503379999, 41.882331946000065], [-87.61499872899991, 41.882331946000065], [-87.61493393499987, 41.882331946000065], [-87.61499768299988, 41.882275893000035], [-87.6154134459999, 41.881909458000045], [-87.61505463299991, 41.88175247700007], [-87.61481721299991, 41.88149681100003], [-87.61481034099987, 41.88147116600003], [-87.61480053099989, 41.88139815200008], [-87.6147745299999, 41.88111815200005], [-87.61477087899992, 41.88101600600003], [-87.6147798779999, 41.88059000600003], [-87.61479333999993, 41.88045283500008], [-87.6148171049999, 41.88035225600002], [-87.61483110599991, 41.880305255000046], [-87.61485183099988, 41.88024385700003], [-87.6148769239999, 41.880184111000055], [-87.61490625399993, 41.88012632700007], [-87.6149396689999, 41.88007080500005], [-87.61497699599991, 41.88001783300006], [-87.61501499599987, 41.87996783300007], [-87.6150420799999, 41.879933927000074], [-87.61515292699991, 41.87982308000004], [-87.61528133899989, 41.879733165000054], [-87.61542341399988, 41.87966691400003], [-87.6154508959999, 41.87965750000006], [-87.61542442499989, 41.87875467700007], [-87.61531712399992, 41.87686498300006], [-87.61531605999991, 41.87684021900008], [-87.61528805999987, 41.87588021900006], [-87.61528785899989, 41.87583592600003], [-87.61529147499992, 41.87565605000008], [-87.61619115999989, 41.87564705300007], [-87.62257483099994, 41.87558321700003], [-87.62382509099987, 41.87562789000008], [-87.6242589489999, 41.87568864900003], [-87.62426, 41.87568], [-87.62480534599989, 41.87569081600008], [-87.62564115499991, 41.87567431900003], [-87.62668590799989, 41.87566177900003], [-87.62723787899992, 41.87557397800003], [-87.62768795499993, 41.87564139600004], [-87.62807918599991, 41.8757], [-87.62877596399991, 41.8757], [-87.62933196199991, 41.87570186300007], [-87.63031002799991, 41.87569784300006], [-87.63069771199989, 41.87569028600006], [-87.63135478899993, 41.87567748000002], [-87.6326085139999, 41.87565694000005], [-87.63337470299987, 41.87565], [-87.63633038499992, 41.87565887500004], [-87.63666007599993, 41.875608635000034], [-87.63683920199989, 41.875830998000076], [-87.6373685169999, 41.87646276100008], [-87.63753926299992, 41.87690670200003], [-87.6376246369999, 41.87731649400006], [-87.63765878599989, 41.87793118200005], [-87.63781245799991, 41.87842634800005], [-87.63793198099992, 41.878989812000036], [-87.63805150399989, 41.879484977000054], [-87.63815395199991, 41.880133815000065], [-87.63827347399989, 41.88071435400006], [-87.6382905489999, 41.88129489200007], [-87.63825639999992, 41.88168761000007], [-87.63820517599991, 41.882148626000046], [-87.6381881009999, 41.88267794100005], [-87.63815395199991, 41.883173106000065], [-87.63805150399989, 41.88358289800003], [-87.6379149059999, 41.88395854100008], [-87.63782953299993, 41.88465860200005], [-87.63772708499992, 41.885444037000056], [-87.63788010299992, 41.88570635500007], [-87.6379661299999, 41.885853829000034], [-87.63837592199988, 41.88636606900007], [-87.63861496799989, 41.886792936000056], [-87.63864154499991, 41.886841258000054], [-87.63797899999989, 41.886924], [-87.63792591499991, 41.88690302300005], [-87.63775691599992, 41.886864024000026], [-87.63740689299992, 41.886885939000074], [-87.63710091599994, 41.88703902300005], [-87.63654691499993, 41.88742102200006], [-87.63636391499993, 41.88750802200007], [-87.63606591599992, 41.88762802300004], [-87.63585891299992, 41.887710020000036], [-87.63538191599989, 41.88773502200006], [-87.63518091599988, 41.88774802300008], [-87.63403591599989, 41.88777902200006], [-87.63395491599991, 41.887781022000055], [-87.63290291599989, 41.88779502200003], [-87.63259991499987, 41.887799022000074], [-87.63242291499994, 41.88779802200008], [-87.63101693299988, 41.88774300200003], [-87.62973891599994, 41.887717022000054], [-87.62954191599992, 41.887721022000036], [-87.62834391599992, 41.88777302300007], [-87.6280889159999, 41.887811023000026], [-87.62791591499987, 41.887830022000045], [-87.62781791699989, 41.88783202300004], [-87.62738991699989, 41.88792102300004], [-87.62698791599988, 41.88808602300007], [-87.62690591899988, 41.888122022000054], [-87.62601791599988, 41.88874202300006], [-87.62571191499988, 41.88895702100006], [-87.6255679159999, 41.889026022000046], [-87.62519191599993, 41.88917802100007], [-87.62511291399993, 41.88919702200007], [-87.62504591899993, 41.88920102300005], [-87.62500491799993, 41.88921215700008], [-87.62478991599991, 41.88919102100004], [-87.6245499149999, 41.88912502200008], [-87.62440906599993, 41.88909871500003]]]}, "properties": {"city": "Chicago", "name": "The Loop", "regionid": "269593", "geo_point_2d": [41.881626576937165, -87.626990711443], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.72712091529293, 41.9412220216832], [-87.72711592765161, 41.94107835895091], [-87.72754375639506, 41.94133871443075], [-87.7284815259211, 41.94191244867433], [-87.73025768264688, 41.94300068566356], [-87.73167595469398, 41.94386995128704], [-87.73239710739091, 41.94431420353767], [-87.73359520502021, 41.94504589013486], [-87.73484197765174, 41.945795736540184], [-87.73548015108564, 41.94618042035644], [-87.7353006275037, 41.94618377626303], [-87.73449051551056, 41.9462], [-87.7328713063409, 41.9462], [-87.7318193469706, 41.94620338136181], [-87.73084915158586, 41.946219089012985], [-87.72987582329198, 41.94624318683175], [-87.7284192201061, 41.94627444327639], [-87.72725508400116, 41.946299888874364], [-87.7272550473749, 41.94629814954933], [-87.72723191591291, 41.945484022094725], [-87.72722891577475, 41.94538302193556], [-87.7272099157982, 41.944711022434795], [-87.72720291547557, 41.94446502188687], [-87.72718591559124, 41.943894022545365], [-87.72716991575305, 41.94309402254028], [-87.72716091533816, 41.94264802266582], [-87.72712091529293, 41.9412220216832]]]}, "properties": {"city": "Chicago", "name": "South Old Irving Park", "regionid": "403316", "geo_point_2d": [41.94451169522103, -87.72993822366946], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.63119791553696, 41.82362402193054], [-87.6313679161787, 41.82362002264563], [-87.6325439155296, 41.823622021838354], [-87.6332099156531, 41.823622021838354], [-87.633747094191, 41.82361796409746], [-87.63531191542138, 41.82358602197749], [-87.63602991524374, 41.823574022323854], [-87.63603809737072, 41.823574024387106], [-87.63605272409632, 41.82414590051702], [-87.63608591071988, 41.82491393380324], [-87.63630908728459, 41.825389941459285], [-87.63624991549251, 41.826291022562515], [-87.63622691533152, 41.82666602186223], [-87.63619891583956, 41.82722902263347], [-87.63619906682703, 41.8272358089226], [-87.63323462452473, 41.827265003103186], [-87.63218894664598, 41.82727723038291], [-87.63075917675701, 41.82742305727711], [-87.63065626909773, 41.823628050273804], [-87.63066091506471, 41.823628022114995], [-87.63089691605092, 41.8236270220689], [-87.63119791553696, 41.82362402193054]]]}, "properties": {"city": "Chicago", "name": "Wentworth Gardens", "regionid": "403341", "geo_point_2d": [41.82545409463409, -87.63342962313487], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.74905299998147, 41.77473200029376], [-87.75195299966258, 41.774632000180766], [-87.75295299989375, 41.77473200029376], [-87.75445299979084, 41.774632000180766], [-87.75565400029416, 41.774632000180766], [-87.75659765079189, 41.774632000180766], [-87.75661488035429, 41.775041864089445], [-87.75662690981464, 41.77598424406582], [-87.75663667247892, 41.77689055798563], [-87.75664992400192, 41.77764169408677], [-87.7566599999999, 41.778080000000124], [-87.75665996476941, 41.77808051469552], [-87.7561635399503, 41.77808439345167], [-87.75437404725477, 41.778101680474514], [-87.75250306022254, 41.77812955810689], [-87.75136317696808, 41.778144295703015], [-87.75014569761466, 41.77816600828444], [-87.7489254549322, 41.77819271962353], [-87.74753562542162, 41.77822173957645], [-87.74703800396176, 41.77823218319221], [-87.74702945841491, 41.777846286302655], [-87.74700488669141, 41.77695187556669], [-87.74697979485194, 41.77613956357667], [-87.7469583310749, 41.775570773486606], [-87.74700979641257, 41.774832000406924], [-87.74705300041836, 41.774832000406924], [-87.7474529999713, 41.77473200029376], [-87.74905299998147, 41.77473200029376]]]}, "properties": {"city": "Chicago", "name": "Chrysler Village", "regionid": "403336", "geo_point_2d": [41.77641409131696, -87.75180866438609], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.74745089215018, 41.96019648498359], [-87.74742882402454, 41.95934727008616], [-87.747394872725, 41.95831618175101], [-87.74736241624838, 41.95746557371615], [-87.74732584234243, 41.956553585592374], [-87.7473147209223, 41.956189395051894], [-87.74730023232132, 41.9557023181252], [-87.74727562633598, 41.95454786661197], [-87.74725968600825, 41.95381977140411], [-87.74723739484352, 41.95272604475343], [-87.74720246243463, 41.9517541735636], [-87.74716893374033, 41.950840990127716], [-87.74714194135866, 41.95005231895064], [-87.74711417108156, 41.94920347461315], [-87.74708276134321, 41.948232840301465], [-87.74704697873298, 41.94732051086023], [-87.74702631113479, 41.946594313272115], [-87.7470077834036, 41.94549752367644], [-87.7469789513854, 41.94483168740757], [-87.74695436145957, 41.94398291619719], [-87.74694332884022, 41.94355816035185], [-87.746922478122, 41.94295151549421], [-87.74691, 41.94239], [-87.7478917488204, 41.94236112503484], [-87.75086420088114, 41.942338184738425], [-87.75353752116318, 41.942309005887445], [-87.7558456653862, 41.94227427485091], [-87.75851664335426, 41.94223090456115], [-87.76076255091215, 41.94220716284989], [-87.76367691879258, 41.94217526009576], [-87.76592323281174, 41.94214761440984], [-87.7681703680309, 41.94211631902001], [-87.77072032068506, 41.942080000000125], [-87.77254226657656, 41.942061197258006], [-87.77497144447423, 41.94201997465845], [-87.77625697194769, 41.94201134506826], [-87.7776419021208, 41.942006662958796], [-87.78013262760035, 41.94197231323964], [-87.78256107122193, 41.94194233245413], [-87.78486803721813, 41.941910904566285], [-87.7861, 41.9419], [-87.78611673247899, 41.94228484701707], [-87.78616152805328, 41.94331514522916], [-87.78620082875526, 41.94422426856159], [-87.78623293302142, 41.945012725072075], [-87.78619, 41.94558000000026], [-87.78615634259597, 41.94586412029119], [-87.78618540614745, 41.946778066967404], [-87.78623069228335, 41.94762592251815], [-87.7862679260421, 41.94853778126878], [-87.78631163435693, 41.94969218161939], [-87.78635031115736, 41.95054059367546], [-87.78638615946248, 41.951333962226414], [-87.78641775208416, 41.95193930627552], [-87.78644984889195, 41.95280592008326], [-87.7864520440139, 41.952805887469786], [-87.78648691522568, 41.95371602204798], [-87.78650991538649, 41.954366022332955], [-87.78653291554727, 41.955009023194755], [-87.78658991637613, 41.956465022862254], [-87.78662291519944, 41.95727002219851], [-87.7866729157056, 41.95847002265591], [-87.78671684578818, 41.96010106996042], [-87.7867249999409, 41.96013200034294], [-87.78608208375883, 41.960115309989845], [-87.78498958890604, 41.96013393779589], [-87.78371455012206, 41.9601444995788], [-87.78231815935442, 41.960158242674005], [-87.7811649640112, 41.96018049646269], [-87.7800110148715, 41.960194707196344], [-87.77873664293347, 41.960210303422606], [-87.77770466559113, 41.96022730056408], [-87.77630648214682, 41.960250138501834], [-87.77515296783386, 41.9602644838703], [-87.77393885476201, 41.960278799109595], [-87.7728458365023, 41.96029336802917], [-87.77138911132695, 41.96031065949486], [-87.7700529751886, 41.96032630854021], [-87.76902153261646, 41.96033980778984], [-87.76792685487396, 41.96035773032209], [-87.7660449172269, 41.96038354351564], [-87.7634948561026, 41.96040975627684], [-87.76106572755468, 41.960434262418495], [-87.7585758222476, 41.96046456314344], [-87.75645093023495, 41.96048795434975], [-87.75329370735366, 41.960525494110556], [-87.75183694521989, 41.960538884246404], [-87.75056262125905, 41.96056380406239], [-87.7492875602707, 41.960589532876845], [-87.74837596710239, 41.960604860048115], [-87.74745994902854, 41.960617629834054], [-87.74745089215018, 41.96019648498359]]]}, "properties": {"city": "Chicago", "name": "Portage Park", "regionid": "275083", "geo_point_2d": [41.95125275418938, -87.76678625697218], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "MultiPolygon", "coordinates": [[[[-87.54572965930703, 41.66670683049445], [-87.54572924514196, 41.66601329976792], [-87.5454505336964, 41.66471264635467], [-87.54480020698969, 41.662390050974025], [-87.54480020698969, 41.660996493745486], [-87.54452149554413, 41.65988164796286], [-87.5442427840983, 41.659417128886766], [-87.53922597807622, 41.659324225071636], [-87.53718209414114, 41.659324225071636], [-87.53681047888017, 41.658766802180146], [-87.53681047888017, 41.658395186919186], [-87.53606724835835, 41.65783776402797], [-87.5354169216518, 41.65746614876691], [-87.5353240178365, 41.65681582206037], [-87.5349524025758, 41.65616549535385], [-87.53523111402136, 41.65579388009288], [-87.53550982546695, 41.65579388009288], [-87.53504530639094, 41.65532936101679], [-87.53458078731485, 41.65495774575591], [-87.53458078731485, 41.6542145152339], [-87.53458078731485, 41.65369000000016], [-87.53497999999979, 41.65369000000016], [-87.53500780257072, 41.65197550813329], [-87.53411626823873, 41.65197652278486], [-87.53411626823858, 41.65184000000031], [-87.53411626823858, 41.64910480539643], [-87.53411626823858, 41.64873319013555], [-87.53950468952179, 41.64873319013555], [-87.5396842075027, 41.644334999607246], [-87.5404430002239, 41.644334999607246], [-87.54060194025605, 41.64436678779186], [-87.54219258208383, 41.645582601020536], [-87.54370372979353, 41.646758750136506], [-87.54616313923827, 41.64867956618601], [-87.54887901653295, 41.65078857844818], [-87.55183203586364, 41.653016718865594], [-87.55388953606985, 41.654605831482996], [-87.55512054969118, 41.65559866531085], [-87.55617301609198, 41.65649144477516], [-87.5576481706417, 41.657728800366755], [-87.55792, 41.65799000000023], [-87.5580492883855, 41.65838573031364], [-87.55806539434091, 41.658859969919455], [-87.55807, 41.65921000000027], [-87.5591073945347, 41.659200000000254], [-87.55882868308913, 41.662111339528366], [-87.55882868308913, 41.66396941583284], [-87.55882868308913, 41.66527006924608], [-87.5586428754587, 41.66629201121357], [-87.55734186822315, 41.666514970173665], [-87.55594944299105, 41.66652450928361], [-87.55483045330438, 41.666521573331416], [-87.55427335241795, 41.666500779265924], [-87.55434444174671, 41.66660785170698], [-87.55371897325149, 41.666663626474445], [-87.55176799313172, 41.66684943410497], [-87.55037443590335, 41.66684943410497], [-87.54777312907714, 41.66684943410497], [-87.54721570618572, 41.66684943410497], [-87.54643455751581, 41.66675179052127], [-87.54643491580663, 41.666712022071046], [-87.54572965930703, 41.66670683049445]]], [[[-87.5714641374855, 41.713440300727115], [-87.5729500630037, 41.713394245533685], [-87.57378619734065, 41.71441618750118], [-87.57908171480858, 41.72008332023013], [-87.57901324935438, 41.721289331425396], [-87.57902288170982, 41.72193951543035], [-87.57896302695035, 41.72242213728403], [-87.57809741969089, 41.72243541455079], [-87.57725970881529, 41.72245344698855], [-87.57577415497916, 41.72247647028498], [-87.5748451738079, 41.72248422928393], [-87.57317302725512, 41.722498521134824], [-87.57196549400554, 41.72251699418832], [-87.57103678815145, 41.72253131448508], [-87.57020010766053, 41.72254], [-87.56889889959035, 41.72255659632251], [-87.56713427404652, 41.72259139546653], [-87.56544703946228, 41.7226278764604], [-87.56473991514115, 41.72263902265761], [-87.56375791573954, 41.72264702212705], [-87.56255591519023, 41.72268202194182], [-87.56159391581122, 41.72269402249499], [-87.56126891521927, 41.72269802178017], [-87.56072691541583, 41.722707022194896], [-87.56072373497592, 41.722706955294], [-87.56067262237598, 41.72138423067982], [-87.5606697267511, 41.72038339839273], [-87.56080999999985, 41.72035000000031], [-87.56038331953225, 41.72010416967836], [-87.5601229800772, 41.71979539497565], [-87.55986020998716, 41.719386377977436], [-87.5598088179886, 41.718503169267265], [-87.5597988153297, 41.71785299644786], [-87.55977377427223, 41.716274004885825], [-87.55973745265096, 41.714416642975245], [-87.55971999999981, 41.71364000000024], [-87.5606867593935, 41.713630000000215], [-87.56180272147954, 41.7136233383488], [-87.56273483316568, 41.71357932576536], [-87.56412465898721, 41.713543994685786], [-87.56523975952491, 41.71352958387377], [-87.56644721871059, 41.71350420118929], [-87.56756310865666, 41.71348079476885], [-87.56876845900304, 41.713465378382274], [-87.5701634455669, 41.71345388795393], [-87.5714641374855, 41.713440300727115]]]]}, "properties": {"city": "Chicago", "name": "Hegewisch", "regionid": "138732", "geo_point_2d": [41.67713213293271, -87.55388868444703], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.73619319927751, 41.978901344051536], [-87.73820313601367, 41.978901344051536], [-87.7382222812317, 41.982588720620335], [-87.73821778201257, 41.982588893394926], [-87.73717664142987, 41.982628334910665], [-87.73562614920235, 41.9826512217741], [-87.73562629404418, 41.981890480736105], [-87.73562629404418, 41.98168433337852], [-87.73567783088362, 41.98142664918144], [-87.73583244140178, 41.981117428145104], [-87.73583244140178, 41.98101435446644], [-87.73578090456236, 41.98080820710877], [-87.73572936772302, 41.98070513343011], [-87.73557475720486, 41.98049898607244], [-87.73547168352601, 41.980344375554274], [-87.73547168352601, 41.9799836176787], [-87.73557475720486, 41.97982900716053], [-87.73578090456236, 41.97972593348161], [-87.73593551508053, 41.979622859802944], [-87.73619319927751, 41.979622859802944], [-87.73619319927751, 41.978901344051536]]]}, "properties": {"city": "Chicago", "name": "Sauganash Woods", "regionid": "403272", "geo_point_2d": [41.98082763663508, -87.73696808140092], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.63101627699989, 41.88774298900005], [-87.63101791599989, 41.887743022000045], [-87.63242291499994, 41.88779802200008], [-87.63259991499987, 41.887799022000074], [-87.63290291599989, 41.88779502200003], [-87.63395491599991, 41.887781022000055], [-87.63403591599989, 41.88777902200006], [-87.63518091599988, 41.88774802300008], [-87.63538191599989, 41.88773502200006], [-87.63585891299992, 41.887710020000036], [-87.63606591599992, 41.88762802300004], [-87.63636391499993, 41.88750802200007], [-87.63654691499993, 41.88742102200006], [-87.63710091599994, 41.88703902300005], [-87.63740689299992, 41.886885939000074], [-87.63775691599992, 41.886864024000026], [-87.63792591499991, 41.88690302300005], [-87.63797899999989, 41.886924], [-87.63864154499991, 41.886841258000054], [-87.63880278899987, 41.88713443000005], [-87.63899060999994, 41.88812476100008], [-87.63924672999988, 41.88855162700007], [-87.6393491779999, 41.88889312100007], [-87.6393491779999, 41.88920046500004], [-87.6396906719999, 41.88957610800007], [-87.6405614799999, 41.89054936400002], [-87.64097127199993, 41.89107867900008], [-87.6410395709999, 41.89142017200003], [-87.6414322879999, 41.89188118800006], [-87.64201372299992, 41.89245144100004], [-87.64232017099991, 41.89275199600006], [-87.64284948599992, 41.89319593800007], [-87.64331050199989, 41.89341790800005], [-87.6436519949999, 41.89377647700006], [-87.64399348899988, 41.894135045000034], [-87.64411301199992, 41.89440823900008], [-87.64413008599989, 41.89536442100005], [-87.64414144799989, 41.89645780500006], [-87.64383799999987, 41.896454], [-87.64383866999991, 41.896465116000044], [-87.64291, 41.89648], [-87.64225592199992, 41.89648484400003], [-87.6405171019999, 41.89651], [-87.63914373899989, 41.896522104000034], [-87.63867, 41.89653], [-87.63767937899989, 41.89654267100007], [-87.6365354089999, 41.89656057700005], [-87.63516288399987, 41.89657352700003], [-87.63388024399987, 41.8966], [-87.63205057899992, 41.896618193000045], [-87.63122999999989, 41.89663], [-87.63122999999989, 41.89616913300005], [-87.63122651999991, 41.895344677000075], [-87.63120044899993, 41.89442875600007], [-87.63117066499989, 41.89365192800005], [-87.6311513569999, 41.89287430000007], [-87.6311099999999, 41.89195816200002], [-87.63110456599989, 41.89117984500006], [-87.6310806369999, 41.89044828500004], [-87.6310620119999, 41.88953270500007], [-87.63104, 41.88879993300007], [-87.6310348469999, 41.888252222000055], [-87.63101627699989, 41.88774298900005]]]}, "properties": {"city": "Chicago", "name": "River North", "regionid": "403300", "geo_point_2d": [41.89247888059017, -87.63664824827097], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.52477199895795, 41.65346116968829], [-87.52476639044252, 41.65346001497571], [-87.52474299999987, 41.65303500000014], [-87.5246659999999, 41.65173600000012], [-87.52464299999986, 41.65133500000017], [-87.52464299999986, 41.6502340000002], [-87.52464299999986, 41.65003367900005], [-87.52464282720305, 41.650007], [-87.52464299999986, 41.6499805350001], [-87.52464299999986, 41.64973500000011], [-87.5246727871128, 41.64919221000636], [-87.52471499999986, 41.6484230000001], [-87.52474299999987, 41.647935], [-87.52474299999987, 41.647334], [-87.52474299999987, 41.64613500000015], [-87.52466799999988, 41.644637], [-87.52498299999984, 41.644636000000105], [-87.52504853799965, 41.64463588296804], [-87.525543, 41.644635], [-87.525743, 41.644535000000104], [-87.52582678211743, 41.644555945529504], [-87.5261429999998, 41.644635], [-87.53385591399653, 41.644570185596756], [-87.53804299999985, 41.644535000000104], [-87.53844751299992, 41.644535000000104], [-87.539043, 41.644535000000104], [-87.53964299999983, 41.644335], [-87.53968420749297, 41.644335], [-87.53950468976072, 41.648733190067865], [-87.53411626811905, 41.648733190067865], [-87.53411626811905, 41.649104805198355], [-87.53411626811905, 41.6518400000002], [-87.53411626807952, 41.65197652292828], [-87.53394739099983, 41.65197671500005], [-87.53339200499988, 41.65198675000009], [-87.53293234999985, 41.651990000000175], [-87.5324527789999, 41.65200174200009], [-87.53197499399987, 41.65199839300008], [-87.53136106299988, 41.65200378000008], [-87.53094006999987, 41.65200569100014], [-87.53036520499982, 41.652000000000115], [-87.52994332599991, 41.65199753000009], [-87.52996289099983, 41.64883313400011], [-87.52532670199983, 41.648852292000086], [-87.52530754399986, 41.65212827600008], [-87.52539771899987, 41.65230862600008], [-87.5252628789999, 41.65268503900005], [-87.52516326399981, 41.653541725000046], [-87.52477199895795, 41.65346116968829]]]}, "properties": {"city": "Chicago", "name": "Powder Horn Lake", "regionid": "403389", "geo_point_2d": [41.6474686270833, -87.53186717673991], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "MultiPolygon", "coordinates": [[[[-87.72326591599993, 41.80046802300006], [-87.7233019159999, 41.80046702200007], [-87.72333704899994, 41.80046715400003], [-87.72333191599994, 41.80028302200008], [-87.72333091599988, 41.800223022000054], [-87.7233287589999, 41.80015791800007], [-87.72339091499991, 41.80013302300005], [-87.72581691499992, 41.79918402200006], [-87.72603591599993, 41.79909802300005], [-87.72661091499991, 41.798906022000025], [-87.72678491599991, 41.798857022000035], [-87.72806117299989, 41.79836314300007], [-87.72814999999989, 41.79841], [-87.73044949499987, 41.79753762900002], [-87.7326979289999, 41.79665780800007], [-87.7325574139999, 41.79666054500006], [-87.73346023799988, 41.79623568600004], [-87.73468080999992, 41.79574745800004], [-87.73550501399987, 41.79539422700008], [-87.73552386999994, 41.795373954000056], [-87.73563434799989, 41.79533879800004], [-87.73579391599989, 41.795288022000065], [-87.73585319799989, 41.79527695600006], [-87.73720332399992, 41.79452688600008], [-87.73785252099991, 41.79387769000003], [-87.73785983399993, 41.79333500200004], [-87.73794791499992, 41.793160022000045], [-87.73799591599992, 41.79300502200003], [-87.73799949999992, 41.79299283500006], [-87.73972673399992, 41.79301401000003], [-87.74187, 41.79298], [-87.7419996009999, 41.79303456700005], [-87.74221922399991, 41.79327617800003], [-87.7423326349999, 41.793880607000055], [-87.74253845699988, 41.79425722200004], [-87.7427008409999, 41.794771941000064], [-87.74272934199989, 41.795422240000036], [-87.74275642899993, 41.79615358000006], [-87.74278624199988, 41.79704787300004], [-87.74283533799989, 41.79802493600004], [-87.74293, 41.79858], [-87.74293937999992, 41.80144279100005], [-87.74298055199992, 41.80347774700003], [-87.74302009699991, 41.804696809000056], [-87.74308688999989, 41.80673190100003], [-87.74310725999992, 41.80754741100008], [-87.74232972399992, 41.80755], [-87.74110943799988, 41.80756061100003], [-87.74029626199987, 41.80757873900006], [-87.73883250399989, 41.80761581900003], [-87.7382738939999, 41.80762110500006], [-87.7382739159999, 41.80763202200006], [-87.73827491599988, 41.80853402200005], [-87.73831591599989, 41.81003302200008], [-87.73834991599993, 41.811420022000036], [-87.7383849159999, 41.81327402200008], [-87.73841291599989, 41.815066022000046], [-87.73848491599993, 41.817948023000035], [-87.7384919159999, 41.81825402300006], [-87.7384939159999, 41.81836502200007], [-87.73849691499991, 41.818486023000034], [-87.73849997099988, 41.81865205400003], [-87.73850191599988, 41.81872502300006], [-87.73851791499992, 41.81954202300005], [-87.7385309999999, 41.820371], [-87.73853914599992, 41.82088715400004], [-87.73441315499988, 41.82217397300007], [-87.72919548299987, 41.82384662400005], [-87.72542577699993, 41.825019976000064], [-87.72372816099993, 41.825519275000026], [-87.72312900299993, 41.82576892500002], [-87.72073236899989, 41.82649290800002], [-87.7174619609999, 41.82759136500003], [-87.71432292899992, 41.82855051400007], [-87.7143114929999, 41.82812296000007], [-87.71426251699992, 41.82627524200005], [-87.71426091499988, 41.826195023000075], [-87.71425191599988, 41.82604202300006], [-87.7142469149999, 41.82554702200007], [-87.71421491699992, 41.82458302200006], [-87.7142509149999, 41.82413802200005], [-87.7142509159999, 41.823914022000054], [-87.7142509159999, 41.82358902300005], [-87.71424591499992, 41.82328902200003], [-87.71424491499991, 41.82324602200003], [-87.71423491699993, 41.82235402200006], [-87.7142689159999, 41.82210402200006], [-87.71426191599993, 41.82134902300003], [-87.71410191599993, 41.81536702300007], [-87.71403691699992, 41.81267102200007], [-87.71405391499988, 41.81250002200004], [-87.7140419159999, 41.81181002200008], [-87.7140299149999, 41.81113902200006], [-87.7138729149999, 41.80922902300005], [-87.7138699159999, 41.80880102300005], [-87.71385191599988, 41.808200022000044], [-87.71384991599989, 41.808138023000076], [-87.71384791599989, 41.80809102300008], [-87.7138459159999, 41.80804102200004], [-87.7138409159999, 41.807913023000076], [-87.71381491499992, 41.80726902300005], [-87.71380591599987, 41.80672702300006], [-87.71380291599989, 41.80658502200003], [-87.7137929069999, 41.80602899100005], [-87.71376991499992, 41.805651023000046], [-87.71370991599989, 41.80514202300008], [-87.71368091499993, 41.80415702200007], [-87.71367991499991, 41.80411102300007], [-87.71367791499992, 41.80406302200002], [-87.71367596999993, 41.80397003400008], [-87.71383402199989, 41.80392340000007], [-87.71461120199989, 41.803787529000026], [-87.71532188899992, 41.803569976000055], [-87.71604391499993, 41.80330402200006], [-87.7173749159999, 41.80279002200005], [-87.71789391499992, 41.802585022000066], [-87.71885091499988, 41.80220802200005], [-87.7195799159999, 41.80191202300005], [-87.72012291599991, 41.80170502200007], [-87.72117691599992, 41.80122902200008], [-87.72259791899987, 41.80058702400003], [-87.72326591599993, 41.80046802300006]]], [[[-87.70320999999987, 41.86642], [-87.7030932299999, 41.86496162500002], [-87.7030403079999, 41.863313980000044], [-87.70300748599992, 41.86185019400004], [-87.70297298299991, 41.860691857000056], [-87.70294594999987, 41.85971611400003], [-87.70288745099991, 41.857887147000035], [-87.70283612699988, 41.85611509500005], [-87.70285952999987, 41.85544960400004], [-87.6992499159999, 41.85550402200004], [-87.69919791599993, 41.855506022000036], [-87.69914191599989, 41.855506022000036], [-87.69882091599992, 41.85550402200004], [-87.69608491499989, 41.85555402300007], [-87.69556179199989, 41.85555316600004], [-87.6955370229999, 41.85531687100006], [-87.69538906699992, 41.85534801000005], [-87.69552999999989, 41.85523], [-87.69796194799991, 41.85452814900008], [-87.6992435479999, 41.85403992100003], [-87.70028103399993, 41.85391786300004], [-87.7009523489999, 41.85391786300004], [-87.70286118599992, 41.85379076400004], [-87.70361491499993, 41.853625023000056], [-87.70372891499993, 41.85361202300003], [-87.70406491499993, 41.853600022000066], [-87.70523091499989, 41.85359102200004], [-87.7052929149999, 41.85359002200004], [-87.70534991599992, 41.853589022000044], [-87.7065049159999, 41.85358202200006], [-87.7077179159999, 41.853563022000046], [-87.70893291499992, 41.853559022000056], [-87.71006191599989, 41.853544022000044], [-87.71110506099988, 41.85341090000007], [-87.71266983499991, 41.85361272100004], [-87.71413452099992, 41.85361272100004], [-87.71688080699992, 41.85361272100004], [-87.71794289599987, 41.85349829500006], [-87.71926627899988, 41.853027784000055], [-87.7204672529999, 41.852597015000065], [-87.7214665959999, 41.852242141000026], [-87.72250840799987, 41.851880808000026], [-87.72657582499991, 41.85029344200006], [-87.72868579799989, 41.84955207500007], [-87.73041960999987, 41.84894676100003], [-87.73178390199988, 41.84847043500007], [-87.73256912999993, 41.848192429000036], [-87.7336449819999, 41.847814410000055], [-87.73493217499991, 41.847403949000054], [-87.73597648499991, 41.84706515300007], [-87.73786105399994, 41.84637326000006], [-87.73904525599991, 41.845956177000055], [-87.73905601299991, 41.84603147400007], [-87.7392560129999, 41.84903147400006], [-87.7393563399999, 41.85043161700007], [-87.73935601299988, 41.85048858700008], [-87.73935601299988, 41.85147436100003], [-87.73935568599988, 41.85153133000004], [-87.73942809899994, 41.85196398800007], [-87.73945601299992, 41.85213147400008], [-87.73947501299992, 41.852340474000066], [-87.7395560129999, 41.85323147400003], [-87.73965633999988, 41.85413161700005], [-87.73965601299993, 41.85418858700006], [-87.73965601299993, 41.85437436100004], [-87.73965601299993, 41.85443147400007], [-87.73965601299993, 41.85448858700004], [-87.73965601499991, 41.85487435600003], [-87.7396562139999, 41.85493132000004], [-87.73965744299991, 41.85497140200004], [-87.73965784999987, 41.854988532000036], [-87.73965895599989, 41.85502847800006], [-87.73966054999993, 41.85508554700004], [-87.73975447699988, 41.85857440300002], [-87.73975562899992, 41.85863148500004], [-87.73982701299988, 41.85955547400004], [-87.7398528339999, 41.859890255000025], [-87.73985625999988, 41.85993173200006], [-87.73985283299992, 41.859947372000065], [-87.7397560729999, 41.860431388000045], [-87.7397560129999, 41.86048858700007], [-87.7397560129999, 41.861974361000065], [-87.7397560129999, 41.86203147400004], [-87.7397560129999, 41.86208858700006], [-87.7397560129999, 41.862278361000044], [-87.7397560129999, 41.86233547400006], [-87.7397560129999, 41.86239258700004], [-87.7397560129999, 41.863174361000056], [-87.73975565899988, 41.86323161100005], [-87.7397379699999, 41.863249517000035], [-87.7395669889999, 41.86342049800003], [-87.73955615699991, 41.863431146000025], [-87.73956699199992, 41.86347761200005], [-87.73971401299988, 41.86409547400006], [-87.73977705899992, 41.86451104700006], [-87.73985633999987, 41.86503161700005], [-87.73985601299992, 41.86507536100003], [-87.73985601299992, 41.865088587000045], [-87.73985601299992, 41.865132474000056], [-87.73985601299992, 41.865189587000074], [-87.73985601299992, 41.86557436100007], [-87.73985601299992, 41.86563147400005], [-87.73985601299992, 41.86568858700007], [-87.73985601299992, 41.86583383200007], [-87.73985601299992, 41.86587436100006], [-87.73985601299992, 41.86589094500003], [-87.7398560209999, 41.86593145300008], [-87.74110971999993, 41.86587925800005], [-87.7422569949999, 41.86583151400002], [-87.74294439599991, 41.86586011500008], [-87.74465703699988, 41.86593141200006], [-87.74579840599993, 41.865836358000074], [-87.7458570129999, 41.86583147400005], [-87.74586416499993, 41.86583087800005], [-87.7470569789999, 41.86573155200004], [-87.74805535699991, 41.86580827000005], [-87.74835704699989, 41.865831395000036], [-87.74872306199988, 41.86580097000005], [-87.74955699799992, 41.86573151600003], [-87.7508570129999, 41.86573147400008], [-87.75191704499989, 41.86573147400008], [-87.75205701299987, 41.86573147400008], [-87.75206308699988, 41.86573147400008], [-87.75315701299989, 41.86573147400008], [-87.75445701699992, 41.86573146400007], [-87.75730954699989, 41.86567325900005], [-87.7592347119999, 41.865633970000026], [-87.75926273599991, 41.86563339800005], [-87.7593570089999, 41.865631484000055], [-87.76010830799991, 41.86563147400005], [-87.76033797699992, 41.86563147400005], [-87.7619570129999, 41.86563147300005], [-87.76295701299992, 41.86563047400005], [-87.76413299999989, 41.86563047400005], [-87.76415, 41.86642872200007], [-87.76412931899993, 41.86795411000008], [-87.76415714599993, 41.86886869300007], [-87.76422348099992, 41.87003047700005], [-87.7643478499999, 41.87122353000006], [-87.76362092899987, 41.871302119000056], [-87.7621559969999, 41.871462957000055], [-87.7605762559999, 41.87152069800003], [-87.75825900899989, 41.871585702000054], [-87.75636654299987, 41.871590165000036], [-87.75496382399989, 41.871600163000025], [-87.75350018899988, 41.87164904300003], [-87.7518517589999, 41.87167972500003], [-87.7507544499999, 41.87169565400005], [-87.74928786099989, 41.87168008500003], [-87.74611300599989, 41.87171827000008], [-87.74482, 41.87175], [-87.74379437399993, 41.871742163000064], [-87.74159693199991, 41.87177090300007], [-87.74086477199988, 41.87178248800005], [-87.7399767139999, 41.871841717000045], [-87.73997652599991, 41.871833520000045], [-87.73957314399989, 41.87185399000003], [-87.73915570799991, 41.87191783300005], [-87.73853162999988, 41.872047093000035], [-87.73783057499989, 41.87227054800006], [-87.73738378699993, 41.872464793000056], [-87.73693788599988, 41.872667838000034], [-87.73654253199992, 41.872847865000026], [-87.7360679099999, 41.873057888000055], [-87.7356637649999, 41.87322558500005], [-87.73467957299992, 41.87355973100004], [-87.73421292099992, 41.873691114000046], [-87.7338257029999, 41.873767467000036], [-87.7333545629999, 41.87384649100004], [-87.7331072339999, 41.87387784600008], [-87.73171059699987, 41.873926481000055], [-87.72957474999991, 41.87396], [-87.72859853599992, 41.87396266800003], [-87.72640190499992, 41.873986960000074], [-87.7253, 41.874], [-87.72377780799988, 41.874021805000034], [-87.72285713199992, 41.87393537100006], [-87.7207863659999, 41.87373890600008], [-87.72024380899987, 41.87374511700005], [-87.7196266409999, 41.87374834700005], [-87.71961018999988, 41.87314242300004], [-87.71957219999989, 41.87216510900002], [-87.71953995099992, 41.87131188800003], [-87.71950941599992, 41.87039543600007], [-87.71947757399992, 41.869358867000074], [-87.71946209199987, 41.868565769000035], [-87.71942710399989, 41.86758882200006], [-87.71940758399988, 41.866975783000036], [-87.71938, 41.86616], [-87.71834538199992, 41.86616623300005], [-87.71724686599991, 41.86617769300005], [-87.71602648399993, 41.86619386300003], [-87.7144398489999, 41.86620702600003], [-87.71199638199992, 41.86626635500004], [-87.71101943699993, 41.866318193000026], [-87.70992354899991, 41.86635], [-87.70888588399991, 41.86635690200005], [-87.7075434899999, 41.86637448800007], [-87.70638419399988, 41.86638732000005], [-87.70563999999987, 41.86639], [-87.7056436379999, 41.866611937000066], [-87.7056664749999, 41.86728244300007], [-87.70577310499992, 41.86821983500004], [-87.70576546999987, 41.869485811000054], [-87.70579761299989, 41.87039639300007], [-87.70581645299988, 41.87125038300007], [-87.70584337699994, 41.87216768100006], [-87.7058646799999, 41.872657637000074], [-87.70588214499989, 41.87314435000006], [-87.70591043399989, 41.874231236000064], [-87.7054078489999, 41.87430387000005], [-87.70486379599991, 41.87439572300007], [-87.70436942299992, 41.874481505000055], [-87.7038092259999, 41.874554571000026], [-87.70314489099988, 41.87463834100003], [-87.7025303429999, 41.874732618000046], [-87.70187446399987, 41.87483323600008], [-87.7013781719999, 41.87490086700006], [-87.70033984799991, 41.87500988200007], [-87.69966303499987, 41.87507263200006], [-87.69893692999993, 41.87512595400005], [-87.6982060629999, 41.87515587800004], [-87.69741305299992, 41.875174112000074], [-87.69680179599992, 41.87518625600005], [-87.69611, 41.8752], [-87.69611780699994, 41.87565039400004], [-87.69545893699988, 41.87565561100006], [-87.69454074999992, 41.87567989300003], [-87.69289641899988, 41.87569396600008], [-87.69161497499992, 41.87571], [-87.69134345799992, 41.875712032000074], [-87.69133029299988, 41.87539265600008], [-87.69132466699992, 41.87525618800003], [-87.69131158299989, 41.87493879200007], [-87.69126627599988, 41.87383969100005], [-87.69122884499987, 41.87293162100007], [-87.69119133799987, 41.872021691000036], [-87.69115384699988, 41.87111215000004], [-87.6912965209999, 41.871109536000056], [-87.69124659499992, 41.87103322600006], [-87.69123249499994, 41.87101167500003], [-87.69122396699991, 41.870625717000046], [-87.69126344799989, 41.87030348800004], [-87.69122428299991, 41.87019579200005], [-87.6912131219999, 41.87010424800007], [-87.6912120349999, 41.87004873400007], [-87.6912228279999, 41.869794488000075], [-87.69124000599992, 41.869389865000066], [-87.6911927719999, 41.86929465800005], [-87.69119005099991, 41.86919954500006], [-87.6911854, 41.86903824800003], [-87.69117608299989, 41.868715107000064], [-87.69117550999992, 41.86863483400002], [-87.69117310099993, 41.86855455200002], [-87.69116977399993, 41.868474264000035], [-87.69116175599991, 41.86835827400005], [-87.6911506089999, 41.86823403400006], [-87.69113998399989, 41.86814924600007], [-87.69112794599992, 41.868068223000066], [-87.69111166399993, 41.86795253100006], [-87.69109032199991, 41.867838182000064], [-87.6910727239999, 41.86776227400008], [-87.69105237299992, 41.867686350000035], [-87.69103018099989, 41.867611102000055], [-87.69100523099988, 41.86753618100004], [-87.6909779839999, 41.86746159100004], [-87.69094843499992, 41.86738767400003], [-87.69095013099988, 41.86729408600006], [-87.69087163199991, 41.867220531000044], [-87.69086213199994, 41.86716760800004], [-87.6908422089999, 41.86705662700006], [-87.69071411799989, 41.86695620100004], [-87.69064909799991, 41.86684984100008], [-87.69058454199988, 41.866743139000036], [-87.6904747849999, 41.86656757900004], [-87.69052782399989, 41.86656682000005], [-87.69073116099989, 41.86656461200003], [-87.69075383699992, 41.86656436100007], [-87.69074491799994, 41.86655175300005], [-87.69179906899988, 41.866537599000075], [-87.69277451899988, 41.86653], [-87.69423920499989, 41.86652], [-87.69539874799993, 41.86651], [-87.6966803639999, 41.86649039400003], [-87.69759577699993, 41.86648], [-87.69887726099992, 41.86647546900007], [-87.69985345899991, 41.86646533100008], [-87.70131886599992, 41.866451532000035], [-87.70247811699991, 41.866431998000046], [-87.70320999999987, 41.86642]]]]}, "properties": {"city": "Chicago", "name": "Archer Heights", "regionid": "269568", "geo_point_2d": [41.84160335295369, -87.72601193764415], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.62431281660412, 41.71501650940129], [-87.62351446236424, 41.71472805339215], [-87.62361, 41.71367295697925], [-87.62359687578122, 41.71292941015772], [-87.62357682578846, 41.71200081206073], [-87.62355285156637, 41.711165546992596], [-87.62353005568218, 41.71032753353894], [-87.62351366526764, 41.70930620882321], [-87.62350140219895, 41.70856300665282], [-87.62347865863875, 41.707539333512344], [-87.62346352776115, 41.706703179998804], [-87.62343106959013, 41.70568208770569], [-87.62341145195462, 41.704939693821096], [-87.62338737584969, 41.70401127548964], [-87.62331602050176, 41.70159704559116], [-87.62328, 41.69984], [-87.62404718858109, 41.699833498401894], [-87.62488317489475, 41.69983729611888], [-87.62636971668212, 41.69982790392119], [-87.62739157733624, 41.699815478874335], [-87.62906413905712, 41.69980333846548], [-87.63045680649697, 41.6998013836895], [-87.63296592764027, 41.69976], [-87.63483036792154, 41.69968522057525], [-87.63593988513787, 41.69967479088135], [-87.63724041764961, 41.69966406164509], [-87.63835519630335, 41.699654957407375], [-87.6396557802866, 41.699644153106895], [-87.64086400960446, 41.699628294924565], [-87.64197815560267, 41.699614618092106], [-87.64269485739167, 41.69960402402151], [-87.64275191573817, 41.70144302225813], [-87.64279891610613, 41.7032660217558], [-87.64282191536756, 41.70412502268201], [-87.64282691559809, 41.705104022844374], [-87.64287491601215, 41.706857022712846], [-87.64287581054032, 41.706886378599975], [-87.64287999999988, 41.70689], [-87.64291170572689, 41.70781983392245], [-87.64295331214663, 41.70958389970931], [-87.64300162986773, 41.71135030510718], [-87.64307438881043, 41.71329764038794], [-87.64298430636661, 41.71480632060753], [-87.64216393298024, 41.71482205836947], [-87.63528957262844, 41.71489626220046], [-87.63333805194259, 41.71474546452159], [-87.63333891530432, 41.71478202290351], [-87.63334291548858, 41.71490902246247], [-87.6333430242796, 41.71491438922053], [-87.63287371570948, 41.71491852959061], [-87.6307462528778, 41.714998404010494], [-87.62924939372498, 41.71487887630738], [-87.62794357234088, 41.71503927230085], [-87.6271956957907, 41.71517804175429], [-87.62618429230155, 41.71526330336565], [-87.62524892291437, 41.715199338437195], [-87.62466769024145, 41.71510363734703], [-87.62431281660412, 41.71501650940129]]]}, "properties": {"city": "Chicago", "name": "Fernwood", "regionid": "403377", "geo_point_2d": [41.707339592182805, -87.63317019597554], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.64559816591213, 41.87671355800389], [-87.64547964306549, 41.875734260218984], [-87.6461483424506, 41.87577291712252], [-87.64834961791426, 41.875802662164595], [-87.64837, 41.87665], [-87.64838011380863, 41.877292226852106], [-87.64840875649702, 41.87839266372813], [-87.64845276838867, 41.87973329684465], [-87.64848681735053, 41.88101395088089], [-87.64851, 41.881750000000146], [-87.64822274205049, 41.88175495272334], [-87.6476126484593, 41.88176537883968], [-87.64681934635689, 41.88178446260118], [-87.64584382055028, 41.88180316591475], [-87.64588, 41.88040426502477], [-87.64591283385796, 41.87912366231974], [-87.6457586469701, 41.87771173409705], [-87.64559816591213, 41.87671355800389]]]}, "properties": {"city": "Chicago", "name": "Greektown", "regionid": "403320", "geo_point_2d": [41.878737205852204, -87.64709654402405], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.57772531683436, 41.72352076123646], [-87.58041952752616, 41.72352076123646], [-87.58030623817945, 41.724641001770756], [-87.58034178069788, 41.72547286815827], [-87.58035967993038, 41.72649495673222], [-87.5803878893323, 41.727516679972275], [-87.58040798504203, 41.72863059396159], [-87.58041413432204, 41.72918795466878], [-87.58042, 41.729710000000104], [-87.57949025961376, 41.72971774783652], [-87.57837545491961, 41.72973362169492], [-87.57716790545794, 41.7297467115252], [-87.57596041883711, 41.72976274930973], [-87.57475284972838, 41.72977261917119], [-87.5737307882272, 41.72978490857438], [-87.57242920382214, 41.729801303164244], [-87.5711288877394, 41.7298224772501], [-87.57067, 41.72983], [-87.57063939715118, 41.72844522109537], [-87.57060964852948, 41.72677177647775], [-87.57055571532578, 41.72482193666094], [-87.57052419392733, 41.723707757095355], [-87.5741949717899, 41.72361366505337], [-87.57642466339698, 41.72352076123646], [-87.57772531683436, 41.72352076123646]]]}, "properties": {"city": "Chicago", "name": "Pill Hill", "regionid": "403363", "geo_point_2d": [41.72667118164577, -87.57550034706442], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.63065650156882, 41.823628048864705], [-87.63063703931647, 41.82304190427983], [-87.63061, 41.8216768439968], [-87.63058147651367, 41.8201938256827], [-87.63055612933483, 41.81898452671808], [-87.63052600558042, 41.81793019531519], [-87.63049814269971, 41.81695499449332], [-87.63043955216237, 41.81578262158736], [-87.63039726163096, 41.81500147974638], [-87.63033202451133, 41.814376551970796], [-87.63029716965714, 41.81390670269993], [-87.63031289906203, 41.81343875290546], [-87.63031741208275, 41.812659359779246], [-87.63033733719628, 41.812151450747606], [-87.6303803922814, 41.81180764631131], [-87.63055588431011, 41.811287054517635], [-87.6307616426127, 41.81088462542768], [-87.63097764299846, 41.8105516140691], [-87.63127703809437, 41.81008650297759], [-87.63151582367706, 41.80965835264596], [-87.63167099956759, 41.80931025097296], [-87.63182531709914, 41.80886379502342], [-87.63190083638855, 41.80836467843154], [-87.63192, 41.80785672783449], [-87.63188637269944, 41.807271610431606], [-87.6318451987831, 41.80664443637803], [-87.63179502294805, 41.80594531974358], [-87.63171708028194, 41.80484963486306], [-87.6316628150991, 41.8039135604616], [-87.63162659472415, 41.803288758993524], [-87.63158274792409, 41.80258631442737], [-87.63154955201283, 41.80211742218145], [-87.63149112812906, 41.80125729797928], [-87.63147456598591, 41.80071215671844], [-87.63145058126044, 41.79989187910975], [-87.63143640051771, 41.79938482088154], [-87.63141802113027, 41.79864351892173], [-87.63140183993993, 41.79786335825925], [-87.6313835133602, 41.79719999433191], [-87.63136549937667, 41.796497471948804], [-87.63134204847377, 41.79544218131801], [-87.63133075658448, 41.794934046305805], [-87.63131883538836, 41.794454039555724], [-87.63131991576485, 41.79445402267909], [-87.63150891548383, 41.79445002249465], [-87.63160691640402, 41.79444902244855], [-87.63173691520188, 41.79445402267909], [-87.6329239159593, 41.794426022287574], [-87.63413191498658, 41.794398021896335], [-87.6352759155605, 41.79436802141272], [-87.63536390612717, 41.79435094418678], [-87.63564852691304, 41.79435495712467], [-87.63568150941073, 41.79630092448943], [-87.63583766891537, 41.80094666975311], [-87.63595478854394, 41.80781768795818], [-87.63595478854394, 41.81016008052805], [-87.63597427886742, 41.81145618704127], [-87.63599605037336, 41.811566020979754], [-87.63598691595949, 41.81162802293842], [-87.6359569154759, 41.81185002238002], [-87.63592591584558, 41.812469022135375], [-87.63596291305458, 41.81394902290932], [-87.63595584741462, 41.813971546976994], [-87.63607914065939, 41.813994664460466], [-87.63604755517272, 41.81448423950418], [-87.63604755517272, 41.81530013537771], [-87.63604755517272, 41.815795500729465], [-87.63622239000279, 41.81699020540146], [-87.63622239000279, 41.81763126644491], [-87.63622239000279, 41.81847630145692], [-87.63630980741779, 41.82150677184418], [-87.63630980741779, 41.82357372192024], [-87.63624390892056, 41.823574076283094], [-87.63602991524374, 41.823574022323854], [-87.63531191542138, 41.82358602197749], [-87.633747094191, 41.82361796409746], [-87.6332099156531, 41.823622021838354], [-87.6325439155296, 41.823622021838354], [-87.6313679161787, 41.82362002264563], [-87.63119791553696, 41.82362402193054], [-87.63089691605092, 41.8236270220689], [-87.63066091506471, 41.823628022114995], [-87.63065650156882, 41.823628048864705]]]}, "properties": {"city": "Chicago", "name": "Fuller Park", "regionid": "269581", "geo_point_2d": [41.81002198594253, -87.63350506210172], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.62898582706111, 41.82370803659457], [-87.62898504465097, 41.823664889821565], [-87.62862091545645, 41.82366902220631], [-87.62757891508797, 41.82367502248277], [-87.62649391543518, 41.82368602209056], [-87.62641295525805, 41.82368699756742], [-87.62641, 41.82338075149137], [-87.62641, 41.82268397286404], [-87.62638259382129, 41.821500049768396], [-87.62636790010066, 41.82080310896198], [-87.62635336122833, 41.81989862392775], [-87.6263242896758, 41.81885309087649], [-87.62629234327834, 41.817810656574146], [-87.62625691915292, 41.816828902237205], [-87.62622303225005, 41.816149096750465], [-87.62619163493845, 41.81557809630443], [-87.62618282399768, 41.81516066789721], [-87.62616184219479, 41.81425552998726], [-87.62614655614762, 41.813628802053266], [-87.62612191983072, 41.81258235229743], [-87.62610121737026, 41.811675520870054], [-87.62608317891625, 41.81055846128381], [-87.62606820785027, 41.8096552069713], [-87.6260567101084, 41.808889055961295], [-87.6260397919019, 41.80826139484852], [-87.62601873038324, 41.80770413686175], [-87.62600404295232, 41.807075761048765], [-87.62598243254864, 41.80624018417094], [-87.62595029463631, 41.80491679427593], [-87.62593093817524, 41.80400986164409], [-87.62590082278835, 41.80310509504939], [-87.62588256625314, 41.80254827072265], [-87.62586533200836, 41.80185502209748], [-87.62586501989979, 41.80185502209748], [-87.62584616949988, 41.80108389874501], [-87.62580051526366, 41.799552881591424], [-87.62577348693885, 41.79844029938969], [-87.62574374032107, 41.79732012307496], [-87.62569459435413, 41.79551008223827], [-87.62569999999982, 41.79457000000027], [-87.6262249045568, 41.79455500272697], [-87.6271523830477, 41.79453195233897], [-87.62797713870339, 41.794516930558096], [-87.62848682122615, 41.79451068150401], [-87.62848686161543, 41.79451202085611], [-87.62947391520673, 41.794495021871136], [-87.6304489160841, 41.794470022517174], [-87.63056191589708, 41.7944670214798], [-87.63059091543533, 41.7944670214798], [-87.63072891550127, 41.79446402224063], [-87.63092991577362, 41.79446102210244], [-87.63112791500804, 41.794457021918085], [-87.63131883538836, 41.794454039555724], [-87.63133075658448, 41.794934046305805], [-87.63134204847377, 41.79544218131801], [-87.63136549937667, 41.796497471948804], [-87.6313835133602, 41.79719999433191], [-87.63140183993993, 41.79786335825925], [-87.63141802113027, 41.79864351892173], [-87.63143640051771, 41.79938482088154], [-87.63145058126044, 41.79989187910975], [-87.63147456598591, 41.80071215671844], [-87.63149112812906, 41.80125729797928], [-87.63154955201283, 41.80211742218145], [-87.63158274792409, 41.80258631442737], [-87.63162659472415, 41.803288758993524], [-87.6316628150991, 41.8039135604616], [-87.63171708028194, 41.80484963486306], [-87.63179502294805, 41.80594531974358], [-87.6318451987831, 41.80664443637803], [-87.63188637269944, 41.807271610431606], [-87.63192, 41.80785672783449], [-87.63190083638855, 41.80836467843154], [-87.63182531709914, 41.80886379502342], [-87.63167099956759, 41.80931025097296], [-87.63151582367706, 41.80965835264596], [-87.63127703809437, 41.81008650297759], [-87.63097764299846, 41.8105516140691], [-87.6307616426127, 41.81088462542768], [-87.63055588431011, 41.811287054517635], [-87.6303803922814, 41.81180764631131], [-87.63033733719628, 41.812151450747606], [-87.63031741208275, 41.812659359779246], [-87.63031289906203, 41.81343875290546], [-87.63029716965714, 41.81390670269993], [-87.63033202451133, 41.814376551970796], [-87.63039726163096, 41.81500147974638], [-87.63043955216237, 41.81578262158736], [-87.63049814269971, 41.81695499449332], [-87.63052600558042, 41.81793019531519], [-87.63055612933483, 41.81898452671808], [-87.63058147651367, 41.8201938256827], [-87.63061, 41.8216768439968], [-87.63063703931647, 41.82304190427983], [-87.63065650156882, 41.823628048864705], [-87.63049591555261, 41.82362902216108], [-87.63026791583458, 41.82363402239171], [-87.63003191574762, 41.823637022529795], [-87.62961691550333, 41.8236580225986], [-87.62934986002807, 41.82366102183759], [-87.62935004798636, 41.823700151339], [-87.62898582706111, 41.82370803659457]]]}, "properties": {"city": "Chicago", "name": "The Robert Taylor Homes", "regionid": "403350", "geo_point_2d": [41.80805336765706, -87.6285965818895], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.70902028099992, 41.99026176100005], [-87.70852529099993, 41.988536867000036], [-87.7080433389999, 41.98683664600008], [-87.70733379799991, 41.98460092200008], [-87.70687861999991, 41.98316845200008], [-87.7068224309999, 41.982965544000066], [-87.7070014749999, 41.98296339800003], [-87.70784750699993, 41.982955961000066], [-87.70929200199991, 41.982931633000064], [-87.71068309899991, 41.98290873000007], [-87.71197108499987, 41.98289374100005], [-87.7130533689999, 41.98289510800002], [-87.7141869689999, 41.9829], [-87.71547550599989, 41.98287450800005], [-87.71676320099992, 41.98284386100004], [-87.7178976679999, 41.98282094800003], [-87.7210307689999, 41.98278379300006], [-87.7213505899999, 41.98278], [-87.72676191899987, 41.98271194000005], [-87.72834, 41.98272], [-87.7283583469999, 41.98338508400006], [-87.72837113299992, 41.98384856200004], [-87.7283926779999, 41.98456934600005], [-87.72839855099993, 41.98478430000005], [-87.72841804899988, 41.985497881000065], [-87.72843334299992, 41.98611712600006], [-87.72844539899987, 41.98668377400007], [-87.7284605759999, 41.987301904000034], [-87.72847785499991, 41.98787011300003], [-87.72849709399989, 41.98838476200006], [-87.7285168369999, 41.98931114800006], [-87.72857, 41.99007], [-87.72727632099989, 41.98997032500005], [-87.72531892599993, 41.99004], [-87.7232574529999, 41.99016], [-87.72202056799993, 41.99016], [-87.7206811419999, 41.99016958200007], [-87.71944372599992, 41.99018], [-87.71876, 41.99014], [-87.71815537699989, 41.990144997000066], [-87.71717642599991, 41.990156074000076], [-87.71563075199991, 41.99018148700008], [-87.71439422199988, 41.99020219800006], [-87.71326065499993, 41.99022071000007], [-87.71217703199991, 41.99023], [-87.7102186319999, 41.99024], [-87.7091713959999, 41.990259978000026], [-87.70902028099992, 41.99026176100005]]]}, "properties": {"city": "Chicago", "name": "Hollywood Park", "regionid": "403268", "geo_point_2d": [41.986431988849084, -87.71818584258276], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.67296195459818, 41.95417427664474], [-87.67379143542257, 41.95416410716499], [-87.67378791557026, 41.95421602261311], [-87.67378382455445, 41.95427401989085], [-87.67382791561556, 41.956066022456206], [-87.67386391547643, 41.9578730230149], [-87.67391891531365, 41.95969502246666], [-87.67396091545118, 41.96153202081129], [-87.67241091504795, 41.9615490224942], [-87.67165991550297, 41.96155802200983], [-87.671636915342, 41.961559022055916], [-87.67082391563702, 41.96156702242463], [-87.66930891594787, 41.961586022401164], [-87.6692740094022, 41.961588349587956], [-87.66911738599455, 41.95998650383405], [-87.66909076102644, 41.95905970136302], [-87.66904439123125, 41.95720540979517], [-87.66899743571763, 41.95529866012244], [-87.668972348249, 41.95438547626307], [-87.66911, 41.95428], [-87.66996715575817, 41.954214962179684], [-87.6710186943349, 41.95419513146137], [-87.67182835338137, 41.95418502225172], [-87.67296195459818, 41.95417427664474]]]}, "properties": {"city": "Chicago", "name": "South East Ravenswood", "regionid": "403293", "geo_point_2d": [41.95787110649144, -87.6714753098802], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.63167391589513, 41.91117402242847], [-87.63160491930552, 41.90988854750011], [-87.63155848016704, 41.908270755742045], [-87.63151279257715, 41.906409628863024], [-87.63145453902325, 41.90462589875515], [-87.63141, 41.90389], [-87.63215632027983, 41.90388197505081], [-87.63377482094442, 41.90385439957842], [-87.63563826512801, 41.90383450590851], [-87.63822952384396, 41.90378259088512], [-87.6431983270477, 41.90371243973028], [-87.64322360469514, 41.9044654676256], [-87.6432571252119, 41.905517389194664], [-87.64332890860403, 41.90802730248022], [-87.64337402989685, 41.90981279974093], [-87.64340343910216, 41.91045864902635], [-87.64341964599532, 41.911029492784586], [-87.6434169158154, 41.91103002118635], [-87.64286689226088, 41.91104215304053], [-87.64274593884392, 41.910991890831745], [-87.64221291517373, 41.91099802240929], [-87.6416559155784, 41.91100002250147], [-87.6410019151088, 41.91101002206319], [-87.64040991569867, 41.911013022201374], [-87.6397829155744, 41.911022022616265], [-87.63856991541712, 41.911043021785794], [-87.63756591590088, 41.911045022777074], [-87.63665091599074, 41.911082022684035], [-87.63566991573602, 41.911075022361395], [-87.63468991552746, 41.91108802206146], [-87.6344229158102, 41.91110002261446], [-87.63394286130837, 41.91110585651663], [-87.63386597017411, 41.911162188249776], [-87.63321191574528, 41.911165022013485], [-87.63305091551825, 41.911166022059575], [-87.63247991617676, 41.91116302192122], [-87.63167391589513, 41.91117402242847]]]}, "properties": {"city": "Chicago", "name": "Old Town", "regionid": "403301", "geo_point_2d": [41.90743359545495, -87.63742462690102], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.69115999999988, 41.96129], [-87.6911607569999, 41.96128604400006], [-87.6918229159999, 41.961289022000074], [-87.69237591499989, 41.96128302200008], [-87.69359691599992, 41.96127302200004], [-87.69454099999989, 41.961263], [-87.69470771099991, 41.96126155000007], [-87.69470931899988, 41.96127977400003], [-87.69481641899993, 41.96158768800005], [-87.69504400799988, 41.96192237700006], [-87.69523143399992, 41.962230291000076], [-87.69539208499991, 41.96263191800005], [-87.6955527359999, 41.963020158000056], [-87.69567322399989, 41.96323435900006], [-87.69620872699991, 41.96383680000007], [-87.69679777999991, 41.96451956600002], [-87.69745377099989, 41.96506845600003], [-87.69797588599992, 41.965563796000076], [-87.6980685709999, 41.96565442100006], [-87.69790597699993, 41.96565524000004], [-87.69696, 41.96566], [-87.69582853499989, 41.96567098500008], [-87.69472767699993, 41.965681673000056], [-87.69387, 41.96569], [-87.69239181999988, 41.96571], [-87.69129255399989, 41.96571338000006], [-87.69129999999988, 41.96571], [-87.6912655769999, 41.96508098600008], [-87.69124208299989, 41.96411927900003], [-87.69122214799992, 41.96350128300008], [-87.6911919979999, 41.96274590600005], [-87.6911799199999, 41.96219635800007], [-87.69115999999988, 41.96129]]]}, "properties": {"city": "Chicago", "name": "Ravenswood Gardens", "regionid": "403261", "geo_point_2d": [41.96372316252739, -87.69374958720542], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.78618999999989, 41.94558], [-87.78699537399987, 41.94556721600008], [-87.78814954199991, 41.94554293800007], [-87.7893627479999, 41.94551871500004], [-87.79124545099988, 41.94547717200003], [-87.7927634629999, 41.94544727000004], [-87.79391620099993, 41.94542226800006], [-87.7952516969999, 41.945395081000065], [-87.79652628299992, 41.94536924100003], [-87.7976793049999, 41.945345627000044], [-87.79895528099989, 41.94532], [-87.80029081299993, 41.945295779000034], [-87.80132372299991, 41.94528026700004], [-87.80265921699991, 41.94525872200005], [-87.8038744929999, 41.94523341400007], [-87.80490760699988, 41.94520801000004], [-87.80630213999989, 41.94517940600007], [-87.80688880799991, 41.94516980000008], [-87.80690107199987, 41.945502156000074], [-87.8069212899999, 41.946108686000045], [-87.80696281499988, 41.94738445300003], [-87.80699125499987, 41.94823763800007], [-87.80702716799992, 41.949329349000024], [-87.80705667699992, 41.95023919700007], [-87.8070850319999, 41.951027692000075], [-87.80712363899993, 41.951997680000034], [-87.80718418599989, 41.952651368000026], [-87.80718404399991, 41.952662540000055], [-87.8071837609999, 41.95268956800003], [-87.8071832499999, 41.95270846600003], [-87.8071831059999, 41.95271963700007], [-87.8071759129999, 41.953048835000054], [-87.8071746649999, 41.95310592000003], [-87.8071734179999, 41.95316300600006], [-87.8071731939999, 41.95317324100005], [-87.80717194599991, 41.95323032700008], [-87.80717069799992, 41.95328741100007], [-87.80716226199988, 41.953673386000034], [-87.80716084399987, 41.95373048700007], [-87.80126100999988, 41.956531512000076], [-87.80038617699992, 41.957009090000035], [-87.80036285499992, 41.95702182400002], [-87.80016199499993, 41.95713144200004], [-87.8000515709999, 41.95718232300003], [-87.7988610289999, 41.95773050300004], [-87.79874741199987, 41.957791665000045], [-87.79764500499991, 41.95838546100002], [-87.79656100999993, 41.958930468000055], [-87.79519100399992, 41.95959745500004], [-87.79411601099989, 41.96006926300003], [-87.7919650119999, 41.96009047500006], [-87.79118401299988, 41.96010047400005], [-87.79039301599992, 41.96011146700004], [-87.78918000899994, 41.960111482000045], [-87.78796701599991, 41.96013146500006], [-87.7867268009999, 41.960131675000056], [-87.78671684599992, 41.96010107000006], [-87.78667291599992, 41.95847002300007], [-87.78662291499987, 41.95727002200004], [-87.78658991599988, 41.95646502300007], [-87.78653291599991, 41.95500902300005], [-87.78650991499987, 41.95436602200004], [-87.78648691499988, 41.95371602200003], [-87.78645094599992, 41.95280590400005], [-87.78641775199989, 41.95193930600003], [-87.78638615899989, 41.95133396200003], [-87.7863503109999, 41.95054059400007], [-87.78631163399989, 41.94969218200003], [-87.78626792599992, 41.94853778100007], [-87.78623069199993, 41.947625923000075], [-87.78618540599989, 41.946778067000075], [-87.78615634299987, 41.94586412000006], [-87.78618999999989, 41.94558]]]}, "properties": {"city": "Chicago", "name": "Dunning", "regionid": "138129", "geo_point_2d": [41.95188164243885, -87.79585431496993], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.75095493799994, 41.80744582700002], [-87.75285399999987, 41.80743556700007], [-87.75285399999987, 41.807532], [-87.75285399999987, 41.809432], [-87.75293836699989, 41.81054369800005], [-87.7529839999999, 41.811145], [-87.753154, 41.812832], [-87.753154, 41.813432], [-87.748854, 41.814732], [-87.74859899999988, 41.81481], [-87.744954, 41.815932], [-87.744954, 41.816532], [-87.744954, 41.816832], [-87.7450539999999, 41.817132], [-87.74500025199988, 41.81734699200006], [-87.744954, 41.817532], [-87.744954, 41.818605846000075], [-87.744954, 41.818732], [-87.744954, 41.81896583400003], [-87.74341938599989, 41.81940801100005], [-87.7434078669999, 41.818992750000056], [-87.74331387399988, 41.81560442000004], [-87.74326724299992, 41.813814561000065], [-87.74322842499991, 41.81218856800005], [-87.74317540699988, 41.81014916000003], [-87.74312959799992, 41.80844170600004], [-87.7431072659999, 41.807547666000055], [-87.74310725399992, 41.80754715600005], [-87.74387645099993, 41.80754401300003], [-87.7450969349999, 41.80753367000005], [-87.74623640799987, 41.807518787000035], [-87.74713128999988, 41.807504359000056], [-87.74851422199991, 41.80748177200007], [-87.74989717199989, 41.807459214000055], [-87.75095493799994, 41.80744582700002]]]}, "properties": {"city": "Chicago", "name": "LeClaire Courts", "regionid": "403334", "geo_point_2d": [41.81166844808208, -87.7475615156852], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.66513796688217, 41.88298621486289], [-87.66415250467797, 41.88285633911876], [-87.66287141699146, 41.88288784219181], [-87.66110094149514, 41.88292103611426], [-87.66012422811109, 41.88293], [-87.65920820391442, 41.88294036914491], [-87.65804809331581, 41.882962777109725], [-87.65725532810217, 41.882973925223524], [-87.65609595083303, 41.88298508152413], [-87.65487536623228, 41.883002225135016], [-87.65365520278964, 41.883028646883886], [-87.65249395278015, 41.88305262690843], [-87.6512737782105, 41.883073495284705], [-87.65017601469094, 41.88308615504576], [-87.64919891641506, 41.88310350212518], [-87.64854999999986, 41.88312000000017], [-87.64851, 41.881750000000146], [-87.64848681735053, 41.88101395088089], [-87.64845276838867, 41.879733296844826], [-87.64840875649702, 41.878392663728306], [-87.64838011380863, 41.87729222685228], [-87.64836999999982, 41.87665], [-87.64901723310834, 41.87664442040424], [-87.65029862698574, 41.8766400000002], [-87.65139717582281, 41.87662181280481], [-87.65353403283999, 41.87657418784901], [-87.65513159458759, 41.87621616047219], [-87.65823160672153, 41.87616165391515], [-87.65981990102861, 41.87614873788406], [-87.6615293453486, 41.87611941377745], [-87.66390866077666, 41.87606621049238], [-87.66598334359286, 41.876041730459264], [-87.66720375402922, 41.876019694355044], [-87.66860685719298, 41.87598892165838], [-87.67037651651351, 41.87593861570334], [-87.67166010092453, 41.87589999889101], [-87.67550449211926, 41.87584363709445], [-87.67788379131763, 41.875824111526434], [-87.67995944866327, 41.87582018191976], [-87.68282762353182, 41.875797920794106], [-87.68526886130712, 41.87577062627119], [-87.68954028090018, 41.875726284529804], [-87.691229971124, 41.87571302170581], [-87.69122996337943, 41.87571274291459], [-87.69161497457462, 41.87571000000017], [-87.69289641949999, 41.87569396616093], [-87.69454075038372, 41.87567989280237], [-87.6954589368243, 41.87565561124054], [-87.6961178074452, 41.87565039429304], [-87.6961293350725, 41.87631539768441], [-87.69614756748425, 41.87698621317842], [-87.69618019733097, 41.87796374928958], [-87.69621048611614, 41.87881711829029], [-87.69622915471598, 41.879671116936755], [-87.6962506318936, 41.880648435219506], [-87.69627326333317, 41.881686744994816], [-87.69631036400976, 41.882540479376324], [-87.69633, 41.88289000000023], [-87.69570378164063, 41.882897893508826], [-87.69332387340063, 41.88292359842871], [-87.69220988041904, 41.88293282685296], [-87.69220991160086, 41.88293388009107], [-87.69106624475455, 41.882946386208026], [-87.68929591884584, 41.882969756846954], [-87.68776914924746, 41.88298754623887], [-87.6871589251005, 41.88299367914478], [-87.68550966498697, 41.883004937246966], [-87.68441274594078, 41.88302443755359], [-87.68307024696585, 41.88304501637614], [-87.68197202529147, 41.88306294652586], [-87.6809348159511, 41.88307992105005], [-87.68038570631573, 41.88308892284739], [-87.67959254795363, 41.88310192544347], [-87.67831024758134, 41.883125954594576], [-87.6777002531868, 41.8831356516803], [-87.67702906651726, 41.883144909427386], [-87.67623583015941, 41.88315521520301], [-87.67550359790911, 41.883164218058624], [-87.67446551865022, 41.88317698132808], [-87.6736721554438, 41.88319122632296], [-87.67294011989473, 41.88320622705144], [-87.67202382539766, 41.88322500357791], [-87.67141451511826, 41.88323000000012], [-87.67104834364778, 41.88323000000012], [-87.66976616770025, 41.88324842457869], [-87.66903440055992, 41.88327000000022], [-87.66726504196457, 41.88329219899282], [-87.66610555336689, 41.883270116498416], [-87.66594317556644, 41.8832219768506], [-87.66548515653076, 41.88306843756744], [-87.66513796688217, 41.88298621486289]]]}, "properties": {"city": "Chicago", "name": "Near West Side", "regionid": "269599", "geo_point_2d": [41.87951726620309, -87.67280902043542], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.68567512399991, 41.854465301000054], [-87.68565350799989, 41.85349243800004], [-87.68561930799987, 41.852453504000025], [-87.68561238699992, 41.852032423000075], [-87.68560526499992, 41.85159909500003], [-87.6855856579999, 41.85068494800003], [-87.6855505169999, 41.849771638000036], [-87.68552429999993, 41.848791329000036], [-87.68550675599992, 41.847998268000076], [-87.68548031999988, 41.84708342000004], [-87.68546192599987, 41.846168218000045], [-87.68544500499988, 41.845435135000045], [-87.68542261099991, 41.84484968700007], [-87.68541, 41.84452], [-87.6853689599999, 41.84384957100007], [-87.6853098419999, 41.842750498000036], [-87.68526594699993, 41.84177601800006], [-87.68523199899991, 41.84092156500008], [-87.68519515099987, 41.83963927000006], [-87.68513869699987, 41.83872818900004], [-87.68512556699991, 41.83825737100004], [-87.68507, 41.838305], [-87.68508452299993, 41.837864138000036], [-87.68615592799992, 41.83752741100004], [-87.68720445499989, 41.83712797200006], [-87.68850263199991, 41.83675349800006], [-87.69062465199993, 41.836079445000045], [-87.69374526899989, 41.835055882000056], [-87.6970656059999, 41.83398239000007], [-87.69771469499993, 41.83385756500006], [-87.69831385299992, 41.83375770500003], [-87.6988880469999, 41.833582951000025], [-87.69946223999993, 41.83333330100003], [-87.7006355919999, 41.832933862000054], [-87.70230824299992, 41.83243456300004], [-87.70490459699992, 41.831610720000064], [-87.7071764059999, 41.83081184200005], [-87.7102221289999, 41.82988814000002], [-87.71386700999992, 41.82868982300004], [-87.71432292899992, 41.82855051400007], [-87.7174619609999, 41.82759136500003], [-87.72073236899989, 41.82649290800002], [-87.72312900299993, 41.82576892500002], [-87.72372816099993, 41.825519275000026], [-87.72542577699993, 41.825019976000064], [-87.72919548299987, 41.82384662400005], [-87.73441315499988, 41.82217397300007], [-87.73853914599992, 41.82088715400004], [-87.7394560729999, 41.82060118200007], [-87.74335060299988, 41.819427830000045], [-87.74341938599989, 41.81940801100005], [-87.744954, 41.81896583400003], [-87.744954, 41.819330630000074], [-87.744954, 41.819332], [-87.7450539999999, 41.820332], [-87.74496598799992, 41.821916226000035], [-87.744954, 41.822132], [-87.743454, 41.822132], [-87.74125399999988, 41.822232], [-87.73937375799993, 41.822159683000045], [-87.738654, 41.822132], [-87.73864137499992, 41.822182500000046], [-87.73857363799992, 41.82245344800002], [-87.73855399999988, 41.822532], [-87.73857490399989, 41.82255290300007], [-87.738654, 41.822632], [-87.73925399999987, 41.829532], [-87.738955, 41.832332], [-87.738855, 41.832932], [-87.7387687269999, 41.83349277200004], [-87.738655, 41.834232], [-87.738655, 41.836432], [-87.738655, 41.836911], [-87.738655, 41.83692150000006], [-87.738655, 41.836932], [-87.7387559999999, 41.840431], [-87.738814, 41.842423], [-87.738855, 41.843832], [-87.738955, 41.845332], [-87.7389616479999, 41.84537853500005], [-87.7390442429999, 41.845956703000056], [-87.73786105399994, 41.84637326000006], [-87.73597648499991, 41.84706515300007], [-87.73493217499991, 41.847403949000054], [-87.7336449819999, 41.847814410000055], [-87.73256912999993, 41.848192429000036], [-87.73178390199988, 41.84847043500007], [-87.73041960999987, 41.84894676100003], [-87.72868579799989, 41.84955207500007], [-87.72657582499991, 41.85029344200006], [-87.72250840799987, 41.851880808000026], [-87.7214665959999, 41.852242141000026], [-87.7204672529999, 41.852597015000065], [-87.71926627899988, 41.853027784000055], [-87.71794289599987, 41.85349829500006], [-87.71688080699992, 41.85361272100004], [-87.71413452099992, 41.85361272100004], [-87.71266983499991, 41.85361272100004], [-87.71110506699992, 41.85341065400006], [-87.71110505599994, 41.85341114500004], [-87.71006191599989, 41.853544022000044], [-87.70893291499992, 41.853559022000056], [-87.7077179159999, 41.853563022000046], [-87.7065049159999, 41.85358202200006], [-87.70534991599992, 41.853589022000044], [-87.7052929149999, 41.85359002200004], [-87.70523091499989, 41.85359102200004], [-87.70406491499993, 41.853600022000066], [-87.70372891499993, 41.85361202300003], [-87.70361491499993, 41.853625023000056], [-87.7028617689999, 41.85379090600008], [-87.70286178899991, 41.85379138600006], [-87.70286, 41.853790000000004], [-87.7009523489999, 41.85391786300004], [-87.70028103399993, 41.85391786300004], [-87.6992435479999, 41.85403992100003], [-87.69796194799991, 41.85452814900008], [-87.69552999999989, 41.85523], [-87.69538906699992, 41.85534801000005], [-87.6943009159999, 41.85557702300007], [-87.69310391599991, 41.85584802300007], [-87.6916469159999, 41.85617602200005], [-87.69152191599991, 41.856192022000066], [-87.69073191499989, 41.85637702300004], [-87.69018291599988, 41.856513022000044], [-87.68997091599992, 41.856552023000056], [-87.68896191399992, 41.85678602300003], [-87.68818189499989, 41.85692198600008], [-87.68764811899989, 41.85703032100007], [-87.68576972899989, 41.85740238500004], [-87.68575650499989, 41.85703031600006], [-87.68573783499993, 41.85635717300004], [-87.68570142999992, 41.855320043000056], [-87.68567512399991, 41.854465301000054]]]}, "properties": {"city": "Chicago", "name": "Little Village", "regionid": "403117", "geo_point_2d": [41.84040323420177, -87.7138696971508], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.63205087651625, 41.847768027838434], [-87.63207950261454, 41.84736694488234], [-87.6341439087778, 41.84743380626961], [-87.63630167320494, 41.84738446850436], [-87.63818230940733, 41.847343089288664], [-87.64041216672382, 41.84715], [-87.64117795932594, 41.84714286314393], [-87.641848944309, 41.847342988718616], [-87.64178173972346, 41.847585094029085], [-87.64200431901793, 41.84771577647306], [-87.64204894268755, 41.84774197626037], [-87.64232309201499, 41.84806305311073], [-87.64222473406386, 41.84811897475307], [-87.64235710527332, 41.84833000786201], [-87.64228283476359, 41.84849101348489], [-87.64229979058123, 41.84862106804073], [-87.6423438132938, 41.84869810756312], [-87.64242907876768, 41.848768331217926], [-87.64205049771566, 41.84901976578097], [-87.64120481028199, 41.84943102290586], [-87.63998789369255, 41.85025985355051], [-87.63936699876348, 41.851251897272704], [-87.63854, 41.8528], [-87.63769396044216, 41.852802925963694], [-87.6363708770109, 41.852856019362], [-87.63532568265032, 41.85286], [-87.63295680380214, 41.85288465472736], [-87.632, 41.85289], [-87.63199654751806, 41.85250198204111], [-87.63198, 41.85173964232105], [-87.63197596861494, 41.85125052490471], [-87.63195188527516, 41.85055336132992], [-87.6319247720079, 41.849579480706396], [-87.63190953535745, 41.84867385876479], [-87.63205087651625, 41.847768027838434]]]}, "properties": {"city": "Chicago", "name": "Chinatown", "regionid": "137799", "geo_point_2d": [41.84984886437441, -87.63638390323771], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.76647891562953, 41.91652202253871], [-87.76564791599412, 41.91652102249244], [-87.76560091562635, 41.91652102249244], [-87.76553491528188, 41.91652102249244], [-87.76492191580297, 41.91652002244635], [-87.76399691543173, 41.91649102290838], [-87.76316491575022, 41.91642102237963], [-87.76145891535062, 41.91622002300666], [-87.76058591557769, 41.916102022963145], [-87.75975691513509, 41.91601502255005], [-87.759676915944, 41.91600202195078], [-87.75906091542753, 41.915902021837795], [-87.75877391658702, 41.91585502236912], [-87.75818991574698, 41.91578702193263], [-87.757552916061, 41.91571302211863], [-87.75643891597078, 41.91558402156821], [-87.75583491151082, 41.9154440250072], [-87.75574591550213, 41.91545302182489], [-87.75519391613715, 41.91545002168662], [-87.75470291598675, 41.915397022840764], [-87.75274291556977, 41.9151560225233], [-87.75211591544542, 41.915076022432835], [-87.7499839151936, 41.914830022784265], [-87.74936291624533, 41.91478102232413], [-87.7493229162, 41.91477602209376], [-87.74849091561914, 41.91466602241897], [-87.74843491573573, 41.914661022188525], [-87.74755591568629, 41.91458102209807], [-87.74709891620392, 41.91454802237538], [-87.74675091545082, 41.91451102246867], [-87.74626091534653, 41.914455022585194], [-87.74613991516483, 41.914440022792945], [-87.74597491565281, 41.914417022632136], [-87.74596976789938, 41.91441671981251], [-87.7459166574065, 41.913666129582005], [-87.74589905749617, 41.91293348738052], [-87.74589135630289, 41.91268961626238], [-87.74587376900212, 41.912140993066785], [-87.74585970077385, 41.91171425680553], [-87.74583137435405, 41.91116544950583], [-87.74582299571068, 41.91097554932705], [-87.74584737192009, 41.910732082518734], [-87.74599500002361, 41.9102299999212], [-87.7460520197592, 41.909575861685006], [-87.74606, 41.909520000000114], [-87.74598906333459, 41.9095207755506], [-87.74599, 41.909520000000114], [-87.74603526575143, 41.909024797603614], [-87.7459561321766, 41.906707843847016], [-87.745946233557, 41.90554855301906], [-87.7459291380082, 41.90469586423282], [-87.74590479545499, 41.90365515914165], [-87.7458929042872, 41.902863597210185], [-87.74587988824607, 41.90227094174228], [-87.74666209542156, 41.902260973807266], [-87.74928595802561, 41.90222860291535], [-87.75154348498681, 41.90221720185498], [-87.75239761616399, 41.902203262542706], [-87.75368003642774, 41.902186220185754], [-87.7551453552169, 41.90216892860301], [-87.7561829112999, 41.90215585788828], [-87.75722069637368, 41.902150000000155], [-87.75868792176605, 41.902139166567686], [-87.75984492524547, 41.90213], [-87.76161595562428, 41.9021105181691], [-87.76277533814601, 41.902096878374735], [-87.76430119890996, 41.90208041979829], [-87.76491131892055, 41.90207036287498], [-87.76607022485757, 41.90205568922887], [-87.76741298964875, 41.902039101870365], [-87.7685111860717, 41.90202094861173], [-87.76973094574451, 41.90199101944769], [-87.77089066473486, 41.901981956315936], [-87.77192785855965, 41.90196486902418], [-87.77321012381043, 41.90194365652344], [-87.77461335170369, 41.90191994014818], [-87.7752580001528, 41.90190350300791], [-87.7752580001528, 41.902031000266355], [-87.7752580001528, 41.90373100038933], [-87.77531151431026, 41.90383802780497], [-87.77535800026597, 41.90393099971639], [-87.7753320557246, 41.90437205691701], [-87.7752580001528, 41.90563099983947], [-87.7753659997354, 41.9059659999933], [-87.77541907952025, 41.906811237695166], [-87.77542903501508, 41.90696975759194], [-87.77545800037888, 41.907431000075704], [-87.7755514289745, 41.909112725248775], [-87.77539999999989, 41.90922], [-87.77513702099492, 41.909821496119925], [-87.77513952819115, 41.91018437509941], [-87.77516966503828, 41.91079916678115], [-87.77519495964833, 41.91146876254888], [-87.77522484370019, 41.91238520680749], [-87.77527081867993, 41.9135434203014], [-87.77534206291932, 41.91568045385845], [-87.77536217265391, 41.916375071033336], [-87.77530891526955, 41.91637602210377], [-87.77504291559836, 41.916380022288216], [-87.76961991562821, 41.916464022563126], [-87.76803691550231, 41.91650902193944], [-87.76647891562953, 41.91652202253871]]]}, "properties": {"city": "Chicago", "name": "North Austin", "regionid": "403323", "geo_point_2d": [41.9089835384844, -87.76108492306871], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.70940498399992, 41.991623819000026], [-87.70933299999989, 41.991576], [-87.70933822299989, 41.99159416000003], [-87.7085315829999, 41.99107895000008], [-87.70771213499994, 41.990589144000054], [-87.70689004199993, 41.98996609700003], [-87.70601890399988, 41.98958534200004], [-87.7050404209999, 41.98899410500006], [-87.7040666469999, 41.98839439900006], [-87.70270831999991, 41.987610587000056], [-87.7019844719999, 41.98719044400006], [-87.7010323129999, 41.98648728800002], [-87.7003752459999, 41.98609355200005], [-87.6997667189999, 41.98573289800003], [-87.69917398599993, 41.98538304900005], [-87.6991646779999, 41.98508194000003], [-87.69912502999993, 41.983915767000035], [-87.69908999999988, 41.98301], [-87.69909112899988, 41.98300595300008], [-87.70028191599991, 41.982990022000074], [-87.70153491599987, 41.98299602200006], [-87.70274491599989, 41.98298702200008], [-87.7039389159999, 41.982980022000035], [-87.7049799149999, 41.98297402200005], [-87.70579591499988, 41.98297002200006], [-87.70670099999988, 41.982967], [-87.7068224309999, 41.982965544000066], [-87.70687861999991, 41.98316845200008], [-87.70733379799991, 41.98460092200008], [-87.7080433389999, 41.98683664600008], [-87.70852529099993, 41.988536867000036], [-87.70902028099992, 41.99026176100005], [-87.70940498399992, 41.991623819000026]]]}, "properties": {"city": "Chicago", "name": "Peterson Park", "regionid": "403256", "geo_point_2d": [41.98608287604842, -87.70445147408513], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.77476889008675, 41.98343053855702], [-87.77365129783664, 41.98220920030009], [-87.7714879078016, 41.97983356778273], [-87.77007072400914, 41.978273365881705], [-87.76894504933176, 41.97704373518623], [-87.76835, 41.9764], [-87.76836999999988, 41.97515], [-87.76930449190729, 41.97561882650141], [-87.77051627678748, 41.97636081265508], [-87.7716959544328, 41.97700895254652], [-87.77380926758822, 41.97818991126908], [-87.77501514252937, 41.97885331597925], [-87.77612771346875, 41.97947395595254], [-87.77796, 41.98051000000016], [-87.77795244811807, 41.98122742875843], [-87.77794334278904, 41.98209243503517], [-87.77793001933257, 41.982848902341296], [-87.77792778688422, 41.98284888919234], [-87.77789691585706, 41.983780022939605], [-87.7778819160648, 41.98468302229677], [-87.7778619160423, 41.98650702274007], [-87.77785291562729, 41.98679502252596], [-87.77728430489678, 41.986185378202045], [-87.77573857238548, 41.98449245001872], [-87.77476889008675, 41.98343053855702]]]}, "properties": {"city": "Chicago", "name": "Gladstone Park", "regionid": "403285", "geo_point_2d": [41.98058970372861, -87.77419597844545], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.66524791580851, 41.79394202246008], [-87.66460791598435, 41.79392902186081], [-87.66451191515634, 41.79393102195317], [-87.66450674120493, 41.793931134435894], [-87.66448808159755, 41.79298188948845], [-87.66444658353055, 41.791272323809295], [-87.66441230610484, 41.78956369216628], [-87.66434124844926, 41.78777619262697], [-87.66429301171625, 41.78622837355272], [-87.66425228537298, 41.784191644656254], [-87.66420850306844, 41.782484015385386], [-87.66416361216055, 41.78045074114973], [-87.66411908190837, 41.778825762382276], [-87.66405414255723, 41.77695278908369], [-87.66399819653218, 41.77491780734377], [-87.66395512252318, 41.77336845984379], [-87.66389939089666, 41.77133570682188], [-87.66383322442907, 41.769303835157096], [-87.66376952081104, 41.76751546460588], [-87.66371991070604, 41.765888749700174], [-87.66367318908544, 41.76442175442185], [-87.66363797336248, 41.76352717423637], [-87.6636171937622, 41.76271182612859], [-87.6635994974942, 41.761738425661775], [-87.6635721790468, 41.7609201574406], [-87.66354328308624, 41.759944455797466], [-87.66352053008285, 41.758968576756686], [-87.66349074457722, 41.75771502256034], [-87.66353391503998, 41.75771502256034], [-87.66471591556684, 41.757695022537725], [-87.66592391549352, 41.75767702260729], [-87.66624091571703, 41.75767702260729], [-87.66722691530282, 41.75765702258459], [-87.66777391533662, 41.757656022538484], [-87.66847691536691, 41.757656022538484], [-87.66956491515805, 41.75764602207741], [-87.6712749157421, 41.75764602207741], [-87.67211891507742, 41.757640021800874], [-87.67318791579132, 41.75763702256197], [-87.67392291549812, 41.757625022008796], [-87.6744839152779, 41.757624021962705], [-87.67528191519072, 41.757623021916515], [-87.67603491482792, 41.757596022470636], [-87.67689691589243, 41.757590022194194], [-87.67844591624946, 41.757548025653904], [-87.67849391486486, 41.75754802205669], [-87.6794479156736, 41.75753302226453], [-87.68150491606542, 41.757524022748896], [-87.68170091520784, 41.75751702242636], [-87.68175191576007, 41.75751502233409], [-87.68290791598768, 41.75750102258811], [-87.68296883886875, 41.75750101910946], [-87.68299330788908, 41.7583189313458], [-87.68303204176145, 41.75994791177102], [-87.68309677246758, 41.7623058723205], [-87.6831366759162, 41.763771673722005], [-87.68319624248585, 41.76572331499309], [-87.68324702175393, 41.76767624524749], [-87.68330007830444, 41.76946566941588], [-87.683350634679, 41.77117510231725], [-87.68339408801852, 41.77296396083643], [-87.68343702784264, 41.77450885384328], [-87.68350052777986, 41.7766251056323], [-87.6835385939874, 41.77841464927419], [-87.68355, 41.779160000000125], [-87.68357999110957, 41.7804496177139], [-87.68361761198264, 41.781671856118784], [-87.68365502712066, 41.782649994725524], [-87.6836785607612, 41.78362451463918], [-87.68370721993321, 41.784356888367945], [-87.68373882375403, 41.78541338982271], [-87.68375593910119, 41.785982475119376], [-87.68376929353069, 41.786450033196516], [-87.68376915317344, 41.786450029207536], [-87.68378865471179, 41.787125300569556], [-87.68383987539768, 41.788995389716916], [-87.6838870552221, 41.79062165666478], [-87.68395690761432, 41.79257301118249], [-87.68398999999985, 41.7938], [-87.68260345440483, 41.79374585050673], [-87.68187074839543, 41.79375000000018], [-87.6807312115169, 41.79375379164051], [-87.6795059410407, 41.79375425737002], [-87.678766550276, 41.79376872243081], [-87.67876502178585, 41.79375799419313], [-87.67838691532852, 41.79377402280979], [-87.67710791572613, 41.793807022532214], [-87.67667491555147, 41.793818022140016], [-87.67586991531593, 41.79380802257831], [-87.6754669156247, 41.79381102181738], [-87.67425791565205, 41.793832021886], [-87.67303291584095, 41.79384902177034], [-87.67226791565062, 41.793855022046976], [-87.6718239158681, 41.79387402202351], [-87.67112491602228, 41.79388602257668], [-87.67059791601099, 41.79388002230005], [-87.66936891511624, 41.79390402250711], [-87.66816791551221, 41.79390802179201], [-87.66694691498628, 41.79392102239145], [-87.66574291614334, 41.79393602218354], [-87.66524791580851, 41.79394202246008]]]}, "properties": {"city": "Chicago", "name": "West Englewood", "regionid": "269612", "geo_point_2d": [41.775720400948686, -87.67373879499955], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.62221337599992, 41.91458974200008], [-87.62200334999991, 41.914580102000045], [-87.62164625299991, 41.91459359200006], [-87.62140914099987, 41.914632023000074], [-87.62137247599989, 41.91464099800003], [-87.62127491999988, 41.91469615200002], [-87.620452, 41.914232], [-87.62118470499988, 41.913761088000044], [-87.62316788699988, 41.91248648900005], [-87.62465299999988, 41.911532], [-87.6246951919999, 41.911441588000045], [-87.62514691599993, 41.91146002200008], [-87.62540791599989, 41.91147202300003], [-87.62559891599992, 41.91148502200007], [-87.62573891399988, 41.91148602100008], [-87.62592095199992, 41.91144110100003], [-87.62597888899988, 41.91139997700008], [-87.62616291499987, 41.91133602200005], [-87.6263429069999, 41.91127397000007], [-87.62694691599994, 41.911255023000066], [-87.6276419149999, 41.911233022000026], [-87.62885091499992, 41.91120402200004], [-87.63019891499991, 41.91118502200003], [-87.63167391599987, 41.911174022000054], [-87.6324799159999, 41.911163022000025], [-87.63305091599992, 41.911166022000074], [-87.63321191599991, 41.91116502200002], [-87.6338659699999, 41.91116218800004], [-87.63394286099991, 41.911105857000045], [-87.63442291599993, 41.91110002300006], [-87.63468991599987, 41.91108802200005], [-87.63566991599991, 41.91107502200002], [-87.63665091599994, 41.911082023000056], [-87.63756591599989, 41.911045023000035], [-87.63856991499992, 41.91104302200005], [-87.63978291599993, 41.91102202300004], [-87.64040991599987, 41.91101302200007], [-87.64100191499992, 41.91101002200008], [-87.64165591599992, 41.911000023000035], [-87.64221291499989, 41.91099802200005], [-87.64274593899992, 41.91099189100002], [-87.6428668919999, 41.91104215300003], [-87.64324495099991, 41.91103381400005], [-87.64322999999987, 41.91108], [-87.64276231599987, 41.912318203000034], [-87.64154790899991, 41.91401837200004], [-87.64049542299993, 41.91563758200004], [-87.63952389799991, 41.91733775100004], [-87.6388, 41.91832], [-87.6368549879999, 41.918347961000045], [-87.63611, 41.91847], [-87.62819765299989, 41.91860078300003], [-87.62780488899992, 41.91860727500005], [-87.62738130799991, 41.918174470000054], [-87.62737188899987, 41.918135027000055], [-87.62730674399988, 41.91809828300005], [-87.62630582399991, 41.91753373400007], [-87.6259949579999, 41.91728189300004], [-87.62588020199988, 41.91716859400008], [-87.62579713999989, 41.91707033400006], [-87.62570616399992, 41.916940661000055], [-87.6256461119999, 41.916815112000045], [-87.6256090129999, 41.916718849000056], [-87.62558801399989, 41.91664378400008], [-87.62556032099988, 41.91661780300006], [-87.62540185999988, 41.916451716000026], [-87.62532805599992, 41.91638737400005], [-87.62528856299988, 41.91636224300003], [-87.62508389499992, 41.91625737200007], [-87.6249704599999, 41.916188223000056], [-87.62480145999989, 41.91606722300007], [-87.6247055929999, 41.91598800900004], [-87.6245922229999, 41.91588030800005], [-87.62444082799992, 41.915758350000026], [-87.62435613599988, 41.91568073300004], [-87.62384946099989, 41.915336258000075], [-87.62320812699991, 41.91493298300007], [-87.6227997919999, 41.91476820100006], [-87.62232451399987, 41.91460921500003], [-87.62221337599992, 41.91458974200008]]]}, "properties": {"city": "Chicago", "name": "Old Town Triangle", "regionid": "403304", "geo_point_2d": [41.91449549304275, -87.63254992134222], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.72004095799988, 41.95431249300008], [-87.71893518599987, 41.95354345800006], [-87.71732717999991, 41.95372277400007], [-87.7161126149999, 41.95361564900008], [-87.7148985879999, 41.953635971000026], [-87.71336115599989, 41.95366698900005], [-87.71230940399992, 41.953691572000075], [-87.71102600199991, 41.95375944400006], [-87.70987984299987, 41.95382440400004], [-87.70874708499989, 41.95385177800005], [-87.70761164299992, 41.95387], [-87.70607332199988, 41.953856581000075], [-87.7049399819999, 41.95385504100004], [-87.70380661299991, 41.95386432600008], [-87.7027540159999, 41.95387], [-87.70113501599988, 41.95387624000005], [-87.70040594399988, 41.95387777200005], [-87.6990300449999, 41.95387385300005], [-87.6982299999999, 41.95388], [-87.69627755799989, 41.95389631600005], [-87.6946413459999, 41.95391226900005], [-87.694641, 41.953914], [-87.69441403699992, 41.95391621300006], [-87.69444156699988, 41.95387644700008], [-87.69452189299992, 41.95376934700003], [-87.69465576799988, 41.95352837000006], [-87.6948833569999, 41.953193681000066], [-87.69515110899992, 41.95284560400005], [-87.69544563499991, 41.95241720200004], [-87.6957133859999, 41.95201557500007], [-87.69596774999988, 41.95158717300006], [-87.69618195099989, 41.951158770000056], [-87.69638276499987, 41.95086424400005], [-87.69655680299988, 41.950609880000044], [-87.69663712899988, 41.95038229100003], [-87.69666390399992, 41.95016809000003], [-87.69670406699991, 41.949820013000064], [-87.69671745399994, 41.949525487000074], [-87.69666390399992, 41.94907030900002], [-87.69655680299988, 41.94853480700003], [-87.6964095399999, 41.94766461500006], [-87.69623550199991, 41.946955073000076], [-87.6961922489999, 41.94669555700004], [-87.69643399999987, 41.946692], [-87.69687891499987, 41.94668702200004], [-87.69802091599989, 41.94667502200007], [-87.69923791599987, 41.94667102200008], [-87.70047491599989, 41.946659022000055], [-87.70170291499991, 41.94664702200002], [-87.7021359159999, 41.946644023000026], [-87.70225691599987, 41.94664502300008], [-87.70291391499988, 41.94664802200002], [-87.70414191499991, 41.94665002200007], [-87.70534591499988, 41.946637022000054], [-87.70658191599992, 41.94663502200006], [-87.7078099149999, 41.946633022000064], [-87.70879391599992, 41.94661602200005], [-87.7098169149999, 41.94659802200003], [-87.7114319149999, 41.94658802300005], [-87.71263491599987, 41.94655402200003], [-87.71387191499988, 41.94654102200007], [-87.71505491499993, 41.946532022000035], [-87.71631991599989, 41.94644302200003], [-87.7173999159999, 41.94640802200006], [-87.71742991599989, 41.94640702200006], [-87.71752691599993, 41.94640502200007], [-87.71763991499994, 41.94640502200007], [-87.71784091599993, 41.946404022000074], [-87.71801691599987, 41.94640302200008], [-87.71834891599988, 41.94640202200008], [-87.71839757299993, 41.946401790000024], [-87.71869341499989, 41.94683172200006], [-87.71891317299989, 41.94708317300006], [-87.71980970399993, 41.94774573100005], [-87.72295847799987, 41.94931813700002], [-87.72484068699991, 41.95039471100005], [-87.7274195169999, 41.951818299000074], [-87.72745002899988, 41.95304124200003], [-87.72747738799987, 41.953850053000046], [-87.7274957729999, 41.954497215000046], [-87.72753418299992, 41.95587476000008], [-87.7275517929999, 41.956526344000046], [-87.7275765559999, 41.957496642000066], [-87.72760159199991, 41.95862953500004], [-87.7276272809999, 41.95950887700007], [-87.7268412199999, 41.958971], [-87.72560558299993, 41.95811225900007], [-87.72473581399991, 41.957514558000064], [-87.7228683829999, 41.95622689300005], [-87.72220027799992, 41.955770849000075], [-87.72004095799988, 41.95431249300008]]]}, "properties": {"city": "Chicago", "name": "Irving Park", "regionid": "269587", "geo_point_2d": [41.951006966487, -87.71170256983801], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.61746771075008, 41.83919975509129], [-87.61745, 41.83836], [-87.61874321243344, 41.83839592386429], [-87.61999743296924, 41.83838591184135], [-87.62111322508156, 41.83836849464095], [-87.62243594232723, 41.83834290984001], [-87.62344261535742, 41.83832918952925], [-87.6234564678801, 41.83933577944751], [-87.62351617071745, 41.841495975112004], [-87.62354284966041, 41.84275459230175], [-87.62360495662381, 41.84449360576459], [-87.6236222819004, 41.84505048965754], [-87.62364196092244, 41.845623022473454], [-87.62353091554803, 41.845623022473454], [-87.62337791568984, 41.84562602261164], [-87.62212991571785, 41.84564502258816], [-87.620481916193, 41.845678022310665], [-87.6196289155435, 41.8456840225874], [-87.61886291530702, 41.84570002242556], [-87.61795191558114, 41.8456860217802], [-87.61762444079102, 41.845681162883366], [-87.6176144022715, 41.84505047516123], [-87.6176185534397, 41.84421480036697], [-87.61758996698046, 41.84351919652587], [-87.61756995645904, 41.84303227383703], [-87.61754708729158, 41.84247579076408], [-87.61751530945922, 41.841428306747396], [-87.6175, 41.84073053930872], [-87.6174863199581, 41.839896958827374], [-87.61746771075008, 41.83919975509129]]]}, "properties": {"city": "Chicago", "name": "South Commons", "regionid": "403345", "geo_point_2d": [41.84201866467182, -87.6205347383223], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.66761677970446, 41.92515319179259], [-87.668139752184, 41.925145246075445], [-87.66816378554864, 41.92568112123356], [-87.66819318196114, 41.92664977923359], [-87.6682300819453, 41.92778270419544], [-87.66824969071696, 41.9284297936579], [-87.66825803013093, 41.92948210544138], [-87.66829519139077, 41.93037111712024], [-87.66835614194008, 41.931829241800166], [-87.66837760209036, 41.9323951922203], [-87.66825791606368, 41.932399022366], [-87.66789991574893, 41.932406021789355], [-87.6671689162266, 41.93242002243463], [-87.66596791572312, 41.93243602227297], [-87.66474691519728, 41.93245802238768], [-87.66355176833615, 41.93248697704413], [-87.66351548215064, 41.93158267367399], [-87.66349087762137, 41.93093592604796], [-87.66346021217323, 41.93013056954904], [-87.66343632344092, 41.92932058151169], [-87.66341099953928, 41.92826998618087], [-87.66337885958885, 41.92745905564774], [-87.66334096478181, 41.92616389757441], [-87.66330458663695, 41.92521016348663], [-87.66397406698013, 41.92520476746406], [-87.6651074409714, 41.925195483689336], [-87.66640235235673, 41.92517407390854], [-87.66761677970446, 41.92515319179259]]]}, "properties": {"city": "Chicago", "name": "West De Paul", "regionid": "403309", "geo_point_2d": [41.92880862503354, -87.66583881067834], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.58581211015132, 41.75146889377227], [-87.58579178409602, 41.75037535278864], [-87.58580631599685, 41.74989733567315], [-87.58585406533209, 41.74832664276034], [-87.58577852588388, 41.74646975350288], [-87.58571730314651, 41.74479460629348], [-87.5857399999996, 41.74416000000028], [-87.58673677555629, 41.744136454120905], [-87.58757312143203, 41.74413000000011], [-87.58878069067592, 41.74412394377597], [-87.58989571685612, 41.74412000000017], [-87.5908252716838, 41.7441165703898], [-87.5924977831812, 41.744087439496184], [-87.59351923335026, 41.744063566935196], [-87.59463348911702, 41.74403783540415], [-87.59621418210082, 41.743988162602285], [-87.5978139551132, 41.74397298066871], [-87.59781452349836, 41.74395588929897], [-87.59870303064685, 41.74394078595692], [-87.59853577183337, 41.7451673505883], [-87.59816415656537, 41.746653811659506], [-87.59714308862782, 41.750497832130854], [-87.59728209591064, 41.75059695075155], [-87.59724091595477, 41.75086402241464], [-87.5972089495534, 41.75107204189522], [-87.59715497404366, 41.75127312130772], [-87.59694691589225, 41.75129802263565], [-87.59683891541043, 41.751323021989165], [-87.59657491583145, 41.751367022218766], [-87.59644391608803, 41.75137702178048], [-87.59538168438127, 41.75139053864597], [-87.59444805887476, 41.751398174922286], [-87.59370487234892, 41.75140428102138], [-87.59249641549728, 41.75141425503891], [-87.59119589239536, 41.751425072860926], [-87.58868760717554, 41.75146320654718], [-87.58720150616423, 41.75147], [-87.58581439883125, 41.75148253732185], [-87.58581211015132, 41.75146889377227]]]}, "properties": {"city": "Chicago", "name": "Avalon Park", "regionid": "269572", "geo_point_2d": [41.74765394709123, -87.59186483271884], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.74683831217865, 41.981895731004855], [-87.7465237295887, 41.98185640818119], [-87.74613050135127, 41.98185640818119], [-87.74581591876124, 41.98185640818119], [-87.74550133617142, 41.9819350538287], [-87.74522607640523, 41.98209234512345], [-87.74467555687278, 41.98213166794729], [-87.74420368298792, 41.98213166794729], [-87.74381045475049, 41.98213166794729], [-87.74333858086561, 41.98185640818119], [-87.74270941568568, 41.98162047123868], [-87.74247347874325, 41.98150250276742], [-87.74180499073972, 41.981030628882564], [-87.74129379403094, 41.98075536911639], [-87.74078259732232, 41.98032281805519], [-87.74074327449866, 41.98004755828892], [-87.74074327449866, 41.979693652875326], [-87.74086124296991, 41.97918245616671], [-87.74097921144109, 41.97882855075285], [-87.74097921144109, 41.978356676868], [-87.74086124296991, 41.97816006274941], [-87.74042869190872, 41.97788480298323], [-87.74007478649503, 41.977688188864384], [-87.73964223543385, 41.977570220393204], [-87.73885577895898, 41.97741292909838], [-87.7385805191928, 41.97741292909838], [-87.73834458225036, 41.97729496062712], [-87.73810006233492, 41.977252165190954], [-87.73696625186835, 41.9768914073152], [-87.73552322036544, 41.97653064943945], [-87.7348841737611, 41.976351212857224], [-87.7340977172862, 41.97623324438589], [-87.73362584340128, 41.97607595309088], [-87.73350787493003, 41.97584001614863], [-87.73311464669258, 41.97580069332487], [-87.73224954457038, 41.97564340202986], [-87.73173834786168, 41.97564340202986], [-87.73150241091925, 41.97564340202986], [-87.73118782832931, 41.9755254335586], [-87.73089987863581, 41.97537388108848], [-87.72954494699611, 41.97538], [-87.72813906878866, 41.975382148672075], [-87.72811335661761, 41.97452398529456], [-87.72806004245376, 41.97287118870713], [-87.72803622739201, 41.9720987339531], [-87.72801393973151, 41.971171830335166], [-87.72799959524501, 41.97065542882037], [-87.72795684146752, 41.96911050319765], [-87.72791, 41.96813], [-87.72851421020829, 41.96813], [-87.72975176825243, 41.96812005254886], [-87.73098917159257, 41.968099849646045], [-87.73232873923357, 41.96807845719947], [-87.7336166590202, 41.96806387984416], [-87.73490509988495, 41.968046527274645], [-87.73598711125169, 41.968034546736035], [-87.73706932554711, 41.96803], [-87.73774999999989, 41.96803], [-87.7377555397285, 41.968025157386904], [-87.73906891566975, 41.96801102179536], [-87.73991891528176, 41.96799702204921], [-87.74083691533018, 41.96798702248767], [-87.74126691536674, 41.96798202225731], [-87.74265791563504, 41.96796902255724], [-87.74373091563413, 41.96796502237297], [-87.74393191590627, 41.96796502237297], [-87.74515291553294, 41.967966022419056], [-87.74560291559193, 41.96795902209643], [-87.74637191596669, 41.967945022350264], [-87.74658591503953, 41.967941022165995], [-87.74690091517067, 41.96794202221208], [-87.74711791528108, 41.967945022350264], [-87.7474219158047, 41.96794802248862], [-87.7475509154559, 41.96795002168144], [-87.74752991538712, 41.9682790224581], [-87.74750286288129, 41.96854102194483], [-87.74752091677097, 41.96864001931397], [-87.74758284588462, 41.96878007163287], [-87.74766398541588, 41.96885597621078], [-87.74770591360765, 41.968976020413265], [-87.74773191570547, 41.96924302192902], [-87.74773091565937, 41.969749022770955], [-87.74776491542798, 41.97018302299178], [-87.74778091526623, 41.9707090220576], [-87.7478029153811, 41.97162702210611], [-87.74782791563419, 41.97252702222428], [-87.74783291586455, 41.9726320225678], [-87.74783891524181, 41.972781022241556], [-87.74784991574897, 41.97304802195882], [-87.74793100491831, 41.9752480037616], [-87.7479619155156, 41.97674702249848], [-87.74798491567657, 41.977275022555574], [-87.74800891588363, 41.97783702238162], [-87.74804691583658, 41.97890602309544], [-87.7480749162281, 41.9799420222881], [-87.74808491578983, 41.98003102189401], [-87.7481359154425, 41.98188602196765], [-87.7481449158575, 41.98216202209991], [-87.74815391537292, 41.98242602257809], [-87.74815689048155, 41.98254767189803], [-87.74700103489367, 41.98256587434817], [-87.74699128543847, 41.98256581185611], [-87.74691695782614, 41.98236760488972], [-87.7468776350023, 41.98213166794729], [-87.74683831217865, 41.981895731004855]]]}, "properties": {"city": "Chicago", "name": "North Mayfair", "regionid": "403271", "geo_point_2d": [41.97355804925192, -87.73943836881178], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.66030799034026, 41.99133971020868], [-87.66028970223944, 41.99079996701649], [-87.66029, 41.9908], [-87.6609161690148, 41.99078998129571], [-87.66208427276328, 41.990770696192065], [-87.66380180155171, 41.990751013979036], [-87.66613924054047, 41.99072718922389], [-87.66888787670261, 41.99069260423542], [-87.6700700828403, 41.990672568051515], [-87.67009999999989, 41.9916], [-87.67016, 41.99276], [-87.67019, 41.99431], [-87.67008999999987, 41.99431], [-87.66833716027219, 41.99433260885892], [-87.66620689607161, 41.9943521048314], [-87.66462624151886, 41.99436], [-87.66352677941642, 41.99436920712858], [-87.66215037517927, 41.994392026419916], [-87.66091231956648, 41.99442990721727], [-87.66040894220696, 41.994450486437735], [-87.66039849573791, 41.99408569836705], [-87.6603674873538, 41.993124620617564], [-87.66033463641351, 41.99209518316567], [-87.66030799034026, 41.99133971020868]]]}, "properties": {"city": "Chicago", "name": "Edgewater Glen", "regionid": "403258", "geo_point_2d": [41.99254952608784, -87.66524536642738], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.68713, 41.910320000000134], [-87.68715983146814, 41.90939483169053], [-87.68711992510991, 41.90786879001315], [-87.68707732441734, 41.90597595326092], [-87.68702474672591, 41.904204278069656], [-87.68704999999987, 41.90302], [-87.68698999999977, 41.90255763899052], [-87.68697703218976, 41.9017052882972], [-87.6869508530414, 41.900788386873614], [-87.6869320518034, 41.8999343829623], [-87.68690422720033, 41.89889630081264], [-87.68688, 41.89810255276597], [-87.68686438957194, 41.897310335953634], [-87.68683908341637, 41.89633522477304], [-87.68681999999984, 41.89575000000021], [-87.68832051048211, 41.89572615815882], [-87.68984550412627, 41.89570548370645], [-87.69106599502216, 41.89569556101623], [-87.69289673136629, 41.895680676980895], [-87.69466644316572, 41.89566628908007], [-87.69576491790683, 41.89566000000023], [-87.69669086627175, 41.895658607090446], [-87.69793091601863, 41.89564602250107], [-87.69914791546084, 41.89564102227062], [-87.69982191595304, 41.89560202227165], [-87.7017409153796, 41.89560202227165], [-87.7019629157204, 41.89559502194884], [-87.70215391553164, 41.895597022041024], [-87.70219407601014, 41.89559892033037], [-87.70595632014168, 41.89555482666566], [-87.70821414392854, 41.89553575566434], [-87.7106568594916, 41.89552193653717], [-87.71187586719661, 41.89551573444056], [-87.71401140886027, 41.89547505887133], [-87.71565795380415, 41.8954428042409], [-87.71706285729536, 41.895415737896], [-87.71883218398835, 41.89538613026694], [-87.72066358054728, 41.895355958040085], [-87.72322606862991, 41.895313866634346], [-87.72420423544963, 41.89531000000022], [-87.72511878773268, 41.89530280675867], [-87.72589999999984, 41.89529000000017], [-87.72598877927186, 41.89816483306137], [-87.72603686780377, 41.899933361621244], [-87.72608471948887, 41.90170158466976], [-87.72610999999989, 41.90257000000016], [-87.7261481691646, 41.90383944324615], [-87.72617000909385, 41.904815413773655], [-87.72621860705027, 41.90652447978326], [-87.72625224890764, 41.90738234714778], [-87.7262757853258, 41.9083562948214], [-87.72640985081988, 41.90977841754782], [-87.72548614646057, 41.909788232484594], [-87.72475395802623, 41.90980322149463], [-87.72328883683042, 41.90982148256622], [-87.7219463718807, 41.909837484757084], [-87.72096992792427, 41.909844274120466], [-87.7198108423252, 41.90985559173238], [-87.71840704368913, 41.90987445150337], [-87.71718665195107, 41.909894559887306], [-87.71566128566161, 41.90991096779594], [-87.71425657716745, 41.90992], [-87.71285305659009, 41.909949275112716], [-87.71084273614069, 41.91002329925296], [-87.70894719499523, 41.9100748816022], [-87.70778793304665, 41.91008957677208], [-87.70688102461217, 41.910106175544925], [-87.70665691536111, 41.91010802185291], [-87.70639291668157, 41.91011202203728], [-87.7056399152456, 41.910118022313824], [-87.70556991561622, 41.910119022359915], [-87.70449491552515, 41.91013602224426], [-87.70335591608107, 41.910148021898145], [-87.70219291553079, 41.910159022405324], [-87.70205191532648, 41.910160022451585], [-87.70184078149366, 41.91016189304114], [-87.70184691576924, 41.91023402316453], [-87.70083098391318, 41.91017029255529], [-87.6997322972407, 41.91018413896269], [-87.69851185194796, 41.91019516803759], [-87.69759498846672, 41.91020790023084], [-87.69643604980328, 41.910224431601776], [-87.69521566070107, 41.91023000000014], [-87.69393395861795, 41.910234803618096], [-87.69265244457122, 41.910245307831595], [-87.69118785827101, 41.91025737307231], [-87.69039473614436, 41.910267727867726], [-87.68917412857353, 41.910283787572766], [-87.68801497992311, 41.910295643144046], [-87.68718999999982, 41.91032000000031], [-87.68719085081719, 41.91038240014563], [-87.68713, 41.910320000000134]]]}, "properties": {"city": "Chicago", "name": "Humboldt Park", "regionid": "269585", "geo_point_2d": [41.902805371025906, -87.70653334566144], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.62500196944762, 41.75260545005808], [-87.62491288815391, 41.750898021355475], [-87.62491291630089, 41.75089802218335], [-87.62492153003943, 41.75089827554095], [-87.62486892562349, 41.748979049659255], [-87.62476920201837, 41.7452617978116], [-87.62469, 41.741822813177635], [-87.62460828958763, 41.7389466046597], [-87.62472986887889, 41.73620557012454], [-87.62479919892121, 41.73038638166091], [-87.6247838533302, 41.730119359946485], [-87.62584949401975, 41.730116932240804], [-87.62666848657548, 41.73011693224098], [-87.62661040783047, 41.730051401049394], [-87.62817208944355, 41.72993112460672], [-87.63003016578284, 41.72983822078973], [-87.63281728029172, 41.72983822078973], [-87.63373970264257, 41.72996999541126], [-87.63374091494927, 41.72998102205413], [-87.63384091596163, 41.731177022327095], [-87.63387591577641, 41.73164802245488], [-87.63390791545282, 41.7326640225243], [-87.63396691547467, 41.73436202255518], [-87.63400691641901, 41.73561102167398], [-87.63404791561103, 41.735892022036424], [-87.6340550904022, 41.73593407073739], [-87.63401575585539, 41.7359789496045], [-87.63399691595811, 41.73617802173021], [-87.63399729032345, 41.73628518137306], [-87.6358831062513, 41.73884989103495], [-87.63690504823788, 41.73987183302145], [-87.63727666350587, 41.74024344828935], [-87.6373695673227, 41.74126539027585], [-87.63770056595307, 41.741959704621785], [-87.6370452635601, 41.742710818266666], [-87.63628, 41.74356000000012], [-87.63628160024403, 41.74356310980809], [-87.63596891602907, 41.74397502231593], [-87.635878966739, 41.74409720780405], [-87.63581376327204, 41.744071903404404], [-87.63582, 41.74420000000023], [-87.63568776285483, 41.744680067152395], [-87.63547055451072, 41.745621988455305], [-87.63524411548842, 41.74653939543973], [-87.63499352641742, 41.747475422783545], [-87.63477350720078, 41.74828509350086], [-87.63454642870302, 41.74915348159963], [-87.6342840701897, 41.750179004730036], [-87.63415417783254, 41.75067596271511], [-87.63403896499372, 41.7511167376911], [-87.63379807890986, 41.75205535541065], [-87.63352691252531, 41.753054333102945], [-87.63329895826797, 41.75381312519592], [-87.63294313366661, 41.75456689121646], [-87.63270378080998, 41.755052065925405], [-87.63235456641587, 41.75580813756634], [-87.63203740147569, 41.756483794148394], [-87.63171055853729, 41.75715589371222], [-87.63129289801991, 41.75802230481181], [-87.63030795853571, 41.75804836736543], [-87.62891535306994, 41.75808261912997], [-87.62715014745713, 41.7580900000001], [-87.62531520821562, 41.75809221714284], [-87.62529443761392, 41.75715441667337], [-87.62500196944762, 41.75260545005808]]]}, "properties": {"city": "Chicago", "name": "West Chatham", "regionid": "403357", "geo_point_2d": [41.743413940790234, -87.62982428832278], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.69632999999986, 41.88289000000021], [-87.69631036400969, 41.882540479376395], [-87.69627326333313, 41.88168674499489], [-87.69625063189356, 41.88064843521966], [-87.69622915471591, 41.87967111693683], [-87.6962104861161, 41.87881711829028], [-87.69618019733093, 41.87796374928975], [-87.69614756748419, 41.87698621317848], [-87.69612933507243, 41.87631539768449], [-87.69611780731218, 41.87565039408297], [-87.69610999999976, 41.87520000000023], [-87.69680179609351, 41.87518625585272], [-87.69741305309232, 41.87517411176326], [-87.6982060628739, 41.87515587800315], [-87.69893692987628, 41.875125954043305], [-87.69966303517741, 41.875072632242116], [-87.70033984815674, 41.87500988180687], [-87.70137817219738, 41.87490086699884], [-87.701874464175, 41.87483323579154], [-87.70253034301719, 41.874732617915015], [-87.70314489076047, 41.874638340829506], [-87.70380922587182, 41.87455457115084], [-87.70436942310737, 41.87448150484975], [-87.70486379598671, 41.874395722879065], [-87.7054078490184, 41.874303869821816], [-87.70591043433512, 41.87423123632562], [-87.70593623786014, 41.87534070371007], [-87.70595442275152, 41.87576960952796], [-87.70597724780286, 41.876499544808176], [-87.70598898890944, 41.8769280951989], [-87.70602176805544, 41.87778277555703], [-87.7060393318564, 41.878148102614], [-87.70605540483332, 41.87906181866562], [-87.70605969564326, 41.87961104234939], [-87.70607730978361, 41.88022122286156], [-87.70608470040035, 41.88046511321711], [-87.70609999999981, 41.880970000000154], [-87.70699419019938, 41.88095431245272], [-87.7078485764336, 41.880950000000105], [-87.70937302946557, 41.88093571152872], [-87.7104131783924, 41.88091906914578], [-87.71169545002934, 41.880878393773905], [-87.71279182477633, 41.880829959548215], [-87.71474416359904, 41.88078308632398], [-87.71573, 41.88077000000013], [-87.71570202234663, 41.87985373185344], [-87.71566668469092, 41.87869722542149], [-87.71564344807172, 41.87784257570178], [-87.71560267773634, 41.876439621148364], [-87.71554436396262, 41.874667473795235], [-87.7155164016568, 41.87380989105637], [-87.71614803604656, 41.87380019618284], [-87.71779626110461, 41.873758113889096], [-87.71907783616003, 41.87375244320295], [-87.71962664099077, 41.87374834663401], [-87.72024380905782, 41.87374511745104], [-87.72078636607517, 41.87373890613567], [-87.72285713223148, 41.873935371148285], [-87.72377780817152, 41.8740218051414], [-87.7253, 41.874000000000244], [-87.72531350525479, 41.87454536784837], [-87.72534066953335, 41.875520755536144], [-87.72537052297832, 41.87643919701137], [-87.72539596506861, 41.87729246302442], [-87.72542810915752, 41.8784532747302], [-87.72545137246117, 41.879065880375876], [-87.72549029554175, 41.88003602549566], [-87.72551896835776, 41.881198101474574], [-87.72555179849157, 41.882601148720966], [-87.72558147708708, 41.883396010663446], [-87.72559358371761, 41.88406826268799], [-87.72561180420496, 41.88473809855902], [-87.72564779594254, 41.886019511038974], [-87.72568283833188, 41.887177988293715], [-87.72570853476532, 41.88789202772699], [-87.72573, 41.888582094532815], [-87.72573, 41.889314437473956], [-87.72575829344747, 41.89035099906109], [-87.72578016088094, 41.89102160035523], [-87.72580400664451, 41.89187841196765], [-87.72581824440607, 41.89248908778121], [-87.72584323944731, 41.893220563699984], [-87.72588390231185, 41.894195357220376], [-87.72590501247194, 41.89528991782857], [-87.72589999999984, 41.89529000000017], [-87.72511878773268, 41.89530280675867], [-87.72420423544963, 41.89531000000022], [-87.72322606862991, 41.895313866634346], [-87.72066358054728, 41.895355958040085], [-87.71883218398835, 41.89538613026694], [-87.71706285729536, 41.895415737896], [-87.71565795380415, 41.8954428042409], [-87.71401140886027, 41.89547505887133], [-87.71187586719661, 41.89551573444056], [-87.7106568594916, 41.89552193653717], [-87.70821414392854, 41.89553575566434], [-87.70595632014168, 41.89555482666566], [-87.70219407601014, 41.89559892033037], [-87.70215391553164, 41.895597022041024], [-87.7019629157204, 41.89559502194884], [-87.7017409153796, 41.89560202227165], [-87.69982191595304, 41.89560202227165], [-87.69914791546084, 41.89564102227062], [-87.69793091601863, 41.89564602250107], [-87.69669086627175, 41.895658607090446], [-87.6966529155632, 41.89472502231438], [-87.69664904554195, 41.89433070690561], [-87.69668034658325, 41.893708495119874], [-87.6967413751617, 41.89059603762067], [-87.69664650871049, 41.88817694311818], [-87.69659081449022, 41.88817762205029], [-87.69643324837611, 41.88718514244104], [-87.69639754899463, 41.88589877323486], [-87.69637239434931, 41.884799905429226], [-87.69635199056506, 41.88400571223765], [-87.696335097723, 41.88327232924226], [-87.69632999999986, 41.88289000000021]]]}, "properties": {"city": "Chicago", "name": "East Garfield Park", "regionid": "269577", "geo_point_2d": [41.8857322151232, -87.71118026470171], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.6529053759359, 41.89236385309441], [-87.65110313107562, 41.89114221389928], [-87.64871473720652, 41.88956682712837], [-87.64822433620468, 41.889040139771716], [-87.64824091507663, 41.88904002197901], [-87.64826635216906, 41.88903978286307], [-87.64799133322634, 41.888754068077205], [-87.64772421817929, 41.88847059903749], [-87.64727978375332, 41.88788543596892], [-87.64711547589773, 41.88767418308297], [-87.64672016484272, 41.88714107004251], [-87.64629410575338, 41.88637478512166], [-87.64610694281775, 41.8857091590355], [-87.64599041092117, 41.88570457097561], [-87.6460199999999, 41.88568000000013], [-87.6457159890394, 41.88429146778737], [-87.64576999999983, 41.88351672276226], [-87.64578999999989, 41.882052036821186], [-87.64584382070014, 41.881803165943154], [-87.64681934635684, 41.88178446260134], [-87.64761264845933, 41.881765378839745], [-87.64822274205045, 41.881754952723426], [-87.64851, 41.88175000000022], [-87.64855, 41.88312000000024], [-87.64919891641499, 41.88310350212517], [-87.65017601469087, 41.883086155045845], [-87.65127377821042, 41.883073495284776], [-87.65249395278009, 41.88305262690842], [-87.65365520278955, 41.88302864688388], [-87.65487536623229, 41.883002225134994], [-87.65609595083306, 41.882985081524204], [-87.65725532810208, 41.8829739252236], [-87.65804809331574, 41.88296277710971], [-87.65920820391443, 41.88294036914498], [-87.66012422811103, 41.882930000000144], [-87.66110094149506, 41.88292103611434], [-87.66287141699142, 41.882887842191884], [-87.66415250467793, 41.882856339118845], [-87.66513796688213, 41.88298621486295], [-87.66548515653079, 41.883068437567516], [-87.66594317556637, 41.88322197685067], [-87.66610555336682, 41.883270116498494], [-87.66726504196448, 41.8832921989929], [-87.66903440055988, 41.883270000000216], [-87.66976616770016, 41.88324842457876], [-87.67104834364771, 41.88323], [-87.6714145151182, 41.88323], [-87.67202382539759, 41.883225003577984], [-87.67294011989466, 41.88320622705152], [-87.67367215544373, 41.883191226323035], [-87.67446551865014, 41.883176981328155], [-87.67550359790907, 41.88316421805861], [-87.67623583015937, 41.883155215203075], [-87.67702906651719, 41.88314490942747], [-87.67770025318674, 41.88313565168036], [-87.6783102475813, 41.88312595459457], [-87.67959254795356, 41.883101925443455], [-87.68038570631576, 41.88308892284747], [-87.68093481595102, 41.883079921050125], [-87.68197202529142, 41.88306294652594], [-87.68307024696581, 41.88304501637613], [-87.68441274594073, 41.883024437553566], [-87.68550966498691, 41.88300493724695], [-87.68715892510043, 41.882993679144775], [-87.68776914924739, 41.88298754623886], [-87.68929591884587, 41.882969756847025], [-87.6910662447545, 41.8829463862081], [-87.69220989600987, 41.88293335347196], [-87.69332387340056, 41.88292359842878], [-87.69570378164055, 41.88289789350882], [-87.69632999999986, 41.88289000000021], [-87.696335097723, 41.88327232924226], [-87.69635199056506, 41.88400571223765], [-87.69637239434931, 41.884799905429226], [-87.69639754899463, 41.88589877323486], [-87.69643324837611, 41.88718514244104], [-87.69659081449022, 41.88817762205029], [-87.69664650871049, 41.88817694311818], [-87.6967413751617, 41.89059603762067], [-87.69668034658325, 41.893708495119874], [-87.69664904554195, 41.89433070690561], [-87.6966529155632, 41.89472502231438], [-87.69669086627175, 41.895658607090446], [-87.69576491790683, 41.89566000000023], [-87.69466644316572, 41.89566628908007], [-87.69289673136629, 41.895680676980895], [-87.69106599502216, 41.89569556101623], [-87.68984550412627, 41.89570548370645], [-87.68832051048211, 41.89572615815882], [-87.68681999999984, 41.89575000000021], [-87.68569472978754, 41.89575683911634], [-87.6833142214474, 41.89580186541795], [-87.68081287243574, 41.8958391567835], [-87.67861554001848, 41.895884335409505], [-87.67705, 41.89591], [-87.67654112303018, 41.89591827442235], [-87.67550342987236, 41.89593000000019], [-87.67385631751368, 41.89595914363703], [-87.67172026902936, 41.89599099327426], [-87.67043795003278, 41.89601421008052], [-87.66903446254513, 41.89603133119784], [-87.66805830020813, 41.89604792455777], [-87.66736999999982, 41.89606], [-87.66628880977528, 41.89608181013123], [-87.66549561470343, 41.89609208628804], [-87.66452008761534, 41.89610866471983], [-87.66384697139453, 41.8961200000001], [-87.66299433287892, 41.89612975288145], [-87.66183547578385, 41.89615117869741], [-87.66110068536574, 41.896160000000215], [-87.66030835607839, 41.89617465183], [-87.6592707510221, 41.8961939426871], [-87.65842038992388, 41.89620617466253], [-87.65726437007459, 41.89538530822904], [-87.65638822116162, 41.89475030703519], [-87.65378080816537, 41.89295725918152], [-87.6529053759359, 41.89236385309441]]]}, "properties": {"city": "Chicago", "name": "West Town", "regionid": "269615", "geo_point_2d": [41.88914493953482, -87.6728622539351], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.831761, 41.95393], [-87.831761, 41.95223], [-87.83171915599989, 41.95223], [-87.83066099999989, 41.95223], [-87.829361, 41.95213], [-87.82806676699992, 41.95213], [-87.828061, 41.95213], [-87.82800402999992, 41.95213517900003], [-87.8269609999999, 41.95223], [-87.826874, 41.952232], [-87.82276099999989, 41.95233], [-87.82206099999989, 41.95233], [-87.820761, 41.95243], [-87.81987600799991, 41.952356251000076], [-87.8195609999999, 41.95233], [-87.81928194899987, 41.95233], [-87.81923587499993, 41.95127030300005], [-87.81911443499988, 41.95017733700006], [-87.81905371399993, 41.94920581200005], [-87.81914247899991, 41.94786777100005], [-87.8209299019999, 41.947012658000055], [-87.82276800399988, 41.94613052400006], [-87.8241144369999, 41.94548868800006], [-87.82534, 41.94495], [-87.82609582799994, 41.94491952300005], [-87.8265801659999, 41.94490430300004], [-87.82725067499989, 41.944883233000034], [-87.82858720599992, 41.94484998700004], [-87.82974186199992, 41.94481594000006], [-87.8308944469999, 41.94478429300004], [-87.8322325199999, 41.94475143700003], [-87.83344651499992, 41.944715196000054], [-87.83453831399991, 41.944678701000036], [-87.83587300099991, 41.944641357000066], [-87.8364600439999, 41.944626888000066], [-87.83646691599994, 41.94480702200007], [-87.83647991599989, 41.94533102200005], [-87.8364869149999, 41.94563502300007], [-87.8365269149999, 41.94647602300005], [-87.83655491599987, 41.94718202200004], [-87.83666991499987, 41.948265023000026], [-87.8367029149999, 41.94849002200004], [-87.83674591499991, 41.94892102200003], [-87.83678091499988, 41.949540024000044], [-87.83676991599991, 41.95011702300007], [-87.83676191599993, 41.95050602200006], [-87.83677891599989, 41.95195202300004], [-87.83677391599988, 41.95379602300004], [-87.8367889159999, 41.95426302200007], [-87.83678091499988, 41.954637022000036], [-87.83673091599988, 41.95556202300008], [-87.83666891499989, 41.95662802200006], [-87.8366669139999, 41.95927103600007], [-87.83666690599993, 41.959271710000046], [-87.83666609499991, 41.95933466300004], [-87.832661, 41.95943], [-87.8326409999999, 41.959326], [-87.831961, 41.95573], [-87.831761, 41.95393]]]}, "properties": {"city": "Chicago", "name": "Irving Woods", "regionid": "403281", "geo_point_2d": [41.9502463622046, -87.82972107914391], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.7618640019999, 42.00841336200005], [-87.7618664789999, 42.00840224200004], [-87.7619619309999, 42.00782952500002], [-87.7619620279999, 42.00777225900003], [-87.7619620279999, 42.00738748500003], [-87.7619620279999, 42.007330378000056], [-87.7619620279999, 42.00727325900003], [-87.7619620279999, 42.00648748500003], [-87.7619620279999, 42.006430378000054], [-87.76206196799988, 42.005530480000054], [-87.7621619679999, 42.004630480000046], [-87.7621620279999, 42.00457325900003], [-87.7621620279999, 42.003787485000075], [-87.7621620279999, 42.00373037800004], [-87.7621620279999, 42.00367325900003], [-87.7621620279999, 42.002687485000074], [-87.7621620279999, 42.00263037800004], [-87.76223136499992, 42.00186724100006], [-87.76223757599989, 42.00178828700005], [-87.76224283299987, 42.00173046100007], [-87.7622481099999, 42.001675868000056], [-87.76225698799992, 42.00158538600004], [-87.7622615529999, 42.00153009600006], [-87.76225635199992, 42.00152746300005], [-87.76066146099991, 42.00082963100003], [-87.75976141499989, 42.00032964000006], [-87.75856132999989, 41.99972966700005], [-87.75756132999992, 41.999329667000026], [-87.7565614149999, 41.99892964000007], [-87.75556137699992, 41.99842965000004], [-87.75554707399992, 41.99842384500004], [-87.7545103389999, 41.99795289500007], [-87.75446152499987, 41.99792962600008], [-87.75391808099992, 41.997584754000044], [-87.7535000109999, 41.997318474000046], [-87.75347737499992, 41.99730414200007], [-87.75336104599988, 41.997229946000054], [-87.7525598629999, 41.997229946000054], [-87.75224689399992, 41.99723033000003], [-87.75206001299992, 41.99723047400004], [-87.75081247299993, 41.997230675000075], [-87.75076004599991, 41.997229946000054], [-87.74996022299989, 41.99722972700004], [-87.7497688839999, 41.99717754400005], [-87.74886004599993, 41.996929946000066], [-87.74827837899993, 41.996929946000066], [-87.7482773329999, 41.99687389300003], [-87.74827513699992, 41.99637251100005], [-87.74826887899988, 41.995238659000044], [-87.74826531199992, 41.99431100900005], [-87.74826194199993, 41.99343489600004], [-87.74826280699993, 41.99245559200004], [-87.7482670759999, 41.99173410100007], [-87.74827, 41.99065174300005], [-87.74827, 41.99023944900004], [-87.7482581929999, 41.98931165000004], [-87.74825170099989, 41.98905413000004], [-87.74823696199988, 41.98833277300007], [-87.74822600699989, 41.98740523900005], [-87.74821261699991, 41.986271587000026], [-87.748204, 41.984726003000034], [-87.74817527399993, 41.98361007700002], [-87.74817303699989, 41.983523159000065], [-87.74817191499992, 41.98316202200003], [-87.74815688999989, 41.98254767200005], [-87.74815391499992, 41.982426023000066], [-87.7481448979999, 41.98216147200008], [-87.74856143399988, 41.98260893700005], [-87.7489856859999, 41.98306527600004], [-87.74951722499989, 41.98364273300006], [-87.7500050719999, 41.98364273300006], [-87.75046890399992, 41.98359119600006], [-87.75088119899992, 41.98338504900004], [-87.75119041999992, 41.98312736500008], [-87.75149964099987, 41.982972754000045], [-87.75196347199991, 41.98292121800005], [-87.75230153999989, 41.98312189300008], [-87.75230100499991, 41.98313513000004], [-87.75232175599987, 41.983133893000044], [-87.75260893699992, 41.98327694500005], [-87.75281538099993, 41.98336542200007], [-87.75290385799988, 41.98342440600003], [-87.75305131799988, 41.98348339000005], [-87.7531692869999, 41.98354237400007], [-87.75325776299988, 41.98357186600003], [-87.75340522399993, 41.98360135800004], [-87.75358217599988, 41.98360135800004], [-87.75375912899992, 41.98360135800004], [-87.7538770979999, 41.98354237400007], [-87.7540540499999, 41.98351288200006], [-87.75417201899987, 41.98339491400008], [-87.75431947899993, 41.983188469000076], [-87.7544079559999, 41.982923040000045], [-87.75449643199991, 41.98271659500005], [-87.75467338499992, 41.982628119000076], [-87.75485033799991, 41.98253964200005], [-87.7550272899999, 41.982451166000075], [-87.75526322699993, 41.982362690000045], [-87.75544017999992, 41.982362690000045], [-87.75567611699992, 41.982392182000055], [-87.75582357699992, 41.982392182000055], [-87.75600052999992, 41.982392182000055], [-87.75617748299992, 41.982451166000075], [-87.75632494299991, 41.98253964200005], [-87.75632494299991, 41.98274608700006], [-87.75626595899989, 41.982952532000056], [-87.75623646699994, 41.98309999200005], [-87.75623646699994, 41.98324745300004], [-87.75623646699994, 41.98342440600003], [-87.75629545099989, 41.98360135800004], [-87.7564724039999, 41.98377831100002], [-87.7565608799999, 41.98380780300003], [-87.7567378329999, 41.98395526400008], [-87.7569442779999, 41.984102724000074], [-87.7570917379999, 41.98425018500005], [-87.75720970699989, 41.98436815400003], [-87.75732767499993, 41.98451561400008], [-87.75738665999992, 41.984604090000055], [-87.75738665999992, 41.98484002700007], [-87.75726869099991, 41.984957996000034], [-87.75703275399988, 41.984957996000034], [-87.75676732499993, 41.985016980000054], [-87.75664935699989, 41.985134949000035], [-87.75667884899988, 41.98537088600004], [-87.75682630899988, 41.985547838000045], [-87.75703275399988, 41.985606822000065], [-87.75726869099991, 41.98566580700008], [-87.75750462799992, 41.985783775000066], [-87.75762259699988, 41.98596072800006], [-87.75777005699987, 41.98610818800005], [-87.75788802599992, 41.98622615700002], [-87.75797650199989, 41.98649158600005], [-87.75812396299992, 41.98681599900003], [-87.75812396299992, 41.98705193600005], [-87.75824193099992, 41.987287873000064], [-87.75835989999989, 41.98746482600005], [-87.75853685199989, 41.98752381000007], [-87.75871380499989, 41.98752381000007], [-87.75889075799994, 41.98770076300007], [-87.7590087259999, 41.98790720800007], [-87.7591561869999, 41.98805466800006], [-87.7593626319999, 41.98823162100006], [-87.7595395839999, 41.988320097000035], [-87.7596870449999, 41.988320097000035], [-87.75977552099992, 41.98826111300007], [-87.7598934899999, 41.98817263700004], [-87.76007044199989, 41.98811365300008], [-87.76021790299987, 41.98805466800006], [-87.76036536399991, 41.98796619200003], [-87.76048185299992, 41.98786119900006], [-87.7616040319999, 41.99020759300003], [-87.7631376229999, 41.99318629700007], [-87.76443527599989, 41.99578160300006], [-87.76476416599989, 41.99640806100006], [-87.76563, 41.99729], [-87.7658224589999, 41.99761152100006], [-87.76910685499988, 42.00380383700006], [-87.76937569599991, 42.00431234200004], [-87.76936256999993, 42.00457882800002], [-87.76936123399989, 42.00462971900003], [-87.7679614609999, 42.00422963100004], [-87.7661613209999, 42.003230653000045], [-87.76607534999992, 42.00331666500005], [-87.7658002999999, 42.00360851900006], [-87.7657650279999, 42.003645946000056], [-87.76580038799992, 42.00366459500008], [-87.76706143699991, 42.004329635000026], [-87.7675813949999, 42.00457764500003], [-87.76776504599988, 42.00464750400004], [-87.76776402599988, 42.00470441000004], [-87.76756505899992, 42.00842904000005], [-87.76753499699991, 42.00843007500003], [-87.76742855899991, 42.00843113500007], [-87.7642250139999, 42.00849347000007], [-87.76255701199993, 42.008514476000066], [-87.7618569189999, 42.008526373000045], [-87.76185104399988, 42.00852641800007], [-87.76185686299992, 42.00846973600004], [-87.76186103399992, 42.008429448000065], [-87.7618640019999, 42.00841336200005]]]}, "properties": {"city": "Chicago", "name": "Edgebrook", "regionid": "51847", "geo_point_2d": [41.994430050053715, -87.75757557123396], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.61875320499995, 41.90013074100005], [-87.61835402799991, 41.900030946000065], [-87.61787902799993, 41.89933994600004], [-87.61748002799997, 41.898758946000044], [-87.61615402799995, 41.89683094600007], [-87.61551702799991, 41.89587594600003], [-87.61447502799993, 41.894311946000066], [-87.61403002799993, 41.893644946000045], [-87.61384829699993, 41.89337279300002], [-87.61380312699997, 41.893344816000074], [-87.61373400099994, 41.89333518000006], [-87.61364602099995, 41.89335501600004], [-87.61343000199996, 41.893370200000035], [-87.6133811699999, 41.89338197700003], [-87.61285743599994, 41.89335903000005], [-87.61235349199993, 41.893431696000036], [-87.61035302899994, 41.896230990000035], [-87.60974102899995, 41.896257989000034], [-87.60805304599992, 41.89633094600003], [-87.60705301399994, 41.89633128400003], [-87.60675402799994, 41.89643094600005], [-87.60655300099994, 41.896231151000045], [-87.60605232099991, 41.896331653000054], [-87.60595214199996, 41.89643079900002], [-87.60535204599994, 41.896330947000074], [-87.60255302799997, 41.896331946000025], [-87.60255112599997, 41.89627489000002], [-87.60245492999995, 41.89338899700003], [-87.60245302799994, 41.89333194600004], [-87.60694903499994, 41.89320997300006], [-87.60985303499993, 41.89313097300003], [-87.60985402799996, 41.89307425900005], [-87.60985402799996, 41.89218848500006], [-87.60985337099997, 41.89213124000001], [-87.60888940899996, 41.89213972200002], [-87.60887089499991, 41.89213989000006], [-87.59875302799996, 41.89223194600003], [-87.59866016599993, 41.891489053000036], [-87.59865302799993, 41.89143194600006], [-87.59865916899994, 41.89143194600006], [-87.59868804599994, 41.89143194600006], [-87.60095204599996, 41.89143194600006], [-87.60105265899992, 41.89123268400004], [-87.60774884299995, 41.89099562300003], [-87.60928546999995, 41.89094138900002], [-87.60938570599993, 41.89093785100004], [-87.60955203199994, 41.890931981000044], [-87.60955302799991, 41.89087525900003], [-87.60955302799991, 41.88938948500003], [-87.60955302799991, 41.889332594000045], [-87.61132698999994, 41.88933194600002], [-87.61135204599995, 41.88933194600002], [-87.61164832699994, 41.89062413900006], [-87.61165032299992, 41.89063275500006], [-87.61165402799995, 41.89063094600003], [-87.61171424599995, 41.89062863100001], [-87.61172080399994, 41.890620647000034], [-87.61172745299994, 41.890611150000026], [-87.61178502899995, 41.89060798800004], [-87.61326902399996, 41.890520006000024], [-87.61360702699992, 41.89050299700005], [-87.61387102799992, 41.89049099200001], [-87.61404602299993, 41.890480009000036], [-87.61421402199991, 41.89046901200003], [-87.61527603899995, 41.89045096300002], [-87.61615203299992, 41.890423977000054], [-87.61623304099997, 41.89042295800004], [-87.61686203799994, 41.890410965000044], [-87.61693003999993, 41.89040996100004], [-87.61716104399993, 41.89040895100003], [-87.61724204599993, 41.89040894600003], [-87.61733501699995, 41.89037729400006], [-87.61736639599995, 41.89034168000005], [-87.61737855199998, 41.89028116400003], [-87.61738291199997, 41.890258545000044], [-87.61737866599995, 41.890223444000064], [-87.61737224999999, 41.89018053600005], [-87.61425304599996, 41.890230946000024], [-87.61320250599992, 41.89023099100006], [-87.61275917599991, 41.89025076500007], [-87.61259502799997, 41.89021594600006], [-87.61257102799998, 41.890192946000056], [-87.61249302799995, 41.89007594600002], [-87.61249259799995, 41.89001925400003], [-87.61249046399993, 41.889737474000064], [-87.61249003099991, 41.88968037000006], [-87.61248977399995, 41.88962325500005], [-87.61248957999996, 41.889580054000064], [-87.61248932599993, 41.88952340400004], [-87.61265350199993, 41.88928822100006], [-87.61265402799994, 41.889230783000066], [-87.61265402799994, 41.88918848500002], [-87.61265402799994, 41.88913159400004], [-87.61298152999994, 41.88897811200007], [-87.61371424399992, 41.88863617900005], [-87.6141438499999, 41.88843569600004], [-87.61415270399992, 41.88843160000003], [-87.61715104099993, 41.88845287700007], [-87.61717980799993, 41.88868417400004], [-87.61727391599997, 41.888665024000034], [-87.61741091499992, 41.88866102200005], [-87.61755391499995, 41.88866002200005], [-87.61782091599996, 41.88866302300004], [-87.61925191499994, 41.88871302200003], [-87.62031301899997, 41.88879744800004], [-87.62044591499992, 41.88880802200002], [-87.62053391599991, 41.888816022000064], [-87.62057891499992, 41.888820024000026], [-87.62062091799997, 41.88883002100003], [-87.62073041599993, 41.888835522000065], [-87.62171391599996, 41.88894002300002], [-87.62208991599992, 41.88900302200004], [-87.62269891599998, 41.889102022000024], [-87.62296391399995, 41.889143020000056], [-87.62396691499997, 41.88909002200005], [-87.62420792799996, 41.889062888000055], [-87.62429091599994, 41.889079022000026], [-87.62440921999993, 41.88909874000005], [-87.62433402299993, 41.889394702000054], [-87.62423794599992, 41.889894381000026], [-87.62409541599997, 41.890540381000044], [-87.62414301599993, 41.89134937400006], [-87.62420125299995, 41.89174844700004], [-87.62426209899995, 41.89216539600004], [-87.62427421199996, 41.89296726000003], [-87.62428988699996, 41.89362216400007], [-87.62433162899997, 41.89442786800003], [-87.62435, 41.89523554600002], [-87.62435719899996, 41.89604395400004], [-87.62431, 41.89671], [-87.62519406599993, 41.896704872000036], [-87.62657021899996, 41.89669213600006], [-87.62770318599995, 41.89667104000005], [-87.6282, 41.89666], [-87.62820828499994, 41.89701623800004], [-87.62835142399994, 41.89799923100002], [-87.62824, 41.89839300400006], [-87.62824622299995, 41.89872047100003], [-87.62826916899994, 41.899367573000056], [-87.62829989899994, 41.900256654000025], [-87.62831571699995, 41.900702096000025], [-87.62411739399992, 41.90080912100006], [-87.61941723699992, 41.90100622500005], [-87.61937966099995, 41.90100683200005], [-87.61875320499995, 41.90013074100005]]]}, "properties": {"city": "Chicago", "name": "Streeterville", "regionid": "403299", "geo_point_2d": [41.894285565503225, -87.61664607025648], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.6980685709999, 41.96565442100006], [-87.69797588599992, 41.965563796000076], [-87.69745377099989, 41.96506845600003], [-87.69679777999991, 41.96451956600002], [-87.69620872699991, 41.96383680000007], [-87.69567322399989, 41.96323435900006], [-87.6955527359999, 41.963020158000056], [-87.69539208499991, 41.96263191800005], [-87.69523143399992, 41.962230291000076], [-87.69504400799988, 41.96192237700006], [-87.69481641899993, 41.96158768800005], [-87.69470931899988, 41.96127977400003], [-87.69470771099991, 41.96126155000007], [-87.694886, 41.96126], [-87.69725391499989, 41.961249022000054], [-87.69851987299987, 41.96124103500005], [-87.69852191499989, 41.961241022000074], [-87.6997049149999, 41.96124202200007], [-87.70092891599988, 41.961231022000035], [-87.70215591599991, 41.96121602200008], [-87.7033779159999, 41.961212022000026], [-87.7033800659999, 41.96121201700003], [-87.7034203769999, 41.962434780000066], [-87.7034676889999, 41.96397141600004], [-87.70350133999993, 41.96510374600006], [-87.70354128699994, 41.96615473600007], [-87.7035816939999, 41.96745399300004], [-87.70367, 41.96849], [-87.70315881799989, 41.96849], [-87.70202475899991, 41.96849446500004], [-87.70120001799994, 41.96850196500007], [-87.70105250499994, 41.96850330600005], [-87.7008274379999, 41.96830824800008], [-87.7003990359999, 41.96789323400007], [-87.69993047099987, 41.967478219000036], [-87.6996091699999, 41.96719708000005], [-87.69918076699989, 41.96678206500007], [-87.69857832699994, 41.966152849000075], [-87.6980685709999, 41.96565442100006]]]}, "properties": {"city": "Chicago", "name": "Ravenswood Manor", "regionid": "403290", "geo_point_2d": [41.964233774504365, -87.70016622558667], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.65442602799996, 41.997264946000016], [-87.65442602799996, 41.99720825900004], [-87.65442602799996, 41.99699048500001], [-87.65442602799996, 41.99693359400003], [-87.65441502799993, 41.99683594600002], [-87.65439002799995, 41.996625946000044], [-87.65430302799994, 41.996193946000055], [-87.65430397999992, 41.99613729100001], [-87.65430408599995, 41.996130963000034], [-87.65430501599994, 41.99607561800007], [-87.65428602799992, 41.99601094600006], [-87.65427728899992, 41.99596724900003], [-87.65426802799996, 41.99592094600006], [-87.65427665899992, 41.995910876000025], [-87.65428539899993, 41.99590068000002], [-87.65430912799995, 41.995885509000054], [-87.65441206899997, 41.99588990000006], [-87.65445509399994, 41.99589385900003], [-87.65448541299996, 41.995858684000034], [-87.65452256399996, 41.99579869800003], [-87.65458165899996, 41.99568068400002], [-87.65458202799992, 41.99562325900006], [-87.65458202799992, 41.99556448500004], [-87.65458197099996, 41.995507395000054], [-87.65452702799995, 41.99538594600004], [-87.65450409999998, 41.99525574800003], [-87.65449902799995, 41.99522694600005], [-87.65450399599993, 41.99519922100006], [-87.65452992199992, 41.99505453500006], [-87.65458633399997, 41.99493847000007], [-87.65458866899993, 41.99493368200005], [-87.65458702799995, 41.99488194600007], [-87.65456302799996, 41.99448094600006], [-87.65443502799995, 41.99393794600006], [-87.65442602799996, 41.99382794600006], [-87.65440702799995, 41.99359194600003], [-87.65431702799995, 41.99322494600004], [-87.65408202799993, 41.99294794600006], [-87.65407751899993, 41.99286356400005], [-87.65407502799994, 41.992816946000055], [-87.65407737099994, 41.99280707800005], [-87.65415087799991, 41.992497575000066], [-87.65425284199992, 41.99213760300005], [-87.65429984499994, 41.991969601000044], [-87.65434196799995, 41.99159548100004], [-87.65434327099995, 41.991538308000024], [-87.65434776499994, 41.99133347900003], [-87.65434791099993, 41.991326862000044], [-87.65434901299993, 41.99127662600005], [-87.65434791799997, 41.991269288000055], [-87.65423302799994, 41.99057994600002], [-87.65422802799996, 41.990501946000045], [-87.65419002799993, 41.99000494600005], [-87.65411202799997, 41.98915094600004], [-87.65405602799994, 41.988873946000055], [-87.65398002799996, 41.98859094600003], [-87.65378602799994, 41.98833494600006], [-87.65350148699996, 41.988015442000055], [-87.65322671899995, 41.98775865800007], [-87.65307296899992, 41.987614888000046], [-87.65272656199991, 41.98727562600003], [-87.65188468799992, 41.98669764700002], [-87.65171902799993, 41.98655194600007], [-87.65140661699995, 41.986185632000044], [-87.65120638799992, 41.98603264700005], [-87.65095627399994, 41.98591569400002], [-87.65058214899993, 41.98579079200003], [-87.65035913099995, 41.98575162100007], [-87.65011411599993, 41.98485274400007], [-87.65010391199995, 41.98481530900005], [-87.65011373299993, 41.98479637600007], [-87.65030964299994, 41.984418688000055], [-87.65042478699995, 41.984088636000074], [-87.65069178199994, 41.98333663900006], [-87.65072594099993, 41.98312643000002], [-87.65077258999995, 41.98238095700002], [-87.65077316199992, 41.98234448600003], [-87.65077334499993, 41.98233233000007], [-87.65077347299994, 41.98232380600006], [-87.65077401699993, 41.98228761500007], [-87.65077335099993, 41.98227476400005], [-87.65075602799998, 41.98196294600006], [-87.65069402799996, 41.98140494600005], [-87.65069390899995, 41.98134825700002], [-87.65069314799995, 41.980984483000036], [-87.65069302899994, 41.98092759100006], [-87.65066242799996, 41.980640746000056], [-87.6503690499999, 41.98005399100003], [-87.65035702799996, 41.98002994600005], [-87.65036879799996, 41.97999757500002], [-87.65038836999994, 41.97994375300003], [-87.65040343599995, 41.97990232100005], [-87.65038861099997, 41.97988593500001], [-87.65005663699995, 41.97954063500003], [-87.64907062399996, 41.978755633000034], [-87.64843869799994, 41.978264650000014], [-87.64830846099993, 41.97814872600002], [-87.64805615299996, 41.97813078700005], [-87.64695702799995, 41.977930946000065], [-87.64687521799998, 41.97768551700005], [-87.64685653299993, 41.97762946100004], [-87.64687318999994, 41.97762946400007], [-87.647077, 41.977618], [-87.64881891599995, 41.97752002100002], [-87.64966904599997, 41.977377161000035], [-87.64982577999993, 41.97673688500003], [-87.65007191499993, 41.97667802200004], [-87.65016791599997, 41.976649022000046], [-87.65024091599996, 41.97662702200005], [-87.65042391599994, 41.97658102300004], [-87.65059091599993, 41.97653802300003], [-87.65085491499995, 41.97649002200006], [-87.65107691499996, 41.97645802400006], [-87.65179491499997, 41.97642002300005], [-87.65215491599996, 41.976402023000034], [-87.65244091499993, 41.97639902300004], [-87.65345591499991, 41.97638002300004], [-87.65499039199993, 41.97635499900007], [-87.65640291499993, 41.976333022000055], [-87.65704791599994, 41.97632302200002], [-87.65780291499996, 41.97630802200006], [-87.65865891499996, 41.976300023000064], [-87.6588379149999, 41.97629902300007], [-87.65984927899993, 41.97629395000006], [-87.65987141199996, 41.97676909600005], [-87.65988673899993, 41.97711204400007], [-87.65991432899992, 41.97772935100004], [-87.65993441299995, 41.978483825000055], [-87.65993816799993, 41.97882741700003], [-87.65994567899996, 41.97951460200004], [-87.65997454199997, 41.98054458400002], [-87.65999332499993, 41.98103026100004], [-87.66000924899996, 41.98144200500007], [-87.66003352499993, 41.982128019000065], [-87.66005400699996, 41.98274588000003], [-87.66007900099997, 41.98352917700003], [-87.66009209299993, 41.983979862000055], [-87.66010511499994, 41.98446429400006], [-87.66012173099993, 41.98508238700003], [-87.66013867399991, 41.98556348800002], [-87.66015965099994, 41.98624971700007], [-87.66016896, 41.986524308000064], [-87.66020101799994, 41.98769203900007], [-87.66020733399994, 41.98796837200006], [-87.66021679099997, 41.98838208700004], [-87.66024393799992, 41.98941066500003], [-87.66027294599994, 41.99030542300004], [-87.66028984499994, 41.99079998300004], [-87.66030798999996, 41.99133971000003], [-87.66033463599994, 41.99209518300005], [-87.66036748699996, 41.99312462100004], [-87.66039849599996, 41.99408569800004], [-87.66040894199992, 41.994450486000055], [-87.66042798999997, 41.995115640000044], [-87.66045031399993, 41.995941614000046], [-87.66046825799995, 41.996902845000065], [-87.6605187179999, 41.99772596800005], [-87.660552, 41.99807092200005], [-87.66054391599992, 41.99807102300007], [-87.65965991499995, 41.99809602300006], [-87.65949191499996, 41.99810202200006], [-87.65928591599997, 41.99812102200006], [-87.65891491499995, 41.99815502300003], [-87.65862691499993, 41.99815602200003], [-87.65701491599997, 41.998165022000066], [-87.65620891599995, 41.99816702200006], [-87.65606291399995, 41.99815302000006], [-87.65596292999996, 41.99811584600007], [-87.65518515599994, 41.99828931200004], [-87.65480440599997, 41.99835224300005], [-87.65476458599993, 41.99835771500004], [-87.65467002799993, 41.998288946000045], [-87.65462902799997, 41.99823894600006], [-87.6545600279999, 41.99815594600004], [-87.65448002799995, 41.997823946000025], [-87.65442602799996, 41.997264946000016]]]}, "properties": {"city": "Chicago", "name": "Edgewater Beach", "regionid": "403257", "geo_point_2d": [41.98585732009902, -87.65593424851126], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.68772409996505, 41.92778776790994], [-87.68769640185243, 41.926908452935685], [-87.68766035075976, 41.926053319873006], [-87.68764269678884, 41.92538246998057], [-87.68759779684578, 41.9242800861583], [-87.6875672100193, 41.923671175841115], [-87.68754478981059, 41.922940904816485], [-87.68752344108469, 41.9222699558385], [-87.68750090979364, 41.921171843881055], [-87.68747615882249, 41.920319331778366], [-87.68743976793948, 41.91946064786719], [-87.68741413804011, 41.91879104205799], [-87.68739482821988, 41.91793696123272], [-87.6873802268919, 41.917203504202675], [-87.68733848118299, 41.916167105788126], [-87.68731883783248, 41.91525211388615], [-87.68728766004253, 41.91421597275964], [-87.68728208114936, 41.91399402011903], [-87.68727512511369, 41.91372601096531], [-87.68725185075694, 41.91292946580091], [-87.6872358121434, 41.9123815529587], [-87.6872159172334, 41.9115883841907], [-87.68719814688741, 41.91091754088286], [-87.68719085081719, 41.91038240014563], [-87.68718999999982, 41.91032000000031], [-87.68801497992311, 41.910295643144046], [-87.68917412857353, 41.910283787572766], [-87.69039473614436, 41.910267727867726], [-87.69118785827101, 41.91025737307231], [-87.69265244457122, 41.910245307831595], [-87.69393395861795, 41.910234803618096], [-87.69521566070107, 41.91023000000014], [-87.69643604980328, 41.910224431601776], [-87.69759498846672, 41.91020790023084], [-87.69851185194796, 41.91019516803759], [-87.6997322972407, 41.91018413896269], [-87.70083098391318, 41.91017029255529], [-87.70184691576924, 41.91023402316453], [-87.70184078149366, 41.91016189304114], [-87.70205191532648, 41.910160022451585], [-87.70219291553079, 41.910159022405324], [-87.70335591608107, 41.910148021898145], [-87.70449491552515, 41.91013602224426], [-87.70556991561622, 41.910119022359915], [-87.7056399152456, 41.910118022313824], [-87.70639291668157, 41.91011202203728], [-87.70665691536111, 41.91010802185291], [-87.70688102461217, 41.910106175544925], [-87.70778793304665, 41.91008957677208], [-87.70894719499523, 41.9100748816022], [-87.71084273614069, 41.91002329925296], [-87.71285305659009, 41.909949275112716], [-87.71425657716745, 41.90992], [-87.71566128566161, 41.90991096779594], [-87.71718665195107, 41.909894559887306], [-87.71840704368913, 41.90987445150337], [-87.7198108423252, 41.90985559173238], [-87.72096992792427, 41.909844274120466], [-87.7219463718807, 41.909837484757084], [-87.72328883683042, 41.90982148256622], [-87.72475395802623, 41.90980322149463], [-87.72548614646057, 41.909788232484594], [-87.72640985081988, 41.90977841754782], [-87.7262687923841, 41.91092075248931], [-87.72630886666542, 41.91220115551966], [-87.72635216200783, 41.91363218089306], [-87.72637373439652, 41.91439424757358], [-87.72641864543068, 41.915922422769036], [-87.72645436703503, 41.917264333068715], [-87.72648075269652, 41.91830050206145], [-87.72650104409433, 41.9189108889472], [-87.72652791583657, 41.91981502234948], [-87.7265589154669, 41.920744022905076], [-87.72657391525898, 41.921445022843166], [-87.72658991599661, 41.92173802016167], [-87.72661483890779, 41.921845058368945], [-87.72674991527816, 41.92202102241512], [-87.72688280000051, 41.92225615285753], [-87.72726808574568, 41.92241998235167], [-87.72707099662492, 41.922644187830215], [-87.7267788015017, 41.92266087295177], [-87.72665273993553, 41.923100022690555], [-87.72675496766911, 41.92344801265185], [-87.72676879384619, 41.92448415933337], [-87.72667338203274, 41.92501553698743], [-87.72668560376108, 41.925568570193505], [-87.7267167020412, 41.92672662858349], [-87.72674892424709, 41.92770403549889], [-87.72677243111802, 41.92861804698992], [-87.72678554947633, 41.929228050659006], [-87.72682474648428, 41.93045060927868], [-87.72685004327775, 41.93124194750509], [-87.72685819836572, 41.931686035716446], [-87.72686, 41.93169000000011], [-87.72687732954806, 41.93270607964968], [-87.72690831592037, 41.9339259005013], [-87.72692924399786, 41.93472040127207], [-87.7269631219091, 41.935756779184274], [-87.72698861048788, 41.93667177720491], [-87.72700503815912, 41.937648472496306], [-87.72704678068553, 41.939017890306744], [-87.7262198351398, 41.93903338987691], [-87.72542470219179, 41.93904592782803], [-87.72414194407818, 41.93907082508597], [-87.72286160686211, 41.93908000000014], [-87.72176309215644, 41.93908000000014], [-87.72029840621535, 41.93909000000025], [-87.71895560281826, 41.93909537510666], [-87.7181012510874, 41.939106045031515], [-87.71663865010618, 41.93915162177052], [-87.71535899216131, 41.93920971587977], [-87.71444527909799, 41.93924742512208], [-87.71328098676058, 41.93928311205407], [-87.71157132082236, 41.939296956194305], [-87.71047392877534, 41.93930620586964], [-87.7093754630653, 41.93931545423705], [-87.7080599999998, 41.9393200000001], [-87.70568967612094, 41.937996491149136], [-87.70158802610042, 41.9356319230622], [-87.6971427561109, 41.93298010621859], [-87.6955572987716, 41.932108846050745], [-87.69592105202251, 41.93210768779877], [-87.69315042898945, 41.93070916978964], [-87.68948889405182, 41.928781891878856], [-87.68772409996505, 41.92778776790994]], [[-87.70691155288995, 41.91799581549108], [-87.70688, 41.917390000000154], [-87.70589669075287, 41.91740011621344], [-87.7045525832139, 41.91742003805983], [-87.70284414674721, 41.91743688127413], [-87.7012575277584, 41.917453071798775], [-87.69960907262548, 41.917465085904226], [-87.69887687747251, 41.91747578121375], [-87.697656565476, 41.91748532778954], [-87.69655817147509, 41.917495014746386], [-87.69521563050124, 41.917505986524894], [-87.69277462036553, 41.91752701408077], [-87.68929632475512, 41.917552384441336], [-87.68854999999988, 41.917560000000194], [-87.69004144334427, 41.91846404440006], [-87.69213028039081, 41.9197232141511], [-87.69337734478407, 41.920491415718885], [-87.69484040243695, 41.92136945410683], [-87.69605413406887, 41.922112636167796], [-87.69773712882346, 41.92315296397551], [-87.69857813866255, 41.923663862516236], [-87.69925628775978, 41.92408002323705], [-87.70041, 41.92479000000012], [-87.7014416846215, 41.924770894729335], [-87.70388202282537, 41.92474627566539], [-87.70595683671112, 41.92472188112069], [-87.70719999999987, 41.92471], [-87.70717530030878, 41.92391960988494], [-87.70710901133306, 41.922208929316184], [-87.70707890566052, 41.921048113213736], [-87.7069779434413, 41.91982559114901], [-87.70694723032358, 41.918849679388536], [-87.70691155288995, 41.91799581549108]]]}, "properties": {"city": "Chicago", "name": "Logan Square", "regionid": "269592", "geo_point_2d": [41.92369794352411, -87.7094260474836], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.72589999999984, 41.89529000000017], [-87.72590501247194, 41.89528991782857], [-87.72646305612945, 41.89528076957184], [-87.7276833988717, 41.89525647842728], [-87.72884232115979, 41.89523101834718], [-87.73024548747343, 41.895199575208956], [-87.73146575523313, 41.89517958459322], [-87.73268603091985, 41.895160063001406], [-87.73390530836083, 41.895135367291225], [-87.73500632546906, 41.89511617066578], [-87.73622626468583, 41.89509485192078], [-87.73787335202371, 41.895047260356144], [-87.73866601355374, 41.895029693220664], [-87.7397054943677, 41.89501], [-87.74037546570672, 41.895006804069645], [-87.74104623872695, 41.895000693520245], [-87.74232879909918, 41.89498550987195], [-87.74489428132254, 41.89494373486432], [-87.74571, 41.89493], [-87.74571898659586, 41.895356863306176], [-87.74574533196389, 41.896334848269944], [-87.74576458169699, 41.89718693444063], [-87.74579175679018, 41.89834720823465], [-87.74579999999986, 41.899140038599036], [-87.74581818649449, 41.89993828937385], [-87.74585003356373, 41.90091157749665], [-87.74587988824611, 41.90227094174235], [-87.74589290428715, 41.90286359721017], [-87.74590479545485, 41.90365515914172], [-87.74592913800822, 41.90469586423289], [-87.74594623355686, 41.905548553019145], [-87.74595613217656, 41.9067078438471], [-87.74603526575144, 41.9090247976036], [-87.74598969056683, 41.909520513589904], [-87.74562542541405, 41.909524751203186], [-87.7443439032033, 41.90954284827813], [-87.74324538898513, 41.90955164265172], [-87.74220794633378, 41.909558335830354], [-87.74133271476212, 41.90957737917592], [-87.7408040087793, 41.90959000000012], [-87.7401931548398, 41.90959377427862], [-87.73909471722763, 41.90961263197517], [-87.73763026759764, 41.90962777034467], [-87.73415178228726, 41.909686058109685], [-87.7330533793201, 41.909690000000225], [-87.73152776046601, 41.90970802132902], [-87.73067347400647, 41.90971451298725], [-87.72963540746575, 41.909709776613106], [-87.72817268249314, 41.909719042830766], [-87.72713485881512, 41.90976682074892], [-87.72640985081988, 41.90977841754782], [-87.7262757853258, 41.9083562948214], [-87.72625224890764, 41.90738234714778], [-87.72621860705027, 41.90652447978326], [-87.72617000909385, 41.904815413773655], [-87.7261481691646, 41.90383944324615], [-87.72610999999989, 41.90257000000016], [-87.72608471948887, 41.90170158466976], [-87.72603686780377, 41.899933361621244], [-87.72598877927186, 41.89816483306137], [-87.72589999999984, 41.89529000000017]]]}, "properties": {"city": "Chicago", "name": "West Humboldt Park", "regionid": "403318", "geo_point_2d": [41.90236465071005, -87.73600670726647], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.74232972399992, 41.80755], [-87.7431072659999, 41.807547666000055], [-87.74312959799992, 41.80844170600004], [-87.74317540699988, 41.81014916000003], [-87.74322842499991, 41.81218856800005], [-87.74326724299992, 41.813814561000065], [-87.74331387399988, 41.81560442000004], [-87.7434078669999, 41.818992750000056], [-87.74341938599989, 41.81940801100005], [-87.74335060299988, 41.819427830000045], [-87.7394560729999, 41.82060118200007], [-87.73853914599992, 41.82088715400004], [-87.7385309999999, 41.820371], [-87.73851791499992, 41.81954202300005], [-87.73850191599988, 41.81872502300006], [-87.73849997099988, 41.81865205400003], [-87.73849691499991, 41.818486023000034], [-87.7384939159999, 41.81836502200007], [-87.7384919159999, 41.81825402300006], [-87.73848491599993, 41.817948023000035], [-87.73841291599989, 41.815066022000046], [-87.7383849159999, 41.81327402200008], [-87.73834991599993, 41.811420022000036], [-87.73831591599989, 41.81003302200008], [-87.73827491599988, 41.80853402200005], [-87.7382739159999, 41.80763202200006], [-87.7382738939999, 41.80762110500006], [-87.73883250399989, 41.80761581900003], [-87.74029626199987, 41.80757873900006], [-87.74110943799988, 41.80756061100003], [-87.74232972399992, 41.80755]]]}, "properties": {"city": "Chicago", "name": "Sleepy Hollow", "regionid": "403335", "geo_point_2d": [41.813880536138115, -87.74078814156515], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.63898878899992, 41.86000915200008], [-87.63898291599989, 41.85991102300004], [-87.63895591599992, 41.85887202300005], [-87.63894691599988, 41.85809902300007], [-87.63894004799992, 41.857652886000025], [-87.6389153879999, 41.85709007000007], [-87.63888593599994, 41.85642005000004], [-87.6388313509999, 41.85556479400008], [-87.6387622539999, 41.85512607400005], [-87.63870494799993, 41.854782916000026], [-87.63864500999993, 41.85442399500003], [-87.63856158099989, 41.853931852000066], [-87.63853249799989, 41.85311510000007], [-87.63853999999989, 41.8528], [-87.63947570299987, 41.852760450000055], [-87.64004605799992, 41.852748099000046], [-87.64043944699993, 41.85273793500004], [-87.64082399999987, 41.852728], [-87.64082035399989, 41.852733683000054], [-87.64297597899989, 41.85269077100008], [-87.64580144499989, 41.85264156700003], [-87.64647388299993, 41.85262985700006], [-87.6464974549999, 41.85300464400006], [-87.6465144999999, 41.85367497700007], [-87.64651891499993, 41.85440841700005], [-87.6465481209999, 41.85507866800002], [-87.64657290599989, 41.85574880800003], [-87.64658501799993, 41.856175674000035], [-87.6466146839999, 41.85708976600005], [-87.6466302509999, 41.85782164100003], [-87.64666305099989, 41.85886205000007], [-87.6466807019999, 41.85941111600004], [-87.6466997139999, 41.85990510300006], [-87.6466629149999, 41.85990602300006], [-87.64485291499992, 41.85995502200006], [-87.64476091599991, 41.859956022000055], [-87.64464891499992, 41.85995702200005], [-87.64457991599993, 41.85995702200005], [-87.64442991599991, 41.859958021000054], [-87.64442283999992, 41.85995797300006], [-87.64428291599994, 41.85995702200005], [-87.6442069159999, 41.85995902100005], [-87.6441209149999, 41.85995302200006], [-87.64402091599992, 41.85994702300007], [-87.64384891499991, 41.85995002200007], [-87.64292591599988, 41.85997302200006], [-87.64192591499992, 41.85998502300003], [-87.64092391499989, 41.85999802200006], [-87.64043991599992, 41.85999602200007], [-87.63898878899992, 41.86000915200008]]]}, "properties": {"city": "Chicago", "name": "East Pilsen", "regionid": "403331", "geo_point_2d": [41.85631216962779, -87.64270424985413], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.68128029899991, 41.69906098500007], [-87.68125737699992, 41.69834736500007], [-87.68123648099991, 41.69760296000003], [-87.68123, 41.69676446300007], [-87.68120725599992, 41.695927470000065], [-87.68117321499993, 41.69490674100006], [-87.68115292299987, 41.694346687000056], [-87.68111088599993, 41.693326436000056], [-87.68107734099992, 41.69230553500005], [-87.68114, 41.69175], [-87.6817403679999, 41.69175508800003], [-87.6832268569999, 41.691745490000066], [-87.68434140899987, 41.69172745700006], [-87.68527046599989, 41.69171547600006], [-87.68657067799988, 41.69170070900008], [-87.68796446499992, 41.69168406200002], [-87.68917212799992, 41.69167411500006], [-87.69000819399989, 41.69166715800008], [-87.69086, 41.69178], [-87.69096517999989, 41.691901290000075], [-87.6909579999999, 41.691738], [-87.695749, 41.691733], [-87.69584899999988, 41.692633], [-87.6959329999999, 41.695316], [-87.695993, 41.697221], [-87.6960491819999, 41.69900700200007], [-87.69604999999989, 41.699033], [-87.691149, 41.699033], [-87.69114949599991, 41.698968015000034], [-87.69111591599993, 41.698968022000074], [-87.68995291599991, 41.69899002200008], [-87.6894989159999, 41.69899002200008], [-87.68869091499991, 41.69899002200008], [-87.68800991599993, 41.69898302200004], [-87.6874389149999, 41.699001022000054], [-87.6868949149999, 41.69900402200005], [-87.6862379159999, 41.69900202200005], [-87.6861059159999, 41.699009022000034], [-87.68553391599991, 41.69901002200003], [-87.68501791499992, 41.69901002200003], [-87.68378691599989, 41.69904402200006], [-87.6825879159999, 41.69904402200006], [-87.6818489159999, 41.69905202300002], [-87.6812804, 41.69906098300003], [-87.68128029899991, 41.69906098500007]]]}, "properties": {"city": "Chicago", "name": "West Morgan Park", "regionid": "403373", "geo_point_2d": [41.69537256805963, -87.68855485985269], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.52582891599984, 41.69179502200012], [-87.52497691599984, 41.69179202300014], [-87.52485351758597, 41.69179679896572], [-87.52484399999987, 41.691635000000204], [-87.52484399999987, 41.689235], [-87.52481299999978, 41.686938], [-87.52474299999987, 41.681735000000145], [-87.52482099999985, 41.680952], [-87.52484299999988, 41.68073500000019], [-87.52484299999988, 41.67623500000018], [-87.52484299999988, 41.67556002600011], [-87.52484299999988, 41.675436000000154], [-87.52484299999988, 41.675035], [-87.52484299999988, 41.67447772200009], [-87.52484299999988, 41.674135000000156], [-87.52478499999988, 41.67373300000011], [-87.52474299999987, 41.67343500000015], [-87.52482776299986, 41.673265474000154], [-87.52484299999988, 41.67323500000012], [-87.52484299999988, 41.66781951000004], [-87.52484299999988, 41.6674350000002], [-87.52490899999984, 41.66677500000012], [-87.5249429999999, 41.666435], [-87.52475733411733, 41.66170051999557], [-87.52474299999987, 41.661335000000136], [-87.52474299999987, 41.660435000000106], [-87.52474299999987, 41.659198], [-87.52474299999987, 41.6567370000002], [-87.52474299999987, 41.65668567900013], [-87.52474299999987, 41.656435], [-87.524793, 41.655632], [-87.52484299999988, 41.654835], [-87.52483099999986, 41.6546340000001], [-87.52476639044252, 41.65346001497571], [-87.52477199895795, 41.65346116968829], [-87.52516326399981, 41.653541725000046], [-87.5252628789999, 41.65268503900005], [-87.5256783879999, 41.65264444400015], [-87.52624907599987, 41.652872719000115], [-87.526251, 41.65286200000021], [-87.52631879399985, 41.65289423000004], [-87.52647817699992, 41.653143266000214], [-87.52665748399991, 41.65342218700011], [-87.52680690599993, 41.65361145500014], [-87.52699617299987, 41.653850530000106], [-87.5276835139999, 41.65492636800013], [-87.52823139499985, 41.65576313100017], [-87.52865973699993, 41.65644051], [-87.52878923599991, 41.65670947000005], [-87.52891873499986, 41.656968468000166], [-87.52909804199987, 41.65710792800016], [-87.52925091699981, 41.657048598000046], [-87.52940731399988, 41.65693688600003], [-87.52967552399986, 41.656764465000066], [-87.53174456699992, 41.655155210000046], [-87.53134225299992, 41.654772054000155], [-87.53134225299992, 41.65367], [-87.53163999999977, 41.65367], [-87.5321468799999, 41.65367], [-87.53248075799988, 41.65367], [-87.53247376099993, 41.654320749000206], [-87.5331752739999, 41.65433608200014], [-87.53318240799986, 41.65367623200008], [-87.53356455799977, 41.65368], [-87.53392855599978, 41.65368], [-87.53427395199984, 41.653686998000076], [-87.5345807871049, 41.653690000000154], [-87.53458078715738, 41.654214515117005], [-87.53458078715738, 41.65495774587802], [-87.53504530619537, 41.65532936100852], [-87.53550982523336, 41.65579388004651], [-87.53523111401057, 41.65579388004651], [-87.53495240278774, 41.656165495177], [-87.53532401791816, 41.656815822030346], [-87.53541692182583, 41.657466148883515], [-87.53606724817905, 41.65783776401418], [-87.53681047893998, 41.65839518695967], [-87.53681047893998, 41.65876680209016], [-87.53718209407047, 41.659324225036], [-87.53922597803788, 41.659324225036], [-87.5398905870021, 41.659336532881596], [-87.53988999999983, 41.65987275500004], [-87.53989119899983, 41.660786351000134], [-87.53989806099986, 41.66190376700004], [-87.53988376699994, 41.663072715000126], [-87.53972393699976, 41.6640063030001], [-87.53955686699989, 41.664786305000156], [-87.53933935899991, 41.665879888000134], [-87.53917798999989, 41.66666955500001], [-87.53892778199989, 41.66779854200007], [-87.53879400999992, 41.668554802000116], [-87.5388485939999, 41.66906250300012], [-87.5390858759999, 41.66975827500013], [-87.53935604799987, 41.670306936000145], [-87.53958721199982, 41.67077701500005], [-87.53977310799982, 41.671259324000076], [-87.5399197669999, 41.67180325200013], [-87.53998072799988, 41.672463493000144], [-87.53998941899987, 41.673580354000094], [-87.53999692499988, 41.67454491600014], [-87.53999999999988, 41.67540822300007], [-87.54000774999987, 41.67647365600003], [-87.54002353299987, 41.67733644800019], [-87.5400421009999, 41.67835149900006], [-87.54005418699982, 41.67901221000011], [-87.54007368299992, 41.6800780130001], [-87.5400854499999, 41.68095746800012], [-87.53913289399988, 41.68095873400004], [-87.5367176139998, 41.68095334300004], [-87.53177373499992, 41.68093818100004], [-87.53114334599991, 41.681714045000106], [-87.53077173099989, 41.68264308300006], [-87.53040538599987, 41.68321295300014], [-87.53040623299994, 41.68357222500001], [-87.53042078399979, 41.68456288500009], [-87.53033091599983, 41.68494902400016], [-87.53030791599987, 41.68609602300012], [-87.5303169159999, 41.686407023000086], [-87.53033091499992, 41.686878023000155], [-87.53030791599987, 41.688407022000106], [-87.53037004899986, 41.69181815500017], [-87.53020891599986, 41.691817022000166], [-87.52944991499984, 41.69181102300011], [-87.52822591599993, 41.69180502200006], [-87.5270169159999, 41.69180002200018], [-87.52582891599984, 41.69179502200012]], [[-87.53047608499975, 41.66895954500012], [-87.53028947099968, 41.66890367900047], [-87.53024738499974, 41.668910628000425], [-87.5301888179998, 41.6689163960002], [-87.53013, 41.66891832200018], [-87.53003099999978, 41.66891832200018], [-87.53001939899981, 41.668918247000086], [-87.52998627099973, 41.6689781100005], [-87.52938258499987, 41.67009797800044], [-87.52778007599963, 41.673356002], [-87.52777762899969, 41.673376385], [-87.52776804199978, 41.67343444900018], [-87.52776697799979, 41.673439715000086], [-87.52774297799985, 41.673556715000174], [-87.5277306779997, 41.67360876200013], [-87.52771359499981, 41.67366507800017], [-87.52769286499968, 41.673720156000115], [-87.52766857799973, 41.67377376000029], [-87.52764083599992, 41.6738256610004], [-87.52763488199979, 41.67383581900036], [-87.52762701299964, 41.67384904500021], [-87.52757671599986, 41.673979926000115], [-87.5276498729996, 41.67400473500048], [-87.52765999999981, 41.67400467800003], [-87.5277035269996, 41.67400573200011], [-87.52790486799974, 41.67401548900012], [-87.52832300199961, 41.674011852000085], [-87.52888308599985, 41.673986816000145], [-87.5289757189997, 41.6739769190003], [-87.52902563999983, 41.67395815900018], [-87.52905292299978, 41.673948405000075], [-87.52910923899982, 41.673931322], [-87.52916655099986, 41.67391795800013], [-87.52922461499976, 41.67390837200041], [-87.52928318199982, 41.67390260400011], [-87.5293419999997, 41.67390067800013], [-87.52940081899979, 41.67390260400011], [-87.52942617799984, 41.673904626000095], [-87.52954317799967, 41.673915626000124], [-87.52957638499984, 41.67391937200007], [-87.52963444899976, 41.67392895800016], [-87.5296917619997, 41.67394232100047], [-87.5297102099998, 41.673947476000095], [-87.52976318699966, 41.67396288700003], [-87.5298144529998, 41.673964174000055], [-87.53000001299979, 41.67392251700007], [-87.53002155099963, 41.67391795800013], [-87.5300796149998, 41.67390837200041], [-87.5301381819996, 41.67390260400011], [-87.53019699999966, 41.67390067800013], [-87.53024620099966, 41.673902025000146], [-87.53039220099984, 41.673910024000186], [-87.53040181799982, 41.67391060300015], [-87.53046038499971, 41.6739163710001], [-87.53047692999982, 41.67391870600038], [-87.53059745899968, 41.673936858000104], [-87.53068494799989, 41.673941], [-87.53090034899978, 41.67393413400005], [-87.53098781799983, 41.67393560300009], [-87.53100648899986, 41.67393702200011], [-87.53119148899971, 41.67395302200002], [-87.53123138499981, 41.67395737100047], [-87.53128944899991, 41.67396695800011], [-87.53134676199986, 41.67398032100006], [-87.53140307799988, 41.67399740500041], [-87.53143745199972, 41.6740098580001], [-87.53154564899978, 41.67405156300009], [-87.53163652399977, 41.674049333000085], [-87.53173834599984, 41.67403395800032], [-87.53178118599983, 41.673990446000275], [-87.53180118899988, 41.6739530560002], [-87.53180956299977, 41.67392183200045], [-87.53181037199985, 41.673913615000046], [-87.53181995899976, 41.673855551000216], [-87.53183332199988, 41.67379823800018], [-87.53185040499977, 41.67374192200014], [-87.53187123949968, 41.673686592500395], [-87.5320103429997, 41.67335134100006], [-87.53203442199982, 41.673298240000015], [-87.53204942599982, 41.6732691620001], [-87.53209157399985, 41.67319099700011], [-87.53220217899975, 41.67295256100011], [-87.53221142199989, 41.6729332400002], [-87.5322391639997, 41.67288133900007], [-87.53227024099962, 41.67283136300038], [-87.5323045199997, 41.672783527000156], [-87.53232751299983, 41.672754798000355], [-87.53244851299972, 41.67260979800044], [-87.53246285399975, 41.672593036000286], [-87.53250308299978, 41.672550084000285], [-87.53254603599962, 41.672509854000154], [-87.53259152699981, 41.67247252000009], [-87.5326043279999, 41.67246287800036], [-87.53269897999984, 41.672392893000165], [-87.53275597599983, 41.672330492000064], [-87.53278408399977, 41.67230108300003], [-87.53282703599977, 41.672260854000164], [-87.53287252799971, 41.67222352000009], [-87.53292036399984, 41.67218924100018], [-87.5329703389999, 41.67215816400001], [-87.53302223999967, 41.67213042200003], [-87.53307584499983, 41.67210613400015], [-87.53313092299977, 41.672085405000125], [-87.53313692699983, 41.672083390000225], [-87.53346338599984, 41.67197513000014], [-87.53354553599989, 41.67193376400045], [-87.5335522399998, 41.671930423000084], [-87.53360584499964, 41.671906135000214], [-87.5336281409998, 41.671897246000455], [-87.53385419899975, 41.67181060300048], [-87.53402026699982, 41.67171751600032], [-87.53406223999966, 41.67169542300018], [-87.53409094799972, 41.671681890000045], [-87.53413208099992, 41.671663380000105], [-87.53436830199966, 41.671442606000134], [-87.53438357199977, 41.67142374300009], [-87.53440855499987, 41.671379420000065], [-87.53441316399973, 41.67137133900019], [-87.5344442409998, 41.67132136400006], [-87.53447851999971, 41.671273528000185], [-87.53449384099983, 41.67125412000017], [-87.53454355099967, 41.671192897000346], [-87.53462572999963, 41.671050214], [-87.53461924099982, 41.67104063600009], [-87.53458816399983, 41.67099066100014], [-87.53456042299973, 41.6709387600002], [-87.53453613499978, 41.67088515500011], [-87.53451540499984, 41.67083007700017], [-87.53450277599967, 41.67078979100022], [-87.53444577599984, 41.67059179100017], [-87.53444132199975, 41.67057576100045], [-87.5344279579997, 41.67051844900014], [-87.53441837199962, 41.67046038500014], [-87.53441260399968, 41.67040181800017], [-87.53441110399964, 41.67037065900017], [-87.53440710299975, 41.67024066000017], [-87.53440860299972, 41.67015418200047], [-87.53441046699976, 41.67013053100003], [-87.53410534899974, 41.67004844000007], [-87.53404992299969, 41.67003159500019], [-87.53399484499985, 41.67001086500007], [-87.53398396099983, 41.670006273000226], [-87.53351158299992, 41.66980329800004], [-87.53310517299978, 41.669673113000364], [-87.5327886909998, 41.669603080000165], [-87.5327502379998, 41.669593678000275], [-87.53274599499981, 41.66959253000001], [-87.5324108359998, 41.66950096700016], [-87.53156803899978, 41.66937788200001], [-87.53152255099984, 41.669370041000164], [-87.53146523799971, 41.669356678000035], [-87.53140892199966, 41.66933959500013], [-87.53135384399981, 41.66931886500019], [-87.53133512499979, 41.66931086200015], [-87.53117412499988, 41.66923986200014], [-87.53113923999966, 41.669223577000125], [-87.5310873389997, 41.66919583600005], [-87.53103736299983, 41.669164759000054], [-87.53099923799978, 41.669137828000046], [-87.53096250899986, 41.66911042600009], [-87.5308369079996, 41.669058132000195], [-87.53065489699972, 41.66900245800004], [-87.5305334499997, 41.668974675000136], [-87.5305012379998, 41.668966678000274], [-87.53047608499975, 41.66895954500012]], [[-87.52769162499986, 41.66905218900051], [-87.5281944019998, 41.66809386199999], [-87.52864180299979, 41.6671528450002], [-87.5286474229998, 41.66714124000005], [-87.5286751639998, 41.66708933900011], [-87.52870027499974, 41.667048410000184], [-87.52876453299982, 41.6669496850001], [-87.5289520759997, 41.66649481400021], [-87.52904971899987, 41.66622009000004], [-87.52908422899971, 41.66608821200014], [-87.52912333999966, 41.665834344000096], [-87.5291465429999, 41.66487375100028], [-87.52912374799968, 41.66304722300014], [-87.52912367999987, 41.66303775000016], [-87.5291216959997, 41.662018242], [-87.52910187299992, 41.661066732000144], [-87.5291016779998, 41.66104800000038], [-87.52910169599973, 41.66104227800008], [-87.52910739299985, 41.66014701000008], [-87.52892400899977, 41.66023641650024], [-87.52841896099972, 41.66048249100003], [-87.5267699939998, 41.661306476000085], [-87.5267657599997, 41.66130857800008], [-87.52671215499991, 41.661332865000226], [-87.5266570769996, 41.66135359499999], [-87.52661759499979, 41.6613659920004], [-87.52657259599971, 41.661378992000046], [-87.5265679049996, 41.661380333000075], [-87.52639281899958, 41.66142988600017], [-87.52621216599982, 41.66152459700011], [-87.52617697699966, 41.662306879000354], [-87.5261301909998, 41.66505033400006], [-87.52612959899983, 41.66507104200018], [-87.52605780599968, 41.66686095600013], [-87.52610799799973, 41.66872985600004], [-87.52610832099977, 41.668752578000145], [-87.52611132149971, 41.67064928900014], [-87.5261108859997, 41.670677974000114], [-87.52608097799973, 41.67163903200005], [-87.52606672199978, 41.67221925900019], [-87.5262066189999, 41.671950866000095], [-87.52668380599978, 41.671003453000125], [-87.52669086299974, 41.67098973400015], [-87.52718500699977, 41.6700491710004], [-87.52768622599977, 41.66906264600004], [-87.52769162499986, 41.66905218900051]], [[-87.53108027899984, 41.66177780800019], [-87.5310747039999, 41.66234698000021], [-87.53108018199983, 41.662346603000444], [-87.5311389999998, 41.66234467800019], [-87.53119781899969, 41.662346603000444], [-87.53124806299977, 41.66235131500012], [-87.53146906199967, 41.66237831600013], [-87.53147340599976, 41.66237885700009], [-87.53159140599976, 41.66239385700008], [-87.53159538499973, 41.662394372000044], [-87.53165435399983, 41.66240413900023], [-87.53318225999983, 41.662711320000156], [-87.53323776199974, 41.66272432100025], [-87.53324339199966, 41.662725850000115], [-87.53380258099966, 41.66287957900049], [-87.53457452499967, 41.663048707000094], [-87.53531612099984, 41.66317275300041], [-87.53546649099965, 41.66267765100012], [-87.5354754049997, 41.66264992300013], [-87.53549613499972, 41.66259484400011], [-87.53552042299978, 41.66254124000012], [-87.53554816499975, 41.662489339000174], [-87.53557779599971, 41.66244153400004], [-87.53566151799973, 41.662315448000086], [-87.53569651899987, 41.662266527000035], [-87.53573385299977, 41.66222103600044], [-87.53577408299971, 41.66217808300017], [-87.53581703499988, 41.662137854000136], [-87.53586252699992, 41.66210052000007], [-87.53589520799979, 41.66207659300013], [-87.53602033699984, 41.661989238000096], [-87.53607877699979, 41.66193988900001], [-87.53611152799984, 41.66191352000004], [-87.53615936399989, 41.661879241000136], [-87.53620933899967, 41.66184816400015], [-87.53626123999979, 41.66182042300007], [-87.5363148449997, 41.661796135000195], [-87.5363699229998, 41.66177540500007], [-87.53637524899963, 41.661773615000286], [-87.53717524899983, 41.661507615000126], [-87.53722623899986, 41.6614923210001], [-87.5372801469998, 41.661479642000025], [-87.53763279399966, 41.661408079000054], [-87.5376645049998, 41.661391459000285], [-87.53768423999983, 41.661381422000076], [-87.53773779077751, 41.6613569273241], [-87.53757322099962, 41.66139988200018], [-87.5375204489997, 41.66141204200012], [-87.53746238499977, 41.661421628000205], [-87.53740381899964, 41.66142739600014], [-87.53734499999966, 41.661429322000124], [-87.53728618199986, 41.66142739600014], [-87.53722761499962, 41.661421628000205], [-87.5372091799996, 41.66141900600034], [-87.53684917999983, 41.661364007000145], [-87.53680955099978, 41.66135704200019], [-87.5367522379998, 41.66134367800015], [-87.53669592199978, 41.66132659500007], [-87.53664084399983, 41.66130586500013], [-87.53658723999975, 41.661281577000075], [-87.53653533899963, 41.66125383600018], [-87.53648536299984, 41.66122275900018], [-87.53643752699972, 41.6611884800001], [-87.53639203599968, 41.66115114600002], [-87.53634908299985, 41.661110917000165], [-87.53630885399971, 41.661067964000246], [-87.53627151999976, 41.6610224720004], [-87.53624632399962, 41.66098797000003], [-87.53599632399981, 41.66062797000008], [-87.53598724099966, 41.66061463700014], [-87.53595616399976, 41.66056466100009], [-87.53592842299979, 41.660512760000145], [-87.53590413499964, 41.66045915600033], [-87.53589526199978, 41.660436900000086], [-87.53575726199992, 41.66007690000014], [-87.53574540499991, 41.66004407800027], [-87.53572832199966, 41.65998776200022], [-87.53571495799972, 41.659930449000015], [-87.53571020499984, 41.659904580000095], [-87.5356633969998, 41.65962707400023], [-87.53563331699986, 41.65952621900014], [-87.53557284299958, 41.65952262400011], [-87.53486306299983, 41.659610561000186], [-87.53464255099979, 41.65963762500012], [-87.53459181899977, 41.65964239700016], [-87.53454763699985, 41.659644203000134], [-87.53415718699988, 41.65965055900013], [-87.53367052699966, 41.659701446000156], [-87.53363581899973, 41.659704397000134], [-87.53357699999974, 41.659706322000034], [-87.53351818199965, 41.659704397000134], [-87.5334596149997, 41.65969862900037], [-87.53340256399979, 41.65968924300012], [-87.53287605749969, 41.65958514250017], [-87.53281823899984, 41.659571678000084], [-87.5328125379999, 41.65957013100022], [-87.53257445199992, 41.65950465700013], [-87.53253936799977, 41.65949517500006], [-87.53248492299971, 41.65947859500011], [-87.53242984499975, 41.659457864999986], [-87.53240774699972, 41.659448364000184], [-87.53135239999978, 41.658977777000054], [-87.53083017999974, 41.658756486000144], [-87.53094688699981, 41.65914822900031], [-87.53095367899974, 41.65917223800007], [-87.5309597999998, 41.659196420000114], [-87.53102679899959, 41.6594774200001], [-87.53103404199983, 41.65951055100008], [-87.53104362799965, 41.65956861500045], [-87.53104939599976, 41.65962718200006], [-87.53105088699981, 41.65965804800047], [-87.53106988699969, 41.66026904700014], [-87.5310703009998, 41.66029089000044], [-87.53108030199975, 41.66176289000002], [-87.53108032199982, 41.66176900000017], [-87.53108027899984, 41.66177780800019]], [[-87.52673497299985, 41.659141223000084], [-87.5279325639999, 41.658599966000146], [-87.52805675999977, 41.658543450000096], [-87.52832895599974, 41.65837439900041], [-87.5283061759998, 41.65833117200047], [-87.52809372999982, 41.65805474550011], [-87.5280592409998, 41.65800663600008], [-87.52805562999978, 41.65800119000017], [-87.52785962999981, 41.65770319100012], [-87.52783216399975, 41.657658661000156], [-87.52780442199978, 41.65760676000004], [-87.5277801349998, 41.65755315600004], [-87.52775940499978, 41.6574980780001], [-87.52774232099986, 41.65744176200007], [-87.52772895799976, 41.65738444900039], [-87.5277265189998, 41.65737171100004], [-87.52771618199993, 41.657315515000015], [-87.52739371699981, 41.65676598400011], [-87.52701086099974, 41.65615341500014], [-87.52643711599978, 41.655239564000105], [-87.52623045199974, 41.65493298200014], [-87.52624111299984, 41.655138385], [-87.52624232199966, 41.655185], [-87.52624224199984, 41.655197013000226], [-87.52623524999989, 41.65572035900037], [-87.52623225799972, 41.65597170600011], [-87.5262310069997, 41.65600960400023], [-87.52621303099973, 41.65634174300009], [-87.52624010799967, 41.65672693700016], [-87.52624039699977, 41.656731181000154], [-87.52624224599982, 41.656778307000145], [-87.52625224599967, 41.657547307000186], [-87.5262503959999, 41.65761781900016], [-87.52624462799977, 41.657676385000386], [-87.52623504199967, 41.65773444900021], [-87.52622167799981, 41.65779176200043], [-87.52620459499983, 41.65784807800046], [-87.52619040099962, 41.657886932000125], [-87.52622143199979, 41.65858400500052], [-87.52622039699972, 41.65868281800025], [-87.52621462799969, 41.65874138500004], [-87.52620504199967, 41.65879944900023], [-87.52620363999993, 41.65880635500008], [-87.5261626399999, 41.6590043550003], [-87.52615067899981, 41.65905476200048], [-87.52614656599981, 41.6590695990002], [-87.52610346999981, 41.65922008300004], [-87.52608932199969, 41.65932459400007], [-87.52608932199969, 41.65937900500021], [-87.52673497299985, 41.659141223000084]], [[-87.53103544099977, 41.66589517300007], [-87.53104150199972, 41.66592706100004], [-87.53104962799988, 41.66597761500013], [-87.53105539699992, 41.66603618100036], [-87.53105732199971, 41.66609500000017], [-87.53105539699992, 41.666153818000055], [-87.53105073599968, 41.666203641000365], [-87.53102773599984, 41.6663926400001], [-87.53102662799992, 41.6664013850002], [-87.53101704199968, 41.666459449000214], [-87.53101119699961, 41.666486631000154], [-87.53090319699984, 41.66695363100006], [-87.53089567899983, 41.666983761000246], [-87.53089018499982, 41.66700337200019], [-87.53086686499984, 41.66708313300012], [-87.53391593699979, 41.66794239300017], [-87.53392103999977, 41.66794384800038], [-87.53484480399975, 41.6682100680002], [-87.53511530099965, 41.66828854000012], [-87.53539717599978, 41.66835409200004], [-87.53554409999977, 41.66803222500032], [-87.53562428399975, 41.66775856900005], [-87.53571371499979, 41.66731878700044], [-87.53572632199975, 41.667265238000184], [-87.53574340499993, 41.66720892200014], [-87.53576413499977, 41.667153844000026], [-87.53577677999985, 41.667124775000374], [-87.53588078099993, 41.66689677600042], [-87.53589242299974, 41.66687224000052], [-87.53592016499971, 41.66682033900041], [-87.53593967599974, 41.66678812100041], [-87.53603867599983, 41.66663212100028], [-87.53605024099973, 41.666614364000154], [-87.53608451999973, 41.666566528000104], [-87.53612185399979, 41.66652103600006], [-87.53616208399976, 41.66647808300051], [-87.53619628299982, 41.666445631000315], [-87.53631135299975, 41.666342037000085], [-87.53632176399982, 41.666215992000055], [-87.5363215089998, 41.66620565000026], [-87.53626506199983, 41.66613568400016], [-87.53625151999968, 41.66611847300042], [-87.53622845399966, 41.666087024000035], [-87.53611145499981, 41.66592002500016], [-87.53610024099962, 41.665903637000056], [-87.53606916399971, 41.665853661000185], [-87.53604142299989, 41.66580176000006], [-87.53601713499994, 41.66574815600007], [-87.53599640499972, 41.66569307800013], [-87.53597932199973, 41.66563676200009], [-87.53596595799971, 41.665579449000056], [-87.53595637199977, 41.66552138600014], [-87.53595060299993, 41.66546281900035], [-87.53594873499978, 41.6654141050004], [-87.53594678899978, 41.66524090800019], [-87.53379490899974, 41.66485413800017], [-87.53377855099981, 41.66485104200045], [-87.53376358399971, 41.66484793200037], [-87.53233374499982, 41.6645381660005], [-87.53163815599973, 41.6644270650001], [-87.5316042344999, 41.66442097900013], [-87.53105634899971, 41.664311589000135], [-87.53103544099977, 41.66589517300007]]]}, "properties": {"city": "Chicago", "name": "Wolf Lake", "regionid": "403387", "geo_point_2d": [41.67207435775845, -87.53154385195313], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.71834538244588, 41.866166232635955], [-87.71938, 41.86616], [-87.71940758418471, 41.866975783203955], [-87.71942710402111, 41.86758882197401], [-87.71946209193572, 41.86856576937068], [-87.71947757371959, 41.86935886704489], [-87.7195094164054, 41.87039543643449], [-87.71953995128617, 41.87131188815621], [-87.71957220042464, 41.872165108707044], [-87.71961018975294, 41.873142422506255], [-87.71962664098284, 41.87374834654228], [-87.71907783616008, 41.87375244320288], [-87.71779626110475, 41.87375811388894], [-87.71614803604673, 41.873800196182756], [-87.7154153306637, 41.87381144238159], [-87.71413393326172, 41.8738275605879], [-87.71309661155532, 41.87384060865959], [-87.71218132769665, 41.87385212166428], [-87.71120425738056, 41.87386441185688], [-87.71053216307075, 41.87387647877518], [-87.70998334963942, 41.87389208484442], [-87.7090076813172, 41.87391982896734], [-87.7083328674629, 41.87395032564611], [-87.70765516857576, 41.8740182762581], [-87.70709923607208, 41.874075634373604], [-87.70613430428382, 41.8741988829516], [-87.70590958338967, 41.87423135942334], [-87.70588214500485, 41.873144350146084], [-87.70586467987631, 41.87265763715449], [-87.70584337744822, 41.87216768131245], [-87.70581645296426, 41.871250383392876], [-87.7057976130315, 41.87039639293541], [-87.70576547005169, 41.86948581119021], [-87.7057731052398, 41.868219834807995], [-87.70566647469951, 41.86728244339107], [-87.7056436383141, 41.86661193716346], [-87.70564, 41.86639], [-87.70638419394346, 41.86638732009318], [-87.70754348975595, 41.86637448798571], [-87.70888588380531, 41.86635690193647], [-87.709923549473, 41.86635], [-87.7110194373308, 41.866318193121295], [-87.71199638223612, 41.8662663552283], [-87.71443984946922, 41.8662070260374], [-87.71602648432736, 41.86619386263069], [-87.71724686632034, 41.86617769323046], [-87.71834538244588, 41.866166232635955]]]}, "properties": {"city": "Chicago", "name": "Homan Square", "regionid": "403327", "geo_point_2d": [41.870072771870674, -87.71263333583163], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.71888391510932, 41.94640002205088], [-87.7190489155206, 41.94639602186661], [-87.71918491549445, 41.94639802195879], [-87.71964491511515, 41.94637902198226], [-87.7199209161467, 41.946372022559], [-87.72050991541865, 41.94635602272066], [-87.72237491595308, 41.9463500224442], [-87.72350491588158, 41.94633702184485], [-87.72478791476878, 41.94632902237541], [-87.72593391633417, 41.94631802186833], [-87.72609791580018, 41.94631602177615], [-87.7272550473749, 41.94629814954933], [-87.72726407983758, 41.946727088407364], [-87.7272885436967, 41.94745491930727], [-87.7273174922926, 41.94842588389822], [-87.7273535024692, 41.9495605567952], [-87.72737932206083, 41.950207232410925], [-87.7274195172052, 41.95181829934554], [-87.72484068723446, 41.95039471114532], [-87.72295847791246, 41.94931813748373], [-87.71980970419924, 41.94774573057398], [-87.71891317345103, 41.947083173451134], [-87.71869341534757, 41.94683172215135], [-87.71839757305814, 41.94640179043019], [-87.71855891541628, 41.946401022096964], [-87.71867991559797, 41.946401022096964], [-87.71883891573283, 41.94640002205088], [-87.71888391510932, 41.94640002205088]]]}, "properties": {"city": "Chicago", "name": "The Villa", "regionid": "403289", "geo_point_2d": [41.94818659176937, -87.7241308492728], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.74544218073902, 41.92417], [-87.74626, 41.92416], [-87.74626, 41.92489409869122], [-87.74629714518608, 41.925873617425886], [-87.74632602858355, 41.92648253356046], [-87.74638091858579, 41.92752123191579], [-87.7464, 41.92831169908262], [-87.7464335034689, 41.929046078631266], [-87.74648245447497, 41.93014628981191], [-87.74651946223183, 41.93099790021827], [-87.74656315268832, 41.931975147317615], [-87.74658448355562, 41.9327669706852], [-87.74660990926856, 41.93374289049567], [-87.7466428756976, 41.93459627092742], [-87.74668106797539, 41.93557876245315], [-87.7467264127928, 41.93661409743223], [-87.74676440515059, 41.937466913314054], [-87.74679464250171, 41.938259036722265], [-87.74680984130339, 41.93872513330811], [-87.74552591563983, 41.938760022407365], [-87.74511991581033, 41.93876702183055], [-87.74429791569054, 41.93878002242998], [-87.7437889156097, 41.93878802189952], [-87.74312391553248, 41.93879802236042], [-87.74208091062114, 41.938813022152495], [-87.74200391516554, 41.93697402281649], [-87.74199991588044, 41.93687702284168], [-87.74192791525952, 41.93517602267243], [-87.74184891521516, 41.933196022232735], [-87.74178391581623, 41.93150702261665], [-87.74175091609362, 41.93043202252548], [-87.74174991604752, 41.930401022895246], [-87.7417219156562, 41.929440022663144], [-87.74172091561002, 41.92940302275618], [-87.74169391526475, 41.928525022753], [-87.7416889159336, 41.928367022664155], [-87.74168191561097, 41.928174021861444], [-87.74163791628064, 41.927146022138324], [-87.7416359152892, 41.92709102230092], [-87.74160891584313, 41.92619902255157], [-87.74157491607463, 41.9251340229214], [-87.7415479157292, 41.924209022550166], [-87.74208615177515, 41.924205726521286], [-87.74312336778945, 41.92419533847336], [-87.74452701529606, 41.92417754974508], [-87.74544218073902, 41.92417]]]}, "properties": {"city": "Chicago", "name": "Kelvin Park", "regionid": "403315", "geo_point_2d": [41.93149124770993, -87.7441620838778], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.64413826099994, 41.969792920000025], [-87.64417701, 41.96978932200004], [-87.64930008899992, 41.96920480600004], [-87.65053779099995, 41.96913669600007], [-87.6522558599999, 41.96912372900003], [-87.65397415399995, 41.96909623000004], [-87.65481, 41.96908], [-87.65481, 41.96961788200007], [-87.65485680799995, 41.97099566100007], [-87.65489983599994, 41.97271179500006], [-87.65492725899996, 41.973672927000045], [-87.65495094399994, 41.97484079700006], [-87.65497641299997, 41.97587088100004], [-87.65499039199993, 41.97635499900007], [-87.65345591499991, 41.97638002300004], [-87.65244091499993, 41.97639902300004], [-87.65215491599996, 41.976402023000034], [-87.65179491499997, 41.97642002300005], [-87.65107691499996, 41.97645802400006], [-87.65085491499995, 41.97649002200006], [-87.65059091599993, 41.97653802300003], [-87.65042391599994, 41.97658102300004], [-87.65024091599996, 41.97662702200005], [-87.65016791599997, 41.976649022000046], [-87.65007191499993, 41.97667802200004], [-87.64982577999993, 41.97673688500003], [-87.64966904599997, 41.977377161000035], [-87.64881891599995, 41.97752002100002], [-87.647077, 41.977618], [-87.64687318999994, 41.97762946400007], [-87.64685653299993, 41.97762946100004], [-87.64681016399996, 41.97749035300006], [-87.64655702799995, 41.97673094600002], [-87.64630573099997, 41.97467031300004], [-87.64605702799996, 41.97263094600003], [-87.64455702799992, 41.97033094600004], [-87.64417903799993, 41.96984537500002], [-87.64417375199996, 41.96983858400005], [-87.64417038999991, 41.969834265000024], [-87.64416702799994, 41.969829946000054], [-87.64413826099994, 41.969792920000025]]]}, "properties": {"city": "Chicago", "name": "Margate Park", "regionid": "403266", "geo_point_2d": [41.9729270841255, -87.6502444143752], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.6928835919999, 41.70719910800004], [-87.69255135599988, 41.70718409100003], [-87.69365, 41.70717589300005], [-87.6939239149999, 41.70717589300005], [-87.69427291599992, 41.70717589300005], [-87.6942788499999, 41.70717589300005], [-87.69565291499991, 41.70714602300006], [-87.69615991399989, 41.707142021000024], [-87.69641892299987, 41.70708004200008], [-87.6963879159999, 41.70629302200006], [-87.69638591599993, 41.70624102200007], [-87.69638491599993, 41.706188022000035], [-87.6963679159999, 41.70515402200003], [-87.6963279159999, 41.704209022000036], [-87.69627791499991, 41.70257702200006], [-87.69624091499989, 41.70137302300008], [-87.69619291499991, 41.700428022000025], [-87.69614300499988, 41.699008004000056], [-87.6960491819999, 41.69900700200007], [-87.695993, 41.697221], [-87.6959329999999, 41.695316], [-87.69584899999988, 41.692633], [-87.695749, 41.691733], [-87.69584839199987, 41.691732909000045], [-87.69584491499988, 41.691634022000066], [-87.69584291499989, 41.69159102200006], [-87.69584191599992, 41.69155802300008], [-87.69572294999993, 41.68815070800007], [-87.69605009299991, 41.688232371000076], [-87.6965500129999, 41.68823247400007], [-87.69715001299991, 41.68823247400007], [-87.69835002799992, 41.68823242800004], [-87.6994419669999, 41.68813320600008], [-87.69944937299991, 41.688132326000066], [-87.69944195899988, 41.68807609200006], [-87.69941783499989, 41.68790722400006], [-87.69938999199988, 41.68771232500006], [-87.69936030699994, 41.68750453200004], [-87.69935001299987, 41.68743247400004], [-87.69925477299994, 41.68667055200006], [-87.6992497919999, 41.68663219300003], [-87.69925477299994, 41.68661343400003], [-87.6993480769999, 41.68624021600004], [-87.69935042399992, 41.68623279200005], [-87.70051900399993, 41.68623247400005], [-87.70054952199989, 41.68623225800007], [-87.70055001299993, 41.68617536100003], [-87.70055001299993, 41.68616960900004], [-87.70055001299993, 41.686112496000064], [-87.70055001299993, 41.686055383000046], [-87.70055001299993, 41.685905907000055], [-87.70055001299993, 41.68584879400002], [-87.70055001299993, 41.68579168100006], [-87.70055001299993, 41.68568958700006], [-87.70055033999992, 41.68563261700005], [-87.7005459139999, 41.68557918900006], [-87.7005401259999, 41.68550394000004], [-87.70045439299989, 41.68438941400005], [-87.7004503509999, 41.68433259100003], [-87.70045438499992, 41.68433230100003], [-87.70051690399991, 41.68432982100005], [-87.70173601299989, 41.68428147300004], [-87.7029500049999, 41.68423249400006], [-87.70535034599992, 41.68423226500004], [-87.70535069299989, 41.68421658500005], [-87.70535031399992, 41.68417537100003], [-87.70535012099991, 41.684159478000076], [-87.70534942299989, 41.68410237300007], [-87.7053397109999, 41.68330757700004], [-87.70533951799987, 41.68325060900007], [-87.7053053399999, 41.682973516000054], [-87.70530270599991, 41.68295429100004], [-87.70530532899993, 41.68291639800008], [-87.70532727699991, 41.68255546400007], [-87.7053369749999, 41.68239599800005], [-87.70533931999994, 41.68236265600006], [-87.7053369749999, 41.682338879000035], [-87.7053111079999, 41.682039534000054], [-87.7053050399999, 41.68196930500004], [-87.70525326399988, 41.68137009000007], [-87.70524974999994, 41.68133223100005], [-87.70525326399988, 41.68131297200005], [-87.70528372499987, 41.68113020100008], [-87.7053504709999, 41.68073271900004], [-87.70765099799988, 41.68053251800006], [-87.71014965899988, 41.68053226500006], [-87.71130079099991, 41.68049532300005], [-87.71129988699992, 41.680552637000055], [-87.71126060199991, 41.68127451200006], [-87.71125155399992, 41.68137589300004], [-87.7112535819999, 41.681431946000075], [-87.71175104599989, 41.681431946000075], [-87.71176126099992, 41.681498447000074], [-87.71183726699988, 41.68187847600005], [-87.71185048099991, 41.68193220400007], [-87.71183994899988, 41.681933959000034], [-87.71125202799992, 41.68203259400007], [-87.71125202799992, 41.68208948500005], [-87.71125202799992, 41.68227525900004], [-87.7112520249999, 41.682332384000055], [-87.71125163999992, 41.68238948800006], [-87.71124640999992, 41.68316626800002], [-87.7112460299999, 41.68322337200003], [-87.71124634499992, 41.683280477000025], [-87.7112504059999, 41.68401622800007], [-87.71125072099993, 41.68407334500005], [-87.71125102799992, 41.68412894600004], [-87.71091136199992, 41.684086639000036], [-87.71025, 41.6841], [-87.7102600299999, 41.68450120600004], [-87.71032226099992, 41.68645330100007], [-87.71035871199989, 41.68747394200005], [-87.71054999999988, 41.68859], [-87.71304901599989, 41.68855], [-87.71435200699993, 41.68852622800005], [-87.71657936099987, 41.688403119000036], [-87.72017619599988, 41.68840041600003], [-87.72010310599988, 41.686452369000044], [-87.72004391199992, 41.684964493000045], [-87.7200065179999, 41.684132474000066], [-87.72015101299989, 41.684132474000066], [-87.7212510129999, 41.684132474000066], [-87.72235101299991, 41.684132474000066], [-87.72375101399987, 41.68413247200004], [-87.72451802399989, 41.68412944200008], [-87.7248460099999, 41.684107483000034], [-87.72623601299992, 41.68403847300004], [-87.7283510109999, 41.68393247800008], [-87.72966401299992, 41.68387947400003], [-87.73085100499989, 41.68383249300006], [-87.73745202199987, 41.68383245000006], [-87.73796301299988, 41.68380847300005], [-87.73950282099989, 41.68373482700007], [-87.73955205899992, 41.68373235100005], [-87.73955355499992, 41.68378954600006], [-87.73974721599987, 41.69096041000005], [-87.73974795399992, 41.69101731000005], [-87.73864199599988, 41.690998521000026], [-87.7373420259999, 41.691098435000065], [-87.73495201299993, 41.69109747400005], [-87.7327235409999, 41.691097919000065], [-87.7303744899999, 41.69109786000006], [-87.72994602799989, 41.69109799300008], [-87.72961031999994, 41.69111362800003], [-87.72795464599993, 41.69119123400007], [-87.72588801399989, 41.69128747100006], [-87.7256510209999, 41.691297453000054], [-87.72474850399992, 41.69129768900005], [-87.72474801299988, 41.69135458700003], [-87.72474801299988, 41.691375361000034], [-87.7247475219999, 41.69143225800007], [-87.72425100799988, 41.691432486000046], [-87.72035150899987, 41.69153267700005], [-87.72035101299988, 41.69158958700006], [-87.72035101299988, 41.691689569000054], [-87.72035101299988, 41.69174668200007], [-87.72034865599991, 41.691803995000036], [-87.7203520279999, 41.69317525900003], [-87.7203520279999, 41.693232378000054], [-87.7203520279999, 41.69328948500003], [-87.7203520279999, 41.69497525900004], [-87.7203520279999, 41.69503194600008], [-87.72044832299989, 41.69613375500005], [-87.72055202799991, 41.69863259400005], [-87.72055202799991, 41.69868948500003], [-87.72055202799991, 41.699475259000046], [-87.72055197499989, 41.69953247100005], [-87.72045498199991, 41.70050240500006], [-87.72045202799987, 41.70053194600006], [-87.72045132899989, 41.70056080600005], [-87.72054832299993, 41.701433755000025], [-87.72065302799987, 41.70233259400003], [-87.72065302799987, 41.70238948500007], [-87.72065302799987, 41.70317525900003], [-87.72065302799987, 41.70323237800004], [-87.72065302799987, 41.70328948500007], [-87.72065302799987, 41.70407525900003], [-87.72065302799987, 41.70413237800005], [-87.72065302799987, 41.70418948500008], [-87.72065302799987, 41.70582651400007], [-87.72065302799987, 41.705875259000024], [-87.72065302799987, 41.705883633000035], [-87.7195522829999, 41.70591], [-87.71797248099993, 41.70593828700003], [-87.71667129699992, 41.70595513300008], [-87.71555673799989, 41.70596291300006], [-87.71434818499992, 41.70597466300005], [-87.71314079499987, 41.70599743500008], [-87.71221208199992, 41.70603165200002], [-87.71105103599989, 41.706132431000064], [-87.71050705099991, 41.70608302300006], [-87.71044440799992, 41.70607670800006], [-87.7103510019999, 41.706068216000055], [-87.70995100399993, 41.70603210400008], [-87.70965662399992, 41.70607739300004], [-87.70957791699992, 41.70608994800004], [-87.7086510529999, 41.70623235900006], [-87.70735097899988, 41.70613255200003], [-87.70615102699989, 41.70623243200003], [-87.7049510339999, 41.70623243400006], [-87.70411247599989, 41.706162596000055], [-87.70399835999989, 41.70615308600008], [-87.70375101699993, 41.706132030000035], [-87.7034194759999, 41.70616013700003], [-87.70330362399991, 41.70616975600006], [-87.70255101399991, 41.706232470000074], [-87.7013332039999, 41.70632615200008], [-87.70129421099993, 41.70632870200006], [-87.70125195999991, 41.70633249200005], [-87.70115634099993, 41.707097437000066], [-87.7011510269999, 41.70713250800007], [-87.69844497799991, 41.70722938500005], [-87.6984271189999, 41.70722976300003], [-87.69835101999989, 41.70723245600003], [-87.69715101299992, 41.707232474000065], [-87.69625101299991, 41.707232474000065], [-87.69427987099992, 41.707232474000065], [-87.69427393699992, 41.707232474000065], [-87.69392493599989, 41.707232474000065], [-87.69365102299989, 41.70723245300007], [-87.6928835919999, 41.70719910800004]]]}, "properties": {"city": "Chicago", "name": "Mount Greenwood", "regionid": "269596", "geo_point_2d": [41.693950742612174, -87.71268107922974], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.7222430219999, 41.76140716100007], [-87.7222349159999, 41.76120302300005], [-87.7222219149999, 41.76084102300006], [-87.72214991499992, 41.75893602200005], [-87.72216891499993, 41.75783402300003], [-87.7221709159999, 41.75701802300005], [-87.7221709159999, 41.75680302300008], [-87.72217191599991, 41.75630202200004], [-87.72216391599993, 41.755742022000035], [-87.72216191599993, 41.75558202200005], [-87.7221440459999, 41.75500488900007], [-87.72206678699987, 41.755005154000045], [-87.72206391499991, 41.75487602300005], [-87.72205886399988, 41.754792892000054], [-87.7221369679999, 41.75487714900003], [-87.72204304099992, 41.753799879000034], [-87.72200078799993, 41.75380015900003], [-87.72199691599988, 41.75373802300003], [-87.7219649159999, 41.75331202300003], [-87.72196294199993, 41.75328775600007], [-87.72195978199991, 41.75324889200003], [-87.72202804599992, 41.75324889200003], [-87.72202778299992, 41.75331215600004], [-87.7244509149999, 41.75327102200003], [-87.72534391499993, 41.75326202200005], [-87.72577591499991, 41.753264022000046], [-87.72643991499987, 41.753248023000076], [-87.72678491599991, 41.75324002200005], [-87.7275949159999, 41.75322202200004], [-87.7291779159999, 41.75319602200005], [-87.72996691599988, 41.753182022000026], [-87.73038991599991, 41.75317502200004], [-87.7316999159999, 41.75314702200006], [-87.73224391599989, 41.75313602200003], [-87.73352291499991, 41.75311702200007], [-87.73670405199992, 41.753117144000036], [-87.7366957769999, 41.75304590100006], [-87.73724891499987, 41.75303202300005], [-87.73820003399992, 41.753050169000055], [-87.73821381999991, 41.75298486300004], [-87.73843791599987, 41.75304802100003], [-87.73893291499989, 41.75311502300007], [-87.7394429159999, 41.75314202200007], [-87.74014591599988, 41.753194023000034], [-87.7412909149999, 41.75319302300004], [-87.74133791499989, 41.75319602200005], [-87.74138200499992, 41.75319800100004], [-87.7414540279999, 41.753201735000054], [-87.7414540279999, 41.75325884200003], [-87.7414540279999, 41.75347425900008], [-87.7414540279999, 41.75353094600006], [-87.74152583599992, 41.75410936200006], [-87.7415266339999, 41.754126645000035], [-87.7415530129999, 41.754331474000075], [-87.7416509149999, 41.754625178000076], [-87.74165332699991, 41.75463164400008], [-87.74165091699989, 41.75468229800003], [-87.7415917249999, 41.75611724000003], [-87.74155502799992, 41.757031378000065], [-87.74155502799992, 41.75708848500004], [-87.74155502799992, 41.75857375100008], [-87.74155502799992, 41.75863087000004], [-87.74155502799992, 41.75868797700007], [-87.74155502799992, 41.75927425900004], [-87.74155502799992, 41.759331378000056], [-87.74155502799992, 41.75938848500003], [-87.74155502799992, 41.75997425900003], [-87.74155502799992, 41.76003094600007], [-87.74164561599989, 41.76050422000008], [-87.74195502799994, 41.76203159400005], [-87.74195502799994, 41.76208848500004], [-87.74195502799994, 41.76247425900004], [-87.7419550319999, 41.762531140000064], [-87.74195630399991, 41.76258821100002], [-87.74198505099989, 41.76404666400003], [-87.74198632299992, 41.76410375500005], [-87.74198758999992, 41.764160834000045], [-87.74205005599991, 41.76697566400007], [-87.7420550279999, 41.76703159400005], [-87.7420550279999, 41.76708848500004], [-87.7420550279999, 41.767274259000075], [-87.7420550279999, 41.767330946000065], [-87.74215502799987, 41.76763159400007], [-87.74215502799987, 41.76768848500005], [-87.74215502799987, 41.768002261000035], [-87.74215502799987, 41.768059380000054], [-87.73826115299993, 41.76669785600005], [-87.72849658099993, 41.76376848400007], [-87.72288195199988, 41.761978312000046], [-87.7222430219999, 41.76140716100007]]]}, "properties": {"city": "Chicago", "name": "Ford City", "regionid": "403338", "geo_point_2d": [41.75917134196632, -87.73284854835086], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.76631013826386, 41.994990195253145], [-87.76568, 41.9947], [-87.76475, 41.9943], [-87.76497843905553, 41.9936835771517], [-87.76546874594129, 41.9923736978496], [-87.7663937871602, 41.99276475384758], [-87.76891909229045, 41.99395010115332], [-87.76969214488128, 41.99420778535042], [-87.77056827115086, 41.994362395868585], [-87.77120096922789, 41.99482859445178], [-87.77107779128113, 41.99503997631367], [-87.77102986281318, 41.99548002177711], [-87.77109454072091, 41.995929794658714], [-87.77020659246892, 41.99555787204254], [-87.7699430204735, 41.99545772862303], [-87.76945329321119, 41.995202400734065], [-87.7688, 41.99488], [-87.76863241400508, 41.99533124078541], [-87.7688160186117, 41.995702353692735], [-87.7688160186117, 41.99596003788966], [-87.76799142918128, 41.99626925892599], [-87.7676306713057, 41.99611464840782], [-87.76670300819654, 41.99575389053208], [-87.76608456612388, 41.99544466949574], [-87.76631013826386, 41.994990195253145]]]}, "properties": {"city": "Chicago", "name": "Old Edgebrook", "regionid": "403274", "geo_point_2d": [41.99447504255374, -87.76763859366996], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.62817323885213, 41.72171268416431], [-87.62687087028394, 41.72162930728327], [-87.6257563788614, 41.7216462002709], [-87.62445597739587, 41.72164054760625], [-87.62445593676536, 41.72164], [-87.62444662072116, 41.72164050693407], [-87.62431753570249, 41.72164753111674], [-87.6236963244985, 41.72168133442729], [-87.6236937498777, 41.72147549838556], [-87.62365875985358, 41.72101363006882], [-87.62359577781052, 41.72018226709853], [-87.62351663145326, 41.719460630856204], [-87.62329538918465, 41.71863308971731], [-87.62304701657399, 41.717938661281835], [-87.62281786742928, 41.71730437776831], [-87.62263519572822, 41.71654418744304], [-87.62268053001804, 41.71616156660616], [-87.62351446236424, 41.71472805339207], [-87.62431281660412, 41.71501650940119], [-87.62466769024145, 41.71510363734694], [-87.62524892291437, 41.715199338437195], [-87.62618429230155, 41.71526330336565], [-87.6271956957907, 41.71517804175419], [-87.62794357234088, 41.71503927230085], [-87.62924939372498, 41.714878876307296], [-87.6307462528778, 41.7149984040104], [-87.63287371570955, 41.71491852959061], [-87.6333430242796, 41.71491438922044], [-87.63334291548858, 41.71490902246237], [-87.63333891530432, 41.714782022903414], [-87.63333805194267, 41.7147454645215], [-87.63528957262851, 41.71489626220037], [-87.64216393298024, 41.714822058369386], [-87.64848090872984, 41.71470087567102], [-87.65115058268843, 41.7142537748892], [-87.65247500764846, 41.71412593702197], [-87.65332, 41.71409], [-87.65352667106215, 41.71355535094794], [-87.65400518010163, 41.71228587244997], [-87.65458700707003, 41.71069079061696], [-87.65506833747729, 41.70936256240439], [-87.65540300852088, 41.708452362985255], [-87.65580655041224, 41.70735063065625], [-87.65602, 41.70679], [-87.65602691477626, 41.70678596607306], [-87.65617101180203, 41.70677885544063], [-87.65618691631228, 41.706809023197984], [-87.65625391580329, 41.706956022779565], [-87.65658891595722, 41.70758402205064], [-87.6567449159538, 41.70789902218186], [-87.65701491580931, 41.70844502216966], [-87.65751891565964, 41.709359022033894], [-87.6579259155352, 41.71016702240771], [-87.65832291584908, 41.71097102169778], [-87.65844191503932, 41.711158024022524], [-87.65850991547596, 41.71133802242733], [-87.65883691616028, 41.71210802194889], [-87.65931891589578, 41.71304502287321], [-87.65984391581486, 41.71402902236683], [-87.66077591560946, 41.71583302278752], [-87.6612539151605, 41.716787022696806], [-87.66155591559192, 41.717370022591396], [-87.6621479159014, 41.718529022957476], [-87.66259991515331, 41.71945402242933], [-87.66316391597083, 41.72047002249865], [-87.66357891531565, 41.7211330224837], [-87.66361591522272, 41.72119202250546], [-87.6636317154838, 41.72121738556112], [-87.66213750004442, 41.7212784374992], [-87.66092960108803, 41.7211666714157], [-87.65990781921788, 41.721172815091315], [-87.65851566364502, 41.72133498206361], [-87.65730725183876, 41.72135402290131], [-87.65600505891778, 41.72122966844764], [-87.65489052236825, 41.72124802422339], [-87.65368325579595, 41.721263907739406], [-87.65238262940643, 41.72127913722285], [-87.6512680100482, 41.72129430466616], [-87.65006022964218, 41.72143], [-87.64875886744473, 41.72133861144609], [-87.64755123061153, 41.721365199028185], [-87.64634416248259, 41.72139320219917], [-87.645043328961, 41.72139523805846], [-87.64392857783714, 41.72139], [-87.64262740009958, 41.72153560178657], [-87.64132834295782, 41.721541194284065], [-87.63984041097592, 41.72142780153678], [-87.63891118711416, 41.721440626014264], [-87.63779663628758, 41.72145833380807], [-87.63640391660293, 41.72149367663256], [-87.63510257674547, 41.72149309894657], [-87.63435971191075, 41.721494813773255], [-87.63351211086355, 41.72150603721037], [-87.63290775929937, 41.721514010465135], [-87.63142437148875, 41.72169251366331], [-87.6305875886847, 41.7217], [-87.62928693524714, 41.72171], [-87.62817323885213, 41.72171268416431]]]}, "properties": {"city": "Chicago", "name": "Longwood Manor", "regionid": "403376", "geo_point_2d": [41.71753028813542, -87.64397000133886], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.69623550199996, 41.94695507300002], [-87.69640954, 41.94766461500006], [-87.69655680299995, 41.94853480700003], [-87.69666390399996, 41.94907030900002], [-87.69671745399994, 41.949525487000074], [-87.69670406699997, 41.949820013000064], [-87.69666390399996, 41.95016809000003], [-87.69663712899992, 41.95038229100003], [-87.69655680299995, 41.950609880000044], [-87.69638276499994, 41.95086424400005], [-87.69618195099996, 41.951158770000056], [-87.69596774999995, 41.95158717300006], [-87.69571338599994, 41.95201557500007], [-87.69544563499991, 41.95241720200004], [-87.69515110899992, 41.95284560400005], [-87.69488335699997, 41.953193681000066], [-87.69465576799995, 41.95352837000006], [-87.69452189299992, 41.95376934700003], [-87.69444156699994, 41.95387644700002], [-87.69441403699992, 41.95391621300006], [-87.69464117299997, 41.953913135000036], [-87.69627755799993, 41.95389631600005], [-87.69823, 41.95388], [-87.69827757299997, 41.95498503900006], [-87.69833295999993, 41.956769785000056], [-87.6983942809999, 41.95862985400002], [-87.69845202799995, 41.96032814100004], [-87.69852067799994, 41.96124080800006], [-87.69725391499996, 41.961249022000054], [-87.694886, 41.96126], [-87.69470771099995, 41.96126155000007], [-87.694541, 41.961263], [-87.69359691599996, 41.96127302200004], [-87.69237591499993, 41.96128302200002], [-87.69182291599996, 41.961289022000074], [-87.69116075699995, 41.96128604400006], [-87.69115591599993, 41.96128602200002], [-87.68992191599993, 41.96129502200005], [-87.68877991499993, 41.961298023000026], [-87.68870691499994, 41.96130002300002], [-87.68870275099991, 41.961300206000026], [-87.68863891599993, 41.96130302300001], [-87.68833791499996, 41.96131402200005], [-87.68749391499993, 41.96132402200003], [-87.68627091499995, 41.96133702300005], [-87.68506991599995, 41.96135402200007], [-87.68385291499997, 41.96138102200006], [-87.68222591599992, 41.961406022000055], [-87.68142491599995, 41.96142002200001], [-87.68060991499998, 41.96143902200003], [-87.67897091599994, 41.961451022000055], [-87.67746791499994, 41.96146202200002], [-87.67599191599997, 41.96148902200007], [-87.67523791499995, 41.961501022000036], [-87.67516391599996, 41.96150302200003], [-87.67451391599994, 41.96152002200005], [-87.67418991499991, 41.961527022000034], [-87.67396091499992, 41.961532021000025], [-87.67391891499993, 41.959695022000055], [-87.67386391499996, 41.95787302300004], [-87.67382791599991, 41.95606602200006], [-87.67378382499996, 41.95427402000006], [-87.67378791599997, 41.954216023000065], [-87.67379143499993, 41.95416410700005], [-87.67379191599996, 41.95415702300004], [-87.67373791599991, 41.95240202300003], [-87.67370491499996, 41.950577022000054], [-87.67366791499994, 41.94874802200007], [-87.67364991599997, 41.94863902200006], [-87.67361585299992, 41.947155486000035], [-87.67371616799993, 41.94694017000005], [-87.67372245199992, 41.94692668200001], [-87.68830469899993, 41.946722724000026], [-87.69619224899995, 41.94669555700004], [-87.69623550199996, 41.94695507300002]]]}, "properties": {"city": "Chicago", "name": "North Center", "regionid": "269600", "geo_point_2d": [41.95421339118978, -87.68553307687955], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.6062299999999, 41.65913841700006], [-87.60629380299989, 41.65830369800005], [-87.60631, 41.65734], [-87.60513, 41.65729], [-87.60512132099988, 41.656908123000036], [-87.60510865899992, 41.656350988000035], [-87.60510177299993, 41.65570123000002], [-87.60511526899987, 41.65533010600006], [-87.6050558509999, 41.654591074000045], [-87.60498064499988, 41.654031853000035], [-87.6048791239999, 41.65347846400004], [-87.60478447599989, 41.65308492500007], [-87.60474, 41.6529], [-87.60479397399988, 41.652534131000024], [-87.60492142399991, 41.65250625100003], [-87.6050627199999, 41.65241728000006], [-87.60519999999988, 41.65182], [-87.60471721899992, 41.65183611100007], [-87.6046268459999, 41.651518255000035], [-87.60470320199992, 41.65019424900002], [-87.60475926599989, 41.64955002700008], [-87.60478726799992, 41.64955269400007], [-87.6059047289999, 41.64966083600006], [-87.60763499099993, 41.64966083600006], [-87.60947339399989, 41.64966083600006], [-87.61087923199993, 41.64973293000008], [-87.61187825499988, 41.64989406300002], [-87.61199669299991, 41.649913166000026], [-87.6121932859999, 41.64994785900006], [-87.61212978799992, 41.65027484100005], [-87.6120625689999, 41.651148689000074], [-87.6120625689999, 41.652728054000065], [-87.6121554729999, 41.654864842000045], [-87.61234128099993, 41.65820937900007], [-87.61243418399991, 41.65905], [-87.61122651499991, 41.65905477100005], [-87.60964711899992, 41.65915324400003], [-87.60667414799988, 41.65916], [-87.6062299999999, 41.65913841700006]]]}, "properties": {"city": "Chicago", "name": "Golden Gate", "regionid": "403380", "geo_point_2d": [41.654261390579094, -87.60864041335047], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.63930492051986, 41.75059507746363], [-87.639576044528, 41.750594887706804], [-87.63964891569263, 41.752399023025546], [-87.63966391548473, 41.75290202282993], [-87.6396748804819, 41.75423370971763], [-87.63718374400862, 41.754271171999704], [-87.63616210708734, 41.75428972183603], [-87.63504756775718, 41.75430797408668], [-87.63432, 41.75432], [-87.63431719698042, 41.75371424298667], [-87.6342978315223, 41.75315747283614], [-87.63424039068151, 41.75232546954286], [-87.63424000117476, 41.75067497859513], [-87.63544491615625, 41.75065002244243], [-87.6366649157368, 41.75063402260427], [-87.63787691494856, 41.75062602223535], [-87.63799591503808, 41.75062402214317], [-87.638681915184, 41.750615022627734], [-87.63885391501874, 41.750603022973756], [-87.63922391588673, 41.750596022651116], [-87.63930492051986, 41.75059507746363]]]}, "properties": {"city": "Chicago", "name": "Winneconna Parkway", "regionid": "403364", "geo_point_2d": [41.752454554165716, -87.63695294882403], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.64287491601209, 41.70685702271291], [-87.64282691559805, 41.70510402284445], [-87.6428219153675, 41.70412502268209], [-87.64279891610607, 41.70326602175588], [-87.64275191573813, 41.7014430222582], [-87.64269483902204, 41.699603431967795], [-87.64382591578311, 41.699593022415115], [-87.6450199150646, 41.69958202190776], [-87.64624391572879, 41.69957302239232], [-87.64745191565538, 41.69957302239232], [-87.64929891536029, 41.699590022276666], [-87.64935191600486, 41.69958902223075], [-87.64985991513988, 41.699585022046385], [-87.6507289165272, 41.69957702257686], [-87.65231591503874, 41.699555022462064], [-87.65252291558743, 41.69954902218552], [-87.6526239157466, 41.699547022093164], [-87.6528339155343, 41.69954402195489], [-87.65326791575522, 41.69953702253145], [-87.65365491560827, 41.69953102225509], [-87.65378291611259, 41.699529022162906], [-87.65386991562633, 41.699527022070725], [-87.65400091536989, 41.699525021978545], [-87.65424191568712, 41.699521021794006], [-87.65442791526775, 41.69951802255492], [-87.65474991572188, 41.69950902214029], [-87.65512791605929, 41.69950602200202], [-87.65694091509607, 41.69944802202617], [-87.65712991571442, 41.69944102260291], [-87.65805791532465, 41.69942702195764], [-87.65838991623967, 41.69942702195764], [-87.65874987707637, 41.69941996857488], [-87.65837991577858, 41.700488022302466], [-87.65798291546454, 41.70147002260345], [-87.65772891544735, 41.70220902249461], [-87.65741091607694, 41.70312102226648], [-87.65732591485686, 41.70336402267614], [-87.65725291598821, 41.703537022556695], [-87.65692091507354, 41.70433002223933], [-87.65668491588598, 41.70497602234003], [-87.65609391562256, 41.706603022695276], [-87.65604691435507, 41.706733023291875], [-87.65602538140516, 41.70678649862661], [-87.65602, 41.70679], [-87.6558065504122, 41.707350630656414], [-87.6554030085209, 41.708452362985426], [-87.65506833747723, 41.70936256240454], [-87.65458700706996, 41.71069079061712], [-87.65400518010159, 41.71228587245014], [-87.65352667106217, 41.713555350948106], [-87.65332, 41.71409], [-87.65247500764839, 41.71412593702196], [-87.65115058268839, 41.71425377488919], [-87.64848090872985, 41.71470087567118], [-87.64298430636664, 41.71480632060752], [-87.64307438881026, 41.713297640388014], [-87.64300162986768, 41.71135030510725], [-87.64295331214655, 41.7095838997093], [-87.64291170572682, 41.70781983392252], [-87.64287999999979, 41.70689000000013], [-87.64287581054027, 41.70688637860006], [-87.64287491601209, 41.70685702271291]]]}, "properties": {"city": "Chicago", "name": "Washington Heights", "regionid": "269610", "geo_point_2d": [41.706483755007255, -87.64949105898711], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.68401068199992, 41.933501878000065], [-87.68367796099992, 41.93343362700005], [-87.68332988399992, 41.933313139000056], [-87.68312906999992, 41.93283118600004], [-87.68289719999989, 41.932388524000075], [-87.68321399999991, 41.932229], [-87.68321167399992, 41.93222344500003], [-87.68394, 41.93221], [-87.68394869999989, 41.931792884000046], [-87.68402114299992, 41.93159290400006], [-87.68440906599992, 41.931249985000065], [-87.68489062899992, 41.93086103000007], [-87.6864847539999, 41.92957399300007], [-87.68746928599988, 41.928806116000075], [-87.68775, 41.92861], [-87.68772409999988, 41.92778776800008], [-87.68948889399988, 41.92878189200006], [-87.69315042899987, 41.93070917000005], [-87.69592105199987, 41.93210768800008], [-87.6955572989999, 41.93210884600005], [-87.69714275599992, 41.93298010600005], [-87.7015880259999, 41.935631923000074], [-87.70568967599988, 41.93799649100003], [-87.70806, 41.93932], [-87.70937546299989, 41.939315454000045], [-87.71047392899992, 41.939306206000076], [-87.71157132099994, 41.93929695600007], [-87.71328098699988, 41.93928311200004], [-87.71444527899989, 41.93924742500007], [-87.71535899199992, 41.93920971600004], [-87.71663864999992, 41.93915162200005], [-87.71810125099992, 41.93910604500006], [-87.71895560299991, 41.939095375000065], [-87.7202984059999, 41.93909], [-87.72176309199993, 41.93908], [-87.7228616069999, 41.93908], [-87.72414194399992, 41.93907082500004], [-87.72542470199991, 41.93904592800003], [-87.72621983499991, 41.93903339000007], [-87.72704678099987, 41.93901789000005], [-87.72707791599991, 41.93988802300002], [-87.7271079159999, 41.940793023000076], [-87.72711191599988, 41.940918022000055], [-87.72711591599992, 41.94107802200004], [-87.72711592799992, 41.94107835900006], [-87.72712091499993, 41.941222022000034], [-87.72716091499989, 41.94264802300005], [-87.72716991599991, 41.943094023000064], [-87.72718591599993, 41.943894023000034], [-87.72720291499989, 41.94446502200008], [-87.72720991599991, 41.94471102200003], [-87.72722891599993, 41.94538302200003], [-87.72723191599991, 41.94548402200006], [-87.7272550469999, 41.94629815000007], [-87.72609791599987, 41.94631602200006], [-87.72593391599989, 41.94631802200006], [-87.7247879149999, 41.94632902200003], [-87.7235049159999, 41.94633702200007], [-87.72237491599991, 41.94635002200004], [-87.72050991499991, 41.946356023000064], [-87.7199209159999, 41.94637202300003], [-87.71964491499993, 41.94637902200008], [-87.7191849149999, 41.94639802200003], [-87.71904891599988, 41.94639602200004], [-87.7188839149999, 41.94640002200003], [-87.71883891599991, 41.94640002200003], [-87.71867991599991, 41.94640102200008], [-87.71855891499989, 41.94640102200008], [-87.71839757299993, 41.946401790000024], [-87.71834891599988, 41.94640202200008], [-87.71801691599987, 41.94640302200008], [-87.71784091599993, 41.946404022000074], [-87.71763991499994, 41.94640502200007], [-87.71752691599993, 41.94640502200007], [-87.71742991599989, 41.94640702200006], [-87.7173999159999, 41.94640802200006], [-87.71631991599989, 41.94644302200003], [-87.71505491499993, 41.946532022000035], [-87.71387191499988, 41.94654102200007], [-87.71263491599987, 41.94655402200003], [-87.7114319149999, 41.94658802300005], [-87.7098169149999, 41.94659802200003], [-87.70879391599992, 41.94661602200005], [-87.7078099149999, 41.946633022000064], [-87.70658191599992, 41.94663502200006], [-87.70534591499988, 41.946637022000054], [-87.70414191499991, 41.94665002200007], [-87.70291391499988, 41.94664802200002], [-87.70225691599987, 41.94664502300008], [-87.7021359159999, 41.946644023000026], [-87.70170291499991, 41.94664702200002], [-87.70047491599989, 41.946659022000055], [-87.69923791599987, 41.94667102200008], [-87.69802091599989, 41.94667502200007], [-87.69687891499987, 41.94668702200004], [-87.69643399999987, 41.946692], [-87.6961922489999, 41.94669555700004], [-87.69616856399993, 41.946553446000046], [-87.69604807599988, 41.945924230000045], [-87.69603468799991, 41.94553599100003], [-87.69595436299993, 41.945174526000066], [-87.69574016199992, 41.94438466000007], [-87.69563306099988, 41.943755444000026], [-87.69553934799988, 41.94327349100006], [-87.69545902299988, 41.942711213000045], [-87.6953117589999, 41.94238991200005], [-87.69480303199991, 41.94176069600007], [-87.69422736599994, 41.941104705000036], [-87.69394622699988, 41.940703078000055], [-87.69370525099991, 41.94040855100007], [-87.69330362399988, 41.94010063700005], [-87.69304925999988, 41.93981949800007], [-87.69250036999992, 41.939484809000064], [-87.69212551799988, 41.93916350700003], [-87.69173727799992, 41.93898946900003], [-87.69142936399992, 41.938788655000046], [-87.69116161299989, 41.93846735400006], [-87.69089386099994, 41.93817282700006], [-87.69074659799989, 41.93789168800003], [-87.69063949699992, 41.93758377400007], [-87.69057255899989, 41.937222309000035], [-87.69042529599992, 41.936954558000075], [-87.69015754499989, 41.936726969000055], [-87.6898630179999, 41.93660648100006], [-87.68943461599989, 41.93652615600006], [-87.68903298899993, 41.93648599300006], [-87.6885644239999, 41.936365505000076], [-87.68816279699989, 41.936298567000044], [-87.6878147199999, 41.93608436600003], [-87.68741309299989, 41.935682739000065], [-87.68710517899989, 41.935281112000034], [-87.6869445279999, 41.93502674800003], [-87.68652951299991, 41.93450463300007], [-87.68628853699991, 41.93433059400006], [-87.68600739799989, 41.93406284300004], [-87.68565932099993, 41.933902192000055], [-87.68511043099993, 41.933674603000036], [-87.6846686409999, 41.93360766500006], [-87.68420007599988, 41.933540728000075], [-87.68401068199992, 41.933501878000065]]]}, "properties": {"city": "Chicago", "name": "Avondale", "regionid": "269573", "geo_point_2d": [41.94001899297501, -87.70499957193], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.60691091599992, 41.831425155000034], [-87.6062449099999, 41.831289940000026], [-87.60612491499988, 41.83133002300006], [-87.6059689399999, 41.831391154000066], [-87.60463591599994, 41.83140502200006], [-87.6028139999999, 41.831425], [-87.60189063099993, 41.83142295800007], [-87.60183290699987, 41.83126373700007], [-87.60174638299989, 41.831097780000036], [-87.60172880999991, 41.83106696400005], [-87.60145203499988, 41.83074552200008], [-87.6013531109999, 41.83064081300005], [-87.60121429799993, 41.830540224000046], [-87.60115834399993, 41.830496196000055], [-87.60110604699989, 41.83044788000006], [-87.60105773699988, 41.83039557800003], [-87.60101371599988, 41.83033961900007], [-87.60097425899991, 41.83028035200004], [-87.60093961499989, 41.83021815000005], [-87.60089861499993, 41.83013715100008], [-87.60086787399987, 41.83006964600007], [-87.60084280099991, 41.82999983800004], [-87.60082356499991, 41.82992820100007], [-87.6008102989999, 41.829855223000024], [-87.60080309099992, 41.829781399000055], [-87.60080199199987, 41.82970723300007], [-87.60080700899991, 41.82963322800002], [-87.60081400799992, 41.829569228000075], [-87.60082556899991, 41.829494910000044], [-87.6008430229999, 41.82942175100003], [-87.60086625699992, 41.829350218000044], [-87.60089512599987, 41.82928076700005], [-87.60086958599992, 41.82914900800005], [-87.6008336099999, 41.82901409800006], [-87.6008174059999, 41.82897109500003], [-87.60065786099992, 41.828669159000064], [-87.60063966699992, 41.82863275800007], [-87.60052535899989, 41.82839049200003], [-87.60032596099988, 41.82806039600006], [-87.60000535099992, 41.82759851400004], [-87.59992970299992, 41.827576498000035], [-87.59988828699993, 41.82756335200008], [-87.5994244059999, 41.82740364100003], [-87.59906952599991, 41.82727693000003], [-87.59895559499994, 41.82722711200005], [-87.59889835699991, 41.82719721100005], [-87.59880335999989, 41.82716892600006], [-87.59872834599989, 41.82714293400004], [-87.59865591599991, 41.827110428000076], [-87.59858663499992, 41.82707166000005], [-87.59854463499993, 41.82704566000006], [-87.59848446599992, 41.82700496200003], [-87.5984277779999, 41.82695954100006], [-87.59837494099989, 41.826909694000044], [-87.59832629999991, 41.82685574400005], [-87.59828217099991, 41.826798045000075], [-87.59824284199992, 41.82673697300003], [-87.59811077499988, 41.82651245700002], [-87.59796029399989, 41.82631617800006], [-87.5978951639999, 41.826218661000034], [-87.5978714749999, 41.82617487500004], [-87.59782744399988, 41.826087813000065], [-87.59773641299992, 41.82590775000005], [-87.59767926299989, 41.82576587800008], [-87.59764826299991, 41.82566487900003], [-87.59762920199994, 41.825592031000035], [-87.59761630099989, 41.82551784300005], [-87.59760965199987, 41.82544283700007], [-87.59760930099989, 41.82536753700002], [-87.59761525099991, 41.82529247100006], [-87.59763931599988, 41.825094518000064], [-87.5976361769999, 41.824768399000035], [-87.59762034799988, 41.82472876000008], [-87.59752933899993, 41.824685836000064], [-87.5974515429999, 41.82463540200006], [-87.59742254299991, 41.82461440200005], [-87.59736431299989, 41.82456877400005], [-87.59731019299993, 41.82451834000005], [-87.59726057699991, 41.824463468000026], [-87.59721582899992, 41.82440455900006], [-87.59717627499991, 41.82434204400004], [-87.59714220399988, 41.824276380000065], [-87.5971125279999, 41.824253787000075], [-87.59706333799991, 41.82421448100006], [-87.59701701999987, 41.82417182800002], [-87.59697379899988, 41.82412603900007], [-87.59693388799988, 41.82407733900003], [-87.59682788799992, 41.82393833900004], [-87.59677541199993, 41.82386160300006], [-87.59661241199989, 41.823594603000046], [-87.59656699299988, 41.82351044700003], [-87.59650599199993, 41.82338144700008], [-87.5964721859999, 41.82329979900003], [-87.59641318599994, 41.82313479900005], [-87.59638049599992, 41.823019759000026], [-87.59636215099992, 41.822933827000035], [-87.59625998499989, 41.82275486500003], [-87.59620180099992, 41.82263241200007], [-87.59617256699991, 41.82258573700005], [-87.59576355799992, 41.822098660000044], [-87.5955706949999, 41.82192731500003], [-87.59548312799987, 41.82183786500006], [-87.59536110399993, 41.821694469000064], [-87.5952691649999, 41.82156366100008], [-87.5952489619999, 41.82152669500004], [-87.5951845259999, 41.82140193900005], [-87.5950749499999, 41.82124838200007], [-87.5950281639999, 41.82117566100003], [-87.59498371999989, 41.821087924000025], [-87.59489271999988, 41.820880925000026], [-87.59487560999989, 41.82083208400006], [-87.59485388999988, 41.82080847000008], [-87.5946789959999, 41.82063019500002], [-87.59459714699993, 41.82056135000004], [-87.59451726999991, 41.82052433000006], [-87.59443995299992, 41.82048222400004], [-87.59436551899994, 41.82043520900003], [-87.59429427999991, 41.820383482000054], [-87.59422194999989, 41.82034413200005], [-87.59416025799992, 41.82030596500004], [-87.59410184499988, 41.82026294800004], [-87.59404708999992, 41.82021536100006], [-87.5939963509999, 41.82016351300006], [-87.59394995899991, 41.82010774200005], [-87.59390821399988, 41.82004841300004], [-87.59387138999992, 41.81998591000006], [-87.59385692699992, 41.81995893200006], [-87.5937969049999, 41.81994919400005], [-87.5937376989999, 41.81993533100006], [-87.5936795929999, 41.81991741000006], [-87.5936228649999, 41.819895515000034], [-87.59352886499991, 41.81985551500008], [-87.59346664499989, 41.81982617900007], [-87.59340684899988, 41.819792173000046], [-87.59334982699993, 41.81975369600007], [-87.59329591299989, 41.819710973000035], [-87.59324542199992, 41.81966425500006], [-87.59319864999989, 41.81961381400003], [-87.59299841099993, 41.81938628300007], [-87.5928992169999, 41.81935367400006], [-87.5926252169999, 41.819245674000044], [-87.5925330819999, 41.81920320700004], [-87.59246908199991, 41.81916920800005], [-87.59244133899992, 41.81915383600005], [-87.59230616199991, 41.81905813000008], [-87.59214139799988, 41.81891433800006], [-87.5920486619999, 41.81882049100005], [-87.59196766199993, 41.81872549100006], [-87.5918818399999, 41.818606363000065], [-87.5918308089999, 41.81852172700006], [-87.5916783249999, 41.81833152300003], [-87.5916011649999, 41.818218661000046], [-87.59155115399993, 41.81811798900007], [-87.5915111539999, 41.81802298900003], [-87.59148367099992, 41.817948737000044], [-87.59146282499994, 41.81787235500006], [-87.59144877899989, 41.817794435000046], [-87.59142877899987, 41.81764643500008], [-87.5914216609999, 41.81756803500007], [-87.59142142699991, 41.81748931200006], [-87.5914280779999, 41.81741087100004], [-87.5914415629999, 41.817333312000066], [-87.59145735499992, 41.817254110000064], [-87.5914436919999, 41.817171713000036], [-87.5914288629999, 41.817104650000026], [-87.59232499999987, 41.816992], [-87.59485291599991, 41.816874023000075], [-87.59503191599991, 41.81688102200007], [-87.59517191599988, 41.81688102200007], [-87.5959789149999, 41.81685002300008], [-87.59625791599989, 41.81683902200006], [-87.59644691499993, 41.81682802300008], [-87.59666891599991, 41.81681802200005], [-87.59746291499991, 41.81681502200006], [-87.59878491499987, 41.81679102300005], [-87.5994019159999, 41.816782022000034], [-87.60061691599991, 41.81676102200002], [-87.60184091599989, 41.81674802200006], [-87.60403791499992, 41.816722022000064], [-87.60404390599989, 41.81672193200005], [-87.60405086999991, 41.81672182700003], [-87.60406333999991, 41.81697544300003], [-87.60413875699992, 41.81767160000003], [-87.6042391969999, 41.81857412400006], [-87.60430341699991, 41.81912404200004], [-87.60440999999989, 41.82003], [-87.60456138099993, 41.82026834400005], [-87.60487373899988, 41.820760142000026], [-87.60567917799992, 41.82207612100005], [-87.60692999999989, 41.82399], [-87.60695162999988, 41.824011119000026], [-87.60709038699989, 41.82432884700005], [-87.6077260259999, 41.82557983900005], [-87.60818876599987, 41.82649753100008], [-87.60867005899988, 41.827442287000046], [-87.60899509299989, 41.82808018600007], [-87.60991491099989, 41.82984906700005], [-87.61038087799992, 41.83073366800004], [-87.61057874999993, 41.831190444000065], [-87.60920891599993, 41.83117802300005], [-87.6089489159999, 41.83119202300008], [-87.60828491599992, 41.83120002200006], [-87.6074848939999, 41.83121494300007], [-87.60723696099987, 41.83137214600002], [-87.60712391599989, 41.83138302300005], [-87.60691091599992, 41.831425155000034]]]}, "properties": {"city": "Chicago", "name": "Oakland", "regionid": "274850", "geo_point_2d": [41.82363678696339, -87.60156960051839], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.60503034099989, 41.83842983500004], [-87.60504782499991, 41.83835136800006], [-87.60505782499989, 41.83831436800004], [-87.60508091299992, 41.83824141400003], [-87.60514891699994, 41.83809671900007], [-87.60514771199988, 41.83775586600005], [-87.6051480879999, 41.83772085100008], [-87.6051548269999, 41.83749771900004], [-87.60512029499992, 41.83724537000005], [-87.60503935799993, 41.83693590500007], [-87.6048602799999, 41.836582997000036], [-87.60477412699991, 41.83648384000008], [-87.60468291899988, 41.836358495000056], [-87.6046255509999, 41.836263384000056], [-87.6044569829999, 41.836094816000056], [-87.60421555699989, 41.83590741500005], [-87.6041549569999, 41.83585592500008], [-87.60409914199992, 41.83579928300008], [-87.6040485489999, 41.83573793200008], [-87.6040035739999, 41.83567235200007], [-87.60397057299988, 41.83561935200003], [-87.60395516399988, 41.83559366100007], [-87.60390744199991, 41.83549132000006], [-87.60385516499989, 41.83541666100006], [-87.60379026999992, 41.835278285000065], [-87.60375159099992, 41.83517343500005], [-87.60371033999988, 41.83502016500005], [-87.6037057149999, 41.83499117800004], [-87.60369671599993, 41.834928178000034], [-87.60368968199991, 41.834861], [-87.60368770899991, 41.83479348300004], [-87.60369080999992, 41.83472600900005], [-87.6036989659999, 41.83465895800003], [-87.60371213199993, 41.83459270800005], [-87.60372213099991, 41.834550708000044], [-87.60374418499991, 41.83447354100007], [-87.60375401999993, 41.834448006000045], [-87.6046206059999, 41.83472380600005], [-87.61100796899987, 41.83472380600005], [-87.61135632599989, 41.83472042400006], [-87.61159050399992, 41.83462832900004], [-87.61171318999992, 41.83443442400004], [-87.61170380799989, 41.834062692000074], [-87.6116248369999, 41.833836075000065], [-87.6114940649999, 41.83348473800003], [-87.61128856999991, 41.83295249600007], [-87.61114347499993, 41.83257978500006], [-87.61082867899994, 41.831731228000066], [-87.6105827129999, 41.831190480000075], [-87.6106279389999, 41.831190891000055], [-87.61081490699989, 41.83127310400004], [-87.61092091599993, 41.831261023000074], [-87.61108295499989, 41.831265798000054], [-87.61109, 41.83127], [-87.6123319729999, 41.83125317700006], [-87.61414309399991, 41.831225530000026], [-87.61553716999991, 41.83122], [-87.61730017599989, 41.83113094000003], [-87.6173212609999, 41.83243819400007], [-87.61740418099991, 41.83543534400007], [-87.61741988999994, 41.83613176800007], [-87.61743435699991, 41.836898039000054], [-87.61744580799987, 41.83773401700005], [-87.61745179999988, 41.838360050000055], [-87.61744999999989, 41.83836], [-87.6164438449999, 41.83841972100003], [-87.61470269399989, 41.83844602600003], [-87.6126117709999, 41.83848363700007], [-87.61086962399989, 41.83852347600003], [-87.60940618299989, 41.83854672200004], [-87.60591649599989, 41.838622224000055], [-87.60591699999989, 41.838659], [-87.60502381599989, 41.83875936100008], [-87.60502152799987, 41.83874212700003], [-87.6050177629999, 41.83867131900006], [-87.6050167199999, 41.83859216000007], [-87.60503034099989, 41.83842983500004]]]}, "properties": {"city": "Chicago", "name": "Lake Meadows", "regionid": "403144", "geo_point_2d": [41.83546540006359, -87.61194364325623], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.65481, 41.96961788200007], [-87.65481, 41.96908], [-87.65397415399995, 41.96909623000004], [-87.6522558599999, 41.96912372900003], [-87.65053779099995, 41.96913669600007], [-87.64930008899992, 41.96920480600004], [-87.64417701, 41.96978932200004], [-87.64413826099994, 41.969792920000025], [-87.64385660599997, 41.969430630000026], [-87.64363335199991, 41.969263190000035], [-87.64265663299993, 41.96853063400003], [-87.64206120799994, 41.96806056200005], [-87.64075627299994, 41.96703069500005], [-87.63895619199995, 41.96643075100006], [-87.63595507299998, 41.96573089300005], [-87.63235602799992, 41.965530946000044], [-87.63193202799991, 41.96340894600006], [-87.63186652499995, 41.96308315400005], [-87.63185602799993, 41.96303094600005], [-87.63186554499995, 41.963026505000066], [-87.63335505599994, 41.96233140000003], [-87.63645517099994, 41.960131554000036], [-87.63805500999999, 41.959631264000045], [-87.63955511899995, 41.959830825000026], [-87.64096305399991, 41.96032754400004], [-87.64125553599993, 41.96043106000006], [-87.64125602799996, 41.960488485000035], [-87.64125602799996, 41.961074259000036], [-87.64125500599992, 41.96113123800006], [-87.64055541499992, 41.96133064000002], [-87.63955508799994, 41.960830868000066], [-87.63835502199997, 41.96073131200006], [-87.63725521099995, 41.96113158800005], [-87.63586254199996, 41.96222582800004], [-87.63585602799996, 41.96223094600003], [-87.63585232299991, 41.962232755000045], [-87.63585965799994, 41.96228409900004], [-87.63594070099997, 41.962851401000066], [-87.63595232299996, 41.96293275500004], [-87.63595602799994, 41.96293094600002], [-87.63685510499992, 41.963030844000045], [-87.64016223799995, 41.96269193400006], [-87.64075749799996, 41.96263079800007], [-87.64215441099995, 41.96248595300004], [-87.64326318399996, 41.96235760600006], [-87.64436382399997, 41.96222880800007], [-87.64758269299993, 41.96188553900002], [-87.64909544699995, 41.96185568200007], [-87.65046928599995, 41.961831088000054], [-87.65170611899995, 41.96181086500006], [-87.65356126499992, 41.96178448400007], [-87.65589866999993, 41.96175691300004], [-87.65704067599994, 41.96174735300007], [-87.65713599199994, 41.96174655500004], [-87.65885320099993, 41.96172], [-87.65957, 41.96172], [-87.65957582399992, 41.96247058200003], [-87.65958, 41.96315757500002], [-87.65957867499995, 41.96418746300002], [-87.65959504799994, 41.96480556900007], [-87.65961956699992, 41.966250163000026], [-87.65964319199996, 41.967900379000056], [-87.65967, 41.96902], [-87.66064019399991, 41.969012813000056], [-87.66235568399996, 41.96897443100004], [-87.66359594899994, 41.96895535100003], [-87.66565827999993, 41.96892901200004], [-87.66689494999994, 41.96890915500005], [-87.66746751099997, 41.96890188500002], [-87.66755164599991, 41.96948358200005], [-87.66763138199995, 41.970238412000064], [-87.66771905499994, 41.97106839100007], [-87.66785517599996, 41.972168998000015], [-87.66797583199994, 41.97293389800006], [-87.66813078199992, 41.973869504000035], [-87.66826679399995, 41.97497114500003], [-87.66836072499993, 41.97580138600005], [-87.66838984199993, 41.97615806900005], [-87.66474391599995, 41.97620002200005], [-87.66351391599991, 41.97622702200003], [-87.66231091499995, 41.976239022000065], [-87.66109291499995, 41.97625602200002], [-87.65984927899993, 41.97629395000006], [-87.6588379149999, 41.97629902300007], [-87.65865891499996, 41.976300023000064], [-87.65780291499996, 41.97630802200006], [-87.65704791599994, 41.97632302200002], [-87.65640291499993, 41.976333022000055], [-87.65499039199993, 41.97635499900007], [-87.65497641299997, 41.97587088100004], [-87.65495094399994, 41.97484079700006], [-87.65492725899996, 41.973672927000045], [-87.65489983599994, 41.97271179500006], [-87.65485680799995, 41.97099566100007], [-87.65481, 41.96961788200007]]]}, "properties": {"city": "Chicago", "name": "Uptown", "regionid": "269609", "geo_point_2d": [41.96778739988586, -87.6524276979242], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.6121932859999, 41.64994785900006], [-87.61199669299991, 41.649913166000026], [-87.61187825499988, 41.64989406300002], [-87.611953, 41.649521], [-87.61196966499993, 41.649524423000045], [-87.61196966499993, 41.64919770900007], [-87.6120625689999, 41.64845447900005], [-87.61196966499993, 41.64696801800004], [-87.61224837699989, 41.64687511400006], [-87.61224837699989, 41.64452912500007], [-87.61234128099993, 41.644524701000066], [-87.614246, 41.644434], [-87.616746, 41.644434], [-87.616746, 41.64455798300007], [-87.616746, 41.645634], [-87.61654599999989, 41.646534], [-87.61684599999988, 41.646934], [-87.616746, 41.647834], [-87.61654599999989, 41.648634], [-87.616646, 41.649734], [-87.61717731999991, 41.64977487100003], [-87.620546, 41.650034], [-87.62045793799989, 41.65016572900002], [-87.62042499999987, 41.650215], [-87.6203459999999, 41.650334], [-87.62030499999989, 41.650415], [-87.620246, 41.650534], [-87.62014464799994, 41.65078026100007], [-87.61880960099987, 41.650525967000064], [-87.61762004599989, 41.65027363700005], [-87.61715143299993, 41.65023759000007], [-87.61592583099988, 41.65016549500006], [-87.61426766299988, 41.65012944800008], [-87.6131862489999, 41.65005735400006], [-87.61260949499987, 41.650021307000074], [-87.6121932859999, 41.64994785900006]]]}, "properties": {"city": "Chicago", "name": "Riverdale", "regionid": "275192", "geo_point_2d": [41.64752914320945, -87.61474834456715], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.60383542899989, 41.83379283400006], [-87.60373792099993, 41.833764178000074], [-87.60360743599993, 41.833771071000044], [-87.6035350249999, 41.83377197600003], [-87.60346277599989, 41.833767052000034], [-87.6033911579999, 41.83375633100007], [-87.60332063399993, 41.83373988200008], [-87.60318663299994, 41.83370288100007], [-87.60306814399989, 41.83366105700008], [-87.60281914499987, 41.83355305700006], [-87.60275592799988, 41.83352265900003], [-87.60269527399991, 41.83348742400004], [-87.60263755099992, 41.83344756800005], [-87.6025831119999, 41.83340333400007], [-87.6025322859999, 41.83335498900003], [-87.60248538399989, 41.83330283000003], [-87.60214268899993, 41.83288658700006], [-87.6020939569999, 41.83282162300008], [-87.60203395699992, 41.832733624000056], [-87.6019930539999, 41.83266769400005], [-87.6019579869999, 41.83259848300007], [-87.60192901699989, 41.832526507000054], [-87.60190635899993, 41.83245230100004], [-87.60189018299988, 41.83237641900006], [-87.6018806089999, 41.832299424000034], [-87.6018777069999, 41.83222189100007], [-87.60187873599993, 41.832128781000044], [-87.60189234099991, 41.83198283400008], [-87.6019197829999, 41.83187021800005], [-87.60193023499993, 41.83183684600004], [-87.60191795299991, 41.83152571700003], [-87.60190742899992, 41.83146929100006], [-87.60189063099993, 41.83142295800007], [-87.6028139999999, 41.831425], [-87.60463591599994, 41.83140502200006], [-87.6059689399999, 41.831391154000066], [-87.60612491499988, 41.83133002300006], [-87.6062449099999, 41.831289940000026], [-87.60691091599992, 41.831425155000034], [-87.60712391599989, 41.83138302300005], [-87.60723696099987, 41.83137214600002], [-87.6074848939999, 41.83121494300007], [-87.60828491599992, 41.83120002200006], [-87.6089489159999, 41.83119202300008], [-87.60920891599993, 41.83117802300005], [-87.61057874999993, 41.831190444000065], [-87.6105827129999, 41.831190480000075], [-87.61082867899994, 41.831731228000066], [-87.61114347499993, 41.83257978500006], [-87.61128856999991, 41.83295249600007], [-87.6114940649999, 41.83348473800003], [-87.6116248369999, 41.833836075000065], [-87.61170380799989, 41.834062692000074], [-87.61171318999992, 41.83443442400004], [-87.61159050399992, 41.83462832900004], [-87.61135632599989, 41.83472042400006], [-87.61100796899987, 41.83472380600005], [-87.6046206059999, 41.83472380600005], [-87.60375401999993, 41.834448006000045], [-87.60377302999987, 41.83439864700005], [-87.60380843799992, 41.83432662300003], [-87.60385012499992, 41.83425804200003], [-87.6039051259999, 41.834176042000074], [-87.60396307999989, 41.83409892700006], [-87.6040540899999, 41.83400791600007], [-87.60399452599994, 41.833866874000066], [-87.60397695599991, 41.833821238000034], [-87.60393720099994, 41.833812516000066], [-87.60386744999992, 41.83379507100005], [-87.60383542899989, 41.83379283400006]]]}, "properties": {"city": "Chicago", "name": "Groveland Park", "regionid": "403348", "geo_point_2d": [41.83296573029524, -87.60701056231142], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.64344391399992, 41.848713024000055], [-87.6434059779999, 41.84866984500007], [-87.64335191599989, 41.84867902200004], [-87.6427379179999, 41.848815022000046], [-87.6426369189999, 41.848857208000084], [-87.6426009159999, 41.848845023000074], [-87.64250891599991, 41.84881302100006], [-87.6424299159999, 41.84876902100006], [-87.64242907899987, 41.84876833100008], [-87.64234381299988, 41.84869810800005], [-87.6422997909999, 41.84862106800005], [-87.64228283499993, 41.84849101300005], [-87.6423571049999, 41.84833000800005], [-87.64222473399988, 41.84811897500003], [-87.6423230919999, 41.848063053000025], [-87.64204894299989, 41.84774197600005], [-87.64200431899988, 41.84771577600007], [-87.64178173999991, 41.84758509400006], [-87.64184894399993, 41.847342989000076], [-87.64117795899993, 41.84714286300003], [-87.6404121669999, 41.84715], [-87.63818230899993, 41.84734308900004], [-87.63630167299992, 41.84738446900006], [-87.63414390899992, 41.84743380600007], [-87.63207950299993, 41.847366945000026], [-87.6315630579999, 41.84735021800008], [-87.63141330899991, 41.846233489000035], [-87.63150214699992, 41.845188010000065], [-87.63103275199987, 41.83982565600006], [-87.63104926399988, 41.838777704000044], [-87.63103806199989, 41.83822005300003], [-87.63102965899989, 41.83780175300006], [-87.63099862499989, 41.836546596000055], [-87.63096935499992, 41.835362796000084], [-87.63093813399989, 41.834109332000075], [-87.63090302499988, 41.83271666000007], [-87.63086967099989, 41.83139362100007], [-87.63085598099988, 41.83095006200005], [-87.6308309369999, 41.83013859800007], [-87.6308033369999, 41.82909415900007], [-87.63075917699989, 41.82742305700002], [-87.6321889469999, 41.82727723000004], [-87.63323462499993, 41.82726500300003], [-87.63619906699994, 41.827235809000044], [-87.63619891599988, 41.82722902300008], [-87.63622691499987, 41.82666602200003], [-87.63624991499994, 41.82629102300006], [-87.6363090869999, 41.825389941000076], [-87.63608591099988, 41.824913934000044], [-87.63605272399988, 41.82414590100006], [-87.6360380969999, 41.823574024000024], [-87.6362439089999, 41.82357407600005], [-87.63630980699989, 41.823573722000035], [-87.63643991499991, 41.823573022000055], [-87.63656891599992, 41.82357202200006], [-87.63730891599991, 41.823566022000065], [-87.6386189159999, 41.82356002300008], [-87.63982891499988, 41.82354802200006], [-87.6410529159999, 41.823524022000065], [-87.64226791599991, 41.823507022000044], [-87.64348091499993, 41.82347702200007], [-87.64471091599994, 41.823465022000036], [-87.64586591499993, 41.82345502200007], [-87.6459219149999, 41.82345502200007], [-87.64592332599993, 41.82345499200004], [-87.64596891599989, 41.82345402200007], [-87.64820791499993, 41.82341402200005], [-87.65077891599991, 41.82338002200002], [-87.65315091599989, 41.82335802200003], [-87.65388191499991, 41.82335902200003], [-87.65594693899992, 41.823299891000026], [-87.65611193199992, 41.82336807800004], [-87.6565199149999, 41.82334602300005], [-87.6567049159999, 41.82333802200008], [-87.65691991499992, 41.82333002300003], [-87.65710391499991, 41.823329023000035], [-87.65746891499991, 41.82331602200008], [-87.6579619149999, 41.823303022000054], [-87.6589479159999, 41.82328702200003], [-87.65951391599991, 41.823201023000074], [-87.65982591599992, 41.82320602200007], [-87.66050991499989, 41.82319402200004], [-87.66180491499993, 41.82318602200006], [-87.66266291499993, 41.82317302200004], [-87.66469891599989, 41.823153022000035], [-87.66529491499992, 41.82314302200005], [-87.66538091599992, 41.823142022000056], [-87.6654035749999, 41.823142022000056], [-87.66540663699992, 41.82334798500004], [-87.66540886099992, 41.823848227000035], [-87.66542522299991, 41.82434765000004], [-87.66543361799994, 41.82472960500006], [-87.6654556489999, 41.82537601600006], [-87.66547866799992, 41.82605252000006], [-87.66547387299991, 41.826493718000044], [-87.66547638299993, 41.82708232800007], [-87.66550059099988, 41.82796352400003], [-87.66552784799988, 41.82902118100008], [-87.66555281099993, 41.83013830300006], [-87.66556413199991, 41.83072596200008], [-87.6655726199999, 41.83110791300003], [-87.66558895699988, 41.831843087000024], [-87.6656079079999, 41.832579865000056], [-87.66562052399989, 41.83296255800006], [-87.66563286699991, 41.83351951200005], [-87.66563849499991, 41.834048568000064], [-87.66564727799991, 41.83440106900007], [-87.6656563899999, 41.834665313000066], [-87.6656727139999, 41.83501815400005], [-87.6656763379999, 41.83528266200005], [-87.66568224499989, 41.835664477000066], [-87.6656874299999, 41.835928933000034], [-87.66569230099991, 41.83625328200008], [-87.66569719999987, 41.83666475800003], [-87.66570518299993, 41.83716434300003], [-87.6657309159999, 41.83789915900007], [-87.66575999999989, 41.83828147200005], [-87.66577119999992, 41.83848225100006], [-87.6657657499999, 41.838485521000045], [-87.66578999999987, 41.83922212300006], [-87.6658099999999, 41.84166193700007], [-87.6658206049999, 41.84324926100003], [-87.66583460399991, 41.84450713000005], [-87.66584711199991, 41.84479220800005], [-87.66583446699988, 41.84479220800005], [-87.66493572899991, 41.84459248900003], [-87.6637830319999, 41.84427700600008], [-87.66265610399994, 41.84480632100007], [-87.66147795199991, 41.84538686000007], [-87.66086326299987, 41.84572835300008], [-87.66053884499988, 41.84595032400005], [-87.66026564999991, 41.84620644400006], [-87.66021442599991, 41.846633311000055], [-87.6600266049999, 41.84692358000006], [-87.65965096199992, 41.84709432700003], [-87.65927531899987, 41.84707725200007], [-87.65845573499989, 41.847538269000026], [-87.65780689699993, 41.84789683700005], [-87.65738003099993, 41.848118807000034], [-87.65693608899993, 41.848272479000045], [-87.65642384899989, 41.84830662900003], [-87.6559457579999, 41.84830662900003], [-87.65529692099992, 41.84823833000006], [-87.65469930699992, 41.848118807000034], [-87.65410169399989, 41.84803343400005], [-87.6535382299999, 41.84796513500003], [-87.65299183999988, 41.84796513500003], [-87.65247959999991, 41.84803343400005], [-87.65195028499993, 41.848152957000075], [-87.65142097099991, 41.84823833000006], [-87.65065260999994, 41.84834077800007], [-87.64949153299993, 41.848426151000055], [-87.6486548739999, 41.84859689800003], [-87.64807433499992, 41.84873349600007], [-87.64732304999988, 41.84900669000005], [-87.64624734499992, 41.849296960000046], [-87.64553020899987, 41.84939940800006], [-87.6451204169999, 41.84943355700004], [-87.64428375799989, 41.84945063200006], [-87.6442400699999, 41.849457056000055], [-87.643801, 41.849217], [-87.64363591499993, 41.84895602200003], [-87.64348491599989, 41.848786022000056], [-87.64344391399992, 41.848713024000055]], [[-87.6646224329999, 41.84365832300005], [-87.6645845319999, 41.843630700000055], [-87.6645753269999, 41.84365406100004], [-87.66450378599988, 41.84390222000007], [-87.66448892699991, 41.84398426400008], [-87.66448473599989, 41.84400838200007], [-87.66454904999993, 41.84404183600008], [-87.66460442699989, 41.84403939000003], [-87.66460468599989, 41.84397914400005], [-87.66460597699994, 41.843934674000074], [-87.66461797699989, 41.84371167400006], [-87.6646224329999, 41.84365832300005]]]}, "properties": {"city": "Chicago", "name": "Bridgeport", "regionid": "273035", "geo_point_2d": [41.83571991928011, -87.64851829258937], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.69086, 41.69178], [-87.69000819373142, 41.69166715803591], [-87.68917212806822, 41.691674114644094], [-87.6879644649858, 41.69168406176234], [-87.68657067786744, 41.691700709154155], [-87.68527046618725, 41.69171547628674], [-87.68434140900861, 41.69172745676415], [-87.68322685701754, 41.69174549028771], [-87.68174036765758, 41.69175508786147], [-87.68114, 41.69175], [-87.68103928499943, 41.691189254975946], [-87.68101611067102, 41.69044623910943], [-87.68095485913295, 41.68933535964865], [-87.68089120555197, 41.68840772769659], [-87.68086518405856, 41.68766107393494], [-87.6808366835384, 41.68682463246622], [-87.68082145611136, 41.68571106028909], [-87.68081, 41.68524438951968], [-87.68081, 41.68448], [-87.68136977390998, 41.68447151857715], [-87.68248283862897, 41.684453863676055], [-87.68387767953227, 41.684444899012554], [-87.6848991603523, 41.68445], [-87.68574900009166, 41.68453299999711], [-87.6860489996232, 41.68453299999408], [-87.68704899985445, 41.68453299999408], [-87.68834900042475, 41.68453299999408], [-87.6895390004209, 41.68455600015498], [-87.69082899963092, 41.684539000270625], [-87.69551431361951, 41.68433451422609], [-87.69554899957097, 41.684332999767925], [-87.69564899968395, 41.688132999567195], [-87.69572193829777, 41.68815123422054], [-87.69584191595038, 41.69155802254957], [-87.69584291509729, 41.69159102227209], [-87.69584491518947, 41.69163402245568], [-87.69584839173457, 41.69173290880813], [-87.69574899989847, 41.691733000019696], [-87.69095800018152, 41.69173800027013], [-87.69096517967743, 41.69190128962976], [-87.69086, 41.69178]]]}, "properties": {"city": "Chicago", "name": "Kennedy Park", "regionid": "403371", "geo_point_2d": [41.6881023788618, -87.68830826966604], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.72881527687589, 41.99723099996968], [-87.72880715938743, 41.99657857401953], [-87.72880179735199, 41.99565094190503], [-87.7287920523757, 41.99498176768561], [-87.72877679559514, 41.994466851344576], [-87.72876850059339, 41.993894371221714], [-87.72876852011342, 41.993894345894155], [-87.72875823977161, 41.99328019982469], [-87.72875238350173, 41.99224949631113], [-87.72874999999972, 41.991270185505265], [-87.72871, 41.99068], [-87.7286706488618, 41.990508541469595], [-87.72862312497499, 41.990301473106435], [-87.7285708770729, 41.99007382153363], [-87.72856999999983, 41.99007000000032], [-87.72851683703965, 41.98931114838891], [-87.72849709391998, 41.98838476236076], [-87.72847785469867, 41.98787011319009], [-87.72846057642244, 41.98730190405515], [-87.72844539945349, 41.98668377432961], [-87.72843334309793, 41.98611712561465], [-87.72841804863936, 41.98549788073224], [-87.72839855098759, 41.98478430003452], [-87.72918432868845, 41.98479357107667], [-87.73042112976708, 41.984783323918876], [-87.73114255993397, 41.98477475512326], [-87.73204999999973, 41.98476000000018], [-87.73204237284436, 41.98451974460166], [-87.73201497182471, 41.9835927427384], [-87.73198999999983, 41.982700000000264], [-87.7313487363745, 41.982700000000264], [-87.73016331082773, 41.98270521181906], [-87.72887497254953, 41.98271561497935], [-87.72833999999989, 41.982720000000235], [-87.72829917823482, 41.98122773769931], [-87.72825397379525, 41.979575264297104], [-87.72819515598019, 41.977374807949296], [-87.72816473576025, 41.976274609996636], [-87.72814005936827, 41.97538214715799], [-87.72954494699611, 41.97538], [-87.73089987863581, 41.97537388108848], [-87.73118782832931, 41.9755254335586], [-87.73150241091925, 41.97564340202986], [-87.73173834786168, 41.97564340202986], [-87.73224954457038, 41.97564340202986], [-87.73311464669258, 41.97580069332487], [-87.73350787493003, 41.97584001614863], [-87.73362584340128, 41.97607595309088], [-87.7340977172862, 41.97623324438589], [-87.7348841737611, 41.976351212857224], [-87.73552322036544, 41.97653064943945], [-87.7356262940441, 41.97890134405135], [-87.73619319927751, 41.978901344051536], [-87.73619319927751, 41.97962285980312], [-87.73593551508044, 41.97962285980312], [-87.73578090456226, 41.97972593348178], [-87.73557475720486, 41.97982900716053], [-87.73547168352594, 41.9799836176787], [-87.73547168352594, 41.980344375554445], [-87.73557475720486, 41.980498986072604], [-87.73572936772302, 41.980705133430284], [-87.73578090456226, 41.98080820710894], [-87.73583244140178, 41.98101435446661], [-87.73583244140178, 41.981117428145275], [-87.73567783088362, 41.98142664918161], [-87.7356262940441, 41.98168433337852], [-87.7356262940441, 41.9818904807362], [-87.73562614920235, 41.98265122177427], [-87.73717664142987, 41.98262833491084], [-87.73821778201268, 41.982588893394926], [-87.73822228123163, 41.982588720620505], [-87.73820313601367, 41.978901344051536], [-87.73810006233492, 41.977252165190954], [-87.73834458225036, 41.97729496062712], [-87.7385805191928, 41.97741292909838], [-87.73885577895898, 41.97741292909838], [-87.73964223543385, 41.977570220393204], [-87.74007478649503, 41.977688188864384], [-87.74042869190872, 41.97788480298323], [-87.74086124296991, 41.97816006274941], [-87.74097921144109, 41.978356676868], [-87.74097921144109, 41.97882855075285], [-87.74086124296991, 41.97918245616671], [-87.74074327449866, 41.979693652875326], [-87.74074327449866, 41.98004755828892], [-87.74078259732232, 41.98032281805519], [-87.74129379403094, 41.98075536911639], [-87.74180499073972, 41.981030628882564], [-87.74247347874325, 41.98150250276742], [-87.74270941568568, 41.98162047123868], [-87.74333858086561, 41.98185640818119], [-87.74381045475049, 41.98213166794729], [-87.74420368298792, 41.98213166794729], [-87.74467555687278, 41.98213166794729], [-87.74522607640523, 41.98209234512345], [-87.74550133617142, 41.9819350538287], [-87.74581591876124, 41.98185640818119], [-87.74613050135127, 41.98185640818119], [-87.7465237295887, 41.98185640818119], [-87.74683831217865, 41.981895731004855], [-87.7468776350023, 41.98213166794729], [-87.74691695782614, 41.98236760488972], [-87.74699128543847, 41.98256581185611], [-87.74700103489367, 41.98256587434817], [-87.74815689048171, 41.98254767189778], [-87.74817191530354, 41.98316202233079], [-87.74817303675805, 41.983523158581555], [-87.74820400006568, 41.984726003221716], [-87.74821261716801, 41.98627158691742], [-87.7482260067626, 41.9874052392392], [-87.74823696188508, 41.988332772956944], [-87.7482517007466, 41.98905412962024], [-87.74825819286066, 41.989311650150476], [-87.74826999999983, 41.99023944871752], [-87.74826999999983, 41.9906517434326], [-87.74826707632296, 41.991734101418594], [-87.74826280714835, 41.9924555919086], [-87.74826194190723, 41.993434895892904], [-87.74826531157375, 41.994311009202285], [-87.74826887945609, 41.99523865858873], [-87.74827513726056, 41.99637251077129], [-87.74827733324405, 41.996930999630436], [-87.74805900002679, 41.996930999630436], [-87.74695899968255, 41.99703099974352], [-87.74575900012451, 41.99723099996968], [-87.74529295885867, 41.99723099996968], [-87.74455899966728, 41.99723099996968], [-87.74335900010942, 41.99723099996968], [-87.7422589997653, 41.99723099996968], [-87.74105900020724, 41.99723099996968], [-87.73985899975004, 41.99723099996968], [-87.73908099985964, 41.9972299999235], [-87.73875900030527, 41.99723099996968], [-87.73755899984803, 41.99723099996968], [-87.73593499963062, 41.99717099990183], [-87.7348590003928, 41.997130999856516], [-87.73375900004885, 41.99723099996968], [-87.73245900037755, 41.99723099996968], [-87.73125899992061, 41.997130999856516], [-87.72985900013641, 41.99723099996968], [-87.72895900001834, 41.99723099996968], [-87.72881527687589, 41.99723099996968]]]}, "properties": {"city": "Chicago", "name": "Sauganash", "regionid": "140292", "geo_point_2d": [41.98793542681414, -87.73774810722011], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.61782091583791, 41.888663022565915], [-87.61755391522127, 41.888660022427544], [-87.61741091492462, 41.888661022473634], [-87.6172739158041, 41.88866502355746], [-87.61717980804883, 41.88868417372069], [-87.61715104143512, 41.88845287708791], [-87.61415200031826, 41.88843200022634], [-87.61404199974436, 41.88780099991781], [-87.61368699956772, 41.885777000147705], [-87.61358300016974, 41.88518699993049], [-87.61345100038015, 41.88443300024718], [-87.61356533244343, 41.8843529677755], [-87.61525840679172, 41.884355914124356], [-87.6160947015051, 41.8843547277498], [-87.61790773532418, 41.88452010784165], [-87.61909085400782, 41.88449905360409], [-87.62065, 41.88449], [-87.62056218939398, 41.884696781819585], [-87.62061235752594, 41.88553418412673], [-87.62064170050145, 41.88636917482083], [-87.62064345886618, 41.8869977404076], [-87.62050422596482, 41.88748863201133], [-87.62036331180093, 41.8884623801268], [-87.62031301856007, 41.88879744819805], [-87.61925191525216, 41.88871302217259], [-87.61782091583791, 41.888663022565915]]]}, "properties": {"city": "Chicago", "name": "New Eastside", "regionid": "403342", "geo_point_2d": [41.88645776142526, -87.61720240667323], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.64113599999985, 41.955369000000175], [-87.64099910024218, 41.95473177594798], [-87.6410539999998, 41.954732], [-87.6440419799999, 41.954744197000075], [-87.64413985099986, 41.954658847000125], [-87.64427091599977, 41.95465802200014], [-87.64453991499991, 41.95464602300013], [-87.6446609159999, 41.95464402300014], [-87.64480091699994, 41.954642022000066], [-87.6449109159999, 41.95464802300011], [-87.64509191599986, 41.954642022000066], [-87.6452329159998, 41.95464102200016], [-87.64589791599984, 41.95462002200019], [-87.6466059149999, 41.95460102200004], [-87.64672291499981, 41.954605023000106], [-87.64713791599978, 41.95459402199999], [-87.64791891599991, 41.95458102200017], [-87.64968391599984, 41.95454602300003], [-87.65048391499985, 41.954530022000206], [-87.65210391599987, 41.95451402200013], [-87.6545149159998, 41.954475022000096], [-87.65609891599983, 41.954457022000035], [-87.65671691599991, 41.95445002300008], [-87.656829191983, 41.95445161484531], [-87.65690477822693, 41.95758089591638], [-87.6569863111721, 41.961054196982694], [-87.65704067616525, 41.961747352881595], [-87.65589866999989, 41.96175691300016], [-87.65356126499992, 41.961784484000034], [-87.65170611899991, 41.96181086500018], [-87.6504692859999, 41.96183108800017], [-87.64909544699991, 41.96185568200007], [-87.64758269299988, 41.96188553900014], [-87.64436382399992, 41.962228808000035], [-87.64326318399988, 41.962357606000026], [-87.64215441099991, 41.96248595300008], [-87.64075682308881, 41.9626314302847], [-87.6423549999999, 41.962132], [-87.6423549999999, 41.961918084999986], [-87.6423315009999, 41.96185875900006], [-87.64222690099986, 41.96159467800015], [-87.64228299999989, 41.96144900000017], [-87.64229599999989, 41.961267], [-87.64223720099992, 41.961037761000156], [-87.641855, 41.96073200000017], [-87.64186978699988, 41.96057596000006], [-87.64179511299993, 41.9605045650001], [-87.64152494199988, 41.95982247800006], [-87.64172799999989, 41.9584210000001], [-87.64170199999985, 41.958096], [-87.64151499999983, 41.957328], [-87.64139099999987, 41.95651], [-87.64113599999985, 41.955369000000175]]]}, "properties": {"city": "Chicago", "name": "Buena Park", "regionid": "403264", "geo_point_2d": [41.95823227740516, -87.64919895339959], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.74898568593846, 41.9830652756315], [-87.74856143373167, 41.98260893712319], [-87.74814489191542, 41.98216128790896], [-87.74813591544252, 41.98188602196754], [-87.74808491578976, 41.98003102189399], [-87.74807491622806, 41.979942022288085], [-87.74804691583654, 41.978906023095334], [-87.74800891588367, 41.97783702238161], [-87.74798491567651, 41.97727502255556], [-87.74796191551562, 41.97674702249837], [-87.74793101277801, 41.97524838492255], [-87.74856177794423, 41.975259572393234], [-87.7499535151316, 41.975240566888864], [-87.75098423174028, 41.97523915484559], [-87.75179999999982, 41.975300000000104], [-87.75259596137076, 41.97578594740011], [-87.75413492903027, 41.976725626204384], [-87.75494106611156, 41.9772213083106], [-87.75494445083275, 41.9772257231684], [-87.75523091604387, 41.97776402261378], [-87.75628199686705, 41.97980500248298], [-87.75628167120712, 41.97981689613811], [-87.75627596871158, 41.980025161012435], [-87.75624891620545, 41.97999602207939], [-87.75617791563063, 41.97992002217329], [-87.75611387400947, 41.979852963326984], [-87.75592899669964, 41.97986679612998], [-87.75557137089805, 41.97992696068154], [-87.75517999999988, 41.97985000000033], [-87.75477826017944, 41.980280513192895], [-87.75405246691946, 41.98109055031312], [-87.75365848187852, 41.981524439571515], [-87.75308801757356, 41.9821842721091], [-87.75270593853502, 41.982641686051025], [-87.75232175605781, 41.98313389273106], [-87.75230100509552, 41.98313513019985], [-87.75230154008199, 41.983121892648], [-87.75196347213092, 41.98292121752384], [-87.75149964057644, 41.982972754363345], [-87.7511904195401, 41.983127364881504], [-87.75088119850378, 41.98338504907833], [-87.75046890378859, 41.98359119643601], [-87.7500050722341, 41.98364273327551], [-87.74951722549393, 41.98364273327551], [-87.74898568593846, 41.9830652756315]]]}, "properties": {"city": "Chicago", "name": "Forest Glen", "regionid": "269580", "geo_point_2d": [41.979150667660114, -87.75128302427716], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.78762591556887, 41.98242502253182], [-87.78762891570715, 41.98229302274242], [-87.78763591602997, 41.98204002277113], [-87.78763791612215, 41.981974022426826], [-87.78763991531503, 41.98189702247454], [-87.78765991533766, 41.980999022448565], [-87.78769391600552, 41.979188022604696], [-87.78773791533575, 41.97737702276074], [-87.78775691531229, 41.97620802193367], [-87.78777091595765, 41.97558202185567], [-87.78777096306005, 41.97557999197461], [-87.78863294185915, 41.97557280881788], [-87.78966511601676, 41.97556430232779], [-87.79100071632365, 41.975546488061276], [-87.79203247532917, 41.97553477662333], [-87.79330800925743, 41.97552436057983], [-87.79446116191131, 41.9755096473015], [-87.79561564119255, 41.975495242634885], [-87.79689078889224, 41.975490000000114], [-87.79816631302513, 41.97548423266363], [-87.79919816195917, 41.97548], [-87.80047401186987, 41.97546517063524], [-87.80180979085952, 41.97545411940447], [-87.80302412081176, 41.97544381093332], [-87.80417811828089, 41.975427791064156], [-87.8056348206327, 41.975411337753265], [-87.80695, 41.97540000000012], [-87.80693588966743, 41.975985578801755], [-87.80692178206232, 41.9766535188477], [-87.80690518082264, 41.97744071501071], [-87.80689479544411, 41.97822993257055], [-87.80688377348079, 41.978897552483055], [-87.80688993085148, 41.98065907452942], [-87.80684999999987, 41.9823900000001], [-87.80556573526258, 41.98240747298964], [-87.8024198129331, 41.98240032618596], [-87.8001131308393, 41.98241554015533], [-87.79660003486322, 41.98246], [-87.79413560941686, 41.98248718260091], [-87.7907813077788, 41.98262518254288], [-87.7893682699588, 41.9827231038026], [-87.78847339299098, 41.98282780851665], [-87.78761404451808, 41.98282915536888], [-87.78761791609953, 41.98268702291792], [-87.7876219153846, 41.982539022390974], [-87.78762591556887, 41.98242502253182]]]}, "properties": {"city": "Chicago", "name": "Union Ridge", "regionid": "403278", "geo_point_2d": [41.978998779525796, -87.79724995441418], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.53681047899991, 41.72454270300005], [-87.53578853699989, 41.723149146000075], [-87.53281561499993, 41.723149146000075], [-87.52612653999994, 41.72333495400004], [-87.52448946699988, 41.72333495400004], [-87.52414922699988, 41.72334977800005], [-87.52415199999989, 41.723134], [-87.52415399999988, 41.722959], [-87.524161, 41.722437], [-87.52417979999989, 41.72106223300005], [-87.52423004699992, 41.721020223000025], [-87.5244138179999, 41.72083712100003], [-87.52450880399992, 41.720694875000035], [-87.52456670399992, 41.720539340000066], [-87.52462361399992, 41.72036088100002], [-87.52468924499992, 41.720000673000065], [-87.52470116799991, 41.71978927700006], [-87.52471429899991, 41.71956601600003], [-87.52471093399991, 41.71950151400005], [-87.5247027649999, 41.71934923000002], [-87.52463805199991, 41.71913479800008], [-87.52457124499992, 41.719000696000066], [-87.52445744999991, 41.71884858300006], [-87.52436714399991, 41.71875484100007], [-87.5242130099999, 41.718642744000064], [-87.52421699999988, 41.718354], [-87.524231, 41.717273], [-87.524236, 41.716979], [-87.52422674499988, 41.716562333000056], [-87.52424034399989, 41.71655800600006], [-87.52429487399989, 41.716542532000055], [-87.52431887499992, 41.71653653200006], [-87.52438083399991, 41.71652334100008], [-87.52454422499993, 41.71650973600003], [-87.5250665689999, 41.71651807200003], [-87.52530906799988, 41.71649773400008], [-87.52543010499991, 41.71648135900005], [-87.5255456649999, 41.71645871100008], [-87.5264018099999, 41.71617178700005], [-87.52682515899991, 41.71601492800004], [-87.52741315799992, 41.71568634100003], [-87.52762579599991, 41.71551608900005], [-87.5276546779999, 41.715482853000076], [-87.52738285299992, 41.71508816500005], [-87.52734116399994, 41.71502266100003], [-87.52729329099989, 41.71492702400007], [-87.5271112919999, 41.71450202500005], [-87.52709406499989, 41.71445873100003], [-87.5269836799999, 41.71415892800007], [-87.52670361399991, 41.713843243000035], [-87.5264687289999, 41.713592955000074], [-87.52601298599987, 41.71326902400005], [-87.52589808399993, 41.713171917000054], [-87.52581708299991, 41.71309091700003], [-87.52577044099988, 41.71304057100007], [-87.52572777299989, 41.712986814000026], [-87.52568932899992, 41.712929960000054], [-87.5256553329999, 41.712870340000045], [-87.52562598199992, 41.712808301000045], [-87.52560144799989, 41.71274420400003], [-87.52556544799988, 41.71263820400008], [-87.52553134099989, 41.71250516500003], [-87.5255176779999, 41.712349], [-87.52553134099989, 41.71219283400006], [-87.52557191399988, 41.71204141400004], [-87.52563816499989, 41.71189933900006], [-87.5256865839999, 41.71182433600005], [-87.52573758399988, 41.711753336000065], [-87.52577802399989, 41.71170118700006], [-87.5258221799999, 41.71165214300003], [-87.5258698139999, 41.71160646900005], [-87.5259206679999, 41.71156441200007], [-87.52644491199992, 41.71116386000005], [-87.5263035859999, 41.71123108600005], [-87.5262266199999, 41.71125525000007], [-87.52612861999994, 41.71128125000006], [-87.52605416499992, 41.711297660000064], [-87.52596264399992, 41.71130899600006], [-87.52585164399991, 41.71131699600004], [-87.5257804949999, 41.71131929800003], [-87.52570938599989, 41.71131596600008], [-87.52563876299989, 41.71130702100003], [-87.52556906999989, 41.711292517000054], [-87.5255007409999, 41.71127254700007], [-87.52536374099991, 41.71122654600003], [-87.52529154999988, 41.71119879900004], [-87.52522200899993, 41.71116495100005], [-87.5251556339999, 41.71112525300003], [-87.52509291499989, 41.71108], [-87.52503431599989, 41.711029524000075], [-87.52489659799991, 41.71090017700003], [-87.52468517499993, 41.71074853600003], [-87.52446319699992, 41.710615698000076], [-87.52439826899987, 41.710572928000026], [-87.5240925719999, 41.71035202700005], [-87.5240887769999, 41.71035089300005], [-87.524044, 41.708335], [-87.524044, 41.70819518200005], [-87.524044, 41.707755], [-87.524044, 41.707435], [-87.524044, 41.70699934900004], [-87.524044, 41.706735], [-87.52422627999991, 41.70657094800002], [-87.5250439999999, 41.705835], [-87.52497536599991, 41.704462313000064], [-87.524944, 41.703835], [-87.5250439999999, 41.702835], [-87.524944, 41.702635], [-87.52502, 41.701034], [-87.5250439999999, 41.700535], [-87.524944, 41.698835], [-87.524944, 41.69745], [-87.524944, 41.697035], [-87.524944, 41.695600999999996], [-87.524944, 41.695595], [-87.524944, 41.69558750000004], [-87.524944, 41.695577], [-87.524944, 41.695135], [-87.524944, 41.69382], [-87.524944, 41.693335], [-87.52485351699993, 41.69179679900008], [-87.52497691599987, 41.691792023000055], [-87.52582891599988, 41.69179502200005], [-87.5270169159999, 41.69180002200004], [-87.52822591599993, 41.691805022000025], [-87.52944991499992, 41.691811023000064], [-87.5302089159999, 41.69181702200006], [-87.53037004899993, 41.69181815500007], [-87.5303079159999, 41.68840702200003], [-87.53033091499991, 41.68687802300007], [-87.53031691599992, 41.686407023000065], [-87.5303079159999, 41.686096023000054], [-87.53033091599988, 41.68494902400004], [-87.53042078399992, 41.684562885000034], [-87.53040623299994, 41.68357222500003], [-87.5304053859999, 41.68321295300006], [-87.53077173099989, 41.682643083000066], [-87.5311433459999, 41.68171404500003], [-87.53177373499992, 41.68093818100004], [-87.53671761399993, 41.68095334300004], [-87.53913289399989, 41.68095873400004], [-87.5400854499999, 41.680957468000074], [-87.54322074699988, 41.68095330200002], [-87.54488999999987, 41.68094], [-87.5449016349999, 41.68162257900007], [-87.54491113399988, 41.68217984000005], [-87.54487999999989, 41.68276], [-87.54487998199991, 41.68276002900005], [-87.54525999799993, 41.683204925000034], [-87.54517591599992, 41.68365902200008], [-87.5451679159999, 41.68392702200002], [-87.54519291599989, 41.68477602200004], [-87.5452379149999, 41.68627402400004], [-87.5452069159999, 41.68750102300003], [-87.54515891499993, 41.690396023000055], [-87.54519691499989, 41.692254022000036], [-87.54517791499987, 41.69369202200004], [-87.54517291599988, 41.694119023000034], [-87.54518091599992, 41.69551202200005], [-87.5451859159999, 41.696284023000075], [-87.54517791499987, 41.69666202200005], [-87.54520291599991, 41.69720502300003], [-87.54520391599992, 41.69729602300003], [-87.54521091599989, 41.69773102300006], [-87.54520191599993, 41.69952802200003], [-87.5451909159999, 41.700006023000064], [-87.54518291499991, 41.70038202200003], [-87.5450519909999, 41.701105057000056], [-87.5447559159999, 41.70146102200003], [-87.54412491499993, 41.70232802300005], [-87.54386991599992, 41.702652022000045], [-87.54379550699988, 41.702776527000076], [-87.54379, 41.70278], [-87.54340664999994, 41.703081922000074], [-87.54331374599988, 41.70336063300004], [-87.54312793799993, 41.70419676800008], [-87.54312793799993, 41.70605484400004], [-87.54312793799993, 41.707076786000066], [-87.54238470799991, 41.70800582400005], [-87.54145566999993, 41.70902776600007], [-87.54024791999989, 41.709585189000045], [-87.53959759299991, 41.710421323000034], [-87.5393188819999, 41.71125745800003], [-87.53904016999988, 41.71237230400003], [-87.5388761009999, 41.71348748700006], [-87.53886476799991, 41.71460207100005], [-87.53887238899992, 41.71599194700007], [-87.5388899999999, 41.71961880100007], [-87.53941178599989, 41.71971170500006], [-87.53941178599989, 41.722591723000036], [-87.53996920899992, 41.722591723000036], [-87.53996753699991, 41.72295262000005], [-87.53792532499992, 41.722970000000004], [-87.53792532499992, 41.72379947300003], [-87.53857565099989, 41.72500722200004], [-87.53828961599991, 41.725106204000035], [-87.53888, 41.72592], [-87.5380897839999, 41.72624916900008], [-87.53792532499992, 41.72575045300004], [-87.53681047899991, 41.72454270300005]], [[-87.52670535499992, 41.71096254700005], [-87.52668858999992, 41.71095883200007], [-87.52657125299991, 41.711071244000024], [-87.52670535499992, 41.71096254700005]]]}, "properties": {"city": "Chicago", "name": "East Side", "regionid": "138166", "geo_point_2d": [41.70178951314494, -87.53462446901159], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.89416601299993, 42.005371361000066], [-87.89416601299993, 42.00408558700007], [-87.89416601299993, 42.00402847400004], [-87.89416601299993, 42.00397136100002], [-87.89416601299993, 42.00318558700006], [-87.89416599499995, 42.00312853100006], [-87.89335486899992, 42.00322049800006], [-87.89335567599991, 42.00327748800003], [-87.89336501799993, 42.00367152400002], [-87.89336523299993, 42.003728567000046], [-87.89276597599995, 42.00412815300006], [-87.89106618299994, 42.00402834800007], [-87.89007059099997, 42.003530763000064], [-87.89006584999997, 42.003528588000044], [-87.89007058199995, 42.003473646000025], [-87.89016648799996, 42.00232871600007], [-87.89406600799997, 42.00222848600002], [-87.89436558199996, 42.00222817200005], [-87.89446104599993, 42.001753307000044], [-87.89446625999992, 42.00172873200005], [-87.89446104599993, 42.00169618900002], [-87.89427479999993, 42.000485588000046], [-87.8942663399999, 42.000428617000054], [-87.89427479199996, 42.00042847400003], [-87.89596552199997, 42.00042825800005], [-87.89596601299993, 42.00037136100007], [-87.89596601299993, 42.00021358700007], [-87.89596601299993, 42.00015647400004], [-87.89596601299993, 42.00009936100002], [-87.89596601299993, 41.99998558700002], [-87.89596603399995, 41.99992843400002], [-87.89356604299994, 41.99972843600005], [-87.88446504999997, 41.99812896200006], [-87.88426468599992, 41.99632833000003], [-87.88426501299992, 41.99627136100001], [-87.88426501299992, 41.995492587000044], [-87.88426502099996, 41.99543545700003], [-87.88406498999994, 41.99542853500002], [-87.88066504699997, 41.99572839300002], [-87.87997431499996, 41.99567566600006], [-87.87847801299992, 41.991502474000065], [-87.87816501299993, 41.990629474000045], [-87.87736501299996, 41.988329474000054], [-87.87636401299993, 41.98542747400005], [-87.87633368799997, 41.985350325000056], [-87.87604133799991, 41.98534862200005], [-87.87579701299994, 41.98396147400006], [-87.87516501299993, 41.982329474000025], [-87.87486501299995, 41.980929474000035], [-87.87446501299993, 41.980129474000066], [-87.87348034699991, 41.97688332300003], [-87.87346436999995, 41.976829480000035], [-87.87348033899994, 41.976826209000066], [-87.87396398999991, 41.97672957600002], [-87.87586352199996, 41.97672925800002], [-87.87586401299994, 41.97667236100004], [-87.87586401299994, 41.97408658700004], [-87.87586401299994, 41.97402947400006], [-87.87276404199994, 41.97402942300003], [-87.87186398399994, 41.97393052400002], [-87.87076401299996, 41.97392947400004], [-87.86996401299994, 41.97392947400004], [-87.86926401299995, 41.97392947400004], [-87.86856409299996, 41.97392937100005], [-87.86817602799992, 41.97383247800007], [-87.86816465199995, 41.973830039000056], [-87.86817603799993, 41.97388959400006], [-87.86825259099996, 41.974272360000036], [-87.86826335499994, 41.974329215000054], [-87.86825259999993, 41.97432947400005], [-87.86789301299994, 41.97432947400005], [-87.86747829799995, 41.97432947400005], [-87.86746409299995, 41.97432937100005], [-87.86747829099993, 41.97427236300006], [-87.86754973499994, 41.97398658500003], [-87.86756393299997, 41.97392957600004], [-87.86754972899993, 41.97392947400004], [-87.86666401299996, 41.97392947400004], [-87.86645801299991, 41.97392947400004], [-87.86506401299991, 41.97392947400004], [-87.86474701299991, 41.97392947400004], [-87.863764, 41.973929510000055], [-87.86236408599994, 41.97402935100006], [-87.86208144899994, 41.97397296100006], [-87.86188723799992, 41.97393390800004], [-87.86187904599994, 41.97387610200006], [-87.86187019399993, 41.97380749500007], [-87.86184221499995, 41.97359765100003], [-87.86181889899993, 41.973495060000054], [-87.86172264899993, 41.973217422000054], [-87.86174779499997, 41.97321239300003], [-87.86177469899994, 41.973207258000045], [-87.86188063699994, 41.973186149000064], [-87.86216398999991, 41.97312957600007], [-87.86356401299994, 41.97312947400007], [-87.86504901299996, 41.97312947400007], [-87.86526401299993, 41.97312947400007], [-87.86645801299991, 41.97312947400007], [-87.86666403499997, 41.97312938800006], [-87.86726399099994, 41.97302955900005], [-87.86826401299997, 41.973029474000036], [-87.86876396399995, 41.97302954800005], [-87.86936406799997, 41.973130383000054], [-87.87078501299996, 41.97308347400007], [-87.87246500199993, 41.973029499000056], [-87.87498350599996, 41.97307676200005], [-87.87512705399995, 41.97307892800005], [-87.87632305499992, 41.97310292700006], [-87.87668813699996, 41.973110755000036], [-87.87706905499992, 41.973118925000044], [-87.87766305299992, 41.973128930000016], [-87.87783016299991, 41.97309585200002], [-87.87810182399991, 41.97304194400004], [-87.87816498999993, 41.97302957600004], [-87.88086452199995, 41.97302925800004], [-87.88086501299992, 41.97297236100006], [-87.88086501199993, 41.972386588000056], [-87.88086489199996, 41.972329477000066], [-87.88086453099997, 41.97227236600001], [-87.88080857799994, 41.96562258100004], [-87.88080822899995, 41.965571806000064], [-87.88080867099994, 41.96556568000005], [-87.88111123999992, 41.96556275500005], [-87.88231204199995, 41.965552954000024], [-87.88243971399997, 41.965549224000014], [-87.88542590699993, 41.96544962600006], [-87.88543703299995, 41.96544897900002], [-87.88543802799994, 41.96539225900005], [-87.88543802799994, 41.965365485000056], [-87.88543736999996, 41.96530824300003], [-87.88080302799993, 41.96537094600006], [-87.88080167899992, 41.96535821400004], [-87.88080139699994, 41.965313873000056], [-87.88080103299993, 41.96530113800002], [-87.88080062299996, 41.965244255000066], [-87.88076678799993, 41.960505610000034], [-87.88076556699994, 41.96044860900001], [-87.88076516299996, 41.96039149800003], [-87.88076441799994, 41.960286584000066], [-87.88076444299996, 41.96022961400006], [-87.88066401299992, 41.958429474000056], [-87.88056871699993, 41.95728591600004], [-87.88056463799992, 41.957229559000034], [-87.88056870999996, 41.95722880300006], [-87.88336386099994, 41.95682968100003], [-87.88496410799993, 41.95742939900003], [-87.8870080149999, 41.95769747100007], [-87.88891201199992, 41.95793347500005], [-87.88924630699992, 41.95797595700002], [-87.88966411599993, 41.95802883000004], [-87.89076404599994, 41.958028946000056], [-87.89114203099994, 41.95802904200007], [-87.89117251299993, 41.95802947400005], [-87.89296400099991, 41.95802922700004], [-87.89296335599992, 41.95797240900002], [-87.89288100399993, 41.95511831800007], [-87.89287902999999, 41.955061426000036], [-87.89287747299993, 41.95500419300002], [-87.89286564399995, 41.95458654100002], [-87.89286442499994, 41.954529462000025], [-87.89266598699997, 41.95215311000003], [-87.89266370499995, 41.952129292000045], [-87.89266598299994, 41.95209599200006], [-87.89276449, 41.95042971700007], [-87.89516400199993, 41.95042949500003], [-87.89736403599993, 41.950529417000034], [-87.90166494699994, 41.95022958800006], [-87.90386500199996, 41.95062948500002], [-87.90856495999996, 41.951628511000074], [-87.91176480099993, 41.95272798300004], [-87.91216522299996, 41.951828965000054], [-87.91446484599996, 41.95262815800003], [-87.91456374399993, 41.95391197000003], [-87.91456488499992, 41.95392862300002], [-87.91456330399993, 41.95396906200005], [-87.91456322399992, 41.95398554400003], [-87.91447643599994, 41.956762924000024], [-87.91447468799997, 41.956820187000055], [-87.91447475999993, 41.956876997000045], [-87.914464, 41.95707189300002], [-87.91446602799992, 41.95712794600006], [-87.91506527299997, 41.957327695000046], [-87.91531295399993, 41.957300510000024], [-87.91542363399996, 41.957288519000016], [-87.91557211899998, 41.95727213400004], [-87.91558938499996, 41.95727020300006], [-87.91596492899995, 41.957228621000056], [-87.91720401299993, 41.95749447300005], [-87.92028205499994, 41.958151425000025], [-87.92198960599994, 41.95830313000003], [-87.92216426599997, 41.95743716300006], [-87.92216625899992, 41.957428733000015], [-87.92216425499997, 41.95738004500002], [-87.92206656699992, 41.954685531000045], [-87.92206490799997, 41.954628477000014], [-87.92206311399994, 41.954571424000044], [-87.92203821899993, 41.953819514000024], [-87.92203611499995, 41.95377077900007], [-87.92203696399996, 41.95376261200004], [-87.922035, 41.95371394200003], [-87.92347801899994, 41.95407046800005], [-87.92386002, 41.95415846700006], [-87.92527602299995, 41.95445746400002], [-87.92607494299992, 41.95460462300007], [-87.92607693599994, 41.95466348500001], [-87.92607890699995, 41.95472151400002], [-87.92624427599996, 41.95900571600002], [-87.92631602099998, 41.95901746400005], [-87.92723301299996, 41.959147473000016], [-87.92763900199995, 41.959204486000026], [-87.92809001699997, 41.959282469000065], [-87.93124701299995, 41.95979147400003], [-87.93191538899995, 41.95989938600006], [-87.93191598599996, 41.95995642500002], [-87.93202403999997, 41.96465452300002], [-87.93202468299992, 41.96471149100005], [-87.93316801299991, 41.96471347300007], [-87.93322801199994, 41.964713475000046], [-87.93399701299992, 41.96471547300006], [-87.93516221199995, 41.96471746600001], [-87.93516280099998, 41.96477457500002], [-87.93519122499997, 41.96683637400002], [-87.93519183399994, 41.96689333800003], [-87.9356009099999, 41.96677829600002], [-87.93592095699992, 41.96652644100003], [-87.93650594699993, 41.965937455000066], [-87.93666605099996, 41.96572848200002], [-87.93669779599996, 41.96569250000003], [-87.93677797099994, 41.965440504000064], [-87.93686016599995, 41.96505584900005], [-87.93686625299992, 41.96502873900004], [-87.93686016599995, 41.96499873100003], [-87.93680801299993, 41.96473347400006], [-87.93680425399997, 41.96472285000004], [-87.93678536699994, 41.96466852000002], [-87.93680424499995, 41.96466573600002], [-87.93706599299996, 41.96462854600002], [-87.93925401299997, 41.96462647400005], [-87.93930801, 41.96462647900005], [-87.93958501499996, 41.96462946800004], [-87.93958501299993, 41.96468658700007], [-87.93958501399996, 41.96471135900002], [-87.93958524499996, 41.96476846800003], [-87.93958579899997, 41.96482557700005], [-87.93958935399993, 41.96513836900003], [-87.93959000199997, 41.96519547500002], [-87.93959063999992, 41.965252581000065], [-87.93960638599997, 41.966686369000044], [-87.93960701199993, 41.96674347500004], [-87.93960763799996, 41.96680058100002], [-87.93961238899996, 41.967234366000056], [-87.93961291599996, 41.967268375000046], [-87.93961284099996, 41.96729133600006], [-87.93961418899993, 41.96732561200002], [-87.93961411599994, 41.96734857200004], [-87.93961464899995, 41.96738258300007], [-87.93963237699994, 41.968972367000035], [-87.93963267999997, 41.96901823800004], [-87.93963359999996, 41.96902958800006], [-87.93963340199996, 41.96907534000007], [-87.93963426999993, 41.96908651100004], [-87.93964649999995, 41.969274611000046], [-87.93964664099995, 41.96933157900002], [-87.93967338399993, 41.97176337000006], [-87.93967352599996, 41.97182033700004], [-87.93968839899995, 41.97207761500004], [-87.93968824599993, 41.97213458600003], [-87.93969277999994, 41.97324536200005], [-87.93969303699997, 41.97330247300003], [-87.93969334099994, 41.97335958400003], [-87.93969368499992, 41.973419363000055], [-87.93969398299997, 41.973476475000034], [-87.93969422199993, 41.97353358700007], [-87.93970280399996, 41.97587936200005], [-87.93970304399993, 41.97593647300003], [-87.93970334799995, 41.97599358400003], [-87.93970767799996, 41.97673136300004], [-87.93970759999993, 41.976788334000034], [-87.93987407099995, 41.97700808000007], [-87.93989110099992, 41.977030820000024], [-87.93987408599995, 41.97706519500007], [-87.93971241899997, 41.97739701100004], [-87.93971276699993, 41.97745397100004], [-87.93977940499994, 41.98372136800002], [-87.93977962899996, 41.983749364000055], [-87.93977993999992, 41.98377847700003], [-87.93978009399993, 41.98380647100003], [-87.93978040999994, 41.98383558400006], [-87.93978064199993, 41.98386357900005], [-87.93978538499992, 41.98429436600002], [-87.93978600199993, 41.98435132700006], [-87.93978792499995, 41.98440852300006], [-87.93979010099996, 41.984473425000026], [-87.93979178699996, 41.98453048600003], [-87.93979335399993, 41.98458770100005], [-87.93980100399992, 41.98503937800006], [-87.93980191699994, 41.98508938100007], [-87.93980204099994, 41.98509647300005], [-87.93980297599992, 41.98514647500007], [-87.93980309899995, 41.98515356800003], [-87.93980399199994, 41.98520357100004], [-87.93982803399993, 41.98660638100005], [-87.93982815699995, 41.98661587600002], [-87.93982827999996, 41.98662537100005], [-87.93982898299997, 41.98666347800001], [-87.93982906499996, 41.98667207400006], [-87.93982881799995, 41.986682334000065], [-87.93983021599995, 41.986720703000046], [-87.93982998099992, 41.98672737100002], [-87.93983011399995, 41.98673956800007], [-87.93983083299995, 41.98678357700003], [-87.93983174799993, 41.98687224000002], [-87.93983360499996, 41.98692968300002], [-87.93983426499992, 41.986986523000034], [-87.93984203799992, 41.98746537700003], [-87.9398429089999, 41.987522478000066], [-87.93984360899996, 41.98757958100003], [-87.93990441599996, 41.99340536800002], [-87.93990438099993, 41.99346224400005], [-87.93906700299996, 41.99342849300005], [-87.93899030999992, 41.99342884500003], [-87.93773, 41.99583547600002], [-87.93559874299994, 41.99967136600002], [-87.93556721999991, 41.999728316000066], [-87.93559874999987, 41.99972847400004], [-87.93586676699992, 41.999728098000034], [-87.93736701299994, 41.99742883100003], [-87.93746735799992, 41.99832856600005], [-87.94006244599994, 41.99812882500003], [-87.94006697899992, 41.99812855000004], [-87.94006245299992, 41.998185938000056], [-87.94004470599997, 41.99840954600005], [-87.94004171199997, 41.998443280000046], [-87.94004470599997, 41.998466665000066], [-87.94016801299992, 41.99952847400003], [-87.94025849099992, 41.999709430000046], [-87.94026823399996, 41.99972875400004], [-87.94025849399996, 41.999766549000064], [-87.94016809299994, 42.000128371000066], [-87.94016801299992, 42.00018558700002], [-87.94016801299992, 42.00086736100007], [-87.94016751499996, 42.00092427500004], [-87.93704301299991, 42.001030473000064], [-87.93476996699995, 42.00110465100004], [-87.93477018699994, 42.00116141700004], [-87.93483380799994, 42.004167387000045], [-87.93483501499996, 42.004224474000054], [-87.93483622799991, 42.00428156100002], [-87.93486613099996, 42.005671520000064], [-87.93486637799997, 42.00572834700005], [-87.93487169899996, 42.00580618800005], [-87.93487310599993, 42.00586262000006], [-87.93496490799997, 42.00797171000004], [-87.93496637399993, 42.00802794600002], [-87.9346680049999, 42.00802809200006], [-87.93326802799996, 42.00822794600002], [-87.93325701299993, 42.00808947400002], [-87.93319201299992, 42.00722047400005], [-87.93306988799995, 42.00556738500005], [-87.93306674599994, 42.005528236000025], [-87.93306988799995, 42.00551026700003], [-87.93307921499994, 42.00545119500002], [-87.93308801299996, 42.00539547400005], [-87.93307920799997, 42.00539408200006], [-87.931185, 42.00509448900004], [-87.93036701099997, 42.004995476000026], [-87.92961704699994, 42.00490943600004], [-87.92948914799994, 42.00488240700001], [-87.92933599199992, 42.00480099700006], [-87.92923887299996, 42.00491438400007], [-87.92545012499993, 42.007244053000065], [-87.92280114, 42.00613480800006], [-87.92076474499996, 42.00365923700007], [-87.92056741099996, 42.00362883300004], [-87.92036656399992, 42.00482819900002], [-87.91366749599993, 42.00502870300004], [-87.91346653599997, 42.00842823000004], [-87.91296701299994, 42.00842847400002], [-87.91296701299994, 42.00837136100005], [-87.91296701299994, 42.00618558700005], [-87.91296701299994, 42.00612847400003], [-87.91257273199993, 42.00612847400003], [-87.91256703899992, 42.00612842700007], [-87.91257272499995, 42.00607136200005], [-87.91266130199993, 42.00518558600003], [-87.91266698699995, 42.00512852000002], [-87.91266129499994, 42.00512847400005], [-87.91126700799997, 42.00512848600003], [-87.90896301299992, 42.00518647300002], [-87.90726651299994, 42.00522867700004], [-87.90726602999997, 42.00528558600006], [-87.90726699599996, 42.00857136100006], [-87.90726651199995, 42.00862827400004], [-87.90523499699995, 42.00869254200005], [-87.90430814399996, 42.00884526600003], [-87.89526606899992, 42.00592844900006], [-87.89416567599993, 42.00542873700004], [-87.89416601299993, 42.005371361000066]], [[-87.89465208699991, 41.96715215500006], [-87.89478532599996, 41.96707748300001], [-87.89482723999996, 41.96705542200004], [-87.89488084499997, 41.967031134000074], [-87.8949359229999, 41.96701040500004], [-87.89499223899996, 41.96699332100006], [-87.89504955099993, 41.96697995800002], [-87.89510761499996, 41.966970371000045], [-87.89516618199995, 41.96696460300006], [-87.89517129199997, 41.96696428300003], [-87.89540529199995, 41.96695028300006], [-87.895459, 41.966948678000044], [-87.89548172099992, 41.96694896500002], [-87.89575871999992, 41.966955965000054], [-87.89579481799994, 41.966957604000065], [-87.89585338499994, 41.96696337200006], [-87.89588097499993, 41.96696744000006], [-87.89660864899994, 41.967086300000055], [-87.89921178399996, 41.96707280700007], [-87.89940384799995, 41.967062169000016], [-87.89931169999993, 41.96619391200004], [-87.89930860299992, 41.96615781900004], [-87.89930667799993, 41.966099], [-87.89930714899995, 41.96606989200006], [-87.89932604599991, 41.965486364000064], [-87.89930918599997, 41.96546855800004], [-87.89913919899996, 41.965289282000064], [-87.89867154299996, 41.96483523800003], [-87.89866208299995, 41.964825917000034], [-87.89862864999996, 41.964790626000074], [-87.89719897299993, 41.963197363000056], [-87.89657197099993, 41.96268803800006], [-87.89654603599993, 41.96266614600005], [-87.89651630099995, 41.962638864000034], [-87.89619471599995, 41.96233024700006], [-87.89600691299992, 41.96219906100002], [-87.89597857999993, 41.96218220200006], [-87.89577845999995, 41.96210450100004], [-87.89527472599995, 41.96202255400003], [-87.89486943899996, 41.96198458700006], [-87.89448592499996, 41.96196872700005], [-87.89443955899995, 41.96199235300002], [-87.89419432799994, 41.96225024500005], [-87.89357045699994, 41.96298103900004], [-87.89192904299993, 41.96538589500004], [-87.89191602099997, 41.96555357500005], [-87.89186800299994, 41.96683595100005], [-87.89188341699995, 41.96699009300004], [-87.89193740799998, 41.96700522200006], [-87.89376743499996, 41.96708758700002], [-87.89378581799996, 41.96708860300003], [-87.89379282799993, 41.96708909000005], [-87.89465208699991, 41.96715215500006]]]}, "properties": {"city": "Chicago", "name": "O'Hare International Airport", "regionid": "403159", "geo_point_2d": [41.98013517007267, -87.90873316502218], "county": "Du Page", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.6257, 41.79457], [-87.62561110093723, 41.79349545944941], [-87.62559988936965, 41.793083434336005], [-87.62557162994472, 41.7921566516416], [-87.62554476740904, 41.79133286721382], [-87.62551433984707, 41.79035277485437], [-87.6254872486267, 41.789372696444026], [-87.6254738010654, 41.78880484901699], [-87.62544689832225, 41.78756732282667], [-87.62542992318923, 41.78679477687661], [-87.62541628286088, 41.78586723454212], [-87.62540019179401, 41.784991274582886], [-87.62539230946349, 41.78463065795788], [-87.62537992294402, 41.78406397469027], [-87.62536794838695, 41.78349716128659], [-87.62543891626609, 41.783062021851805], [-87.6253587848746, 41.78306115400583], [-87.62531291585375, 41.78071802171994], [-87.6252959159695, 41.78028602249058], [-87.62528204752437, 41.77992088875337], [-87.62525, 41.77889600016223], [-87.62525, 41.77816214101065], [-87.6252199999999, 41.77755000000014], [-87.62516255267575, 41.776743465724245], [-87.62513999999985, 41.77633], [-87.62511765886967, 41.77525849513428], [-87.6251, 41.77446000000018], [-87.62517302315135, 41.77438697684865], [-87.62518420939637, 41.77419899595806], [-87.6251688530367, 41.773074444138935], [-87.62511710570752, 41.77132904649581], [-87.62557786151977, 41.76981232231454], [-87.62659849224016, 41.768017006651384], [-87.62724407858359, 41.76638339256728], [-87.62803132676099, 41.76493061663217], [-87.6284908341588, 41.76397050902178], [-87.62909113537022, 41.76269669814608], [-87.62997533394899, 41.76080957231918], [-87.6308421662814, 41.75896308512501], [-87.63138000000001, 41.75802], [-87.63251457432307, 41.75799283454851], [-87.63381769496335, 41.75796971459758], [-87.6349023517383, 41.75795138657626], [-87.63564378053853, 41.757950890446715], [-87.6363259153985, 41.757938022048016], [-87.63654091541666, 41.75793102262448], [-87.63692791526952, 41.7579280224863], [-87.63838991521365, 41.75791102260187], [-87.63879791603453, 41.75791102260187], [-87.63898591570751, 41.75791602283241], [-87.6392030453203, 41.757935155908605], [-87.64415, 41.75794000000021], [-87.64415254315482, 41.7579367591433], [-87.64415278876385, 41.75794415092755], [-87.64419291561352, 41.75794402232456], [-87.64535591616378, 41.757937022001926], [-87.64658691535215, 41.75792202220968], [-87.64778891500218, 41.75790202218714], [-87.64841591602581, 41.75789402181842], [-87.64899691492894, 41.757886022348806], [-87.6502159153627, 41.757865022280185], [-87.65029391536098, 41.757864022234095], [-87.65142791547372, 41.75784402221139], [-87.65263891553883, 41.75782202209668], [-87.65383891959326, 41.757806967445816], [-87.65582791595128, 41.75779002242009], [-87.65724391557345, 41.7577660222132], [-87.65868191531061, 41.75774102196013], [-87.66042291552509, 41.75773102239841], [-87.66095091558228, 41.75774102196013], [-87.66211491527943, 41.757748022282755], [-87.66310291585687, 41.75772102193742], [-87.66342391536529, 41.75771502256034], [-87.66347691600986, 41.75771502256034], [-87.66349074457722, 41.75771502256034], [-87.66352053008285, 41.758968576756686], [-87.66354328308624, 41.759944455797466], [-87.6635721790468, 41.7609201574406], [-87.6635994974942, 41.761738425661775], [-87.6636171937622, 41.76271182612859], [-87.66363797336248, 41.76352717423637], [-87.66367318908544, 41.76442175442185], [-87.66371991070604, 41.765888749700174], [-87.66376952081104, 41.76751546460588], [-87.66383322442907, 41.769303835157096], [-87.66389939089666, 41.77133570682188], [-87.66395512252318, 41.77336845984379], [-87.66399819653218, 41.77491780734377], [-87.66405414255723, 41.77695278908369], [-87.66411908190837, 41.778825762382276], [-87.66416361216055, 41.78045074114973], [-87.66420850306844, 41.782484015385386], [-87.66425228537298, 41.784191644656254], [-87.66429301171625, 41.78622837355272], [-87.66434124844926, 41.78777619262697], [-87.66441230610484, 41.78956369216628], [-87.66444658353055, 41.791272323809295], [-87.66448808159755, 41.79298188948845], [-87.66450674120493, 41.793931134435894], [-87.66441991541217, 41.79393302204525], [-87.66373491531213, 41.793945022598336], [-87.66329791585241, 41.793962022482695], [-87.66207791537248, 41.7939930221132], [-87.66088691533024, 41.79403902243488], [-87.66063091522061, 41.794047021904326], [-87.65966591570341, 41.794057022365145], [-87.65884991586024, 41.79406102254959], [-87.65732291561783, 41.794095022318196], [-87.65724991585007, 41.79409602236428], [-87.65599691564773, 41.794121022617354], [-87.65542691545288, 41.79413202222514], [-87.65481391507477, 41.79413307623057], [-87.65358991530985, 41.79413502236351], [-87.6524009153596, 41.79414202178686], [-87.65115891566425, 41.794149022109494], [-87.65017591531746, 41.79415202224786], [-87.64995891520704, 41.79415902257049], [-87.64871191618043, 41.79419602247745], [-87.64751791599993, 41.794193022339094], [-87.64678891567038, 41.79421002222343], [-87.64626491489803, 41.79422202187724], [-87.64596291536616, 41.79422802215387], [-87.64512391536192, 41.79424302194612], [-87.6450689155246, 41.794244021992114], [-87.64501291564113, 41.79424502203822], [-87.64386891596658, 41.79425702259138], [-87.64265091557891, 41.79426902224528], [-87.64142591576791, 41.79428402203727], [-87.64028091514808, 41.79431102238269], [-87.64000091573091, 41.79431702265915], [-87.63995791554741, 41.794318022705234], [-87.63903591621373, 41.79436302208155], [-87.63807791522072, 41.79439102247298], [-87.63778091591904, 41.79438502219661], [-87.63536390612717, 41.79435094418678], [-87.63527591556041, 41.7943680214129], [-87.63413191498658, 41.794398021896335], [-87.63292391595921, 41.79442602228776], [-87.63173691520188, 41.79445402267927], [-87.63160691640392, 41.79444902244855], [-87.63150891548383, 41.79445002249465], [-87.63131991576475, 41.79445402267927], [-87.63112791500804, 41.794457021918085], [-87.63092991577354, 41.79446102210244], [-87.6307289155012, 41.79446402224081], [-87.63059091543533, 41.7944670214798], [-87.63056191589708, 41.7944670214798], [-87.63044891608403, 41.79447002251735], [-87.62947391520673, 41.794495021871136], [-87.62848686161543, 41.79451202085611], [-87.62848682122605, 41.79451068150401], [-87.62797713870339, 41.794516930558096], [-87.6271523830477, 41.79453195233915], [-87.6262249045568, 41.79455500272697], [-87.6257, 41.79457]]]}, "properties": {"city": "Chicago", "name": "Englewood", "regionid": "138261", "geo_point_2d": [41.776517427741865, -87.64512995131874], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.8066479159999, 41.99770602300003], [-87.80666369599992, 41.99689430500006], [-87.80666391499992, 41.99688302200007], [-87.8066879159999, 41.99597702200003], [-87.80670291499989, 41.99570302300002], [-87.8067037829999, 41.99533988900003], [-87.8067130229999, 41.99472139700002], [-87.80671558399989, 41.99398811400003], [-87.8067137659999, 41.99314855200003], [-87.8067199999999, 41.99241458600004], [-87.80673582199994, 41.99131396300003], [-87.80675797999992, 41.990423197000034], [-87.80677623199992, 41.98953178500005], [-87.80678423499988, 41.98900760300005], [-87.80679144199989, 41.98822040700008], [-87.80679679699993, 41.986699758000036], [-87.8068053309999, 41.98533687600007], [-87.80682666099993, 41.98376362000004], [-87.80682608499987, 41.98323980800005], [-87.80684692499993, 41.98249928400008], [-87.80803030499992, 41.982376913000046], [-87.80960278199989, 41.98237], [-87.81086091199987, 41.98236512700002], [-87.8119099339999, 41.98236670500005], [-87.81290440799994, 41.98237925800003], [-87.81369151799991, 41.982459569000056], [-87.8145464329999, 41.982589951000065], [-87.81512129899993, 41.982709505000045], [-87.81580298599992, 41.98286640100007], [-87.81648537899989, 41.983029004000045], [-87.81713056499987, 41.98321071000002], [-87.8175647449999, 41.983323034000065], [-87.81863127899992, 41.983615107000034], [-87.81988841199988, 41.98384789400006], [-87.8212349059999, 41.98418398400007], [-87.8223003019999, 41.984316260000064], [-87.82336199999989, 41.98453], [-87.823962, 41.98463], [-87.82386199999989, 41.98513], [-87.823262, 41.98543], [-87.8231619999999, 41.98583], [-87.8231619999999, 41.98663], [-87.823062, 41.98753], [-87.822877, 41.988531], [-87.822562, 41.99023], [-87.822562, 41.99083], [-87.822362, 41.99183], [-87.8222180459999, 41.99240581700006], [-87.82219071599988, 41.99251513600005], [-87.822162, 41.99263], [-87.822032, 41.993215], [-87.82187642599987, 41.99421630300008], [-87.82175, 41.99503], [-87.821662, 41.99593], [-87.8217147519999, 41.99675331000008], [-87.82161379999988, 41.99675493900008], [-87.82151352399987, 41.99675655600003], [-87.82135847999989, 41.99675905700007], [-87.82129700299991, 41.99676404400008], [-87.82039091499989, 41.996806023000026], [-87.8188509149999, 41.996853022000025], [-87.81660791499989, 41.99692502300006], [-87.81538991599989, 41.99694302300008], [-87.81345691599992, 41.99699602200006], [-87.8114669159999, 41.99703202200003], [-87.81031189599989, 41.99707110100007], [-87.8101837489999, 41.99700211800006], [-87.8098152449999, 41.99709151100006], [-87.80929019599994, 41.99711782700006], [-87.80887251199992, 41.997205286000046], [-87.80853410999991, 41.99728465500004], [-87.80820346499989, 41.997392866000034], [-87.80765495099988, 41.99758082700003], [-87.80719098199992, 41.99769], [-87.80665, 41.99771], [-87.80664794499988, 41.99770929700007], [-87.8066479159999, 41.99770602300003]]]}, "properties": {"city": "Chicago", "name": "Norwood Park West", "regionid": "403276", "geo_point_2d": [41.989873224599215, -87.81439074709768], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.7154153306637, 41.87381144238159], [-87.71551640165696, 41.87380989105629], [-87.71554436396258, 41.87466747379507], [-87.71560267773638, 41.876439621148286], [-87.71564344807176, 41.87784257570162], [-87.71566668469096, 41.87869722542141], [-87.71570202234672, 41.879853731853366], [-87.71573, 41.88077], [-87.71474416359911, 41.88078308632382], [-87.7127918247764, 41.88082995954814], [-87.71169545002942, 41.88087839377383], [-87.71041317839244, 41.88091906914561], [-87.70937302946574, 41.88093571152856], [-87.70784857643365, 41.88095], [-87.70699419019945, 41.880954312452644], [-87.70609999999988, 41.88097], [-87.70608470040042, 41.88046511321694], [-87.70607730978368, 41.88022122286148], [-87.7060596956433, 41.87961104234923], [-87.70605540483338, 41.87906181866545], [-87.70603933185649, 41.878148102613835], [-87.70602176805556, 41.877782775556874], [-87.70598898890951, 41.87692809519883], [-87.70597724780293, 41.87649954480801], [-87.70595442275157, 41.875769609527794], [-87.7059362378602, 41.87534070371], [-87.70591128547551, 41.87423111343931], [-87.70613430428382, 41.8741988829516], [-87.70709923607208, 41.874075634373604], [-87.70765516857576, 41.8740182762581], [-87.7083328674629, 41.87395032564611], [-87.7090076813172, 41.87391982896734], [-87.70998334963942, 41.87389208484442], [-87.71053216307075, 41.87387647877518], [-87.71120425738056, 41.87386441185688], [-87.71218132769665, 41.87385212166428], [-87.71309661155532, 41.87384060865959], [-87.71413393326172, 41.8738275605879], [-87.7154153306637, 41.87381144238159]]]}, "properties": {"city": "Chicago", "name": "Fifth City", "regionid": "403319", "geo_point_2d": [41.87740045395838, -87.71082857479162], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.58584491599997, 41.78629502200003], [-87.58492987899993, 41.786066096000056], [-87.5844219499999, 41.785556835000065], [-87.58353092499998, 41.785942104000064], [-87.58220791499994, 41.785780022000026], [-87.58112587799994, 41.78566285300007], [-87.58078096399996, 41.78590219100004], [-87.57946691599993, 41.78589302300002], [-87.57755891499993, 41.786087023000036], [-87.57749291599995, 41.78609902200002], [-87.57742591599997, 41.78611202300004], [-87.57732591599994, 41.78613015500007], [-87.577093, 41.786072], [-87.57636979699997, 41.78620856700007], [-87.57636583599997, 41.786208553000044], [-87.57634850599996, 41.78603262700004], [-87.57606260699995, 41.78585727500006], [-87.57604399599994, 41.78581633200001], [-87.57603113099995, 41.78578316000004], [-87.57602201799993, 41.78575966300007], [-87.57591061899993, 41.78544086300007], [-87.57556677699993, 41.78465905600006], [-87.57555680699993, 41.78463548200005], [-87.57554530999992, 41.78460718000007], [-87.57548607799997, 41.784569725000026], [-87.57515986099995, 41.78437026100004], [-87.57513972499996, 41.784357580000055], [-87.57418012599997, 41.78373536900004], [-87.57350013299995, 41.78340040600005], [-87.57277756899997, 41.78307842500003], [-87.57218094899994, 41.78285434700006], [-87.57155864999997, 41.78266215300004], [-87.57108566399992, 41.78253872600003], [-87.57059596599997, 41.78244959200003], [-87.57046674499996, 41.78242514600002], [-87.56901226499996, 41.781534033000064], [-87.56899707599995, 41.78153943700004], [-87.56871966099997, 41.781682380000056], [-87.56860102799996, 41.78150500500003], [-87.56856759499998, 41.781455017000035], [-87.56859998799995, 41.781448120000036], [-87.568844, 41.78139616500004], [-87.56915171399996, 41.781322750000065], [-87.57104802199993, 41.78063331200007], [-87.57234812299995, 41.77983350300007], [-87.57274058399994, 41.779592915000066], [-87.57277195399993, 41.77957368400006], [-87.57274147799994, 41.779535237000054], [-87.57255816099993, 41.77931478000004], [-87.57245931299994, 41.77929567500006], [-87.57222264099991, 41.77920563600002], [-87.57215202799992, 41.77914894600007], [-87.57205022699996, 41.77903294600002], [-87.57174902799991, 41.77903294600002], [-87.56985641699995, 41.77714076000006], [-87.56982349699994, 41.77712762700001], [-87.56969032499994, 41.77704766900006], [-87.56948749299991, 41.776967628000065], [-87.56904265099995, 41.77670263800007], [-87.56845302799994, 41.77621994600002], [-87.56818402799995, 41.775917946000035], [-87.56816588499996, 41.775894747000045], [-87.56605446799993, 41.77441675500005], [-87.56603532799991, 41.77440403000002], [-87.56605289699995, 41.77436102100005], [-87.56626778799993, 41.77384929400005], [-87.566258, 41.773844500000045], [-87.56674486899998, 41.77352485200002], [-87.56739091599995, 41.773543022000055], [-87.56828391599996, 41.77353602300006], [-87.56996791599992, 41.77352602200005], [-87.57159591599996, 41.77351502200002], [-87.57278791599995, 41.773506022000035], [-87.57405191499996, 41.77348002300003], [-87.57528491599997, 41.77345802300004], [-87.57640406299996, 41.77344011600007], [-87.57640991499993, 41.77344002300002], [-87.57766191499995, 41.773423022000024], [-87.57894691499995, 41.77342102300002], [-87.58002491599993, 41.77345102200006], [-87.58012691599997, 41.77344602200002], [-87.58066291499993, 41.77342002300002], [-87.58139008699996, 41.77340064000003], [-87.58263991599993, 41.773370022000044], [-87.58339991599995, 41.773361023000064], [-87.58386491499991, 41.77334502300003], [-87.58504991499994, 41.773321022000054], [-87.58571791499996, 41.77330902200002], [-87.58605691599995, 41.773303023000025], [-87.58615991599997, 41.77330402300003], [-87.58625207699995, 41.77330601400006], [-87.58641691499994, 41.77329702200006], [-87.58877691599994, 41.77328102200004], [-87.59120291599996, 41.77326602300007], [-87.59131391599996, 41.77327802100005], [-87.59152291599996, 41.77326702100003], [-87.59173098899997, 41.77322309000004], [-87.59187291599993, 41.773178023000064], [-87.59203694899996, 41.773156098000065], [-87.59205885499992, 41.773138858000024], [-87.59285791599994, 41.773099022000046], [-87.59357891499995, 41.773100022000044], [-87.59442091499992, 41.77310202100004], [-87.59598691599993, 41.77307202200005], [-87.59722391499994, 41.773069023000055], [-87.59833291499994, 41.77307502200004], [-87.59958491499995, 41.77306102200003], [-87.60084791599994, 41.773055022000044], [-87.60205791599995, 41.773032023000034], [-87.60280891499994, 41.77301602200003], [-87.60325291599995, 41.77301602200003], [-87.60446791499996, 41.77301602200003], [-87.60543191599996, 41.77299002300003], [-87.60565058999997, 41.77299014600003], [-87.60568878999997, 41.77404947100006], [-87.60574164199991, 41.775789814000056], [-87.60579267299991, 41.77767029400007], [-87.60582123899995, 41.77948403200002], [-87.60584137699996, 41.781365112000046], [-87.60587373599996, 41.783175856000064], [-87.60591724899994, 41.78498944400006], [-87.60593518199993, 41.78573395000005], [-87.60575491499995, 41.785730023000035], [-87.60429291499996, 41.785727022000046], [-87.60274791499994, 41.785726022000055], [-87.60107091499995, 41.78575002300004], [-87.59778291599994, 41.78582402200003], [-87.59619491499994, 41.78585802200006], [-87.59454891499995, 41.785902022000066], [-87.59294291599994, 41.78592902200005], [-87.59221891599995, 41.78594602300005], [-87.59142091599993, 41.786033023000066], [-87.59084391599997, 41.78607302200003], [-87.59034091499993, 41.78611102300004], [-87.58965391499996, 41.786106022000055], [-87.58923091599995, 41.78608002200007], [-87.58910291599993, 41.78608102300006], [-87.58907720299997, 41.786081305000025], [-87.58901191599992, 41.786082023000056], [-87.58812291599996, 41.786090022000046], [-87.58657991599992, 41.786087022000046], [-87.58653078199994, 41.78608689000002], [-87.58653805899993, 41.786375146000054], [-87.58649491499993, 41.78637902300004], [-87.58627691599996, 41.786387021000046], [-87.58608391299995, 41.786366021000035], [-87.58584491599997, 41.78629502200003]]]}, "properties": {"city": "Chicago", "name": "Woodlawn", "regionid": "276138", "geo_point_2d": [41.779332582005196, -87.58830824677976], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.68549455999992, 41.71363247400006], [-87.68435104599988, 41.71363194600006], [-87.68305004599992, 41.71363194600006], [-87.6817525139999, 41.71363194600006], [-87.68174880899993, 41.71363375500004], [-87.68174899999988, 41.713575893000026], [-87.6817317459999, 41.71302287200005], [-87.6816756889999, 41.71098049600004], [-87.68162335799988, 41.70921741200003], [-87.68156892699993, 41.70726227200003], [-87.68152496799989, 41.70559093700005], [-87.6814259909999, 41.70364149300007], [-87.6813562259999, 41.701874380000035], [-87.68131435199989, 41.700107740000064], [-87.68127966099989, 41.699060995000075], [-87.6818489159999, 41.69905202300002], [-87.6825879159999, 41.69904402200006], [-87.68378691599989, 41.69904402200006], [-87.68501791499992, 41.69901002200003], [-87.68553391599991, 41.69901002200003], [-87.6861059159999, 41.699009022000034], [-87.6862379159999, 41.69900202200005], [-87.6868949149999, 41.69900402200005], [-87.6874389149999, 41.699001022000054], [-87.68800991599993, 41.69898302200004], [-87.68869091499991, 41.69899002200008], [-87.6894989159999, 41.69899002200008], [-87.68995291599991, 41.69899002200008], [-87.69111591599993, 41.698968022000074], [-87.69114949599991, 41.698968015000034], [-87.691149, 41.699033], [-87.69604999999989, 41.699033], [-87.6960491819999, 41.69900700200007], [-87.69614300499988, 41.699008004000056], [-87.69619291499991, 41.700428022000025], [-87.69624091499989, 41.70137302300008], [-87.69627791499991, 41.70257702200006], [-87.6963279159999, 41.704209022000036], [-87.6963679159999, 41.70515402200003], [-87.69638491599993, 41.706188022000035], [-87.69638591599993, 41.70624102200007], [-87.6963879159999, 41.70629302200006], [-87.69641892299987, 41.70708004200008], [-87.69615991399989, 41.707142021000024], [-87.69565291499991, 41.70714602300006], [-87.6942788499999, 41.70717589300005], [-87.69427291599992, 41.70717589300005], [-87.6939239149999, 41.70717589300005], [-87.69365, 41.70717589300005], [-87.69255135599988, 41.70718409100003], [-87.6913625729999, 41.70713240400004], [-87.69135215499993, 41.707132584000064], [-87.6913481429999, 41.70713321400006], [-87.6913598449999, 41.70719136300004], [-87.69149430299989, 41.70786365400004], [-87.69154451299991, 41.70811470500006], [-87.69155197499991, 41.70813247100004], [-87.69154816799993, 41.708170542000055], [-87.6915196179999, 41.70845604000004], [-87.69145202799989, 41.709132378000035], [-87.69145202799989, 41.70918948500007], [-87.69145202799989, 41.70997525900008], [-87.69145202799989, 41.710031946000065], [-87.69154832299994, 41.710933755000035], [-87.69161964199989, 41.71157562300003], [-87.69165202799988, 41.71183259400003], [-87.69165202799988, 41.71188948500008], [-87.69165202799988, 41.712445066000036], [-87.69165202799988, 41.712502185000055], [-87.69165202799988, 41.71255929200003], [-87.69165202799988, 41.712584281000034], [-87.69165202799988, 41.712641400000045], [-87.69165202799988, 41.71267525900002], [-87.69165202799988, 41.71269850700008], [-87.69165195999992, 41.712732492000036], [-87.69155101699992, 41.71353203000006], [-87.69121875599987, 41.71355971900004], [-87.69106448699993, 41.71357305400004], [-87.69035101299991, 41.71363247400006], [-87.68915104699992, 41.71373239500007], [-87.68785099299991, 41.713632510000075], [-87.6867119809999, 41.71363247400006], [-87.68665101299987, 41.71363247400006], [-87.6858510129999, 41.71363247400006], [-87.68582155099993, 41.71363247400006], [-87.68549455999992, 41.71363247400006]]]}, "properties": {"city": "Chicago", "name": "West Beverly", "regionid": "403374", "geo_point_2d": [41.7056166236578, -87.68807403703572], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.67811691547297, 41.7500730228243], [-87.67807891551985, 41.7493400223105], [-87.67803291609742, 41.74846002311423], [-87.67803491618972, 41.74833702284027], [-87.67804022218749, 41.74798227256169], [-87.6783401506022, 41.74693252311024], [-87.67871176587, 41.746096388757806], [-87.6790833811379, 41.745353158221945], [-87.6801053231244, 41.74479573532024], [-87.6808485536601, 41.74460992768634], [-87.6817775918297, 41.74451702386952], [-87.68214920709748, 41.74451702386952], [-87.68260684014183, 41.74451732326256], [-87.68261663498873, 41.74488857449711], [-87.68263021682615, 41.74572452736085], [-87.68265777748712, 41.74693110210052], [-87.68268531226842, 41.74776936805407], [-87.68270732060597, 41.748883495759976], [-87.68273180389816, 41.74971870525248], [-87.68274, 41.75002], [-87.68271412645261, 41.75005973867451], [-87.68214920709748, 41.75002], [-87.681034361294, 41.75005000000012], [-87.67963956669116, 41.75005609114317], [-87.67811691547297, 41.7500730228243]]]}, "properties": {"city": "Chicago", "name": "Beverly View", "regionid": "403368", "geo_point_2d": [41.747525968036776, -87.68055044440987], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.64846276299993, 41.91717460000006], [-87.64836154299988, 41.914419744000035], [-87.64828793699989, 41.912074719000074], [-87.64826105699989, 41.91090986400007], [-87.6494739149999, 41.910898022000076], [-87.6510949149999, 41.91087402200008], [-87.65308991599993, 41.91083502200006], [-87.6540909159999, 41.91081502200006], [-87.6554889159999, 41.910799022000035], [-87.65599391599991, 41.91079002200006], [-87.65639799999988, 41.910783], [-87.6567575479999, 41.91078014400005], [-87.65706346999988, 41.91154494900007], [-87.6574650979999, 41.91254901700005], [-87.6577997869999, 41.913419209000025], [-87.65798721299988, 41.913914549000076], [-87.6583486769999, 41.91434295100002], [-87.65892434299988, 41.91490522900005], [-87.65966065899993, 41.91541395700005], [-87.66037019999992, 41.91581558400007], [-87.66085215299992, 41.916083336000035], [-87.66116006699988, 41.91612349800003], [-87.66152153099993, 41.91602978500002], [-87.66207042199993, 41.91578880900004], [-87.66244527399992, 41.91557460800003], [-87.66287367599992, 41.91550767000005], [-87.66322175299987, 41.91552105800007], [-87.6635296669999, 41.915628158000054], [-87.66379741799992, 41.91586913500003], [-87.66398484399988, 41.91625737400005], [-87.66419904499989, 41.91679287700003], [-87.66425933899988, 41.91701252100005], [-87.66403499999988, 41.917018], [-87.66227445399993, 41.91706100100003], [-87.6613031239999, 41.91707477800008], [-87.65944031099988, 41.91710148300005], [-87.65817999999989, 41.91714], [-87.65820330099989, 41.91766261400004], [-87.65821999999989, 41.91803], [-87.6566058529999, 41.91806320100004], [-87.65522903899992, 41.91807255500003], [-87.65409559399987, 41.91808566200005], [-87.65271962299992, 41.918117510000044], [-87.65158686999987, 41.91813914300006], [-87.65045355999989, 41.91815], [-87.64940132899991, 41.91815765300002], [-87.64849999999988, 41.91818], [-87.64846276299993, 41.91717460000006]]]}, "properties": {"city": "Chicago", "name": "Ranch Triangle", "regionid": "403305", "geo_point_2d": [41.91474497118594, -87.65400135912189], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.83926295199988, 41.966529560000076], [-87.83856001299992, 41.96652947400007], [-87.83806301299988, 41.96652947400007], [-87.83783192899993, 41.96652947400007], [-87.83676301299991, 41.96652947400007], [-87.83676301299991, 41.96647236100005], [-87.83676301299991, 41.96528658700003], [-87.8367630259999, 41.965229448000045], [-87.83679201199992, 41.96468647400007], [-87.836863, 41.963329499000054], [-87.8368630129999, 41.96327236100006], [-87.8368630129999, 41.96308658700007], [-87.83686333999991, 41.96302961700006], [-87.83676586899989, 41.96157231100005], [-87.83676276599991, 41.96152921500003], [-87.83676586899989, 41.96151519200003], [-87.83685485399988, 41.96107027000004], [-87.8368632599999, 41.961029732000036], [-87.83685485099993, 41.961013150000035], [-87.83676637899991, 41.96083620700004], [-87.8367627079999, 41.96082928700007], [-87.83676637599987, 41.960779087000034], [-87.83682801199987, 41.959855475000026], [-87.83686205299993, 41.959329516000075], [-87.83666710799989, 41.959334137000035], [-87.83666609499991, 41.95927755600007], [-87.83666783599993, 41.95927103500008], [-87.83739296899994, 41.95925666100003], [-87.83878906099993, 41.959230903000034], [-87.84042775399988, 41.95920881400007], [-87.84103488899989, 41.95920233200006], [-87.8418848769999, 41.95919325800007], [-87.84249061399991, 41.95919969700008], [-87.8429686889999, 41.959248822000056], [-87.84392413099991, 41.959484857000064], [-87.8442477219999, 41.95959886100007], [-87.8446814689999, 41.95982499900002], [-87.8452514009999, 41.96019467200004], [-87.84586419299988, 41.960826236000045], [-87.84618722199988, 41.96135351700007], [-87.84629741499991, 41.96162050400004], [-87.84641544199988, 41.96200885800004], [-87.84646487999991, 41.96256378200007], [-87.84646127399989, 41.96323138400003], [-87.8464515999999, 41.96396121700007], [-87.84645665999992, 41.964629102000046], [-87.84646, 41.96529670300002], [-87.84646, 41.96578246600006], [-87.8466415599999, 41.96627459600005], [-87.8466592429999, 41.96632252600005], [-87.84666179999992, 41.966329498000036], [-87.8466425799999, 41.96633117700008], [-87.84622091099992, 41.966366316000055], [-87.84546302699991, 41.966429432000034], [-87.84416301299987, 41.966429474000044], [-87.84286299699991, 41.96642952000007], [-87.84176306199991, 41.96652937100003], [-87.84113607599993, 41.96645110700006], [-87.8409629569999, 41.96642965900007], [-87.84085910399992, 41.96645545100005], [-87.8405630159999, 41.96652940900008], [-87.83976309199993, 41.966629324000046], [-87.83926295199988, 41.966529560000076]]]}, "properties": {"city": "Chicago", "name": "Schorsch Forest View", "regionid": "403280", "geo_point_2d": [41.962986570330564, -87.84144479017925], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.66231091532124, 41.97623902246355], [-87.66351391591672, 41.976227021910375], [-87.66353412686175, 41.97683661823837], [-87.6635614181649, 41.97752279957448], [-87.66361363961857, 41.97876371122729], [-87.66364048192507, 41.97938108427748], [-87.66368260607555, 41.98034146246024], [-87.66372201844403, 41.98123316729848], [-87.66377118209353, 41.982539104678395], [-87.66378994345294, 41.98347717265245], [-87.66290877307947, 41.98349620898816], [-87.6621526662879, 41.983503338800986], [-87.6609850559568, 41.98351563403249], [-87.66007816157598, 41.98352848327984], [-87.66005400708406, 41.98274588036973], [-87.66003352549863, 41.98212801921042], [-87.66000924880696, 41.98144200486704], [-87.65999332499688, 41.981030260633666], [-87.65997454193902, 41.98054458442456], [-87.65994567871208, 41.97951460216338], [-87.6599381684968, 41.978827417456465], [-87.6599344133891, 41.97848382510315], [-87.65991432853683, 41.97772935101167], [-87.65988673939387, 41.977112043938924], [-87.65987141209222, 41.97676909556505], [-87.65985046058252, 41.97629391419835], [-87.66109291493359, 41.97625602234772], [-87.66231091532124, 41.97623902246355]]]}, "properties": {"city": "Chicago", "name": "Lakewood - Balmoral", "regionid": "403260", "geo_point_2d": [41.97989235502535, -87.6618140521251], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.58002491555817, 41.77345102218306], [-87.57894691532881, 41.77342102259891], [-87.57766191544997, 41.77342302179172], [-87.5764099152936, 41.77344002257543], [-87.57640406317411, 41.773440116208974], [-87.57641535452632, 41.77248153978515], [-87.57639866610259, 41.77052928651552], [-87.57635958306145, 41.76885693592933], [-87.57629720459585, 41.76681495626283], [-87.57627999999987, 41.7661], [-87.57698227683967, 41.766094336477025], [-87.57846817108413, 41.76609441541406], [-87.57967629699054, 41.76608], [-87.58069804526444, 41.766058311792804], [-87.58123, 41.76605], [-87.58127441678589, 41.767370820792074], [-87.58131430908345, 41.76904037238514], [-87.5813416076179, 41.770808881813096], [-87.58137203187951, 41.77257346645684], [-87.58139, 41.7734], [-87.58138992345368, 41.773401048469545], [-87.58066291529013, 41.77342002255281], [-87.58012691576351, 41.773446021952516], [-87.58002491555817, 41.77345102218306]]]}, "properties": {"city": "Chicago", "name": "Jackson Park Highlands", "regionid": "403354", "geo_point_2d": [41.76975591818472, -87.57884586650198], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.5873200044695, 41.802450011272654], [-87.5871086023473, 41.80185029191093], [-87.58689956875916, 41.80080512397002], [-87.58676021303383, 41.79996898961742], [-87.58669053517104, 41.79857543236302], [-87.58669053517104, 41.797181875108635], [-87.58689956875916, 41.79578831785433], [-87.58731763593552, 41.7936979819727], [-87.58766602524919, 41.79216506899277], [-87.58829312601357, 41.78958698807221], [-87.58870069601586, 41.78788992266506], [-87.58885054891527, 41.787078585014314], [-87.58907720344376, 41.786081305089354], [-87.58910291583828, 41.78608102252152], [-87.58923091634261, 41.786080022475524], [-87.58965391515717, 41.78610602187532], [-87.59034091534927, 41.78611102300514], [-87.59084391605268, 41.78607302215263], [-87.59142091567072, 41.78603302300685], [-87.59221891558371, 41.785946022593755], [-87.5929429156826, 41.785929021810226], [-87.59454891507013, 41.78590202236416], [-87.5961949154019, 41.78585802213457], [-87.59778291575836, 41.785824022365794], [-87.60107091533841, 41.78575002255214], [-87.60274791530061, 41.78572602234507], [-87.60429291547347, 41.78572702239116], [-87.60575491541752, 41.78573002252953], [-87.60593591836498, 41.78573396605653], [-87.60594391603587, 41.78627202233278], [-87.60604696844781, 41.7870950215993], [-87.60597891585066, 41.787608021864585], [-87.60596291601239, 41.78946202189204], [-87.60604691538792, 41.79120002196834], [-87.60607091559491, 41.79199202250381], [-87.6061019161246, 41.79303802305691], [-87.6061149158245, 41.794054022226774], [-87.60616491543135, 41.794877022392846], [-87.6062369160523, 41.79670702221332], [-87.60629491602786, 41.79851902300273], [-87.60630791572795, 41.79887302223382], [-87.6063239164654, 41.80030902277789], [-87.60634191549663, 41.80128802294043], [-87.60634991586535, 41.80155802279606], [-87.60636091547298, 41.802093022276686], [-87.60636390931603, 41.80215302594175], [-87.60621891522267, 41.802155022436715], [-87.604113915316, 41.80218702211314], [-87.60374491539356, 41.80219402243577], [-87.60283991504497, 41.80219602252795], [-87.60154791574274, 41.80221402245839], [-87.6013339157707, 41.80220802218194], [-87.59974291527605, 41.80224802222689], [-87.59907191582147, 41.80225802178861], [-87.59821491498764, 41.802266022157326], [-87.59664191532279, 41.802277021765136], [-87.59499891512912, 41.80229702178784], [-87.59338391532688, 41.802311022433194], [-87.59177891598563, 41.80233702183298], [-87.59047591527694, 41.802358021901775], [-87.58916091581386, 41.80240402222355], [-87.5879649155411, 41.80244402226869], [-87.5878879155888, 41.80244702240706], [-87.5878109156367, 41.80244702240706], [-87.5874049158072, 41.80245002254532], [-87.5873200044695, 41.802450011272654]]]}, "properties": {"city": "Chicago", "name": "Hyde Park", "regionid": "269586", "geo_point_2d": [41.794283972931176, -87.59682773413114], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.65618691631228, 41.70680902319808], [-87.65617101180203, 41.70677885544072], [-87.65602377390007, 41.70678612106322], [-87.65604691435522, 41.70673302329189], [-87.65609391562253, 41.70660302269528], [-87.65668491588593, 41.70497602233987], [-87.65692091507351, 41.704330022239255], [-87.65725291598835, 41.70353702255671], [-87.65732591485683, 41.70336402267608], [-87.65741091607703, 41.70312102226641], [-87.65772891544732, 41.702209022494536], [-87.6579829154647, 41.701470022603374], [-87.65837991577865, 41.70048802230248], [-87.65874987707633, 41.6994199685748], [-87.66045391605479, 41.69939202214279], [-87.66216291479412, 41.69937602230445], [-87.6643289157139, 41.699348021913195], [-87.66527891543899, 41.69932702184448], [-87.66679091588918, 41.699285022606446], [-87.66895991514848, 41.69922202240032], [-87.66946191490662, 41.69922202240032], [-87.66962391517971, 41.69922902272297], [-87.66991691519614, 41.69921502207751], [-87.66994356204061, 41.69921421187237], [-87.66986286899859, 41.69973893992955], [-87.66971966283246, 41.700940525041936], [-87.66955213750623, 41.702255345026295], [-87.66938805207705, 41.70362933651354], [-87.66914623867336, 41.705583620759356], [-87.66888237612955, 41.7073660118864], [-87.66875083060307, 41.70839152129526], [-87.66867604442565, 41.70912343727544], [-87.66855472081006, 41.70968002221544], [-87.6684, 41.71023], [-87.66817647361142, 41.71060713107678], [-87.66780485835045, 41.71135036159843], [-87.66743324308959, 41.712465207381314], [-87.66723999999986, 41.71286], [-87.66731, 41.71391], [-87.66594670007646, 41.71394789782446], [-87.66371712180262, 41.71395812076802], [-87.66223117294783, 41.71397449588217], [-87.66102411998367, 41.714003341120446], [-87.65983969138534, 41.7140211045819], [-87.65931891589578, 41.71304502287321], [-87.65883691616028, 41.71210802194898], [-87.65850991547596, 41.71133802242733], [-87.65844191503932, 41.711158024022524], [-87.65832291584908, 41.71097102169788], [-87.6579259155352, 41.71016702240771], [-87.65751891565964, 41.709359022033894], [-87.65701491580931, 41.70844502216974], [-87.65674491595371, 41.70789902218186], [-87.65658891595722, 41.707584022050725], [-87.65625391580329, 41.706956022779565], [-87.65618691631228, 41.70680902319808]]]}, "properties": {"city": "Chicago", "name": "East Beverly", "regionid": "403375", "geo_point_2d": [41.706239465724664, -87.66325589540513], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.78536295799994, 41.924224761000055], [-87.7853, 41.92253], [-87.7860607299999, 41.922771207000025], [-87.78687161099991, 41.92303990700008], [-87.78803368999988, 41.92318704500007], [-87.78888538999989, 41.92327439900003], [-87.79033091499988, 41.92356202200005], [-87.7903249159999, 41.92347902200003], [-87.79028691599987, 41.92294002300007], [-87.79022691599992, 41.921369023000054], [-87.79021591499989, 41.92106702300003], [-87.79017491499988, 41.92012502200002], [-87.79015691499988, 41.91915302200004], [-87.79013091599988, 41.91885102300005], [-87.79009291499993, 41.91856302300003], [-87.79005591499991, 41.918373022000075], [-87.79000691399993, 41.91821802200007], [-87.78992491499989, 41.91806902300004], [-87.78979391499988, 41.917864022000046], [-87.78962994899989, 41.91768100200004], [-87.78911120899991, 41.91723107300003], [-87.78960491499991, 41.91741502200005], [-87.79110791599992, 41.91794202200003], [-87.79155591599992, 41.91811702200004], [-87.79254591599991, 41.91850302200003], [-87.79408491599989, 41.91904002300003], [-87.79500291599992, 41.91938802200007], [-87.79562091499989, 41.91962202200005], [-87.79693691499993, 41.92010102200003], [-87.79716091499992, 41.92018302200005], [-87.79870291499991, 41.920728022000034], [-87.79986391599992, 41.92113802200003], [-87.80056391599992, 41.92138802300007], [-87.80240891499993, 41.92206802300005], [-87.8028459159999, 41.92223302200004], [-87.80344791599991, 41.92246402200003], [-87.80425891499992, 41.92271002300003], [-87.80521391599987, 41.92297802200004], [-87.80609546399988, 41.923193649000034], [-87.80609999999989, 41.9232], [-87.8062571819999, 41.92321281500004], [-87.80625899999987, 41.923231], [-87.8061589999999, 41.923431], [-87.80625899999987, 41.924431], [-87.806359, 41.925131], [-87.806359, 41.926131], [-87.80636, 41.926931], [-87.80645999999989, 41.927631], [-87.80645999999989, 41.928831], [-87.80645999999989, 41.929331], [-87.80645899999988, 41.930679], [-87.80645999999989, 41.932631], [-87.80656, 41.934331], [-87.80649791599988, 41.93443600800002], [-87.8065579159999, 41.93627602300005], [-87.80660191499992, 41.93750102300003], [-87.80663012899991, 41.93790851400006], [-87.8060129159999, 41.93791402200008], [-87.80538591499992, 41.937920022000064], [-87.80521891499988, 41.93792102200006], [-87.8041519159999, 41.93793702200007], [-87.80388291599988, 41.93794102200007], [-87.8029389149999, 41.93795602200003], [-87.8024769159999, 41.93796402300005], [-87.80110191499992, 41.93798902200006], [-87.7992639159999, 41.93801502200005], [-87.79802091499992, 41.938040022000045], [-87.79681791599987, 41.93804902200008], [-87.79558091499989, 41.93806502200003], [-87.7943759159999, 41.93808702200004], [-87.79314291599988, 41.938108022000044], [-87.79194591599989, 41.93812002300006], [-87.79085000299993, 41.938139075000024], [-87.79066291499991, 41.93814202200007], [-87.78962991599988, 41.93816302200007], [-87.78948591599992, 41.938165022000064], [-87.78840891599991, 41.93818402200008], [-87.78826291599992, 41.93818702200007], [-87.7871679159999, 41.93820802200007], [-87.78702191599987, 41.93821102100008], [-87.7859301849999, 41.93823293700006], [-87.7859239999999, 41.938233060000066], [-87.78586062499991, 41.93793168800004], [-87.78577227199989, 41.936794968000065], [-87.78574339899987, 41.93563463700007], [-87.78571143799988, 41.934842146000044], [-87.78567326099991, 41.93386479000008], [-87.78564756999991, 41.933196811000066], [-87.7856206629999, 41.93221714100007], [-87.78559811699994, 41.93148521300008], [-87.78556757299991, 41.93045019800007], [-87.78553946399987, 41.92941017300006], [-87.78549836199991, 41.92800576200005], [-87.7854309759999, 41.92605319800003], [-87.78536295799994, 41.924224761000055]]]}, "properties": {"city": "Chicago", "name": "Montclare", "regionid": "274579", "geo_point_2d": [41.9294771961269, -87.79587744700854], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.74682694928438, 41.939238478530534], [-87.74681001609804, 41.938725128558325], [-87.74680984130339, 41.93872513330811], [-87.74679464250171, 41.938259036722265], [-87.74676440515059, 41.937466913314054], [-87.7467264127928, 41.93661409743223], [-87.74668106797539, 41.93557876245315], [-87.7466428756976, 41.93459627092742], [-87.74660990926856, 41.93374289049567], [-87.74658448355562, 41.9327669706852], [-87.74656315268832, 41.931975147317615], [-87.74651946223183, 41.93099790021827], [-87.74648245447497, 41.93014628981191], [-87.7464335034689, 41.929046078631266], [-87.7464, 41.92831169908262], [-87.74638091858579, 41.92752123191579], [-87.74632602858355, 41.92648253356046], [-87.74629714518608, 41.925873617425886], [-87.74626, 41.92489409869122], [-87.74626, 41.92416], [-87.74625043193993, 41.92373422132649], [-87.74622021104898, 41.922819801363985], [-87.74619999999987, 41.921659584035204], [-87.74618805818587, 41.92025533827482], [-87.74613128163315, 41.91909881307006], [-87.74610388627063, 41.91842654099169], [-87.74608519497237, 41.91732754750872], [-87.74604585092095, 41.91623507185416], [-87.74602239446347, 41.915805818684], [-87.74597764980864, 41.914886638334295], [-87.74603440210707, 41.914425314765786], [-87.74613991516483, 41.914440022792945], [-87.74626091534653, 41.914455022585194], [-87.74675091545082, 41.91451102246867], [-87.74709891620392, 41.91454802237538], [-87.74755591568629, 41.91458102209807], [-87.74843491573573, 41.914661022188525], [-87.74849091561914, 41.91466602241897], [-87.7493229162, 41.91477602209376], [-87.74936291624533, 41.91478102232413], [-87.7499839151936, 41.914830022784265], [-87.75211591544542, 41.915076022432835], [-87.75274291556977, 41.9151560225233], [-87.75470291598675, 41.915397022840764], [-87.75519391613715, 41.91545002168662], [-87.75574591550213, 41.91545302182489], [-87.75582868823756, 41.91544465413274], [-87.7558609732618, 41.91647122993934], [-87.75589580253842, 41.91763107192232], [-87.7559824648942, 41.91994909728011], [-87.75603644759472, 41.92178081618041], [-87.75604483184935, 41.92263486644968], [-87.756061647439, 41.92361248731597], [-87.75607, 41.92398000000017], [-87.7566734575809, 41.924050856703346], [-87.75789465997657, 41.92402537721371], [-87.75911328854635, 41.924005215415], [-87.7603348206667, 41.923985979983705], [-87.76173681117642, 41.923960000000115], [-87.76295738274496, 41.923960000000115], [-87.76417672609819, 41.92393138373769], [-87.76533749730311, 41.92391], [-87.76590811791128, 41.92390775947266], [-87.76594116051932, 41.92471127050024], [-87.76597205942268, 41.92550581372941], [-87.76600918076885, 41.92660487691334], [-87.76603303615086, 41.92733644196143], [-87.76604879989283, 41.927945995175634], [-87.7660782481157, 41.92886058260526], [-87.76611770537886, 41.92989886674863], [-87.7661377062591, 41.93063063478919], [-87.7661795393671, 41.93209587392393], [-87.76623765060282, 41.93386596386548], [-87.76631084675313, 41.9356376089054], [-87.76638048142644, 41.9375898439845], [-87.76641030990427, 41.93849703675518], [-87.76641035608021, 41.93849703649327], [-87.76643274930359, 41.93917489600995], [-87.76646784588958, 41.94003045546062], [-87.76650292690101, 41.941005449329644], [-87.76652891192428, 41.94179996568297], [-87.76654010842927, 41.94213902331009], [-87.76592323281182, 41.94214761440984], [-87.76367691879268, 41.94217526009576], [-87.76076255091215, 41.94220716284971], [-87.75851664335426, 41.94223090456115], [-87.7558456653863, 41.94227427485091], [-87.75353752116327, 41.942309005887445], [-87.75086420088124, 41.942338184738425], [-87.7478917488204, 41.94236112503484], [-87.74691, 41.94239000000019], [-87.74690123057741, 41.941982221853394], [-87.74688127201638, 41.941067240740914], [-87.74685686632922, 41.940152856208286], [-87.74682694928438, 41.939238478530534]]]}, "properties": {"city": "Chicago", "name": "Cragin", "regionid": "28173", "geo_point_2d": [41.9304111305193, -87.75531125182412], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.67461287090819, 41.99063813028491], [-87.67456691820459, 41.99052702278766], [-87.67455691594503, 41.99043402209802], [-87.67455991608323, 41.9901290224277], [-87.67452991559979, 41.98823702244737], [-87.67451491670691, 41.98678002273378], [-87.67453191659125, 41.986690024880346], [-87.67456691550657, 41.98661002299114], [-87.67460191442217, 41.986517021402136], [-87.67461891430634, 41.98642302246497], [-87.67460291536744, 41.98445802271681], [-87.67458991566764, 41.98411502219407], [-87.6745660011102, 41.983261716325075], [-87.67452191613025, 41.98238702257884], [-87.67452591541534, 41.98149102264522], [-87.67448891550839, 41.98059202257315], [-87.67446091511695, 41.979664022963014], [-87.67437691574149, 41.97875702252222], [-87.6743509172409, 41.977936022448425], [-87.67436691617995, 41.977857022404045], [-87.6744079675326, 41.97771603209218], [-87.67439391562584, 41.97695402214761], [-87.67438899168468, 41.97605407509846], [-87.67473181682531, 41.9760521409308], [-87.67521015000887, 41.97604], [-87.67603510913148, 41.97603361289001], [-87.67692851605236, 41.976019286917946], [-87.67754680208417, 41.97600675400511], [-87.67850894016661, 41.97599583491619], [-87.67899000599245, 41.975992648983606], [-87.68029575982675, 41.97598411822209], [-87.6813951509869, 41.97597465321872], [-87.68249622311414, 41.975961904190754], [-87.68366406084796, 41.9759540381592], [-87.68503935276244, 41.97593458111202], [-87.68579502602711, 41.975919032193076], [-87.6873063790763, 41.975885580920384], [-87.68908006719748, 41.97586058194399], [-87.68912921285143, 41.9762861397941], [-87.68912858885422, 41.97710754015144], [-87.68916569304984, 41.978073408757865], [-87.6891884502084, 41.978759920957664], [-87.68921206283245, 41.979789530184334], [-87.68921729185703, 41.98075167522496], [-87.68926123022999, 41.981780105958244], [-87.68930904992857, 41.98239492686851], [-87.68935922809312, 41.98305073489098], [-87.68942791526455, 41.984843022477584], [-87.6894849160934, 41.98671002220485], [-87.68954191602297, 41.98854802239423], [-87.68959804810669, 41.99037515067934], [-87.68954670722685, 41.99037602551573], [-87.68836591588625, 41.9903950220495], [-87.687495916069, 41.99041302201475], [-87.68714991523852, 41.99042002217609], [-87.6856269152095, 41.990449021945246], [-87.68369291515064, 41.990468021933495], [-87.68173391591475, 41.99051802218651], [-87.6797999158558, 41.990553022093835], [-87.6783839160447, 41.99057602217442], [-87.67705491504071, 41.99059702315831], [-87.67523591588407, 41.99077707489077], [-87.67495591522577, 41.990731021199956], [-87.67461287090819, 41.99063813028491]]]}, "properties": {"city": "Chicago", "name": "Bowmanville", "regionid": "403263", "geo_point_2d": [41.983269193948175, -87.68191600962888], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.6422559218246, 41.89648484390365], [-87.64291, 41.896480000000146], [-87.64294315560524, 41.89710521998651], [-87.64300636460577, 41.89863839446641], [-87.64305421179267, 41.90001345366314], [-87.64308542589384, 41.90082492971642], [-87.64313747631259, 41.9021184409772], [-87.6431800916386, 41.90316920200499], [-87.6431983270477, 41.90371243973028], [-87.63895626934394, 41.90377233045945], [-87.63895626934394, 41.90278639103477], [-87.63891819288966, 41.90197084266221], [-87.63889158868157, 41.90174217989287], [-87.63888, 41.901526425042256], [-87.63887364322643, 41.90134201034257], [-87.63882999999987, 41.90048], [-87.6388179561186, 41.900245144314255], [-87.63880514990696, 41.89999542318568], [-87.63877953748339, 41.89949598092844], [-87.63875722988887, 41.899060982833205], [-87.6387455090483, 41.898778535021314], [-87.63873033044798, 41.89830040911], [-87.63870396600527, 41.89776830026625], [-87.63868019766409, 41.89695320306364], [-87.63866999999986, 41.89653], [-87.63914373903135, 41.896522104349486], [-87.64051710205615, 41.89651], [-87.6422559218246, 41.89648484390365]]]}, "properties": {"city": "Chicago", "name": "Cabrini Green", "regionid": "403302", "geo_point_2d": [41.900115880892905, -87.640937744877], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.6337397026425, 41.72996999541133], [-87.63373542905391, 41.72993112460679], [-87.63448954899702, 41.72993112460697], [-87.63560439480042, 41.72993112460697], [-87.63690504823784, 41.72993112460697], [-87.63801989404132, 41.72993112460697], [-87.639041836028, 41.72993112460697], [-87.6420147581706, 41.72993112460697], [-87.6443373535946, 41.72993112460697], [-87.64508058413047, 41.73020983605787], [-87.64545219939808, 41.73039564369151], [-87.64600962229997, 41.73095306659348], [-87.64638123756785, 41.731510489495264], [-87.64638123756785, 41.73188210476307], [-87.64651938130645, 41.73255153457954], [-87.64656691532939, 41.73255102271148], [-87.6466929157417, 41.732547022526944], [-87.6470449148806, 41.73253302278105], [-87.64723191630597, 41.73253002264278], [-87.64782991599282, 41.732520022181696], [-87.6479299161057, 41.7325110217667], [-87.64801891571179, 41.732487022459274], [-87.64822891549957, 41.73245602282868], [-87.64837691602662, 41.73245902206776], [-87.64872491588042, 41.732454021837306], [-87.64960191493826, 41.73246802248275], [-87.6508309149338, 41.73244502232186], [-87.65202091582931, 41.73241702193052], [-87.65293291560127, 41.732402022138174], [-87.65320691564119, 41.73237302170093], [-87.65338691494536, 41.732354022623596], [-87.65441191542959, 41.73233702184005], [-87.65566791487093, 41.732312022486006], [-87.65685991495951, 41.73229302250965], [-87.65807891539326, 41.73227702177221], [-87.65928591527401, 41.7322670222105], [-87.66052291563811, 41.73224802223397], [-87.66173991597968, 41.73223102234962], [-87.66231491550545, 41.73223102234962], [-87.66251191559334, 41.73222402292619], [-87.66259991515315, 41.73221702260356], [-87.66273591512702, 41.73220302285766], [-87.6628109158864, 41.73219302239676], [-87.66297091606731, 41.7321650229046], [-87.66312891615597, 41.73215402239744], [-87.66340291529659, 41.73212402281328], [-87.66361191593748, 41.732112022260026], [-87.66381291531027, 41.732102022698314], [-87.66392312209717, 41.7320983489737], [-87.66449748187512, 41.73169629712908], [-87.66496200095958, 41.731510489495264], [-87.66535024960986, 41.73132554195639], [-87.66535119534508, 41.7313698766809], [-87.66589103912933, 41.73104597041038], [-87.66644846203114, 41.730581451325676], [-87.66664582759238, 41.73020940025324], [-87.6666258211182, 41.730210219165464], [-87.66693164262313, 41.72966107742716], [-87.66703556231765, 41.729437250392735], [-87.66729878548287, 41.72899478322459], [-87.66742160883605, 41.728667372530836], [-87.66754591561835, 41.72889802249378], [-87.66797087045829, 41.729835048599206], [-87.66824691016039, 41.73016492891285], [-87.66851691541217, 41.730463022688994], [-87.66925291606422, 41.73118002246553], [-87.66940091569198, 41.731305021932485], [-87.66986191535877, 41.73169502282291], [-87.66995191501087, 41.73178602252117], [-87.67072491557008, 41.73257602296455], [-87.67153891622033, 41.733494023012966], [-87.67250291479206, 41.734693022524915], [-87.6727929155695, 41.73512402260729], [-87.67301991524162, 41.73559602278135], [-87.67304685083572, 41.735652095509856], [-87.6730749159783, 41.735710022640326], [-87.67313991537723, 41.735845022568185], [-87.67337491631734, 41.73625702177485], [-87.67375391580158, 41.73675402220179], [-87.67437791578757, 41.73747902234687], [-87.6748589154769, 41.73802802247276], [-87.67525391659807, 41.738523021908335], [-87.67565091601256, 41.73902102238137], [-87.67652591497827, 41.73997202215257], [-87.67727291523815, 41.74093702256886], [-87.67729591539904, 41.74097802266], [-87.67738391495895, 41.741143022172366], [-87.67747791479539, 41.7414050225582], [-87.67760891543817, 41.74188802233988], [-87.67785691427946, 41.742623020248104], [-87.67788191543198, 41.74283802206481], [-87.67795091501544, 41.743818022273615], [-87.67797391517634, 41.74480002257407], [-87.67803691538236, 41.74644302276774], [-87.67804291565909, 41.746600022810576], [-87.67804891593553, 41.74734902226326], [-87.67804891593553, 41.74740102286157], [-87.6780402221874, 41.74798227256168], [-87.67803491618956, 41.74833702284043], [-87.67803291609738, 41.7484600231144], [-87.67807891551978, 41.74934002231075], [-87.67811691547283, 41.750073022824544], [-87.679639566691, 41.75005609114343], [-87.68103436129394, 41.75005000000037], [-87.68214920709744, 41.75002000000019], [-87.68271412645264, 41.750059738674594], [-87.68285999999983, 41.75007], [-87.68278220070998, 41.75093166828541], [-87.68280431174573, 41.75157971681972], [-87.68283593247702, 41.75278950165949], [-87.68285098696428, 41.7533465176832], [-87.68286983851687, 41.75399680024717], [-87.68289866283384, 41.75492494325158], [-87.68293035355411, 41.75613166728943], [-87.68295073690085, 41.75687410758996], [-87.68296965068002, 41.75750101906309], [-87.68290791598764, 41.75750102258827], [-87.68175191575999, 41.75751502233434], [-87.68170091520777, 41.75751702242652], [-87.68150491606535, 41.75752402274916], [-87.67944791567356, 41.757533022264774], [-87.6784939148648, 41.75754802205677], [-87.67844591624942, 41.75754802565389], [-87.67689691589236, 41.75759002219418], [-87.67603491482788, 41.7575960224709], [-87.67528191519067, 41.75762302191677], [-87.67448391527783, 41.75762402196286], [-87.67392291549804, 41.75762502200878], [-87.67318791579108, 41.75763702256222], [-87.67211891507745, 41.757640021801116], [-87.67127491574206, 41.75764602207766], [-87.66956491515799, 41.75764602207766], [-87.66847691536682, 41.75765602253875], [-87.66777391533655, 41.75765602253875], [-87.66722691530275, 41.75765702258484], [-87.66624091571695, 41.75767702260745], [-87.66592391549347, 41.75767702260745], [-87.6647159155668, 41.75769502253788], [-87.66353391504002, 41.75771502256032], [-87.66349074457716, 41.75771502256032], [-87.66347691600981, 41.75771502256032], [-87.66342391536523, 41.75771502256032], [-87.66310291585683, 41.75772102193768], [-87.66211491527939, 41.757748022282925], [-87.66095091558213, 41.757741021960115], [-87.66042291552513, 41.757731022398396], [-87.65868191531054, 41.757741021960115], [-87.6572439155734, 41.75776602221336], [-87.6558279159512, 41.75779002242034], [-87.65383891959321, 41.75780696744599], [-87.65263891553877, 41.75782202209666], [-87.65142791547356, 41.757844022211636], [-87.6502939153609, 41.75786402223425], [-87.65021591536252, 41.757865022280164], [-87.64899691492886, 41.75788602234905], [-87.64841591602575, 41.75789402181841], [-87.64778891500211, 41.75790202218731], [-87.64658691535209, 41.757922022209925], [-87.64535591616355, 41.75793702200191], [-87.64419291561327, 41.75794402232472], [-87.64415278876389, 41.757944150927806], [-87.64415254315485, 41.75793675914337], [-87.64415, 41.757940000000204], [-87.63920304532022, 41.757935155908854], [-87.63898591570745, 41.75791602283239], [-87.63879791603446, 41.75791102260194], [-87.63838991521368, 41.75791102260194], [-87.63692791526957, 41.75792802248629], [-87.63654091541669, 41.757931022624554], [-87.63632591539846, 41.757938022048], [-87.63564378053849, 41.757950890446786], [-87.63490235173833, 41.75795138657633], [-87.63381769496327, 41.757969714597564], [-87.632514574323, 41.757992834548496], [-87.63138000000001, 41.75802000000025], [-87.63129289801986, 41.758022304811874], [-87.63171055853722, 41.75715589371221], [-87.63203740147564, 41.75648379414848], [-87.6323545664158, 41.75580813756642], [-87.63270378080993, 41.75505206592539], [-87.63294313366657, 41.75456689121645], [-87.6332989582679, 41.753813125196004], [-87.63352691252524, 41.753054333103016], [-87.63379807890978, 41.752055355410626], [-87.63403896499368, 41.751116737691085], [-87.6341541778325, 41.75067596271528], [-87.63428407018964, 41.75017900473011], [-87.63454642870305, 41.749153481599876], [-87.63477350720073, 41.74828509350085], [-87.63499352641736, 41.74747542278371], [-87.63524411548835, 41.746539395439896], [-87.63547055451056, 41.74562198845556], [-87.63568776285467, 41.74468006715265], [-87.63582, 41.74420000000021], [-87.63581376327195, 41.74407190340467], [-87.63587896673876, 41.74409720780431], [-87.635968916029, 41.74397502231591], [-87.6362808001218, 41.74356155490431], [-87.63704526356011, 41.742710818266914], [-87.6377005659529, 41.74195970462203], [-87.63736956732254, 41.741265390276105], [-87.6372766635058, 41.740243448289604], [-87.63690504823784, 41.739871833021624], [-87.63588310625124, 41.738849891034945], [-87.63399729032328, 41.73628518137323], [-87.63399691595804, 41.73617802173047], [-87.63401575585533, 41.73597894960475], [-87.63405509040216, 41.73593407073755], [-87.6340479156108, 41.73589202203667], [-87.63400691641885, 41.735611021674146], [-87.6339669154746, 41.73436202255544], [-87.63390791545275, 41.732664022524546], [-87.63387591577644, 41.73164802245505], [-87.63384091596156, 41.73117702232708], [-87.63374091494921, 41.729981022054396], [-87.6337397026425, 41.72996999541133]], [[-87.63930492051979, 41.75059507746389], [-87.63922391588656, 41.75059602265129], [-87.63885391501877, 41.75060302297391], [-87.63868191518395, 41.75061502262781], [-87.63799591503792, 41.75062402214325], [-87.63787691494841, 41.75062602223561], [-87.63666491573674, 41.750634022604515], [-87.63544491615617, 41.750650022442585], [-87.6342400011748, 41.75067497859529], [-87.63424039068136, 41.75232546954294], [-87.63429783152223, 41.753157472836214], [-87.63431719698026, 41.75371424298674], [-87.63432, 41.75432000000018], [-87.63504756775711, 41.75430797408676], [-87.63616210708726, 41.754289721836024], [-87.63718374400865, 41.75427117199969], [-87.63967488048173, 41.7542337097177], [-87.63966391548465, 41.75290202283018], [-87.63964891569259, 41.75239902302571], [-87.63957604452791, 41.75059488770688], [-87.63930492051979, 41.75059507746389]]]}, "properties": {"city": "Chicago", "name": "Gresham", "regionid": "269571", "geo_point_2d": [41.74517709208001, -87.65641161948184], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.75203297083796, 41.79718685841956], [-87.75256, 41.79706], [-87.75261090923198, 41.79794727541], [-87.75264205068159, 41.798677545791584], [-87.7526740312944, 41.799652954483356], [-87.75269, 41.80014], [-87.75284944055254, 41.800324046218876], [-87.75275399971366, 41.80033199955651], [-87.75275399971366, 41.80203199967976], [-87.75275399971366, 41.802425572677144], [-87.75275399971366, 41.8038319999159], [-87.7528477918067, 41.80561403799318], [-87.75285399982675, 41.80573200026505], [-87.75285399982675, 41.80576881311319], [-87.75285399982675, 41.80693199982308], [-87.75285399982675, 41.807435567164944], [-87.75095493826603, 41.80744582695646], [-87.74989717207637, 41.80745921379876], [-87.74851422241687, 41.80748177244135], [-87.74713129022801, 41.80750435883926], [-87.74623640759717, 41.807518787181074], [-87.74509693526318, 41.8075336700402], [-87.74387645137092, 41.80754401312392], [-87.74310725356129, 41.80754715576793], [-87.74308688984524, 41.806731900526394], [-87.7430200965072, 41.80469680868682], [-87.74298055185625, 41.80347774742332], [-87.74293937984619, 41.801442790778005], [-87.74292999999977, 41.79858000000014], [-87.74346726880754, 41.79850259686663], [-87.74468716012758, 41.79833368130554], [-87.74599805625574, 41.79815394203044], [-87.7469779179694, 41.79801094887774], [-87.74843491735739, 41.79779845241251], [-87.74924063008785, 41.79766843659725], [-87.7508588429108, 41.79737143106828], [-87.75203297083796, 41.79718685841956]]]}, "properties": {"city": "Chicago", "name": "Vittum Park", "regionid": "403333", "geo_point_2d": [41.8026745060038, -87.74798456435497], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.6393744189999, 41.88571680100005], [-87.63839794299989, 41.88570620700004], [-87.63813705399988, 41.885709142000046], [-87.63788010299992, 41.88570635500007], [-87.63772708499992, 41.885444037000056], [-87.63782953299993, 41.88465860200005], [-87.6379149059999, 41.88395854100008], [-87.63805150399989, 41.88358289800003], [-87.63815395199991, 41.883173106000065], [-87.6381881009999, 41.88267794100005], [-87.63820517599991, 41.882148626000046], [-87.63825639999992, 41.88168761000007], [-87.6382905489999, 41.88129489200007], [-87.63827347399989, 41.88071435400006], [-87.63815395199991, 41.880133815000065], [-87.63805150399989, 41.879484977000054], [-87.63793198099992, 41.878989812000036], [-87.63781245799991, 41.87842634800005], [-87.63765878599989, 41.87793118200005], [-87.6376246369999, 41.87731649400006], [-87.63753926299992, 41.87690670200003], [-87.6373685169999, 41.87646276100008], [-87.63683920199989, 41.875830998000076], [-87.63666007599993, 41.875608635000034], [-87.6370159779999, 41.87555440100004], [-87.63888784299988, 41.87563411200004], [-87.64004582599993, 41.87562], [-87.64163146299988, 41.875681835000066], [-87.64401268299989, 41.87574], [-87.64548307099993, 41.87576258600006], [-87.64559816599987, 41.876713558000056], [-87.6457586469999, 41.87771173400005], [-87.64591283399993, 41.87912366200003], [-87.64588, 41.88040426500003], [-87.6458438209999, 41.88180316600004], [-87.64579, 41.882052037000044], [-87.6457699999999, 41.883516723000064], [-87.64571598899991, 41.88429146800006], [-87.64601999999988, 41.88568], [-87.64599041099989, 41.88570457100007], [-87.64559825399992, 41.885689131000035], [-87.64480596599991, 41.88570441900002], [-87.6433413689999, 41.88571], [-87.6417546449999, 41.88571601100005], [-87.64047274299989, 41.88573779700004], [-87.6393744189999, 41.88571680100005]]]}, "properties": {"city": "Chicago", "name": "West Loop Gate", "regionid": "403322", "geo_point_2d": [41.880641123489006, -87.64181328920239], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.721050999712, 41.735033000425865], [-87.72155100027733, 41.735033000425865], [-87.72109718543598, 41.735786601173096], [-87.71968902667447, 41.736907169922894], [-87.71840928034838, 41.73793121699038], [-87.71715391431762, 41.73891696440679], [-87.71589386053809, 41.73989799000425], [-87.71480929519123, 41.74074671756423], [-87.71375377838002, 41.741566173441605], [-87.71268164123944, 41.74240451103454], [-87.71214177915152, 41.7428250351872], [-87.71203, 41.73862], [-87.71202448481647, 41.738107087932995], [-87.71203153062078, 41.73717688737376], [-87.71202785348494, 41.73643560454965], [-87.71199647207982, 41.73550769031672], [-87.7120212608108, 41.73513299963966], [-87.71205100032954, 41.73513299963966], [-87.71425100011874, 41.73513299963966], [-87.71665100013382, 41.73513299963966], [-87.71905100014907, 41.735033000425865], [-87.721050999712, 41.735033000425865]]]}, "properties": {"city": "Chicago", "name": "Parkview", "regionid": "403365", "geo_point_2d": [41.73769075577763, -87.71537212729538], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.6665, 41.867222092086486], [-87.66636663505001, 41.8664280563162], [-87.66634614545252, 41.86563538163092], [-87.66633231407114, 41.86472029024045], [-87.66630137572878, 41.8637456099652], [-87.66627042925246, 41.86282974561132], [-87.66624618950794, 41.8621014111441], [-87.66621, 41.86099717708792], [-87.66618756277938, 41.85962602225051], [-87.66618791597189, 41.85962602225051], [-87.66623991567093, 41.85962002197406], [-87.66637091631362, 41.859605023081166], [-87.66687291517245, 41.859596022666445], [-87.66869291633058, 41.85957802183664], [-87.6693059158095, 41.85957202245938], [-87.67113091539943, 41.85955202243658], [-87.67245991550789, 41.859530022322055], [-87.67359991589746, 41.8595190218148], [-87.6738899121783, 41.85951801997023], [-87.67471492952357, 41.8592938477665], [-87.67488191563804, 41.85936602285587], [-87.67506788733971, 41.859457195224095], [-87.67603091554287, 41.85933402228026], [-87.67880991504235, 41.858882023028364], [-87.68130491853736, 41.858668023056126], [-87.68156178266698, 41.85869118271165], [-87.68264377542903, 41.85837294954632], [-87.68343714693377, 41.85800677808269], [-87.68508491852026, 41.85757957804166], [-87.68576937380186, 41.8574101550026], [-87.68579101201885, 41.85794469952831], [-87.68577872831156, 41.858251792209366], [-87.68579267181114, 41.85892343622364], [-87.68581, 41.85935], [-87.68581596558501, 41.85935393322516], [-87.685806149847, 41.85935409896724], [-87.68582330898182, 41.85996221316358], [-87.68583932377675, 41.86069423184418], [-87.68586436357774, 41.861549349140525], [-87.68587858207593, 41.861914967669314], [-87.68590000198043, 41.862465765212356], [-87.68593828211283, 41.86350969267763], [-87.68594689775801, 41.86453831929165], [-87.68596, 41.86526917757727], [-87.68600011008766, 41.866306678837994], [-87.68604101661845, 41.86660965826656], [-87.68601999999989, 41.86661], [-87.6860555431998, 41.866763308218346], [-87.68593, 41.86672], [-87.68490101873799, 41.86662816329562], [-87.68359, 41.86669], [-87.68259156285832, 41.86759948632564], [-87.68111043121377, 41.86881409707788], [-87.6799484117533, 41.86977314211584], [-87.6791132918209, 41.870467640479006], [-87.67748930535427, 41.87182735719796], [-87.67663360207979, 41.872558341074416], [-87.67605965493452, 41.873019051932324], [-87.67543758742146, 41.87356358177621], [-87.67444950910185, 41.87440374241513], [-87.67391174804227, 41.87485469996359], [-87.67305364713754, 41.87557278086651], [-87.67267662258213, 41.87588509588445], [-87.67166010092461, 41.87589999889101], [-87.67037651651351, 41.87593861570334], [-87.6686068571931, 41.875988921658205], [-87.66720375402929, 41.876019694355044], [-87.66673899631708, 41.87602808616238], [-87.66669525914, 41.87454608277258], [-87.66666784384728, 41.87375172183485], [-87.66662896309604, 41.87241012900167], [-87.66659903735761, 41.871188802736214], [-87.66658508221144, 41.870639900322374], [-87.66657422820902, 41.87021297622258], [-87.66656333478518, 41.86978450155118], [-87.66653616043557, 41.868869724285695], [-87.66650590061094, 41.86801639723135], [-87.6665, 41.867222092086486]]]}, "properties": {"city": "Chicago", "name": "Illinois Medical District", "regionid": "403328", "geo_point_2d": [41.866042396655054, -87.67496864448727], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.57944951599994, 41.798432626000064], [-87.57704904599994, 41.79693294600001], [-87.57683641699997, 41.79693294600001], [-87.57578330899995, 41.79693294600001], [-87.57525002799994, 41.79693294600001], [-87.57486589099993, 41.79578053500005], [-87.57485002799994, 41.79573294600004], [-87.57486497199993, 41.79572398000004], [-87.57584911499997, 41.79513349400001], [-87.57668010799992, 41.79505748400003], [-87.57694901499995, 41.795033038000035], [-87.57852776199994, 41.79414547100002], [-87.57854909699994, 41.79413347000007], [-87.57852868599996, 41.79408782400002], [-87.57830425499995, 41.793613332000064], [-87.57828707799995, 41.79357701700007], [-87.57829686399997, 41.79356856800006], [-87.57830222699992, 41.79355727900002], [-87.57847606099993, 41.79319134800004], [-87.57848485599997, 41.79316245300002], [-87.57854616099995, 41.792962184000025], [-87.57851876999997, 41.792833444000046], [-87.57833269199993, 41.79231069500002], [-87.57823147099997, 41.79207210400005], [-87.57813762099993, 41.79189315200005], [-87.57790813399995, 41.79148937100006], [-87.57789024199995, 41.791456301000025], [-87.57782951599992, 41.79133822100004], [-87.57767355499998, 41.79111842200006], [-87.57762816399996, 41.79104766100004], [-87.57757821699992, 41.790947140000064], [-87.57738385999993, 41.79048577800005], [-87.57734422799994, 41.790406856000054], [-87.57727902099997, 41.79034023100006], [-87.57706964299994, 41.79015276600005], [-87.57673419699994, 41.78989718800005], [-87.57649177699992, 41.78972728300004], [-87.57645422899996, 41.78969961200004], [-87.57597186199995, 41.788679136000034], [-87.57595002799994, 41.78863294600005], [-87.57597089699998, 41.78862251200007], [-87.57654906999991, 41.78833342500006], [-87.57704912999995, 41.788261646000066], [-87.57724741199993, 41.78823305100007], [-87.57704923499992, 41.78820426100002], [-87.57655002799996, 41.78813294600007], [-87.57637120499997, 41.786264647000046], [-87.57636583599997, 41.786208553000044], [-87.57636979699997, 41.78620856700007], [-87.577093, 41.786072], [-87.57732591599994, 41.78613015500007], [-87.57742591599997, 41.78611202300004], [-87.57749291599995, 41.78609902200002], [-87.57755891499993, 41.786087023000036], [-87.57946691599993, 41.78589302300002], [-87.58078096399996, 41.78590219100004], [-87.58112587799994, 41.78566285300007], [-87.58220791499994, 41.785780022000026], [-87.58353092499998, 41.785942104000064], [-87.5844219499999, 41.785556835000065], [-87.58492987899993, 41.786066096000056], [-87.58584491599997, 41.78629502200003], [-87.58608391299995, 41.786366021000035], [-87.58627691599996, 41.786387021000046], [-87.58649491499993, 41.78637902300004], [-87.58653805899993, 41.786375146000054], [-87.58653078199994, 41.78608689000002], [-87.58657991599992, 41.786087022000046], [-87.58812291599996, 41.786090022000046], [-87.58901191599992, 41.786082023000056], [-87.58907720299997, 41.786081305000025], [-87.58885054899996, 41.78707858500007], [-87.58870069599996, 41.78788992300007], [-87.58829312599993, 41.78958698800004], [-87.58766602499996, 41.79216506900007], [-87.58731763599995, 41.79369798200002], [-87.58689956899995, 41.795788318000064], [-87.58669053499992, 41.79718187500003], [-87.58669053499992, 41.79857543200006], [-87.58676021299996, 41.79996899000001], [-87.58689956899995, 41.80080512400007], [-87.58710860199993, 41.80185029200003], [-87.58732000399992, 41.80245001200006], [-87.58734687699994, 41.80245001600007], [-87.58738731399993, 41.80254707100004], [-87.58014969099995, 41.80253367600004], [-87.58015002799992, 41.802476259000045], [-87.58015002799992, 41.799690485000035], [-87.58014992399993, 41.79963334400002], [-87.57944951599994, 41.798432626000064]]]}, "properties": {"city": "Chicago", "name": "East Hyde Park", "regionid": "403352", "geo_point_2d": [41.79337785804735, -87.58254865056637], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.77895053299991, 42.01522832100004], [-87.77696302799994, 42.015128946000054], [-87.77696129999997, 42.015071879000054], [-87.77693137499993, 42.01415320900002], [-87.77693150599995, 42.014096529000014], [-87.77692960299997, 42.014039209000025], [-87.77687074099995, 42.01203789000004], [-87.77686934699993, 42.011980982000054], [-87.77687022299993, 42.01197541900005], [-87.77686874199996, 42.01192321700005], [-87.77686858299992, 42.011917823000026], [-87.77686592099995, 42.01186097000003], [-87.77686470799992, 42.011786091000026], [-87.77686206199995, 42.01172891200007], [-87.77406203199993, 42.01162934400001], [-87.77356302799993, 42.01182894600003], [-87.77226302799991, 42.009428946000064], [-87.77176202799994, 42.00852894600006], [-87.77136202799991, 42.00762894600006], [-87.77076202799996, 42.00652894600005], [-87.77062206899994, 42.00625142400002], [-87.77038514799992, 42.006369527000054], [-87.77029199599994, 42.006415885000074], [-87.76959328399991, 42.00507405800004], [-87.76936459799995, 42.00463488100006], [-87.76936123399996, 42.00462971900003], [-87.76936256999993, 42.00457882800002], [-87.76937569599991, 42.00431234200004], [-87.76910685499995, 42.00380383700006], [-87.7658224589999, 41.99761152100006], [-87.76563, 41.99729], [-87.76476416599994, 41.99640806100006], [-87.76443527599996, 41.99578160300006], [-87.76313762299995, 41.99318629700007], [-87.76160403199997, 41.99020759300003], [-87.76048185299997, 41.98786119900006], [-87.76069591599996, 41.98776002300007], [-87.76110991499995, 41.98761102300005], [-87.76149591299996, 41.987557023000065], [-87.76177298099995, 41.98757686000004], [-87.76183791499993, 41.98763102400004], [-87.76206791599996, 41.987997022000044], [-87.76217004699997, 41.98821399800004], [-87.76217608399996, 41.98833511700007], [-87.76206796099994, 41.988396148000035], [-87.76176991399993, 41.98842302600007], [-87.76165080999992, 41.988470881000055], [-87.76154091599993, 41.98862902500007], [-87.76149586399997, 41.98876201300003], [-87.76148878399992, 41.98886304400001], [-87.76196279999992, 41.99031908800003], [-87.76205088199997, 41.99042706300001], [-87.76217991799994, 41.99051502100007], [-87.76228090499994, 41.990549198000046], [-87.76279595299994, 41.99042009900005], [-87.76309391499994, 41.99019002200002], [-87.76353391599996, 41.98975702200004], [-87.76377791599997, 41.98957402300003], [-87.76390989799994, 41.98950597200007], [-87.76400491599996, 41.98948602400002], [-87.76418793899994, 41.989492944000055], [-87.76432291499995, 41.989581022000024], [-87.76467591499993, 41.989838022000065], [-87.76502091399998, 41.99002702000006], [-87.76547491599995, 41.99014902200001], [-87.76581291599993, 41.99020315500007], [-87.76599891599992, 41.99015602300005], [-87.76618291499993, 41.99010902200006], [-87.76661291599997, 41.98993902300002], [-87.76695194299995, 41.989784069000045], [-87.76758491599996, 41.98932302300005], [-87.76809287099994, 41.98907289500004], [-87.76864791199993, 41.98900502400004], [-87.76926491499992, 41.989059023000024], [-87.77008691599995, 41.98910002400003], [-87.77062291299993, 41.98921502400002], [-87.77087291499996, 41.98936402300006], [-87.77121901699995, 41.98961393500007], [-87.77134703999995, 41.98981097700004], [-87.77137300199995, 41.990004528000036], [-87.77129291399994, 41.99028502300006], [-87.77091491599995, 41.990911023000066], [-87.77081991599992, 41.991114026000034], [-87.77077978199995, 41.99129002100004], [-87.77080676199994, 41.99146612200007], [-87.77088787099994, 41.99152014600002], [-87.77126091599993, 41.991554022000045], [-87.77163991299992, 41.99156102400006], [-87.77187691599991, 41.99162202300004], [-87.77226991499997, 41.991771023000055], [-87.77300091499993, 41.99218402400003], [-87.77312701899993, 41.992291937000054], [-87.77332996499995, 41.99265700400007], [-87.77344499799993, 41.99312402700007], [-87.77337003099993, 41.99336808800006], [-87.77320794999996, 41.99354406300005], [-87.77258491599996, 41.99393602300005], [-87.77142591599993, 41.99461302300005], [-87.77123981299994, 41.994761936000025], [-87.77120096899992, 41.99482859400006], [-87.77056827099992, 41.99436239600003], [-87.76969214499994, 41.99420778500006], [-87.76891909199995, 41.99395010100004], [-87.76639378699996, 41.99276475400005], [-87.76546874599994, 41.992373698000044], [-87.76497843899993, 41.99368357700006], [-87.76475, 41.9943], [-87.76568, 41.9947], [-87.76631013799992, 41.99499019500006], [-87.76608456599993, 41.99544466900004], [-87.76670300799995, 41.99575389100004], [-87.76763067099992, 41.996114648000045], [-87.76799142899993, 41.99626925900002], [-87.76881601899996, 41.99596003800007], [-87.76881601899996, 41.99570235400005], [-87.76863241399994, 41.995331241000024], [-87.7688, 41.99488], [-87.76945329299996, 41.995202401000014], [-87.76994301999996, 41.99545772900006], [-87.77020659199994, 41.995557872000056], [-87.77109454099995, 41.995929795000045], [-87.77110478999997, 41.99600106700006], [-87.77125391599996, 41.99627202200002], [-87.77143686599997, 41.99652908800003], [-87.77175191899993, 41.99667102200003], [-87.77266591599992, 41.99687402100004], [-87.77354691599997, 41.99693502100006], [-87.77405092599996, 41.996908075000064], [-87.77449791599994, 41.99678602200003], [-87.77512090899994, 41.99657593900002], [-87.77524993399996, 41.99659697300006], [-87.77573091499993, 41.996833022000025], [-87.77621889799994, 41.997097071000034], [-87.77635391599992, 41.99712402200003], [-87.77669891499993, 41.997145022000026], [-87.77720691599994, 41.99712402200003], [-87.77750191599993, 41.997077023000024], [-87.77773189199995, 41.997074013000024], [-87.77783391399997, 41.99711151300004], [-87.77798321699996, 41.99711151300004], [-87.77824864599994, 41.99711151300004], [-87.77836477199995, 41.99711151300004], [-87.77853066499993, 41.997028567000015], [-87.77861361099995, 41.99694562000006], [-87.77879609399996, 41.996812906000045], [-87.77896198699992, 41.996746548000026], [-87.77914446999993, 41.99671337000005], [-87.77932695199995, 41.99669678000002], [-87.77955920199996, 41.99669678000002], [-87.77967532799993, 41.99669678000002], [-87.77989098899994, 41.99686267400005], [-87.77999052499993, 41.99711151300004], [-87.78002370299997, 41.99754283600002], [-87.78002370299997, 41.99799074700007], [-87.78002370299997, 41.99842206900006], [-87.78002370299997, 41.99872067700004], [-87.77994075699995, 41.999085642000054], [-87.77997393499997, 41.999400839000025], [-87.78007347099995, 41.999716036000045], [-87.78013982899992, 42.00008100100006], [-87.78017300699996, 42.00062844800003], [-87.78017300699996, 42.00104318100005], [-87.78027254299995, 42.00155745000006], [-87.78047161499994, 42.00202195100007], [-87.78053797199996, 42.00233714800003], [-87.78073704399996, 42.002718702000024], [-87.78100247299993, 42.00288459500006], [-87.78126790199995, 42.002950953000024], [-87.78156650999993, 42.002950953000024], [-87.78179875999994, 42.002818238000025], [-87.78194806399993, 42.002503041000054], [-87.78218031399994, 42.00220443300002], [-87.78236279699995, 42.00182287900002], [-87.78251210099994, 42.00150768200006], [-87.78272776199996, 42.00124225300003], [-87.78279411899993, 42.00099341300005], [-87.78287706599995, 42.00076116300004], [-87.78289365499994, 42.00052891200004], [-87.78289365499994, 42.000329841000045], [-87.78294342299994, 42.00011418000002], [-87.78312590499995, 41.99996487600003], [-87.78329179899997, 41.99996487600003], [-87.78349086999997, 41.99996487600003], [-87.78365676399993, 41.99996487600003], [-87.78393878199992, 42.00004782200004], [-87.78417103199993, 42.00014735800004], [-87.78446963999991, 42.00016394800007], [-87.78461894399997, 42.00016394800007], [-87.78483460499994, 42.00004782200004], [-87.78486778399994, 41.99993169700007], [-87.78500049799997, 41.999848751000044], [-87.78510003399997, 41.999782393000025], [-87.78521615899994, 41.99973262500003], [-87.78538205199997, 41.99973262500003], [-87.78569724899994, 41.99973262500003], [-87.78597926799993, 41.99973262500003], [-87.78619492899995, 41.99973262500003], [-87.78632764399997, 41.99973262500003], [-87.78652671499995, 41.99973262500003], [-87.78672578699995, 41.99973262500003], [-87.78689167999988, 41.99973262500003], [-87.78714051999997, 41.99973262500003], [-87.78742253799997, 41.999699447000054], [-87.78763819899994, 41.999649679000065], [-87.78775432499995, 41.99959991100007], [-87.78785386099996, 41.99959991100007], [-87.78806952199993, 41.999699447000054], [-87.78833495099995, 41.99981557200004], [-87.78866673699996, 41.99993169700007], [-87.78884921999995, 42.000031233000065], [-87.78908146999996, 42.00016394800007], [-87.78938007799994, 42.00031325100007], [-87.78956255999995, 42.00036301900007], [-87.78974504299997, 42.00041278700006], [-87.78984457899995, 42.00042937700003], [-87.79014318599995, 42.00057868000004], [-87.79022613299996, 42.00066162700004], [-87.79024311299995, 42.00068513800004], [-87.79027357899996, 42.00072654900003], [-87.79024413099994, 42.00074172000005], [-87.78941603599995, 42.00116831100006], [-87.78716203399992, 42.00232934900003], [-87.78296222099998, 42.00402959600006], [-87.78271434399993, 42.00422924000002], [-87.78247725999992, 42.00441762900004], [-87.78246302799992, 42.00442894600006], [-87.78245932299994, 42.00443075500004], [-87.78246347699994, 42.00443444900002], [-87.78247670599993, 42.00447474200006], [-87.78247827199992, 42.00447944000006], [-87.78256167499994, 42.00472964900006], [-87.78270457799992, 42.004958294000055], [-87.78285972299992, 42.005206526000045], [-87.78286532099996, 42.00521281800007], [-87.78294665199994, 42.00535048100004], [-87.78295205699997, 42.00535913000004], [-87.78305932299996, 42.005530755000045], [-87.78424379899995, 42.00686329000006], [-87.78466302799993, 42.00732894600002], [-87.78477412199997, 42.007428931000035], [-87.78497388099997, 42.007608878000035], [-87.78498427599997, 42.00761999000007], [-87.78566174799992, 42.00822942200004], [-87.78596167099991, 42.00852988700006], [-87.78626167999992, 42.00872941100004], [-87.78676170299995, 42.009529886000045], [-87.78746164699997, 42.00992941200007], [-87.78901656199996, 42.011970437000045], [-87.78906108699992, 42.01202942900004], [-87.78901686399995, 42.01202751600003], [-87.78837502899992, 42.011996952000054], [-87.78696203099994, 42.01192898500005], [-87.78513878999993, 42.011999773000056], [-87.78436167999993, 42.012029636000065], [-87.78336168699995, 42.012029627000054], [-87.78296958299994, 42.01200164200002], [-87.78196204599996, 42.011928946000054], [-87.78171096199995, 42.011928946000054], [-87.78066204599995, 42.011928946000054], [-87.78055196199992, 42.011928946000054], [-87.7804400919999, 42.01192894500007], [-87.77996050199992, 42.011928105000024], [-87.77906299599992, 42.01162943900004], [-87.77903050099995, 42.01213311700007], [-87.77893967699995, 42.01354088000004], [-87.77886578199994, 42.01468625000006], [-87.77886302799998, 42.01472894600004], [-87.77886210899997, 42.01474468400004], [-87.77894778299995, 42.01517305700002], [-87.77896146999993, 42.01522886800005], [-87.77895053299991, 42.01522832100004]]]}, "properties": {"city": "Chicago", "name": "Wildwood", "regionid": "403275", "geo_point_2d": [42.002302598009074, -87.77544796167604], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.74345737199988, 41.960843971000074], [-87.74161503599991, 41.96071], [-87.74007678699988, 41.96071], [-87.73886109699991, 41.960721733000064], [-87.73756596399993, 41.96074005800005], [-87.73651407299988, 41.96075265600006], [-87.73497594599989, 41.96076369400002], [-87.73319476499992, 41.96077938500007], [-87.73206170199988, 41.96079], [-87.73060427699991, 41.96080420200007], [-87.7295353309999, 41.960819922000034], [-87.72952796299988, 41.96081485300003], [-87.72887991499992, 41.960821022000054], [-87.7276639159999, 41.96083402200008], [-87.72764676799993, 41.96024777300004], [-87.72763941099991, 41.95992409800003], [-87.7276272809999, 41.95950887700007], [-87.72760159199991, 41.95862953500004], [-87.7275765559999, 41.957496642000066], [-87.7275517929999, 41.956526344000046], [-87.72753418299992, 41.95587476000008], [-87.7274957729999, 41.954497215000046], [-87.72747738799987, 41.953850053000046], [-87.72745002899988, 41.95304124200003], [-87.7274195169999, 41.951818299000074], [-87.72737932199992, 41.95020723200007], [-87.72735350199989, 41.949560557000034], [-87.72731749199993, 41.94842588400006], [-87.7272885439999, 41.94745491900004], [-87.72726407999993, 41.94672708800004], [-87.7272550619999, 41.94629884600004], [-87.72841921999989, 41.94627444300004], [-87.7298758229999, 41.946243187000036], [-87.73084915199988, 41.94621908900005], [-87.73181934699993, 41.946203381000075], [-87.73287130599992, 41.9462], [-87.73449051599992, 41.9462], [-87.73530062799993, 41.94618377600004], [-87.73548015099992, 41.94618042000007], [-87.73619074399993, 41.94616713700008], [-87.7384574089999, 41.946144636000035], [-87.7404812799999, 41.946122489000054], [-87.74137178299989, 41.94611506800003], [-87.7422498959999, 41.94610000200003], [-87.7434786549999, 41.946086403000024], [-87.7442056869999, 41.94607386700004], [-87.7449342389999, 41.94606399700006], [-87.74630977099989, 41.94603623400007], [-87.74701671299991, 41.94602616700006], [-87.74702631099987, 41.94659431300004], [-87.74704697899992, 41.947320511000036], [-87.74708276099993, 41.94823284000005], [-87.7471141709999, 41.94920347500004], [-87.74714194099992, 41.95005231900006], [-87.74716893399992, 41.95084099000006], [-87.74720246199989, 41.95175417400008], [-87.74723739499987, 41.95272604500007], [-87.74725968599992, 41.953819771000035], [-87.74727562599989, 41.95454786700007], [-87.74730023199992, 41.955702318000064], [-87.74731472099987, 41.956189395000074], [-87.74732584199988, 41.956553586000034], [-87.7473624159999, 41.957465574000075], [-87.74739487299989, 41.958316182000026], [-87.74742882399993, 41.959347270000066], [-87.74745089199989, 41.960196485000026], [-87.74745924199992, 41.96061764000006], [-87.74746787399991, 41.961135762000076], [-87.74747652399991, 41.96170235300007], [-87.74744037299989, 41.964284558000074], [-87.74739791599988, 41.96409402600005], [-87.74719691499988, 41.96366902200002], [-87.7470789149999, 41.96345002300006], [-87.74696091499992, 41.963234022000044], [-87.74682291899987, 41.96298902100005], [-87.74681091599989, 41.96293002200007], [-87.74679941899988, 41.96287601900006], [-87.74680160799993, 41.96285587900006], [-87.74652, 41.9628], [-87.7457502849999, 41.96203258900003], [-87.74503410099992, 41.96159752400006], [-87.74436379699992, 41.96122490000004], [-87.74345737199988, 41.960843971000074]]]}, "properties": {"city": "Chicago", "name": "Old Irving Park", "regionid": "403288", "geo_point_2d": [41.953608716248056, -87.73752792438319], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.64445, 41.86056997703897], [-87.64442283978877, 41.859957973272024], [-87.64442991574646, 41.859958021366964], [-87.64457991636586, 41.85995702222007], [-87.64464891504986, 41.85995702222007], [-87.64476091571603, 41.85995602217398], [-87.64485291546039, 41.85995502212781], [-87.64666291525816, 41.85990602256722], [-87.64670291530331, 41.859905022521126], [-87.64674291534861, 41.85990402247504], [-87.64685791615315, 41.85990202238277], [-87.6476629154893, 41.85990002229041], [-87.64877291539517, 41.85988202235997], [-87.64984691544016, 41.85985802215317], [-87.65047491561052, 41.859852021876534], [-87.65092691576176, 41.859840022222734], [-87.65223391575552, 41.85982602247676], [-87.65284191590325, 41.85981802210804], [-87.65342891598219, 41.85980702250023], [-87.65452291515042, 41.859791022661966], [-87.65647691529098, 41.85975902208619], [-87.65768991634746, 41.85974302224801], [-87.65890391475212, 41.859727022409764], [-87.65957191586715, 41.859716021902685], [-87.66135891550404, 41.85968402222627], [-87.66379591542606, 41.85964502222712], [-87.66612591581188, 41.85962602225051], [-87.66618756277938, 41.85962602225051], [-87.66621, 41.86099717708792], [-87.66624618950794, 41.8621014111441], [-87.66627042925246, 41.86282974561132], [-87.66630137572878, 41.8637456099652], [-87.66633231407114, 41.86472029024045], [-87.66634614545252, 41.86563538163092], [-87.66636663505001, 41.8664280563162], [-87.6665, 41.867222092086486], [-87.66650590061094, 41.86801639723135], [-87.66653616043557, 41.868869724285695], [-87.66656333478518, 41.86978450155118], [-87.66657422820902, 41.87021297622258], [-87.66658508221144, 41.870639900322374], [-87.66659903735761, 41.871188802736214], [-87.66662896309604, 41.87241012900167], [-87.66666784384728, 41.87375172183485], [-87.66669525914, 41.87454608277258], [-87.66673899631708, 41.87602808616238], [-87.66598334359294, 41.876041730459086], [-87.66390866077666, 41.876066210492205], [-87.6615293453486, 41.87611941377745], [-87.65981990102871, 41.87614873788388], [-87.65823160672153, 41.87616165391496], [-87.65513159458759, 41.87621616047219], [-87.65353403284008, 41.87657418784901], [-87.65139717582281, 41.87662181280463], [-87.65029862698584, 41.87664], [-87.64901723310834, 41.87664442040424], [-87.64837, 41.87665], [-87.64834961791426, 41.875802662164595], [-87.6461483424506, 41.87577291712252], [-87.6455600433639, 41.87573890808711], [-87.64560016259362, 41.874728699250255], [-87.64561197794684, 41.87369087529096], [-87.64560567283202, 41.87283613201951], [-87.64527289428987, 41.87162951478848], [-87.64505143998173, 41.87000097329999], [-87.64494978441758, 41.869301357161454], [-87.6448432867677, 41.86819505885622], [-87.6446755881217, 41.866307019374304], [-87.64469386185579, 41.8648414443344], [-87.64456792699112, 41.86325232079577], [-87.64452081171828, 41.86197208371791], [-87.64445, 41.86056997703897]]]}, "properties": {"city": "Chicago", "name": "University Village - Little Italy", "regionid": "403141", "geo_point_2d": [41.86795119805566, -87.65572947040228], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.66545105198473, 41.96165484415551], [-87.66594693723438, 41.96164233482939], [-87.66595080612164, 41.961654180219135], [-87.66606648939553, 41.96219604404175], [-87.66625016872754, 41.96310751833912], [-87.66641804275486, 41.9639125050223], [-87.66656951861268, 41.96461162436631], [-87.66680745988238, 41.96570441706379], [-87.66686813371652, 41.965981089747316], [-87.66705067457059, 41.96681105805332], [-87.6671817998073, 41.967415920527195], [-87.66728914287276, 41.96791813272667], [-87.66740324077571, 41.9684575279563], [-87.66746751114286, 41.96890188474296], [-87.66689494950431, 41.96890915462304], [-87.66565827983135, 41.96892901220616], [-87.66359594872732, 41.968955350982306], [-87.66235568443844, 41.968974430703575], [-87.66064019449351, 41.96901281337413], [-87.65967, 41.96902], [-87.65964319195795, 41.96790037913829], [-87.65961956698389, 41.966250162522925], [-87.65959504819085, 41.96480556920756], [-87.6595786745126, 41.964187462852045], [-87.65958, 41.96315757478921], [-87.65957582443875, 41.96247058154186], [-87.65957, 41.96172], [-87.66036518744261, 41.96171423777211], [-87.66167175890251, 41.96169983438162], [-87.66304618143218, 41.96167497374595], [-87.6642826086042, 41.96166], [-87.66545105198473, 41.96165484415551]]]}, "properties": {"city": "Chicago", "name": "Sheridan Park", "regionid": "403265", "geo_point_2d": [41.965445485647244, -87.66317461722956], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.60420312581385, 41.73664474661429], [-87.60487, 41.73664], [-87.60486434183143, 41.73782762227047], [-87.6048945550089, 41.7390368079203], [-87.60490853406485, 41.73968683401406], [-87.60492672825644, 41.740616967129384], [-87.60495502932565, 41.741731524931616], [-87.60498511637513, 41.74266096602503], [-87.60504583128049, 41.744794045556354], [-87.60507804651792, 41.74646916308611], [-87.6051380760922, 41.748604692357475], [-87.60512333676972, 41.750369809273494], [-87.6051429464414, 41.751114880408586], [-87.60511291550107, 41.751115022293575], [-87.60392691568921, 41.751138022454384], [-87.60273391555461, 41.751147021970006], [-87.60147291588284, 41.75118602196924], [-87.60029891572475, 41.75120802208376], [-87.59950691608839, 41.75120802208376], [-87.59909891526755, 41.75121802254485], [-87.59790391594005, 41.75122802210656], [-87.59757491516346, 41.751229022152735], [-87.59733591763609, 41.751229023951296], [-87.5971549740438, 41.751273121307634], [-87.59720894955348, 41.75107204189505], [-87.5972409159549, 41.75086402241457], [-87.59728209591071, 41.75059695075148], [-87.59714308862794, 41.75049783213078], [-87.59816415656555, 41.74665381165934], [-87.59853577183343, 41.745167350588225], [-87.59881448328441, 41.74312346661504], [-87.59955771381993, 41.73968602538756], [-87.60037403429514, 41.736692170429656], [-87.60169494315254, 41.73667583378455], [-87.60308870978577, 41.7366589292474], [-87.60420312581385, 41.73664474661429]]]}, "properties": {"city": "Chicago", "name": "East Chatham", "regionid": "403360", "geo_point_2d": [41.7445939396903, -87.60178029859162], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.64269482677581, 41.69960303726506], [-87.64265891504846, 41.698324022374344], [-87.6426289154643, 41.697803022639704], [-87.64260291516496, 41.69735302258071], [-87.64255791578866, 41.6959880217121], [-87.6425109154207, 41.69416302212207], [-87.64250491604345, 41.69346802246043], [-87.64245091535294, 41.6923770225312], [-87.64244991530677, 41.692350022185934], [-87.64244891526067, 41.692327022025054], [-87.64240491593046, 41.691079022952444], [-87.64239791560772, 41.69053002282637], [-87.64238791514673, 41.69009702265154], [-87.64233391535561, 41.689218022602084], [-87.64231591542517, 41.68871302180641], [-87.64227191519558, 41.68818002241797], [-87.64227191609487, 41.687142023133305], [-87.64227291524166, 41.686879021801666], [-87.64219691983216, 41.685059966730286], [-87.64430091519604, 41.68504202255783], [-87.6453539160716, 41.685048021935096], [-87.64551491539925, 41.68504902198118], [-87.64694491566696, 41.685015022212404], [-87.64758791473001, 41.6850130221203], [-87.64820691538483, 41.6850130221203], [-87.64945491535673, 41.68499802232824], [-87.65063891507653, 41.68497402212134], [-87.65187191525634, 41.684947022675296], [-87.65307691594411, 41.68492302246832], [-87.65429491543239, 41.684909021822776], [-87.65546991563654, 41.68489402203088], [-87.65670291581638, 41.68487502205427], [-87.65789091572042, 41.68485302193957], [-87.65834891524895, 41.68484402242393], [-87.65911891566981, 41.68483102182468], [-87.65998491511981, 41.684820022216876], [-87.66039991536428, 41.68481302189424], [-87.66152504816189, 41.684791149483154], [-87.66147691554754, 41.68333002188158], [-87.6614739154092, 41.683100022970464], [-87.6614619157553, 41.68236602241049], [-87.66142091566407, 41.68117402232207], [-87.66135791545787, 41.679364022524126], [-87.6613299159659, 41.67853902226606], [-87.66132191559717, 41.67796602283229], [-87.6613149134758, 41.677525012396394], [-87.6613537363084, 41.6775250159936], [-87.66134800043261, 41.67763400033365], [-87.66194800021144, 41.67753400022074], [-87.66264800010356, 41.67753400022074], [-87.66293496476989, 41.67751008634861], [-87.6638479996614, 41.677434000107574], [-87.66494800000574, 41.67733399999458], [-87.66556423344662, 41.6774367052682], [-87.66614799956358, 41.67753400022074], [-87.66644799990283, 41.67753400022074], [-87.6664979581409, 41.67741263761197], [-87.66704200030449, 41.67609100025295], [-87.66714799979474, 41.6758340000974], [-87.66852233292816, 41.67612751812346], [-87.66854799957885, 41.67613300039045], [-87.66839277209988, 41.67642128076496], [-87.66784799968684, 41.677433000061484], [-87.66864799969193, 41.677433000061484], [-87.67034799981506, 41.6773329999485], [-87.67055472607056, 41.6773329999485], [-87.67104799970707, 41.6773329999485], [-87.67154800027231, 41.6773329999485], [-87.67184799971216, 41.677433000061484], [-87.67214800005131, 41.6773329999485], [-87.67324800039547, 41.6773329999485], [-87.6736479999484, 41.6773329999485], [-87.67504799973233, 41.677433000061484], [-87.67684799996857, 41.6773329999485], [-87.67884899957761, 41.6773329999485], [-87.6806026601812, 41.6773329999485], [-87.68060865044856, 41.678555421329136], [-87.68063657858181, 41.679484186403855], [-87.68065, 41.680506294942994], [-87.68065918338988, 41.68115550169837], [-87.68068724397813, 41.68189730427374], [-87.68070257380451, 41.682268286070844], [-87.68075775730885, 41.68337907088866], [-87.68078018500411, 41.68384792208679], [-87.68081, 41.68448], [-87.68081, 41.68524438951968], [-87.68082145611136, 41.68571106028909], [-87.6808366835384, 41.68682463246622], [-87.68086518405856, 41.68766107393494], [-87.68089120555197, 41.68840772769659], [-87.68095485913295, 41.68933535964865], [-87.68101611067102, 41.69044623910943], [-87.68103928499943, 41.691189254975946], [-87.68114, 41.69175], [-87.68107734054678, 41.692305535312016], [-87.68111088552246, 41.693326436047435], [-87.68115292345435, 41.69434668734218], [-87.68117321525733, 41.69490674110359], [-87.68120725557299, 41.695927469940685], [-87.68123, 41.696764462607774], [-87.68123648142938, 41.697602960050986], [-87.68125737667306, 41.69834736520326], [-87.68128029878251, 41.699060984610384], [-87.68127791549408, 41.69906102217331], [-87.68095191575499, 41.699057021988864], [-87.68006891552123, 41.699058022034954], [-87.67888291570938, 41.69907202178111], [-87.67766991555209, 41.699079022103746], [-87.67648491488711, 41.699107022495085], [-87.6753369159274, 41.69911502196462], [-87.67432291595024, 41.69913602203332], [-87.67410491489454, 41.69914302235596], [-87.67341291537127, 41.699150021779225], [-87.67290091515208, 41.69915802214812], [-87.67271191633245, 41.69915802214812], [-87.671100914916, 41.69917902221683], [-87.66991691519614, 41.69921502207751], [-87.66962391517971, 41.69922902272297], [-87.66946191490662, 41.69922202240032], [-87.66895991514848, 41.69922202240032], [-87.66679091588918, 41.699285022606446], [-87.66527891543899, 41.69932702184448], [-87.6643289157139, 41.699348021913195], [-87.66216291479412, 41.69937602230445], [-87.66045391605479, 41.69939202214279], [-87.65874987707633, 41.6994199685748], [-87.65838991623964, 41.69942702195757], [-87.65805791532462, 41.69942702195757], [-87.65712991571458, 41.69944102260283], [-87.65694091509613, 41.69944802202618], [-87.65512791605937, 41.69950602200195], [-87.65474991572194, 41.69950902214021], [-87.6544279152679, 41.69951802255493], [-87.65424191568718, 41.69952102179402], [-87.65400091536986, 41.69952502197846], [-87.65386991562637, 41.69952702207064], [-87.65378291611258, 41.69952902216274], [-87.65365491560824, 41.699531022255], [-87.6532679157552, 41.69953702253146], [-87.65283391553434, 41.69954402195481], [-87.65262391574676, 41.69954702209318], [-87.6525229155875, 41.69954902218544], [-87.6523159150388, 41.6995550224619], [-87.65072891652716, 41.699577022576776], [-87.64985991514004, 41.699585022046314], [-87.64935191600483, 41.699589022230676], [-87.64929891536035, 41.69959002227668], [-87.64745191565542, 41.699573022392336], [-87.64624391572886, 41.699573022392336], [-87.64501991506457, 41.69958202190778], [-87.64382591578315, 41.69959302241503], [-87.64269482677581, 41.69960303726506]]]}, "properties": {"city": "Chicago", "name": "Morgan Park", "regionid": "269595", "geo_point_2d": [41.68983783502428, -87.66359647770392], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.6247838533302, 41.730119359946485], [-87.62474841197482, 41.72902002793111], [-87.62458391552414, 41.729022021914396], [-87.62443571651383, 41.72902265257756], [-87.62469635880807, 41.727510383601484], [-87.62464432466405, 41.72518893455257], [-87.62460207830155, 41.723609650176144], [-87.62445597739587, 41.72164054760643], [-87.6257563788614, 41.72164620027099], [-87.62687087028394, 41.72162930728345], [-87.62817323885213, 41.72171268416449], [-87.62928693524714, 41.72171], [-87.63058758868463, 41.7217000000002], [-87.63142437148865, 41.721692513663484], [-87.63290775929937, 41.72151401046531], [-87.6335121111602, 41.72150604789204], [-87.63351391527732, 41.72157102288915], [-87.63351591536933, 41.72166102254116], [-87.63354391576075, 41.72292502235123], [-87.6335489159913, 41.72332702199635], [-87.63355791550691, 41.723996022258206], [-87.63359891559813, 41.72522202211521], [-87.63360691596685, 41.725475022086314], [-87.6336159154825, 41.725734022334244], [-87.63364191578164, 41.726533022293225], [-87.63365291538928, 41.72696902260625], [-87.63370086094471, 41.72892202000267], [-87.63370391594168, 41.72906202285909], [-87.63371991577984, 41.72979002224306], [-87.63373970264257, 41.72996999541126], [-87.63281728029172, 41.72983822078973], [-87.63003016578284, 41.72983822078973], [-87.62817208944355, 41.72993112460672], [-87.62661040783047, 41.730051401049394], [-87.62666848657555, 41.730116932240804], [-87.62584949401975, 41.730116932240804], [-87.6247838533302, 41.730119359946485]]]}, "properties": {"city": "Chicago", "name": "Princeton Park", "regionid": "403362", "geo_point_2d": [41.725800856651134, -87.62909619827239], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.7618848579999, 41.78500995800004], [-87.76186990799992, 41.78452109000005], [-87.76181458099987, 41.78297332600003], [-87.7617950139999, 41.78224150700002], [-87.76175569199994, 41.781183037000055], [-87.76171579199989, 41.78036729400007], [-87.7616735799999, 41.77947055600002], [-87.7616448299999, 41.77849541700003], [-87.76164, 41.77802], [-87.76080144599989, 41.77803358000004], [-87.75836042099989, 41.77806722800004], [-87.7566599769999, 41.77808034300005], [-87.7566499239999, 41.77764169400005], [-87.75663667199989, 41.776890558000034], [-87.7566269099999, 41.77598424400008], [-87.7566148799999, 41.775041864000045], [-87.75659869699989, 41.77463094600005], [-87.75685504599993, 41.77463094600005], [-87.75925502999989, 41.77453098800003], [-87.76165504599992, 41.77453094600008], [-87.76375502699992, 41.77443099400004], [-87.76415504599991, 41.77443094600004], [-87.7662550459999, 41.77443094600004], [-87.76655527299991, 41.77453069500005], [-87.7689550299999, 41.774430988000056], [-87.77145502999988, 41.774330986000045], [-87.77395502999988, 41.77423098600007], [-87.77635504599992, 41.77423094600003], [-87.7788550459999, 41.77423094600003], [-87.78125502999987, 41.774130988000074], [-87.78365504599991, 41.774130946000064], [-87.78405500199993, 41.774031203000035], [-87.7861560459999, 41.77403094600003], [-87.78855602999992, 41.77393098800006], [-87.79105606499989, 41.774030908000036], [-87.79345604599987, 41.77403094600003], [-87.79465604599991, 41.77403094600003], [-87.79595601899989, 41.77393102400002], [-87.7985560309999, 41.77383098500007], [-87.79987200699989, 41.77365908000006], [-87.80085600699992, 41.77353107900005], [-87.80095332299992, 41.774833755000074], [-87.80105702799993, 41.775631594000046], [-87.80105702799993, 41.77568848500004], [-87.80105702799993, 41.77578091600008], [-87.80105702799993, 41.775838035000035], [-87.8001848149999, 41.77670654200005], [-87.7988828719999, 41.77808985600005], [-87.79798778599988, 41.77914768500005], [-87.79668584399988, 41.78053099900005], [-87.79603487199994, 41.781588828000054], [-87.7950584149999, 41.78248391400006], [-87.79416332899989, 41.783134885000024], [-87.79310549999988, 41.78337899900003], [-87.7912339569999, 41.78378585700004], [-87.78976927099988, 41.78370448500004], [-87.78708401399989, 41.783623114000065], [-87.78431738499994, 41.783623114000065], [-87.7804115559999, 41.78378585700004], [-87.77650572699989, 41.78386722800007], [-87.77300675599989, 41.784029971000045], [-87.77032149799992, 41.784029971000045], [-87.7683685839999, 41.78419271400008], [-87.76682252599993, 41.78451820000003], [-87.76600881199991, 41.78468094200007], [-87.76364903999988, 41.78492505700006], [-87.76218435399993, 41.78492505700006], [-87.76203387299994, 41.78499401400005], [-87.76189, 41.78501], [-87.7618848579999, 41.78500995800004]]]}, "properties": {"city": "Chicago", "name": "Clearing (W)", "regionid": "137841", "geo_point_2d": [41.7787587616771, -87.77887943869912], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.70200002771223, 41.73634110711115], [-87.70206919634937, 41.73542618300356], [-87.70215099992963, 41.73543299997881], [-87.70709200016438, 41.73527900007452], [-87.71175099999049, 41.73513299963966], [-87.7120212608108, 41.73513299963966], [-87.71199647207982, 41.73550769031672], [-87.71202785348494, 41.73643560454965], [-87.71203153062078, 41.73717688737376], [-87.71202448481647, 41.738107087932995], [-87.71203, 41.73862], [-87.71057750185787, 41.73864805394551], [-87.70788366857681, 41.73867950223936], [-87.70602638597681, 41.73870790192675], [-87.70351859701447, 41.738755560379936], [-87.70204, 41.73878], [-87.70198411938674, 41.73829190910576], [-87.70198498142021, 41.73754982591238], [-87.70199193361317, 41.73699133307347], [-87.70200002771223, 41.73634110711115]]]}, "properties": {"city": "Chicago", "name": "Marycrest", "regionid": "403366", "geo_point_2d": [41.73699066934628, -87.70705163499036], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.60452191523768, 41.68550602236289], [-87.60584391502351, 41.68548402224809], [-87.60665591558177, 41.685490022524824], [-87.60702686152729, 41.68548405529233], [-87.60697449894376, 41.6847794461382], [-87.60696, 41.68404], [-87.60816061248325, 41.684038099489236], [-87.61011158132936, 41.684033409660366], [-87.61251321759363, 41.684018756639524], [-87.61251091568057, 41.68402902262678], [-87.61234991545355, 41.68479802210216], [-87.61223391640159, 41.685391021558374], [-87.61222693676348, 41.68542807542474], [-87.61207691545972, 41.68542902241088], [-87.61193391516316, 41.68543102250306], [-87.61132889896602, 41.68543596787481], [-87.61132491586869, 41.6854630221794], [-87.61117991547985, 41.6863490225514], [-87.6110199152991, 41.68733202289838], [-87.6110059155531, 41.68740702275831], [-87.61060191581572, 41.68913202223525], [-87.61018291538706, 41.6909100223567], [-87.60997291559927, 41.69182302217458], [-87.60975816656683, 41.692765920519044], [-87.6088088188448, 41.69278523783994], [-87.60769869957771, 41.6928110932121], [-87.60667461566699, 41.692824791392646], [-87.60435173178601, 41.69283472085017], [-87.60286482262825, 41.69283843773608], [-87.60119276374586, 41.69285505743059], [-87.59924521408425, 41.69280631284507], [-87.59823983551087, 41.69239984667225], [-87.59826691558595, 41.692348022093675], [-87.59872691520674, 41.69147002209039], [-87.59966791541605, 41.689746022659534], [-87.60000991589261, 41.68911702244318], [-87.60032791526282, 41.68853002236405], [-87.60087291520445, 41.687510022110374], [-87.60090591582632, 41.687431022066], [-87.60110291591413, 41.687081022120005], [-87.60139291489298, 41.68652902275503], [-87.60151891620461, 41.68623102250788], [-87.60154187679537, 41.68618495923371], [-87.60201091550182, 41.686155022602044], [-87.60215791508331, 41.6861430220487], [-87.60235791530936, 41.68611702264891], [-87.60255491539726, 41.68608502207329], [-87.60270791525546, 41.68605502248888], [-87.60310991310193, 41.685960022606345], [-87.6034049150092, 41.68582902196364], [-87.60383591599113, 41.68565702212892], [-87.60413291619182, 41.68555702291521], [-87.60429091628066, 41.68552302314661], [-87.60452191523768, 41.68550602236289]]]}, "properties": {"city": "Chicago", "name": "Pullman", "regionid": "140024", "geo_point_2d": [41.68890508629328, -87.60568827026648], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.72791, 41.96813], [-87.72786441454124, 41.96712447747555], [-87.72783, 41.96607594716683], [-87.72778683398815, 41.965187809185984], [-87.7277340120724, 41.96348638028493], [-87.72769774281619, 41.96203022155809], [-87.72766391604165, 41.96083402191058], [-87.72887991543786, 41.96082102221085], [-87.72952796329474, 41.960814852861645], [-87.72953533126604, 41.960819921599075], [-87.73060427650972, 41.96080420181609], [-87.73206170175591, 41.96079000000011], [-87.73319476466129, 41.96077938466678], [-87.73497594610389, 41.96076369369274], [-87.73651407278493, 41.960752656173405], [-87.73756596350906, 41.96074005766424], [-87.73886109657047, 41.96072173291505], [-87.74007678714649, 41.96071], [-87.74161503585775, 41.96071], [-87.74345737247604, 41.960843971335954], [-87.7443637968525, 41.96122490017811], [-87.74503410100512, 41.96159752437192], [-87.74575028518427, 41.962032589203275], [-87.74652, 41.96280000000015], [-87.7468016076896, 41.962855878843385], [-87.74679941858585, 41.96287601891317], [-87.74681091551867, 41.962930022301606], [-87.7468229187698, 41.962989021424264], [-87.7469609152386, 41.96323402192583], [-87.74707891528203, 41.96345002288952], [-87.74719691532556, 41.963669022192754], [-87.74739791559763, 41.964094025596154], [-87.74744091488184, 41.96428602275547], [-87.74746591513492, 41.9646030220796], [-87.74747091536544, 41.96550902247431], [-87.74756191506364, 41.96777402166243], [-87.7475509154559, 41.96795002168144], [-87.7474219158047, 41.96794802248862], [-87.74711791528108, 41.967945022350264], [-87.74690091517067, 41.96794202221208], [-87.74658591503953, 41.967941022165995], [-87.74637191596669, 41.967945022350264], [-87.74560291559193, 41.96795902209643], [-87.74515291553294, 41.967966022419056], [-87.74393191590627, 41.96796502237297], [-87.74373091563413, 41.96796502237297], [-87.74265791563504, 41.96796902255724], [-87.74126691536674, 41.96798202225731], [-87.74083691533018, 41.96798702248767], [-87.73991891528176, 41.96799702204921], [-87.73906891566975, 41.96801102179536], [-87.7377555397285, 41.968025157386904], [-87.73774999999989, 41.96803], [-87.73706932554711, 41.96803], [-87.73598711125169, 41.968034546736035], [-87.73490509988495, 41.968046527274645], [-87.7336166590202, 41.96806387984416], [-87.73232873923357, 41.96807845719947], [-87.73098917159257, 41.968099849646045], [-87.72975176825243, 41.96812005254886], [-87.72851421020829, 41.96813], [-87.72791, 41.96813]]]}, "properties": {"city": "Chicago", "name": "Mayfair", "regionid": "403286", "geo_point_2d": [41.964492132833634, -87.73727496106227], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.65187999999989, 41.94728], [-87.65181, 41.94543], [-87.65247737785772, 41.94541914832748], [-87.65427, 41.94539], [-87.65424999999986, 41.94446], [-87.65520999999988, 41.9456], [-87.65620190892805, 41.945358692564945], [-87.65814456760188, 41.9453314611695], [-87.65912, 41.94531999999989], [-87.65912484085618, 41.94575567706631], [-87.65914103522826, 41.94672762049818], [-87.65915426306255, 41.94753646481823], [-87.65917749961318, 41.9485891551947], [-87.65920495241437, 41.94956183468214], [-87.65925, 41.95106], [-87.65806365951028, 41.95106693766369], [-87.65701062943363, 41.95108080710705], [-87.65603991229416, 41.95110523357804], [-87.6548255763047, 41.95111664812983], [-87.65353005702015, 41.95113], [-87.65255972513143, 41.95114600687176], [-87.65199999999986, 41.95116], [-87.65198273319133, 41.950693796169006], [-87.65195398621793, 41.94947862179268], [-87.65195, 41.94908], [-87.65187999999989, 41.94728]]]}, "properties": {"city": "Chicago", "name": "Wrigleyville", "regionid": "403294", "geo_point_2d": [41.948216854924475, -87.65553617100254], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.89416601299993, 42.005371361000066], [-87.89416601299993, 42.00408558700007], [-87.89416601299993, 42.00402847400004], [-87.89416601299993, 42.00397136100002], [-87.89416601299993, 42.00318558700006], [-87.89416599499995, 42.00312853100006], [-87.89335486899992, 42.00322049800006], [-87.89335567599991, 42.00327748800003], [-87.89336501799993, 42.00367152400002], [-87.89336523299993, 42.003728567000046], [-87.89276597599995, 42.00412815300006], [-87.89106618299994, 42.00402834800007], [-87.89007059099997, 42.003530763000064], [-87.89006584999997, 42.003528588000044], [-87.89007058199995, 42.003473646000025], [-87.89016648799996, 42.00232871600007], [-87.89406600799997, 42.00222848600002], [-87.89436558199996, 42.00222817200005], [-87.89446104599993, 42.001753307000044], [-87.89446625999992, 42.00172873200005], [-87.89446104599993, 42.00169618900002], [-87.89427479999993, 42.000485588000046], [-87.8942663399999, 42.000428617000054], [-87.89427479199996, 42.00042847400003], [-87.89596552199997, 42.00042825800005], [-87.89596601299993, 42.00037136100007], [-87.89596601299993, 42.00021358700007], [-87.89596601299993, 42.00015647400004], [-87.89596601299993, 42.00009936100002], [-87.89596601299993, 41.99998558700002], [-87.89596603399995, 41.99992843400002], [-87.89356604299994, 41.99972843600005], [-87.88446504999997, 41.99812896200006], [-87.88426468599992, 41.99632833000003], [-87.88426501299992, 41.99627136100001], [-87.88426501299992, 41.995492587000044], [-87.88426502099996, 41.99543545700003], [-87.88406498999994, 41.99542853500002], [-87.88066504699997, 41.99572839300002], [-87.87997431499996, 41.99567566600006], [-87.87847801299992, 41.991502474000065], [-87.87816501299993, 41.990629474000045], [-87.87736501299996, 41.988329474000054], [-87.87636401299993, 41.98542747400005], [-87.87633368799997, 41.985350325000056], [-87.87604133799991, 41.98534862200005], [-87.87579701299994, 41.98396147400006], [-87.87516501299993, 41.982329474000025], [-87.87486501299995, 41.980929474000035], [-87.87446501299993, 41.980129474000066], [-87.87348034699991, 41.97688332300003], [-87.87346436999995, 41.976829480000035], [-87.87348033899994, 41.976826209000066], [-87.87396398999991, 41.97672957600002], [-87.87586352199996, 41.97672925800002], [-87.87586401299994, 41.97667236100004], [-87.87586401299994, 41.97408658700004], [-87.87586401299994, 41.97402947400006], [-87.87276404199994, 41.97402942300003], [-87.87186398399994, 41.97393052400002], [-87.87076401299996, 41.97392947400004], [-87.86996401299994, 41.97392947400004], [-87.86926401299995, 41.97392947400004], [-87.86856409299996, 41.97392937100005], [-87.86817602799992, 41.97383247800007], [-87.86816465199995, 41.973830039000056], [-87.86817603799993, 41.97388959400006], [-87.86825259099996, 41.974272360000036], [-87.86826335499994, 41.974329215000054], [-87.86825259999993, 41.97432947400005], [-87.86789301299994, 41.97432947400005], [-87.86747829799995, 41.97432947400005], [-87.86746409299995, 41.97432937100005], [-87.86747829099993, 41.97427236300006], [-87.86754973499994, 41.97398658500003], [-87.86756393299997, 41.97392957600004], [-87.86754972899993, 41.97392947400004], [-87.86666401299996, 41.97392947400004], [-87.86645801299991, 41.97392947400004], [-87.86506401299991, 41.97392947400004], [-87.86474701299991, 41.97392947400004], [-87.863764, 41.973929510000055], [-87.86236408599994, 41.97402935100006], [-87.86208144899994, 41.97397296100006], [-87.86188723799992, 41.97393390800004], [-87.86187904599994, 41.97387610200006], [-87.86187019399993, 41.97380749500007], [-87.86184221499995, 41.97359765100003], [-87.86181889899993, 41.973495060000054], [-87.86172264899993, 41.973217422000054], [-87.86174779499997, 41.97321239300003], [-87.86177469899994, 41.973207258000045], [-87.86188063699994, 41.973186149000064], [-87.86216398999991, 41.97312957600007], [-87.86356401299994, 41.97312947400007], [-87.86504901299996, 41.97312947400007], [-87.86526401299993, 41.97312947400007], [-87.86645801299991, 41.97312947400007], [-87.86666403499997, 41.97312938800006], [-87.86726399099994, 41.97302955900005], [-87.86826401299997, 41.973029474000036], [-87.86876396399995, 41.97302954800005], [-87.86936406799997, 41.973130383000054], [-87.87078501299996, 41.97308347400007], [-87.87246500199993, 41.973029499000056], [-87.87498350599996, 41.97307676200005], [-87.87512705399995, 41.97307892800005], [-87.87632305499992, 41.97310292700006], [-87.87668813699996, 41.973110755000036], [-87.87706905499992, 41.973118925000044], [-87.87766305299992, 41.973128930000016], [-87.87783016299991, 41.97309585200002], [-87.87810182399991, 41.97304194400004], [-87.87816498999993, 41.97302957600004], [-87.88086452199995, 41.97302925800004], [-87.88086501299992, 41.97297236100006], [-87.88086501199993, 41.972386588000056], [-87.88086489199996, 41.972329477000066], [-87.88086453099997, 41.97227236600001], [-87.88080857799994, 41.96562258100004], [-87.88080822899995, 41.965571806000064], [-87.88080867099994, 41.96556568000005], [-87.88111123999992, 41.96556275500005], [-87.88231204199995, 41.965552954000024], [-87.88243971399997, 41.965549224000014], [-87.88542590699993, 41.96544962600006], [-87.88543703299995, 41.96544897900002], [-87.88543802799994, 41.96539225900005], [-87.88543802799994, 41.965365485000056], [-87.88543736999996, 41.96530824300003], [-87.88080302799993, 41.96537094600006], [-87.88080167899992, 41.96535821400004], [-87.88080139699994, 41.965313873000056], [-87.88080103299993, 41.96530113800002], [-87.88080062299996, 41.965244255000066], [-87.88076678799993, 41.960505610000034], [-87.88076556699994, 41.96044860900001], [-87.88076516299996, 41.96039149800003], [-87.88076441799994, 41.960286584000066], [-87.88076444299996, 41.96022961400006], [-87.88066401299992, 41.958429474000056], [-87.88056871699993, 41.95728591600004], [-87.88056463799992, 41.957229559000034], [-87.88056870999996, 41.95722880300006], [-87.88336386099994, 41.95682968100003], [-87.88496410799993, 41.95742939900003], [-87.8870080149999, 41.95769747100007], [-87.88891201199992, 41.95793347500005], [-87.88924630699992, 41.95797595700002], [-87.88966411599993, 41.95802883000004], [-87.89076404599994, 41.958028946000056], [-87.89114203099994, 41.95802904200007], [-87.89117251299993, 41.95802947400005], [-87.89296400099991, 41.95802922700004], [-87.89296335599992, 41.95797240900002], [-87.89288100399993, 41.95511831800007], [-87.89287902999999, 41.955061426000036], [-87.89287747299993, 41.95500419300002], [-87.89286564399995, 41.95458654100002], [-87.89286442499994, 41.954529462000025], [-87.89266598699997, 41.95215311000003], [-87.89266370499995, 41.952129292000045], [-87.89266598299994, 41.95209599200006], [-87.89276449, 41.95042971700007], [-87.89516400199993, 41.95042949500003], [-87.89736403599993, 41.950529417000034], [-87.90166494699994, 41.95022958800006], [-87.90386500199996, 41.95062948500002], [-87.90856495999996, 41.951628511000074], [-87.91176480099993, 41.95272798300004], [-87.91216522299996, 41.951828965000054], [-87.91446484599996, 41.95262815800003], [-87.91456374399993, 41.95391197000003], [-87.91456488499992, 41.95392862300002], [-87.91456330399993, 41.95396906200005], [-87.91456322399992, 41.95398554400003], [-87.91447643599994, 41.956762924000024], [-87.91447468799997, 41.956820187000055], [-87.91447475999993, 41.956876997000045], [-87.914464, 41.95707189300002], [-87.91446602799992, 41.95712794600006], [-87.91506527299997, 41.957327695000046], [-87.91531295399993, 41.957300510000024], [-87.91542363399996, 41.957288519000016], [-87.91557211899998, 41.95727213400004], [-87.91558938499996, 41.95727020300006], [-87.91596492899995, 41.957228621000056], [-87.91720401299993, 41.95749447300005], [-87.92028205499994, 41.958151425000025], [-87.92198960599994, 41.95830313000003], [-87.92216426599997, 41.95743716300006], [-87.92216625899992, 41.957428733000015], [-87.92216425499997, 41.95738004500002], [-87.92206656699992, 41.954685531000045], [-87.92206490799997, 41.954628477000014], [-87.92206311399994, 41.954571424000044], [-87.92203821899993, 41.953819514000024], [-87.92203611499995, 41.95377077900007], [-87.92203696399996, 41.95376261200004], [-87.922035, 41.95371394200003], [-87.92347801899994, 41.95407046800005], [-87.92386002, 41.95415846700006], [-87.92527602299995, 41.95445746400002], [-87.92607494299992, 41.95460462300007], [-87.92607693599994, 41.95466348500001], [-87.92607890699995, 41.95472151400002], [-87.92624427599996, 41.95900571600002], [-87.92631602099998, 41.95901746400005], [-87.92723301299996, 41.959147473000016], [-87.92763900199995, 41.959204486000026], [-87.92809001699997, 41.959282469000065], [-87.93124701299995, 41.95979147400003], [-87.93191538899995, 41.95989938600006], [-87.93191598599996, 41.95995642500002], [-87.93202403999997, 41.96465452300002], [-87.93202468299992, 41.96471149100005], [-87.93316801299991, 41.96471347300007], [-87.93322801199994, 41.964713475000046], [-87.93399701299992, 41.96471547300006], [-87.93516221199995, 41.96471746600001], [-87.93516280099998, 41.96477457500002], [-87.93519122499997, 41.96683637400002], [-87.93519183399994, 41.96689333800003], [-87.9356009099999, 41.96677829600002], [-87.93592095699992, 41.96652644100003], [-87.93650594699993, 41.965937455000066], [-87.93666605099996, 41.96572848200002], [-87.93669779599996, 41.96569250000003], [-87.93677797099994, 41.965440504000064], [-87.93686016599995, 41.96505584900005], [-87.93686625299992, 41.96502873900004], [-87.93686016599995, 41.96499873100003], [-87.93680801299993, 41.96473347400006], [-87.93680425399997, 41.96472285000004], [-87.93678536699994, 41.96466852000002], [-87.93680424499995, 41.96466573600002], [-87.93706599299996, 41.96462854600002], [-87.93925401299997, 41.96462647400005], [-87.93930801, 41.96462647900005], [-87.93958501499996, 41.96462946800004], [-87.93958501299993, 41.96468658700007], [-87.93958501399996, 41.96471135900002], [-87.93958524499996, 41.96476846800003], [-87.93958579899997, 41.96482557700005], [-87.93958935399993, 41.96513836900003], [-87.93959000199997, 41.96519547500002], [-87.93959063999992, 41.965252581000065], [-87.93960638599997, 41.966686369000044], [-87.93960701199993, 41.96674347500004], [-87.93960763799996, 41.96680058100002], [-87.93961238899996, 41.967234366000056], [-87.93961291599996, 41.967268375000046], [-87.93961284099996, 41.96729133600006], [-87.93961418899993, 41.96732561200002], [-87.93961411599994, 41.96734857200004], [-87.93961464899995, 41.96738258300007], [-87.93963237699994, 41.968972367000035], [-87.93963267999997, 41.96901823800004], [-87.93963359999996, 41.96902958800006], [-87.93963340199996, 41.96907534000007], [-87.93963426999993, 41.96908651100004], [-87.93964649999995, 41.969274611000046], [-87.93964664099995, 41.96933157900002], [-87.93967338399993, 41.97176337000006], [-87.93967352599996, 41.97182033700004], [-87.93968839899995, 41.97207761500004], [-87.93968824599993, 41.97213458600003], [-87.93969277999994, 41.97324536200005], [-87.93969303699997, 41.97330247300003], [-87.93969334099994, 41.97335958400003], [-87.93969368499992, 41.973419363000055], [-87.93969398299997, 41.973476475000034], [-87.93969422199993, 41.97353358700007], [-87.93970280399996, 41.97587936200005], [-87.93970304399993, 41.97593647300003], [-87.93970334799995, 41.97599358400003], [-87.93970767799996, 41.97673136300004], [-87.93970759999993, 41.976788334000034], [-87.93987407099995, 41.97700808000007], [-87.93989110099992, 41.977030820000024], [-87.93987408599995, 41.97706519500007], [-87.93971241899997, 41.97739701100004], [-87.93971276699993, 41.97745397100004], [-87.93977940499994, 41.98372136800002], [-87.93977962899996, 41.983749364000055], [-87.93977993999992, 41.98377847700003], [-87.93978009399993, 41.98380647100003], [-87.93978040999994, 41.98383558400006], [-87.93978064199993, 41.98386357900005], [-87.93978538499992, 41.98429436600002], [-87.93978600199993, 41.98435132700006], [-87.93978792499995, 41.98440852300006], [-87.93979010099996, 41.984473425000026], [-87.93979178699996, 41.98453048600003], [-87.93979335399993, 41.98458770100005], [-87.93980100399992, 41.98503937800006], [-87.93980191699994, 41.98508938100007], [-87.93980204099994, 41.98509647300005], [-87.93980297599992, 41.98514647500007], [-87.93980309899995, 41.98515356800003], [-87.93980399199994, 41.98520357100004], [-87.93982803399993, 41.98660638100005], [-87.93982815699995, 41.98661587600002], [-87.93982827999996, 41.98662537100005], [-87.93982898299997, 41.98666347800001], [-87.93982906499996, 41.98667207400006], [-87.93982881799995, 41.986682334000065], [-87.93983021599995, 41.986720703000046], [-87.93982998099992, 41.98672737100002], [-87.93983011399995, 41.98673956800007], [-87.93983083299995, 41.98678357700003], [-87.93983174799993, 41.98687224000002], [-87.93983360499996, 41.98692968300002], [-87.93983426499992, 41.986986523000034], [-87.93984203799992, 41.98746537700003], [-87.9398429089999, 41.987522478000066], [-87.93984360899996, 41.98757958100003], [-87.93990441599996, 41.99340536800002], [-87.93990438099993, 41.99346224400005], [-87.93906700299996, 41.99342849300005], [-87.93899030999992, 41.99342884500003], [-87.93773, 41.99583547600002], [-87.93559874299994, 41.99967136600002], [-87.93556721999991, 41.999728316000066], [-87.93559874999987, 41.99972847400004], [-87.93586676699992, 41.999728098000034], [-87.93736701299994, 41.99742883100003], [-87.93746735799992, 41.99832856600005], [-87.94006244599994, 41.99812882500003], [-87.94006697899992, 41.99812855000004], [-87.94006245299992, 41.998185938000056], [-87.94004470599997, 41.99840954600005], [-87.94004171199997, 41.998443280000046], [-87.94004470599997, 41.998466665000066], [-87.94016801299992, 41.99952847400003], [-87.94025849099992, 41.999709430000046], [-87.94026823399996, 41.99972875400004], [-87.94025849399996, 41.999766549000064], [-87.94016809299994, 42.000128371000066], [-87.94016801299992, 42.00018558700002], [-87.94016801299992, 42.00086736100007], [-87.94016751499996, 42.00092427500004], [-87.93704301299991, 42.001030473000064], [-87.93476996699995, 42.00110465100004], [-87.93477018699994, 42.00116141700004], [-87.93483380799994, 42.004167387000045], [-87.93483501499996, 42.004224474000054], [-87.93483622799991, 42.00428156100002], [-87.93486613099996, 42.005671520000064], [-87.93486637799997, 42.00572834700005], [-87.93487169899996, 42.00580618800005], [-87.93487310599993, 42.00586262000006], [-87.93496490799997, 42.00797171000004], [-87.93496637399993, 42.00802794600002], [-87.9346680049999, 42.00802809200006], [-87.93326802799996, 42.00822794600002], [-87.93325701299993, 42.00808947400002], [-87.93319201299992, 42.00722047400005], [-87.93306988799995, 42.00556738500005], [-87.93306674599994, 42.005528236000025], [-87.93306988799995, 42.00551026700003], [-87.93307921499994, 42.00545119500002], [-87.93308801299996, 42.00539547400005], [-87.93307920799997, 42.00539408200006], [-87.931185, 42.00509448900004], [-87.93036701099997, 42.004995476000026], [-87.92961704699994, 42.00490943600004], [-87.92948914799994, 42.00488240700001], [-87.92933599199992, 42.00480099700006], [-87.92923887299996, 42.00491438400007], [-87.92545012499993, 42.007244053000065], [-87.92280114, 42.00613480800006], [-87.92076474499996, 42.00365923700007], [-87.92056741099996, 42.00362883300004], [-87.92036656399992, 42.00482819900002], [-87.91366749599993, 42.00502870300004], [-87.91346653599997, 42.00842823000004], [-87.91296701299994, 42.00842847400002], [-87.91296701299994, 42.00837136100005], [-87.91296701299994, 42.00618558700005], [-87.91296701299994, 42.00612847400003], [-87.91257273199993, 42.00612847400003], [-87.91256703899992, 42.00612842700007], [-87.91257272499995, 42.00607136200005], [-87.91266130199993, 42.00518558600003], [-87.91266698699995, 42.00512852000002], [-87.91266129499994, 42.00512847400005], [-87.91126700799997, 42.00512848600003], [-87.90896301299992, 42.00518647300002], [-87.90726651299994, 42.00522867700004], [-87.90726602999997, 42.00528558600006], [-87.90726699599996, 42.00857136100006], [-87.90726651199995, 42.00862827400004], [-87.90523499699995, 42.00869254200005], [-87.90430814399996, 42.00884526600003], [-87.89526606899992, 42.00592844900006], [-87.89416567599993, 42.00542873700004], [-87.89416601299993, 42.005371361000066]], [[-87.89465208699991, 41.96715215500006], [-87.89478532599996, 41.96707748300001], [-87.89482723999996, 41.96705542200004], [-87.89488084499997, 41.967031134000074], [-87.8949359229999, 41.96701040500004], [-87.89499223899996, 41.96699332100006], [-87.89504955099993, 41.96697995800002], [-87.89510761499996, 41.966970371000045], [-87.89516618199995, 41.96696460300006], [-87.89517129199997, 41.96696428300003], [-87.89540529199995, 41.96695028300006], [-87.895459, 41.966948678000044], [-87.89548172099992, 41.96694896500002], [-87.89575871999992, 41.966955965000054], [-87.89579481799994, 41.966957604000065], [-87.89585338499994, 41.96696337200006], [-87.89588097499993, 41.96696744000006], [-87.89660864899994, 41.967086300000055], [-87.89921178399996, 41.96707280700007], [-87.89940384799995, 41.967062169000016], [-87.89931169999993, 41.96619391200004], [-87.89930860299992, 41.96615781900004], [-87.89930667799993, 41.966099], [-87.89930714899995, 41.96606989200006], [-87.89932604599991, 41.965486364000064], [-87.89930918599997, 41.96546855800004], [-87.89913919899996, 41.965289282000064], [-87.89867154299996, 41.96483523800003], [-87.89866208299995, 41.964825917000034], [-87.89862864999996, 41.964790626000074], [-87.89719897299993, 41.963197363000056], [-87.89657197099993, 41.96268803800006], [-87.89654603599993, 41.96266614600005], [-87.89651630099995, 41.962638864000034], [-87.89619471599995, 41.96233024700006], [-87.89600691299992, 41.96219906100002], [-87.89597857999993, 41.96218220200006], [-87.89577845999995, 41.96210450100004], [-87.89527472599995, 41.96202255400003], [-87.89486943899996, 41.96198458700006], [-87.89448592499996, 41.96196872700005], [-87.89443955899995, 41.96199235300002], [-87.89419432799994, 41.96225024500005], [-87.89357045699994, 41.96298103900004], [-87.89192904299993, 41.96538589500004], [-87.89191602099997, 41.96555357500005], [-87.89186800299994, 41.96683595100005], [-87.89188341699995, 41.96699009300004], [-87.89193740799998, 41.96700522200006], [-87.89376743499996, 41.96708758700002], [-87.89378581799996, 41.96708860300003], [-87.89379282799993, 41.96708909000005], [-87.89465208699991, 41.96715215500006]]]}, "properties": {"city": "Chicago", "name": "O'Hare International Airport", "regionid": "403159", "geo_point_2d": [41.98013517007267, -87.90873316502218], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.55846241232251, 41.730304632929126], [-87.55477807404505, 41.728210000000246], [-87.55212033956366, 41.7257664491917], [-87.55030108376894, 41.72436218136435], [-87.5495673012112, 41.72371223546254], [-87.54914794740654, 41.723297947406856], [-87.54872192261617, 41.72283999391945], [-87.54872791538598, 41.7228400220304], [-87.54881991513007, 41.722841022076665], [-87.54891491501279, 41.722841022076665], [-87.54970391541009, 41.7228340226533], [-87.55040191520982, 41.72281202253851], [-87.5510469152647, 41.7227920225159], [-87.55152291562284, 41.72277602267764], [-87.55253891569227, 41.7227610219862], [-87.55300491558951, 41.722748022286304], [-87.55386491566252, 41.72275602175566], [-87.55451991617805, 41.72276302207838], [-87.55518191521782, 41.72275702180174], [-87.55660391511661, 41.72274602219412], [-87.5579829157312, 41.722734022540145], [-87.55804791602942, 41.722733022494054], [-87.55814991623457, 41.72273202244805], [-87.55827091551708, 41.72273002235587], [-87.55844891562845, 41.72272802226369], [-87.55976791617519, 41.722699021826166], [-87.55986991548117, 41.7226980217799], [-87.56010891570656, 41.72269402249499], [-87.56072691541583, 41.722707022194896], [-87.56126891521927, 41.7226980217799], [-87.56159391581122, 41.72269402249499], [-87.56255591519023, 41.72268202194182], [-87.56375791573954, 41.72264702212705], [-87.56473991514115, 41.72263902265761], [-87.56544703946228, 41.7226278764604], [-87.56713427404652, 41.72259139546653], [-87.56889889959035, 41.72255659632251], [-87.57020010766053, 41.72254], [-87.57103678815145, 41.72253131448508], [-87.57196549400554, 41.72251699418832], [-87.57317302725512, 41.722498521134824], [-87.5748451738079, 41.72248422928393], [-87.57577415497916, 41.72247647028498], [-87.57725970881529, 41.72245344698855], [-87.57809741969089, 41.72243541455079], [-87.57930480578386, 41.72241689485017], [-87.5819059031251, 41.722392795419715], [-87.5842280307736, 41.722373827055854], [-87.58664988874943, 41.72231697439914], [-87.58677791568294, 41.722459022454146], [-87.58683991584297, 41.72251802247581], [-87.58695095603325, 41.72262498783808], [-87.58722291551156, 41.72307402292452], [-87.58726191551062, 41.723140022369726], [-87.58729291514095, 41.723191022921945], [-87.58734695809979, 41.7232779190136], [-87.58745291532169, 41.7234480221783], [-87.58755791566551, 41.723625022243404], [-87.58758591515739, 41.72367202261144], [-87.58839791571565, 41.72562902199088], [-87.58850288458301, 41.725910043937134], [-87.58860391621835, 41.726124022325656], [-87.5895169160363, 41.727706022405386], [-87.59035191585627, 41.72910802318094], [-87.59057491624306, 41.72950602174208], [-87.59099891600306, 41.73026402340859], [-87.59151791564535, 41.73134902216199], [-87.5917179158714, 41.731717022038445], [-87.59207791627858, 41.732297022694034], [-87.59259591587488, 41.73313402170671], [-87.59271491596455, 41.733275022810304], [-87.59299491538174, 41.73360802287205], [-87.59313491553985, 41.73377402243024], [-87.59321491563033, 41.73386902231269], [-87.59326791537563, 41.73394202208051], [-87.59354391550762, 41.734323022556126], [-87.5941249153102, 41.735056022170824], [-87.59452391571624, 41.73555802282813], [-87.59483791580129, 41.73600502274903], [-87.59488191603089, 41.73608502283949], [-87.59508091531177, 41.73644802248522], [-87.59517891533267, 41.73662702264276], [-87.59524492736823, 41.73674702457692], [-87.59349891523203, 41.736778022408785], [-87.59299491538174, 41.7367890220165], [-87.59249691580789, 41.736793022200764], [-87.59176891552453, 41.73680002252349], [-87.59140291574029, 41.736804021808574], [-87.59056491578222, 41.73681302222338], [-87.59026791558135, 41.73681702240784], [-87.58932091599478, 41.73682702196955], [-87.5890269150327, 41.73683002210773], [-87.58813591532963, 41.73685302226871], [-87.58780691545233, 41.73686102173824], [-87.58680791526719, 41.73688102265995], [-87.58562691568586, 41.736909022152105], [-87.58543191569014, 41.73691302233655], [-87.58520991534918, 41.73691902261301], [-87.58484691570338, 41.73692702208253], [-87.58274291584291, 41.736967022127864], [-87.58057991596085, 41.736982021919935], [-87.57935891543485, 41.737008022219094], [-87.57815491569255, 41.73703302247216], [-87.57694391562752, 41.73705302249478], [-87.57568291595572, 41.73708002194065], [-87.57448891577486, 41.73710702228608], [-87.57331591566289, 41.73713302258524], [-87.57213691527437, 41.73714302214686], [-87.57085891571813, 41.73715102251567], [-87.5696579152149, 41.73716502226166], [-87.56892991583094, 41.737183022192085], [-87.56876591546548, 41.73718702147717], [-87.56860591528482, 41.737191022560815], [-87.56848152969117, 41.737193437388], [-87.5650724717349, 41.73483670572617], [-87.56176665749341, 41.732684533347374], [-87.55846241232251, 41.730304632929126]], [[-87.58030623817945, 41.724641001770756], [-87.58041952752616, 41.72352076123646], [-87.57772531683436, 41.72352076123646], [-87.57642466339698, 41.72352076123646], [-87.5741949717899, 41.72361366505337], [-87.57052419392733, 41.723707757095355], [-87.57055571532578, 41.72482193666094], [-87.57060964852948, 41.72677177647775], [-87.57063939715118, 41.72844522109537], [-87.57067, 41.72983], [-87.5711288877394, 41.7298224772501], [-87.57242920382214, 41.729801303164244], [-87.5737307882272, 41.72978490857438], [-87.57475284972838, 41.72977261917119], [-87.57596041883711, 41.72976274930973], [-87.57716790545794, 41.7297467115252], [-87.57837545491961, 41.72973362169492], [-87.57949025961376, 41.72971774783652], [-87.58042, 41.729710000000104], [-87.58041413432204, 41.72918795466878], [-87.58040798504203, 41.72863059396159], [-87.5803878893323, 41.727516679972275], [-87.58035967993038, 41.72649495673222], [-87.58034178069788, 41.72547286815827], [-87.58030623817945, 41.724641001770756]]]}, "properties": {"city": "Chicago", "name": "Calumet Heights", "regionid": "273111", "geo_point_2d": [41.72973650176269, -87.57370893549677], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.52869355599992, 41.732790927000046], [-87.52867072299992, 41.73275678800007], [-87.52860128199988, 41.73263091400007], [-87.52855425899989, 41.73253164600003], [-87.52848825799991, 41.73236701300003], [-87.52843850399994, 41.732279817000055], [-87.52821124299992, 41.73194262100008], [-87.52816150999988, 41.73185949600007], [-87.52805271899987, 41.73165319500002], [-87.52794090899992, 41.73149324400003], [-87.5278919659999, 41.731414955000034], [-87.5269966029999, 41.72980429600004], [-87.52671641599991, 41.72932453700007], [-87.52666369099987, 41.72921888900003], [-87.52648972299993, 41.72880417600003], [-87.5259785299999, 41.72794189000006], [-87.52542472899992, 41.727036121000076], [-87.5254052139999, 41.72700260100003], [-87.52489397399992, 41.72607919700004], [-87.5241589259999, 41.72481740600005], [-87.52413259799992, 41.72476862600006], [-87.5241333619999, 41.72471151700005], [-87.52414225799991, 41.72404648800004], [-87.52414302199989, 41.723989391000075], [-87.52414375599992, 41.72393228100008], [-87.52415051499989, 41.72340626600003], [-87.52415124899991, 41.72334916800008], [-87.52448946699988, 41.72333495400004], [-87.52612653999994, 41.72333495400004], [-87.53281561499993, 41.723149146000075], [-87.53578853699989, 41.723149146000075], [-87.53681047899991, 41.72454270300005], [-87.53792532499992, 41.72575045300004], [-87.5380897839999, 41.72624916900008], [-87.53888, 41.72592], [-87.53828961599991, 41.725106204000035], [-87.53857565099989, 41.72500722200004], [-87.53792532499992, 41.72379947300003], [-87.53792532499992, 41.722970000000004], [-87.53996753699991, 41.72295262000005], [-87.53996920899992, 41.722591723000036], [-87.53941178599989, 41.722591723000036], [-87.53941178599989, 41.71971170500006], [-87.5388899999999, 41.71961880100007], [-87.53887238899992, 41.71599194700007], [-87.53886476799991, 41.71460207100005], [-87.5388761009999, 41.71348748700006], [-87.53904016999988, 41.71237230400003], [-87.5393188819999, 41.71125745800003], [-87.53959759299991, 41.710421323000034], [-87.54024791999989, 41.709585189000045], [-87.54145566999993, 41.70902776600007], [-87.54238470799991, 41.70800582400005], [-87.54312793799993, 41.707076786000066], [-87.54312793799993, 41.70605484400004], [-87.54312793799993, 41.70419676800008], [-87.54331374599988, 41.70336063300004], [-87.54340664999994, 41.703081922000074], [-87.54379, 41.70278], [-87.54379550699988, 41.702776527000076], [-87.54386991599992, 41.702652022000045], [-87.54412491499993, 41.70232802300005], [-87.5447559159999, 41.70146102200003], [-87.5450519909999, 41.701105057000056], [-87.54518291499991, 41.70038202200003], [-87.5451909159999, 41.700006023000064], [-87.54520191599993, 41.69952802200003], [-87.54521091599989, 41.69773102300006], [-87.54520391599992, 41.69729602300003], [-87.54520291599991, 41.69720502300003], [-87.54517791499987, 41.69666202200005], [-87.5451859159999, 41.696284023000075], [-87.54518091599992, 41.69551202200005], [-87.54517291599988, 41.694119023000034], [-87.54517791499987, 41.69369202200004], [-87.54519691499989, 41.692254022000036], [-87.54515891499993, 41.690396023000055], [-87.5452069159999, 41.68750102300003], [-87.5452379149999, 41.68627402400004], [-87.54519291599989, 41.68477602200004], [-87.5451679159999, 41.68392702200002], [-87.54517591599992, 41.68365902200008], [-87.54525999799993, 41.683204925000034], [-87.54487999099989, 41.682760015000035], [-87.54491113399988, 41.68217984000005], [-87.5449016349999, 41.68162257900007], [-87.54488999999987, 41.68094], [-87.54322074699988, 41.68095330200002], [-87.5400854499999, 41.680957468000074], [-87.54007368299992, 41.68007801300006], [-87.54005418699987, 41.679012210000046], [-87.5400421009999, 41.67835149900008], [-87.54002353299988, 41.67733644800005], [-87.5400077499999, 41.67647365600003], [-87.53999999999989, 41.67540822300002], [-87.5399969249999, 41.67454491600005], [-87.53998941899991, 41.673580354000066], [-87.5399807279999, 41.67246349300006], [-87.53991976699989, 41.67180325200007], [-87.53977310799989, 41.67125932400005], [-87.53958721199989, 41.67077701500005], [-87.53935604799993, 41.670306936000074], [-87.53908587599992, 41.66975827500004], [-87.53884859399993, 41.66906250300006], [-87.53879400999992, 41.66855480200006], [-87.5389277819999, 41.667798542000064], [-87.53917798999991, 41.66666955500005], [-87.53933935899991, 41.665879888000056], [-87.53955686699992, 41.66478630500006], [-87.53972393699989, 41.66400630300005], [-87.53988376699992, 41.66307271500004], [-87.53989806099989, 41.66190376700006], [-87.53989119899988, 41.660786351000056], [-87.53988999999987, 41.65987275500003], [-87.53989058699993, 41.65933653300004], [-87.54424278399993, 41.65941712900006], [-87.54452149599989, 41.65988164800006], [-87.5448002069999, 41.66099649400007], [-87.5448002069999, 41.66239005100003], [-87.54545053399991, 41.66471264600004], [-87.54572924499988, 41.66601330000008], [-87.54572965899989, 41.66670683000006], [-87.54643491599988, 41.66671202200006], [-87.54643455799992, 41.666751791000074], [-87.54721570599989, 41.66684943400008], [-87.54777312899992, 41.66684943400008], [-87.5503744359999, 41.66684943400008], [-87.55176799299993, 41.66684943400008], [-87.55371897299993, 41.666663626000044], [-87.55434444199992, 41.66660785200007], [-87.55427335199991, 41.66650077900004], [-87.5548304529999, 41.666521573000054], [-87.5559494429999, 41.66652450900004], [-87.55734186799991, 41.66651497000003], [-87.55864287499989, 41.666292011000046], [-87.55882868299992, 41.66527006900008], [-87.55882868299992, 41.663969416000036], [-87.55882868299992, 41.66211134000007], [-87.55910739499987, 41.6592], [-87.56112143899989, 41.65917054900005], [-87.5635589149999, 41.65918], [-87.56364917199993, 41.66425303500005], [-87.56364917199993, 41.66593488000006], [-87.56364917199993, 41.66723022100006], [-87.56364917199993, 41.66795356700003], [-87.56382968599989, 41.67643771600006], [-87.5639199429999, 41.680409019000074], [-87.5639199429999, 41.68374852400006], [-87.5642809699999, 41.685914689000064], [-87.5643712269999, 41.69114958900007], [-87.5644614839999, 41.69475986500003], [-87.56247614799989, 41.69575253300006], [-87.56245193199992, 41.695649617000065], [-87.56170901399989, 41.695482672000075], [-87.56096598899988, 41.69549500400006], [-87.55946, 41.69552], [-87.55950136999988, 41.69973722900004], [-87.55873577899989, 41.701502557000076], [-87.55762093299987, 41.70336063300004], [-87.55687770299991, 41.70391805600007], [-87.55520543399987, 41.70503290200008], [-87.55409058899991, 41.70559032500005], [-87.5534402619999, 41.70605484400004], [-87.55251122399993, 41.707076786000066], [-87.55167508899989, 41.708377440000035], [-87.55083895499989, 41.710421323000034], [-87.5503744359999, 41.71190778400005], [-87.54851635999994, 41.71664587900004], [-87.54721570599989, 41.71952589700003], [-87.54676459999988, 41.720628], [-87.54779203399993, 41.72164470200004], [-87.54841950699993, 41.72231666800007], [-87.54891565899992, 41.72284101600006], [-87.54881991499988, 41.72284102200006], [-87.54872791499989, 41.722840022000064], [-87.54872192299987, 41.722839994000076], [-87.54851491499993, 41.722839023000056], [-87.5481799149999, 41.72281302300006], [-87.5475559149999, 41.72281902200006], [-87.54742691599994, 41.722821022000055], [-87.54694391499991, 41.72281602300005], [-87.54623291499989, 41.722844022000054], [-87.54572491499991, 41.72285302300007], [-87.54562505999989, 41.722854964000064], [-87.54550491599989, 41.72285702200008], [-87.5442929149999, 41.72288402200006], [-87.543682, 41.722908], [-87.54321128199987, 41.722896041000055], [-87.54321128199987, 41.72315014600007], [-87.54309400199989, 41.72367790300006], [-87.54287898999989, 41.72422520700006], [-87.5426835249999, 41.72457704500004], [-87.54212998699988, 41.72507715300008], [-87.54126611499991, 41.72508628100007], [-87.54103577999987, 41.72508447700004], [-87.54156063799988, 41.72535623600003], [-87.54200226399989, 41.725568504000066], [-87.54217531399988, 41.72602349000003], [-87.54225350099992, 41.726492607000054], [-87.54217531399988, 41.72682489900005], [-87.54192120899992, 41.72721583000003], [-87.54147163799989, 41.72762630700003], [-87.54035748499989, 41.728290890000025], [-87.53918469099993, 41.72881864700007], [-87.53828554999988, 41.72913139200004], [-87.53613542899988, 41.73003053400003], [-87.5324606759999, 41.73153561900004], [-87.53003690299992, 41.73264977200005], [-87.5292354939999, 41.73288433100003], [-87.52869355599992, 41.732790927000046]], [[-87.54244645599992, 41.716706853000055], [-87.5423565709999, 41.71653609300005], [-87.54235432199991, 41.71658], [-87.5423406589999, 41.71673616500003], [-87.54230547799992, 41.71687234900008], [-87.54229495899992, 41.71690294800003], [-87.5427144799999, 41.71718313600006], [-87.5427487339999, 41.717204043000024], [-87.54272759899992, 41.71715633100006], [-87.54268192099991, 41.71708483400003], [-87.54249169599989, 41.71678648300007], [-87.54244645599992, 41.716706853000055]], [[-87.55167193299988, 41.67575210900008], [-87.55171101999989, 41.67516864700007], [-87.55167056799988, 41.675187626000074], [-87.55149257699989, 41.675253356000034], [-87.55133230299991, 41.67529581500007], [-87.55109509699989, 41.675333920000064], [-87.55089243799993, 41.67539679700006], [-87.55090712999991, 41.67540350700005], [-87.55120019099988, 41.675517144000075], [-87.55133425299992, 41.67556878100003], [-87.55146334299988, 41.675630719000075], [-87.55167193299988, 41.67575210900008]], [[-87.56255519599993, 41.66716915700004], [-87.5625856739999, 41.66712508300008], [-87.56252666199991, 41.66716697300006], [-87.56255519599993, 41.66716915700004]]]}, "properties": {"city": "Chicago", "name": "Calumet River", "regionid": "403388", "geo_point_2d": [41.69160764191001, -87.5494011639897], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.6745169079999, 41.92506100300004], [-87.6744673119999, 41.92482541900005], [-87.6744137619999, 41.92451750500003], [-87.6742531109999, 41.92431669200004], [-87.67383809599993, 41.923995390000066], [-87.6733695309999, 41.923674088000034], [-87.67271353999989, 41.92320552300003], [-87.67227174999991, 41.92287083400002], [-87.67187012299989, 41.92267002000005], [-87.67140155799989, 41.92253614500003], [-87.67095976899991, 41.92248259400007], [-87.6703707149999, 41.92241565700004], [-87.66962101199991, 41.922429044000054], [-87.66908550899988, 41.92240226900008], [-87.6685901689999, 41.92225500600005], [-87.66826886699992, 41.92206758000008], [-87.6681617659999, 41.921759666000064], [-87.66798772799989, 41.92150530200006], [-87.66751916299994, 41.92103673700005], [-87.6671041479999, 41.92056817200005], [-87.66652848299991, 41.92005944400005], [-87.66622056899988, 41.91971136700004], [-87.66577877899992, 41.91908215100005], [-87.66552441499991, 41.91873407500008], [-87.66524327599987, 41.91843954800003], [-87.6649219739999, 41.91815840900005], [-87.6645471219999, 41.91787727000006], [-87.66438647099991, 41.91747564300004], [-87.66425933899988, 41.91701252100005], [-87.66419904499989, 41.91679287700003], [-87.66398484399988, 41.91625737400005], [-87.66379741799992, 41.91586913500003], [-87.6635296669999, 41.915628158000054], [-87.66322175299987, 41.91552105800007], [-87.66287367599992, 41.91550767000005], [-87.66244527399992, 41.91557460800003], [-87.66207042199993, 41.91578880900004], [-87.66152153099993, 41.91602978500002], [-87.66116006699988, 41.91612349800003], [-87.66085215299992, 41.916083336000035], [-87.66037019999992, 41.91581558400007], [-87.65966065899993, 41.91541395700005], [-87.65892434299988, 41.91490522900005], [-87.6583486769999, 41.91434295100002], [-87.65798721299988, 41.913914549000076], [-87.6577997869999, 41.913419209000025], [-87.6574650979999, 41.91254901700005], [-87.65706346999988, 41.91154494900007], [-87.6567575479999, 41.91078014400005], [-87.65711062499989, 41.91077733900004], [-87.65903490199989, 41.91075245900002], [-87.6600871029999, 41.91073488200003], [-87.66089673299989, 41.91072337500008], [-87.66259557799991, 41.91070716700004], [-87.6638115729999, 41.910680720000066], [-87.6651057369999, 41.91066401300008], [-87.66507792399992, 41.91061533200007], [-87.66519, 41.91066], [-87.6662871489999, 41.91065247500005], [-87.66677580199988, 41.91064493900006], [-87.66762, 41.91063], [-87.66848523599991, 41.91062092300006], [-87.66927835299991, 41.91060380900006], [-87.6705596179999, 41.91058258700007], [-87.67184186899992, 41.91056190500006], [-87.67306223499989, 41.910546047000075], [-87.67446555799992, 41.91052899900006], [-87.67550277299993, 41.910511987000056], [-87.67654016399989, 41.91049599800004], [-87.67885918999988, 41.91045267400005], [-87.68087362899989, 41.91041856000003], [-87.68300893199994, 41.910368542000064], [-87.68423051699993, 41.91036], [-87.6853887099999, 41.91033854800007], [-87.6863656229999, 41.91032658900007], [-87.6871299999999, 41.91032], [-87.68719085099988, 41.910382400000046], [-87.68719814699993, 41.91091754100005], [-87.68721591699993, 41.911588384000076], [-87.6872358119999, 41.91238155300004], [-87.68725185099991, 41.91292946600003], [-87.6872751249999, 41.913726011000044], [-87.68728197199992, 41.91399402200005], [-87.68728218999989, 41.91399401800003], [-87.6872876599999, 41.914215973000054], [-87.68731883799988, 41.91525211400004], [-87.68733848099991, 41.91616710600004], [-87.68738022699993, 41.917203504000035], [-87.6873948279999, 41.91793696100006], [-87.68741413799992, 41.91879104200006], [-87.6874397679999, 41.91946064800003], [-87.6874761589999, 41.92031933200007], [-87.68750090999991, 41.92117184400006], [-87.68752344099993, 41.92226995600002], [-87.68754478999993, 41.92294090500007], [-87.68756720999991, 41.92367117600007], [-87.6875977969999, 41.92428008600006], [-87.68764269699993, 41.92538247000004], [-87.6876603509999, 41.92605332000004], [-87.68769640199989, 41.92690845300006], [-87.68772409999988, 41.92778776800008], [-87.68775, 41.92861], [-87.68746928599988, 41.928806116000075], [-87.6864847539999, 41.92957399300007], [-87.68489062899992, 41.93086103000007], [-87.68440906599992, 41.931249985000065], [-87.68402114299992, 41.93159290400006], [-87.68394869999989, 41.931792884000046], [-87.68394, 41.93221], [-87.68321167399992, 41.93222344500003], [-87.68321399999991, 41.932229], [-87.68289719999989, 41.932388524000075], [-87.68283454399993, 41.932268908000076], [-87.68263372999989, 41.93193421900007], [-87.6821785529999, 41.93165308000004], [-87.68136191099987, 41.93131839100004], [-87.68101383399993, 41.93113096500007], [-87.68078624499991, 41.93070256300006], [-87.68046494399991, 41.930006409000036], [-87.68009009199992, 41.929296868000044], [-87.67978217799991, 41.92861410200003], [-87.67963491399989, 41.92848022600002], [-87.67919312399991, 41.928332963000045], [-87.6784300329999, 41.928172312000065], [-87.67754645299992, 41.927917948000065], [-87.67686368699991, 41.92773052200003], [-87.67619430899991, 41.927435995000046], [-87.67521701599989, 41.92687371700003], [-87.67469490099988, 41.92643192800004], [-87.67449408699989, 41.926097238000054], [-87.67448069999992, 41.92581609900008], [-87.67449408699989, 41.92544124700003], [-87.67452086199992, 41.92507978300005], [-87.6745169079999, 41.92506100300004]]]}, "properties": {"city": "Chicago", "name": "Bucktown", "regionid": "137603", "geo_point_2d": [41.91843161239443, -87.67631513616733], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.65704067649581, 41.96174735264455], [-87.65698631134448, 41.96105419696529], [-87.65690477845419, 41.95758089583246], [-87.65682919194938, 41.954451614535714], [-87.65685791576658, 41.95445202180076], [-87.65697591581011, 41.95445202180076], [-87.65921891578277, 41.95442602240089], [-87.66220777623073, 41.95437394192052], [-87.66220999999989, 41.95437], [-87.66337164521885, 41.95437], [-87.66383619024441, 41.954365360753805], [-87.66533038543304, 41.95434600754073], [-87.66703076499545, 41.954318507130345], [-87.6682675527664, 41.9543048112825], [-87.66897, 41.95430000000011], [-87.66899743571763, 41.95529866012244], [-87.66904439123125, 41.95720540979517], [-87.66909076102644, 41.95905970136302], [-87.66911738599455, 41.95998650383405], [-87.6692740094022, 41.961588349587956], [-87.66923391518857, 41.96159102263161], [-87.66917291507463, 41.96159602196288], [-87.66771091602988, 41.96161502193913], [-87.66667991616839, 41.961621022215944], [-87.66595080612164, 41.961654180219135], [-87.6659469372343, 41.96164233482939], [-87.66545105198463, 41.96165484415568], [-87.6642826086042, 41.96166000000014], [-87.66304618143218, 41.96167497374612], [-87.66167175890251, 41.96169983438172], [-87.66036518744261, 41.9617142377722], [-87.65956999999987, 41.961720000000135], [-87.65885320109874, 41.961720000000135], [-87.6571359917943, 41.96174655469938], [-87.65704067649581, 41.96174735264455]]]}, "properties": {"city": "Chicago", "name": "Graceland West", "regionid": "403267", "geo_point_2d": [41.958028114829474, -87.6629986337845], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.6893599999999, 41.98305], [-87.68930904999988, 41.98239492700008], [-87.68926122999993, 41.98178010600008], [-87.68921729199988, 41.980751675000064], [-87.68921206299987, 41.979789530000055], [-87.68918844999988, 41.978759921000055], [-87.68916569299992, 41.97807340900004], [-87.68912858899994, 41.97710754000007], [-87.68912921299993, 41.97628614000006], [-87.68908011199991, 41.97586097000003], [-87.68908, 41.97586], [-87.68978054599994, 41.975856482000026], [-87.69094885199989, 41.97584901700007], [-87.69177353799992, 41.97584374700005], [-87.69287326099992, 41.97583410000003], [-87.69362915399994, 41.97582541200005], [-87.69493477699987, 41.97581494100006], [-87.69569074299993, 41.97581035900002], [-87.6974086669999, 41.97579218600003], [-87.69823445199994, 41.97577490500003], [-87.7002268169999, 41.975765337000034], [-87.7020132839999, 41.97576533400006], [-87.7037317219999, 41.97576], [-87.70448803999992, 41.97575593500004], [-87.70460446899993, 41.97575530900007], [-87.7046295079999, 41.97584545000006], [-87.7048035469999, 41.97640772800007], [-87.70495080999987, 41.976876293000075], [-87.70509807299992, 41.97739840900004], [-87.7052587239999, 41.97780003600006], [-87.70533904999988, 41.97813472500007], [-87.70559341399989, 41.97896475400005], [-87.7058076149999, 41.97966090800008], [-87.70608875399994, 41.98055787500003], [-87.70628956699993, 41.981200479000044], [-87.7064903809999, 41.98182969400005], [-87.70663764399987, 41.982298259000046], [-87.7068224309999, 41.982965544000066], [-87.70670099999988, 41.982967], [-87.70579591499988, 41.98297002200006], [-87.7049799149999, 41.98297402200005], [-87.7039389159999, 41.982980022000035], [-87.70274491599989, 41.98298702200008], [-87.70153491599987, 41.98299602200006], [-87.70028191599991, 41.982990022000074], [-87.69909112899988, 41.98300595300008], [-87.69908591499987, 41.98300602200003], [-87.69787791499992, 41.983009022000026], [-87.69667391599987, 41.98301302200007], [-87.69516591499988, 41.98302302200004], [-87.69505091499988, 41.98302402200004], [-87.6949539159999, 41.98302402200004], [-87.69420891499992, 41.983029022000025], [-87.69300491599994, 41.98303302200008], [-87.69176291599987, 41.983043022000054], [-87.69109891499991, 41.98304802200004], [-87.6905639149999, 41.983043022000054], [-87.68936101199992, 41.98305086900006], [-87.6893599999999, 41.98305]]]}, "properties": {"city": "Chicago", "name": "Budlong Woods", "regionid": "403262", "geo_point_2d": [41.97947779486626, -87.6974812075734], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.62078214386248, 41.70112998592466], [-87.62085, 41.69988], [-87.62014468797254, 41.69989137600034], [-87.6190301298926, 41.69990966450181], [-87.61772983240158, 41.699931150288386], [-87.61652137352878, 41.69994548054491], [-87.61531327738948, 41.699961037354804], [-87.61382724607226, 41.699985366998874], [-87.61299142680959, 41.69999460985887], [-87.61187662522947, 41.700007573996366], [-87.61020479307355, 41.70003455621034], [-87.6093683585727, 41.70004], [-87.6087192288335, 41.700054399098825], [-87.60805041036338, 41.70007043756494], [-87.60792591566837, 41.70066602241384], [-87.60762991551368, 41.70193202231617], [-87.6074309153337, 41.7028270222038], [-87.60723691538406, 41.70373702278297], [-87.6070559160339, 41.70463602285503], [-87.60685991509291, 41.7056030224643], [-87.60666191585831, 41.7064920220753], [-87.60646591581651, 41.70739802247001], [-87.60645991553996, 41.707425022815265], [-87.60645914781134, 41.707428203208366], [-87.60555921096903, 41.70744078789268], [-87.60277265388291, 41.707485929765504], [-87.60281105642866, 41.70568225715122], [-87.60269952631533, 41.704753771908386], [-87.60268799836261, 41.70419658752629], [-87.60266202073831, 41.70345441015952], [-87.60263390830413, 41.70289840868329], [-87.60262, 41.702], [-87.60314380295252, 41.70196], [-87.60444427619281, 41.701952602371165], [-87.60537317040723, 41.70194103937253], [-87.60626, 41.70193], [-87.60623679163821, 41.7012260130256], [-87.60621527471113, 41.70057333290507], [-87.6062, 41.69973738469521], [-87.60620808342071, 41.69899412093818], [-87.60614999999989, 41.698250000000165], [-87.60667409444613, 41.69824514727376], [-87.60751003832036, 41.698235554947274], [-87.6084909156326, 41.69822302221509], [-87.60859991616059, 41.697801022547615], [-87.60895891562211, 41.69623302221386], [-87.60915991589428, 41.69540402267054], [-87.60929191568387, 41.69488502212887], [-87.60949491514883, 41.69389802249698], [-87.60975791558104, 41.692767022522425], [-87.60997291559927, 41.69182302217458], [-87.61018291538706, 41.6909100223567], [-87.61060191581572, 41.68913202223525], [-87.6110059155531, 41.68740702275831], [-87.6110199152991, 41.68733202289838], [-87.61117991547985, 41.6863490225514], [-87.61132491586869, 41.6854630221794], [-87.61132889896602, 41.68543596787481], [-87.61193391516316, 41.68543102250306], [-87.61207691545972, 41.68542902241088], [-87.61222693676348, 41.68542807542474], [-87.61249391579622, 41.685425022226426], [-87.61311891582831, 41.685413022572625], [-87.61437991550018, 41.68540202206545], [-87.61564391531034, 41.68539102245783], [-87.6167199154475, 41.68537502261957], [-87.61742591561614, 41.68536902234303], [-87.61801591583344, 41.685364022112495], [-87.61874491526369, 41.685363022066404], [-87.62075991641746, 41.685333022482155], [-87.6227709155881, 41.685316022597725], [-87.62281791595605, 41.685317022643815], [-87.62285691595518, 41.685320022782186], [-87.62410591597317, 41.685301021906376], [-87.62499691567645, 41.68528002183757], [-87.62514691539637, 41.68527902179149], [-87.62526091525534, 41.68527802174539], [-87.62557191520222, 41.68527602255259], [-87.62648191578137, 41.68526602209159], [-87.62762591545584, 41.68526302195341], [-87.62885991568173, 41.685255022483794], [-87.62946291559895, 41.68525102229952], [-87.63004991567807, 41.685232022323], [-87.63124391585866, 41.685199022600386], [-87.63243991523223, 41.685209022162105], [-87.63354391576075, 41.68520302188547], [-87.63486591554664, 41.68518702204731], [-87.63604591598119, 41.685172022255145], [-87.63727591512345, 41.685157022462896], [-87.6377179157131, 41.685152022232536], [-87.63851191544154, 41.685129022071635], [-87.6397189153222, 41.68509502230286], [-87.64035191572293, 41.68507902246479], [-87.64095691573247, 41.685064022672705], [-87.64205491508517, 41.685060022488265], [-87.64211991628275, 41.685060022488265], [-87.64219691983216, 41.685059966730286], [-87.64227291524166, 41.686879021801666], [-87.64227191609487, 41.687142023133305], [-87.64227191519558, 41.68818002241797], [-87.64231591542517, 41.68871302180641], [-87.64233391535561, 41.689218022602084], [-87.64238791514673, 41.69009702265154], [-87.64239791560772, 41.69053002282637], [-87.64240491593046, 41.691079022952444], [-87.64244891526067, 41.692327022025054], [-87.64244991530677, 41.692350022185934], [-87.64245091535294, 41.6923770225312], [-87.64250491604345, 41.69346802246043], [-87.6425109154207, 41.69416302212207], [-87.64255791578866, 41.6959880217121], [-87.64260291516496, 41.69735302258071], [-87.6426289154643, 41.697803022639704], [-87.64265891504846, 41.698324022374344], [-87.64269482677581, 41.69960303726506], [-87.64269485739167, 41.69960402402151], [-87.64197815560267, 41.699614618092106], [-87.64086400960446, 41.699628294924565], [-87.6396557802866, 41.699644153106895], [-87.63835519630335, 41.699654957407375], [-87.63724041764961, 41.69966406164509], [-87.63593988513787, 41.69967479088135], [-87.63483036792154, 41.69968522057525], [-87.63296592764027, 41.69976], [-87.63045680649697, 41.6998013836895], [-87.62906413905712, 41.69980333846548], [-87.62739157733624, 41.699815478874335], [-87.62636971668212, 41.69982790392119], [-87.62488317489475, 41.69983729611888], [-87.62404718858109, 41.699833498401894], [-87.62328, 41.69984], [-87.62331602050176, 41.70159704559116], [-87.62338737584969, 41.70401127548964], [-87.62341145195462, 41.704939693821096], [-87.62343106959013, 41.70568208770569], [-87.62346352776115, 41.706703179998804], [-87.62347865863875, 41.707539333512344], [-87.62350140219895, 41.70856300665282], [-87.62351366526764, 41.70930620882321], [-87.62353005568218, 41.71032753353894], [-87.62355285156637, 41.711165546992596], [-87.62357682578846, 41.71200081206073], [-87.62359687578122, 41.71292941015772], [-87.62361, 41.71367295697925], [-87.62351446236424, 41.71472805339215], [-87.62268053001804, 41.71616156660625], [-87.62263519572822, 41.71654418744312], [-87.62281786742928, 41.717304377768386], [-87.62304701657399, 41.71793866128193], [-87.62329538918465, 41.71863308971731], [-87.62351663145326, 41.719460630856204], [-87.62359577781052, 41.72018226709861], [-87.62365875985358, 41.72101363006882], [-87.6236937498777, 41.72147549838565], [-87.6236963244985, 41.72168133442729], [-87.62297189525505, 41.72172075436438], [-87.62139325086571, 41.72177782043101], [-87.62058226695405, 41.72178765356451], [-87.62058, 41.72138397364302], [-87.62057114476323, 41.72054853435913], [-87.6205559675226, 41.71961949924061], [-87.62052906072876, 41.7185998744941], [-87.62049655921241, 41.717854786218055], [-87.62044760964802, 41.71683267536477], [-87.62042716063398, 41.715812104091675], [-87.62039682466026, 41.71497554524423], [-87.62016728129443, 41.7131266002847], [-87.62007587274871, 41.71238077703585], [-87.6199795054391, 41.71162599710183], [-87.6199100851552, 41.711172095245644], [-87.61986498356261, 41.710609276757296], [-87.6198586524817, 41.71033070919982], [-87.6199612174886, 41.70994757564064], [-87.61987144514747, 41.70939957454831], [-87.61995599883346, 41.708562777665406], [-87.62006679346791, 41.70783929353565], [-87.62021732437752, 41.70690380037663], [-87.62040203450027, 41.70577296194804], [-87.62047241489547, 41.70512998721201], [-87.6205287493128, 41.70447650797124], [-87.62065473320713, 41.70298477789489], [-87.6207445138059, 41.70197078525099], [-87.62078214386248, 41.70112998592466]]]}, "properties": {"city": "Chicago", "name": "Roseland", "regionid": "275251", "geo_point_2d": [41.69536525715409, -87.62448669124214], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.83505129299992, 41.98615154400004], [-87.83496304599991, 41.98612894600006], [-87.83488876299991, 41.986129535000074], [-87.83426305699992, 41.98612940500003], [-87.83371701299988, 41.98604547400003], [-87.8329630229999, 41.98592946300004], [-87.83186297899992, 41.98572951700003], [-87.83056304199992, 41.98562943600006], [-87.82936302499988, 41.98542946100007], [-87.82886299199991, 41.98532949500008], [-87.82857669299993, 41.985288571000055], [-87.82816301699994, 41.98522946900003], [-87.8268629899999, 41.985029503000064], [-87.8256630399999, 41.984929439000034], [-87.82546338599988, 41.984896203000055], [-87.82537062899989, 41.98488074300007], [-87.82446302499993, 41.98472946100007], [-87.82396423899988, 41.98463006200005], [-87.82395094199987, 41.98462818600006], [-87.82336199999989, 41.98453], [-87.82330665599991, 41.98374673300008], [-87.82336029799987, 41.98339063600007], [-87.8234637819999, 41.98272007700005], [-87.8235815179999, 41.98180962700007], [-87.82371929999991, 41.980778887000035], [-87.82381967499991, 41.980055385000064], [-87.82392821599987, 41.979203998000045], [-87.82406171599987, 41.978234081000025], [-87.8242032339999, 41.977204782000044], [-87.82434669999991, 41.976227470000076], [-87.8244647779999, 41.97531655300008], [-87.82456570499991, 41.97453189600008], [-87.82462572899992, 41.974043189000035], [-87.82476296999992, 41.97402957100007], [-87.82576305499991, 41.97412938100006], [-87.82686250599993, 41.97402930400005], [-87.82686301299991, 41.973972361000044], [-87.82686301299991, 41.97358658700005], [-87.82686301299991, 41.97352947400008], [-87.82686301299991, 41.973472361000056], [-87.82686301299991, 41.97312222500004], [-87.82686301299991, 41.973086587000054], [-87.82686301299991, 41.97306511200008], [-87.8268635079999, 41.97302968000007], [-87.83205026099994, 41.97292971900002], [-87.83206313099991, 41.97292956100005], [-87.83205026699991, 41.97298683000007], [-87.83194376499993, 41.97346609000004], [-87.8318752509999, 41.97377440200005], [-87.8318630329999, 41.97382945400005], [-87.83187525799988, 41.97383151500008], [-87.83195693899992, 41.97384512800004], [-87.8320766979999, 41.973865088000075], [-87.83246306099993, 41.97392940000008], [-87.83320231399993, 41.97392947400004], [-87.83325586799991, 41.97392947400004], [-87.8332629789999, 41.97392953100005], [-87.83325587399989, 41.97398658600007], [-87.83320897899993, 41.97436174500005], [-87.83317015199991, 41.97467236200003], [-87.8331630469999, 41.97472941700004], [-87.83317015899989, 41.974729474000064], [-87.83356716599988, 41.974729474000064], [-87.83374396999992, 41.974729474000064], [-87.8337628989999, 41.97472959900005], [-87.83374397599992, 41.97478658400006], [-87.83366312699988, 41.975029348000064], [-87.83366301299989, 41.97508658700007], [-87.83366301299989, 41.97525808100004], [-87.83366301299989, 41.97531519400007], [-87.83366301299989, 41.975372334000035], [-87.8336630189999, 41.97542945800006], [-87.8365372029999, 41.975336758000076], [-87.8367488579999, 41.975329931000026], [-87.83676259799994, 41.97532915200008], [-87.83674884899993, 41.97527281700007], [-87.83667015299991, 41.97495803600008], [-87.83666279199988, 41.97492919300002], [-87.83667015199988, 41.97490091700006], [-87.8367634239999, 41.97452979200004], [-87.84141790099993, 41.97452947400006], [-87.84148792799992, 41.97452947400006], [-87.84176252199988, 41.97452925800006], [-87.84176301299992, 41.974472361000025], [-87.84176301299992, 41.97368658700008], [-87.84176299199991, 41.97362954700007], [-87.84106303299987, 41.97372940100006], [-87.83856301299993, 41.97372947400003], [-87.83816333999992, 41.97372961700005], [-87.83806271399992, 41.97142927700008], [-87.83799795599991, 41.97142405300007], [-87.83686299199988, 41.97132951300005], [-87.8368630129999, 41.97127236100003], [-87.8368630129999, 41.97126589300006], [-87.8368630129999, 41.97120878000003], [-87.8368630129999, 41.97115166700006], [-87.8368630129999, 41.97088658700004], [-87.83686333999991, 41.97082961700004], [-87.83677107599992, 41.97018591600004], [-87.83676766799992, 41.970162062000036], [-87.83676335999988, 41.97012956100007], [-87.83677106799988, 41.97012880300007], [-87.83796299799988, 41.970029516000054], [-87.83856301299993, 41.97002947400005], [-87.83926301299992, 41.97002947400005], [-87.84046303199993, 41.970029410000045], [-87.8412629139999, 41.96992964000008], [-87.8416621409999, 41.97002932200006], [-87.8415637189999, 41.97151899900006], [-87.84156302899991, 41.97152944200006], [-87.8415630129999, 41.97158658700003], [-87.8415630129999, 41.971859361000035], [-87.84156301199992, 41.97191647500004], [-87.84556301199987, 41.971929475000074], [-87.84646809199987, 41.971934411000056], [-87.84666252299988, 41.971935255000055], [-87.84666301299988, 41.971878361000044], [-87.84666301299988, 41.971486587000065], [-87.84666333999989, 41.97142961700006], [-87.8465626859999, 41.969929330000056], [-87.8465630129999, 41.96987236100004], [-87.8465630129999, 41.969186587000074], [-87.8465630129999, 41.96912947400005], [-87.8465630129999, 41.96907236100007], [-87.8465630129999, 41.96818658700004], [-87.84656302599988, 41.968129447000074], [-87.84665917699992, 41.966378892000066], [-87.84666179999992, 41.966329498000036], [-87.8466592429999, 41.96632252600005], [-87.8466415599999, 41.96627459600005], [-87.84646, 41.96578246600006], [-87.84646, 41.96529670300002], [-87.84645665999992, 41.964629102000046], [-87.8464515999999, 41.96396121700007], [-87.84646127399989, 41.96323138400003], [-87.84646487999991, 41.96256378200007], [-87.84641544199988, 41.96200885800004], [-87.84629741499991, 41.96162050400004], [-87.84618722199988, 41.96135351700007], [-87.84586419299988, 41.960826236000045], [-87.8452514009999, 41.96019467200004], [-87.8446814689999, 41.95982499900002], [-87.8442477219999, 41.95959886100007], [-87.84392413099991, 41.959484857000064], [-87.8429686889999, 41.959248822000056], [-87.84249061399991, 41.95919969700008], [-87.8418848769999, 41.95919325800007], [-87.84103488899989, 41.95920233200006], [-87.84042775399988, 41.95920881400007], [-87.83878906099993, 41.959230903000034], [-87.83739296899994, 41.95925666100003], [-87.83666783599993, 41.95927103500008], [-87.83666891499989, 41.95662802200006], [-87.83673091599988, 41.95556202300008], [-87.83678091499988, 41.954637022000036], [-87.8367889159999, 41.95426302200007], [-87.83677391599988, 41.95379602300004], [-87.83677891599989, 41.95195202300004], [-87.83676191599993, 41.95050602200006], [-87.83676991599991, 41.95011702300007], [-87.83678091499988, 41.949540024000044], [-87.83674591499991, 41.94892102200003], [-87.8367029149999, 41.94849002200004], [-87.83666991499987, 41.948265023000026], [-87.83655491599987, 41.94718202200004], [-87.8365269149999, 41.94647602300005], [-87.8364869149999, 41.94563502300007], [-87.83647991599989, 41.94533102200005], [-87.83646691599994, 41.94480702200007], [-87.8364600439999, 41.944626888000066], [-87.8363568689999, 41.94368412900007], [-87.8362972129999, 41.94283611300005], [-87.83622619099991, 41.94137049600004], [-87.83612656299992, 41.939429254000046], [-87.83608245399994, 41.93863896200008], [-87.83605204699988, 41.93809417600004], [-87.83602896699992, 41.937539708000074], [-87.83616200399987, 41.93752950000003], [-87.84416200799987, 41.93732948600006], [-87.84666201999991, 41.93732945600003], [-87.8494217649999, 41.93723431000006], [-87.84956172599992, 41.937229649000066], [-87.84963607899991, 41.93728464600007], [-87.84971111999992, 41.93734001500007], [-87.85156260599987, 41.938728630000064], [-87.85337932299988, 41.943093755000056], [-87.85339523999993, 41.94312287300004], [-87.85341973799989, 41.94318310100003], [-87.8545637489999, 41.94593055600006], [-87.85458036299988, 41.94598586500006], [-87.85496032299993, 41.947030755000064], [-87.85525325599988, 41.94878835200006], [-87.85526367599992, 41.94882968000007], [-87.85525661099987, 41.94884439800005], [-87.85406302699991, 41.951328994000036], [-87.85372684899993, 41.95134500200003], [-87.84989095199988, 41.95152766400007], [-87.84776397499988, 41.95162947100004], [-87.84766625999991, 41.95260661800005], [-87.84766402799993, 41.95262894600005], [-87.84766631199993, 41.952663210000075], [-87.8478178859999, 41.95499420300006], [-87.84786032299989, 41.955630755000065], [-87.84786402799992, 41.95562894600003], [-87.8517630349999, 41.95552897200007], [-87.8522310459999, 41.955528946000065], [-87.8522630699999, 41.95551342500004], [-87.85256307099988, 41.955528895000036], [-87.85276304599994, 41.955528946000065], [-87.8528950009999, 41.95550513300008], [-87.8530440949999, 41.955518859000044], [-87.8578096179999, 41.955344784000026], [-87.8578919839999, 41.95534213200005], [-87.85793420399992, 41.95534072100008], [-87.85802301199992, 41.95533747500007], [-87.85826267699989, 41.95532935200003], [-87.85914789299991, 41.957197553000064], [-87.85916303999993, 41.95722983000007], [-87.8591478959999, 41.95725466900006], [-87.85867101299992, 41.95804947400005], [-87.8577540129999, 41.95957847400006], [-87.85646315799993, 41.96172941700006], [-87.8557629079999, 41.96452952400005], [-87.8549631679999, 41.96612937200006], [-87.85485948599987, 41.96706277700007], [-87.85470968899993, 41.96842182200003], [-87.85465884799991, 41.96888015600006], [-87.8545452839999, 41.96990373700003], [-87.85454394299988, 41.96990855200004], [-87.85436991199991, 41.971476294000055], [-87.85436379199987, 41.97152958300005], [-87.85436991899988, 41.971533412000035], [-87.85581575299992, 41.97249687600003], [-87.85586454699988, 41.972528626000035], [-87.85603146599992, 41.97247188200002], [-87.85608506899987, 41.97245368300007], [-87.85655239299989, 41.97229337300006], [-87.85687408999989, 41.97218297400006], [-87.85880586399989, 41.971520652000024], [-87.8589370429999, 41.97147583200006], [-87.8589519499999, 41.971470753000055], [-87.85936369299989, 41.97132980000003], [-87.85987811799991, 41.97175414300004], [-87.85988375599992, 41.97175767700008], [-87.8599033549999, 41.971773747000036], [-87.86003388499992, 41.97188160100006], [-87.8603316559999, 41.97212763900006], [-87.86152847399988, 41.97311615900008], [-87.86166465599989, 41.97322863900006], [-87.86172264899993, 41.973217422000054], [-87.86181889899989, 41.973495060000054], [-87.86184221499988, 41.97359765100003], [-87.86187019399993, 41.97380749500007], [-87.86187904599987, 41.97387610200006], [-87.86188723799992, 41.97393390800004], [-87.86188006799993, 41.97393268500007], [-87.86186424299987, 41.973929932000026], [-87.8617463449999, 41.97414800200004], [-87.86174169199987, 41.97415617700005], [-87.8617463449999, 41.97420511700004], [-87.86175967399991, 41.97434614200005], [-87.8617632819999, 41.97438170900005], [-87.86175967399991, 41.97440326000008], [-87.86165582699992, 41.97507956500004], [-87.86125384899991, 41.975736429000044], [-87.86085094299992, 41.97607540600005], [-87.86064573799987, 41.97619258000008], [-87.8604638609999, 41.97652945500004], [-87.86013101199988, 41.97686347300002], [-87.85826401299992, 41.97872947400002], [-87.8566638859999, 41.98032933300003], [-87.85666210899987, 41.98038652700006], [-87.85665691399991, 41.98054242500006], [-87.85665502999989, 41.98059947500007], [-87.8566531789999, 41.98065652800005], [-87.8565658509999, 41.98337241400008], [-87.8565648599999, 41.98342946400003], [-87.85656521299988, 41.983486411000044], [-87.85658448199989, 41.98419240300005], [-87.85658562999991, 41.98424948500007], [-87.8566587379999, 41.98479044300006], [-87.85666421199993, 41.98482977000002], [-87.8566587629999, 41.984847557000066], [-87.85645701299988, 41.98554247400005], [-87.8560890129999, 41.98680847300004], [-87.85576408199994, 41.98792940600003], [-87.85566928799992, 41.988876732000044], [-87.85566373299989, 41.98892958100003], [-87.8556692949999, 41.988933850000066], [-87.85635790199989, 41.98950446100007], [-87.85644503799993, 41.98957677700007], [-87.85635804899992, 41.98956088400006], [-87.8562735109999, 41.989546461000074], [-87.85600710199991, 41.98949826400007], [-87.85010701299989, 41.988433473000036], [-87.84837601299994, 41.98812047400008], [-87.84806168099992, 41.98806373100006], [-87.8474242099999, 41.98794794300004], [-87.84676409799991, 41.987828853000046], [-87.84643887599991, 41.98779633100008], [-87.84624203499992, 41.98777664700008], [-87.84576412699992, 41.98772881600006], [-87.84396400199994, 41.98746937000003], [-87.8419440129999, 41.98717747400007], [-87.84122101299992, 41.98707347300006], [-87.83676301199992, 41.986429474000026], [-87.83602901299989, 41.986324474000064], [-87.83536305299991, 41.98622943500003], [-87.83505129299992, 41.98615154400004]]]}, "properties": {"city": "Chicago", "name": "O'Hare", "regionid": "269603", "geo_point_2d": [41.966175627980896, -87.84462902341924], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.71234999972324, 41.71343300028936], [-87.71215000039655, 41.711532999939855], [-87.7110500000523, 41.711532999939855], [-87.7110500000523, 41.709832999816875], [-87.7110500000523, 41.70778900002434], [-87.7110500000261, 41.70613300006541], [-87.71221208231198, 41.70603165233395], [-87.7131407945199, 41.70599743540165], [-87.714348184948, 41.70597466324184], [-87.71555673840636, 41.705962913048836], [-87.71667129709117, 41.70595513303808], [-87.71797248055623, 41.70593828739711], [-87.71955228343204, 41.705910000000145], [-87.72065100015924, 41.70588425470642], [-87.72065100015924, 41.705932999904526], [-87.72085100038532, 41.707833000253935], [-87.72080099987907, 41.70873000023365], [-87.72075100027224, 41.709632999590625], [-87.72095099959893, 41.711432999827046], [-87.72095099959893, 41.71233299994494], [-87.72095099959893, 41.713149161666266], [-87.72095099959893, 41.713233000063106], [-87.71975100004109, 41.713333000176284], [-87.71855099958384, 41.713333000176284], [-87.71735100002583, 41.71343300028936], [-87.71605100035498, 41.71343300028936], [-87.71485099989758, 41.71343300028936], [-87.71355100022663, 41.71343300028936], [-87.71234999972324, 41.71343300028936]]]}, "properties": {"city": "Chicago", "name": "Tally's Corner", "regionid": "403370", "geo_point_2d": [41.70960881622243, -87.7160854159501], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.76643274930359, 41.93917489601013], [-87.76641035608013, 41.93849703649327], [-87.76641030990417, 41.93849703675518], [-87.76638048142644, 41.93758984398468], [-87.76631084675313, 41.9356376089054], [-87.76623765060273, 41.93386596386548], [-87.7661795393671, 41.932095873924105], [-87.766137706259, 41.93063063478919], [-87.76611770537886, 41.92989886674881], [-87.7660782481157, 41.92886058260526], [-87.76604879989273, 41.927945995175634], [-87.76603303615086, 41.92733644196161], [-87.76600918076876, 41.92660487691334], [-87.76597205942268, 41.92550581372941], [-87.76594116051932, 41.92471127050042], [-87.76588605012948, 41.923371127917825], [-87.76578920715717, 41.92086542187703], [-87.76572470776023, 41.91946480462676], [-87.76564349575432, 41.917812150227384], [-87.765607926068, 41.91652102249262], [-87.76564791599412, 41.91652102249262], [-87.76647891562946, 41.916522022538885], [-87.76803691550231, 41.91650902193944], [-87.76961991562811, 41.916464022563126], [-87.77504291559836, 41.91638002228839], [-87.77530891526945, 41.91637602210377], [-87.77536281341762, 41.916375059590756], [-87.77538792524625, 41.917201002243964], [-87.77542676667348, 41.91848234311808], [-87.77545766203575, 41.91964753964081], [-87.77548, 41.92049], [-87.77618028653953, 41.920613128353274], [-87.77709777847723, 41.92078064150408], [-87.77834327128267, 41.92101353853259], [-87.77953632680914, 41.92123133494274], [-87.78095160597002, 41.921471174540706], [-87.78204084107159, 41.921650501266434], [-87.7831635123868, 41.92191599517641], [-87.78451657462266, 41.92230255392283], [-87.7853, 41.922530000000194], [-87.78536295789466, 41.92422476140492], [-87.78543097641898, 41.92605319824552], [-87.78549836156888, 41.9280057623616], [-87.78553946412845, 41.929410172754466], [-87.78556757291159, 41.93045019773305], [-87.78559811692939, 41.931485212512], [-87.785620663232, 41.93221714099614], [-87.78564756966277, 41.93319681123422], [-87.78567326116182, 41.93386479020929], [-87.78571143836463, 41.93484214633011], [-87.78574339861864, 41.935634637149484], [-87.7857722717706, 41.9367949684313], [-87.78586062517864, 41.937931688268314], [-87.78592208452233, 41.938233096998104], [-87.78592591544572, 41.93823302239619], [-87.78593018540255, 41.93823293668137], [-87.78596739967563, 41.93882239157054], [-87.78600109179843, 41.93979681515166], [-87.78604462096784, 41.9407070258089], [-87.78608216903204, 41.941494345480756], [-87.7861, 41.9419], [-87.78486803721813, 41.941910904566285], [-87.78256107122193, 41.94194233245413], [-87.78013262760035, 41.94197231323964], [-87.7776419021208, 41.942006662958796], [-87.77625697194769, 41.94201134506826], [-87.77497144447423, 41.94201997465845], [-87.77254226657656, 41.942061197258006], [-87.77072032068506, 41.942080000000125], [-87.7681703680309, 41.94211631902001], [-87.76654010842917, 41.94213902331009], [-87.76652891192428, 41.94179996568297], [-87.76650292690101, 41.941005449329644], [-87.76646784588948, 41.9400304554608], [-87.76643274930359, 41.93917489601013]]]}, "properties": {"city": "Chicago", "name": "Belmont Central", "regionid": "403109", "geo_point_2d": [41.930343766804356, -87.77528540884309], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.68296919787696, 41.75750101908877], [-87.6829507369009, 41.756874107589695], [-87.68293035355426, 41.75613166728927], [-87.68289866283389, 41.7549249432515], [-87.68286983851691, 41.75399680024692], [-87.68285098696441, 41.75334651768294], [-87.68283593247726, 41.75278950165924], [-87.6828043117459, 41.75157971681947], [-87.68278220071005, 41.75093166828517], [-87.68286, 41.75007], [-87.68271412645261, 41.75005973867451], [-87.68274, 41.75002], [-87.68273180389816, 41.74971870525248], [-87.68270732060597, 41.748883495759976], [-87.68268531226842, 41.74776936805407], [-87.68265777748712, 41.74693110210052], [-87.68263021682615, 41.74572452736085], [-87.68261663498873, 41.74488857449711], [-87.68260684014183, 41.74451732326256], [-87.68257888336147, 41.74358917082978], [-87.6825, 41.74274], [-87.68372841634556, 41.74273146247069], [-87.68484232275455, 41.742707319932414], [-87.68577131674421, 41.74268862492358], [-87.68764, 41.74267], [-87.68763053229816, 41.738292468133075], [-87.68763053229816, 41.73624858415999], [-87.68759286997886, 41.735533000091976], [-87.68815000024468, 41.735533000091976], [-87.68885000013678, 41.735533000091976], [-87.68965000014167, 41.735533000091976], [-87.68994999958164, 41.73543299997881], [-87.69115000003886, 41.735533000091976], [-87.69234999959671, 41.735533000091976], [-87.69355000005403, 41.73543299997881], [-87.69494999983813, 41.73543299997881], [-87.6960500001822, 41.73543299997881], [-87.69734999985322, 41.73543299997881], [-87.69782926564977, 41.73528921997003], [-87.69835000008437, 41.73513299963966], [-87.69929247248403, 41.735258579169205], [-87.69985100002765, 41.73533299986582], [-87.70095100037179, 41.73533299986582], [-87.70206919634937, 41.73542618300356], [-87.70200002771223, 41.73634110711115], [-87.70199193361317, 41.73699133307347], [-87.70198498142021, 41.73754982591238], [-87.70198411938674, 41.73829190910576], [-87.70204, 41.73878], [-87.7019956994393, 41.739036474768575], [-87.7020225969406, 41.740057908221594], [-87.70205010150642, 41.740893146698824], [-87.70208489713563, 41.74219526824115], [-87.70209, 41.74243], [-87.70209890970729, 41.74275253140333], [-87.70211686083506, 41.743402362234704], [-87.702147046749, 41.74442321547539], [-87.70217915826032, 41.74525811476982], [-87.70222537448878, 41.74646412942107], [-87.70225727224238, 41.74730258465662], [-87.70229175636405, 41.7483291072916], [-87.70230694708768, 41.748883568699235], [-87.70233, 41.75038], [-87.7016741546089, 41.750683400649294], [-87.70042897952754, 41.751252454243314], [-87.69906513453736, 41.751873355568954], [-87.69738321612961, 41.752594771141304], [-87.69620736583897, 41.753128777857754], [-87.69442767827746, 41.753934144220416], [-87.69337269825989, 41.75441935424546], [-87.69143883360641, 41.755292130833446], [-87.68911545394042, 41.75631215797691], [-87.68769990206818, 41.75691292793149], [-87.6866753838882, 41.75736582416951], [-87.6866528925673, 41.75736589135932], [-87.6865719158131, 41.75740202252094], [-87.68649493924337, 41.75743715363673], [-87.68549491562975, 41.75745702235834], [-87.68490691550471, 41.7574710221045], [-87.68427491514991, 41.7574710221045], [-87.68390584777832, 41.75748403349549], [-87.68406298811507, 41.7574330095609], [-87.68321491575027, 41.75745502226616], [-87.68304291411698, 41.75747702328041], [-87.68297091529463, 41.757501018990716], [-87.68296919787696, 41.75750101908877]]]}, "properties": {"city": "Chicago", "name": "Wrightwood", "regionid": "403151", "geo_point_2d": [41.74581331726431, -87.69249795950904], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.54800120099989, 41.75195988900003], [-87.54800499299994, 41.75148464200003], [-87.54799272699994, 41.75083454700006], [-87.54797421499988, 41.74981290600004], [-87.54796028299988, 41.74897700400003], [-87.54793651999992, 41.74804995900006], [-87.54789602799988, 41.74711864200008], [-87.54786342499993, 41.746283301000055], [-87.54785383999989, 41.74526059200007], [-87.54785, 41.7447], [-87.5484605479999, 41.74469491200005], [-87.54948489099989, 41.744683846000044], [-87.5505052159999, 41.74465585600006], [-87.5513411869999, 41.744642049000035], [-87.55134063299992, 41.74405250500006], [-87.55134063299992, 41.743123467000075], [-87.5511548249999, 41.74265894800003], [-87.55059740199992, 41.74228733200004], [-87.55022578699992, 41.742008621000075], [-87.54976126799993, 41.74154410200003], [-87.54929674899994, 41.74107958300004], [-87.54883222999987, 41.74042925600007], [-87.5484606139999, 41.739871833000045], [-87.54827480699991, 41.73931441000007], [-87.54818190299991, 41.73894279500006], [-87.54808899899989, 41.73838537200003], [-87.54808899899989, 41.73743], [-87.54660111499992, 41.73744892000006], [-87.54660253799992, 41.739407314000054], [-87.54465116399987, 41.739290559000025], [-87.5296276109999, 41.73940885500007], [-87.52872015199992, 41.739416], [-87.52869874399991, 41.73926637800007], [-87.52869558799993, 41.739241937000074], [-87.52826858899991, 41.73553593800005], [-87.52826479999992, 41.735494750000036], [-87.52821861499989, 41.73482369900005], [-87.52815321199992, 41.734537056000065], [-87.52814265299989, 41.73448326900006], [-87.52811231299994, 41.73429918000004], [-87.52810902599991, 41.73427990000004], [-87.5281034309999, 41.73424891700005], [-87.52809555599991, 41.73420068400008], [-87.52806168199992, 41.73399544900006], [-87.52804967799992, 41.733849], [-87.52805405399992, 41.73376039500003], [-87.52806405299991, 41.73365939500006], [-87.52807333999992, 41.73359183400002], [-87.5281139129999, 41.73344041300003], [-87.52818016399992, 41.733298339000044], [-87.5282700789999, 41.733169927000056], [-87.52835871399989, 41.73307834500007], [-87.52860971499989, 41.732853345000066], [-87.5286319269999, 41.732834079000035], [-87.52869355599992, 41.732790927000046], [-87.5292354939999, 41.73288433100003], [-87.53003690299992, 41.73264977200005], [-87.5324606759999, 41.73153561900004], [-87.53613542899988, 41.73003053400003], [-87.53828554999988, 41.72913139200004], [-87.53918469099993, 41.72881864700007], [-87.54035748499989, 41.728290890000025], [-87.54147163799989, 41.72762630700003], [-87.54192120899992, 41.72721583000003], [-87.54217531399988, 41.72682489900005], [-87.54225350099992, 41.726492607000054], [-87.54217531399988, 41.72602349000003], [-87.54200226399989, 41.725568504000066], [-87.54203404499991, 41.72540420400007], [-87.54205821399994, 41.725308041000055], [-87.54212998699988, 41.72507715300008], [-87.5426835249999, 41.72457704500004], [-87.54287898999989, 41.72422520700006], [-87.54309400199989, 41.72367790300006], [-87.54321128199987, 41.72315014600007], [-87.54321128199987, 41.722896041000055], [-87.543682, 41.722908], [-87.5442929149999, 41.72288402200006], [-87.54550491599989, 41.72285702200008], [-87.54562505999989, 41.722854964000064], [-87.54572491499991, 41.72285302300007], [-87.54623291499989, 41.722844022000054], [-87.54694391499991, 41.72281602300005], [-87.54742691599994, 41.722821022000055], [-87.5475559149999, 41.72281902200006], [-87.5481799149999, 41.72281302300006], [-87.54851491499993, 41.722839023000056], [-87.54872192299987, 41.722839994000076], [-87.54914794699992, 41.72329794700004], [-87.5495673009999, 41.723712235000036], [-87.55030108399987, 41.724362181000046], [-87.55212033999992, 41.72576644900004], [-87.55477807399988, 41.72821], [-87.55846241199991, 41.73030463300005], [-87.5617666569999, 41.73268453300004], [-87.56507247199993, 41.73483670600006], [-87.56848152999991, 41.73719343700008], [-87.5685358829999, 41.73719238200003], [-87.57213289299992, 41.73980363700007], [-87.57715348699989, 41.743515376000055], [-87.58187945299991, 41.74706243800005], [-87.58580631599992, 41.74989733600005], [-87.58579178399991, 41.750375353000024], [-87.5858102949999, 41.751468898000034], [-87.5858099959999, 41.75146889900007], [-87.58575591599991, 41.75146902200002], [-87.58568091599992, 41.75146902200002], [-87.5855769149999, 41.75146902200002], [-87.58553491499991, 41.751468022000026], [-87.58549891599992, 41.751468022000026], [-87.58544678099989, 41.751469078000056], [-87.5851499159999, 41.751474023000064], [-87.58435191599993, 41.75148702200004], [-87.58329291499989, 41.75151202300008], [-87.5828109159999, 41.75151202300008], [-87.58206891599988, 41.75152202200006], [-87.58162091499989, 41.75151902200007], [-87.58089391499992, 41.75153902200008], [-87.58001491499992, 41.75154702200006], [-87.5796649149999, 41.75154902300004], [-87.57836391499987, 41.75156402200007], [-87.57717191499994, 41.75157802200004], [-87.57593191599992, 41.75159002300005], [-87.57471891599992, 41.75159502200005], [-87.57345591599993, 41.75162002200005], [-87.57236791499989, 41.75164202200005], [-87.57107691599991, 41.75165402200007], [-87.5698669159999, 41.751666023000034], [-87.56923891599988, 41.751676022000076], [-87.56861591599993, 41.75168502200005], [-87.56739691499988, 41.75169102300002], [-87.56619291599989, 41.75172902300004], [-87.56496891499994, 41.75173802200003], [-87.56377291599993, 41.75175802200003], [-87.56247691499992, 41.75177302200006], [-87.5620249159999, 41.751786023000065], [-87.5613289149999, 41.75179702200005], [-87.56013691499992, 41.751820022000054], [-87.55988791599991, 41.75182102200005], [-87.55890791499989, 41.751824023000026], [-87.55842091499993, 41.75183302200008], [-87.5576859159999, 41.751839022000055], [-87.55689291599991, 41.75185202200002], [-87.55648091599991, 41.75184802200003], [-87.55547491499992, 41.751860022000066], [-87.55524291499992, 41.75186102300005], [-87.55466691599989, 41.751868022000046], [-87.55409491499994, 41.75188202300006], [-87.55401291499992, 41.75188402300005], [-87.55282891599988, 41.75190202300006], [-87.55245691599991, 41.75190202300006], [-87.55235390399989, 41.75190188800006], [-87.55226800199993, 41.75192923400004], [-87.55222, 41.75191], [-87.55087597499991, 41.75193984000003], [-87.54985417199991, 41.75195], [-87.5487393259999, 41.75195], [-87.54800120099989, 41.75195988900003]]]}, "properties": {"city": "Chicago", "name": "South Chicago", "regionid": "140437", "geo_point_2d": [41.739595368610686, -87.55560428763064], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.70460446899993, 41.97575530900007], [-87.70456257099993, 41.97560447400007], [-87.7045357949999, 41.97543043600005], [-87.70448224499992, 41.975243010000035], [-87.70438853199992, 41.97496187100005], [-87.70434836899993, 41.974707507000055], [-87.7042948189999, 41.974412980000054], [-87.70424126899988, 41.97421216700007], [-87.70424126899988, 41.97401135300002], [-87.7042126239999, 41.973908231000046], [-87.704329, 41.973907], [-87.7050409159999, 41.97397802300008], [-87.70533991599989, 41.97402502200002], [-87.70559291599992, 41.97406802300008], [-87.70631191599992, 41.974255022000065], [-87.70694291599993, 41.97443602200007], [-87.70739591599992, 41.974566022000026], [-87.70808891599992, 41.97467202200004], [-87.70826591599987, 41.97468702300005], [-87.7085019159999, 41.974686021000075], [-87.70865691599992, 41.97466102300005], [-87.70876591499987, 41.974634021000035], [-87.70898991599994, 41.97454002300008], [-87.7093709149999, 41.97441402300006], [-87.7099539149999, 41.97430202200006], [-87.7100149149999, 41.97429402200003], [-87.7103709159999, 41.974194022000056], [-87.71074291599987, 41.97404202200004], [-87.71106091499988, 41.97385902200005], [-87.71109991499989, 41.97383002200006], [-87.71122691399994, 41.973710024000056], [-87.7118839149999, 41.97281802300005], [-87.7120649159999, 41.972614023000055], [-87.71224091599991, 41.97244302300003], [-87.71238191699989, 41.97233902300007], [-87.71264091499994, 41.972209023000055], [-87.71301391599988, 41.97204602300007], [-87.71338991499992, 41.97194002200006], [-87.7135059149999, 41.97191002300007], [-87.71382591599989, 41.97185002200006], [-87.71465791499989, 41.971779022000064], [-87.71484891499992, 41.97175402300006], [-87.71528191499992, 41.97172002300004], [-87.71568593599991, 41.97175091100006], [-87.7157849149999, 41.97178002300006], [-87.7159699519999, 41.97183398100008], [-87.71622388099989, 41.97201510000007], [-87.7165159339999, 41.972083973000046], [-87.71680691599992, 41.97225602300005], [-87.71724091699988, 41.97256702100008], [-87.71745691499989, 41.97267902300006], [-87.7176519159999, 41.97280102300004], [-87.7180928889999, 41.973123068000056], [-87.7181899149999, 41.97316202300003], [-87.71830791599987, 41.97320302200006], [-87.7186559159999, 41.973316023000045], [-87.7188839149999, 41.97340502300005], [-87.71919591599992, 41.97358502200006], [-87.71941291599987, 41.97368502300008], [-87.7197329129999, 41.973801020000046], [-87.71995291599988, 41.97381902300002], [-87.7201029159999, 41.97385802200005], [-87.72038691499992, 41.97389702200007], [-87.72080791599988, 41.97393402200002], [-87.72139191599989, 41.97396102300006], [-87.72165191499991, 41.97398902300005], [-87.72182991499992, 41.97402102300003], [-87.72228191499991, 41.974172023000044], [-87.7224669159999, 41.97425102300007], [-87.72265591499989, 41.97436102200003], [-87.72303991499989, 41.974618022000065], [-87.7233469159999, 41.97479802300005], [-87.7236379159999, 41.97503702300007], [-87.72402391599988, 41.975444022000026], [-87.7240246499999, 41.975444701000065], [-87.72412991799987, 41.97554202200006], [-87.72426491699989, 41.97562502200008], [-87.72451091599991, 41.97572002200007], [-87.72489991499991, 41.975809022000064], [-87.72522891499993, 41.97588102200006], [-87.72551991599993, 41.97593102200005], [-87.72578891499987, 41.97593702200003], [-87.72596791599993, 41.97593102200005], [-87.72624491499992, 41.975887022000045], [-87.72634891599994, 41.97585302300006], [-87.7265739139999, 41.97575902300007], [-87.72685691599989, 41.975620023000026], [-87.72703691499991, 41.97552602200004], [-87.72722386599989, 41.97539785200006], [-87.72813804799993, 41.97538215000007], [-87.72813906899994, 41.97538214900003], [-87.72814005899991, 41.97538214700005], [-87.72816473599991, 41.97627461000006], [-87.72819515599988, 41.97737480800003], [-87.7282539739999, 41.97957526400005], [-87.7282991779999, 41.98122773800008], [-87.72834, 41.98272], [-87.72676191899987, 41.98271194000005], [-87.7213505899999, 41.98278], [-87.7210307689999, 41.98278379300006], [-87.7178976679999, 41.98282094800003], [-87.71676320099992, 41.98284386100004], [-87.71547550599989, 41.98287450800005], [-87.7141869689999, 41.9829], [-87.7130533689999, 41.98289510800002], [-87.71197108499987, 41.98289374100005], [-87.71068309899991, 41.98290873000007], [-87.70929200199991, 41.982931633000064], [-87.70784750699993, 41.982955961000066], [-87.7070014749999, 41.98296339800003], [-87.7068224309999, 41.982965544000066], [-87.70663764399987, 41.982298259000046], [-87.7064903809999, 41.98182969400005], [-87.70628956699993, 41.981200479000044], [-87.70608875399994, 41.98055787500003], [-87.7058076149999, 41.97966090800008], [-87.70559341399989, 41.97896475400005], [-87.70533904999988, 41.97813472500007], [-87.7052587239999, 41.97780003600006], [-87.70509807299992, 41.97739840900004], [-87.70495080999987, 41.976876293000075], [-87.7048035469999, 41.97640772800007], [-87.7046295079999, 41.97584545000006], [-87.70460446899993, 41.97575530900007]]]}, "properties": {"city": "Chicago", "name": "North Park", "regionid": "276477", "geo_point_2d": [41.97818284952796, -87.71638922338744], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.64053244199992, 41.95343970300006], [-87.64060446499995, 41.953433240000024], [-87.64105232299991, 41.95339304800005], [-87.64573106199997, 41.95297316100004], [-87.64689711599998, 41.95290410400002], [-87.64859208099993, 41.952809076000044], [-87.64962, 41.95275], [-87.64959366699995, 41.95182834300003], [-87.64956, 41.95061250000003], [-87.64954050999994, 41.94947733300005], [-87.64949396999997, 41.94818322700006], [-87.64944153899992, 41.94672770400006], [-87.64942021499995, 41.94591989700006], [-87.64939911199995, 41.94511247500002], [-87.64936797099992, 41.94397269000005], [-87.64934662899996, 41.943001634000034], [-87.64932317399995, 41.94219520900002], [-87.64928295599996, 41.941299679000046], [-87.64926032399994, 41.940571380000044], [-87.64923225899994, 41.93943734200007], [-87.64920022399996, 41.938628550000026], [-87.64913, 41.93684922200002], [-87.64910481599996, 41.93587631700006], [-87.64906484199996, 41.93499020900003], [-87.64903286099994, 41.93401441200005], [-87.64900349299992, 41.93312480000002], [-87.64899098699996, 41.93271706900002], [-87.65021391499992, 41.93269702300005], [-87.65141791599994, 41.932675022000055], [-87.65262091599996, 41.93265602200005], [-87.65313191499995, 41.93264602200002], [-87.65384591499996, 41.932634022000045], [-87.65516591599996, 41.93261102200005], [-87.65582791499996, 41.93260002300007], [-87.65626891599996, 41.93259702200003], [-87.65743991599996, 41.93258802200005], [-87.65869791599992, 41.932562023000045], [-87.65989391499991, 41.932532022000025], [-87.66111591499997, 41.932524022000045], [-87.66134591599993, 41.93251902300004], [-87.66233891599992, 41.93250202300003], [-87.66355111999994, 41.93248699300005], [-87.66474691499994, 41.93245802200004], [-87.66596791599991, 41.93243602200004], [-87.66716891599995, 41.932420022000024], [-87.66789991599994, 41.93240602200005], [-87.66825791599994, 41.93239902200001], [-87.66837760199995, 41.93239519200005], [-87.66838291599996, 41.93239502200002], [-87.66958591599993, 41.93237602200002], [-87.67081791499997, 41.93237202200003], [-87.67206391499994, 41.93234602300003], [-87.67336370799993, 41.93233270800004], [-87.67343591699994, 41.93233196800002], [-87.67388691599994, 41.93232702200003], [-87.67472891599992, 41.932322022000044], [-87.67513991599996, 41.93232002200005], [-87.67565791499997, 41.932317022000056], [-87.67625791599991, 41.932295022000055], [-87.67810491599992, 41.93227802200004], [-87.67820897299997, 41.93227745800003], [-87.67821704799995, 41.932284056000064], [-87.67824661399997, 41.93228383000007], [-87.67856426299991, 41.93227914100002], [-87.67875110099997, 41.932275789000066], [-87.67886000199995, 41.932274985000035], [-87.67921695299995, 41.932272352000034], [-87.67953804999995, 41.93226830800005], [-87.68003513199992, 41.93226208100002], [-87.68041892899997, 41.93225762300004], [-87.68069230999996, 41.93225478600004], [-87.68098774499997, 41.93225171800002], [-87.68128464599991, 41.932248904000055], [-87.68157916999994, 41.93224511500004], [-87.68207624699991, 41.93223926400002], [-87.68233481799994, 41.93223639500007], [-87.68252219899993, 41.93223394400006], [-87.68271179099996, 41.93223082000002], [-87.68281071699992, 41.93222919600003], [-87.68283454399993, 41.93226890800002], [-87.68289719999993, 41.93238852400002], [-87.68312907, 41.93283118600004], [-87.68332988399997, 41.933313139000056], [-87.68367796099992, 41.93343362700005], [-87.68401068199992, 41.933501878000065], [-87.68420007599995, 41.93354072800002], [-87.68466864099994, 41.93360766500006], [-87.68511043099993, 41.933674603000036], [-87.68565932099993, 41.933902192000055], [-87.68600739799994, 41.93406284300004], [-87.68628853699991, 41.93433059400006], [-87.68652951299995, 41.93450463300007], [-87.68694452799996, 41.93502674800003], [-87.68710517899994, 41.935281112000034], [-87.68741309299996, 41.935682739000065], [-87.6878147199999, 41.93608436600003], [-87.68816279699993, 41.936298567000044], [-87.6885644239999, 41.93636550500002], [-87.68903298899993, 41.93648599300006], [-87.68943461599996, 41.93652615600006], [-87.68986301799997, 41.93660648100006], [-87.69015754499996, 41.936726969000055], [-87.69042529599992, 41.93695455800002], [-87.69057255899996, 41.937222309000035], [-87.69063949699992, 41.93758377400007], [-87.69074659799996, 41.93789168800003], [-87.69089386099994, 41.93817282700006], [-87.69116161299993, 41.93846735400006], [-87.69142936399996, 41.938788655000046], [-87.69173727799992, 41.93898946900003], [-87.69212551799993, 41.93916350700003], [-87.69250036999992, 41.939484809000064], [-87.69304925999992, 41.93981949800007], [-87.69330362399994, 41.94010063700005], [-87.69370525099995, 41.94040855100007], [-87.69394622699996, 41.940703078000055], [-87.69422736599994, 41.941104705000036], [-87.69480303199995, 41.94176069600007], [-87.6953117589999, 41.94238991200005], [-87.69545902299993, 41.942711213000045], [-87.69553934799994, 41.94327349100006], [-87.69563306099995, 41.943755444000026], [-87.69574016199992, 41.94438466000001], [-87.69595436299993, 41.945174526000066], [-87.69603468799991, 41.94553599100003], [-87.69604807599993, 41.945924230000045], [-87.69616856399993, 41.946553446000046], [-87.69619224899995, 41.94669555700004], [-87.68830469899993, 41.946722724000026], [-87.68813813699994, 41.93948916300007], [-87.67343996899993, 41.939620538000035], [-87.67343287499992, 41.939620847000064], [-87.67239409399996, 41.93962941700004], [-87.67167350999993, 41.93964434900004], [-87.67125712299996, 41.93965297800002], [-87.67053275499993, 41.93966798900003], [-87.66865, 41.93968], [-87.66901277299995, 41.940208083000066], [-87.66956022599997, 41.940978206000054], [-87.67058174, 41.94245766800002], [-87.67151095899997, 41.943784025000056], [-87.67209746999995, 41.944601982000044], [-87.67278874799995, 41.94558902000005], [-87.67299893999994, 41.94588914200005], [-87.67329881899995, 41.94640485300004], [-87.67361089999996, 41.94694224100003], [-87.67361191599996, 41.94698402300002], [-87.67361585299992, 41.947155486000035], [-87.67364991599997, 41.94863902200006], [-87.67366791499994, 41.94874802200007], [-87.67370491499996, 41.950577022000054], [-87.67373791599991, 41.95240202300003], [-87.67379191599996, 41.95415702300004], [-87.67379143499993, 41.95416410700005], [-87.67296195499992, 41.95417427700004], [-87.67182835299995, 41.95418502200004], [-87.67101869399993, 41.954195131000056], [-87.66996715599997, 41.95421496200002], [-87.66911, 41.95428], [-87.66897234799994, 41.95438547600002], [-87.66897, 41.9543], [-87.66826755299994, 41.954304811000036], [-87.66703076499992, 41.95431850700003], [-87.66533038499992, 41.954346008000066], [-87.66383618999996, 41.95436536100003], [-87.66337164499997, 41.95437], [-87.66221, 41.95437], [-87.66220777599992, 41.95437394200007], [-87.65921891599996, 41.95442602200005], [-87.65697591599995, 41.95445202200005], [-87.65685791599996, 41.95445202200005], [-87.65682919199995, 41.95445161500003], [-87.65671691599991, 41.95445002300004], [-87.65609891599993, 41.954457022000035], [-87.65451491599995, 41.95447502200004], [-87.65210391599997, 41.95451402200006], [-87.65048391499992, 41.954530022000014], [-87.64968391599994, 41.95454602300003], [-87.64791891599997, 41.95458102200006], [-87.64713791599993, 41.95459402200003], [-87.64672291499993, 41.95460502300004], [-87.64660591499992, 41.95460102200007], [-87.64589791599995, 41.954620022000015], [-87.64523291599993, 41.954641022000025], [-87.64509191599994, 41.95464202200002], [-87.64491091599993, 41.95464802300005], [-87.64480091699993, 41.95464202200002], [-87.64466091599994, 41.95464402300006], [-87.64453991499991, 41.95464602300005], [-87.64427091599997, 41.95465802200003], [-87.64413985099996, 41.95465884700007], [-87.64404197999988, 41.95474419700001], [-87.64186572799996, 41.954735313000064], [-87.641054, 41.954732], [-87.64101137999992, 41.95473182600005], [-87.64100024699995, 41.95473161000006], [-87.64099902799992, 41.95472094600006], [-87.64097802799994, 41.95462394600002], [-87.64063702799993, 41.95375694600004], [-87.64053446999995, 41.95349575600004], [-87.64051280999993, 41.953440593000025], [-87.64053244199992, 41.95343970300006]], [[-87.65912484099992, 41.945755677000065], [-87.65912, 41.94532], [-87.65814456799993, 41.94533146100007], [-87.65620190899993, 41.94535869300005], [-87.65521, 41.9456], [-87.65425, 41.94446], [-87.65427, 41.94539], [-87.65247737799994, 41.94541914800003], [-87.65181, 41.94543], [-87.65188, 41.94728], [-87.65195, 41.94908], [-87.65195398599995, 41.94947862200006], [-87.65198273299993, 41.95069379600004], [-87.652, 41.95116], [-87.65255972499992, 41.95114600700002], [-87.65353005699994, 41.95113], [-87.65482557599994, 41.951116648000024], [-87.65603991199997, 41.95110523400005], [-87.65701062899993, 41.95108080700003], [-87.65806365999997, 41.951066938000054], [-87.65925, 41.95106], [-87.65920495199997, 41.94956183500004], [-87.65917749999993, 41.94858915500003], [-87.65915426299992, 41.94753646500003], [-87.65914103499993, 41.94672762000004], [-87.65912484099992, 41.945755677000065]]]}, "properties": {"city": "Chicago", "name": "Lake View", "regionid": "269589", "geo_point_2d": [41.94220774853042, -87.66677048954814], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.67654112303022, 41.895918274422364], [-87.67705, 41.89591], [-87.67706580045628, 41.89639454732419], [-87.67710009806173, 41.89743294185082], [-87.67710999999986, 41.898163581344264], [-87.67711759153214, 41.89926250563334], [-87.67715274521582, 41.899872362481645], [-87.67717936050457, 41.900969826621925], [-87.67723355134109, 41.901888129505174], [-87.67725, 41.90274072472563], [-87.6772520943022, 41.90318166701809], [-87.67666232427207, 41.903189713648395], [-87.67550304312012, 41.90321309650928], [-87.6745271067079, 41.90323565786589], [-87.6727578693891, 41.903261602902504], [-87.67190353714435, 41.903277747363546], [-87.67086525791251, 41.903289999999885], [-87.67019394354992, 41.9033], [-87.66964592469971, 41.903304493037766], [-87.66818274297823, 41.903341396512786], [-87.66739712859197, 41.90334045257915], [-87.66738543400258, 41.90292449423574], [-87.66750954570449, 41.901761824516385], [-87.6675009933979, 41.90084635805616], [-87.6674714890962, 41.900054672889404], [-87.66743959037372, 41.8990773015872], [-87.66741591212396, 41.89828611231301], [-87.66740452458174, 41.89767645868977], [-87.66738347347751, 41.89663935953593], [-87.66736999999989, 41.89606], [-87.66805830020809, 41.89604792455779], [-87.6690344625452, 41.89603133119768], [-87.67043795003292, 41.89601421008044], [-87.67172026902935, 41.895990993274275], [-87.67385631751364, 41.89595914363704], [-87.67550342987234, 41.895930000000114], [-87.67654112303022, 41.895918274422364]]]}, "properties": {"city": "Chicago", "name": "East Ukrainian Village", "regionid": "403311", "geo_point_2d": [41.8996330978912, -87.67229289169595], "county": "Cook", "state": "IL"}},{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[-87.77542676699993, 41.918482343000036], [-87.77538792499989, 41.917201002000056], [-87.77536281299989, 41.91637506000005], [-87.77536217299988, 41.916375071000054], [-87.77534206299991, 41.91568045400003], [-87.77527081899989, 41.913543420000046], [-87.7752248439999, 41.91238520700005], [-87.7751949599999, 41.911468763000066], [-87.77516966499988, 41.910799167000036], [-87.77513952799988, 41.91018437500007], [-87.7751370209999, 41.909821496000056], [-87.7754, 41.90922], [-87.77555142899989, 41.90911272500004], [-87.77555445699993, 41.90916723300005], [-87.775558, 41.909231], [-87.77635799999989, 41.909231], [-87.77684, 41.909244], [-87.777658, 41.909331], [-87.778058, 41.909231], [-87.77885799999987, 41.909131], [-87.77886441899993, 41.909131], [-87.77945799999989, 41.909131], [-87.77995799999988, 41.909131], [-87.78065799999987, 41.909131], [-87.78155799999989, 41.909131], [-87.7816219459999, 41.909131], [-87.782058, 41.909131], [-87.78212264199992, 41.909131], [-87.78315799999989, 41.909131], [-87.7846579999999, 41.909131], [-87.785058, 41.909131], [-87.785958, 41.909131], [-87.78655799999987, 41.909131], [-87.78712200699991, 41.90903699900007], [-87.78715799999989, 41.909031], [-87.78726591599988, 41.909031], [-87.7875152799999, 41.909031], [-87.78813391599991, 41.909031], [-87.78815799999988, 41.909031], [-87.78821607199988, 41.909019385000065], [-87.788658, 41.908931], [-87.7894585039999, 41.909003773000045], [-87.789758, 41.909031], [-87.7898833869999, 41.90899965300007], [-87.79015799999989, 41.908931], [-87.79105799999988, 41.908931], [-87.791458, 41.908931], [-87.79181696699992, 41.90897581500007], [-87.792259, 41.909031], [-87.79251396499988, 41.90896725900006], [-87.79265899999987, 41.908931], [-87.793459, 41.908931], [-87.79350234099988, 41.90894183500006], [-87.793859, 41.909031], [-87.79465899999988, 41.908931], [-87.79468313099991, 41.90892496700007], [-87.795059, 41.908831], [-87.79543203099989, 41.90891389600005], [-87.7959589999999, 41.909031], [-87.79643234099987, 41.90889576000005], [-87.79665899999989, 41.908831], [-87.79695777199991, 41.908890754000076], [-87.79715899999988, 41.908931], [-87.798159, 41.908931], [-87.79855899999987, 41.908931], [-87.79965899999989, 41.908931], [-87.80014940399988, 41.90886094200004], [-87.80035899999989, 41.908831], [-87.80048205099989, 41.90885834500004], [-87.80125899999987, 41.909031], [-87.802059, 41.908931], [-87.802659, 41.908831], [-87.80280803299988, 41.90881745200004], [-87.80375899999987, 41.908731], [-87.804259, 41.908731], [-87.8048792669999, 41.908769767000074], [-87.80558819899993, 41.90881407500006], [-87.805859, 41.908831], [-87.80575899999988, 41.910731], [-87.805859, 41.912531], [-87.805959, 41.914331], [-87.806059, 41.916131], [-87.806059, 41.917931], [-87.80599676299992, 41.919175726000034], [-87.805959, 41.919931], [-87.8060337409999, 41.92034207300008], [-87.8061589999999, 41.921031], [-87.80625899999987, 41.922031], [-87.8061589999999, 41.922231], [-87.8062571819999, 41.92321281500004], [-87.80609999999989, 41.9232], [-87.80609546399988, 41.923193649000034], [-87.80521391599987, 41.92297802200004], [-87.80425891499992, 41.92271002300003], [-87.80344791599991, 41.92246402200003], [-87.8028459159999, 41.92223302200004], [-87.80240891499993, 41.92206802300005], [-87.80056391599992, 41.92138802300007], [-87.79986391599992, 41.92113802200003], [-87.79870291499991, 41.920728022000034], [-87.79716091499992, 41.92018302200005], [-87.79693691499993, 41.92010102200003], [-87.79562091499989, 41.91962202200005], [-87.79500291599992, 41.91938802200007], [-87.79408491599989, 41.91904002300003], [-87.79254591599991, 41.91850302200003], [-87.79155591599992, 41.91811702200004], [-87.79110791599992, 41.91794202200003], [-87.78960491499991, 41.91741502200005], [-87.78911120899991, 41.91723107300003], [-87.78962994899989, 41.91768100200004], [-87.78979391499988, 41.917864022000046], [-87.78992491499989, 41.91806902300004], [-87.79000691399993, 41.91821802200007], [-87.79005591499991, 41.918373022000075], [-87.79009291499993, 41.91856302300003], [-87.79013091599988, 41.91885102300005], [-87.79015691499988, 41.91915302200004], [-87.79017491499988, 41.92012502200002], [-87.79021591499989, 41.92106702300003], [-87.79022691599992, 41.921369023000054], [-87.79028691599987, 41.92294002300007], [-87.7903249159999, 41.92347902200003], [-87.79033091499988, 41.92356202200005], [-87.78888538999989, 41.92327439900003], [-87.78803368999988, 41.92318704500007], [-87.78687161099991, 41.92303990700008], [-87.7860607299999, 41.922771207000025], [-87.7853, 41.92253], [-87.78451657499993, 41.92230255400005], [-87.7831635119999, 41.921915995000056], [-87.7820408409999, 41.92165050100003], [-87.7809516059999, 41.92147117500007], [-87.77953632699989, 41.92123133500006], [-87.7783432709999, 41.92101353900005], [-87.77709777799991, 41.92078064200006], [-87.7761802869999, 41.920613128000056], [-87.77547999999987, 41.92049], [-87.77545766199992, 41.91964754000008], [-87.77542676699993, 41.918482343000036]]]}, "properties": {"city": "Chicago", "name": "Galewood", "regionid": "403325", "geo_point_2d": [41.915212462800014, -87.79075878795798], "county": "Cook", "state": "IL"}}]}

All the relevant data is in the *features* key, which is basically a list of the neighborhoods. 
So, let's define a new variable that includes this data.

In [18]:
neighborhoods_data = new_data['features']


#### Tranform the data into a *pandas* dataframe

The next task is essentially transforming this data of nested Python dictionaries into a *pandas* dataframe. 
So let's start by creating an empty dataframe.

In [19]:
column_names = ['Neighborhood', 'Latitude', 'Longitude'] 

neighborhoods = pd.DataFrame(columns=column_names)


Then let's loop through the data and fill the dataframe one row at a time.

In [20]:
for data in neighborhoods_data:
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['properties']['geo_point_2d']
    neighborhood_lat = neighborhood_latlon[0]
    neighborhood_lon = neighborhood_latlon[1]
    
    neighborhoods = neighborhoods.append({'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

neighborhoods.head()


,Neighborhood,Latitude,Longitude
0,Heart of Italy,41.848657,-87.684331
1,The Bush,41.745141,-87.539765
2,Rogers Park,42.009614,-87.670109
3,Clearing (E),41.776564,-87.744548
4,South Loop,41.860771,-87.625568


#### Use geopy library to get the latitude and longitude values of Chicago.

In [21]:
address = 'Chicago, IL'

geolocator = Nominatim(user_agent="chi_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Chicago are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Chicago are 41.8755616, -87.6244212.


#### Create a map of Chicago with neighborhoods superimposed on top.

In [22]:
# create map of New York using latitude and longitude values
map_chi = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_chi)  
    
map_chi

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [23]:
CLIENT_ID = 'NQ1XBA425AZWHFCTH2VO3ZVSNIVSMFJGVHQXOMJGLLBOHMJT' 
CLIENT_SECRET = 'D2RF04HZPRPUQKBI1LLHA1540D2T5D4302YXOHFKWD4PEAL1' 
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius


## 2. Explore Neighborhoods in Chicago

In [24]:
#function to explore neighborhoods in chicago
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


#### Now the code to run the above function on each neighborhood and create a new dataframe called *chicago_venues*.

In [25]:
#run the above function on each neighborhood and create a new dataframe called chicago_venues
chicago_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                 latitudes=neighborhoods['Latitude'],
                                 longitudes=neighborhoods['Longitude'])


Heart of Italy
The Bush
Rogers Park
Clearing (E)
South Loop
Pulaski Park
Kilbourn Park
Gold Coast
Edgewater
Lake View East
Jefferson Park
Marquette Park
Grand Crossing
Andersonville
Noble Square
West Elsdon
Marynook
Pilsen
Oriole Park
Beverly Woods
Park West
Belmont Gardens
Brighton Park
Stateway Gardens
Ravenswood
Parkway Gardens
South Shore
Washington Park
Hanson Park
Belmont Terrace
Magnolia Glen
Schorsch Village
Roscoe Village
Hermosa
Near North
Dearborn Park
Rosemoor
Fulton River District
Goose Island
Ukrainian Village
Ashburn
Cottage Grove Heights
Chicago Midway Airport
Altgeld Gardens
Lincoln Square
Scottsdale
Trumbull Park
Heart of Chicago
West Rogers Park
Mount Greenwood Heights
North Kenwood
Arcadia Terrace
West Pullman
Wicker Park
Tri-Taylor
Sheffield Neighbors
Norwood Park East
Eden Green
Park Manor
Belmont Heights
South Deering
Stony Island Park
Canaryville
Burnside
Edison Park
London Towne
Douglas Park
Kenwood
Ida B. Wells - Darrow Homes
The Island
McKinley Park
West Wood

This concludes our gathering of data part. We have a list of neighborhoods along with nearby venues.

## Methodology <a name="methodology"></a>


We will focus our efforts on finding neighborhoods that have high density of outdoor public places and low restaurant and fast food density.
We already collected the required data: location and category of Chicago restaurants.
Now we will explore the density of the most common venues per neighborhood in Chicago.

In the final step, we will focus on creating clusters (using k-means clustering) of the neighborhoods based on the most common venues, and 
analyzing clusters to come up with a list of neighborhoods that potentially could work for our client's food truck business.


## Analysis <a name="analysis"></a>

In [30]:
#analyze each neighborhood
chicago_onehot = pd.get_dummies(chicago_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
chicago_onehot['Neighborhood'] = chicago_venues['Neighborhood'] 
# move neighborhood column to the first column
my_col = chicago_onehot['Neighborhood']
chicago_onehot.drop(labels=['Neighborhood'], axis=1,inplace = True)
chicago_onehot.insert(0, 'Neighborhood', my_col)

chicago_onehot.head()


,Neighborhood,ATM,Accessories Store,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat Rental,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Check Cashing Service,Child Care Service,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Cafeteria,College Quad,College Residence Hall,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Cuban Restaurant,Cupcake Shop,Currency Exchange,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Floating Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Forest,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Library,Light Rail Station,Lighthouse,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,National Park,Nature Preserve,New American Restaurant,Nightclub,Nightlife Spot,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Other Great Outdoors,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pastry Shop,Pawn Shop,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Recordin

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [32]:
chicago_grouped = chicago_onehot.groupby('Neighborhood').mean().reset_index()
chicago_grouped.head()


,Neighborhood,ATM,Accessories Store,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat Rental,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Check Cashing Service,Child Care Service,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Cafeteria,College Quad,College Residence Hall,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Cuban Restaurant,Cupcake Shop,Currency Exchange,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Floating Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Forest,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Library,Light Rail Station,Lighthouse,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,National Park,Nature Preserve,New American Restaurant,Nightclub,Nightlife Spot,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Other Great Outdoors,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pastry Shop,Pawn Shop,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Recordin

#### Let's print each neighborhood along with the top 5 most common venues

In [33]:
num_top_venues = 5

for hood in chicago_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = chicago_grouped[chicago_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----Albany Park----
                  venue  freq
0           Bus Station  0.09
1  Fast Food Restaurant  0.05
2         Hot Dog Joint  0.05
3          Cocktail Bar  0.05
4         Train Station  0.05


----Altgeld Gardens----
                   venue  freq
0                   Park   1.0
1         Nightlife Spot   0.0
2  Outdoors & Recreation   0.0
3   Outdoor Supply Store   0.0
4      Outdoor Sculpture   0.0


----Andersonville----
                       venue  freq
0               Burger Joint  0.04
1                Pizza Place  0.04
2  Middle Eastern Restaurant  0.04
3              Grocery Store  0.04
4                Coffee Shop  0.04


----Arcadia Terrace----
                         venue  freq
0    Middle Eastern Restaurant  0.08
1                          Gym  0.08
2                       Bakery  0.08
3    Indian Chinese Restaurant  0.04
4  Eastern European Restaurant  0.04


----Archer Heights----
                venue  freq
0  Mexican Restaurant  0.29
1                Bank  0.

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = chicago_grouped['Neighborhood']

for ind in np.arange(chicago_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(chicago_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Albany Park,Bus Station,Sandwich Place,Coffee Shop,Thai Restaurant,Thrift / Vintage Store,Sushi Restaurant,Chinese Restaurant,Paper / Office Supplies Store,Train Station,Discount Store
1,Altgeld Gardens,Park,Field,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Falafel Restaurant,Farm,Farmers Market
2,Andersonville,Sushi Restaurant,Breakfast Spot,Pizza Place,Coffee Shop,Middle Eastern Restaurant,Burger Joint,Café,Bakery,Grocery Store,Southern / Soul Food Restaurant
3,Arcadia Terrace,Gym,Middle Eastern Restaurant,Bakery,Vietnamese Restaurant,Burger Joint,Greek Restaurant,Sushi Restaurant,Eastern European Restaurant,Turkish Restaurant,Nature Preserve
4,Archer Heights,Mexican Restaurant,Pharmacy,Bank,Cosmetics Shop,Market,Nightclub,Chinese Restaurant,Sandwich Place,Dessert Shop,Seafood Restaurant


##  Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 10 clusters.

In [36]:
# set number of clusters
kclusters = 10

chicago_grouped_clustering = chicago_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(chicago_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


array([2, 7, 2, 2, 6, 2, 1, 2, 2, 2], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [37]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

chicago_merged = neighborhoods

# merge chicago_grouped with chicago_data to add latitude/longitude for each neighborhood
chicago_merged = chicago_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

chicago_merged.head() 


,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Heart of Italy,41.848657,-87.684331,2.0,Italian Restaurant,Mexican Restaurant,Pizza Place,Thrift / Vintage Store,Sandwich Place,Food Truck,Fast Food Restaurant,Shoe Store,Gas Station,Bank
1,The Bush,41.745141,-87.539765,2.0,Concert Hall,Zoo Exhibit,Filipino Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Falafel Restaurant,Farm,Farmers Market
2,Rogers Park,42.009614,-87.670109,2.0,Mexican Restaurant,Pizza Place,American Restaurant,Donut Shop,Theater,Chinese Restaurant,Bakery,Train Station,Bar,Grocery Store
3,Clearing (E),41.776564,-87.744548,1.0,Hotel,American Restaurant,Pizza Place,Fast Food Restaurant,Chinese Restaurant,Coffee Shop,Mexican Restaurant,Martial Arts Dojo,Taco Place,Taxi Stand
4,South Loop,41.860771,-87.625568,2.0,Park,Grocery Store,Coffee Shop,Bar,Dog Run,Steakhouse,Donut Shop,Bank,Breakfast Spot,Salon / Barbershop


In [40]:
chicago_merged.dropna(inplace=True)
chicago_merged['Cluster Labels'] = chicago_merged['Cluster Labels'].astype(int)
chicago_merged.head() 

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Heart of Italy,41.848657,-87.684331,2,Italian Restaurant,Mexican Restaurant,Pizza Place,Thrift / Vintage Store,Sandwich Place,Food Truck,Fast Food Restaurant,Shoe Store,Gas Station,Bank
1,The Bush,41.745141,-87.539765,2,Concert Hall,Zoo Exhibit,Filipino Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Falafel Restaurant,Farm,Farmers Market
2,Rogers Park,42.009614,-87.670109,2,Mexican Restaurant,Pizza Place,American Restaurant,Donut Shop,Theater,Chinese Restaurant,Bakery,Train Station,Bar,Grocery Store
3,Clearing (E),41.776564,-87.744548,1,Hotel,American Restaurant,Pizza Place,Fast Food Restaurant,Chinese Restaurant,Coffee Shop,Mexican Restaurant,Martial Arts Dojo,Taco Place,Taxi Stand
4,South Loop,41.860771,-87.625568,2,Park,Grocery Store,Coffee Shop,Bar,Dog Run,Steakhouse,Donut Shop,Bank,Breakfast Spot,Salon / Barbershop


Finally, let's visualize the resulting clusters

In [41]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(chicago_merged['Latitude'], chicago_merged['Longitude'], chicago_merged['Neighborhood'], chicago_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


##  Examine Clusters

In [48]:
chicago_merged.sort_values(by='Cluster Labels')

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
114,Fuller Park,41.810022,-87.633505,0,Bus Station,Park,Food,Train Station,Baseball Field,Fast Food Restaurant,Zoo Exhibit,Event Space,Exhibit,Eye Doctor
49,Mount Greenwood Heights,41.686302,-87.715167,0,Currency Exchange,Park,Mexican Restaurant,Pizza Place,Performing Arts Venue,Zoo Exhibit,Field,Event Space,Exhibit,Eye Doctor
182,West Beverly,41.705617,-87.688074,0,Park,Convenience Store,Bookstore,Deli / Bodega,Golf Course,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor
194,Gresham,41.745177,-87.656412,0,Pool,Park,Basketball Court,Nightclub,Discount Store,Zoo Exhibit,Fast Food Restaurant,Event Space,Exhibit,Eye Doctor
181,Woodlawn,41.779333,-87.588308,0,Bus Station,BBQ Joint,Park,Soccer Field,Baseball Field,Gym,Athletics & Sports,Food,Design Studio,Exhibit
62,Canaryville,41.814845,-87.641123,0,Baseball Field,Park,Pizza Place,Bar,Shoe Store,Gym,Sandwich Place,Bakery,Zoo Exhibit,Falafel Restaurant
64,Edison Park,42.007704,-87.814129,0,Park,Wine Shop,Theater,Zoo Exhibit,Fast Food Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor
164,Oakland,41.823637,-87.601570,0,Beach,Park,Track,Trail,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Falafel Restaurant
77,River's Edge,41.977877,-87.736786,0,Pool,Park,Gym,Bike Trail,Zoo Exhibit,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor
78,South Austin,41.886741,-87.760133,0,Grocery Store,Park,Bus Station,Shoe Repair,Wings Joint,Food Court,Sandwich Place,Gym,Train Station,Intersection


#### Cluster 1

In [44]:
chicago_merged.loc[chicago_merged['Cluster Labels'] == 0, chicago_merged.columns[[0] + list(range(4, chicago_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Marquette Park,Soccer Field,Park,Zoo Exhibit,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Falafel Restaurant,Farm,Farmers Market
18,Oriole Park,Video Store,Park,Gym,Football Stadium,Zoo Exhibit,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Falafel Restaurant
49,Mount Greenwood Heights,Currency Exchange,Park,Mexican Restaurant,Pizza Place,Performing Arts Venue,Zoo Exhibit,Field,Event Space,Exhibit,Eye Doctor
62,Canaryville,Baseball Field,Park,Pizza Place,Bar,Shoe Store,Gym,Sandwich Place,Bakery,Zoo Exhibit,Falafel Restaurant
64,Edison Park,Park,Wine Shop,Theater,Zoo Exhibit,Fast Food Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor
77,River's Edge,Pool,Park,Gym,Bike Trail,Zoo Exhibit,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor
78,South Austin,Grocery Store,Park,Bus Station,Shoe Repair,Wings Joint,Food Court,Sandwich Place,Gym,Train Station,Intersection
84,West Chesterfield,BBQ Joint,Park,Pizza Place,Field,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Falafel Restaurant
88,Brynford Park,Park,Gymnastics Gym,Kitchen Supply Store,Department Store,Fast Food Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor
114,Fuller Park,Bus Station,Park,Food,Train Station,Baseball Field,Fast Food Restaurant,Zoo Exhibit,Event Space,Exhibit,Eye Doctor


#### Cluster 2

In [49]:
chicago_merged.loc[chicago_merged['Cluster Labels'] == 1, chicago_merged.columns[[0] + list(range(4, chicago_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Clearing (E),Hotel,American Restaurant,Pizza Place,Fast Food Restaurant,Chinese Restaurant,Coffee Shop,Mexican Restaurant,Martial Arts Dojo,Taco Place,Taxi Stand
15,West Elsdon,Mexican Restaurant,Bar,Fast Food Restaurant,Ice Cream Shop,Rental Service,Gas Station,Donut Shop,Gym / Fitness Center,Video Store,Fried Chicken Joint
16,Marynook,Fast Food Restaurant,Jazz Club,Caribbean Restaurant,Optical Shop,Grocery Store,Park,Athletics & Sports,Seafood Restaurant,Supermarket,Film Studio
19,Beverly Woods,Fast Food Restaurant,Snack Place,Pharmacy,Scenic Lookout,Skating Rink,Discount Store,Gas Station,Video Store,Bar,Breakfast Spot
21,Belmont Gardens,Fast Food Restaurant,Discount Store,Department Store,Park,Art Gallery,Pet Store,Diner,Optical Shop,Mexican Restaurant,Fish Market
25,Parkway Gardens,Train Station,American Restaurant,Food,Music Venue,Grocery Store,Fast Food Restaurant,Light Rail Station,Fish Market,Falafel Restaurant,English Restaurant
27,Washington Park,Fast Food Restaurant,Bookstore,Breakfast Spot,Theater,Lighting Store,Train Station,Gas Station,Park,Bus Station,Lounge
36,Rosemoor,Donut Shop,Fast Food Restaurant,Grocery Store,Liquor Store,Southern / Soul Food Restaurant,Zoo Exhibit,Event Space,Exhibit,Eye Doctor,Falafel Restaurant
42,Chicago Midway Airport,Mobile Phone Shop,Gift Shop,Airport Lounge,Airport Service,Fast Food Restaurant,Bagel Shop,Filipino Restaurant,Event Space,Exhibit,Eye Doctor
58,Park Manor,Fast Food Restaurant,Currency Exchange,Lounge,Convenience Store,Seafood Restaurant,Liquor Store,Spa,American Restaurant,Park,Chinese Restaurant


## Results and Discussion <a name="results"></a>

Based on our analysis, we can see that the neighborhoods in Chicago are very diverse. 
Our k-means clustering analysis did not produce a clear winner. Although the neighborhoods within each cluster still have
a good amount of differences, the results might be used to see what list of neighborhoods should be considered above the rest.

For instance, if we look at the cluster #2, we can see that this cluster is probably not the best set of neighborhoods that would
work for our client's food truck business. The reason for that is that this set of neighborhoods have a lot of restaurants, fast food chains,
pizza spots, lounges and food joints. We think that it would be hard to compete in such neighborhoods with such dense population of food places.
For this reason, this cluster is excluded from the list of potential candidates. The rest of the clusters except 1 and 8 are also excluded from
the list of potential neighborhoods. 

If we look at clusters 1 and 8, we can see that the most common venues are parks, soccer fields, pools and beaches along with zoo exhibits, 
farmers markets and bike trails. Based on that, we can say that these two clusters would be the best choice for a food truck locations.

## Conclusion <a name="conclusion"></a>

The purpose of this project was to help a client with expanding his food truck business called "Hot Potatoez" to Chicago.

The process included collecting a list of neighborhoods, their longitude and latitude; retriving venue information from the FourSquare website; 
using the two previous pieces of information to find the most common venues for each neighborhood; after that clustering (using k-means clustering)
our neighborhoods to produce a set of neighborhoods that would be the best potential candidates for the food truck business.

The results of our analysis, allow us to conclude that clusters 1 and 8 are our choice for potential food truck locations.
These two clusters have their most common venues as parks, zoo exhibits, soccer fields, bike trails, beaches, pools and farmers markets.
We think that these public venues would fit our business most and also the lack of restaurants and other food places